In [132]:
import re
import pandas as pd
import numpy as np
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor

Root = "C:/Users/JJvRijn/Documents/Python_Scripts"

In [32]:
test0 = '<tr class="even"><td align="right"><a href="display_sub.php?details=3435">3435</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS</td><td>Peptide-4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>2% hemolysis upto 100μg/ml</td><td>Human</td><td>CTTSTTSHHHHHHHHHHHHHSSTTSSTTTTTTC</td><td align="center"><a href="jmol.php?id=3435&amp;di=1" width="5px" height="5px"><img src="pymol_images/3435.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3436">3436</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS</td><td>Peptide-4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>9% hemolysis upto 200μg/ml</td><td>Human</td><td>CTTSTTSHHHHHHHHHHHHHSSTTSSTTTTTTC</td><td align="center"><a href="jmol.php?id=3436&amp;di=1" width="5px" height="5px"><img src="pymol_images/3436.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1014">1014</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>AAAKAALNAVLVGANA</td><td>S4-(13–28)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>LC<sub>50</sub> =40μM</td><td>Human</td><td>CHHHHHTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=1014&amp;di=1" width="5px" height="5px"><img src="pymol_images/1014.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1014&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1014.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1969">1969</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11242518">11242518</a></td><td align="right">2001</td><td>AAGLAMLFLGILSAAGSTMGARA</td><td>FP-I<sub>VAU</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Hemolytic</td><td>~50% hemolysis at 40µM</td><td>Human</td><td>CCTHHHHHHHHHHHTTCCTTTTC</td><td align="center"><a href="jmol.php?id=1969&amp;di=1" width="5px" height="5px"><img src="pymol_images/1969.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1969&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1969.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1970">1970</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11242518">11242518</a></td><td align="right">2001</td><td>AAGLAMLFLGILSAAGSTMGARA</td><td>FP-I<sub>VAU</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Hemolytic</td><td>55-60% hemolysis at 80µM</td><td>Human</td><td>CCTHHHHHHHHHHHTTCCTTTTC</td><td align="center"><a href="jmol.php?id=1970&amp;di=1" width="5px" height="5px"><img src="pymol_images/1970.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1970&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1970.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1227">1227</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18570895">18570895</a></td><td align="right">2008</td><td>AAGMGFFGAR</td><td>Urechistachykinins II</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>0% hemolysis at 3-100μM (non-hemolytic)</td><td>Human</td><td>CTTSSSSSCC</td><td align="center"><a href="jmol.php?id=1227&amp;di=1" width="5px" height="5px"><img src="pymol_images/1227.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1227&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1227.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2049">2049</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>AARRAARRAARR</td><td>(AARR)<sub>3</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt; 500mg/L</td><td>Human</td><td>CHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2049&amp;di=1" width="5px" height="5px"><img src="pymol_images/2049.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2049&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2049.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3567">3567</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15304333">15304333</a></td><td align="right">2004</td><td>ACGILHDNCVYVPAQNPCCRGLQCRYGKCLVQV</td><td>PcFK1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antiplasmodial</td><td>No hemolysis at 10μM (non hemolytic)</td><td>Human</td><td>CCSSSSSSCCCCTTCCCCCSSCBSSSSBCCCCC</td><td align="center"><a href="jmol.php?id=3567&amp;di=1" width="5px" height="5px"><img src="pymol_images/3567.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3567&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3567.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1099">1099</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17212422">17212422</a></td><td align="right">2007</td><td>ACPC-A-APC-K-APC-L-ACPC-K-APC-A-ACPC-A-ACPC-L-ACPC</td><td>Peptide 9</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>APC and ACPC</td><td align="right">15</td><td>Antibacterial</td><td>&gt;10% hemolysis at &gt;400μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1093">1093</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17212422">17212422</a></td><td align="right">2007</td><td>ACPC-L-APC-K-APC-L-ACPC-K-APC-L-ACPC-L-ACPC-L-ACPC</td><td>Peptide 3</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>APC and ACPC</td><td align="right">15</td><td>Antibacterial</td><td>&gt;10% hemolysis at 100μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1197">1197</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18455267">18455267</a></td><td align="right">2008</td><td>ACYCRRRFCVCVGR</td><td>PC93</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>25-50% hemolysis at 80μg/ml</td><td>Human</td><td>CCCCCCCCSCCTTC</td><td align="center"><a href="jmol.php?id=1197&amp;di=1" width="5px" height="5px"><img src="pymol_images/1197.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1197&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1197.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2929">2929</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>AGWGSIFKHIFKAGKFIHGAIQAHND</td><td>NRC-14</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCSCCTHHHHCSSCCCCCGGGTTCCC</td><td align="center"><a href="jmol.php?id=2929&amp;di=1" width="5px" height="5px"><img src="pymol_images/2929.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2929&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2929.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1647">1647</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16137634">16137634</a></td><td align="right">2005</td><td>AGYLLGKINLKALAALAKKIL</td><td>Transportan 10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>CPP</td><td>29.0±4.9% hemolysis at 50µM</td><td>Bovine</td><td>CCSCCCSSCHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=1647&amp;di=1" width="5px" height="5px"><img src="pymol_images/1647.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1647&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1647.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3569">3569</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18519720">18519720</a></td><td align="right">2008</td><td>AGYLLGKINLKALAALAKKIL</td><td>TP10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>5% hemolysis at 30μM</td><td>Human</td><td>CCSCCCSSCHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3569&amp;di=1" width="5px" height="5px"><img src="pymol_images/3569.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3569&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3569.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1783">1783</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>AhxFTicOicAhxKTicOicAhxFTicOicAhxKTicKKKK</td><td>Peptide-37</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid, Ahx= 6-aminohexanoic acid</td><td align="right">19</td><td>Antibiotic</td><td>44.7% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1784">1784</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>AhxFTicOicAhxKTicOicAhxFTicOicAhxKTicKKKK</td><td>Peptide-37</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid, Ahx= 6-aminohexanoic acid</td><td align="right">19</td><td>Antibiotic</td><td>26.6% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1294">1294</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19399781">19399781</a></td><td align="right">2009</td><td>Aib-GGL-Aib-GIL</td><td>Tric-8</td><td>Methoxy (OMe)</td><td>Acetylated by n-octanoyl</td><td>Linear</td><td>L</td><td>Aib (α-aminoisobutyric acid)</td><td align="right">8</td><td>Antifungal and Antibacterial</td><td>0% hemolysis at 15μM (non-hemolytic)</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1292">1292</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19399781">19399781</a></td><td align="right">2009</td><td>Aib-GL-Aib-GGL-Aib-GI</td><td>Trichogin GA IV</td><td>Lol (leucinol)</td><td>Acetylated by n-octanoyl</td><td>Linear</td><td>L</td><td>Aib (α-aminoisobutyric acid)</td><td align="right">10</td><td>Antifungal and Antibacterial</td><td>2% hemolysis at 15μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1293">1293</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19399781">19399781</a></td><td align="right">2009</td><td>Aib-GL-Aib-GGL-Aib-GIL</td><td>Tric-OMe</td><td>Methoxy (OMe)</td><td>Acetylated by n-octanoyl</td><td>Linear</td><td>L</td><td>Aib (α-aminoisobutyric acid)</td><td align="right">11</td><td>Antifungal and Antibacterial</td><td>2% hemolysis at 15μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3808">3808</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9518597">9518597</a></td><td align="right">1998</td><td>Aib-P-Aib-A-Aib-AQ-Aib-V-Aib-GL-Aib-PV-Aib-Aib-EQ</td><td>Alam-P14</td><td>Pheol</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Aib = α- methylalanine</td><td align="right">19</td><td>Cytolytic</td><td>EC<sub>50</sub> =30μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2062">2062</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>AINPKSVQSLL</td><td>EpVP3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCSHHHHTTTC</td><td align="center"><a href="jmol.php?id=2062&amp;di=1" width="5px" height="5px"><img src="pymol_images/2062.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2314">2314</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>AKAkAyPAKAKAyP</td><td>GS14K4 V3L3/A6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;150µg/ml</td><td>Human</td><td>CCCCCCCTTCCCCC</td><td align="center"><a href="jmol.php?id=2314&amp;di=1" width="5px" height="5px"><img src="pymol_images/2314.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2432">2432</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11682479">11682479</a></td><td align="right">2002</td><td>AKAkAyPAKAKAyP</td><td>V3L3/A6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;800µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2744">2744</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22038181">22038181</a></td><td align="right">2012</td><td>AKKKGKCKGPLKLVAKC</td><td>Las[des1-10, Ala25]-NH<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>&lt;1% hemolytic at 200μM</td><td>Human</td><td>CCSTTTTCCSCCCCCCC</td><td align="center"><a href="jmol.php?id=2744&amp;di=1" width="5px" height="5px"><img src="pymol_images/2744.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3574">3574</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18984589">18984589</a></td><td align="right">2009</td><td>akklohalhoallalohlahollakk</td><td>D-HALO-rev</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>O=Ornithine</td><td align="right">26</td><td>Antimicrobial and Antiplasmodial</td><td>HC<sub>50</sub> =716.8μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3575">3575</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18984589">18984589</a></td><td align="right">2009</td><td>akklohalhoallalohlpholkkff</td><td>D-HALO-P8Frev</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>O=Ornithine</td><td align="right">26</td><td>Antimicrobial and Antiplasmodial</td><td>HC<sub>50</sub> &gt;65.2μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=3575&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3575.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1931">1931</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFS</td><td>HPC3(2–14)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTTGGGGTTTCC</td><td align="center"><a href="jmol.php?id=1931&amp;di=1" width="5px" height="5px"><img src="pymol_images/1931.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1931&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1931.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1930">1930</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFSKI</td><td>HPC2(2–16)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCGGGGGTTTTTSCC</td><td align="center"><a href="jmol.php?id=1930&amp;di=1" width="5px" height="5px"><img src="pymol_images/1930.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1930&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1930.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1929">1929</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFSKIQN</td><td>HPC1(2–18)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CGGGGHHHHTTTCSSCC</td><td align="center"><a href="jmol.php?id=1929&amp;di=1" width="5px" height="5px"><img src="pymol_images/1929.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1929&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1929.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1925">1925</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFSKIQNDK</td><td>HP (2–20)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHTTTHHHHSGGGSCC</td><td align="center"><a href="jmol.php?id=1925&amp;di=1" width="5px" height="5px"><img src="pymol_images/1925.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1925&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1925.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2329">2329</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15019213">15019213</a></td><td align="right">2003</td><td>AKKVFKRLEKLFSKIQNDK</td><td>HP (2–20)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal</td><td>&lt;20% hemolysis at 2.15 to 215µM</td><td>Human</td><td>CHHHHTTTHHHHSGGGSCC</td><td align="center"><a href="jmol.php?id=2329&amp;di=1" width="5px" height="5px"><img src="pymol_images/2329.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2329&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2329.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2381">2381</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12147359">12147359</a></td><td align="right">2002</td><td>AKKVFKRLEKLFSKIQNDK</td><td>HP (2 – 20)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CHHHHTTTHHHHSGGGSCC</td><td align="center"><a href="jmol.php?id=2381&amp;di=1" width="5px" height="5px"><img src="pymol_images/2381.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2381&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2381.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2394">2394</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15358153">15358153</a></td><td align="right">2004</td><td>AKKVFKRLEKLFSKIQNDK</td><td>HP (2–20)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>0% hemolysis at 140µg/ml (non-hemolytic)</td><td>Human</td><td>CHHHHTTTHHHHSGGGSCC</td><td align="center"><a href="jmol.php?id=2394&amp;di=1" width="5px" height="5px"><img src="pymol_images/2394.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2394&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2394.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1935">1935</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFSKIQNWK</td><td>HPA1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CGGGGGGTGGGSCTTTSCC</td><td align="center"><a href="jmol.php?id=1935&amp;di=1" width="5px" height="5px"><img src="pymol_images/1935.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1935&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1935.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2382">2382</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12147359">12147359</a></td><td align="right">2002</td><td>AKKVFKRLEKLFSKIQNWK</td><td>Anal 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal and Anticancer</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CGGGGGGTGGGSCTTTSCC</td><td align="center"><a href="jmol.php?id=2382&amp;di=1" width="5px" height="5px"><img src="pymol_images/2382.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2382&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2382.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1936">1936</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFSKIWNDK</td><td>HPA2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTTTHHHHHHHCCSSCCC</td><td align="center"><a href="jmol.php?id=1936&amp;di=1" width="5px" height="5px"><img src="pymol_images/1936.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1936&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1936.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2383">2383</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12147359">12147359</a></td><td align="right">2002</td><td>AKKVFKRLEKLFSKIWNDK</td><td>Anal 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal and Anticancer</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CTTTTHHHHHHHCCSSCCC</td><td align="center"><a href="jmol.php?id=2383&amp;di=1" width="5px" height="5px"><img src="pymol_images/2383.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2383&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2383.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1828">1828</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFSKIWNWK</td><td>HPA3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>23.11% hemolysis at 100µM</td><td>Human</td><td>CGGGGHHHHHHHHHHSCCC</td><td align="center"><a href="jmol.php?id=1828&amp;di=1" width="5px" height="5px"><img src="pymol_images/1828.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1828&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1828.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2384">2384</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12147359">12147359</a></td><td align="right">2002</td><td>AKKVFKRLEKLFSKIWNWK</td><td>Anal 3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal and Anticancer</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CGGGGHHHHHHHHHHSCCC</td><td align="center"><a href="jmol.php?id=2384&amp;di=1" width="5px" height="5px"><img src="pymol_images/2384.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2384&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2384.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1937">1937</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKSFSKIQNDK</td><td>HPA4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHTTGGGTTCTTSCCC</td><td align="center"><a href="jmol.php?id=1937&amp;di=1" width="5px" height="5px"><img src="pymol_images/1937.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1937&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1937.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2385">2385</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12147359">12147359</a></td><td align="right">2002</td><td>AKKVFKRLEKSFSKIQNDK</td><td>Anal 4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal and Anticancer</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CHHHHTTGGGTTCTTSCCC</td><td align="center"><a href="jmol.php?id=2385&amp;di=1" width="5px" height="5px"><img src="pymol_images/2385.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2385&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2385.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4184">4184</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLKVLKKG</td><td>HM5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis upto 100µM</td><td>Human</td><td>CGGGHHHHCCHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=4184&amp;di=1" width="5px" height="5px"><img src="pymol_images/4184.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4184&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4184.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4168">4168</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLKVLTTG</td><td>HP(2-9)-ME(1-12)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis upto 100µM</td><td>Human</td><td>CHHHHTTTCCTTSCCCCSCC</td><td align="center"><a href="jmol.php?id=4168&amp;di=1" width="5px" height="5px"><img src="pymol_images/4168.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4168&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4168.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4172">4172</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLKVLTWG</td><td>HM2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis upto 100µM</td><td>Human</td><td>CTTTCGGGCCSSSCCCCSCC</td><td align="center"><a href="jmol.php?id=4172&amp;di=1" width="5px" height="5px"><img src="pymol_images/4172.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4172&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4172.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1589">1589</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTTG</td><td>HM1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>46% hemolysis at 100µM</td><td>Human</td><td>CTTSTTTSSCSSCCCSSTTC</td><td align="center"><a href="jmol.php?id=1589&amp;di=1" width="5px" height="5px"><img src="pymol_images/1589.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1589&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1589.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1590">1590</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTTG</td><td>HM1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>30% hemolysis at 50µM</td><td>Human</td><td>CTTSTTTSSCSSCCCSSTTC</td><td align="center"><a href="jmol.php?id=1590&amp;di=1" width="5px" height="5px"><img src="pymol_images/1590.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1590&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1590.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4169">4169</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTTG</td><td>HM1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>13% hemolysis at 25µM</td><td>Human</td><td>CTTSTTTSSCSSCCCSSTTC</td><td align="center"><a href="jmol.php?id=4169&amp;di=1" width="5px" height="5px"><img src="pymol_images/4169.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4169&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4169.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4170">4170</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTTG</td><td>HM1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>1% hemolysis at 12.5µM</td><td>Human</td><td>CTTSTTTSSCSSCCCSSTTC</td><td align="center"><a href="jmol.php?id=4170&amp;di=1" width="5px" height="5px"><img src="pymol_images/4170.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4170&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4170.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4171">4171</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTTG</td><td>HM1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis at 6.25µM</td><td>Human</td><td>CTTSTTTSSCSSCCCSSTTC</td><td align="center"><a href="jmol.php?id=4171&amp;di=1" width="5px" height="5px"><img src="pymol_images/4171.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4171&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4171.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1591">1591</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTWG</td><td>HM3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>86% hemolysis at 100µM</td><td>Human</td><td>CGGGGGGSSCTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=1591&amp;di=1" width="5px" height="5px"><img src="pymol_images/1591.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1591&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1591.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1592">1592</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTWG</td><td>HM3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>69% hemolysis at 50µM</td><td>Human</td><td>CGGGGGGSSCTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=1592&amp;di=1" width="5px" height="5px"><img src="pymol_images/1592.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1592&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1592.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4173">4173</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTWG</td><td>HM3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>44% hemolysis at 25µM</td><td>Human</td><td>CGGGGGGSSCTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=4173&amp;di=1" width="5px" height="5px"><img src="pymol_images/4173.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4173&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4173.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4174">4174</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTWG</td><td>HM3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>31% hemolysis at 12.5µM</td><td>Human</td><td>CGGGGGGSSCTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=4174&amp;di=1" width="5px" height="5px"><img src="pymol_images/4174.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4174&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4174.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4175">4175</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTWG</td><td>HM3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>12% hemolysis at 6.25µM</td><td>Human</td><td>CGGGGGGSSCTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=4175&amp;di=1" width="5px" height="5px"><img src="pymol_images/4175.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4175&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4175.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4176">4176</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTWG</td><td>HM3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>4% hemolysis at 3.12µM</td><td>Human</td><td>CGGGGGGSSCTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=4176&amp;di=1" width="5px" height="5px"><img src="pymol_images/4176.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4176&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4176.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4177">4177</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTWG</td><td>HM3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis at 1.5µM</td><td>Human</td><td>CGGGGGGSSCTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=4177&amp;di=1" width="5px" height="5px"><img src="pymol_images/4177.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4177&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4177.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1822">1822</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLRKLFKKI</td><td>HPC2A3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>4.5% hemolysis at 100µM</td><td>Human</td><td>CTTTHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1822&amp;di=1" width="5px" height="5px"><img src="pymol_images/1822.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1822&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1822.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1938">1938</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVSKRLEKLFSKIQNDK</td><td>HPA5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHHHHHTTTCTTTSCC</td><td align="center"><a href="jmol.php?id=1938&amp;di=1" width="5px" height="5px"><img src="pymol_images/1938.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1938&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1938.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2386">2386</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12147359">12147359</a></td><td align="right">2002</td><td>AKKVSKRLEKLFSKIQNDK</td><td>Anal 5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal and Anticancer</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CHHHHHHHHTTTCTTTSCC</td><td align="center"><a href="jmol.php?id=2386&amp;di=1" width="5px" height="5px"><img src="pymol_images/2386.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2386&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2386.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2312">2312</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>AKLkAyPLKAKLyP</td><td>GS14K4 V3/A3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;150µg/ml</td><td>Human</td><td>CCCCCCCSSSSCCC</td><td align="center"><a href="jmol.php?id=2312&amp;di=1" width="5px" height="5px"><img src="pymol_images/2312.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2312&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2312.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2430">2430</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11682479">11682479</a></td><td align="right">2002</td><td>AKLkAyPLKAKLyP</td><td>V3/A3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;800µg/ml</td><td>Human</td><td>CCCCCCCSSSSCCC</td><td align="center"><a href="jmol.php?id=2430&amp;di=1" width="5px" height="5px"><img src="pymol_images/2430.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2430&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2430.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3946">3946</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>AKLkAyPLKAKLyP</td><td>GS14K4 V3/A3</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>MHC &gt;800μg/ml</td><td>Human</td><td>CCCCCCCSSSSCCC</td><td align="center"><a href="jmol.php?id=3946&amp;di=1" width="5px" height="5px"><img src="pymol_images/3946.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3946&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3946.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1818">1818</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17826814">17826814</a></td><td align="right">2007</td><td>ALAGTIIAGASLTFQVLDKVLEELGKVSRK</td><td>St II 1–30</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Hemolytic</td><td>Hemolytic activity expresssed as the initial rate (∆A/∆t, min<sup>-1</sup>) <i>i.e.</i> at above 100</td><td>Human</td><td>CCSSSCCCTTCCSTHHHHHHHHHHTTSSCC</td><td align="center"><a href="jmol.php?id=1818&amp;di=1" width="5px" height="5px"><img src="pymol_images/1818.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1818&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1818.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3571">3571</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18984589">18984589</a></td><td align="right">2009</td><td>ALALLOHALHOLALLAOHLAHOLALA</td><td>HALO-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>O=Ornithine</td><td align="right">26</td><td>Antimicrobial and Antiplasmodial</td><td>HC<sub>50</sub> &lt;5.6μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3541">3541</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12067731">12067731</a></td><td align="right">2002</td><td>ALLHHGLNCAKGVLA</td><td>15 monomer halocidin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml (non hemolytic)</td><td>Human</td><td>CTTCSSCCGGGGTCC</td><td align="center"><a href="jmol.php?id=3541&amp;di=1" width="5px" height="5px"><img src="pymol_images/3541.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3541&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3541.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3542">3542</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12067731">12067731</a></td><td align="right">2002</td><td>ALLHHGLNCAKGVLAALLHHGLNCAKGVLA</td><td>15 homodimer halocidin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml (non hemolytic)</td><td>Human</td><td>CTTTCCCCCHHHHHHHHHHCSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3542&amp;di=1" width="5px" height="5px"><img src="pymol_images/3542.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3542&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3542.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2881">2881</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>ALPLIGRVLSGIL</td><td>Ta A1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>62.5% hemolytic at 40μM</td><td>Human</td><td>CCCCGGGTTSCCC</td><td align="center"><a href="jmol.php?id=2881&amp;di=1" width="5px" height="5px"><img src="pymol_images/2881.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2881&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2881.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2882">2882</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>ALPLIGRVLSGIL</td><td>Ta A1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolytic at 1.25μM</td><td>Human</td><td>CCCCGGGTTSCCC</td><td align="center"><a href="jmol.php?id=2882&amp;di=1" width="5px" height="5px"><img src="pymol_images/2882.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2882&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2882.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1004">1004</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWDTLLKKVLKAAAKAALDAVLVGANA</td><td>D<sub>4</sub>D<sub>20</sub>-S4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>LC<sub>50</sub> =5±1μM</td><td>Human</td><td>CCTHHHHHHHHHHHHHHHHHHHHTCCCC</td><td align="center"><a href="jmol.php?id=1004&amp;di=1" width="5px" height="5px"><img src="pymol_images/1004.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1004&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1004.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1003">1003</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWDTLLKKVLKAAAKAALNAVLVGANA</td><td>D<sub>4</sub>-S4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>LC<sub>50</sub> =2.3±0.3μM</td><td>Human</td><td>CCTHHHHHHHHHHHHHHHTTTTSCTTCC</td><td align="center"><a href="jmol.php?id=1003&amp;di=1" width="5px" height="5px"><img src="pymol_images/1003.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1003&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1003.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3691">3691</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7989335">7989335</a></td><td align="right">1994</td><td>ALWFTMLKKLGTMALHAGKAALGAAANTISQGTQ</td><td>DS s2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antimicrobial</td><td>100% hemolysis at 70µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3692">3692</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7989335">7989335</a></td><td align="right">1994</td><td>ALWKNMLKGIGKLAGKAALGAVKKLVGAES</td><td>DS s3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>100% hemolysis at 80µM</td><td>Human</td><td>CTHHHHTTTTTTCHHHHTTHHHHTTSCCCC</td><td align="center"><a href="jmol.php?id=3692&amp;di=1" width="5px" height="5px"><img src="pymol_images/3692.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3692&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3692.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1019">1019</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWKTLLKKV</td><td>K<sub>4</sub>-S4-(1–10)a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100μM</td><td>Human</td><td>CTHHHHGGGC</td><td align="center"><a href="jmol.php?id=1019&amp;di=1" width="5px" height="5px"><img src="pymol_images/1019.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1019&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1019.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1018">1018</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWKTLLKKVLKA</td><td>K<sub>4</sub>-S4-(1–13)a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LC<sub>50</sub> =57±3μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=1018&amp;di=1" width="5px" height="5px"><img src="pymol_images/1018.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1018&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1018.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4097">4097</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11897590">11897590</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>P</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>5% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4097&amp;di=1" width="5px" height="5px"><img src="pymol_images/4097.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4097&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4097.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4098">4098</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11937508">11937508</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>K<sub>4</sub>-S4(1-13)<sub>NH2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>3% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4098&amp;di=1" width="5px" height="5px"><img src="pymol_images/4098.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4098&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4098.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4099">4099</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11937508">11937508</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>Glycyl-P</td><td>Amidation</td><td>H<sub>2</sub>N-CH<sub>2</sub>-CO-P</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>2% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4099&amp;di=1" width="5px" height="5px"><img src="pymol_images/4099.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4099&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4099.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4100">4100</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11937508">11937508</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>Aminobutyryl-P</td><td>Amidation</td><td>H<sub>2</sub>N-(CH<sub>2</sub>)<sub>3</sub>-CO-P</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>1% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4100&amp;di=1" width="5px" height="5px"><img src="pymol_images/4100.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4101">4101</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11937508">11937508</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>Aminoheptanoyl-P</td><td>Amidation</td><td>H<sub>2</sub>N-(CH<sub>2</sub>)<sub>6</sub>-CO-P</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>1% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4101&amp;di=1" width="5px" height="5px"><img src="pymol_images/4101.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4102">4102</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11937508">11937508</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>Aminolauroyl-P</td><td>Amidation</td><td>H<sub>2</sub>N-(CH<sub>2</sub>)<sub>11</sub>-CO-P</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>4% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4102&amp;di=1" width="5px" height="5px"><img src="pymol_images/4102.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4103">4103</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11937508">11937508</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>Lysyl-P</td><td>Amidation</td><td>H<sub>2</sub>N-C((CH<sub>2</sub>)<sub>4</sub>-NH<sub>2</sub>)H-CO-P</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>5% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4103&amp;di=1" width="5px" height="5px"><img src="pymol_images/4103.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1017">1017</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWKTLLKKVLKAAA</td><td>K<sub>4</sub>-S4-(1–15)a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>LC<sub>50</sub> =20±1μM</td><td>Human</td><td>CTHHHHTHHHHTTTC</td><td align="center"><a href="jmol.php?id=1017&amp;di=1" width="5px" height="5px"><img src="pymol_images/1017.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1017&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1017.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1015">1015</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWKTLLKKVLKAAAK</td><td>K<sub>4</sub>-S4-(1–16)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>LC<sub>50</sub> =57±3μM</td><td>Human</td><td>CTHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=1015&amp;di=1" width="5px" height="5px"><img src="pymol_images/1015.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1015&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1015.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1016">1016</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWKTLLKKVLKAAAK</td><td>K<sub>4</sub>-S4-(1–16)a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>LC<sub>50</sub> =10±0.5μM</td><td>Human</td><td>NA</td><td align="center"><a href="jmol.php?id=1016&amp;di=1" width="5px" height="5px"><img src="pymol_images/1016.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1016&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1016.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1007">1007</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWKTLLKKVLKAAAKAALKAVLVGANA</td><td>K<sub>4</sub>K<sub>20</sub>-S4-S4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>LC<sub>50</sub> =0.5±0.1μM</td><td>Human</td><td>CTTHHHHHHHHHHHHHHHHHHHSCSCCC</td><td align="center"><a href="jmol.php?id=1007&amp;di=1" width="5px" height="5px"><img src="pymol_images/1007.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1007&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1007.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1006">1006</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWKTLLKKVLKAAAKAALNAVLVGANA</td><td>K<sub>4</sub>-S4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>LC<sub>50</sub> =2±0.1μM</td><td>Human</td><td>CTHHHHHHHHHHHHHHHHHHHSCCCCCC</td><td align="center"><a href="jmol.php?id=1006&amp;di=1" width="5px" height="5px"><img src="pymol_images/1006.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1006&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1006.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3690">3690</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7989335">7989335</a></td><td align="right">1994</td><td>ALWKTMLKKLGTMALHAGKAALGAAADTISQGTQ</td><td>DS s1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antimicrobial</td><td>100% hemolysis at &gt;70µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3986">3986</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8306981">8306981</a></td><td align="right">1994</td><td>ALWKTMLKKLGTMALHAGKAALGAAADTISQGTQ</td><td>Dermaseptin I</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antimicrobial</td><td>100% hemolysis at 100μM</td><td>Rabbit</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1010">1010</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWMTLLKKVLK</td><td>S4-(1–12)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> =100μM</td><td>Human</td><td>CHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=1010&amp;di=1" width="5px" height="5px"><img src="pymol_images/1010.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1010&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1010.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1009">1009</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWMTLLKKVLKAAAK</td><td>S4-(1–16)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>LC<sub>50</sub> =23±1μM</td><td>Human</td><td>CTHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=1009&amp;di=1" width="5px" height="5px"><img src="pymol_images/1009.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1009&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1009.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1002">1002</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWMTLLKKVLKAAAKAALDAVLVGANA</td><td>D<sub>20</sub>-S4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>LC<sub>50</sub> =1.2±0.1μM</td><td>Human</td><td>CCHHHHHHHHHHHHHHHTTTTTSCTTCC</td><td align="center"><a href="jmol.php?id=1002&amp;di=1" width="5px" height="5px"><img src="pymol_images/1002.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1002&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1002.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1005">1005</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWMTLLKKVLKAAAKAALKAVLVGANA</td><td>K<sub>20</sub>-S4-S4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>LC<sub>50</sub> =1.2±0.4μM</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHGGGTCCCCCC</td><td align="center"><a href="jmol.php?id=1005&amp;di=1" width="5px" height="5px"><img src="pymol_images/1005.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1005&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1005.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1008">1008</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWMTLLKKVLKAAAKAALN</td><td>S4-(1–20)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>LC<sub>50</sub> =5±1μM</td><td>Human</td><td>CHHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1008&amp;di=1" width="5px" height="5px"><img src="pymol_images/1008.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1008&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1008.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1001">1001</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10660589">10660589</a></td><td align="right">2000</td><td>ALWMTLLKKVLKAAAKAALNAVLVGANA</td><td>S4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>LC<sub>50</sub> =1.4±0.2μM</td><td>Human</td><td>CHHHHHHHHHHHHHTTTGGGSSCCCSCC</td><td align="center"><a href="jmol.php?id=1001&amp;di=1" width="5px" height="5px"><img src="pymol_images/1001.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1001&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1001.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3693">3693</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7989335">7989335</a></td><td align="right">1994</td><td>ALWMTLLKKVLKAAAKAALNAVLVGANA</td><td>DS s4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>100% hemolysis at 1µM</td><td>Human</td><td>CHHHHHHHHHHHHHTTTGGGSSCCCSCC</td><td align="center"><a href="jmol.php?id=3693&amp;di=1" width="5px" height="5px"><img src="pymol_images/3693.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3693&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3693.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1096">1096</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17212422">17212422</a></td><td align="right">2007</td><td>APC-F-ACPC-L-APC-L-ACPC-F-APC-F-ACPC-K-ACPC-L-APC</td><td>Peptide 6</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>APC and ACPC</td><td align="right">15</td><td>Antibacterial</td><td>&gt;10% hemolysis at 1.6μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1098">1098</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17212422">17212422</a></td><td align="right">2007</td><td>APC-L-ACPC-A-APC-A-ACPC-K-ACPC-L-ACPC-K-ACPC-A-APC</td><td>Peptide 8</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>APC and ACPC</td><td align="right">15</td><td>Antibacterial</td><td>&gt;10% hemolysis at 100μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1097">1097</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17212422">17212422</a></td><td align="right">2007</td><td>APC-L-ACPC-K-ACPC-A-APC-A-ACPC-K-ACPC-A-APC-L-ACPC</td><td>Peptide 7</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>APC and ACPC</td><td align="right">15</td><td>Antibacterial</td><td>&gt;10% hemolysis at &gt;400μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1091">1091</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17212422">17212422</a></td><td align="right">2007</td><td>APC-L-ACPC-K-ACPC-L-APC-L-ACPC-K-ACPC-L-APC-L-ACPC</td><td>Peptide 1</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>APC and ACPC</td><td align="right">15</td><td>Antibacterial</td><td>&gt;10% hemolysis at 3.1μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1092">1092</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17212422">17212422</a></td><td align="right">2007</td><td>APC-L-ACPC-L-APC-L-ACPC-K-ACPC-L-ACPC-K-ACPC-L-APC</td><td>Peptide 2</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>APC and ACPC</td><td align="right">15</td><td>Antibacterial</td><td>&gt;10% hemolysis at 1.6μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1095">1095</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17212422">17212422</a></td><td align="right">2007</td><td>APC-L-ACPC-L-APC-L-ACPC-L-APC-L-ACPC-K-ACPC-L-APC</td><td>Peptide 5</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>APC and ACPC</td><td align="right">15</td><td>Antibacterial</td><td>&gt;10% hemolysis at 1.6μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1094">1094</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17212422">17212422</a></td><td align="right">2007</td><td>APC-L-APC-L-ACPC-K-ACPC-L-ACPC-L-APC-L-ACPC-K-ACPC</td><td>Peptide 4</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>APC and ACPC</td><td align="right">15</td><td>Antibacterial</td><td>&gt;10% hemolysis at 100μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1628">1628</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15629533">15629533</a></td><td align="right">2005</td><td>AQDIISTIGDLVKWIIDTVNKFTKK</td><td>Peptide n-25</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Hemolytic</td><td>&gt;75% hemolysis at 30µM</td><td>Guinea-pig</td><td>CCCTTHHHHHHHHHHHTTTTTSSCC</td><td align="center"><a href="jmol.php?id=1628&amp;di=1" width="5px" height="5px"><img src="pymol_images/1628.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1628&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1628.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3562">3562</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2689223">2689223</a></td><td align="right">1989</td><td>AVAVVGQATQIAKKWKLFKKIEKVGQ</td><td>CA(25-37)CA(1-13)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antibacterial and Antiparasitic</td><td>Lethal concentration calculated from zone of inhibition&gt;300μM</td><td>Sheep</td><td>CCCCTTHHHHHHHHHHHHHHHTTTTC</td><td align="center"><a href="jmol.php?id=3562&amp;di=1" width="5px" height="5px"><img src="pymol_images/3562.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3562&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3562.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4049">4049</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16675060">16675060</a></td><td align="right">2006</td><td>AVDLAKIANKVLSSLF</td><td>Temporin-1TGb</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>LD<sub>50</sub> =250μM</td><td>Human</td><td>CCCHHHHHHHHTTTCC</td><td align="center"><a href="jmol.php?id=4049&amp;di=1" width="5px" height="5px"><img src="pymol_images/4049.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4049&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4049.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1967">1967</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11242518">11242518</a></td><td align="right">2001</td><td>AVGIGALFLGFLGAAGSTMGARS</td><td>FP-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Hemolytic</td><td>50% hemolysis at 40µM</td><td>Human</td><td>CCSSGGGGHHHHHHHTTCSSSCC</td><td align="center"><a href="jmol.php?id=1967&amp;di=1" width="5px" height="5px"><img src="pymol_images/1967.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1967&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1967.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1968">1968</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11242518">11242518</a></td><td align="right">2001</td><td>AVGIGALFLGFLGAAGSTMGARS</td><td>FP-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Hemolytic</td><td>90% hemolysis at 100µM</td><td>Human</td><td>CCSSGGGGHHHHHHHTTCSSSCC</td><td align="center"><a href="jmol.php?id=1968&amp;di=1" width="5px" height="5px"><img src="pymol_images/1968.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1968&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1968.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3811">3811</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9639668">9639668</a></td><td align="right">1998</td><td>AVVKVPLKKFKSIRE</td><td>mPcAP</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>0.40% hemolysis at 200μg/ml</td><td>Human</td><td>CCSCCSSTTTTTCCC</td><td align="center"><a href="jmol.php?id=3811&amp;di=1" width="5px" height="5px"><img src="pymol_images/3811.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3811&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3811.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3812">3812</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9639668">9639668</a></td><td align="right">1998</td><td>AVVKVPLKKFKSIRETMKEKGLLEDF</td><td>hPcAP</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>0.49% hemolysis at 200μg/ml</td><td>Human</td><td>CCSCCTTCCCTTTGGGTTTCCCTTCC</td><td align="center"><a href="jmol.php?id=3812&amp;di=1" width="5px" height="5px"><img src="pymol_images/3812.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3812&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3812.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2106">2106</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>AWASFFKKAAHVAKHVAKAALTHYL</td><td>[A<sup>1,3,13,17</sup>]Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>45% hemolysis at 100µM</td><td>Rabbit</td><td>CHHHHTHHHHHHHHHHHHHHHSSCC</td><td align="center"><a href="jmol.php?id=2106&amp;di=1" width="5px" height="5px"><img src="pymol_images/2106.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2106&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2106.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2107">2107</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>AWASFFKKAAHVAKHVAKAALTHYL</td><td>[A<sup>1,3,13,17</sup>]Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>90% hemolysis at 100µM</td><td>Human</td><td>CHHHHTHHHHHHHHHHHHHHHSSCC</td><td align="center"><a href="jmol.php?id=2107&amp;di=1" width="5px" height="5px"><img src="pymol_images/2107.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2107&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2107.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2102">2102</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>AWASFFKKAAHVGKHVGKAALTHYL</td><td>[A<sup>1,3</sup>]Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>4% hemolysis at 100µM</td><td>Rabbit</td><td>CHHHHHHHHHHHHHHHCTTSSSSCC</td><td align="center"><a href="jmol.php?id=2102&amp;di=1" width="5px" height="5px"><img src="pymol_images/2102.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2102&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2102.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2103">2103</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>AWASFFKKAAHVGKHVGKAALTHYL</td><td>[A<sup>1,3</sup>]Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>8% hemolysis at 100µM</td><td>Human</td><td>CHHHHHHHHHHHHHHHCTTSSSSCC</td><td align="center"><a href="jmol.php?id=2103&amp;di=1" width="5px" height="5px"><img src="pymol_images/2103.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2103&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2103.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1593">1593</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>98% hemolysis at 100µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=1593&amp;di=1" width="5px" height="5px"><img src="pymol_images/1593.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1593&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1593.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1594">1594</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>83% hemolysis at 50µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=1594&amp;di=1" width="5px" height="5px"><img src="pymol_images/1594.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1594&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1594.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4178">4178</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>44% hemolysis at 25µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=4178&amp;di=1" width="5px" height="5px"><img src="pymol_images/4178.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4178&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4178.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4179">4179</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>43% hemolysis at 12.5µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=4179&amp;di=1" width="5px" height="5px"><img src="pymol_images/4179.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4179&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4179.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4180">4180</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>30% hemolysis at 6.25µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=4180&amp;di=1" width="5px" height="5px"><img src="pymol_images/4180.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4180&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4180.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4181">4181</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>29% hemolysis at 3.1µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=4181&amp;di=1" width="5px" height="5px"><img src="pymol_images/4181.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4181&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4181.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4182">4182</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>13% hemolysis at 1.5µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=4182&amp;di=1" width="5px" height="5px"><img src="pymol_images/4182.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4182&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4182.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4183">4183</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis at 0.78µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=4183&amp;di=1" width="5px" height="5px"><img src="pymol_images/4183.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4183&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4183.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3995">3995</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16441062">16441062</a></td><td align="right">2005</td><td>CAESCVWIPCTVTALLGCSCSNNVCYNGIP</td><td>Cycloviolacin H4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>HIV inhibitory and Antimicrobial and Cytotoxic and Neurotensin antagonistic and Trypsin inhibitory and Insecticide</td><td>HD<sub>50</sub> =5.5μM</td><td>Human</td><td>CCSSSSCCGGGGGTSBTTBCCCCCSSSSCC</td><td align="center"><a href="jmol.php?id=3995&amp;di=1" width="5px" height="5px"><img src="pymol_images/3995.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3995&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3995.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3997">3997</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CAESCVYIPCTVTALLGCSCSNRVCYNGIP</td><td>Cycloviolacin O1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCSSCCCCTHHHHCSSCCGGGSSCCSCC</td><td align="center"><a href="jmol.php?id=3997&amp;di=1" width="5px" height="5px"><img src="pymol_images/3997.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2377">2377</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CAKAKAKAGSGAKAKAKAC</td><td>V5</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =1055000nM</td><td>Human</td><td>CBTTBCCCCSSCCCTTSCC</td><td align="center"><a href="jmol.php?id=2377&amp;di=1" width="5px" height="5px"><img src="pymol_images/2377.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2377&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2377.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4004">4004</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CESCVWIPCISSVVGCSCKSKVCYKNGTLP</td><td>Cycloviolacin O8</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCSSCCCSSSSTTSSSCSSCCCCCSTTCC</td><td align="center"><a href="jmol.php?id=4004&amp;di=1" width="5px" height="5px"><img src="pymol_images/4004.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4004&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4004.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2378">2378</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CFKFKFKFGSGFKFKFKFC</td><td>V6</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =35000nM</td><td>Human</td><td>CCCTTSSSSSSSCCCCSCC</td><td align="center"><a href="jmol.php?id=2378&amp;di=1" width="5px" height="5px"><img src="pymol_images/2378.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2378&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2378.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4011">4011</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVFIPCISTLLGCSCKNKVCYRNGVIP</td><td>Circulin B</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">31</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCSSSSSSTHHHHHTTSCCCCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=4011&amp;di=1" width="5px" height="5px"><img src="pymol_images/4011.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4011&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4011.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4136">4136</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10441158">10441158</a></td><td align="right">1999</td><td>CGESCVFIPCISTLLGCSCKNKVCYRNGVIP</td><td>Circulin B</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Uterotonic</td><td>HD<sub>50</sub> =30μM</td><td>Human</td><td>CCCSSSSSSTHHHHHTTSCCCCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=4136&amp;di=1" width="5px" height="5px"><img src="pymol_images/4136.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4136&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4136.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3607">3607</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22074926">22074926</a></td><td align="right">2012</td><td>CGESCVFIPCITSLAGCSCKNKVCYYDGGSVP</td><td>Parigidin_br1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>41% hemolysis at 40µM </td><td>Human</td><td>CCCCCCCCSSSSSSSSSCCCCCCCCCTTTSCC</td><td align="center"><a href="jmol.php?id=3607&amp;di=1" width="5px" height="5px"><img src="pymol_images/3607.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3607&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3607.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3608">3608</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22074926">22074926</a></td><td align="right">2012</td><td>CGESCVFIPCITSLAGCSCKNKVCYYDGGSVP</td><td>Parigidin_br1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>28% hemolysis at 20µM</td><td>Human</td><td>CCCCCCCCSSSSSSSSSCCCCCCCCCTTTSCC</td><td align="center"><a href="jmol.php?id=3608&amp;di=1" width="5px" height="5px"><img src="pymol_images/3608.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3608&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3608.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4012">4012</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVFIPCVTALLGCSCKSKVCYKNSIP</td><td>Cyclopsychotride A</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCSSCHHHHHHHTSCCBTTBCCCCCCCC</td><td align="center"><a href="jmol.php?id=4012&amp;di=1" width="5px" height="5px"><img src="pymol_images/4012.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4012&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4012.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4010">4010</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCISAALGCSCKNKVCYRNGIP</td><td>Circulin A</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCSSSSCTTTSCTTTTTCCCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=4010&amp;di=1" width="5px" height="5px"><img src="pymol_images/4010.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4002">4002</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCISAAVGCSCKSKVCYKNGTLP</td><td>Cycloviolacin O6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">31</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCTTSSSCTTTTTTTTTSCCCCCCCCCSCCC</td><td align="center"><a href="jmol.php?id=4002&amp;di=1" width="5px" height="5px"><img src="pymol_images/4002.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4002&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4002.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4007">4007</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCISAVVGCSCKSKVCYKNGTLP</td><td>Cycloviolacin O11</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">31</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4000">4000</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCISSAIGCSCKNKVCYRNGIP</td><td>Cycloviolacin O4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCSCSCTTTTBTTTBCCCCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=4000&amp;di=1" width="5px" height="5px"><img src="pymol_images/4000.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4000&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4000.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3998">3998</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCISSAIGCSCKSKVCYRNGIP</td><td>Cycloviolacin O2</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCSSSCSTTCSSSCSSSSSSCCCSSSCC</td><td align="center"><a href="jmol.php?id=3998&amp;di=1" width="5px" height="5px"><img src="pymol_images/3998.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3998&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3998.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4001">4001</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCISSAVGCSCKNKVCYKNGTP</td><td>Cycloviolacin O5</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCSSCCCSTTCSSSSCGGGCCSSCCCSCC</td><td align="center"><a href="jmol.php?id=4001&amp;di=1" width="5px" height="5px"><img src="pymol_images/4001.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4001&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4001.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3999">3999</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCLTSAIGCSCKSKVCYRNGIP</td><td>Cycloviolacin O3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4005">4005</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCLTSAVGCSCKSKVCYRNGIP</td><td>Cycloviolacin O9</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCSTTCCSSCCSBSSSBCCCCCCCSSSCC</td><td align="center"><a href="jmol.php?id=4005&amp;di=1" width="5px" height="5px"><img src="pymol_images/4005.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4005&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4005.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4003">4003</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCTITALAGCKCKSKVCYNSIP</td><td>Cycloviolacin O7</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCTTSSSCCSHHHHSSSCCCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=4003&amp;di=1" width="5px" height="5px"><img src="pymol_images/4003.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4003&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4003.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4009">4009</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVYIPCISGVIGCSCTDKVCYLNGTP</td><td>Kalata B5</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCTTCCCGGGCSSSSSBTTBCCCCCSCC</td><td align="center"><a href="jmol.php?id=4009&amp;di=1" width="5px" height="5px"><img src="pymol_images/4009.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4009&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4009.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4008">4008</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVYIPCLTSAIGCSCKSKVCYRNGIP</td><td>Cycloviolacin H1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCSSSSSTTCCSCSCSCSSSSSCCCSSSCC</td><td align="center"><a href="jmol.php?id=4008&amp;di=1" width="5px" height="5px"><img src="pymol_images/4008.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4008&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4008.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4006">4006</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVYIPCLTSAVGCSCKSKVCYRNGIP</td><td>Cycloviolacin O10</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCSCCCCTTCCCCCTTCSSCCCCSSSCC</td><td align="center"><a href="jmol.php?id=4006&amp;di=1" width="5px" height="5px"><img src="pymol_images/4006.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4006&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4006.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2005">2005</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21928440">21928440</a></td><td align="right">2011</td><td>cgetcvggtcntpgctcswpvctrnglnpv</td><td>D-Kalata B1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>D</td><td>None</td><td align="right">29</td><td>Anticancer and Anti-HIV</td><td>IC<sub>50</sub> =77µM</td><td>Human</td><td>CCCSSSSCCCCCTTTTCCCCSSSSSCSCCC</td><td align="center"><a href="jmol.php?id=2005&amp;di=1" width="5px" height="5px"><img src="pymol_images/2005.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2005&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2005.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2004">2004</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21928440">21928440</a></td><td align="right">2011</td><td>CGETCVGGTCNTPGCTCSWPVCTRNGLPV</td><td>Kalata B1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Anticancer and Anti-HIV</td><td>IC<sub>50</sub> =26µM</td><td>Human</td><td>CCCGGGTTTSSCCCCCSSSCCCTTCSSCC</td><td align="center"><a href="jmol.php?id=2004&amp;di=1" width="5px" height="5px"><img src="pymol_images/2004.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2004&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2004.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2484">2484</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10747913">10747913</a></td><td align="right">2000</td><td>CGETCVGGTCNTPGCTCSWPVCTRNGLPV</td><td>Kalata B1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Anti-HIV</td><td>50% hemolytic at 50μM</td><td>Human</td><td>CCCGGGTTTSSCCCCCSSSCCCTTCSSCC</td><td align="center"><a href="jmol.php?id=2484&amp;di=1" width="5px" height="5px"><img src="pymol_images/2484.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2484&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2484.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4135">4135</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10441158">10441158</a></td><td align="right">1999</td><td>CGETCVGGTCNTPGCTCSWPVCTRNGLPV</td><td>Kalata B1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Uterotonic</td><td>HD<sub>50</sub> =50μM</td><td>Human</td><td>CCCGGGTTTSSCCCCCSSSCCCTTCSSCC</td><td align="center"><a href="jmol.php?id=4135&amp;di=1" width="5px" height="5px"><img src="pymol_images/4135.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4135&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4135.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2533">2533</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10821683">10821683</a></td><td align="right">2000</td><td>CH3-S-CKLLLKWLLKLLKC-S-CH3</td><td>Linear-K<sup>4</sup>L<sup>7</sup>W</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Cytolytic</td><td>100% hemolytic at 5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2535">2535</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10821683">10821683</a></td><td align="right">2000</td><td>CH3-S-cklllkwllkllkc-S-CH3</td><td>linear-[D]-L<sup>3,4,8,10</sup>-K<sup>4</sup>L<sup>7</sup>W</td><td>Free</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">14</td><td>Cytolytic</td><td>5% hemolytic at 50μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3091">3091</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20735987">20735987</a></td><td align="right">2010</td><td>CIAKGNGCQPSGVQGNCCSGHCHKEPGWVAGYCK</td><td>Psacotheasin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antifungal</td><td>0% hemolysis at 100μM</td><td>Human</td><td>CCSCSSSCCSSCSSCSSSSCCCSSSSCCCTTTTC</td><td align="center"><a href="jmol.php?id=3091&amp;di=1" width="5px" height="5px"><img src="pymol_images/3091.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3091&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3091.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3517">3517</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9026036">9026036</a></td><td align="right">1997</td><td>CIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Dioleoylmelittin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Membrane disturbing</td><td>~20% hemolysis at 1μM</td><td>Dog</td><td>CHHHHHHHHTTCCHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3517&amp;di=1" width="5px" height="5px"><img src="pymol_images/3517.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3517&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3517.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3518">3518</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9026036">9026036</a></td><td align="right">1997</td><td>CIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Dioleoylmelittin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Membrane disturbing</td><td>100% hemolysis at 10μM</td><td>Dog</td><td>CHHHHHHHHTTCCHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3518&amp;di=1" width="5px" height="5px"><img src="pymol_images/3518.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3518&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3518.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1694">1694</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16848415">16848415</a></td><td align="right">2006</td><td>CIGAVLKVLTTGLPALISWIKRKRQQC</td><td>M1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Hemolytic</td><td>HL<sub>50</sub> = 3.0 at pH 7.4</td><td>Mouse</td><td>CHHHHHHHHHHHCHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1694&amp;di=1" width="5px" height="5px"><img src="pymol_images/1694.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1694&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1694.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1695">1695</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16848415">16848415</a></td><td align="right">2006</td><td>CIGAVLKVLTTGLPALISWIKRKRQQC</td><td>M1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Hemolytic</td><td>HL<sub>50</sub> = 1μM at pH 4.5</td><td>Mouse</td><td>CHHHHHHHHHHHCHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1695&amp;di=1" width="5px" height="5px"><img src="pymol_images/1695.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1695&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1695.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1696">1696</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16848415">16848415</a></td><td align="right">2006</td><td>CKKIGAVLKVLTTGLPALISWIKRKRQQKKC</td><td>M6</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">31</td><td>Hemolytic</td><td>HL<sub>50</sub> = 2.5μM at pH 7.4</td><td>Mouse</td><td>CCCSSCCGGGGGTSSTTHHHHHHTHHHHCCC</td><td align="center"><a href="jmol.php?id=1696&amp;di=1" width="5px" height="5px"><img src="pymol_images/1696.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1696&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1696.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3455">3455</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10604598">10604598</a></td><td align="right">1999</td><td>CKLLKTFLSKWIC</td><td>SC</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>100% hemolysis at 40μg/ml</td><td>Rat</td><td>CCHHHHTGGGTTC</td><td align="center"><a href="jmol.php?id=3455&amp;di=1" width="5px" height="5px"><img src="pymol_images/3455.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3455&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3455.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3456">3456</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10604598">10604598</a></td><td align="right">1999</td><td>CKLLKTFLSKWIC</td><td>SAC</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>100% hemolysis at 5μg/ml</td><td>Rat</td><td>CCHHHHTGGGTTC</td><td align="center"><a href="jmol.php?id=3456&amp;di=1" width="5px" height="5px"><img src="pymol_images/3456.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3456&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3456.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2532">2532</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10821683">10821683</a></td><td align="right">2000</td><td>CKLLLKWLLKLLKC</td><td>Cyclic-K<sup>4</sup>L<sup>7</sup>W</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">14</td><td>Cytolytic</td><td>50% hemolytic at 50μM</td><td>Human</td><td>CTTHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2532&amp;di=1" width="5px" height="5px"><img src="pymol_images/2532.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2532&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2532.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2534">2534</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10821683">10821683</a></td><td align="right">2000</td><td>cklllkwllkllkc</td><td>cyclic-[D]-L<sup>3,4,8,10</sup>-K<sup>4</sup>L<sup>7</sup>W</td><td>Free</td><td>Free</td><td>Cyclic</td><td>D</td><td>None</td><td align="right">14</td><td>Cytolytic</td><td>90% hemolytic at 50μM</td><td>Human</td><td>CTTHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2534&amp;di=1" width="5px" height="5px"><img src="pymol_images/2534.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2534&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2534.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3292">3292</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/6325405">6325405</a></td><td align="right">1984</td><td>CKTRTSTNTDHLWIAKSKARLSET</td><td>Synthetic peptide-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Not mentioned</td><td>0% hemolysis upto 500μg/ml (non hemolytic)</td><td>Sheep</td><td>CCSSSCCSSCCTTTSCCTTSCSCC</td><td align="center"><a href="jmol.php?id=3292&amp;di=1" width="5px" height="5px"><img src="pymol_images/3292.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3292&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3292.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2692">2692</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22083756">22083756</a></td><td align="right">2012</td><td>CLRIGMRGRELMGGIGKTM</td><td>pCM19</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>0.109% hemolytic at 100μg/mL</td><td>Human</td><td>CCCSCCEESSSSEECCCCC</td><td align="center"><a href="jmol.php?id=2692&amp;di=1" width="5px" height="5px"><img src="pymol_images/2692.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2692&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2692.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2693">2693</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22083756">22083756</a></td><td align="right">2012</td><td>CLRIGMRGRELMGGIGKTM</td><td>pCM19</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>0.119% hemolytic at 3000μg/mL</td><td>Human</td><td>CCCSCCEESSSSEECCCCC</td><td align="center"><a href="jmol.php?id=2693&amp;di=1" width="5px" height="5px"><img src="pymol_images/2693.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2693&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2693.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3397">3397</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10430870">10430870</a></td><td align="right">1999</td><td>CSCKNKVCYRNGIPCGESCVWIPCISAALG</td><td>Cir A</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>50% hemolysis at &gt;400μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3398">3398</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10430870">10430870</a></td><td align="right">1999</td><td>CSCKNKVCYRNGVIPCGESCVFIPCISTLLG</td><td>Cir B</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>50% hemolysis at &gt;400μM</td><td>Human</td><td>CCCCCCCSSSSSSSCSSCCSSCCCSSSCCCC</td><td align="center"><a href="jmol.php?id=3398&amp;di=1" width="5px" height="5px"><img src="pymol_images/3398.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3398&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3398.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3399">3399</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10430870">10430870</a></td><td align="right">1999</td><td>CSCKSKVCYKNSIPCGESCVFIPCTVTALLG</td><td>CPT</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>50% hemolysis at &gt;400μM</td><td>Human</td><td>CCSSSSSSSSSSSCSSSCCCCCSSSSHHHHC</td><td align="center"><a href="jmol.php?id=3399&amp;di=1" width="5px" height="5px"><img src="pymol_images/3399.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3399&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3399.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3396">3396</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10430870">10430870</a></td><td align="right">1999</td><td>CTCSWPVCTRNGLPVCGETCVGGTCNTPG</td><td>Kalata B1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>50% hemolysis at &gt;400μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3150">3150</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>CVDIHVWDGVC</td><td>Cyclic C-PIP[59-67]C</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">11</td><td>Cytotoxic</td><td>7.2% hemolysis at 100μM</td><td>Human</td><td>CCCSSCCCTTC</td><td align="center"><a href="jmol.php?id=3150&amp;di=1" width="5px" height="5px"><img src="pymol_images/3150.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3150&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3150.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4023">4023</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>CVHWQTNPARTSCIGP</td><td>Nigroain-D1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CCCCCSSSSSSCSSCC</td><td align="center"><a href="jmol.php?id=4023&amp;di=1" width="5px" height="5px"><img src="pymol_images/4023.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4023&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4023.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4021">4021</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>CVISAGWNHKIRCKLTGNC</td><td>Nigroain-B1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CCCSTTCSSCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=4021&amp;di=1" width="5px" height="5px"><img src="pymol_images/4021.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4021&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4021.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2373">2373</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CVKVKVKVGSGVKVKVKVC</td><td>V1</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =880000nM</td><td>Human</td><td>CCCCSSCCSSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=2373&amp;di=1" width="5px" height="5px"><img src="pymol_images/2373.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2373&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2373.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2376">2376</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CVKVQVKVGSGVKVQVKVC</td><td>V4</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =3226000nM</td><td>Human</td><td>CCCCSSSSSCCSCCCCCCC</td><td align="center"><a href="jmol.php?id=2376&amp;di=1" width="5px" height="5px"><img src="pymol_images/2376.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2376&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2376.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2375">2375</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CVKVRVKVGSGVKVRVKVC</td><td>V3</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =590000nM</td><td>Human</td><td>CCCCSSCCSSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=2375&amp;di=1" width="5px" height="5px"><img src="pymol_images/2375.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2375&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2375.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2374">2374</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CVKVSVKVGSGVKVSVKVC</td><td>V2</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =5,89,000nM</td><td>Human</td><td>CCCCCEECSSSEECCCSCC</td><td align="center"><a href="jmol.php?id=2374&amp;di=1" width="5px" height="5px"><img src="pymol_images/2374.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2374&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2374.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3121">3121</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21241661">21241661</a></td><td align="right">2011</td><td>CVYAYVRVRGVLVRYRRCW</td><td>Arenicin-1(analog-1)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 50μM</td><td>Human</td><td>CCSTTSCCCCCCCCCCSCC</td><td align="center"><a href="jmol.php?id=3121&amp;di=1" width="5px" height="5px"><img src="pymol_images/3121.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3121&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3121.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3122">3122</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21241661">21241661</a></td><td align="right">2011</td><td>CVYAYVRVRGVLVRYRRCW</td><td>Arenicin-1(analog-1)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>4% hemolysis at 100μM</td><td>Human</td><td>CCSTTSCCCCCCCCCCSCC</td><td align="center"><a href="jmol.php?id=3122&amp;di=1" width="5px" height="5px"><img src="pymol_images/3122.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3122&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3122.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2379">2379</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CWKWKWKWGSGWKWKWKWC</td><td>V7</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =45000nM</td><td>Human</td><td>CCCCGGGCCSSSSSCSSCC</td><td align="center"><a href="jmol.php?id=2379&amp;di=1" width="5px" height="5px"><img src="pymol_images/2379.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2379&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2379.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1190">1190</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18455267">18455267</a></td><td align="right">2008</td><td>CYCFRRFCVC</td><td>PC74</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>0-3% hemolysis at 80μg/ml</td><td>Human</td><td>CTTTTSSSCC</td><td align="center"><a href="jmol.php?id=1190&amp;di=1" width="5px" height="5px"><img src="pymol_images/1190.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1190&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1190.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2433">2433</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>CYCRIPACIAGERRYGTCIYQGRLWAFCC</td><td>HNP-2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>0.9% hemolysis at 80µg/ml</td><td>Human</td><td>CCCCCSCCCCSCTTCCEEEETTEEEECCC</td><td align="center"><a href="jmol.php?id=2433&amp;di=1" width="5px" height="5px"><img src="pymol_images/2433.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2433&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2433.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2982">2982</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>CYCRKGHCKRGERVRGTCGIRFLYCCPRRGLL</td><td>Crp4-1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>4.2% hemolytic at 0μg/ml</td><td>Human</td><td>CCCSSCCCSCCCCCCCCCCCCCSCCCTTSSCC</td><td align="center"><a href="jmol.php?id=2982&amp;di=1" width="5px" height="5px"><img src="pymol_images/2982.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2983">2983</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>CYCRKGHCKRGERVRGTCGIRFLYCCPRRGLL</td><td>Crp4-1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>5.4% hemolytic at 12.5μg/ml</td><td>Human</td><td>CCCSSCCCSCCCCCCCCCCCCCSCCCTTSSCC</td><td align="center"><a href="jmol.php?id=2983&amp;di=1" width="5px" height="5px"><img src="pymol_images/2983.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2984">2984</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>CYCRKGHCKRGERVRGTCGIRFLYCCPRRGLL</td><td>Crp4-1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>5.8% hemolytic at 25μg/ml</td><td>Human</td><td>CCCSSCCCSCCCCCCCCCCCCCSCCCTTSSCC</td><td align="center"><a href="jmol.php?id=2984&amp;di=1" width="5px" height="5px"><img src="pymol_images/2984.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2985">2985</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>CYCRKGHCKRGERVRGTCGIRFLYCCPRRGLL</td><td>Crp4-1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>5.9% hemolytic at 50μg/ml</td><td>Human</td><td>CCCSSCCCSCCCCCCCCCCCCCSCCCTTSSCC</td><td align="center"><a href="jmol.php?id=2985&amp;di=1" width="5px" height="5px"><img src="pymol_images/2985.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2986">2986</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>CYCRKGHCKRGERVRGTCGIRFLYCCPRRGLL</td><td>Crp4-1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>4.9% hemolytic at 100μg/mL</td><td>Human</td><td>CCCSSCCCSCCCCCCCCCCCCCSCCCTTSSCC</td><td align="center"><a href="jmol.php?id=2986&amp;di=1" width="5px" height="5px"><img src="pymol_images/2986.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1189">1189</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18455267">18455267</a></td><td align="right">2008</td><td>CYCRRRFCVC</td><td>PC73</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>0-3% hemolysis at 80μg/ml</td><td>Human</td><td>CCCCSSSSCC</td><td align="center"><a href="jmol.php?id=1189&amp;di=1" width="5px" height="5px"><img src="pymol_images/1189.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1189&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1189.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1187">1187</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18455267">18455267</a></td><td align="right">2008</td><td>CYCRRRFCVCV</td><td>PC71</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0-3% hemolysis at 80μg/ml</td><td>Human</td><td>CCCBTTBCCCC</td><td align="center"><a href="jmol.php?id=1187&amp;di=1" width="5px" height="5px"><img src="pymol_images/1187.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1187&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1187.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1180">1180</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18455267">18455267</a></td><td align="right">2008</td><td>CYCRRRFCVCVGR</td><td>PC37</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3-6% hemolysis at 80μg/ml </td><td>Human</td><td>CCEETTEECCCCC</td><td align="center"><a href="jmol.php?id=1180&amp;di=1" width="5px" height="5px"><img src="pymol_images/1180.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1180&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1180.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1030">1030</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18243710">18243710</a></td><td align="right">2008</td><td>DAACAAHCLWR</td><td>DHWR</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;200μg/ml</td><td>Human</td><td>CCSSSCCSTTC</td><td align="center"><a href="jmol.php?id=1030&amp;di=1" width="5px" height="5px"><img src="pymol_images/1030.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1030&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1030.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1031">1031</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18243710">18243710</a></td><td align="right">2008</td><td>DAACAAHCLWR</td><td>DHWRox</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;200μg/ml</td><td>Human</td><td>CCSSSCCSTTC</td><td align="center"><a href="jmol.php?id=1031&amp;di=1" width="5px" height="5px"><img src="pymol_images/1031.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1658">1658</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16226345">16226345</a></td><td align="right">2006</td><td>DAACAAHCLWR</td><td>L1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>100% hemolysis at 50μg/ml</td><td>Human</td><td>CCSSSCCSTTC</td><td align="center"><a href="jmol.php?id=1658&amp;di=1" width="5px" height="5px"><img src="pymol_images/1658.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1658&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1658.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1032">1032</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18243710">18243710</a></td><td align="right">2008</td><td>DAACAAKCLWR</td><td>DKWR</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;200μg/ml</td><td>Human</td><td>CTTTTTTSTTC</td><td align="center"><a href="jmol.php?id=1032&amp;di=1" width="5px" height="5px"><img src="pymol_images/1032.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1032&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1032.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1033">1033</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18243710">18243710</a></td><td align="right">2008</td><td>DAACAAKCLWR</td><td>DKWRox</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;200μg/ml</td><td>Human</td><td>CTTTTTTSTTC</td><td align="center"><a href="jmol.php?id=1033&amp;di=1" width="5px" height="5px"><img src="pymol_images/1033.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1898">1898</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16226345">16226345</a></td><td align="right">2006</td><td>DAACAAKCLWR</td><td>L2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0% hemolysis at 200μg/ml (non-hemolytic)</td><td>Human</td><td>CTTTTTTSTTC</td><td align="center"><a href="jmol.php?id=1898&amp;di=1" width="5px" height="5px"><img src="pymol_images/1898.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1898&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1898.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3025">3025</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22029824">22029824</a></td><td align="right">2012</td><td>DANVENGEDAEDLTDKFIGLMG</td><td>Andersonin-S1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>Non-hemolytic upto 50μg/mL</td><td>Human</td><td>CCCSCCSCCSCCCTHHHHSTTC</td><td align="center"><a href="jmol.php?id=3025&amp;di=1" width="5px" height="5px"><img src="pymol_images/3025.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3025&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3025.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4024">4024</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>DCTRWIIGINGRICRD</td><td>Nigroain-E</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>7.62±1.60% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CCSSSCTTSCCCSSCC</td><td align="center"><a href="jmol.php?id=4024&amp;di=1" width="5px" height="5px"><img src="pymol_images/4024.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4024&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4024.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2125">2125</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>DFKDWMKTAGEWLKKKGPGILKAAMAAAT</td><td>Ponericin-G4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Horse</td><td>CTTTGGGTHHHHTTTTSSCSTHHHHTSCC</td><td align="center"><a href="jmol.php?id=2125&amp;di=1" width="5px" height="5px"><img src="pymol_images/2125.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2125&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2125.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2126">2126</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>DFKDWMKTAGEWLKKKGPGILKAAMAAAT</td><td>Ponericin-G4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Sheep</td><td>CTTTGGGTHHHHTTTTSSCSTHHHHTSCC</td><td align="center"><a href="jmol.php?id=2126&amp;di=1" width="5px" height="5px"><img src="pymol_images/2126.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2126&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2126.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3246">3246</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23262358">23262358</a></td><td align="right">2013</td><td>DLLGTLGNLPLPFI</td><td>frenatin 2D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Immunomodulatory</td><td>No hemolysis upto 500μM</td><td>Human</td><td>CCSSSTTSCCCSCC</td><td align="center"><a href="jmol.php?id=3246&amp;di=1" width="5px" height="5px"><img src="pymol_images/3246.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1632">1632</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15629533">15629533</a></td><td align="right">2005</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide n-16</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Hemolytic</td><td>&lt;25% hemolysis at 30µM</td><td>Guinea-pig</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=1632&amp;di=1" width="5px" height="5px"><img src="pymol_images/1632.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1632&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1632.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3337">3337</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>0% hemolysis at 75μg</td><td>Human</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3337&amp;di=1" width="5px" height="5px"><img src="pymol_images/3337.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3337&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3337.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3338">3338</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>12% hemolysis at 250μg</td><td>Human</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3338&amp;di=1" width="5px" height="5px"><img src="pymol_images/3338.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3338&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3338.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3339">3339</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>18% hemolysis at 500μg</td><td>Human</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3339&amp;di=1" width="5px" height="5px"><img src="pymol_images/3339.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3339&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3339.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3340">3340</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>0% hemolysis at 75μg</td><td>Horse</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3340&amp;di=1" width="5px" height="5px"><img src="pymol_images/3340.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3340&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3340.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3341">3341</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>20% hemolysis at 250μg</td><td>Horse</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3341&amp;di=1" width="5px" height="5px"><img src="pymol_images/3341.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3341&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3341.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3342">3342</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>19% hemolysis at 500μg</td><td>Horse</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3342&amp;di=1" width="5px" height="5px"><img src="pymol_images/3342.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3342&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3342.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3343">3343</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>0% hemolysis at 75μg</td><td>Guinea-pig</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3343&amp;di=1" width="5px" height="5px"><img src="pymol_images/3343.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3343&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3343.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3344">3344</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>23% hemolysis at 250μg</td><td>Guinea-pig</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3344&amp;di=1" width="5px" height="5px"><img src="pymol_images/3344.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3344&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3344.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3345">3345</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>51% hemolysis at 500μg</td><td>Guinea-pig</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3345&amp;di=1" width="5px" height="5px"><img src="pymol_images/3345.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3345&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3345.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3346">3346</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>0% hemolysis at 250μg</td><td>Rabbit</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3346&amp;di=1" width="5px" height="5px"><img src="pymol_images/3346.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3346&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3346.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3347">3347</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>14% hemolysis at 500μg</td><td>Rabbit</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3347&amp;di=1" width="5px" height="5px"><img src="pymol_images/3347.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3347&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3347.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3348">3348</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>0% hemolysis at 250μg</td><td>Sheep</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3348&amp;di=1" width="5px" height="5px"><img src="pymol_images/3348.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3348&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3348.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3349">3349</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>17% hemolysis at 500μg</td><td>Sheep</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3349&amp;di=1" width="5px" height="5px"><img src="pymol_images/3349.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3349&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3349.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2029">2029</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10817697">10817697</a></td><td align="right">2000</td><td>DSHAKRHHGYKIKFHEKHHSLRGY</td><td>R12I/H21L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antifungal</td><td>&lt;10% hemolysis at 200µM</td><td>Human</td><td>CCSSCSSCCCCSSGGGGTSCCSCC</td><td align="center"><a href="jmol.php?id=2029&amp;di=1" width="5px" height="5px"><img src="pymol_images/2029.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2028">2028</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10817697">10817697</a></td><td align="right">2000</td><td>DSHAKRHHGYKIKFHENHHSHRGY</td><td>R12I/K17N</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antifungal</td><td>&lt;10% hemolysis at 200µM</td><td>Human</td><td>CCCTTSCCCCCCCSSCCCCSSSCC</td><td align="center"><a href="jmol.php?id=2028&amp;di=1" width="5px" height="5px"><img src="pymol_images/2028.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2027">2027</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10817697">10817697</a></td><td align="right">2000</td><td>DSHAKRHHGYKRKFHEKHHSHRGY</td><td>Hsn-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antifungal</td><td>&lt;10% hemolysis at 200µM</td><td>Human</td><td>CCSTTTTSSCCHHHHTTTSCCSCC</td><td align="center"><a href="jmol.php?id=2027&amp;di=1" width="5px" height="5px"><img src="pymol_images/2027.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1872">1872</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15196144">15196144</a></td><td align="right">2004</td><td>DSHEKRHHGYKRKFHEKHHSHRGY</td><td>Histatin-5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antifungal</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCSSCCSSSCCSCSSCSSSCCSCC</td><td align="center"><a href="jmol.php?id=1872&amp;di=1" width="5px" height="5px"><img src="pymol_images/1872.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1872&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1872.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3068">3068</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20457198">20457198</a></td><td align="right">2010</td><td>DVNDLKNLCAKTHNLLPMCAMF</td><td>DF-22-amide</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>No hemolysis upto &lt;400μM</td><td>Human</td><td>CCSTHHHHHHHSSTTSSSSTTC</td><td align="center"><a href="jmol.php?id=3068&amp;di=1" width="5px" height="5px"><img src="pymol_images/3068.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3068&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3068.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3067">3067</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20457198">20457198</a></td><td align="right">2010</td><td>DVNDLKNLCAKTHNLLPMCAMFGKK</td><td>DK25</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>No hemolysis upto &lt;400μM</td><td>Human</td><td>CCCTTGGGGGTSSCCCHHHHHTCCC</td><td align="center"><a href="jmol.php?id=3067&amp;di=1" width="5px" height="5px"><img src="pymol_images/3067.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3067&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3067.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1202">1202</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18455267">18455267</a></td><td align="right">2008</td><td>ECYCRRRFCVCVGR</td><td>PC98</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>3-6% hemolysis at 80μg/ml</td><td>Human</td><td>CTTCSSCCSCCTTC</td><td align="center"><a href="jmol.php?id=1202&amp;di=1" width="5px" height="5px"><img src="pymol_images/1202.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1202&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1202.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2909">2909</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21343499">21343499</a></td><td align="right">2011</td><td>EEEEAAAkwklfkkipkflhlakkf</td><td>Ac-E4-A3–D-P18</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">25</td><td>Antimcrobial and Hemolytic</td><td>~3.5% hemolytic upto 500μM</td><td>Human</td><td>CTTTTTHHHHTTTTCSSCSSCTTCC</td><td align="center"><a href="jmol.php?id=2909&amp;di=1" width="5px" height="5px"><img src="pymol_images/2909.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2909&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2909.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1755">1755</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>EGKGLGβAlaβAlaKTicOicβAlaFTicOicβAlaETicOicβAlaFTicOicELMNS</td><td>Peptide-21</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Tic- tetrahydroisoquinolinecarboxylic acid, Oic- octahydroindolecarboxylic acid, βAla- β-alanine</td><td align="right">28</td><td>Antibiotic</td><td>27% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1756">1756</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>EGKGLGβAlaβAlaKTicOicβAlaFTicOicβAlaETicOicβAlaFTicOicELMNS</td><td>Peptide-21</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Tic- tetrahydroisoquinolinecarboxylic acid, Oic- octahydroindolecarboxylic acid, βAla- β-alanine</td><td align="right">28</td><td>Antibiotic</td><td>27% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2030">2030</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10817697">10817697</a></td><td align="right">2000</td><td>EGRERDHELRHRRHHHQSPK</td><td>MUC7 D1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antifungal</td><td>&lt;10% hemolysis at 200µM</td><td>Human</td><td>CTTCCCTTCSSSCSSSSCCC</td><td align="center"><a href="jmol.php?id=2030&amp;di=1" width="5px" height="5px"><img src="pymol_images/2030.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2030&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2030.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1816">1816</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17826814">17826814</a></td><td align="right">2007</td><td>ELAGTIIDGASLTFEVLDKVLGELGKVSRK</td><td>St I 2–31</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Hemolytic</td><td>Hemolytic activity expresssed as the initial rate (∆A/∆t, min<sup>-1</sup>) <i>i.e.</i> at above 100</td><td>Human</td><td>CCSTTSCCSSSSCHHHHHHHTCHHHHSCCC</td><td align="center"><a href="jmol.php?id=1816&amp;di=1" width="5px" height="5px"><img src="pymol_images/1816.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1816&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1816.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1291">1291</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19345214">19345214</a></td><td align="right">2009</td><td>ELLKAVRLIK</td><td>Rev-NIS</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antifungal</td><td>0% hemolysis at 1-100μM(non-hemolytic)</td><td>Human</td><td>CCSSCSCCCC</td><td align="center"><a href="jmol.php?id=1291&amp;di=1" width="5px" height="5px"><img src="pymol_images/1291.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1291&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1291.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2999">2999</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21762675">21762675</a></td><td align="right">2011</td><td>ELLKAVRLIK</td><td>Rev–NIS</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>0% hemolytic upto 100μM(non-hemolytic)</td><td>Human</td><td>CCSSCSCCCC</td><td align="center"><a href="jmol.php?id=2999&amp;di=1" width="5px" height="5px"><img src="pymol_images/2999.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2999&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2999.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1749">1749</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>ELMNSTicOicGKLGKTicOicELMS</td><td>Peptide-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid</td><td align="right">18</td><td>Antibiotic</td><td>23% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1750">1750</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>ELMNSTicOicGKLGKTicOicELMS</td><td>Peptide-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid</td><td align="right">18</td><td>Antibiotic</td><td>23% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1747">1747</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>ELMNSTicOicGLTicOicGKTicOicGLTicOicGKTicOicELMNS</td><td>Peptide-7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid</td><td align="right">28</td><td>Antibiotic</td><td>3% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1748">1748</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>ELMNSTicOicGLTicOicGKTicOicGLTicOicGKTicOicELMNS</td><td>Peptide-7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid</td><td align="right">28</td><td>Antibiotic</td><td>3% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3001">3001</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21762675">21762675</a></td><td align="right">2011</td><td>ELSKAVRLIK</td><td>Anal S</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>0% hemolytic upto 100μM (non-hemolytic)</td><td>Human</td><td>CTTTSSCCCC</td><td align="center"><a href="jmol.php?id=3001&amp;di=1" width="5px" height="5px"><img src="pymol_images/3001.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3001&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3001.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1751">1751</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>ELSTicOicGKGTicOicLKES</td><td>Peptide-9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid</td><td align="right">14</td><td>Antibiotic</td><td>23% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1752">1752</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>ELSTicOicGKGTicOicLKES</td><td>Peptide-9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid</td><td align="right">14</td><td>Antibiotic</td><td>23% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3023">3023</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22029824">22029824</a></td><td align="right">2012</td><td>EMLKKKEVKMERKT</td><td>Andersonin-Q1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>Non-hemolytic upto 50μg/mL</td><td>Human</td><td>CCCSSSCTTTCTTC</td><td align="center"><a href="jmol.php?id=3023&amp;di=1" width="5px" height="5px"><img src="pymol_images/3023.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3023&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3023.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3024">3024</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22029824">22029824</a></td><td align="right">2012</td><td>ENAEEDIVLMENLFCSYIVGSADSFWT</td><td>Andersonin-R1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>Non-hemolytic upto 50μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3022">3022</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22029824">22029824</a></td><td align="right">2012</td><td>ENMFNIKSSVESDSFWG</td><td>Andersonin-N1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>Non-hemolytic upto 50μg/mL</td><td>Human</td><td>CCCCSSCBTTBCHHHHC</td><td align="center"><a href="jmol.php?id=3022&amp;di=1" width="5px" height="5px"><img src="pymol_images/3022.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3022&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3022.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1137">1137</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18098173">18098173</a></td><td align="right">2007</td><td>EWESFLETFESAKETVLHTALEAISS</td><td>-5 Peptide</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC&gt;1000.0μg/ml</td><td>Human</td><td>CCSSHHHHTTTSGGGGGGHHHHTTTC</td><td align="center"><a href="jmol.php?id=1137&amp;di=1" width="5px" height="5px"><img src="pymol_images/1137.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1893">1893</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15949628">15949628</a></td><td align="right">2005</td><td>EWGRRCCGWGPGRRYCRRWC</td><td>Ib-AMP4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>No hemolysis upto 100μM (non-hemolytic)</td><td>Rabbit</td><td>CCSCCCSSSCTTSTTTCSCC</td><td align="center"><a href="jmol.php?id=1893&amp;di=1" width="5px" height="5px"><img src="pymol_images/1893.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1892">1892</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15949628">15949628</a></td><td align="right">2005</td><td>EWGRRCCGWGPGRRYCVRWC</td><td>Ib-AMP1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>No hemolysis upto 100μM (non-hemolytic)</td><td>Rabbit</td><td>CCTTCSSCCCSCCCCCCCCC</td><td align="center"><a href="jmol.php?id=1892&amp;di=1" width="5px" height="5px"><img src="pymol_images/1892.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1892&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1892.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1559">1559</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21110126">21110126</a></td><td align="right">2011</td><td>FAKWAFKWAKK</td><td>GA-K4A</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>MHC ≥200μg/ml</td><td>Human</td><td>CCHHHHHSCCC</td><td align="center"><a href="jmol.php?id=1559&amp;di=1" width="5px" height="5px"><img src="pymol_images/1559.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1559&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1559.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1560">1560</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21110126">21110126</a></td><td align="right">2011</td><td>FAKWAFKWLKK</td><td>GA-K4AL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>MHC ≥200μg/ml</td><td>Human</td><td>CHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=1560&amp;di=1" width="5px" height="5px"><img src="pymol_images/1560.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1560&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1560.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2883">2883</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FAPLIGRVLSGIL</td><td>Ta A2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>15% hemolytic at 40μM</td><td>Human</td><td>CCCCHHHHTSSCC</td><td align="center"><a href="jmol.php?id=2883&amp;di=1" width="5px" height="5px"><img src="pymol_images/2883.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2883&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2883.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2884">2884</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FAPLIGRVLSGIL</td><td>Ta A2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CCCCHHHHTSSCC</td><td align="center"><a href="jmol.php?id=2884&amp;di=1" width="5px" height="5px"><img src="pymol_images/2884.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2884&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2884.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1200">1200</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18455267">18455267</a></td><td align="right">2008</td><td>FCYCRRRFCVCVGR</td><td>PC96</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>&gt;50% hemolysis at 80μg/ml</td><td>Human</td><td>CCCGGGSCCCCSCC</td><td align="center"><a href="jmol.php?id=1200&amp;di=1" width="5px" height="5px"><img src="pymol_images/1200.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1200&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1200.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2060">2060</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>FDLLGLVKKVASAL</td><td>EpVP2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCGGGGTTHHHHTC</td><td align="center"><a href="jmol.php?id=2060&amp;di=1" width="5px" height="5px"><img src="pymol_images/2060.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2060&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2060.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2061">2061</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>FDLLGLVKSVVSAL</td><td>EpVP2b</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>EC<sub>50</sub> =64(68–70)µM</td><td>Human</td><td>CCTTSSSTTTTTTC</td><td align="center"><a href="jmol.php?id=2061&amp;di=1" width="5px" height="5px"><img src="pymol_images/2061.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2061&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2061.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2607">2607</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23226256">23226256</a></td><td align="right">2013</td><td>FDWDSVLKGVEGFVRGYF</td><td>TP1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>0% hemolytic at 100 µg/ml (non-hemolytic)</td><td>Fish</td><td>CCSHHHHHHHHTHHHHCC</td><td align="center"><a href="jmol.php?id=2607&amp;di=1" width="5px" height="5px"><img src="pymol_images/2607.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2607&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2607.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1501">1501</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20308076">20308076</a></td><td align="right">2010</td><td>FFFLSRIF</td><td>Temporin-SHf</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>LC<sub>50</sub> =200μM</td><td>Human</td><td>CCCCSSCC</td><td align="center"><a href="jmol.php?id=1501&amp;di=1" width="5px" height="5px"><img src="pymol_images/1501.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1501&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1501.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3485">3485</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19427345">19427345</a></td><td align="right">2009</td><td>FFGAIAAALPHVISAIKNAL</td><td>Kassinatuerin-2Mb</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>45-50% hemolysis at 120μM</td><td>Horse</td><td>CCCTHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3485&amp;di=1" width="5px" height="5px"><img src="pymol_images/3485.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3616">3616</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19088182">19088182</a></td><td align="right">2009</td><td>FFGHLFKLATKIIPSLFQ</td><td>Meucin-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>74% hemolysis at 6.25µM</td><td>Rabbit</td><td>CTTHHHHHGGGGSSSSCC</td><td align="center"><a href="jmol.php?id=3616&amp;di=1" width="5px" height="5px"><img src="pymol_images/3616.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3616&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3616.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3617">3617</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19088182">19088182</a></td><td align="right">2009</td><td>FFGHLFKLATKIIPSLFQ</td><td>Meucin-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>&gt;90% hemolysis at 12.5µM</td><td>Rabbit</td><td>CTTHHHHHGGGGSSSSCC</td><td align="center"><a href="jmol.php?id=3617&amp;di=1" width="5px" height="5px"><img src="pymol_images/3617.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3617&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3617.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3618">3618</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19088182">19088182</a></td><td align="right">2009</td><td>FFGHLFKLATKIIPSLFQ</td><td>Meucin-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>&gt;95% hemolysis at 25µM</td><td>Rabbit</td><td>CTTHHHHHGGGGSSSSCC</td><td align="center"><a href="jmol.php?id=3618&amp;di=1" width="5px" height="5px"><img src="pymol_images/3618.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3618&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3618.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2615">2615</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23093034">23093034</a></td><td align="right">2013</td><td>FFGSLLSLGSKLLPSVFKLFQRKKE</td><td>Css54</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>83% hemolytic at 25mM</td><td>Human</td><td>CTTSSSSSCCCSCHHHHHHHSSSCC</td><td align="center"><a href="jmol.php?id=2615&amp;di=1" width="5px" height="5px"><img src="pymol_images/2615.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2615&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2615.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1860">1860</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18052076">18052076</a></td><td align="right">2007</td><td>FFGWLIKGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>EC<sub>50</sub> = 1µM</td><td>Human</td><td>CCGGGSTTTGGGTCCCCSSCSSSCC</td><td align="center"><a href="jmol.php?id=1860&amp;di=1" width="5px" height="5px"><img src="pymol_images/1860.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1860&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1860.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2278">2278</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FFGWLIKGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>100% hemolysis at 1μM</td><td>Human</td><td>CCGGGSTTTGGGTCCCCSSCSSSCC</td><td align="center"><a href="jmol.php?id=2278&amp;di=1" width="5px" height="5px"><img src="pymol_images/2278.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2278&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2278.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3543">3543</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FFGWLIKGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>~40% hemolysis at 1μM</td><td>Human</td><td>CCGGGSTTTGGGTCCCCSSCSSSCC</td><td align="center"><a href="jmol.php?id=3543&amp;di=1" width="5px" height="5px"><img src="pymol_images/3543.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3543&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3543.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3544">3544</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FFGWLIKGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>100% hemolysis at 100μM</td><td>Human</td><td>CCGGGSTTTGGGTCCCCSSCSSSCC</td><td align="center"><a href="jmol.php?id=3544&amp;di=1" width="5px" height="5px"><img src="pymol_images/3544.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3544&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3544.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2279">2279</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FFGWLIRGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>100% hemolysis at 1μM</td><td>Human</td><td>CTTTTCGGGTTSSCCSSSSSCCCCC</td><td align="center"><a href="jmol.php?id=2279&amp;di=1" width="5px" height="5px"><img src="pymol_images/2279.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2279&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2279.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3545">3545</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FFGWLIRGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>&gt;40% hemolysis at 1μM</td><td>Human</td><td>CTTTTCGGGTTSSCCSSSSSCCCCC</td><td align="center"><a href="jmol.php?id=3545&amp;di=1" width="5px" height="5px"><img src="pymol_images/3545.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3545&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3545.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3546">3546</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FFGWLIRGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>100% hemolysis at 100μM</td><td>Human</td><td>CTTTTCGGGTTSSCCSSSSSCCCCC</td><td align="center"><a href="jmol.php?id=3546&amp;di=1" width="5px" height="5px"><img src="pymol_images/3546.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3546&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3546.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1112">1112</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17328560">17328560</a></td><td align="right">2007</td><td>FFHHIFRAIVHVAKTIHRLVTG</td><td>Pis-1 AA</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Cytotoxic</td><td>HC<sub>50</sub> =6μM</td><td>Human</td><td>CCHHHHHHHTTHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=1112&amp;di=1" width="5px" height="5px"><img src="pymol_images/1112.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1112&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1112.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1118">1118</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17328560">17328560</a></td><td align="right">2007</td><td>FFHHIFRaIVHVaKTIHRLVTG</td><td>Pis-1 aa</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">22</td><td>Cytotoxic</td><td>HC<sub>50</sub> =8μM</td><td>Human</td><td>CCHHHHHHHTTHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=1118&amp;di=1" width="5px" height="5px"><img src="pymol_images/1118.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1118&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1118.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1110">1110</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17328560">17328560</a></td><td align="right">2007</td><td>FFHHIFRAIVHVGKTIHRLVTG</td><td>Pis-1 AG</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Cytotoxic</td><td>HC<sub>50</sub> =4μM</td><td>Human</td><td>CCHHHHHHHHCTTSTTCCCCCC</td><td align="center"><a href="jmol.php?id=1110&amp;di=1" width="5px" height="5px"><img src="pymol_images/1110.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1110&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1110.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1116">1116</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17328560">17328560</a></td><td align="right">2007</td><td>FFHHIFRAIVHVPKTIHRLVTG</td><td>Pis-1 AP</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Cytotoxic</td><td>HC<sub>50</sub> =75μM</td><td>Human</td><td>CHHHHTTHHHHTTTTCTTTSCC</td><td align="center"><a href="jmol.php?id=1116&amp;di=1" width="5px" height="5px"><img src="pymol_images/1116.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1116&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1116.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1111">1111</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17328560">17328560</a></td><td align="right">2007</td><td>FFHHIFRGIVHVAKTIHRLVTG</td><td>Pis-1 GA</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Cytotoxic</td><td>HC<sub>50</sub> =4μM</td><td>Human</td><td>CCCHHHHHHHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=1111&amp;di=1" width="5px" height="5px"><img src="pymol_images/1111.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1111&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1111.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3411">3411</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11713517">11713517</a></td><td align="right">2001</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Piscidin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolysis at 10μg/ml</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3411&amp;di=1" width="5px" height="5px"><img src="pymol_images/3411.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3412">3412</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11713517">11713517</a></td><td align="right">2001</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Piscidin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>~90% hemolysis at 100μg/ml</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3412&amp;di=1" width="5px" height="5px"><img src="pymol_images/3412.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3415">3415</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolysis upto 1.25μM</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3415&amp;di=1" width="5px" height="5px"><img src="pymol_images/3415.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3416">3416</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>1% hemolysis upto 2.5μM</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3416&amp;di=1" width="5px" height="5px"><img src="pymol_images/3416.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3417">3417</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolysis upto 5μM</td><td>Sheep</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3417&amp;di=1" width="5px" height="5px"><img src="pymol_images/3417.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3418">3418</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>19% hemolysis at 5μM</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3418&amp;di=1" width="5px" height="5px"><img src="pymol_images/3418.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3419">3419</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>11% hemolysis at 10μM</td><td>Sheep</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3419&amp;di=1" width="5px" height="5px"><img src="pymol_images/3419.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3420">3420</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>55% hemolysis at 10μM</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3420&amp;di=1" width="5px" height="5px"><img src="pymol_images/3420.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3421">3421</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>51% hemolysis at 20μM</td><td>Sheep</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3421&amp;di=1" width="5px" height="5px"><img src="pymol_images/3421.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3422">3422</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>100% hemolysis at 80μM</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3422&amp;di=1" width="5px" height="5px"><img src="pymol_images/3422.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3423">3423</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>80% hemolysis at 40μM</td><td>Sheep</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3423&amp;di=1" width="5px" height="5px"><img src="pymol_images/3423.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3424">3424</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>100% hemolysis at 80μM</td><td>Sheep</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3424&amp;di=1" width="5px" height="5px"><img src="pymol_images/3424.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1109">1109</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17328560">17328560</a></td><td align="right">2007</td><td>FFHHIFRGIVHVGKTIHRLVTG</td><td>Piscidin 1 (Pis-1)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Cytotoxic</td><td>HC<sub>50</sub> =11μM</td><td>Human</td><td>CHHHHHHHHHHHTTHHHHHHTC</td><td align="center"><a href="jmol.php?id=1109&amp;di=1" width="5px" height="5px"><img src="pymol_images/1109.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1109&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1109.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1249">1249</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18827353">18827353</a></td><td align="right">2008</td><td>FFHHIFRGIVHVGKTIHRLVTG</td><td>Piscidin 1 (P1)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antifungal</td><td>100% hemolysis at 25μM</td><td>Human</td><td>CHHHHHHHHHHHTTHHHHHHTC</td><td align="center"><a href="jmol.php?id=1249&amp;di=1" width="5px" height="5px"><img src="pymol_images/1249.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1250">1250</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18827353">18827353</a></td><td align="right">2008</td><td>FFHHIFRGIVHVGKTIHRLVTG</td><td>Piscidin 1 (P1)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antifungal</td><td>96% hemolysis at 12.5μM</td><td>Human</td><td>CHHHHHHHHHHHTTHHHHHHTC</td><td align="center"><a href="jmol.php?id=1250&amp;di=1" width="5px" height="5px"><img src="pymol_images/1250.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1251">1251</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18827353">18827353</a></td><td align="right">2008</td><td>FFHHIFRGIVHVGKTIHRLVTG</td><td>Piscidin 1 (P1)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antifungal</td><td>92%hemolysis at 6.25μM</td><td>Human</td><td>CHHHHHHHHHHHTTHHHHHHTC</td><td align="center"><a href="jmol.php?id=1251&amp;di=1" width="5px" height="5px"><img src="pymol_images/1251.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1252">1252</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18827353">18827353</a></td><td align="right">2008</td><td>FFHHIFRGIVHVGKTIHRLVTG</td><td>Piscidin 1 (P1)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antifungal</td><td>45% hemolysis at 3.125μM</td><td>Human</td><td>CHHHHHHHHHHHTTHHHHHHTC</td><td align="center"><a href="jmol.php?id=1252&amp;di=1" width="5px" height="5px"><img src="pymol_images/1252.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1253">1253</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18827353">18827353</a></td><td align="right">2008</td><td>FFHHIFRGIVHVGKTIHRLVTG</td><td>Piscidin 1 (P1)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antifungal</td><td>9% hemolysis at 1.56μM</td><td>Human</td><td>CHHHHHHHHHHHTTHHHHHHTC</td><td align="center"><a href="jmol.php?id=1253&amp;di=1" width="5px" height="5px"><img src="pymol_images/1253.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1254">1254</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18827353">18827353</a></td><td align="right">2008</td><td>FFHHIFRGIVHVGKTIHRLVTG</td><td>Piscidin 1 (P1)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antifungal</td><td>0% hemolysis at 0.78μM</td><td>Human</td><td>CHHHHHHHHHHHTTHHHHHHTC</td><td align="center"><a href="jmol.php?id=1254&amp;di=1" width="5px" height="5px"><img src="pymol_images/1254.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3409">3409</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11713517">11713517</a></td><td align="right">2001</td><td>FFHHIFRGIVHVGKTIHRLVTG</td><td>Piscidin 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolysis at 10μg/ml</td><td>Human</td><td>CHHHHHHHHHHHTTHHHHHHTC</td><td align="center"><a href="jmol.php?id=3409&amp;di=1" width="5px" height="5px"><img src="pymol_images/3409.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3410">3410</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11713517">11713517</a></td><td align="right">2001</td><td>FFHHIFRGIVHVGKTIHRLVTG</td><td>Piscidin 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>~90% hemolysis at 100μg/ml</td><td>Human</td><td>CHHHHHHHHHHHTTHHHHHHTC</td><td align="center"><a href="jmol.php?id=3410&amp;di=1" width="5px" height="5px"><img src="pymol_images/3410.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1114">1114</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17328560">17328560</a></td><td align="right">2007</td><td>FFHHIFRGIVHVPKTIHRLVTG</td><td>Pis-1 GP</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Cytotoxic</td><td>HC<sub>50</sub> &gt;200μM</td><td>Human</td><td>CHHHHHHHHTTCCCGGGGTCCC</td><td align="center"><a href="jmol.php?id=1114&amp;di=1" width="5px" height="5px"><img src="pymol_images/1114.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1114&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1114.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1117">1117</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17328560">17328560</a></td><td align="right">2007</td><td>FFHHIFRPIVHVAKTIHRLVTG</td><td>Pis-1 PA</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Cytotoxic</td><td>HC<sub>50</sub> =32μM</td><td>Human</td><td>CCCCSSCCHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=1117&amp;di=1" width="5px" height="5px"><img src="pymol_images/1117.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1117&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1117.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1113">1113</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17328560">17328560</a></td><td align="right">2007</td><td>FFHHIFRPIVHVGKTIHRLVTG</td><td>Pis-1 PG</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Cytotoxic</td><td>HC<sub>50</sub> &gt;200μM</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=1113&amp;di=1" width="5px" height="5px"><img src="pymol_images/1113.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1113&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1113.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1115">1115</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17328560">17328560</a></td><td align="right">2007</td><td>FFHHIFRPIVHVPKTIHRLVTG</td><td>Pis-1 PP</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Cytotoxic</td><td>HC<sub>50</sub> &gt;200μM</td><td>Human</td><td>CCHHHHHHHTTSCTTGGGTCCC</td><td align="center"><a href="jmol.php?id=1115&amp;di=1" width="5px" height="5px"><img src="pymol_images/1115.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1115&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1115.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2281">2281</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12569105">12569105</a></td><td align="right">2003</td><td>FFHLHFH-dcΔDOPA</td><td>Plicatamide</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>dcΔDOPA=decarboxy-(E)α,β-dehydro3,4-dihydroxyphenylalanine</td><td align="right">8</td><td>Antibacterial</td><td>100% hemolysis at ~40μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2282">2282</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12569105">12569105</a></td><td align="right">2003</td><td>FFHLHFH-dcΔDOPA</td><td>Plicatamide</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>dcΔDOPA=decarboxy-(E)α,β-dehydro3,4-dihydroxyphenylalanine</td><td align="right">8</td><td>Antibacterial</td><td>~10% hemolysis at 80μM</td><td>Sheep</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3572">3572</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18984589">18984589</a></td><td align="right">2009</td><td>FFKKLOHALHOLALLAOHLAHOLKKA</td><td>HALO-F</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>O=Ornithine</td><td align="right">26</td><td>Antimicrobial and Antiplasmodial</td><td>HC<sub>50</sub> &lt;5.6μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3491">3491</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19539775">19539775</a></td><td align="right">2009</td><td>FFPLALLCKVFKKC</td><td>Japonicin-1CDYa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;300μM</td><td>Human</td><td>CCHHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3491&amp;di=1" width="5px" height="5px"><img src="pymol_images/3491.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3491&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3491.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2924">2924</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>FFRLLFHGVHHGGGYLNAA</td><td>NRC-09</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CGGGHHHHSEESSSCEECC</td><td align="center"><a href="jmol.php?id=2924&amp;di=1" width="5px" height="5px"><img src="pymol_images/2924.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2925">2925</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>FFRLLFHGVHHVGKIKPRA</td><td>NRC-10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCCCCCCCCCSGGGCCCCC</td><td align="center"><a href="jmol.php?id=2925&amp;di=1" width="5px" height="5px"><img src="pymol_images/2925.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2925&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2925.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2045">2045</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>FFRRFFRR</td><td>(FFRR)<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt; 500mg/L</td><td>Human</td><td>CCTTSCCC</td><td align="center"><a href="jmol.php?id=2045&amp;di=1" width="5px" height="5px"><img src="pymol_images/2045.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2045&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2045.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2048">2048</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>FFRRFFRRFFRR</td><td>(FFRR)<sub>3</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt; 500mg/L</td><td>Human</td><td>CHHHHTTTSCCC</td><td align="center"><a href="jmol.php?id=2048&amp;di=1" width="5px" height="5px"><img src="pymol_images/2048.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2048&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2048.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2052">2052</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>FFRRFFRRFFRRFFRR</td><td>(FFRR)<sub>4</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>HC<sub>50</sub> ~26mg/L</td><td>Human</td><td>CHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2052&amp;di=1" width="5px" height="5px"><img src="pymol_images/2052.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2052&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2052.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2067">2067</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>FGPVIGLLSGILKSLL</td><td>EpVP6</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CHHHHHHHHHHGGGTC</td><td align="center"><a href="jmol.php?id=2067&amp;di=1" width="5px" height="5px"><img src="pymol_images/2067.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2067&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2067.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4191">4191</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15298176">15298176</a></td><td align="right">2004</td><td>FGVLAKVAAHVVPAIAEHF</td><td>[∆<sup>1,2</sup>]Mac</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>0% hemolysis upto 100µM</td><td>Rabbit</td><td>CCCGGGGGGTTTGGGGTTC</td><td align="center"><a href="jmol.php?id=4191&amp;di=1" width="5px" height="5px"><img src="pymol_images/4191.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4191&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4191.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3635">3635</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17927208">17927208</a></td><td align="right">2007</td><td>FHPSLWVLIPQYIQLIRKILKSG</td><td>Conolysin-Mt</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Cytolytic</td><td>40% hemolysis at 1µM</td><td>Human</td><td>CCSCCCSSHHHHHHHHTTTTTCC</td><td align="center"><a href="jmol.php?id=3635&amp;di=1" width="5px" height="5px"><img src="pymol_images/3635.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3635&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3635.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3636">3636</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17927208">17927208</a></td><td align="right">2007</td><td>FHPSLWVLIPQYIQLIRKILKSG</td><td>Conolysin-Mt</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Cytolytic</td><td>~90% hemolysis at 10µM</td><td>Human</td><td>CCSCCCSSHHHHHHHHTTTTTCC</td><td align="center"><a href="jmol.php?id=3636&amp;di=1" width="5px" height="5px"><img src="pymol_images/3636.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3636&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3636.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3020">3020</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22029824">22029824</a></td><td align="right">2012</td><td>FIFPKKNIINSLFGR</td><td>Andersonin-D1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>Non-hemolytic upto 50μg/mL</td><td>Human</td><td>CCSSSSCGGGTCSCC</td><td align="center"><a href="jmol.php?id=3020&amp;di=1" width="5px" height="5px"><img src="pymol_images/3020.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3020&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3020.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2672">2672</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22792229">22792229</a></td><td align="right">2012</td><td>FIGAIARLLSKIF</td><td>BmKn2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>HC<sub>50</sub> =17.13µg/mL</td><td>Human</td><td>CHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2672&amp;di=1" width="5px" height="5px"><img src="pymol_images/2672.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2672&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2672.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2280">2280</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FIGLLISAGKAIHDLIRRRH</td><td>Chrysophsin-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antibacterial</td><td>100% hemolysis at 5μM</td><td>Human</td><td>CCSCCCCSGGGHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2280&amp;di=1" width="5px" height="5px"><img src="pymol_images/2280.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2280&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2280.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3547">3547</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FIGLLISAGKAIHDLIRRRH</td><td>Chrysophsin-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antibacterial</td><td>0% hemolysis at 1μM</td><td>Human</td><td>CCSCCCCSGGGHHHHTTTCC</td><td align="center"><a href="jmol.php?id=3547&amp;di=1" width="5px" height="5px"><img src="pymol_images/3547.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3547&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3547.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3548">3548</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FIGLLISAGKAIHDLIRRRH</td><td>Chrysophsin-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antibacterial</td><td>100% hemolysis at 100μM</td><td>Human</td><td>CCSCCCCSGGGHHHHTTTCC</td><td align="center"><a href="jmol.php?id=3548&amp;di=1" width="5px" height="5px"><img src="pymol_images/3548.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3548&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3548.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3375">3375</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9022710">9022710</a></td><td align="right">1996</td><td>FIGSALKVLAGVLPSVISWVKQ</td><td>MLP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>Lethal concentration =0.5μM</td><td>Human</td><td>CCCCHHHHHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3375&amp;di=1" width="5px" height="5px"><img src="pymol_images/3375.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2119">2119</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>FIGTALGIASAIPAIVKLFK</td><td>Ponericin-W6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition (3 mm) at 0.4-0.5mM</td><td>Horse</td><td>CCCCSEETTEEHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2119&amp;di=1" width="5px" height="5px"><img src="pymol_images/2119.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2119&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2119.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2120">2120</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>FIGTALGIASAIPAIVKLFK</td><td>Ponericin-W6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition (1 mm) at 0.4-0.5mM</td><td>Sheep</td><td>CCCCSEETTEEHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2120&amp;di=1" width="5px" height="5px"><img src="pymol_images/2120.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2120&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2120.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1255">1255</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18827353">18827353</a></td><td align="right">2008</td><td>FIHHIFRGIVHAGRSIGRFLTG</td><td>Piscidin 3 (P3)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antifungal</td><td>25% hemolysis at 25μM</td><td>Human</td><td>CHHHHHHHHHHHTTSHHHHTTC</td><td align="center"><a href="jmol.php?id=1255&amp;di=1" width="5px" height="5px"><img src="pymol_images/1255.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1256">1256</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18827353">18827353</a></td><td align="right">2008</td><td>FIHHIFRGIVHAGRSIGRFLTG</td><td>Piscidin 3 (P3)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antifungal</td><td>4% hemolysis at 12.5μM</td><td>Human</td><td>CHHHHHHHHHHHTTSHHHHTTC</td><td align="center"><a href="jmol.php?id=1256&amp;di=1" width="5px" height="5px"><img src="pymol_images/1256.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1257">1257</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18827353">18827353</a></td><td align="right">2008</td><td>FIHHIFRGIVHAGRSIGRFLTG</td><td>Piscidin 3 (P3)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antifungal</td><td>0% hemolysis at 0.78-6.25μM</td><td>Human</td><td>CHHHHHHHHHHHTTSHHHHTTC</td><td align="center"><a href="jmol.php?id=1257&amp;di=1" width="5px" height="5px"><img src="pymol_images/1257.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3413">3413</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11713517">11713517</a></td><td align="right">2001</td><td>FIHHIFRGIVHAGRSIGRFLTG</td><td>Piscidin 3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolysis at 10μg/ml</td><td>Human</td><td>CHHHHHHHHHHHTTSHHHHTTC</td><td align="center"><a href="jmol.php?id=3413&amp;di=1" width="5px" height="5px"><img src="pymol_images/3413.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3414">3414</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11713517">11713517</a></td><td align="right">2001</td><td>FIHHIFRGIVHAGRSIGRFLTG</td><td>Piscidin 3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>~5% hemolysis at 100μg/ml</td><td>Human</td><td>CHHHHHHHHHHHTTSHHHHTTC</td><td align="center"><a href="jmol.php?id=3414&amp;di=1" width="5px" height="5px"><img src="pymol_images/3414.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2610">2610</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23226256">23226256</a></td><td align="right">2013</td><td>FIHHIIGGLFSAGKAIHRLIRRRRR</td><td>TP4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>100% hemolytic at 100 µg/ml</td><td>Fish</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2609">2609</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23226256">23226256</a></td><td align="right">2013</td><td>FIHHIIGGLFSVGKHIHSLIHGH</td><td>TP3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>42% hemolytic at 100 µg/ml</td><td>Fish</td><td>CTTTTTTTTSSTTTTHHHHSCCC</td><td align="center"><a href="jmol.php?id=2609&amp;di=1" width="5px" height="5px"><img src="pymol_images/2609.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2609&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2609.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2671">2671</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22792229">22792229</a></td><td align="right">2012</td><td>FIKRIARLLRKIF</td><td>Kn2-7</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>HC<sub>50</sub> =90.27µg/mL</td><td>Human</td><td>CHHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=2671&amp;di=1" width="5px" height="5px"><img src="pymol_images/2671.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2671&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2671.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3488">3488</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19427345">19427345</a></td><td align="right">2009</td><td>FIQYLAPLIPHAVKAISDLI</td><td>Kassinatuerin-2S</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>45-50% hemolysis at 120μM</td><td>Horse</td><td>CCTTCTTTSGGGGGGGSCCC</td><td align="center"><a href="jmol.php?id=3488&amp;di=1" width="5px" height="5px"><img src="pymol_images/3488.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1877">1877</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15196144">15196144</a></td><td align="right">2004</td><td>FKCKKVVISLRRY</td><td>JH8944</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antifungal</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCCCCCCCGGGC</td><td align="center"><a href="jmol.php?id=1877&amp;di=1" width="5px" height="5px"><img src="pymol_images/1877.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1877&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1877.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1873">1873</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15196144">15196144</a></td><td align="right">2004</td><td>FKCRRWQWRMKKLG</td><td>LFB(17–30)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antifungal</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTCSCHHHHTTCC</td><td align="center"><a href="jmol.php?id=1873&amp;di=1" width="5px" height="5px"><img src="pymol_images/1873.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1873&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1873.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2913">2913</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21607695">21607695</a></td><td align="right">2011</td><td>FKCRRWQWRMKKLGA</td><td>LfcinB15</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>&gt;7% hemolytic at 256mg/ml</td><td>Pig</td><td>CCCCSSHHHHTSSCC</td><td align="center"><a href="jmol.php?id=2913&amp;di=1" width="5px" height="5px"><img src="pymol_images/2913.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2913&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2913.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2911">2911</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21607695">21607695</a></td><td align="right">2011</td><td>FKCRRWQWRMKKLGAPSITCVRRAF</td><td>LfcinB (C–C)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>~7% hemolytic at 256mg/ml</td><td>Pig</td><td>CCCCCCHHHHTTSSCCCCGGGGGCC</td><td align="center"><a href="jmol.php?id=2911&amp;di=1" width="5px" height="5px"><img src="pymol_images/2911.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2911&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2911.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2912">2912</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21607695">21607695</a></td><td align="right">2011</td><td>FKCRRWQWRMKKLGAPSITCVRRAF</td><td>LfcinB</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>&lt;10% hemolytic at 256mg/ml</td><td>Pig</td><td>CCCCCCHHHHTTSSCCCCGGGGGCC</td><td align="center"><a href="jmol.php?id=2912&amp;di=1" width="5px" height="5px"><img src="pymol_images/2912.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2912&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2912.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2989">2989</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22189867">22189867</a></td><td align="right">2012</td><td>FKCRRWQWRWKKLGAKPVPIIYCNRRTGKCQRM</td><td>LFT33</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>Non-hemolytic upto 256μg/ml</td><td>Human</td><td>CCCSCCHHHHTTCCSSSCCCCCCCCSCSCCCCC</td><td align="center"><a href="jmol.php?id=2989&amp;di=1" width="5px" height="5px"><img src="pymol_images/2989.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2989&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2989.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2572">2572</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FKDLKKIANIINSIFKK</td><td>LK2(5)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Anticancer</td><td>LC<sub>50</sub> =1,186μM</td><td>Human</td><td>CCCSIIIIIGGGTTSCC</td><td align="center"><a href="jmol.php?id=2572&amp;di=1" width="5px" height="5px"><img src="pymol_images/2572.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2572&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2572.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4192">4192</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16460023">16460023</a></td><td align="right">2006</td><td>FKIKASKVLDKFGKIVGKVLKQLKKVSAVAKV</td><td>P6_1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Hemolytic</td><td>LC<sub>50</sub>=12.6µM</td><td>Human</td><td>CCCSSSGGGTTHHHHHHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=4192&amp;di=1" width="5px" height="5px"><img src="pymol_images/4192.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4192&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4192.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1674">1674</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16460023">16460023</a></td><td align="right">2006</td><td>FKIKPGKVLDKFGKIVGKVLKQLKKVS</td><td>P7</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Hemolytic</td><td>50% hemolysis at 296µM</td><td>Human</td><td>CCCCCCCCCCSTTCCHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1674&amp;di=1" width="5px" height="5px"><img src="pymol_images/1674.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1674&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1674.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1673">1673</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16460023">16460023</a></td><td align="right">2006</td><td>FKIKPGKVLDKFGKIVGKVLKQLKKVSAVAKV</td><td>P6</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Hemolytic</td><td>50% hemolysis at 90µM</td><td>Human</td><td>CCCSCTTCCTTTCHHHHHHHHHHHHHSCGGGC</td><td align="center"><a href="jmol.php?id=1673&amp;di=1" width="5px" height="5px"><img src="pymol_images/1673.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1673&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1673.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2574">2574</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FKKLKKIANIINSIFKK</td><td>LK3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Anticancer</td><td>LC<sub>50</sub> =4,649μM</td><td>Human</td><td>CCGGGGHHHHHTTTTCC</td><td align="center"><a href="jmol.php?id=2574&amp;di=1" width="5px" height="5px"><img src="pymol_images/2574.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2574&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2574.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1832">1832</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>FKLFKKILKFL</td><td>BP66</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>9±2.2% hemolysis at 50µM</td><td>Human</td><td>CHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=1832&amp;di=1" width="5px" height="5px"><img src="pymol_images/1832.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1832&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1832.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1846">1846</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>FKLFKKILKFL</td><td>BP66</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>63±5.9% hemolysis at 150µM</td><td>Human</td><td>CHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=1846&amp;di=1" width="5px" height="5px"><img src="pymol_images/1846.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1846&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1846.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2159">2159</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>FKLFKKIPKFLHLAKKF</td><td>N-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antibacterial</td><td>0.6% hemolysis at 100µM</td><td>Human</td><td>CCCCCCCHHHHTCSSCC</td><td align="center"><a href="jmol.php?id=2159&amp;di=1" width="5px" height="5px"><img src="pymol_images/2159.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2159&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2159.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1934">1934</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>FKRLEKLF</td><td>HPN2C4(6–13)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCSSCTTC</td><td align="center"><a href="jmol.php?id=1934&amp;di=1" width="5px" height="5px"><img src="pymol_images/1934.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1934&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1934.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1824">1824</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>FKRLEKLFKKIWNWK</td><td>HPA3NT2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>14.92% hemolysis at 100µM</td><td>Human</td><td>CTTHHHHHHHHCCCC</td><td align="center"><a href="jmol.php?id=1824&amp;di=1" width="5px" height="5px"><img src="pymol_images/1824.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1824&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1824.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1927">1927</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>FKRLEKLFSKIQNDK</td><td>HPN2(6–20)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHHHHCTTTTCC</td><td align="center"><a href="jmol.php?id=1927&amp;di=1" width="5px" height="5px"><img src="pymol_images/1927.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1927&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1927.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1826">1826</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>FKRLEKLFSKIWNWK</td><td>HPA3NT0</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>4.70% hemolysis at 100µM</td><td>Human</td><td>CTTGGGGTTTTCCCC</td><td align="center"><a href="jmol.php?id=1826&amp;di=1" width="5px" height="5px"><img src="pymol_images/1826.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1826&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1826.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3937">3937</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>FKRLEKLFSKIWNWK</td><td>A3-NT</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>MHC &gt;100μg/ml</td><td>Human</td><td>CTTGGGGTTTTCCCC</td><td align="center"><a href="jmol.php?id=3937&amp;di=1" width="5px" height="5px"><img src="pymol_images/3937.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3937&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3937.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1823">1823</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>FKRLKKLFKKIWNWK</td><td>HPA3NT3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>11.62% hemolysis at 100µM</td><td>Human</td><td>CTTHHHHGGGTCCCC</td><td align="center"><a href="jmol.php?id=1823&amp;di=1" width="5px" height="5px"><img src="pymol_images/1823.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1823&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1823.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1825">1825</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>FKRLKKLFSKIWNWK</td><td>HPA3NT1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>9.65% hemolysis at 100µM</td><td>Human</td><td>CTTTHHHHTTTSSCC</td><td align="center"><a href="jmol.php?id=1825&amp;di=1" width="5px" height="5px"><img src="pymol_images/1825.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1825&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1825.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4022">4022</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>FKTWKRPPFQTSCSGIIKE</td><td>Nigroain-C1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CCCCCCSSSSCBTTBSSCC</td><td align="center"><a href="jmol.php?id=4022&amp;di=1" width="5px" height="5px"><img src="pymol_images/4022.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2082">2082</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22426384">22426384</a></td><td align="right">2012</td><td>FLAGLIGGLAKMLGK</td><td>Temporin-LTe</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>LD<sub>50</sub> =40µM</td><td>Human</td><td>CCSHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2082&amp;di=1" width="5px" height="5px"><img src="pymol_images/2082.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2082&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2082.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2885">2885</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLALIGRVLSGIL</td><td>Ta A3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolytic at 40μM</td><td>Human</td><td>CHHHHTTTTSSCC</td><td align="center"><a href="jmol.php?id=2885&amp;di=1" width="5px" height="5px"><img src="pymol_images/2885.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2885&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2885.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2886">2886</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLALIGRVLSGIL</td><td>Ta A3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CHHHHTTTTSSCC</td><td align="center"><a href="jmol.php?id=2886&amp;di=1" width="5px" height="5px"><img src="pymol_images/2886.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2886&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2886.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2084">2084</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22484288">22484288</a></td><td align="right">2012</td><td>FLFSLIPHAIGGLISAFK</td><td>AamAP1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>0%hemolysis at 10µM</td><td>Horse</td><td>CCCSSSCTTTCBTTBCCC</td><td align="center"><a href="jmol.php?id=2084&amp;di=1" width="5px" height="5px"><img src="pymol_images/2084.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2084&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2084.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2085">2085</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22484288">22484288</a></td><td align="right">2012</td><td>FLFSLIPHAIGGLISAFK</td><td>AamAP1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>100%hemolysis at 100µM</td><td>Horse</td><td>CCCSSSCTTTCBTTBCCC</td><td align="center"><a href="jmol.php?id=2085&amp;di=1" width="5px" height="5px"><img src="pymol_images/2085.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2085&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2085.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2088">2088</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22484288">22484288</a></td><td align="right">2012</td><td>FLFSLIPKAIGGLISAFK</td><td>AamAP-S1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>17.8%hemolysis at 10µM</td><td>Horse</td><td>CCCCSSGGGGCSSCCCCC</td><td align="center"><a href="jmol.php?id=2088&amp;di=1" width="5px" height="5px"><img src="pymol_images/2088.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2088&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2088.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2089">2089</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22484288">22484288</a></td><td align="right">2012</td><td>FLFSLIPKAIGGLISAFK</td><td>AamAP-S1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>53% hemolysis at 120µM</td><td>Horse</td><td>CCCCSSGGGGCSSCCCCC</td><td align="center"><a href="jmol.php?id=2089&amp;di=1" width="5px" height="5px"><img src="pymol_images/2089.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2089&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2089.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3484">3484</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19427345">19427345</a></td><td align="right">2009</td><td>FLGAIAAALPHVINAVTNAL</td><td>Kassinatuerin-2Ma</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>45-50% hemolysis at 120μM</td><td>Horse</td><td>CCTTHHHHHHHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=3484&amp;di=1" width="5px" height="5px"><img src="pymol_images/3484.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3703">3703</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7999137">7999137</a></td><td align="right">1994</td><td>FLGALFKVASKVLPSVKCAITKKC</td><td>Gaegurin 5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>1.01% hemolysis at 100µM</td><td>Human</td><td>CTTTTHHHHTSSSSCCCCCGGGCC</td><td align="center"><a href="jmol.php?id=3703&amp;di=1" width="5px" height="5px"><img src="pymol_images/3703.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3703&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3703.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2920">2920</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>FLGALIKGAIHGGRFIHGMIQNHH</td><td>NRC-05</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CHHHHCCSSCCSTTCCCCTTTCCC</td><td align="center"><a href="jmol.php?id=2920&amp;di=1" width="5px" height="5px"><img src="pymol_images/2920.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2920&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2920.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3160">3160</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>FLGALLGPLMNLLQ</td><td>PFQa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>LC<sub>50</sub> =105μM</td><td>Human</td><td>CCTTSHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3160&amp;di=1" width="5px" height="5px"><img src="pymol_images/3160.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3160&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3160.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3259">3259</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23415044">23415044</a></td><td align="right">2013</td><td>FLGFLHHLF</td><td>H2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antiviral</td><td>HC<sub>50</sub> =203.3μg/ml</td><td>Human</td><td>CCSCCCCCC</td><td align="center"><a href="jmol.php?id=3259&amp;di=1" width="5px" height="5px"><img src="pymol_images/3259.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3259&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3259.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3258">3258</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23415044">23415044</a></td><td align="right">2013</td><td>FLGFLKNLF</td><td>WT</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antiviral</td><td>HC<sub>50</sub> =137.9μg/ml</td><td>Human</td><td>CCSSCCCCC</td><td align="center"><a href="jmol.php?id=3258&amp;di=1" width="5px" height="5px"><img src="pymol_images/3258.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3258&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3258.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2934">2934</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>FLGLLFHGVHHVGKWIHGLIHGHH</td><td>NRC-19</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Anticancer</td><td>50% hemolytic at 16μg/ml</td><td>Human</td><td>CTTSCCCSCCCTTBTTBCCCSSCC</td><td align="center"><a href="jmol.php?id=2934&amp;di=1" width="5px" height="5px"><img src="pymol_images/2934.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2934&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2934.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4027">4027</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>FLGPIIKIATGILPTAICKILKKC</td><td>Gaegurin-RN3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CCTTSCTTTSSCCTTHHHHHTTTC</td><td align="center"><a href="jmol.php?id=4027&amp;di=1" width="5px" height="5px"><img src="pymol_images/4027.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4027&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4027.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1055">1055</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16413829">16413829</a></td><td align="right">2006</td><td>FLGSIVGALASALPSLISKIRN</td><td>Brevinin-1TSa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>LD<sub>50</sub> =12μM</td><td>Human</td><td>CCTTHHHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=1055&amp;di=1" width="5px" height="5px"><img src="pymol_images/1055.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1556">1556</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21110126">21110126</a></td><td align="right">2011</td><td>FLGWLFKWAKK</td><td>GA-K3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>MHC =50μg/ml</td><td>Human</td><td>CCSHHHHTSCC</td><td align="center"><a href="jmol.php?id=1556&amp;di=1" width="5px" height="5px"><img src="pymol_images/1556.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1556&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1556.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1553">1553</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21110126">21110126</a></td><td align="right">2011</td><td>FLGWLFKWASK</td><td>GA-W2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>MHC =25μg/ml</td><td>Human</td><td>CCSSSCSSSCC</td><td align="center"><a href="jmol.php?id=1553&amp;di=1" width="5px" height="5px"><img src="pymol_images/1553.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1553&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1553.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1554">1554</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21110126">21110126</a></td><td align="right">2011</td><td>FLGWLFKWAWK</td><td>GA-W3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>MHC =50μg/ml</td><td>Human</td><td>CCSTTCCCCCC</td><td align="center"><a href="jmol.php?id=1554&amp;di=1" width="5px" height="5px"><img src="pymol_images/1554.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1554&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1554.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3260">3260</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23415044">23415044</a></td><td align="right">2013</td><td>FLHFLHHLF</td><td>H3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antiviral</td><td>HC<sub>50</sub> =416.4μg/ml</td><td>Human</td><td>CCCCCCCCC</td><td align="center"><a href="jmol.php?id=3260&amp;di=1" width="5px" height="5px"><img src="pymol_images/3260.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3260&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3260.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3609">3609</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21956830">21956830</a></td><td align="right">2011</td><td>FLKPLFNAALKLLP</td><td>Temporin-Ra</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>1.3% hemolysis at 60µg/ml</td><td>Human</td><td>CCCGGGHHHHTTCC</td><td align="center"><a href="jmol.php?id=3609&amp;di=1" width="5px" height="5px"><img src="pymol_images/3609.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3609&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3609.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1557">1557</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21110126">21110126</a></td><td align="right">2011</td><td>FLKWLFKWAKK</td><td>GA-K4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>MHC =6.3μg/ml</td><td>Human</td><td>CTTGGGGTSCC</td><td align="center"><a href="jmol.php?id=1557&amp;di=1" width="5px" height="5px"><img src="pymol_images/1557.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1557&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1557.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1558">1558</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21110126">21110126</a></td><td align="right">2011</td><td>FLKWLFKWLKK</td><td>GA-K4L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>MHC =12.5μg/ml</td><td>Human</td><td>CHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1558&amp;di=1" width="5px" height="5px"><img src="pymol_images/1558.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1558&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1558.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1072">1072</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20153801">20153801</a></td><td align="right">2010</td><td>FLLFPLMCKIQGKC</td><td>Japonicin-1Npa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>1.92% hemolysis at 80μg/ml</td><td>Human</td><td>CCCCSCSSSCCSCC</td><td align="center"><a href="jmol.php?id=1072&amp;di=1" width="5px" height="5px"><img src="pymol_images/1072.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1072&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1072.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3686">3686</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8508915">8508915</a></td><td align="right">1993</td><td>FLPAIAGILSQLF</td><td>A1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LC<sub>50</sub> ~20µM</td><td>Human</td><td>CCCHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=3686&amp;di=1" width="5px" height="5px"><img src="pymol_images/3686.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3686&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3686.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2887">2887</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPAIGRVLSGIL</td><td>Ta A4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>11.5% hemolytic at 40μM</td><td>Human</td><td>CCCCTHHHHCCCC</td><td align="center"><a href="jmol.php?id=2887&amp;di=1" width="5px" height="5px"><img src="pymol_images/2887.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2887&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2887.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2888">2888</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPAIGRVLSGIL</td><td>Ta A4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolytic at 1.25μM</td><td>Human</td><td>CCCCTHHHHCCCC</td><td align="center"><a href="jmol.php?id=2888&amp;di=1" width="5px" height="5px"><img src="pymol_images/2888.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2888&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2888.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3161">3161</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22943778">22943778</a></td><td align="right">2012</td><td>FLPFLLSALPKVFCFFSKKC</td><td>Brevinin-1ZHa</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>HC<sub>50</sub> =3μM</td><td>Human</td><td>CCCTTTCSSCCSCCCSSCCC</td><td align="center"><a href="jmol.php?id=3161&amp;di=1" width="5px" height="5px"><img src="pymol_images/3161.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3161&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3161.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1071">1071</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20044030">20044030</a></td><td align="right">2010</td><td>FLPIIAGAAAKVVQKIFCAISKKC</td><td>Brevinin-1Yb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>LD<sub>50</sub> =10μM</td><td>Human</td><td>CCSCHHHHHHHHHHHHHGGGCCCC</td><td align="center"><a href="jmol.php?id=1071&amp;di=1" width="5px" height="5px"><img src="pymol_images/1071.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1071&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1071.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1131">1131</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17451843">17451843</a></td><td align="right">2007</td><td>FLPIIGKLLSGLL</td><td>Temporin-1CSb</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LD<sub>50</sub> =95μM</td><td>Human</td><td>CCHHHHHHHTCCC</td><td align="center"><a href="jmol.php?id=1131&amp;di=1" width="5px" height="5px"><img src="pymol_images/1131.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1131&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1131.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1618">1618</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15325526">15325526</a></td><td align="right">2005</td><td>FLPIIGQLLSGLL</td><td>Temporin-1AUa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;300µM</td><td>Human</td><td>CCTHHHHHHCCCC</td><td align="center"><a href="jmol.php?id=1618&amp;di=1" width="5px" height="5px"><img src="pymol_images/1618.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1618&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1618.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1129">1129</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17451843">17451843</a></td><td align="right">2007</td><td>FLPILAGLAAKIVPKLFCLATKKC</td><td>Brevinin-1CSa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>LD<sub>50</sub> =5μM</td><td>Human</td><td>CCHHHHHHTTTTSGGGTTTSCCCC</td><td align="center"><a href="jmol.php?id=1129&amp;di=1" width="5px" height="5px"><img src="pymol_images/1129.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1129&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1129.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1616">1616</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15325526">15325526</a></td><td align="right">2005</td><td>FLPILAGLAAKLVPKVFCSITKKC</td><td>Brevinin-1AUa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>HC<sub>50</sub> =5µM</td><td>Human</td><td>CCTHHHHHHHHHTTCCSCCSCCCC</td><td align="center"><a href="jmol.php?id=1616&amp;di=1" width="5px" height="5px"><img src="pymol_images/1616.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1616&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1616.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1617">1617</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15325526">15325526</a></td><td align="right">2005</td><td>FLPILAGLAANILPKVFCSITKKC</td><td>Brevinin-1AUb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>HC<sub>50</sub> =7µM</td><td>Human</td><td>CCHHHHHHTTCSSCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=1617&amp;di=1" width="5px" height="5px"><img src="pymol_images/1617.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1617&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1617.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2323">2323</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14531844">14531844</a></td><td align="right">2003</td><td>FLPILASLAAKFGPKLFCLVTKKC</td><td>Brevinin-1BYa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antibacterial</td><td>HC<sub>50</sub> =4µM</td><td>Human</td><td>CCHHHHHHHHHHTTCCCCCCCTTC</td><td align="center"><a href="jmol.php?id=2323&amp;di=1" width="5px" height="5px"><img src="pymol_images/2323.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2323&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2323.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2324">2324</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14531844">14531844</a></td><td align="right">2003</td><td>FLPILASLAAKLGPKLFCLVTKKC</td><td>Brevinin-1BYb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antibacterial</td><td>HC<sub>50</sub> =4µM</td><td>Human</td><td>CCHHHHHHHHHHSSSCCCCCCTTC</td><td align="center"><a href="jmol.php?id=2324&amp;di=1" width="5px" height="5px"><img src="pymol_images/2324.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2324&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2324.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4058">4058</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12804591">12804591</a></td><td align="right">2003</td><td>FLPILGKLLSGIL</td><td>MRP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>LC<sub>50</sub> = 8μM</td><td>Human</td><td>CCCCTHHHHSCCC</td><td align="center"><a href="jmol.php?id=4058&amp;di=1" width="5px" height="5px"><img src="pymol_images/4058.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4058&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4058.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1130">1130</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17451843">17451843</a></td><td align="right">2007</td><td>FLPIVGKLLSGLL</td><td>Temporin-1Csa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LD<sub>50</sub> =75μM</td><td>Human</td><td>CCTTHHHHCCCCC</td><td align="center"><a href="jmol.php?id=1130&amp;di=1" width="5px" height="5px"><img src="pymol_images/1130.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1130&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1130.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2483">2483</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12429503">12429503</a></td><td align="right">2002</td><td>FLPIVGRLISGLL</td><td>Temporin-1AR</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>HC<sub>50</sub> =210µM</td><td>Human</td><td>CCCCHHHHHTCCC</td><td align="center"><a href="jmol.php?id=2483&amp;di=1" width="5px" height="5px"><img src="pymol_images/2483.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2483&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2483.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3708">3708</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19716842">19716842</a></td><td align="right">2010</td><td>FLPKMSTKLRVPYRRGTKDYH</td><td>Scolopin 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>25±2% hemolysis at 50µg/ml</td><td>Human</td><td>CCCCCCSCCCSSSCSCSSCCC</td><td align="center"><a href="jmol.php?id=3708&amp;di=1" width="5px" height="5px"><img src="pymol_images/3708.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3708&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3708.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3709">3709</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19716842">19716842</a></td><td align="right">2010</td><td>FLPKMSTKLRVPYRRGTKDYH</td><td>Scolopin 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>21±5% hemolysis at 50µg/ml</td><td>Rabbit</td><td>CCCCCCSCCCSSSCSCSSCCC</td><td align="center"><a href="jmol.php?id=3709&amp;di=1" width="5px" height="5px"><img src="pymol_images/3709.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3709&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3709.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2889">2889</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLAGRVLSGIL</td><td>Ta A5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3.5% hemolytic at 40μM</td><td>Human</td><td>CCGGGSTTSCSCC</td><td align="center"><a href="jmol.php?id=2889&amp;di=1" width="5px" height="5px"><img src="pymol_images/2889.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2889&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2889.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2890">2890</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLAGRVLSGIL</td><td>Ta A5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CCGGGSTTSCSCC</td><td align="center"><a href="jmol.php?id=2890&amp;di=1" width="5px" height="5px"><img src="pymol_images/2890.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2890&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2890.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1720">1720</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17000029">17000029</a></td><td align="right">2006</td><td>FLPLAVSLAANFLPKLFCKITKKC</td><td>Brevinins-ALb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>96%hemolysis at 20µg/ml</td><td>Rabbit</td><td>CCCCGGGGGTCSSHHHHTTTSCCC</td><td align="center"><a href="jmol.php?id=1720&amp;di=1" width="5px" height="5px"><img src="pymol_images/1720.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1720&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1720.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2891">2891</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIARVLSGIL</td><td>Ta A6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>97% hemolytic at 40μM</td><td>Human</td><td>CCCHHHHTTSSCC</td><td align="center"><a href="jmol.php?id=2891&amp;di=1" width="5px" height="5px"><img src="pymol_images/2891.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2891&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2891.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2892">2892</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIARVLSGIL</td><td>Ta A6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>1% hemolytic at 1.25μM</td><td>Human</td><td>CCCHHHHTTSSCC</td><td align="center"><a href="jmol.php?id=2892&amp;di=1" width="5px" height="5px"><img src="pymol_images/2892.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2892&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2892.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2893">2893</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGAVLSGIL</td><td>Ta A7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>39% hemolytic at 40μM</td><td>Human</td><td>CCGGGGGGTSSCC</td><td align="center"><a href="jmol.php?id=2893&amp;di=1" width="5px" height="5px"><img src="pymol_images/2893.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2893&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2893.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2894">2894</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGAVLSGIL</td><td>Ta A7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>6% hemolytic at 1.25μM</td><td>Human</td><td>CCGGGGGGTSSCC</td><td align="center"><a href="jmol.php?id=2894&amp;di=1" width="5px" height="5px"><img src="pymol_images/2894.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2894&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2894.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2895">2895</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRALSGIL</td><td>Ta A8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>10.5% hemolytic at 40μM</td><td>Human</td><td>CCCCGGGTSCCCC</td><td align="center"><a href="jmol.php?id=2895&amp;di=1" width="5px" height="5px"><img src="pymol_images/2895.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2895&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2895.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2896">2896</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRALSGIL</td><td>Ta A8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>1.5% hemolytic at 1.25μM</td><td>Human</td><td>CCCCGGGTSCCCC</td><td align="center"><a href="jmol.php?id=2896&amp;di=1" width="5px" height="5px"><img src="pymol_images/2896.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2896&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2896.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2897">2897</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVASGIL</td><td>Ta A9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3.5% hemolytic at 40μM</td><td>Human</td><td>CCCCGGGTSSSCC</td><td align="center"><a href="jmol.php?id=2897&amp;di=1" width="5px" height="5px"><img src="pymol_images/2897.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2897&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2897.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2898">2898</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVASGIL</td><td>Ta A9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CCCCGGGTSSSCC</td><td align="center"><a href="jmol.php?id=2898&amp;di=1" width="5px" height="5px"><img src="pymol_images/2898.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2898&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2898.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2899">2899</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLAGIL</td><td>Ta A10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>70% hemolytic at 40μM</td><td>Human</td><td>CCCCTTBTTSBCC</td><td align="center"><a href="jmol.php?id=2899&amp;di=1" width="5px" height="5px"><img src="pymol_images/2899.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2899&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2899.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2900">2900</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLAGIL</td><td>Ta A10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CCCCTTBTTSBCC</td><td align="center"><a href="jmol.php?id=2900&amp;di=1" width="5px" height="5px"><img src="pymol_images/2900.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2900&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2900.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2901">2901</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSAIL</td><td>Ta A11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolytic at 40μM</td><td>Human</td><td>CCSSTHHHHTTTC</td><td align="center"><a href="jmol.php?id=2901&amp;di=1" width="5px" height="5px"><img src="pymol_images/2901.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2901&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2901.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2902">2902</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSAIL</td><td>Ta A11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3.5% hemolytic at 1.25μM</td><td>Human</td><td>CCSSTHHHHTTTC</td><td align="center"><a href="jmol.php?id=2902&amp;di=1" width="5px" height="5px"><img src="pymol_images/2902.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2902&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2902.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2903">2903</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSGAL</td><td>Ta A12</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>1% hemolytic at 40μM</td><td>Human</td><td>CCSSGGGTTSCCC</td><td align="center"><a href="jmol.php?id=2903&amp;di=1" width="5px" height="5px"><img src="pymol_images/2903.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2903&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2903.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2904">2904</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSGAL</td><td>Ta A12</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CCSSGGGTTSCCC</td><td align="center"><a href="jmol.php?id=2904&amp;di=1" width="5px" height="5px"><img src="pymol_images/2904.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2904&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2904.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2905">2905</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSGIA</td><td>Ta A13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3.5% hemolytic at 40μM</td><td>Human</td><td>CCCSGGGTSTTTC</td><td align="center"><a href="jmol.php?id=2905&amp;di=1" width="5px" height="5px"><img src="pymol_images/2905.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2905&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2905.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2906">2906</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSGIA</td><td>Ta A13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CCCSGGGTSTTTC</td><td align="center"><a href="jmol.php?id=2906&amp;di=1" width="5px" height="5px"><img src="pymol_images/2906.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2906&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2906.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2151">2151</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11995990">11995990</a></td><td align="right">2001</td><td>FLPLIGRVLSGIL</td><td>Temporin A (TA)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antifungal</td><td>0% hemolysis at 14µM</td><td>Human</td><td>CCCCHHHHTSCCC</td><td align="center"><a href="jmol.php?id=2151&amp;di=1" width="5px" height="5px"><img src="pymol_images/2151.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2151&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2151.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2152">2152</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11995990">11995990</a></td><td align="right">2001</td><td>FLPLIGRVLSGIL</td><td>Temporin A (TA)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antifungal</td><td>100% hemolysis at 225µM</td><td>Human</td><td>CCCCHHHHTSCCC</td><td align="center"><a href="jmol.php?id=2152&amp;di=1" width="5px" height="5px"><img src="pymol_images/2152.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2152&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2152.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2879">2879</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSGIL</td><td>1Ta</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>27.5% hemolytic at 40μM</td><td>Human</td><td>CCCCHHHHTSCCC</td><td align="center"><a href="jmol.php?id=2879&amp;di=1" width="5px" height="5px"><img src="pymol_images/2879.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2879&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2879.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2880">2880</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSGIL</td><td>1Ta</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0.5% hemolytic at 1.25μM</td><td>Human</td><td>CCCCHHHHTSCCC</td><td align="center"><a href="jmol.php?id=2880&amp;di=1" width="5px" height="5px"><img src="pymol_images/2880.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2880&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2880.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3373">3373</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9022710">9022710</a></td><td align="right">1996</td><td>FLPLIGRVLSGIL</td><td>Temporin A</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>Lethal concentration &gt;120μM</td><td>Human</td><td>CCCCHHHHTSCCC</td><td align="center"><a href="jmol.php?id=3373&amp;di=1" width="5px" height="5px"><img src="pymol_images/3373.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3373&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3373.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4110">4110</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15513914">15513914</a></td><td align="right">2004</td><td>FLPLIGRVLSGIL</td><td>Temporin A</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>LC<sub>50</sub> &gt;120μM</td><td>Human</td><td>CCCCHHHHTSCCC</td><td align="center"><a href="jmol.php?id=4110&amp;di=1" width="5px" height="5px"><img src="pymol_images/4110.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4110&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4110.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3475">3475</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11738089">11738089</a></td><td align="right">2001</td><td>FLPLILRKIVTAL</td><td>Crabrolin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial and Mast cell degranulating</td><td>No hemolysis (non hemolytic)</td><td>Rat</td><td>CCHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3475&amp;di=1" width="5px" height="5px"><img src="pymol_images/3475.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3475&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3475.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2148">2148</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11892852">11892852</a></td><td align="right">2001</td><td>FLPLLAGLAANFLPKIFC-Acm-KITRKC-Acm</td><td>Brevinin1E linear (BL)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>Acm= acetamidomethyl</td><td align="right">26</td><td>Antibacterial</td><td>100% hemolysisat 5μM</td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1602">1602</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15232222">15232222</a></td><td align="right">2004</td><td>FLPLLAGLAANFLPKIFCKITRKC</td><td>Brevinin-1E</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>25.8% hemolysis at 10µg/ml</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHHCSCC</td><td align="center"><a href="jmol.php?id=1602&amp;di=1" width="5px" height="5px"><img src="pymol_images/1602.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1602&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1602.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1606">1606</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15232222">15232222</a></td><td align="right">2004</td><td>FLPLLAGLAANFLPKIFCKITRKC</td><td>Brevinin-1E</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>189.2% hemolysis at 100µg/ml</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHHCSCC</td><td align="center"><a href="jmol.php?id=1606&amp;di=1" width="5px" height="5px"><img src="pymol_images/1606.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1606&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1606.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2147">2147</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11892852">11892852</a></td><td align="right">2001</td><td>FLPLLAGLAANFLPKIFCKITRKC</td><td>Brevinin 1E (B)</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">24</td><td>Antibacterial</td><td>100% hemolysis at 0.7μM</td><td>Rat</td><td>CCHHHHHHHHHHHHHHHHHHCSCC</td><td align="center"><a href="jmol.php?id=2147&amp;di=1" width="5px" height="5px"><img src="pymol_images/2147.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2147&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2147.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3285">3285</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9748603">9748603</a></td><td align="right">1998</td><td>FLPLLAGLAANFLPKIFCKITRKC</td><td>Brevinin 1E</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>HD<sub>50</sub> =1.1μg/ml</td><td>Mouse</td><td>NA</td><td align="center"><a href="jmol.php?id=3285&amp;di=1" width="5px" height="5px"><img src="pymol_images/3285.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3286">3286</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9748603">9748603</a></td><td align="right">1998</td><td>FLPLLAGLAANFLPKIFCKITRKC</td><td>Brevinin 1E (red)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>HD<sub>50</sub> =1.6μg/ml</td><td>Mouse</td><td>NA</td><td align="center"><a href="jmol.php?id=3286&amp;di=1" width="5px" height="5px"><img src="pymol_images/3286.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3302">3302</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8163497">8163497</a></td><td align="right">1994</td><td>FLPLLAGLAANFLPKIFCKITRKC</td><td>Brevinin-1E</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>Lethal concentration =0.5μM</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHHCSCC</td><td align="center"><a href="jmol.php?id=3302&amp;di=1" width="5px" height="5px"><img src="pymol_images/3302.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3683">3683</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8508915">8508915</a></td><td align="right">1993</td><td>FLPLLAGLAANFLPKIFCKITRKC</td><td>Brevinin-1 E</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>LC<sub>50</sub> =5µM</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHHCSCC</td><td align="center"><a href="jmol.php?id=3683&amp;di=1" width="5px" height="5px"><img src="pymol_images/3683.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3704">3704</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7999137">7999137</a></td><td align="right">1994</td><td>FLPLLAGLAANFLPTIICKISYKC</td><td>Gaegurin 6</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>0.86% hemolysis at 100µM</td><td>Human</td><td>CCCCHHHHHHCSSTTSCCCSSCCC</td><td align="center"><a href="jmol.php?id=3704&amp;di=1" width="5px" height="5px"><img src="pymol_images/3704.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3704&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3704.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2150">2150</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11892852">11892852</a></td><td align="right">2001</td><td>FLPLLAGLC-Acm-KITRKC-Acm-AANFLPKIF</td><td>Linear analog of brevinin 1E (BAL)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>Acm=acetamidomethyl</td><td align="right">26</td><td>Antibacterial</td><td>Non-hemolytic</td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2149">2149</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11892852">11892852</a></td><td align="right">2001</td><td>FLPLLAGLCKITRKCAANFLPKIF</td><td>Analog of brevinin 1E (BA)</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">24</td><td>Antibacterial</td><td>100% hemolysis at 7μM</td><td>Rat</td><td>CCTTTSHHHHHHHHHTTCCCTTCC</td><td align="center"><a href="jmol.php?id=2149&amp;di=1" width="5px" height="5px"><img src="pymol_images/2149.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2149&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2149.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3645">3645</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17688900">17688900</a></td><td align="right">2007</td><td>FLPLLLAGLPKLLCLFFKKC</td><td>Brevinin-1DYc</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>LC<sub>50</sub> &lt;5µM</td><td>Human</td><td>CHHHHHSSSCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3645&amp;di=1" width="5px" height="5px"><img src="pymol_images/3645.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3645&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3645.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3162">3162</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22943778">22943778</a></td><td align="right">2012</td><td>FLPLLLSALPSFLCLVFKKC</td><td>Brevinin-1ZHb</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>HC<sub>50</sub> =5μM</td><td>Human</td><td>CHHHHHHHTTTTTSSGGGCC</td><td align="center"><a href="jmol.php?id=3162&amp;di=1" width="5px" height="5px"><img src="pymol_images/3162.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3162&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3162.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4028">4028</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>FLPLVLGALSGILPKILGK</td><td>Temporin-TN1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>95.7±4.12% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CCGGGHHHHTTSCCTTSCC</td><td align="center"><a href="jmol.php?id=4028&amp;di=1" width="5px" height="5px"><img src="pymol_images/4028.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4028&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4028.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1132">1132</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17451843">17451843</a></td><td align="right">2007</td><td>FLPLVTGLLSGLL</td><td>Temporin-1CSc</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LD<sub>50</sub>&gt;300μM</td><td>Human</td><td>CCSCHHHHHHHHC</td><td align="center"><a href="jmol.php?id=1132&amp;di=1" width="5px" height="5px"><img src="pymol_images/1132.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1132&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1132.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3892">3892</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15996769">15996769</a></td><td align="right">2005</td><td>FLPLVTMLLGKLF</td><td>Temporin-1Vc</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LD<sub>50</sub> =30μM</td><td>Human</td><td>CCCTHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3892&amp;di=1" width="5px" height="5px"><img src="pymol_images/3892.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1070">1070</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20044030">20044030</a></td><td align="right">2010</td><td>FLPVIAGAANFLPKLFCAISKKC</td><td>Brevinin-1Ya</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>LD<sub>50</sub> =14μM</td><td>Human</td><td>CCSCCCCCSCCCTTHHHHSSCCC</td><td align="center"><a href="jmol.php?id=1070&amp;di=1" width="5px" height="5px"><img src="pymol_images/1070.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1070&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1070.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4050">4050</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16675060">16675060</a></td><td align="right">2006</td><td>FLPVILPVIGKLLSGIL</td><td>Temporin-1TGc</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>LD<sub>50</sub> =50μM</td><td>Human</td><td>CCTTTTTHHHHHHTSCC</td><td align="center"><a href="jmol.php?id=4050&amp;di=1" width="5px" height="5px"><img src="pymol_images/4050.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4050&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4050.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2450">2450</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>FLPVLAGIAAKVVPALFCKITKKC</td><td>Brevinin-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>89% hemolysis at 80µg/ml</td><td>Human</td><td>CCHHHHHHHHHHSHHHHCCSCCCC</td><td align="center"><a href="jmol.php?id=2450&amp;di=1" width="5px" height="5px"><img src="pymol_images/2450.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2450&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2450.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2451">2451</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>FLPVLAGIAAKVVPALFCKITKKC</td><td>CAM-brevinin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>0.5% hemolysis at 80µg/ml</td><td>Human</td><td>CCHHHHHHHHHHSHHHHCCSCCCC</td><td align="center"><a href="jmol.php?id=2451&amp;di=1" width="5px" height="5px"><img src="pymol_images/2451.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2451&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2451.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3610">3610</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21956830">21956830</a></td><td align="right">2011</td><td>FLPVLAGVLSRA</td><td>Temporin-Rb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>1.1% hemolysis at 60µg/ml</td><td>Human</td><td>CCCCCBTTBCCC</td><td align="center"><a href="jmol.php?id=3610&amp;di=1" width="5px" height="5px"><img src="pymol_images/3610.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3610&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3610.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2836">2836</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FLPWFSKFLGRIL</td><td>[Leu<sup>2</sup>, Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>19% hemolytic at 6μM</td><td>Human</td><td>CCCTTTTTTCCCC</td><td align="center"><a href="jmol.php?id=2836&amp;di=1" width="5px" height="5px"><img src="pymol_images/2836.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2836&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2836.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3534">3534</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005415">12005415</a></td><td align="right">2001</td><td>FLRFIGSVIHGIGHLVHHIGVAL</td><td>Clavaspirin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>~70% hemolysis at 20μg/ml</td><td>Human</td><td>CCTTHHHHTTTHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3534&amp;di=1" width="5px" height="5px"><img src="pymol_images/3534.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3535">3535</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005415">12005415</a></td><td align="right">2001</td><td>FLRFIGSVIHGIGHLVHHIGVAL</td><td>Clavaspirin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>75% hemolysis at 80μg/ml</td><td>Human</td><td>CCTTHHHHTTTHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3535&amp;di=1" width="5px" height="5px"><img src="pymol_images/3535.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3891">3891</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15996769">15996769</a></td><td align="right">2005</td><td>FLSIIAKVLGSLF</td><td>Temporin-IVb</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LD<sub>50</sub> =30μM</td><td>Human</td><td>CHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3891&amp;di=1" width="5px" height="5px"><img src="pymol_images/3891.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3891&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3891.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3643">3643</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17688900">17688900</a></td><td align="right">2007</td><td>FLSLALAALPKFLCLVFKKC</td><td>Brevinin-1DYa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>LC<sub>50</sub> &lt;5µM</td><td>Human</td><td>CHHHHHHHSTTTTCCGGGCC</td><td align="center"><a href="jmol.php?id=3643&amp;di=1" width="5px" height="5px"><img src="pymol_images/3643.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3643&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3643.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1995">1995</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15003829">15003829</a></td><td align="right">2004</td><td>FLSLALAALPKLFCLIFKKC</td><td>Brevinin-1PRa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>HC<sub>50</sub> =7µM</td><td>Human</td><td>CHHHHHTTTTTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=1995&amp;di=1" width="5px" height="5px"><img src="pymol_images/1995.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1995&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1995.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3490">3490</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19539775">19539775</a></td><td align="right">2009</td><td>FLSLALAALPKLFCLIFKKC</td><td>Brevinin-1CDYa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>HC<sub>50</sub> =125μM</td><td>Human</td><td>CHHHHHTTTTTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3490&amp;di=1" width="5px" height="5px"><img src="pymol_images/3490.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3490&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3490.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3644">3644</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17688900">17688900</a></td><td align="right">2007</td><td>FLSLALAALPKLFCLIFKKC</td><td>Brevinin-1DYb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>LC<sub>50</sub> &lt;5µM</td><td>Human</td><td>CHHHHHTTTTTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3644&amp;di=1" width="5px" height="5px"><img src="pymol_images/3644.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3644&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3644.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1916">1916</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16963159">16963159</a></td><td align="right">2006</td><td>FLSLIPHAINAVSAIAKHF</td><td>PS-7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>No hemolysis at 6µM (non-hemolytic)</td><td>Human</td><td>CCSCCGGGGGHHHHHTTCC</td><td align="center"><a href="jmol.php?id=1916&amp;di=1" width="5px" height="5px"><img src="pymol_images/1916.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1916&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1916.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2367">2367</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15752569">15752569</a></td><td align="right">2005</td><td>FLSLIPHAINAVSAIAKHN</td><td>PS-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antiprotozoa</td><td>1.98% hemolysis at 64µM</td><td>Human</td><td>CCSCCCTHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2367&amp;di=1" width="5px" height="5px"><img src="pymol_images/2367.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2367&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2367.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2368">2368</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15752569">15752569</a></td><td align="right">2005</td><td>FLSLIPHAINAVSTLVHHF</td><td>PS-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antiprotozoa</td><td>2.05% hemolysis at 64µM</td><td>Human</td><td>CCSSCCTHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2368&amp;di=1" width="5px" height="5px"><img src="pymol_images/2368.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2368&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2368.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3890">3890</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15996769">15996769</a></td><td align="right">2005</td><td>FLSSIGKILGNLL</td><td>Temporin-1Va</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LD<sub>50</sub> =120μM</td><td>Human</td><td>CCCGGGGGTSCCC</td><td align="center"><a href="jmol.php?id=3890&amp;di=1" width="5px" height="5px"><img src="pymol_images/3890.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3890&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3890.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3267">3267</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23624072">23624072</a></td><td align="right">2013</td><td>FLSTIWNGIKSLL</td><td>Pantinin-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolysis upto 8μM</td><td>Human</td><td>CCHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3267&amp;di=1" width="5px" height="5px"><img src="pymol_images/3267.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3267&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3267.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3270">3270</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23624072">23624072</a></td><td align="right">2013</td><td>FLSTIWNGIKSLL</td><td>Pantinin-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>70% hemolysis at 16μM</td><td>Human</td><td>CCHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3270&amp;di=1" width="5px" height="5px"><img src="pymol_images/3270.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3270&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3270.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3272">3272</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23624072">23624072</a></td><td align="right">2013</td><td>FLSTIWNGIKSLL</td><td>Pantinin-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolysis at 32μM</td><td>Human</td><td>CCHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3272&amp;di=1" width="5px" height="5px"><img src="pymol_images/3272.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3272&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3272.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1555">1555</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21110126">21110126</a></td><td align="right">2011</td><td>FLWWLFKWAWK</td><td>GA-W4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>MHC =25μg/ml</td><td>Human</td><td>CHHHHTTSSCC</td><td align="center"><a href="jmol.php?id=1555&amp;di=1" width="5px" height="5px"><img src="pymol_images/1555.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1555&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1555.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3976">3976</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18723059">18723059</a></td><td align="right">2008</td><td>FMPIIGRLMSGSL</td><td>VESP-VB1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>10.2% hemolysis up to 200μg/ml</td><td>Rabbit</td><td>CCCGGGGTCCCCC</td><td align="center"><a href="jmol.php?id=3976&amp;di=1" width="5px" height="5px"><img src="pymol_images/3976.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3976&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3976.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3977">3977</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18723059">18723059</a></td><td align="right">2008</td><td>FMPIIGRLMSGSL</td><td>VESP-VB1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3.2% hemolysis up to 200μg/ml</td><td>Human</td><td>CCCGGGGTCCCCC</td><td align="center"><a href="jmol.php?id=3977&amp;di=1" width="5px" height="5px"><img src="pymol_images/3977.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3977&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3977.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2795">2795</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fP-Ada-GO-Ada-AfP-Ada-GO-AdaA</td><td>GS6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, Ada=Adamantylalanine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =7.55±1.10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2797">2797</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fP-Chg-O-Cha-fP-Chg-O-Cha</td><td>GS8</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, Chg=Cyclohexylglycine, Cha= Cyclohexylalanine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =9.12±1.13μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2796">2796</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>FP-tBuGly-O-tBuAla-fP-tBuGly-O-tBuAla</td><td>GS7</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, tBuGly= tert-butylglycine, tBuAla=tert-butylalanine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =9.30±1.07μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2086">2086</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22484288">22484288</a></td><td align="right">2012</td><td>FPFSLIPHAIGGLISAIK</td><td>AamAP2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>19%hemolysis at 10µM</td><td>Horse</td><td>CCSSSCCTTTCBTTBCCC</td><td align="center"><a href="jmol.php?id=2086&amp;di=1" width="5px" height="5px"><img src="pymol_images/2086.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2086&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2086.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2087">2087</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22484288">22484288</a></td><td align="right">2012</td><td>FPFSLIPHAIGGLISAIK</td><td>AamAP2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>75%hemolysis at 120µM</td><td>Horse</td><td>CCSSSCCTTTCBTTBCCC</td><td align="center"><a href="jmol.php?id=2087&amp;di=1" width="5px" height="5px"><img src="pymol_images/2087.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2087&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2087.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3672">3672</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>fPLOLOfPOLOL</td><td>GS12FO/LL</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine</td><td align="right">12</td><td>Antibacterial</td><td>100% hemolysis at 170µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=3672&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3672.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2799">2799</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fPO-AdaGly-OfPO-Ada-Gly-O</td><td>GS10</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, Ada=Adamantylglycine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =107±1.07μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2798">2798</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fPOVO</td><td>GS9</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine</td><td align="right">10</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=2798&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2798.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3670">3670</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>fPVKLKfPKVKL</td><td>GS12F</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>100% hemolysis at 400µg/ml</td><td>Human</td><td>CCCSSSCTTCCC</td><td align="center"><a href="jmol.php?id=3670&amp;di=1" width="5px" height="5px"><img src="pymol_images/3670.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3670&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3670.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2794">2794</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fPVO-Ada-AfPVO-AdaA</td><td>GS5</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, Ada=Adamantylalanine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =10.3±1.07μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2792">2792</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fPVOLfP-AdaG-OL</td><td>GS2</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, Ada=Adamantylglycine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =10.1±1.11μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2793">2793</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fPVOLfPVO-AdaA</td><td>GS4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, Ada=Adamantylalanine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =5.20±1.13μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2791">2791</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fPVOLfPVOL</td><td>GS</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =17.5±1.07μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=2791&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2791.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3662">3662</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>fPVOLfPVOL</td><td>GS</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine</td><td align="right">10</td><td>Antibacterial</td><td>100% hemolysis at 39µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=3662&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3662.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3671">3671</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>fPVOLOfPOVOL</td><td>GS12FO</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine</td><td align="right">12</td><td>Antibacterial</td><td>100% hemolysis at 160µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=3671&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3671.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1905">1905</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16616888">16616888</a></td><td align="right">2006</td><td>FQWQRNIRKVR</td><td>LF11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCHHHHHCCCC</td><td align="center"><a href="jmol.php?id=1905&amp;di=1" width="5px" height="5px"><img src="pymol_images/1905.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1905&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1905.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1222">1222</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18566844">18566844</a></td><td align="right">2008</td><td>FQWQRNMRKVR</td><td>hLF(21–31)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>&gt;2-3% at ≥16μM</td><td>Human</td><td>CTTHHHHCCCC</td><td align="center"><a href="jmol.php?id=1222&amp;di=1" width="5px" height="5px"><img src="pymol_images/1222.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1222&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1222.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1639">1639</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15822124">15822124</a></td><td align="right">2005</td><td>FRKLFRVYSNFLRGKLKL</td><td>P1buy</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Hemolytic</td><td>62% hemolysis at 10µM</td><td>Rat</td><td>CCSTHHHHTTTCSSSCCC</td><td align="center"><a href="jmol.php?id=1639&amp;di=1" width="5px" height="5px"><img src="pymol_images/1639.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1639&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1639.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3798">3798</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9204560">9204560</a></td><td align="right">1997</td><td>frlkfh</td><td>66-10</td><td>Free</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">6</td><td>Antimicrobial</td><td>0% hemolysis at 312 to 0μg/ml</td><td>Human</td><td>CCSSCC</td><td align="center"><a href="jmol.php?id=3798&amp;di=1" width="5px" height="5px"><img src="pymol_images/3798.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3798&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3798.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3799">3799</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9204560">9204560</a></td><td align="right">1997</td><td>frlkfh</td><td>66-10</td><td>Free</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">6</td><td>Antimicrobial</td><td>0% hemolysis at 312 to 0μg/ml</td><td>Bovine</td><td>CCSSCC</td><td align="center"><a href="jmol.php?id=3799&amp;di=1" width="5px" height="5px"><img src="pymol_images/3799.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3799&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3799.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3800">3800</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9204560">9204560</a></td><td align="right">1997</td><td>frlkfh</td><td>66-10</td><td>Free</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">6</td><td>Antimicrobial</td><td>0% hemolysis at 312 to 0μg/ml</td><td>Ovine</td><td>CCSSCC</td><td align="center"><a href="jmol.php?id=3800&amp;di=1" width="5px" height="5px"><img src="pymol_images/3800.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3800&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3800.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1369">1369</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19524300">19524300</a></td><td align="right">2009</td><td>FRPKVTITIQGSARF</td><td>C12-26</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Immunomodulatory and Antimicrobial</td><td>0% hemolysis at 40μM</td><td>Chicken</td><td>CCCCCCCCSCSSTTC</td><td align="center"><a href="jmol.php?id=1369&amp;di=1" width="5px" height="5px"><img src="pymol_images/1369.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1369&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1369.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1045">1045</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11087404">11087404</a></td><td align="right">2000</td><td>FRWCFRVCYKGRCRYKCR</td><td>PV8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>100% hemolysis at 85μg/ml</td><td>Human</td><td>CCCCCCCCTTSCCCCCCC</td><td align="center"><a href="jmol.php?id=1045&amp;di=1" width="5px" height="5px"><img src="pymol_images/1045.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1045&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1045.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2588">2588</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23519707">23519707</a></td><td align="right">2013</td><td>FSEAIKKIIDFLGEGLFDIIKKIAESF</td><td>E(AU)2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>~5% hemolytic at 128μmol/L</td><td>Human</td><td>CTHHHHTTTTTTCCCTTGGGGTTTSCC</td><td align="center"><a href="jmol.php?id=2588&amp;di=1" width="5px" height="5px"><img src="pymol_images/2588.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2588&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2588.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2589">2589</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23519707">23519707</a></td><td align="right">2013</td><td>FSEAIKKIIDFLGEGLFDIIKKIAESF</td><td>E(AU)2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>0% hemolytic at 4-32μmol/L</td><td>Human</td><td>CTHHHHTTTTTTCCCTTGGGGTTTSCC</td><td align="center"><a href="jmol.php?id=2589&amp;di=1" width="5px" height="5px"><img src="pymol_images/2589.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2589&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2589.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2613">2613</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23124812">23124812</a></td><td align="right">2013</td><td>FSGGNCRGFRRRCFCTK</td><td>SolyC</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>&lt;5% hemolytic at 50mg/ml</td><td>Mouse</td><td>CCSSCCCSSSSCCSSCC</td><td align="center"><a href="jmol.php?id=2613&amp;di=1" width="5px" height="5px"><img src="pymol_images/2613.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2613&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2613.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4193">4193</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16460023">16460023</a></td><td align="right">2006</td><td>FSISPGKVLDKFGKIVGKVLKQLKKVSAVAKV</td><td>P6_2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">31</td><td>Hemolytic</td><td>LC<sub>50</sub>=28.1µM</td><td>Human</td><td>NA</td><td align="center"><a href="jmol.php?id=4193&amp;di=1" width="5px" height="5px"><img src="pymol_images/4193.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4193&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4193.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3086">3086</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20637182">20637182</a></td><td align="right">2010</td><td>FSPQMLQDIIEAATAIL</td><td>A17</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 200μg/ml</td><td>Mouse</td><td>CCCCTTTTTTHHHHHTC</td><td align="center"><a href="jmol.php?id=3086&amp;di=1" width="5px" height="5px"><img src="pymol_images/3086.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3086&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3086.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3087">3087</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20637182">20637182</a></td><td align="right">2010</td><td>FSPQMLQDIIEKKTKIL</td><td>K17</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 200μg/ml</td><td>Mouse</td><td>CCCHHHHHHTGGGTTCC</td><td align="center"><a href="jmol.php?id=3087&amp;di=1" width="5px" height="5px"><img src="pymol_images/3087.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3087&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3087.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1765">1765</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>FTicOicKTicOicFTicOicKTicKKKK</td><td>Peptide-28</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic- tetrahydroisoquinolinecarboxylic acid, Oic- octahydroindolecarboxylic acid</td><td align="right">15</td><td>Antibiotic</td><td>86.8% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1766">1766</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>FTicOicKTicOicFTicOicKTicKKKK</td><td>Peptide-28</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic- tetrahydroisoquinolinecarboxylic acid, Oic- octahydroindolecarboxylic acid</td><td align="right">15</td><td>Antibiotic</td><td>50% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1785">1785</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>FTicOicKTicOicFTicOicKTicKKKKKK</td><td>Peptide-38</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic- tetrahydroisoquinolinecarboxylic acid, Oic- octahydroindolecarboxylic acid</td><td align="right">17</td><td>Antibiotic</td><td>41.3% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1786">1786</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>FTicOicKTicOicFTicOicKTicKKKKKK</td><td>Peptide-38</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid</td><td align="right">17</td><td>Antibiotic</td><td>33.5% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1998">1998</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21871946">21871946</a></td><td align="right">2012</td><td>FVDLKKIANIINSIF</td><td>Temporin-1CEa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>60% hemolysis at 100µM</td><td>Human</td><td>CCCTTTTHHHHTCCC</td><td align="center"><a href="jmol.php?id=1998&amp;di=1" width="5px" height="5px"><img src="pymol_images/1998.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1998&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1998.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1999">1999</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21871946">21871946</a></td><td align="right">2012</td><td>FVDLKKIANIINSIF</td><td>Temporin-1CEa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>100% hemolysis at 200µM</td><td>Human</td><td>CCCTTTTHHHHTCCC</td><td align="center"><a href="jmol.php?id=1999&amp;di=1" width="5px" height="5px"><img src="pymol_images/1999.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1999&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1999.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2570">2570</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FVDLKKIANIINSIFGK</td><td>Temporin-1CEa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial and Anticancer and moderate Hemolytic</td><td>LC<sub>50</sub> =95.7μM</td><td>Human</td><td>CCCHHHHHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=2570&amp;di=1" width="5px" height="5px"><img src="pymol_images/2570.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2570&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2570.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2571">2571</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FVDLKKIANIINSIFKK</td><td>LK1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Anticancer</td><td>LC<sub>50</sub> =1,467μM</td><td>Human</td><td>CCCHHHHHHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2571&amp;di=1" width="5px" height="5px"><img src="pymol_images/2571.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2571&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2571.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3486">3486</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19427345">19427345</a></td><td align="right">2009</td><td>FVGAIAAALPHVISAIKNAL</td><td>Kassinatuerin-2Mc</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>45-50% hemolysis at 120μM</td><td>Horse</td><td>CCSTHHHHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=3486&amp;di=1" width="5px" height="5px"><img src="pymol_images/3486.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2573">2573</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FVKLKKIANIINSIFKK</td><td>LK2(6)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Anticancer</td><td>LC<sub>50</sub> =1,993μM</td><td>Human</td><td>CCCTHHHHHHHTTTTCC</td><td align="center"><a href="jmol.php?id=2573&amp;di=1" width="5px" height="5px"><img src="pymol_images/2573.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2573&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2573.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2576">2576</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FVKLKKILNIILSIFKK</td><td>LK2(6)AN(2L)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Anticancer</td><td>LC<sub>50</sub> =28.34μM</td><td>Human</td><td>CCCTHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2576&amp;di=1" width="5px" height="5px"><img src="pymol_images/2576.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2576&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2576.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2575">2575</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FVKLKKILNIINSIFKK</td><td>LK2(6)A(L)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Anticancer</td><td>LC<sub>50</sub> =86.99μM</td><td>Human</td><td>CCSTHHHHHHHTTTTCC</td><td align="center"><a href="jmol.php?id=2575&amp;di=1" width="5px" height="5px"><img src="pymol_images/2575.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2575&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2575.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1073">1073</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20153801">20153801</a></td><td align="right">2010</td><td>FVLPLVMCKILRKC</td><td>Japonicin-1Npb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>3.40% hemolysis at 80μg/ml</td><td>Human</td><td>CCCHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=1073&amp;di=1" width="5px" height="5px"><img src="pymol_images/1073.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1073&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1073.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2844">2844</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVPWFSKFLGOIL</td><td>[Pro<sup>3</sup>, Orn<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolytic at 6μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2835">2835</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL<sup>b</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>10% hemolytic at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=2835&amp;di=1" width="5px" height="5px"><img src="pymol_images/2835.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2835&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2835.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3194">3194</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3194&amp;di=1" width="5px" height="5px"><img src="pymol_images/3194.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3194&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3194.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3195">3195</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>92% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3195&amp;di=1" width="5px" height="5px"><img src="pymol_images/3195.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3195&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3195.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3196">3196</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>42% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3196&amp;di=1" width="5px" height="5px"><img src="pymol_images/3196.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3196&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3196.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3197">3197</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>10% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3197&amp;di=1" width="5px" height="5px"><img src="pymol_images/3197.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3197&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3197.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3198">3198</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>6% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3198&amp;di=1" width="5px" height="5px"><img src="pymol_images/3198.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3198&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3198.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3199">3199</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>4.5% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3199&amp;di=1" width="5px" height="5px"><img src="pymol_images/3199.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3199&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3199.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3200">3200</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSkFLGRIL</td><td>[Pro<sup>3</sup>, DLys<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>18% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3200&amp;di=1" width="5px" height="5px"><img src="pymol_images/3200.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3200&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3200.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3201">3201</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSkFLGRIL</td><td>[Pro<sup>3</sup>, DLys<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>8% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3201&amp;di=1" width="5px" height="5px"><img src="pymol_images/3201.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3201&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3201.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3202">3202</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSkFLGRIL</td><td>[Pro<sup>3</sup>, DLys<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>5% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3202&amp;di=1" width="5px" height="5px"><img src="pymol_images/3202.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3202&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3202.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3203">3203</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSkFLGRIL</td><td>[Pro<sup>3</sup>, DLys<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>7% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3203&amp;di=1" width="5px" height="5px"><img src="pymol_images/3203.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3203&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3203.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3204">3204</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSkFLGRIL</td><td>[Pro<sup>3</sup>, DLys<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>4% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3204&amp;di=1" width="5px" height="5px"><img src="pymol_images/3204.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3204&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3204.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3205">3205</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSkFLGRIL</td><td>[Pro<sup>3</sup>, DLys<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3205&amp;di=1" width="5px" height="5px"><img src="pymol_images/3205.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3205&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3205.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3206">3206</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKfLGRIL</td><td>[Pro<sup>3</sup>, DPhe<sup>8</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>94% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3206&amp;di=1" width="5px" height="5px"><img src="pymol_images/3206.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3206&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3206.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3207">3207</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKfLGRIL</td><td>[Pro<sup>3</sup>, DPhe<sup>8</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>19% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3207&amp;di=1" width="5px" height="5px"><img src="pymol_images/3207.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3207&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3207.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3208">3208</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKfLGRIL</td><td>[Pro<sup>3</sup>, DPhe<sup>8</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>19% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3208&amp;di=1" width="5px" height="5px"><img src="pymol_images/3208.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3208&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3208.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3209">3209</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKfLGRIL</td><td>[Pro<sup>3</sup>, DPhe<sup>8</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>17% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3209&amp;di=1" width="5px" height="5px"><img src="pymol_images/3209.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3209&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3209.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3210">3210</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKfLGRIL</td><td>[Pro<sup>3</sup>, DPhe<sup>8</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>5% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3210&amp;di=1" width="5px" height="5px"><img src="pymol_images/3210.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3210&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3210.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3211">3211</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKfLGRIL</td><td>[Pro<sup>3</sup>, DPhe<sup>8</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>4% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3211&amp;di=1" width="5px" height="5px"><img src="pymol_images/3211.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3211&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3211.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3212">3212</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIL</td><td>[Pro<sup>3</sup>, DLeu<sup>9</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>9% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3212&amp;di=1" width="5px" height="5px"><img src="pymol_images/3212.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3212&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3212.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3213">3213</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIL</td><td>[Pro<sup>3</sup>, DLeu<sup>9</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>8% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3213&amp;di=1" width="5px" height="5px"><img src="pymol_images/3213.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3213&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3213.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3214">3214</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIL</td><td>[Pro<sup>3</sup>, DLeu<sup>9</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>7% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3214&amp;di=1" width="5px" height="5px"><img src="pymol_images/3214.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3214&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3214.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3215">3215</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIL</td><td>[Pro<sup>3</sup>, DLeu<sup>9</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3215&amp;di=1" width="5px" height="5px"><img src="pymol_images/3215.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3215&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3215.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3216">3216</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIL</td><td>[Pro<sup>3</sup>, DLeu<sup>9</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3216&amp;di=1" width="5px" height="5px"><img src="pymol_images/3216.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3216&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3216.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3217">3217</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIL</td><td>[Pro<sup>3</sup>, DLeu<sup>9</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3217&amp;di=1" width="5px" height="5px"><img src="pymol_images/3217.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3217&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3217.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3218">3218</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGrIL</td><td>[Pro<sup>3</sup>, DArg<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>22% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3218&amp;di=1" width="5px" height="5px"><img src="pymol_images/3218.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3218&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3218.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3219">3219</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGrIL</td><td>[Pro<sup>3</sup>, DArg<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>16% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3219&amp;di=1" width="5px" height="5px"><img src="pymol_images/3219.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3219&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3219.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3220">3220</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGrIL</td><td>[Pro<sup>3</sup>, DArg<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>8% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3220&amp;di=1" width="5px" height="5px"><img src="pymol_images/3220.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3220&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3220.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3221">3221</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGrIL</td><td>[Pro<sup>3</sup>, DArg<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>6% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3221&amp;di=1" width="5px" height="5px"><img src="pymol_images/3221.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3221&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3221.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3222">3222</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGrIL</td><td>[Pro<sup>3</sup>, DArg<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>4% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3222&amp;di=1" width="5px" height="5px"><img src="pymol_images/3222.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3222&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3222.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3223">3223</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGrIL</td><td>[Pro<sup>3</sup>, DArg<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3223&amp;di=1" width="5px" height="5px"><img src="pymol_images/3223.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3223&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3223.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3224">3224</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRiL</td><td>[Pro<sup>3</sup>, DIle<sup>12</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>17% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3224&amp;di=1" width="5px" height="5px"><img src="pymol_images/3224.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3224&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3224.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3225">3225</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRiL</td><td>[Pro<sup>3</sup>, DIle<sup>12</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>14% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3225&amp;di=1" width="5px" height="5px"><img src="pymol_images/3225.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3225&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3225.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3226">3226</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRiL</td><td>[Pro<sup>3</sup>, DIle<sup>12</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>10% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3226&amp;di=1" width="5px" height="5px"><img src="pymol_images/3226.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3226&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3226.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3227">3227</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRiL</td><td>[Pro<sup>3</sup>, DIle<sup>12</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>5% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3227&amp;di=1" width="5px" height="5px"><img src="pymol_images/3227.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3227&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3227.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3228">3228</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRiL</td><td>[Pro<sup>3</sup>, DIle<sup>12</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>4% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3228&amp;di=1" width="5px" height="5px"><img src="pymol_images/3228.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3228&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3228.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3229">3229</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRiL</td><td>[Pro<sup>3</sup>, DIle<sup>12</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3229&amp;di=1" width="5px" height="5px"><img src="pymol_images/3229.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3229&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3229.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3230">3230</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIl</td><td>[Pro<sup>3</sup>, DLeu<sup>13</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>15% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3230&amp;di=1" width="5px" height="5px"><img src="pymol_images/3230.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3230&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3230.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3231">3231</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIl</td><td>[Pro<sup>3</sup>, DLeu<sup>13</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>17% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3231&amp;di=1" width="5px" height="5px"><img src="pymol_images/3231.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3231&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3231.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3232">3232</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIl</td><td>[Pro<sup>3</sup>, DLeu<sup>13</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>17% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3232&amp;di=1" width="5px" height="5px"><img src="pymol_images/3232.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3232&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3232.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3233">3233</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIl</td><td>[Pro<sup>3</sup>, DLeu<sup>13</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>8% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3233&amp;di=1" width="5px" height="5px"><img src="pymol_images/3233.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3233&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3233.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3234">3234</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIl</td><td>[Pro<sup>3</sup>, DLeu<sup>13</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>6% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3234&amp;di=1" width="5px" height="5px"><img src="pymol_images/3234.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3234&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3234.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3235">3235</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIl</td><td>[Pro<sup>3</sup>, DLeu<sup>13</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3235&amp;di=1" width="5px" height="5px"><img src="pymol_images/3235.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3235&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3235.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2837">2837</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVPWFSKFLPRIL</td><td>[Pro<sup>3</sup>, Pro<sup>10</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>4.5% hemolytic at 6μM</td><td>Human</td><td>CCCCGGGTSSSCC</td><td align="center"><a href="jmol.php?id=2837&amp;di=1" width="5px" height="5px"><img src="pymol_images/2837.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2837&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2837.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2838">2838</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVPWFSKFLpRIL</td><td>[Pro<sup>3</sup>, DPro<sup>10</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0.6% hemolytic at 6μM</td><td>Human</td><td>CCCCGGGTSSSCC</td><td align="center"><a href="jmol.php?id=2838&amp;di=1" width="5px" height="5px"><img src="pymol_images/2838.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2838&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2838.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2839">2839</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVQWFSKFLGKIL</td><td>[Lys<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>14% hemolytic at 6μM</td><td>Human</td><td>CHHHHHHHHTCCC</td><td align="center"><a href="jmol.php?id=2839&amp;di=1" width="5px" height="5px"><img src="pymol_images/2839.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2839&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2839.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2840">2840</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVQWFSKFLGOIL</td><td>[Orn<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Ornithine</td><td align="right">13</td><td>Antimicrobial</td><td>22% hemolytic at 6μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=2840&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2840.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2831">2831</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>48% hemolytic at 6μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=2831&amp;di=1" width="5px" height="5px"><img src="pymol_images/2831.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2831&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2831.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2845">2845</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>fvqwfskflGril</td><td>D-TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>46% hemolytic at 6μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=2845&amp;di=1" width="5px" height="5px"><img src="pymol_images/2845.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2845&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2845.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3188">3188</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolysis at 48μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3188&amp;di=1" width="5px" height="5px"><img src="pymol_images/3188.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3188&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3188.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3189">3189</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>94% hemolysis at 24μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3189&amp;di=1" width="5px" height="5px"><img src="pymol_images/3189.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3189&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3189.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3190">3190</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>92% hemolysis at 12μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3190&amp;di=1" width="5px" height="5px"><img src="pymol_images/3190.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3190&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3190.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3191">3191</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>48% hemolysis at 6μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3191&amp;di=1" width="5px" height="5px"><img src="pymol_images/3191.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3191&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3191.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3192">3192</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>13% hemolysis at 3μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3192&amp;di=1" width="5px" height="5px"><img src="pymol_images/3192.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3192&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3192.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3193">3193</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3% hemolysis at 1.5μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3193&amp;di=1" width="5px" height="5px"><img src="pymol_images/3193.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3193&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3193.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4148">4148</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19635516">19635516</a></td><td align="right">2009</td><td>FVQWFSKFLGRIL</td><td>Temporin L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LC<sub>50</sub> =2μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=4148&amp;di=1" width="5px" height="5px"><img src="pymol_images/4148.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4148&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4148.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2834">2834</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVQWFSKFLLRIL</td><td>[Leu<sup>10</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>80% hemolytic at 6μM</td><td>Human</td><td>CHHHHHHHTTTTC</td><td align="center"><a href="jmol.php?id=2834&amp;di=1" width="5px" height="5px"><img src="pymol_images/2834.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2834&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2834.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2841">2841</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVQWFSRFLGRIL</td><td>[Arg<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>29% hemolytic at 6μM</td><td>Human</td><td>CHHHHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2841&amp;di=1" width="5px" height="5px"><img src="pymol_images/2841.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2841&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2841.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2842">2842</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVRWFSKFLGRIL</td><td>[Arg<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>52% hemolytic at 6μM</td><td>Human</td><td>CHHHHHTTTTCCC</td><td align="center"><a href="jmol.php?id=2842&amp;di=1" width="5px" height="5px"><img src="pymol_images/2842.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2842&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2842.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2843">2843</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVRWFSRFLGRIL</td><td>[Arg<sup>3</sup>, Arg<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>43% hemolytic at 6μM</td><td>Human</td><td>CCGGGHHHHTTCC</td><td align="center"><a href="jmol.php?id=2843&amp;di=1" width="5px" height="5px"><img src="pymol_images/2843.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2843&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2843.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1279">1279</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20033827">20033827</a></td><td align="right">2011</td><td>FWGALAKGALKLIGVGSLFSSFSKKD</td><td>Pin2 [P14GVG]</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>IC<sub>50</sub> =11.5μM </td><td>Human</td><td>CHHHHHHHHHSCCCCCCCCCCSSCCC</td><td align="center"><a href="jmol.php?id=1279&amp;di=1" width="5px" height="5px"><img src="pymol_images/1279.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1279&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1279.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1277">1277</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20033827">20033827</a></td><td align="right">2011</td><td>FWGALAKGALKLIGVSLFSSFSKKD</td><td>Pin2 [P14GV]</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>IC<sub>50</sub> =9.3μM </td><td>Human</td><td>CHHHHHHHHHHTTSCSSCCCCSSCC</td><td align="center"><a href="jmol.php?id=1277&amp;di=1" width="5px" height="5px"><img src="pymol_images/1277.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1277&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1277.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1275">1275</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20033827">20033827</a></td><td align="right">2011</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>IC<sub>50</sub> =3.3μM </td><td>Human</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=1275&amp;di=1" width="5px" height="5px"><img src="pymol_images/1275.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1275&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1275.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2191">2191</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11976325">11976325</a></td><td align="right">2002</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial and Insecticidal</td><td>~100% hemolysis at 50μM</td><td>Pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2191&amp;di=1" width="5px" height="5px"><img src="pymol_images/2191.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2191&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2191.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2192">2192</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11976325">11976325</a></td><td align="right">2002</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial and Insecticidal</td><td>~100% hemolysis at 50μM</td><td>Guinea-pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2192&amp;di=1" width="5px" height="5px"><img src="pymol_images/2192.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2192&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2192.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2193">2193</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11976325">11976325</a></td><td align="right">2002</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial and Insecticidal</td><td>~100% hemolysis at 50μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2193&amp;di=1" width="5px" height="5px"><img src="pymol_images/2193.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2193&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2193.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2342">2342</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14637016">14637016</a></td><td align="right">2003</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>&gt;80% hemolysis at 50μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2342&amp;di=1" width="5px" height="5px"><img src="pymol_images/2342.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2342&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2342.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2343">2343</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14637016">14637016</a></td><td align="right">2003</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>100% hemolysis at 50μM</td><td>Pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2343&amp;di=1" width="5px" height="5px"><img src="pymol_images/2343.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2343&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2343.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2344">2344</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14637016">14637016</a></td><td align="right">2003</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>100% hemolysis at 50μM</td><td>Guinea-pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2344&amp;di=1" width="5px" height="5px"><img src="pymol_images/2344.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2344&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2344.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3554">3554</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11563967">11563967</a></td><td align="right">2001</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>90.5±0.8% hemolysis at 44.5μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3554&amp;di=1" width="5px" height="5px"><img src="pymol_images/3554.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3555">3555</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11563967">11563967</a></td><td align="right">2001</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>51.4±0.5% hemolysis at 22.2μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3555&amp;di=1" width="5px" height="5px"><img src="pymol_images/3555.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3556">3556</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11563967">11563967</a></td><td align="right">2001</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>17.5±1.1% hemolysis at 11.1μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3556&amp;di=1" width="5px" height="5px"><img src="pymol_images/3556.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3557">3557</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11563967">11563967</a></td><td align="right">2001</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>3.7±0.1% hemolysis at 5.5μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3557&amp;di=1" width="5px" height="5px"><img src="pymol_images/3557.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3558">3558</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11563967">11563967</a></td><td align="right">2001</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>1.4±0.3% hemolysis at 2.7μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3558&amp;di=1" width="5px" height="5px"><img src="pymol_images/3558.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4063">4063</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328050">15328050</a></td><td align="right">2004</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>7% hemolysis at 1μM</td><td>Guinea-pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=4063&amp;di=1" width="5px" height="5px"><img src="pymol_images/4063.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4064">4064</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328050">15328050</a></td><td align="right">2004</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>50% hemolysis at 6μM</td><td>Guinea-pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=4064&amp;di=1" width="5px" height="5px"><img src="pymol_images/4064.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4065">4065</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328050">15328050</a></td><td align="right">2004</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>85% hemolysis at 12.5μM</td><td>Guinea-pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=4065&amp;di=1" width="5px" height="5px"><img src="pymol_images/4065.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4066">4066</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328050">15328050</a></td><td align="right">2004</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>100% hemolysis at 25μM</td><td>Guinea-pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=4066&amp;di=1" width="5px" height="5px"><img src="pymol_images/4066.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1278">1278</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20033827">20033827</a></td><td align="right">2011</td><td>FWGALAKGALKLIVGSLFSSFSKKD</td><td>Pin2 [P14VG]</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>IC<sub>50</sub> =8.8μM </td><td>Human</td><td>CHHHHHHHHHHHHTTTTCCSCCCCC</td><td align="center"><a href="jmol.php?id=1278&amp;di=1" width="5px" height="5px"><img src="pymol_images/1278.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1278&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1278.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1276">1276</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20033827">20033827</a></td><td align="right">2011</td><td>FWGALAKGALKLIVSLFSSFSKKD</td><td>Pin2 [P14V]</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>IC<sub>50</sub> =6.4μM </td><td>Human</td><td>CHHHHHHHHHHHHGGGTSSSSCCC</td><td align="center"><a href="jmol.php?id=1276&amp;di=1" width="5px" height="5px"><img src="pymol_images/1276.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1276&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1276.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2117">2117</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>FWGALIKGAAKLIPSVVGLFKKKQ</td><td>Ponericin-W5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition (4 mm) at 0.4-0.5mM</td><td>Horse</td><td>CGGGTTTTTSTTCGGGTSSCCCCC</td><td align="center"><a href="jmol.php?id=2117&amp;di=1" width="5px" height="5px"><img src="pymol_images/2117.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2117&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2117.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2118">2118</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>FWGALIKGAAKLIPSVVGLFKKKQ</td><td>Ponericin-W5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition (1.5 mm) at 0.4-0.5mM</td><td>Sheep</td><td>CGGGTTTTTSTTCGGGTSSCCCCC</td><td align="center"><a href="jmol.php?id=2118&amp;di=1" width="5px" height="5px"><img src="pymol_images/2118.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2118&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2118.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3619">3619</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19085906">19085906</a></td><td align="right">2008</td><td>FWGHIWNAVKRVGANALHGAVTGALS</td><td>Halocyntin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>22% hemolysis at 50µM</td><td>Sheep</td><td>CTTTHHHHHTTSEETTEESSCBSSCC</td><td align="center"><a href="jmol.php?id=3619&amp;di=1" width="5px" height="5px"><img src="pymol_images/3619.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2145">2145</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Aib-Orn-Orn-Aib-Aib-Orn-Orn-Nle-Nle-GY</td><td>P13(5|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">13</td><td>Antimicrobial</td><td>12% hemolysis at 10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2139">2139</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Aib-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P15(7|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">15</td><td>Antimicrobial</td><td>5% hemolysis at 10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2140">2140</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Aib-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P15(7|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">15</td><td>Antimicrobial</td><td>30% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2141">2141</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Aib-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-GY</td><td>P14(7|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">14</td><td>Antimicrobial</td><td>10% hemolysis at 10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2142">2142</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Aib-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-GY</td><td>P14(7|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">14</td><td>Antimicrobial</td><td>30% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2146">2146</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Aib-Orn-Orn-Nle-Aib-Orn-Orn-Nle-Nle-Orn-GY</td><td>P14(6|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">14</td><td>Antimicrobial</td><td>70% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2135">2135</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Nle-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P19(9|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">19</td><td>Antimicrobial</td><td>30% hemolysis at 10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2136">2136</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Nle-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P19(9|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">19</td><td>Antimicrobial</td><td>70% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2143">2143</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P13(7|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">13</td><td>Antimicrobial</td><td>5% hemolysis at 10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2144">2144</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P13(7|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">13</td><td>Antimicrobial</td><td>25% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2137">2137</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P17(9|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">17</td><td>Antimicrobial</td><td>15% hemolysis at 10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2138">2138</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P17(9|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">17</td><td>Antimicrobial</td><td>35% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1464">1464</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20176897">20176897</a></td><td align="right">2010</td><td>G-ΔF-RK-ΔF-HK-ΔF-WA</td><td>ΔFm</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>ΔF=α,β-didehydrophenylalanine</td><td align="right">10</td><td>Antibacterial</td><td>0% hemolysis upto 1358μg/ml(non-hemolytic)</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1465">1465</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20176897">20176897</a></td><td align="right">2010</td><td>G-ΔF-RK-ΔF-KA-ΔF-WH</td><td>ΔFmscr</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>ΔF=α,β-didehydrophenylalanine</td><td align="right">10</td><td>Antibacterial</td><td>0% hemolysis upto 679μg/ml(non-hemolytic)</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1767">1767</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GABAFTicOicGABAKTicOicGABAFTicOicGABAKTicKKKK</td><td>Peptide-29</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic=  tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid, GABA= γ-aminobutyric acid</td><td align="right">19</td><td>Antibiotic</td><td>10.8% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1768">1768</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GABAFTicOicGABAKTicOicGABAFTicOicGABAKTicKKKK</td><td>Peptide-29</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic=  tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid, GABA= γ-aminobutyric acid</td><td align="right">19</td><td>Antibiotic</td><td>1% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1789">1789</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GABAFTicOicGABAKTicOicGABAFTicOicGABAKTicKKKKK</td><td>Peptide-40</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid, GABA= γ-aminobutyric acid,</td><td align="right">18</td><td>Antibiotic</td><td>29.7% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1790">1790</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GABAFTicOicGABAKTicOicGABAFTicOicGABAKTicKKKKK</td><td>Peptide-40</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid, GABA= γ-aminobutyric acid,</td><td align="right">18</td><td>Antibiotic</td><td>10.5% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2076">2076</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22426384">22426384</a></td><td align="right">2012</td><td>GAFGDLLKGVAKEAGMKLLNMAQCKLSGKC</td><td>Brevinin-2LTa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>LD<sub>50</sub> =520µM</td><td>Human</td><td>CTTSCHHHHHHHHTHHHHHTTTSCCCSSCC</td><td align="center"><a href="jmol.php?id=2076&amp;di=1" width="5px" height="5px"><img src="pymol_images/2076.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2076&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2076.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3770">3770</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14636071">14636071</a></td><td align="right">2003</td><td>GALRGCWTKSYPPKPCK</td><td>Ranacyclin T</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>20% hemolysis at 100μM</td><td>Human</td><td>CCCCSSTTSCSCSSCCC</td><td align="center"><a href="jmol.php?id=3770&amp;di=1" width="5px" height="5px"><img src="pymol_images/3770.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1675">1675</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16460028">16460028</a></td><td align="right">2006</td><td>GASLSFKILKTVLEALGNV</td><td>EqTII<sub>11-29</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Hemolytic</td><td>10% hemolysis at 25µM</td><td>Human</td><td>CCSSCHHHHHHHHHTTTCC</td><td align="center"><a href="jmol.php?id=1675&amp;di=1" width="5px" height="5px"><img src="pymol_images/1675.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1675&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1675.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1904">1904</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16460028">16460028</a></td><td align="right">2006</td><td>GASLSFKILKTVLEALGNVKRK</td><td>EqTII<sup>11-32</sup></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCSSSHHHHTTTHHHHGGGGGC</td><td align="center"><a href="jmol.php?id=1904&amp;di=1" width="5px" height="5px"><img src="pymol_images/1904.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1904&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1904.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1943">1943</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17888907">17888907</a></td><td align="right">2008</td><td>GCASRCKAKCAGRRCKGWASASFRGRCYCKCFRC</td><td>Mytilin-A</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antimicrobial</td><td>10% hemolysis 100µM</td><td>Human</td><td>CCCCTTTSSCCCCCCCCCSSSCCSSSCCCCCCCC</td><td align="center"><a href="jmol.php?id=1943&amp;di=1" width="5px" height="5px"><img src="pymol_images/1943.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1943&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1943.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2940">2940</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GCKKWFKKAAHVGKNVGKVALNAYL</td><td>NRC-127</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCTTTHHHHTCSCCCSSSCCTTSCC</td><td align="center"><a href="jmol.php?id=2940&amp;di=1" width="5px" height="5px"><img src="pymol_images/2940.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2940&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2940.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2389">2389</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15546886">15546886</a></td><td align="right">2005</td><td>GCRFCCNCCPNMSGCGVCCRF</td><td>Bass Hepcidin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antibacterial and Antifungal</td><td>0% hemolysis at 100 µg/ml (non-hemolytic)</td><td>Rat</td><td>CCCCCSSSSSCCSCCSSSSCC</td><td align="center"><a href="jmol.php?id=2389&amp;di=1" width="5px" height="5px"><img src="pymol_images/2389.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2389&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2389.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2608">2608</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23226256">23226256</a></td><td align="right">2013</td><td>GECIWDAIFHGAKHFLHRLVNP</td><td>TP2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>60% hemolytic at 100µg/ml</td><td>Fish</td><td>CCSSHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2608&amp;di=1" width="5px" height="5px"><img src="pymol_images/2608.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2608&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2608.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2517">2517</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10973820">10973820</a></td><td align="right">2000</td><td>GEKLKKIGKKIKNFFQKL</td><td>K9-CRAMP-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and Anticancer</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CCSHHHHHHHHTTTTTCC</td><td align="center"><a href="jmol.php?id=2517&amp;di=1" width="5px" height="5px"><img src="pymol_images/2517.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2517&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2517.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2518">2518</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10973820">10973820</a></td><td align="right">2000</td><td>GEKLKKIGQKIKKFFQKL</td><td>K13-CRAMP-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and Anticancer</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CCHHHHHHHHHHGGGGGC</td><td align="center"><a href="jmol.php?id=2518&amp;di=1" width="5px" height="5px"><img src="pymol_images/2518.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2518&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2518.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2519">2519</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10973820">10973820</a></td><td align="right">2000</td><td>GEKLKKIGQKIKNFFKKL</td><td>K16-CRAMP-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and Anticancer</td><td>0.2% hemolytic at 100μM</td><td>Human</td><td>CCSSTTHHHHHHHHCCCC</td><td align="center"><a href="jmol.php?id=2519&amp;di=1" width="5px" height="5px"><img src="pymol_images/2519.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2519&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2519.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2515">2515</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10973820">10973820</a></td><td align="right">2000</td><td>GEKLKKIGQKIKNFFQKL</td><td>CRAMP-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and Anticancer</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CCHHHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2515&amp;di=1" width="5px" height="5px"><img src="pymol_images/2515.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2515&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2515.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2538">2538</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10673369">10673369</a></td><td align="right">1999</td><td>GFCRCLCRRGVCRCLCTK</td><td>RTD</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>EC<sub>25</sub> =2350μM</td><td>Human</td><td>CCTTSSCCCCCCCCCTTC</td><td align="center"><a href="jmol.php?id=2538&amp;di=1" width="5px" height="5px"><img src="pymol_images/2538.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2538&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2538.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3791">3791</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9010936">9010936</a></td><td align="right">1996</td><td>GFFALIAKIISSPLFKTL</td><td>18A</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>100% hemolysis at 33μg/ml</td><td>Human</td><td>CCHHHHHHHHSCSCCCCC</td><td align="center"><a href="jmol.php?id=3791&amp;di=1" width="5px" height="5px"><img src="pymol_images/3791.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3791&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3791.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3797">3797</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9108731">9108731</a></td><td align="right">1996</td><td>GFFALIAKIISSPLFKTL</td><td>PX1–18 (P7A)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Cytolytic</td><td>Conc. Required for hemolysis =7.1nmols</td><td>Human</td><td>CCHHHHHHHHSCSCCCCC</td><td align="center"><a href="jmol.php?id=3797&amp;di=1" width="5px" height="5px"><img src="pymol_images/3797.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3797&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3797.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3794">3794</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9108731">9108731</a></td><td align="right">1996</td><td>GFFALIAKIISSPLFKTLLSAVGSALSSSGEQE</td><td>PX1–33 (P7A)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Cytolytic</td><td>Conc. Required for hemolysis =0.6nmols</td><td>Human</td><td>CCSSHHHHHHTSHHHHHHHHHGGGGTSSCCCCC</td><td align="center"><a href="jmol.php?id=3794&amp;di=1" width="5px" height="5px"><img src="pymol_images/3794.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3794&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3794.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3792">3792</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9010936">9010936</a></td><td align="right">1996</td><td>GFFALIAQIISSPLFQTL</td><td>18Q</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>100% hemolysis at 56μg/ml</td><td>Human</td><td>CHHHHHHHHHHCSSCCCC</td><td align="center"><a href="jmol.php?id=3792&amp;di=1" width="5px" height="5px"><img src="pymol_images/3792.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3792&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3792.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3681">3681</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIIS</td><td>Des-(12-33)-pardaxin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antibacterial</td><td>No hemolysis (Non hemolytic)</td><td>Human</td><td>CCCCCCCTTCC</td><td align="center"><a href="jmol.php?id=3681&amp;di=1" width="5px" height="5px"><img src="pymol_images/3681.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3681&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3681.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3682">3682</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIIS</td><td>Des-(I 2-33)-pardaxin-[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antibacterial</td><td>No hemolysis (Non hemolytic)</td><td>Human</td><td>CCCCCCCTTCC</td><td align="center"><a href="jmol.php?id=3682&amp;di=1" width="5px" height="5px"><img src="pymol_images/3682.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3790">3790</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9010936">9010936</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTL</td><td>18P</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>No hemolysis (Non hemolytic)</td><td>Human</td><td>CCCCCCTTTTSCCCCCCC</td><td align="center"><a href="jmol.php?id=3790&amp;di=1" width="5px" height="5px"><img src="pymol_images/3790.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3790&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3790.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3796">3796</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9108731">9108731</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTL</td><td>PX1–18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Cytolytic</td><td>No hemolysis (Non hemolysis)</td><td>Human</td><td>CCCCCCTTTTSCCCCCCC</td><td align="center"><a href="jmol.php?id=3796&amp;di=1" width="5px" height="5px"><img src="pymol_images/3796.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3796&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3796.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3676">3676</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAV</td><td>Des-(23-33)-pardaxin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antibacterial</td><td>&gt;100% hemolysis at 10µM</td><td>Human</td><td>CCCCCCHHHHTSHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3676&amp;di=1" width="5px" height="5px"><img src="pymol_images/3676.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3676&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3676.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3677">3677</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAV</td><td>Des-(23-33)-pardaxin-[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antibacterial</td><td>&lt;20% hemolysis at 50µM</td><td>Human</td><td>CCCCCCHHHHTSHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3677&amp;di=1" width="5px" height="5px"><img src="pymol_images/3677.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3833">3833</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9914515">9914515</a></td><td align="right">1999</td><td>GFFALIPKIISSPLFKTLLSAV</td><td>Par[1-22]</td><td>NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub></td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>22% hemolysis at 50μM</td><td>Human</td><td>CCCCCCHHHHTSHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3833&amp;di=1" width="5px" height="5px"><img src="pymol_images/3833.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3834">3834</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9914515">9914515</a></td><td align="right">1999</td><td>GFFALIpKIISSPLFKTllSAV</td><td>[D]-P<sup>7</sup>L<sup>18,19</sup>-Par[1-22]</td><td>NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub></td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>1% hemolysis at 50μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3795">3795</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9108731">9108731</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAVGSAL</td><td>PX1–26</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>Conc. Required for hemolysis =1.1nmols</td><td>Human</td><td>CCCCCCHHHHTSHHHHHHHHHHSSCC</td><td align="center"><a href="jmol.php?id=3795&amp;di=1" width="5px" height="5px"><img src="pymol_images/3795.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3795&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3795.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3789">3789</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9010936">9010936</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAVGSALSSSGEQE</td><td>PX</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>100% hemolysis at 14μg/ml</td><td>Human</td><td>CCCSSCCTTTCSHHHHHHHHHHHHHHSSCCSCC</td><td align="center"><a href="jmol.php?id=3789&amp;di=1" width="5px" height="5px"><img src="pymol_images/3789.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3789&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3789.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3793">3793</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9108731">9108731</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAVGSALSSSGEQE</td><td>PX1–33</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Cytolytic</td><td>Conc. Required for hemolysis =1.75nmols</td><td>Human</td><td>CCCSSCCTTTCSHHHHHHHHHHHHHHSSCCSCC</td><td align="center"><a href="jmol.php?id=3793&amp;di=1" width="5px" height="5px"><img src="pymol_images/3793.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3793&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3793.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3673">3673</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAVGSALSSSGGQE</td><td>Pardaxin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>&gt;20% hemolysis at 25µM</td><td>Human</td><td>CCCSSCHHHHTSSTTHHHHHHHHHHHTTCSSCC</td><td align="center"><a href="jmol.php?id=3673&amp;di=1" width="5px" height="5px"><img src="pymol_images/3673.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3673&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3673.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3674">3674</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAVGSALSSSGGQE</td><td>Pardaxin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>&gt;50% hemolysis at 50µM</td><td>Human</td><td>CCCSSCHHHHTSSTTHHHHHHHHHHHTTCSSCC</td><td align="center"><a href="jmol.php?id=3674&amp;di=1" width="5px" height="5px"><img src="pymol_images/3674.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3674&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3674.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3675">3675</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAVGSALSSSGGQE</td><td>Pardaxin- [NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>&gt;100% hemolysis at 12µM</td><td>Human</td><td>CCCSSCHHHHTSSTTHHHHHHHHHHHTTCSSCC</td><td align="center"><a href="jmol.php?id=3675&amp;di=1" width="5px" height="5px"><img src="pymol_images/3675.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1447">1447</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19563807">19563807</a></td><td align="right">2009</td><td>GFFGKMKEYFKKFGASFKRRFANLKKRL</td><td>Ltc5 (native)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial and Cytotoxic</td><td>EC<sub>50</sub> =12μM </td><td>Human</td><td>CCCTTGGGTTTTSTTTGGGGGGGTTTCC</td><td align="center"><a href="jmol.php?id=1447&amp;di=1" width="5px" height="5px"><img src="pymol_images/1447.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4057">4057</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16735513">16735513</a></td><td align="right">2006</td><td>GFFGKMKEYFKKFGASFKRRFANLKKRL</td><td>Ltc5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>20% hemolysis at 40μM</td><td>Rabbit</td><td>CCCTTGGGTTTTSTTTGGGGGGGTTTCC</td><td align="center"><a href="jmol.php?id=4057&amp;di=1" width="5px" height="5px"><img src="pymol_images/4057.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1448">1448</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19563807">19563807</a></td><td align="right">2009</td><td>GFFGKRKEYFKKFGASFKRRFANLKKRL</td><td>Ltc5_M6R</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial and Cytotoxic</td><td>EC<sub>50</sub> &gt;36μM </td><td>Human</td><td>CTTCCCTTTTGGGHHHHHTTTTTSCCCC</td><td align="center"><a href="jmol.php?id=1448&amp;di=1" width="5px" height="5px"><img src="pymol_images/1448.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1448&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1448.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1779">1779</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFFOicGKFOicGFFOicGKFKKKK</td><td>Peptide-35</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Oic= octahydroindolecarboxylic acid</td><td align="right">19</td><td>Antibiotic</td><td>27.30% hemolysisat 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1780">1780</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFFOicGKFOicGFFOicGKFKKKK</td><td>Peptide-35</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Oic= octahydroindolecarboxylic acid</td><td align="right">19</td><td>Antibiotic</td><td>8.9% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1777">1777</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFG-Oic-GKG-Oic-GFG-Oic-GKGKKKK</td><td>Peptide-34</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Oic= octahydroindolecarboxylic acid</td><td align="right">19</td><td>Antibiotic</td><td>3.30% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1778">1778</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFGOicGKGOicGFGOicGKGKKKK</td><td>Peptide-34</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Oic= octahydroindolecarboxylic acid</td><td align="right">19</td><td>Antibiotic</td><td>3.3% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2008">2008</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22123629">22123629</a></td><td align="right">2012</td><td>GFGSLLGKALKIGTNLL</td><td>CPF-PG1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>LC<sub>50</sub> =145µM</td><td>Human</td><td>NA</td><td align="center"><a href="jmol.php?id=2008&amp;di=1" width="5px" height="5px"><img src="pymol_images/2008.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2008&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2008.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2482">2482</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12429503">12429503</a></td><td align="right">2002</td><td>GFISTVKNLATNVAGTVIDTIKCKVTGGC</td><td>Palustrin-2AR</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antibacterial</td><td>HC<sub>50</sub> &gt;300µM</td><td>Human</td><td>CCSSCHHHHTTCCCCCCCCCSSSCCSSCC</td><td align="center"><a href="jmol.php?id=2482&amp;di=1" width="5px" height="5px"><img src="pymol_images/2482.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2482&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2482.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1949">1949</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAAKALKKTVLF</td><td>Lys-14</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>CHHHHHHTTHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=1949&amp;di=1" width="5px" height="5px"><img src="pymol_images/1949.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1949&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1949.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1954">1954</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAAKALkKTVLF</td><td>D-Lys-14</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>CHHHHHHTTHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=1954&amp;di=1" width="5px" height="5px"><img src="pymol_images/1954.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1954&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1954.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1956">1956</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAAKALVKAVLF</td><td>Ala-16</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =22µM</td><td>Human</td><td>CHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1956&amp;di=1" width="5px" height="5px"><img src="pymol_images/1956.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1956&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1956.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1950">1950</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAAKALVKTVKF</td><td>Lys-18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>CHHHHHHHHHHHHHHSCCC</td><td align="center"><a href="jmol.php?id=1950&amp;di=1" width="5px" height="5px"><img src="pymol_images/1950.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1950&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1950.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1955">1955</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAAKALVKTVkF</td><td>D-Lys-18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>CHHHHHHHHHHHHHHSCCC</td><td align="center"><a href="jmol.php?id=1955&amp;di=1" width="5px" height="5px"><img src="pymol_images/1955.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1955&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1955.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1945">1945</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAAKALVKTVLF</td><td>Ascaphin-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =55µM</td><td>Human</td><td>CHHHHHHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=1945&amp;di=1" width="5px" height="5px"><img src="pymol_images/1945.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1945&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1945.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2548">2548</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18068868">18068868</a></td><td align="right">2007</td><td>GFKDLLKGAAKALVKTVLF</td><td>Ascaphin-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =55μM</td><td>Human</td><td>CHHHHHHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=2548&amp;di=1" width="5px" height="5px"><img src="pymol_images/2548.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2548&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2548.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1948">1948</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAKKALVKTVLF</td><td>Lys-10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>CHHHHHHHHHHHHGGGTCC</td><td align="center"><a href="jmol.php?id=1948&amp;di=1" width="5px" height="5px"><img src="pymol_images/1948.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1953">1953</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAkKALVKTVLF</td><td>D-Lys-10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1947">1947</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKKAAKALVKTVLF</td><td>Lys-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =11µM</td><td>Human</td><td>CHHHHHHHHHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=1947&amp;di=1" width="5px" height="5px"><img src="pymol_images/1947.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1947&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1947.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1952">1952</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKkAAKALVKTVLF</td><td>D-Lys-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =150µM</td><td>Human</td><td>CHHHHHHHHHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=1952&amp;di=1" width="5px" height="5px"><img src="pymol_images/1952.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1952&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1952.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2550">2550</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18068868">18068868</a></td><td align="right">2007</td><td>GFKDLLKKAAKALVKTVLF</td><td>Lys<sup>8</sup>-Ascaphin-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =11μM</td><td>Human</td><td>CHHHHHHHHHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=2550&amp;di=1" width="5px" height="5px"><img src="pymol_images/2550.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2550&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2550.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1946">1946</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKKLLKGAAKALVKTVLF</td><td>Lys-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =24µM</td><td>Human</td><td>CHHHHHHHHHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=1946&amp;di=1" width="5px" height="5px"><img src="pymol_images/1946.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1951">1951</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKkLLKGAAKALVKTVLF</td><td>D-Lys-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =300µM</td><td>Human</td><td>CHHHHHHHHHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=1951&amp;di=1" width="5px" height="5px"><img src="pymol_images/1951.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1951&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1951.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2549">2549</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18068868">18068868</a></td><td align="right">2007</td><td>GFKKLLKGAAKALVKTVLF</td><td>Lys<sup>4</sup>-Ascaphin-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =24μM</td><td>Human</td><td>CHHHHHHHHHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=2549&amp;di=1" width="5px" height="5px"><img src="pymol_images/2549.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2549&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2549.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2132">2132</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11557475">11557475</a></td><td align="right">2001</td><td>GFKLKGKAKISCLPNGQWSNFPPKCIRECAMVSS</td><td>S1Δ</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antibacterial</td><td>0% hemolysis at 100µg/ml (non-hemolytic)</td><td>Human</td><td>CCCCCCCCCCCSSCCSSCCCCCGGGHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2132&amp;di=1" width="5px" height="5px"><img src="pymol_images/2132.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2132&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2132.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1539">1539</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20873868">20873868</a></td><td align="right">2010</td><td>GFKLKGMARISCLPNGQWSNFPPKCIRECAMVSS</td><td>Sushi 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antimicrobial</td><td>13% hemolysis at 100μg/ml </td><td>Rabbit</td><td>CCCCCCCCCCCCCSCSSSSCCCTTTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=1539&amp;di=1" width="5px" height="5px"><img src="pymol_images/1539.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1539&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1539.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2131">2131</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11557475">11557475</a></td><td align="right">2001</td><td>GFKLKGMARISCLPNGQWSNFPPKCIRECAMVSS</td><td>S1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antibacterial</td><td>0% hemolysis at 100µg/ml (non-hemolytic)</td><td>Human</td><td>CCCCCCCCCCCCCSCSSSSCCCTTTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2131&amp;di=1" width="5px" height="5px"><img src="pymol_images/2131.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2131&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2131.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2935">2935</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GFLGILFHGVHHGRKKALHMNSERRS</td><td>NRC-20</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCCSSCCBSSSSSBSSCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2935&amp;di=1" width="5px" height="5px"><img src="pymol_images/2935.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2935&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2935.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3476">3476</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11738090">11738090</a></td><td align="right">2001</td><td>GFLGPLLKLAAKGVAKVIPHLIPSRQQ</td><td>XT-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>HC<sub>50</sub> =90μM</td><td>Human</td><td>CCTTTTGGGGTCCCTTTSCCSSCCCCC</td><td align="center"><a href="jmol.php?id=3476&amp;di=1" width="5px" height="5px"><img src="pymol_images/3476.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3476&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3476.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3158">3158</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GFLGPLLKLGLKGAAKLLPQLLPSRQQ</td><td>CPF-SE2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>LC<sub>50</sub> =50μM</td><td>Human</td><td>CCCGGGGGGCTTGGGGTHHHHSSSCCC</td><td align="center"><a href="jmol.php?id=3158&amp;di=1" width="5px" height="5px"><img src="pymol_images/3158.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3157">3157</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GFLGPLLKLGLKGVAKVIPHLIPSRQQ</td><td>CPF-SE1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>LC<sub>50</sub> =50μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3159">3159</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GFLGSLLKTGLKVGSNLL</td><td>CPF-SE3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =220μM</td><td>Human</td><td>CCCHHHHTTTTTCSSCCC</td><td align="center"><a href="jmol.php?id=3159&amp;di=1" width="5px" height="5px"><img src="pymol_images/3159.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3159&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3159.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3479">3479</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11738090">11738090</a></td><td align="right">2001</td><td>GFLGSLLKTGLKVGSNLL</td><td>XT-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;150μM</td><td>Human</td><td>CCCHHHHTTTTTCSSCCC</td><td align="center"><a href="jmol.php?id=3479&amp;di=1" width="5px" height="5px"><img src="pymol_images/3479.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3479&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3479.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3495">3495</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19109988">19109988</a></td><td align="right">2010</td><td>GFLSALKKYLPIVLKHV</td><td>Osmin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antibacterial and Antifungal</td><td>Weak hemolytic activity <i>i.e.</i> 5.0HU/mg</td><td>Human</td><td>CTHHHHTTTTTSSCCCC</td><td align="center"><a href="jmol.php?id=3495&amp;di=1" width="5px" height="5px"><img src="pymol_images/3495.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3495&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3495.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3496">3496</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19109988">19109988</a></td><td align="right">2010</td><td>GFLSALKKYLPIVLKHV</td><td>Osmin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antibacterial and Antifungal</td><td>Hemolytic activity <i>i.e.</i> 16.6HU/mg</td><td>Human</td><td>CTHHHHTTTTTSSCCCC</td><td align="center"><a href="jmol.php?id=3496&amp;di=1" width="5px" height="5px"><img src="pymol_images/3496.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3496&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3496.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3631">3631</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVL</td><td>MEP-10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100µM</td><td>Rat</td><td>CCHHHHTTTC</td><td align="center"><a href="jmol.php?id=3631&amp;di=1" width="5px" height="5px"><img src="pymol_images/3631.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3631&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3631.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3624">3624</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVLAKVMAHMK</td><td>MEP-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =29.7µM</td><td>Rat</td><td>CCHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3624&amp;di=1" width="5px" height="5px"><img src="pymol_images/3624.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3624&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3624.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3625">3625</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVLGKVMAHMK</td><td>MEP-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =41.4µM</td><td>Rat</td><td>CCHHHHHHHHHHHHTSCC</td><td align="center"><a href="jmol.php?id=3625&amp;di=1" width="5px" height="5px"><img src="pymol_images/3625.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3625&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3625.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3622">3622</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVLKKVMAHMK</td><td>MEP-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =27.3µM</td><td>Rat</td><td>CCHHHHTTTHHHHHSSCC</td><td align="center"><a href="jmol.php?id=3622&amp;di=1" width="5px" height="5px"><img src="pymol_images/3622.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3622&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3622.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3623">3623</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVLKVMAHMK</td><td>MEP-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>LC<sub>50</sub> =22.9µM</td><td>Rat</td><td>CCHHHHHHHHHHHSSCC</td><td align="center"><a href="jmol.php?id=3623&amp;di=1" width="5px" height="5px"><img src="pymol_images/3623.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3623&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3623.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3630">3630</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVLP</td><td>MEP-9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100µM</td><td>Rat</td><td>CCHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3630&amp;di=1" width="5px" height="5px"><img src="pymol_images/3630.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3630&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3630.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2748">2748</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSILKKVLPKSXAHSK</td><td>MEP-Ns-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =18.1μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1508">1508</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20499256">20499256</a></td><td align="right">2010</td><td>GFLSILKKVLPKV-Met(O)-AH-Met(O)-K</td><td>[Met (O)<sup>14,17</sup>] MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Met (O)= methionine sulfoxide</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200μM </td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1509">1509</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20499256">20499256</a></td><td align="right">2010</td><td>GFLSILKKVLPKV-Met(O<sub>2</sub>)-AH-Met(O<sub>2</sub>)-K</td><td>[Met (O2)<sup>14,17</sup>] MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Met (O<sub>2</sub>)=methionine sulfone</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200μM </td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1511">1511</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20499256">20499256</a></td><td align="right">2010</td><td>GFLSILKKVLPKV-Met-AH-Nle-K</td><td>[Nle<sup>17</sup>] MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nle= norleucine</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =80μM </td><td>Rat</td><td>NA</td><td align="center"><a href="jmol.php?id=1511&amp;di=1" width="5px" height="5px"><img src="pymol_images/1511.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1512">1512</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20499256">20499256</a></td><td align="right">2010</td><td>GFLSILKKVLPKV-Nle-AH-Met(O)-K</td><td>[Nle<sup>14</sup>, Met(O)<sup>17</sup>] MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nle= norleucine, Met (O)= methionine sulfoxide</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200μM </td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1513">1513</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20499256">20499256</a></td><td align="right">2010</td><td>GFLSILKKVLPKV-Nle-AH-Met-K</td><td>[Nle<sup>14</sup>] MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nle= norleucine</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =56.3μM </td><td>Rat</td><td>NA</td><td align="center"><a href="jmol.php?id=1513&amp;di=1" width="5px" height="5px"><img src="pymol_images/1513.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1510">1510</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20499256">20499256</a></td><td align="right">2010</td><td>GFLSILKKVLPKV-Nle-AH-Nle-K</td><td>[Nle<sup>14,17</sup>] MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nle= norleucine</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =50μM </td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1507">1507</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20499256">20499256</a></td><td align="right">2010</td><td>GFLSILKKVLPKVMAHMK</td><td>MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100μM </td><td>Rat</td><td>CCHHHHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1507&amp;di=1" width="5px" height="5px"><img src="pymol_images/1507.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1507&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1507.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2746">2746</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSILKKVLPKVMAHMK</td><td>MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100μM</td><td>Human</td><td>CCHHHHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=2746&amp;di=1" width="5px" height="5px"><img src="pymol_images/2746.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2746&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2746.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3621">3621</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVLPKVMAHMK</td><td>MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100µM</td><td>Rat</td><td>CCHHHHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=3621&amp;di=1" width="5px" height="5px"><img src="pymol_images/3621.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3621&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3621.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2747">2747</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSILKKVLPKVXAHXK</td><td>MEP-N</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =50μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1514">1514</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20499256">20499256</a></td><td align="right">2010</td><td>GFLSIVKKVLPKV-Nle-AH-Met(O)-K</td><td>[Val<sup>6</sup>, Nle<sup>14</sup>, Met(O)<sup>17</sup>] MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nle= norleucine, Met (O)= methionine sulfoxide</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200μM </td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1515">1515</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20499256">20499256</a></td><td align="right">2010</td><td>GFLSIVKKVLPKV-Nle-AH-Met-K</td><td>[Val<sup>6</sup>, Nle<sup>14</sup>] MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nle= norleucine</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200μM </td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2754">2754</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSSLKKSLAKSXAHSK</td><td>MEP-Ns-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =39.5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2753">2753</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSSLKKSLGKSXAHSK</td><td>MEP-Ns-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =29μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2750">2750</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSSLKKSLPKSXAHSK</td><td>MEP-Ns-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =13.9μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2749">2749</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSSLKKSLPKVXAHXK</td><td>MEP-Ns-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =14.7μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3646">3646</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17272268">17272268</a></td><td align="right">2007</td><td>GFMDTAKNVAKNVAVTLLDNLKCKITKAC</td><td>Odorranain-F1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>15.4±3.5% hemolysis at 50µg/ml</td><td>Human</td><td>CCSCHHHHHHTTCSBTTBCSCCCTTCSCC</td><td align="center"><a href="jmol.php?id=3646&amp;di=1" width="5px" height="5px"><img src="pymol_images/3646.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3646&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3646.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4075">4075</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIGPLIPHAVKAIKKLI</td><td>Kassinatuerin-1 (Lys<sup>18</sup>, Lys<sup>19</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =25μM</td><td>Human</td><td>CCCCCCCTTSHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=4075&amp;di=1" width="5px" height="5px"><img src="pymol_images/4075.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4075&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4075.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3489">3489</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19427345">19427345</a></td><td align="right">2009</td><td>GFMKYIGPLIPHAVKAISDLI</td><td>Kassinatuerin-1S</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>45-50% hemolysis at 120μM</td><td>Horse</td><td>CCCCCCSSSSTTTGGGTCCCC</td><td align="center"><a href="jmol.php?id=3489&amp;di=1" width="5px" height="5px"><img src="pymol_images/3489.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3489&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3489.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4071">4071</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIGPLIPHAVKAISDLI</td><td>Kassinatuerin-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =65μM</td><td>Human</td><td>CCCCCCSSSSTTTGGGTCCCC</td><td align="center"><a href="jmol.php?id=4071&amp;di=1" width="5px" height="5px"><img src="pymol_images/4071.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4071&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4071.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4078">4078</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIGPLIPHAVKAISDLI</td><td>Kassinatuerin-1 (COOH)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>39% hemolysis at 400μM</td><td>Human</td><td>CCCCCCSSSSTTTGGGTCCCC</td><td align="center"><a href="jmol.php?id=4078&amp;di=1" width="5px" height="5px"><img src="pymol_images/4078.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4078&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4078.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4074">4074</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIGPLIPHAVKAISKLI</td><td>Kassinatuerin-1 (Lys<sup>19</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =45μM</td><td>Human</td><td>CCCCCCSSCCHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=4074&amp;di=1" width="5px" height="5px"><img src="pymol_images/4074.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4074&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4074.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4079">4079</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIGPLIPHAVKAISKLI</td><td>Kassinatuerin-1 (Lys<sup>19</sup>)(COOH)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =120μM</td><td>Human</td><td>CCCCCCSSCCHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=4079&amp;di=1" width="5px" height="5px"><img src="pymol_images/4079.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4079&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4079.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4073">4073</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIGPLIPHKVKAISDLI</td><td>Kassinatuerin-1 (Lys<sup>13</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>32% hemolysis at 400μM</td><td>Human</td><td>CTTBCSSSBCSSCTTSSCCCC</td><td align="center"><a href="jmol.php?id=4073&amp;di=1" width="5px" height="5px"><img src="pymol_images/4073.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4073&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4073.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4077">4077</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIKPLIPHAVKAIKKLI</td><td>Kassinatuerin-1 (Lys<sup>7</sup>, Lys<sup>18</sup>, Lys<sup>19</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =40μM</td><td>Human</td><td>CCCCCCCCCCHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=4077&amp;di=1" width="5px" height="5px"><img src="pymol_images/4077.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4077&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4077.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4081">4081</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIKPLIPHAVKAIKKLI</td><td>Kassinatuerin-1 (Lys<sup>7</sup>, Lys<sup>18</sup>, Lys<sup>19</sup>) (COOH)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =200μM</td><td>Human</td><td>CCCCCCCCCCHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=4081&amp;di=1" width="5px" height="5px"><img src="pymol_images/4081.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4081&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4081.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4076">4076</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIKPLIPHAVKAISKLI</td><td>Kassinatuerin-1 (Lys<sup>7</sup>, Lys<sup>19</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =55μM</td><td>Human</td><td>CCCCCCCCCCTTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=4076&amp;di=1" width="5px" height="5px"><img src="pymol_images/4076.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4076&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4076.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4080">4080</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIKPLIPHAVKAISKLI</td><td>Kassinatuerin-1 (Lys<sup>7</sup>, Lys<sup>19</sup>)(COOH)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =135μM</td><td>Human</td><td>CCCCCCCCCCTTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=4080&amp;di=1" width="5px" height="5px"><img src="pymol_images/4080.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4080&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4080.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1771">1771</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFOicGKOicGFOicGKKKKK</td><td>Peptide-31</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Oic= octahydroindolecarboxylic acid</td><td align="right">15</td><td>Antibiotic</td><td>5.9% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1772">1772</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFOicGKOicGFOicGKKKKK</td><td>Peptide-31</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Oic= octahydroindolecarboxylic acid</td><td align="right">15</td><td>Antibiotic</td><td>3.2% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2751">2751</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFRSILKKVSPKVXAHXK</td><td>MEP-Ns-4 cis</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =11μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2752">2752</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFRSILKKVSPKVXAHXK</td><td>MEP-Ns-4 trans</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =20μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1775">1775</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFTicGGKTicGGFTicGGKTicKKKK</td><td>Peptide-33</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid</td><td align="right">19</td><td>Antibiotic</td><td>5.70% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1776">1776</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFTicGGKTicGGFTicGGKTicKKKK</td><td>Peptide-33</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid</td><td align="right">19</td><td>Antibiotic</td><td>3.80% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1773">1773</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFTicGKTicGFTicGKTicKKKK</td><td>Peptide-32</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid</td><td align="right">16</td><td>Antibiotic</td><td>6% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1774">1774</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFTicGKTicGFTicGKTicKKKK</td><td>Peptide-32</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid</td><td align="right">16</td><td>Antibiotic</td><td>4.9% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1759">1759</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFTicOicGKTicOicGFTicOicGKTicKKKK</td><td>Peptide-23</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic- tetrahydroisoquinolinecarboxylic acid, Oic- octahydroindolecarboxylic acid</td><td align="right">19</td><td>Antibiotic</td><td>14% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1760">1760</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFTicOicGKTicOicGFTicOicGKTicKKKK</td><td>Peptide-23</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic- tetrahydroisoquinolinecarboxylic acid, Oic- octahydroindolecarboxylic acid</td><td align="right">19</td><td>Antibiotic</td><td>14% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1761">1761</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFTicOicGKTicOicGFTicOicGKTicKKKK</td><td>Peptide-26</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Tic- tetrahydroisoquinolinecarboxylic acid, Oic- octahydroindolecarboxylic acid</td><td align="right">19</td><td>Antibiotic</td><td>33.4% hemolysis at 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1762">1762</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFTicOicGKTicOicGFTicOicGKTicKKKK</td><td>Peptide-26</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Tic- tetrahydroisoquinolinecarboxylic acid, Oic- octahydroindolecarboxylic acid</td><td align="right">19</td><td>Antibiotic</td><td>14.3% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1787">1787</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFTicOicGKTicOicGFTicOicGKTicKKKKK</td><td>Peptide-39</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid</td><td align="right">20</td><td>Antibiotic</td><td>24.9% hemolysisat 100µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1788">1788</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17547385">17547385</a></td><td align="right">2007</td><td>GFTicOicGKTicOicGFTicOicGKTicKKKKK</td><td>Peptide-39</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Tic= tetrahydroisoquinolinecarboxylic acid, Oic= octahydroindolecarboxylic acid</td><td align="right">20</td><td>Antibiotic</td><td>18.9% hemolysis at 25µM</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2930">2930</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GFWGKLFKLGLHGIGLLHLHL</td><td>NRC-15</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Anticancer</td><td>50% hemolytic at 64μg/ml</td><td>Human</td><td>CCTHHHHTTSCCCTTCCCSCC</td><td align="center"><a href="jmol.php?id=2930&amp;di=1" width="5px" height="5px"><img src="pymol_images/2930.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2930&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2930.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3620">3620</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19085906">19085906</a></td><td align="right">2008</td><td>GFWKKVGSAAWGGVKAAAKGAAVGGLNALAKHIQ</td><td>Papillosin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antimicrobial</td><td>0% hemolysis upto 50µM (Non hemolytic)</td><td>Sheep</td><td>CCSSSSSSTTCSTHHHHHHSTTCCTHHHHGGGTC</td><td align="center"><a href="jmol.php?id=3620&amp;di=1" width="5px" height="5px"><img src="pymol_images/3620.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3620&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3620.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3155">3155</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GFWTTAAEGLKKFAKAGLASILNPK</td><td>XPF-SE3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>LC<sub>50</sub> =105μM</td><td>Human</td><td>CCCCHHHHHHTTGGGCSTTTSSSCC</td><td align="center"><a href="jmol.php?id=3155&amp;di=1" width="5px" height="5px"><img src="pymol_images/3155.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3155&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3155.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3991">3991</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8910461">8910461</a></td><td align="right">1996</td><td>GGLRSLGRKILRAWKKYGPIIVPIIRIG</td><td>BMAP-28</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>0% hemolysis at 100μM</td><td>Human</td><td>CCCCSHHHHTTTTGGGTSCSCCCCCCCC</td><td align="center"><a href="jmol.php?id=3991&amp;di=1" width="5px" height="5px"><img src="pymol_images/3991.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3991&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3991.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1027">1027</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18081258">18081258</a></td><td align="right">2008</td><td>GGTIFDCGETCFLGTCYTPGCSCGNYGFCYGTN</td><td>Cycloviolacin Y1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">33</td><td>Anti-HIV</td><td>Poor hemolytic</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3585">3585</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9335545">9335545</a></td><td align="right">1997</td><td>GIAKFGKAAAHFGKKWVGELMNS</td><td>100°M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>EC<sub>25</sub>=159μM</td><td>Human</td><td>CCGGGCCTHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3585&amp;di=1" width="5px" height="5px"><img src="pymol_images/3585.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3649">3649</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17272268">17272268</a></td><td align="right">2007</td><td>GIFGKILGVGKKVLCGLSGVC</td><td>Odorranain-H1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>21.6±7.3% hemolysis at 50µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2706">2706</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAILKVLATGLPTLISWI</td><td>FL20</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>CPP</td><td>&lt;60% hemolytic at 25μM at pH 5 </td><td>Human</td><td>CCTHHHHGGGSCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2706&amp;di=1" width="5px" height="5px"><img src="pymol_images/2706.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2706&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2706.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2707">2707</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAILKVLATGLPTLISWI</td><td>FL20</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>CPP</td><td>~80% hemolytic at 25μM at pH 8</td><td>Human</td><td>CCTHHHHGGGSCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2707&amp;di=1" width="5px" height="5px"><img src="pymol_images/2707.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2707&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2707.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2704">2704</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAILKVLATGLPTLISWIKNKRKQ</td><td>Florae</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>CPP</td><td>&lt;70% hemolytic at 25μM at pH 6 </td><td>Human</td><td>CCTTSHHHHTTTSHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2704&amp;di=1" width="5px" height="5px"><img src="pymol_images/2704.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2704&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2704.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2705">2705</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAILKVLATGLPTLISWIKNKRKQ</td><td>Florae</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>CPP</td><td>~80% hemolytic at 25μM at pH 8</td><td>Human</td><td>CCTTSHHHHTTTSHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2705&amp;di=1" width="5px" height="5px"><img src="pymol_images/2705.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2705&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2705.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1723">1723</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17065623">17065623</a></td><td align="right">2006</td><td>GIGAVLIVLTTGLPALISWIKRKRQQ</td><td>Mel.subK7I</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antifungal</td><td>30.2±2.8% hemolysis at 100µM</td><td>Human</td><td>CCCCCSSHHHHCSTHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=1723&amp;di=1" width="5px" height="5px"><img src="pymol_images/1723.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1723&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1723.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2476">2476</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12168695">12168695</a></td><td align="right">2002</td><td>GIGAVLK*VLTTGLpALISWIK*RK*RQQ</td><td>Melittin-A1</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>K*: Acetylated Lysin</td><td align="right">26</td><td>Antibacterial</td><td>50% hemolysis at &gt;200µg/ml</td><td>Human</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2476&amp;di=1" width="5px" height="5px"><img src="pymol_images/2476.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2476&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2476.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2475">2475</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12168695">12168695</a></td><td align="right">2002</td><td>GIGAVLK*VLTTGLpALISWIK*XK*RQQ</td><td>Melittin-A1P</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>X: Pcm arg; K*: Acetylated Lysin</td><td align="right">26</td><td>Antibacterial</td><td>50% hemolysis at &gt;200µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1460">1460</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19635451">19635451</a></td><td align="right">2010</td><td>GIGAVLKVLALISWIKRKR</td><td>Mel-H</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>28% hemolysis at 0.4μM </td><td>Human</td><td>CHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=1460&amp;di=1" width="5px" height="5px"><img src="pymol_images/1460.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1460&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1460.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1461">1461</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19635451">19635451</a></td><td align="right">2010</td><td>GIGAVLKVLALISWIKRKR</td><td>Mel-H</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>42% hemolysis at 23μM </td><td>Human</td><td>CHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=1461&amp;di=1" width="5px" height="5px"><img src="pymol_images/1461.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1461&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1461.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1462">1462</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19635451">19635451</a></td><td align="right">2010</td><td>GIGAvLKvLAlISWIkRKR</td><td>Mel-<sup>d</sup>H</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>5% hemolysis at 0.4μM </td><td>Human</td><td>CHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=1462&amp;di=1" width="5px" height="5px"><img src="pymol_images/1462.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1462&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1462.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1463">1463</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19635451">19635451</a></td><td align="right">2010</td><td>GIGAvLKvLAlISWIkRKR</td><td>Mel-<sup>d</sup>H</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>12% hemolysis at 23μM </td><td>Human</td><td>CHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=1463&amp;di=1" width="5px" height="5px"><img src="pymol_images/1463.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1463&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1463.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3071">3071</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20472009">20472009</a></td><td align="right">2010</td><td>GIGAVLKVLSTGLPALISWIKRKRQQ</td><td>melittin-S</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>35% hemolysis at 3μM</td><td>Human</td><td>CCSCCCCTTTSCCHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3071&amp;di=1" width="5px" height="5px"><img src="pymol_images/3071.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3071&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3071.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3563">3563</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2689223">2689223</a></td><td align="right">1989</td><td>GIGAVLKVLTTGLKWKLFKKIEKVGQ</td><td>M(1-13)CA(1-13)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antibacterial and Antiparasitic</td><td>Lethal concentration calculated from zone of inhibition80μM</td><td>Sheep</td><td>CGGGSHHHHHHHCCHHHHTTTTTTCC</td><td align="center"><a href="jmol.php?id=3563&amp;di=1" width="5px" height="5px"><img src="pymol_images/3563.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3563&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3563.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2702">2702</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAVLKVLTTGLPALISWI</td><td>MT20</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>CPP</td><td>&lt;80% hemolytic at 25μM at pH 8</td><td>Human</td><td>CCSBGGGCSSCCCSSSSSBC</td><td align="center"><a href="jmol.php?id=2702&amp;di=1" width="5px" height="5px"><img src="pymol_images/2702.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2702&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2702.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2703">2703</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAVLKVLTTGLPALISWI</td><td>MT20</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>CPP</td><td>40% hemolytic at 25μM at pH 5</td><td>Human</td><td>CCSBGGGCSSCCCSSSSSBC</td><td align="center"><a href="jmol.php?id=2703&amp;di=1" width="5px" height="5px"><img src="pymol_images/2703.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2703&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2703.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2473">2473</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12168695">12168695</a></td><td align="right">2002</td><td>GIGAVLKVLTTGLpALISWIKFKRQQ</td><td>Melittin-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Antibacterial</td><td>50% hemolysis at &gt;80µg/ml</td><td>Human</td><td>CCCCCSSCTTCCCHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2473&amp;di=1" width="5px" height="5px"><img src="pymol_images/2473.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2473&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2473.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1076">1076</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17114219">17114219</a></td><td align="right">2007</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Hemolytic</td><td>100% hemolysis at 2μM</td><td>Rat</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=1076&amp;di=1" width="5px" height="5px"><img src="pymol_images/1076.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1076&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1076.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1692">1692</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16848415">16848415</a></td><td align="right">2006</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>M0</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Hemolytic</td><td>HL<sub>50</sub> = 1.5μM at pH 7.4</td><td>Mouse</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=1692&amp;di=1" width="5px" height="5px"><img src="pymol_images/1692.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1692&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1692.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1693">1693</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16848415">16848415</a></td><td align="right">2006</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>M0</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Hemolytic</td><td>HL<sub>50</sub> = 6μM at pH 4.5</td><td>Mouse</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=1693&amp;di=1" width="5px" height="5px"><img src="pymol_images/1693.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1693&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1693.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1722">1722</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17065623">17065623</a></td><td align="right">2006</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antifungal</td><td>115.7±2.5% hemolysis at 100µM</td><td>Human</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=1722&amp;di=1" width="5px" height="5px"><img src="pymol_images/1722.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1722&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1722.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2354">2354</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antibacterial</td><td>HC<sub>10</sub> =1.78μg/ml</td><td>Human</td><td>CCCSSHHHHHHSCHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2354&amp;di=1" width="5px" height="5px"><img src="pymol_images/2354.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2354&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2354.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2441">2441</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>94.6% hemolysis at 80µg/ml</td><td>Human</td><td>CCCSSHHHHHHSCHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2441&amp;di=1" width="5px" height="5px"><img src="pymol_images/2441.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2441&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2441.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2472">2472</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12168695">12168695</a></td><td align="right">2002</td><td>GIGAVLKVLTTGLpALISWIKRKRQQ</td><td>Melittin-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Antibacterial</td><td>50% hemolysis at &gt;200µg/ml</td><td>Human</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2472&amp;di=1" width="5px" height="5px"><img src="pymol_images/2472.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2472&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2472.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2700">2700</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>CPP</td><td>&gt;80% hemolytic at 25μM at pH 8</td><td>Human</td><td>CCCSSHHHHHHSCHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2700&amp;di=1" width="5px" height="5px"><img src="pymol_images/2700.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2700&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2700.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2701">2701</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>CPP</td><td>&lt;60% hemolytic at 25μM at pH 6</td><td>Human</td><td>CCCSSHHHHHHSCHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2701&amp;di=1" width="5px" height="5px"><img src="pymol_images/2701.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2701&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2701.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2770">2770</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19635451">19635451</a></td><td align="right">2010</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>100% hemolytic at 0.4μM &amp; 23μM</td><td>Human</td><td>CCCSSHHHHHHSCHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2770&amp;di=1" width="5px" height="5px"><img src="pymol_images/2770.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2770&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2770.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3294">3294</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8027544">8027544</a></td><td align="right">1994</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>100% hemolysis at 120μM</td><td>Mouse</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3294&amp;di=1" width="5px" height="5px"><img src="pymol_images/3294.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3294&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3294.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3295">3295</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8027544">8027544</a></td><td align="right">1994</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Ac-Melittin</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>100% hemolysis at 120μM compared with melittin</td><td>Mouse</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3295&amp;di=1" width="5px" height="5px"><img src="pymol_images/3295.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3295&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3295.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3296">3296</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8027544">8027544</a></td><td align="right">1994</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin-Lac</td><td>Lac=p-amidophenyl-β-D-lactopyranoside</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>120% hemolysis at 120μM compared with melittin</td><td>Mouse</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3296&amp;di=1" width="5px" height="5px"><img src="pymol_images/3296.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3296&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3296.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3297">3297</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8027544">8027544</a></td><td align="right">1994</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>LacSuc-Melittin</td><td>Amidation</td><td>Lac=p-amidophenyl-β-D-lactopyranoside and Suc=Succinyl</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>43% hemolysis at 120μM compared with melittin</td><td>Mouse</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3297&amp;di=1" width="5px" height="5px"><img src="pymol_images/3297.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3297&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3297.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3581">3581</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9048567">9048567</a></td><td align="right">1997</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antibacterial</td><td>100% hemolysis at 50μM</td><td>Human</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3581&amp;di=1" width="5px" height="5px"><img src="pymol_images/3581.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3581&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3581.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3582">3582</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9048567">9048567</a></td><td align="right">1997</td><td>GIGAvLKvLTTGLPALiSWIkRKRQQ</td><td>[D]-V<sup>5,8</sup>,I<sup>17</sup>,K<sup>21</sup>-melittin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Antibacterial</td><td>0% hemolysis at 50μM</td><td>Human</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3582&amp;di=1" width="5px" height="5px"><img src="pymol_images/3582.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3582&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3582.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3583">3583</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9048567">9048567</a></td><td align="right">1997</td><td>GIGAvLKvLTTGLPALiSWIkRKRQQ</td><td>[D]-V<sup>5,8</sup>,I<sup>17</sup>,K<sup>21</sup>-melittin</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Antibacterial</td><td>~1-2% hemolysis at 50μM</td><td>Human</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3583&amp;di=1" width="5px" height="5px"><img src="pymol_images/3583.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3583&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3583.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2474">2474</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12168695">12168695</a></td><td align="right">2002</td><td>GIGAVLKVLTTGLpALISWIKXXKRQQ</td><td>Melittin-1P</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>X: Pcm arg</td><td align="right">27</td><td>Antibacterial</td><td>50% hemolysis at &gt;18µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3525">3525</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9166783">9166783</a></td><td align="right">1997</td><td>GIGKFIHAAKKFGKLFIGEIMNS</td><td>I<sup>6</sup>A<sup>8</sup>L<sup>15</sup>I<sup>17</sup>-M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>EC<sub>50</sub> =32μM</td><td>Human</td><td>CCCCTHHHHTTTSSCCCSCGGGC</td><td align="center"><a href="jmol.php?id=3525&amp;di=1" width="5px" height="5px"><img src="pymol_images/3525.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3525&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3525.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3526">3526</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9166783">9166783</a></td><td align="right">1997</td><td>GIGKFIHAAKKFGKLFIGEIMNS</td><td>I<sup>6</sup>A<sup>8</sup>L<sup>15</sup>I<sup>17</sup>-M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>98% hemolysis at 150μM</td><td>Human</td><td>CCCCTHHHHTTTSSCCCSCGGGC</td><td align="center"><a href="jmol.php?id=3526&amp;di=1" width="5px" height="5px"><img src="pymol_images/3526.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3526&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3526.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1270">1270</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19104020">19104020</a></td><td align="right">2009</td><td>GIGKFIHAVKKWGKTFIGEIAKS</td><td>MK5E</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>EC<sub>25</sub> =334μM</td><td>Human</td><td>CCCCTTHHHHTTGGGTCCCGGGC</td><td align="center"><a href="jmol.php?id=1270&amp;di=1" width="5px" height="5px"><img src="pymol_images/1270.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1270&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1270.png" style="width:60px;height:60px;border:1;"></a></td></tr>'
test1 = '<tr class="even"><td align="right"><a href="display_sub.php?details=1841">1841</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>KKLFKKILKKL</td><td>BP125</td><td>Amidation</td><td>Tosylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>2±0.7% hemolysis at 50µM</td><td>Human</td><td>CCSTTTSTTTC</td><td align="center"><a href="jmol.php?id=1841&amp;di=1" width="5px" height="5px"><img src="pymol_images/1841.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1842">1842</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>KKLFKKILKKL</td><td>BP126</td><td>Amidation</td><td>Benzoylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>2±0.4% hemolysis at 50µM</td><td>Human</td><td>CCSTTTSTTTC</td><td align="center"><a href="jmol.php?id=1842&amp;di=1" width="5px" height="5px"><img src="pymol_images/1842.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1843">1843</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>KKLFKKILKFL</td><td>BP76</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>34±2.1% hemolysis at 150µM</td><td>Human</td><td>CTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=1843&amp;di=1" width="5px" height="5px"><img src="pymol_images/1843.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1843&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1843.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1844">1844</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>KKLFKKILKVL</td><td>BP31</td><td>Amidation</td><td>Benzoylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>71±2.3% hemolysis at 150µM</td><td>Human</td><td>CTTHHHHTTTC</td><td align="center"><a href="jmol.php?id=1844&amp;di=1" width="5px" height="5px"><img src="pymol_images/1844.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1845">1845</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>KKLFKKILKWL</td><td>BP52</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>84±6.5% hemolysis at 150µM</td><td>Human</td><td>CTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=1845&amp;di=1" width="5px" height="5px"><img src="pymol_images/1845.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1845&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1845.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1846">1846</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>FKLFKKILKFL</td><td>BP66</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>63±5.9% hemolysis at 150µM</td><td>Human</td><td>CHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=1846&amp;di=1" width="5px" height="5px"><img src="pymol_images/1846.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1846&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1846.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1847">1847</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>KKLFKKILKFL</td><td>BP77</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>40±3.8% hemolysis at 150µM</td><td>Human</td><td>CTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=1847&amp;di=1" width="5px" height="5px"><img src="pymol_images/1847.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1847&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1847.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1848">1848</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>KKLFKKILKFL</td><td>BP78</td><td>Amidation</td><td>Tosylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>85±7.4% hemolysis at 150µM</td><td>Human</td><td>CTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=1848&amp;di=1" width="5px" height="5px"><img src="pymol_images/1848.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1849">1849</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>LKLFKKILKFL</td><td>BP81</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>65±1.5% hemolysis at 150µM</td><td>Human</td><td>CTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=1849&amp;di=1" width="5px" height="5px"><img src="pymol_images/1849.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1849&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1849.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1850">1850</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>KKLFKKILKYL</td><td>BP100</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>22±2.8% hemolysis at 150µM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=1850&amp;di=1" width="5px" height="5px"><img src="pymol_images/1850.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1850&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1850.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1851">1851</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>KKLFKKILKYL</td><td>BP102</td><td>Amidation</td><td>Tosylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>79±6.5% hemolysis at 150µM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=1851&amp;di=1" width="5px" height="5px"><img src="pymol_images/1851.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1852">1852</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>KKLFKKILKYL</td><td>BP103</td><td>Amidation</td><td>Benzoylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>83±13.5% hemolysis at 150µM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=1852&amp;di=1" width="5px" height="5px"><img src="pymol_images/1852.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1853">1853</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>LKLFKKILKYL</td><td>BP105</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>91±6.2% hemolysis at 150µM</td><td>Human</td><td>CTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=1853&amp;di=1" width="5px" height="5px"><img src="pymol_images/1853.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1853&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1853.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1854">1854</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>YKLFKKILKKL</td><td>BP121</td><td>Amidation</td><td>Tosylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>81±3.7% hemolysis at 150µM</td><td>Human</td><td>CTTTTTSGGGC</td><td align="center"><a href="jmol.php?id=1854&amp;di=1" width="5px" height="5px"><img src="pymol_images/1854.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1855">1855</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>KKLFKKILKKL</td><td>BP125</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>8±1.6% hemolysis at 150µM</td><td>Human</td><td>CCSTTTSTTTC</td><td align="center"><a href="jmol.php?id=1855&amp;di=1" width="5px" height="5px"><img src="pymol_images/1855.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1855&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1855.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1856">1856</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17980935">17980935</a></td><td align="right">2007</td><td>KKLFKKILKKL</td><td>BP126</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>14±2.9% hemolysis at 150µM</td><td>Human</td><td>CCSTTTSTTTC</td><td align="center"><a href="jmol.php?id=1856&amp;di=1" width="5px" height="5px"><img src="pymol_images/1856.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1856&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1856.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1857">1857</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17981364">17981364</a></td><td align="right">2007</td><td>SLLSLIRKLIT</td><td>Decoralin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>50-60% hemolysis at 10<sup>-3</sup>M</td><td>Mouse</td><td>CHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1857&amp;di=1" width="5px" height="5px"><img src="pymol_images/1857.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1857&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1857.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1858">1858</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17981364">17981364</a></td><td align="right">2007</td><td>SLLSLIRKLIT</td><td>Decoralin-NH<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>100% hemolysis at 10<sup>-3</sup>M</td><td>Mouse</td><td>CHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1858&amp;di=1" width="5px" height="5px"><img src="pymol_images/1858.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1858&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1858.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1859">1859</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17981364">17981364</a></td><td align="right">2007</td><td>SLLSLIRKLIT</td><td>Decoralin-NH<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Hemolytic</td><td>EC<sub>50</sub> = 80µM</td><td>Human</td><td>CHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1859&amp;di=1" width="5px" height="5px"><img src="pymol_images/1859.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1859&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1859.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1860">1860</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18052076">18052076</a></td><td align="right">2007</td><td>FFGWLIKGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>EC<sub>50</sub> = 1µM</td><td>Human</td><td>CCGGGSTTTGGGTCCCCSSCSSSCC</td><td align="center"><a href="jmol.php?id=1860&amp;di=1" width="5px" height="5px"><img src="pymol_images/1860.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1860&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1860.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1866">1866</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15003352">15003352</a></td><td align="right">2004</td><td>RLRLRIGRR</td><td>9-Fluorenylmethoxycarbonyl (F-moc)</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>0% hemolysis at 100µg/ml (non-hemolytic)</td><td>Rabbit</td><td>CCCCCCCCC</td><td align="center"><a href="jmol.php?id=1866&amp;di=1" width="5px" height="5px"><img src="pymol_images/1866.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1867">1867</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15052574">15052574</a></td><td align="right">2004</td><td>I/LLGTILGLLKGI/L</td><td>Agelaia-CP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Inflammatory</td><td>0% hemolysis at 1x10<sup>-5</sup>M (non-hemolytic)</td><td>Rat</td><td>CTHHHHHHCSCC</td><td align="center"><a href="jmol.php?id=1867&amp;di=1" width="5px" height="5px"><img src="pymol_images/1867.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1868">1868</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15055772">15055772</a></td><td align="right">2004</td><td>KWKLFKKIPKFLHLAKKF</td><td>P18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>Non-hemolytic</td><td>Human</td><td>CCCSCSCCHHHHTCSSCC</td><td align="center"><a href="jmol.php?id=1868&amp;di=1" width="5px" height="5px"><img src="pymol_images/1868.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1868&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1868.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1872">1872</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15196144">15196144</a></td><td align="right">2004</td><td>DSHEKRHHGYKRKFHEKHHSHRGY</td><td>Histatin-5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antifungal</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCSSCCSSSCCSCSSCSSSCCSCC</td><td align="center"><a href="jmol.php?id=1872&amp;di=1" width="5px" height="5px"><img src="pymol_images/1872.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1872&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1872.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1873">1873</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15196144">15196144</a></td><td align="right">2004</td><td>FKCRRWQWRMKKLG</td><td>LFB(17–30)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antifungal</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTCSCHHHHTTCC</td><td align="center"><a href="jmol.php?id=1873&amp;di=1" width="5px" height="5px"><img src="pymol_images/1873.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1873&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1873.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1874">1874</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15196144">15196144</a></td><td align="right">2004</td><td>KRLFKKLLFSLRKY</td><td>Dhvar 4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antifungal</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTHHHHHHHTSCC</td><td align="center"><a href="jmol.php?id=1874&amp;di=1" width="5px" height="5px"><img src="pymol_images/1874.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1874&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1874.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1875">1875</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15196144">15196144</a></td><td align="right">2004</td><td>KRLFRRWQWRMKKY</td><td>JH8194</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antifungal</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTHHHHGGGTCCC</td><td align="center"><a href="jmol.php?id=1875&amp;di=1" width="5px" height="5px"><img src="pymol_images/1875.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1875&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1875.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1876">1876</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15196144">15196144</a></td><td align="right">2004</td><td>KRLFRRLLFSMKKY</td><td>JH8195</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antifungal</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=1876&amp;di=1" width="5px" height="5px"><img src="pymol_images/1876.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1876&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1876.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1877">1877</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15196144">15196144</a></td><td align="right">2004</td><td>FKCKKVVISLRRY</td><td>JH8944</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antifungal</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCCCCCCCGGGC</td><td align="center"><a href="jmol.php?id=1877&amp;di=1" width="5px" height="5px"><img src="pymol_images/1877.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1877&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1877.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1883">1883</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15581688">15581688</a></td><td align="right">2005</td><td>INWKAIIEAAKQAL</td><td>MP II</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Hemolytic</td><td>No hemolysis at 10µM (non-hemolytic)</td><td>Rat</td><td>CCHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1883&amp;di=1" width="5px" height="5px"><img src="pymol_images/1883.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1883&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1883.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1884">1884</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15581688">15581688</a></td><td align="right">2005</td><td>INWLKLGKAVIDAL</td><td>MP III</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Hemolytic</td><td>No hemolysis at 10µM (non-hemolytic)</td><td>Rat</td><td>CBTTTTBTTTSCCC</td><td align="center"><a href="jmol.php?id=1884&amp;di=1" width="5px" height="5px"><img src="pymol_images/1884.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1884&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1884.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1885">1885</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15616319">15616319</a></td><td align="right">2005</td><td>GRDYRTSLTIVQKLKKMVD</td><td>G1(1-19)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>No hemolysis at 10µM (non-hemolytic)</td><td>Rabbit</td><td>CCSSSCCCHHHHHHBTTBC</td><td align="center"><a href="jmol.php?id=1885&amp;di=1" width="5px" height="5px"><img src="pymol_images/1885.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1885&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1885.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1886">1886</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15616319">15616319</a></td><td align="right">2005</td><td>RDVCRNFMRR</td><td>G11 (42-51)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>No hemolysis at 10µM (non-hemolytic)</td><td>Rabbit</td><td>CTTHHHHHTC</td><td align="center"><a href="jmol.php?id=1886&amp;di=1" width="5px" height="5px"><img src="pymol_images/1886.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1886&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1886.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1887">1887</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15616319">15616319</a></td><td align="right">2005</td><td>QRSVSNAATRVSRTGRSRWRDVSRNFMRR</td><td>G9 (23-51)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>No hemolysis at 10µM (non-hemolytic)</td><td>Rabbit</td><td>CCCCCCTTSTTSHHHHCCHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=1887&amp;di=1" width="5px" height="5px"><img src="pymol_images/1887.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1887&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1887.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1888">1888</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15616319">15616319</a></td><td align="right">2005</td><td>QRSVSNAATRVCRT</td><td>G10 (23-36)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>No hemolysis at 10µM (non-hemolytic)</td><td>Rabbit</td><td>CCCCCCGGGGCCCC</td><td align="center"><a href="jmol.php?id=1888&amp;di=1" width="5px" height="5px"><img src="pymol_images/1888.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1888&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1888.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1889">1889</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15616319">15616319</a></td><td align="right">2005</td><td>QSRVIQGLVAGETAQQICED</td><td>G21 (53-72)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>No hemolysis at 10µM (non-hemolytic)</td><td>Rabbit</td><td>CCCCCCBTTBSTHHHHTTTC</td><td align="center"><a href="jmol.php?id=1889&amp;di=1" width="5px" height="5px"><img src="pymol_images/1889.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1889&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1889.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1890">1890</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15822124">15822124</a></td><td align="right">2005</td><td>NRLARHFRDIAGRVNQRL</td><td>P1c9k</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Hemolytic</td><td>Non-hemolytic</td><td>Rat</td><td>CTTCTTTHHHHHHSSSCC</td><td align="center"><a href="jmol.php?id=1890&amp;di=1" width="5px" height="5px"><img src="pymol_images/1890.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1890&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1890.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1891">1891</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15822124">15822124</a></td><td align="right">2005</td><td>LKDVEEAQQKIINIIRRL</td><td>P1qc7</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Hemolytic</td><td>Non-hemolytic</td><td>Rat</td><td>CCTTHHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1891&amp;di=1" width="5px" height="5px"><img src="pymol_images/1891.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1891&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1891.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1892">1892</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15949628">15949628</a></td><td align="right">2005</td><td>EWGRRCCGWGPGRRYCVRWC</td><td>Ib-AMP1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>No hemolysis upto 100μM (non-hemolytic)</td><td>Rabbit</td><td>CCTTCSSCCCSCCCCCCCCC</td><td align="center"><a href="jmol.php?id=1892&amp;di=1" width="5px" height="5px"><img src="pymol_images/1892.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1892&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1892.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1893">1893</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15949628">15949628</a></td><td align="right">2005</td><td>EWGRRCCGWGPGRRYCRRWC</td><td>Ib-AMP4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>No hemolysis upto 100μM (non-hemolytic)</td><td>Rabbit</td><td>CCSCCCSSSCTTSTTTCSCC</td><td align="center"><a href="jmol.php?id=1893&amp;di=1" width="5px" height="5px"><img src="pymol_images/1893.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1894">1894</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15980334">15980334</a></td><td align="right">2005</td><td>QAKIRVRLSA</td><td>M4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>&lt;1% hemolysis at 120μg/ml</td><td>Human</td><td>CCSCSCCCCC</td><td align="center"><a href="jmol.php?id=1894&amp;di=1" width="5px" height="5px"><img src="pymol_images/1894.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1894&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1894.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1895">1895</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15980334">15980334</a></td><td align="right">2005</td><td>KIRVRLSA</td><td>M5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>&lt;1% hemolysis at 120μg/ml</td><td>Human</td><td>CCSSSSCC</td><td align="center"><a href="jmol.php?id=1895&amp;di=1" width="5px" height="5px"><img src="pymol_images/1895.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1895&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1895.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1896">1896</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15980334">15980334</a></td><td align="right">2005</td><td>QKKIRVRLSA</td><td>M6</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>&lt;1% hemolysis at 120μg/ml</td><td>Human</td><td>CCCCCCCSCC</td><td align="center"><a href="jmol.php?id=1896&amp;di=1" width="5px" height="5px"><img src="pymol_images/1896.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1896&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1896.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1897">1897</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16142907">16142907</a></td><td align="right">2005</td><td>KWKKLLKKaLKLaKKLLK</td><td>KLW-L9,13-a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>a=Nala (Ala-peptoid)</td><td align="right">18</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHHHHHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=1897&amp;di=1" width="5px" height="5px"><img src="pymol_images/1897.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1898">1898</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16226345">16226345</a></td><td align="right">2006</td><td>DAACAAKCLWR</td><td>L2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0% hemolysis at 200μg/ml (non-hemolytic)</td><td>Human</td><td>CTTTTTTSTTC</td><td align="center"><a href="jmol.php?id=1898&amp;di=1" width="5px" height="5px"><img src="pymol_images/1898.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1898&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1898.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1899">1899</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16226345">16226345</a></td><td align="right">2006</td><td>KAACAAHCLWR</td><td>L3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0% hemolysis at 200μg/ml (non-hemolytic)</td><td>Human</td><td>CCTTSSCCCCC</td><td align="center"><a href="jmol.php?id=1899&amp;di=1" width="5px" height="5px"><img src="pymol_images/1899.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1899&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1899.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1900">1900</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16226345">16226345</a></td><td align="right">2006</td><td>KAACAAKCLWR</td><td>L4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0% hemolysis at 200μg/ml (non-hemolytic)</td><td>Human</td><td>CBTTBSSSSCC</td><td align="center"><a href="jmol.php?id=1900&amp;di=1" width="5px" height="5px"><img src="pymol_images/1900.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1900&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1900.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1901">1901</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16344012">16344012</a></td><td align="right">2006</td><td>KWKLFKKIGIGKFLHSAKKF</td><td>CA-MA</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCCCCCCSSGGGGCTTSCC</td><td align="center"><a href="jmol.php?id=1901&amp;di=1" width="5px" height="5px"><img src="pymol_images/1901.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1901&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1901.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1902">1902</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16344012">16344012</a></td><td align="right">2006</td><td>KWKKLLKKPLLKKLLKKL</td><td>P5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHTHHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=1902&amp;di=1" width="5px" height="5px"><img src="pymol_images/1902.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1903">1903</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16460028">16460028</a></td><td align="right">2006</td><td>SADVAGAVIDGASLSFKILKTVLEALGNVKRK</td><td>EqTII<sup>1-32</sup></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCCSSSCCCSSSCCHHHHHHHHHHHTTTCTTC</td><td align="center"><a href="jmol.php?id=1903&amp;di=1" width="5px" height="5px"><img src="pymol_images/1903.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1903&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1903.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1904">1904</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16460028">16460028</a></td><td align="right">2006</td><td>GASLSFKILKTVLEALGNVKRK</td><td>EqTII<sup>11-32</sup></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCSSSHHHHTTTHHHHGGGGGC</td><td align="center"><a href="jmol.php?id=1904&amp;di=1" width="5px" height="5px"><img src="pymol_images/1904.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1904&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1904.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1905">1905</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16616888">16616888</a></td><td align="right">2006</td><td>FQWQRNIRKVR</td><td>LF11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCHHHHHCCCC</td><td align="center"><a href="jmol.php?id=1905&amp;di=1" width="5px" height="5px"><img src="pymol_images/1905.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1905&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1905.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1908">1908</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16730857">16730857</a></td><td align="right">2006</td><td>KfKLKQ</td><td>BPC6D</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">6</td><td>Antibacterial</td><td>0% hemolysis at 360µM (non-hemolytic)</td><td>Human</td><td>CCTTCC</td><td align="center"><a href="jmol.php?id=1908&amp;di=1" width="5px" height="5px"><img src="pymol_images/1908.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1908&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1908.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1909">1909</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16730857">16730857</a></td><td align="right">2006</td><td>KLKLKLKE</td><td>BPC8S2</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">8</td><td>Antibacterial</td><td>0% hemolysis at 360µM (non-hemolytic)</td><td>Human</td><td>CCCCCCCC</td><td align="center"><a href="jmol.php?id=1909&amp;di=1" width="5px" height="5px"><img src="pymol_images/1909.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1909&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1909.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1912">1912</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16730859">16730859</a></td><td align="right">2006</td><td>STVLTSKYR</td><td>α133–141</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antibacterial</td><td>Non-hemolytic</td><td>Bovine</td><td>CCSSSCSCC</td><td align="center"><a href="jmol.php?id=1912&amp;di=1" width="5px" height="5px"><img src="pymol_images/1912.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1912&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1912.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1913">1913</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16730859">16730859</a></td><td align="right">2006</td><td>VTLASHLPSDFTPAVHASLDKFLANVSTVL</td><td>α107–136</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antibacterial</td><td>Non-hemolytic</td><td>Bovine</td><td>CCCSSCCSSCCSHHHHHTTHHHHTTTCSCC</td><td align="center"><a href="jmol.php?id=1913&amp;di=1" width="5px" height="5px"><img src="pymol_images/1913.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1913&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1913.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1914">1914</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16730859">16730859</a></td><td align="right">2006</td><td>QADFQKVVAGVANALAHRYH</td><td>β126-145</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antibacterial</td><td>Non-hemolytic</td><td>Bovine</td><td>CCSHHHHHHHHHHHHHCCCC</td><td align="center"><a href="jmol.php?id=1914&amp;di=1" width="5px" height="5px"><img src="pymol_images/1914.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1914&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1914.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1915">1915</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16963159">16963159</a></td><td align="right">2006</td><td>GLWSTIKNVGKEAAIAAGKAALGAL</td><td>DPh-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>No hemolysis at 6µM (non-hemolytic)</td><td>Human</td><td>CTTSGGGTTCHHHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=1915&amp;di=1" width="5px" height="5px"><img src="pymol_images/1915.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1915&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1915.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1916">1916</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16963159">16963159</a></td><td align="right">2006</td><td>FLSLIPHAINAVSAIAKHF</td><td>PS-7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>No hemolysis at 6µM (non-hemolytic)</td><td>Human</td><td>CCSCCGGGGGHHHHHTTCC</td><td align="center"><a href="jmol.php?id=1916&amp;di=1" width="5px" height="5px"><img src="pymol_images/1916.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1916&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1916.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1925">1925</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFSKIQNDK</td><td>HP (2–20)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHTTTHHHHSGGGSCC</td><td align="center"><a href="jmol.php?id=1925&amp;di=1" width="5px" height="5px"><img src="pymol_images/1925.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1925&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1925.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1926">1926</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>KVFKRLEKLFSKIQNDK</td><td>HPN1(4–20)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCGGGTGGGGCTTTSCC</td><td align="center"><a href="jmol.php?id=1926&amp;di=1" width="5px" height="5px"><img src="pymol_images/1926.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1926&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1926.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1927">1927</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>FKRLEKLFSKIQNDK</td><td>HPN2(6–20)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHHHHCTTTTCC</td><td align="center"><a href="jmol.php?id=1927&amp;di=1" width="5px" height="5px"><img src="pymol_images/1927.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1927&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1927.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1928">1928</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>RLEKLFSKIQNDK</td><td>HPN3(8–20)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHHSGGGGCC</td><td align="center"><a href="jmol.php?id=1928&amp;di=1" width="5px" height="5px"><img src="pymol_images/1928.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1928&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1928.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1929">1929</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFSKIQN</td><td>HPC1(2–18)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CGGGGHHHHTTTCSSCC</td><td align="center"><a href="jmol.php?id=1929&amp;di=1" width="5px" height="5px"><img src="pymol_images/1929.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1929&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1929.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1930">1930</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFSKI</td><td>HPC2(2–16)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCGGGGGTTTTTSCC</td><td align="center"><a href="jmol.php?id=1930&amp;di=1" width="5px" height="5px"><img src="pymol_images/1930.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1930&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1930.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1931">1931</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFS</td><td>HPC3(2–14)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTTGGGGTTTCC</td><td align="center"><a href="jmol.php?id=1931&amp;di=1" width="5px" height="5px"><img src="pymol_images/1931.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1931&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1931.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1932">1932</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>KVFKRLEKLFSKI</td><td>HPN1C2(4–16)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTHHHHTTTTCCC</td><td align="center"><a href="jmol.php?id=1932&amp;di=1" width="5px" height="5px"><img src="pymol_images/1932.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1932&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1932.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1933">1933</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>KVFKRLEKLFS</td><td>HPN1C3(4–14)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=1933&amp;di=1" width="5px" height="5px"><img src="pymol_images/1933.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1933&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1933.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1934">1934</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>FKRLEKLF</td><td>HPN2C4(6–13)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCSSCTTC</td><td align="center"><a href="jmol.php?id=1934&amp;di=1" width="5px" height="5px"><img src="pymol_images/1934.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1934&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1934.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1935">1935</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFSKIQNWK</td><td>HPA1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CGGGGGGTGGGSCTTTSCC</td><td align="center"><a href="jmol.php?id=1935&amp;di=1" width="5px" height="5px"><img src="pymol_images/1935.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1935&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1935.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1936">1936</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKLFSKIWNDK</td><td>HPA2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTTTHHHHHHHCCSSCCC</td><td align="center"><a href="jmol.php?id=1936&amp;di=1" width="5px" height="5px"><img src="pymol_images/1936.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1936&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1936.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1937">1937</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVFKRLEKSFSKIQNDK</td><td>HPA4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHTTGGGTTCTTSCCC</td><td align="center"><a href="jmol.php?id=1937&amp;di=1" width="5px" height="5px"><img src="pymol_images/1937.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1937&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1937.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1938">1938</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17961502">17961502</a></td><td align="right">2008</td><td>AKKVSKRLEKLFSKIQNDK</td><td>HPA5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHHHHHTTTCTTTSCC</td><td align="center"><a href="jmol.php?id=1938&amp;di=1" width="5px" height="5px"><img src="pymol_images/1938.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1938&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1938.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1939">1939</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20015460">20015460</a></td><td align="right">2010</td><td>GVIKSVLKGVAKTVALGML</td><td>B2RP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =280µM</td><td>Human</td><td>CCHHHHHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1939&amp;di=1" width="5px" height="5px"><img src="pymol_images/1939.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1939&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1939.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1940">1940</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17888907">17888907</a></td><td align="right">2008</td><td>KWCFRVCYRGICYRRC</td><td>Tachyplesin-I</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>10% hemolysis 100µM</td><td>Human</td><td>CCSCCCCTTCCCCCCC</td><td align="center"><a href="jmol.php?id=1940&amp;di=1" width="5px" height="5px"><img src="pymol_images/1940.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1940&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1940.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1941">1941</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17888907">17888907</a></td><td align="right">2008</td><td>VFQFLGKIIHHVGNFVHGFSHVF</td><td>Clavanin-A</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>20% hemolysis 100 µM</td><td>Human</td><td>CTTCGGGSSTTTCCCCCSCCCCC</td><td align="center"><a href="jmol.php?id=1941&amp;di=1" width="5px" height="5px"><img src="pymol_images/1941.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1941&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1941.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1942">1942</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17888907">17888907</a></td><td align="right">2008</td><td>GIGKFLKKAKKFGKAFVKMKK</td><td>MSI-94</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>45-50% hemolysis 100µM</td><td>Human</td><td>CTHHHHHHHHTTHHHHTTCCC</td><td align="center"><a href="jmol.php?id=1942&amp;di=1" width="5px" height="5px"><img src="pymol_images/1942.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1942&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1942.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1943">1943</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17888907">17888907</a></td><td align="right">2008</td><td>GCASRCKAKCAGRRCKGWASASFRGRCYCKCFRC</td><td>Mytilin-A</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antimicrobial</td><td>10% hemolysis 100µM</td><td>Human</td><td>CCCCTTTSSCCCCCCCCCSSSCCSSSCCCCCCCC</td><td align="center"><a href="jmol.php?id=1943&amp;di=1" width="5px" height="5px"><img src="pymol_images/1943.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1943&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1943.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1945">1945</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAAKALVKTVLF</td><td>Ascaphin-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =55µM</td><td>Human</td><td>CHHHHHHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=1945&amp;di=1" width="5px" height="5px"><img src="pymol_images/1945.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1945&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1945.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1946">1946</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKKLLKGAAKALVKTVLF</td><td>Lys-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =24µM</td><td>Human</td><td>CHHHHHHHHHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=1946&amp;di=1" width="5px" height="5px"><img src="pymol_images/1946.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1947">1947</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKKAAKALVKTVLF</td><td>Lys-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =11µM</td><td>Human</td><td>CHHHHHHHHHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=1947&amp;di=1" width="5px" height="5px"><img src="pymol_images/1947.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1947&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1947.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1948">1948</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAKKALVKTVLF</td><td>Lys-10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>CHHHHHHHHHHHHGGGTCC</td><td align="center"><a href="jmol.php?id=1948&amp;di=1" width="5px" height="5px"><img src="pymol_images/1948.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1949">1949</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAAKALKKTVLF</td><td>Lys-14</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>CHHHHHHTTHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=1949&amp;di=1" width="5px" height="5px"><img src="pymol_images/1949.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1949&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1949.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1950">1950</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAAKALVKTVKF</td><td>Lys-18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>CHHHHHHHHHHHHHHSCCC</td><td align="center"><a href="jmol.php?id=1950&amp;di=1" width="5px" height="5px"><img src="pymol_images/1950.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1950&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1950.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1951">1951</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKkLLKGAAKALVKTVLF</td><td>D-Lys-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =300µM</td><td>Human</td><td>CHHHHHHHHHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=1951&amp;di=1" width="5px" height="5px"><img src="pymol_images/1951.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1951&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1951.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1952">1952</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKkAAKALVKTVLF</td><td>D-Lys-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =150µM</td><td>Human</td><td>CHHHHHHHHHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=1952&amp;di=1" width="5px" height="5px"><img src="pymol_images/1952.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1952&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1952.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1953">1953</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAkKALVKTVLF</td><td>D-Lys-10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1954">1954</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAAKALkKTVLF</td><td>D-Lys-14</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>CHHHHHHTTHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=1954&amp;di=1" width="5px" height="5px"><img src="pymol_images/1954.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1954&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1954.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1955">1955</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAAKALVKTVkF</td><td>D-Lys-18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>CHHHHHHHHHHHHHHSCCC</td><td align="center"><a href="jmol.php?id=1955&amp;di=1" width="5px" height="5px"><img src="pymol_images/1955.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1955&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1955.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1956">1956</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GFKDLLKGAAKALVKAVLF</td><td>Ala-16</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =22µM</td><td>Human</td><td>CHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1956&amp;di=1" width="5px" height="5px"><img src="pymol_images/1956.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1956&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1956.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1957">1957</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GLLGPLLKIAAKVGSNLL</td><td>XT-7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =140µM</td><td>Human</td><td>CCSGGGHHHHHHHTCCCC</td><td align="center"><a href="jmol.php?id=1957&amp;di=1" width="5px" height="5px"><img src="pymol_images/1957.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1957&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1957.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1958">1958</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>KLLGPLLKIAAKVGSNLL</td><td>Lys-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =400µM</td><td>Human</td><td>CCTTTTHHHHHHHSCSCC</td><td align="center"><a href="jmol.php?id=1958&amp;di=1" width="5px" height="5px"><img src="pymol_images/1958.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1958&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1958.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1959">1959</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GLLKPLLKIAAKVGSNLL</td><td>Lys-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;500µM</td><td>Human</td><td>CTTGGGGGGGGGTCCCCC</td><td align="center"><a href="jmol.php?id=1959&amp;di=1" width="5px" height="5px"><img src="pymol_images/1959.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1959&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1959.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1960">1960</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GLLGKLLKIAAKVGSNLL</td><td>Lys-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =15µM</td><td>Human</td><td>CCHHHHHHHHHHHSCCCC</td><td align="center"><a href="jmol.php?id=1960&amp;di=1" width="5px" height="5px"><img src="pymol_images/1960.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1960&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1960.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1961">1961</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GLLGPLLKIAKKVGSNLL</td><td>Lys-11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =350µM</td><td>Human</td><td>CCTTHHHHHHTTTCSSCC</td><td align="center"><a href="jmol.php?id=1961&amp;di=1" width="5px" height="5px"><img src="pymol_images/1961.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1961&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1961.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1962">1962</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GLLGPLLKIAAKVGKNLL</td><td>Lys-15</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =110µM</td><td>Human</td><td>CCTTTHHHHHHHTTCCCC</td><td align="center"><a href="jmol.php?id=1962&amp;di=1" width="5px" height="5px"><img src="pymol_images/1962.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1962&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1962.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1963">1963</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GLLGPLLKIAAKVGSKLL</td><td>Lys-16</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =80µM</td><td>Human</td><td>CCSTTHHHHHHHHHSSCC</td><td align="center"><a href="jmol.php?id=1963&amp;di=1" width="5px" height="5px"><img src="pymol_images/1963.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1963&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1963.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1964">1964</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GLLGPLLKIAAKVGKKLL</td><td>Lys-15, Lys-16</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =60µM</td><td>Human</td><td>CCSTHHHHHHHTTCCCCC</td><td align="center"><a href="jmol.php?id=1964&amp;di=1" width="5px" height="5px"><img src="pymol_images/1964.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1964&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1964.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1965">1965</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GLLGKLLKIAAKVGKKLL</td><td>Lys-5, Lys-15, Lys-16</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =25µM</td><td>Human</td><td>CCTHHHHHHHHHTCSCCC</td><td align="center"><a href="jmol.php?id=1965&amp;di=1" width="5px" height="5px"><img src="pymol_images/1965.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1965&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1965.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1966">1966</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18554256">18554256</a></td><td align="right">2008</td><td>GLLKKLLKIAAKVGKKLL</td><td>Lys-4, Lys-5,Lys-15, Lys-16</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =25µM</td><td>Human</td><td>CTHHHHHHHHHHTCCCCC</td><td align="center"><a href="jmol.php?id=1966&amp;di=1" width="5px" height="5px"><img src="pymol_images/1966.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1966&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1966.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1967">1967</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11242518">11242518</a></td><td align="right">2001</td><td>AVGIGALFLGFLGAAGSTMGARS</td><td>FP-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Hemolytic</td><td>50% hemolysis at 40µM</td><td>Human</td><td>CCSSGGGGHHHHHHHTTCSSSCC</td><td align="center"><a href="jmol.php?id=1967&amp;di=1" width="5px" height="5px"><img src="pymol_images/1967.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1967&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1967.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1968">1968</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11242518">11242518</a></td><td align="right">2001</td><td>AVGIGALFLGFLGAAGSTMGARS</td><td>FP-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Hemolytic</td><td>90% hemolysis at 100µM</td><td>Human</td><td>CCSSGGGGHHHHHHHTTCSSSCC</td><td align="center"><a href="jmol.php?id=1968&amp;di=1" width="5px" height="5px"><img src="pymol_images/1968.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1968&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1968.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1969">1969</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11242518">11242518</a></td><td align="right">2001</td><td>AAGLAMLFLGILSAAGSTMGARA</td><td>FP-I<sub>VAU</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Hemolytic</td><td>~50% hemolysis at 40µM</td><td>Human</td><td>CCTHHHHHHHHHHHTTCCTTTTC</td><td align="center"><a href="jmol.php?id=1969&amp;di=1" width="5px" height="5px"><img src="pymol_images/1969.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1969&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1969.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1970">1970</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11242518">11242518</a></td><td align="right">2001</td><td>AAGLAMLFLGILSAAGSTMGARA</td><td>FP-I<sub>VAU</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Hemolytic</td><td>55-60% hemolysis at 80µM</td><td>Human</td><td>CCTHHHHHHHHHHHTTCCTTTTC</td><td align="center"><a href="jmol.php?id=1970&amp;di=1" width="5px" height="5px"><img src="pymol_images/1970.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1970&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1970.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1973">1973</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11478963">11478963</a></td><td align="right">2001</td><td>LKLKSIVSWAKKVL</td><td>MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Hypotensive</td><td>~85% hemolysis at 15µM</td><td>Guinea-pig</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1973&amp;di=1" width="5px" height="5px"><img src="pymol_images/1973.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1973&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1973.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1974">1974</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11478963">11478963</a></td><td align="right">2001</td><td>LKLKSIVSWAKKVL</td><td>MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Hypotensive</td><td>~95% hemolysis at 20µM</td><td>Guinea-pig</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1974&amp;di=1" width="5px" height="5px"><img src="pymol_images/1974.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1974&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1974.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1975">1975</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11478963">11478963</a></td><td align="right">2001</td><td>lklksivswakkvl</td><td>D-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">14</td><td>Hypotensive</td><td>15% hemolysis at 15µM</td><td>Guinea-pig</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1975&amp;di=1" width="5px" height="5px"><img src="pymol_images/1975.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1975&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1975.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1976">1976</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11478963">11478963</a></td><td align="right">2001</td><td>lklksivswakkvl</td><td>D-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">14</td><td>Hypotensive</td><td>20% hemolysis at 20µM</td><td>Guinea-pig</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1976&amp;di=1" width="5px" height="5px"><img src="pymol_images/1976.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1976&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1976.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1977">1977</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11478963">11478963</a></td><td align="right">2001</td><td>LkLKSIVSWAKKVL</td><td>[D-Lys<sup>2</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">14</td><td>Hypotensive</td><td>10% hemolysis at 15µM</td><td>Guinea-pig</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1977&amp;di=1" width="5px" height="5px"><img src="pymol_images/1977.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1977&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1977.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1978">1978</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11478963">11478963</a></td><td align="right">2001</td><td>LkLKSIVSWAKKVL</td><td>[D-Lys<sup>2</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">14</td><td>Hypotensive</td><td>15% hemolysis at 20µM</td><td>Guinea-pig</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1978&amp;di=1" width="5px" height="5px"><img src="pymol_images/1978.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1978&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1978.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1979">1979</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11478963">11478963</a></td><td align="right">2001</td><td>LKLkSIVSWAKKVL</td><td>[D-Lys<sup>4</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">14</td><td>Hypotensive</td><td>5% hemolysis at 15µM</td><td>Guinea-pig</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1979&amp;di=1" width="5px" height="5px"><img src="pymol_images/1979.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1979&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1979.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1980">1980</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11478963">11478963</a></td><td align="right">2001</td><td>LKLkSIVSWAKKVL</td><td>[D-Lys<sup>4</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">14</td><td>Hypotensive</td><td>10% hemolysis at 20µM</td><td>Guinea-pig</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1980&amp;di=1" width="5px" height="5px"><img src="pymol_images/1980.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1980&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1980.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1981">1981</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11478963">11478963</a></td><td align="right">2001</td><td>LKLKSIVSWAkkVL</td><td>[D-Lys<sup>11,12</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">14</td><td>Hypotensive</td><td>&lt;5% hemolysis at 15µM</td><td>Guinea-pig</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1981&amp;di=1" width="5px" height="5px"><img src="pymol_images/1981.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1981&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1981.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1982">1982</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11478963">11478963</a></td><td align="right">2001</td><td>LKLKSIVSWAkkVL</td><td>[D-Lys<sup>11,12</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">14</td><td>Hypotensive</td><td>&lt;5% hemolysis at 20µM</td><td>Guinea-pig</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=1982&amp;di=1" width="5px" height="5px"><img src="pymol_images/1982.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1982&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1982.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1983">1983</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11520071">11520071</a></td><td align="right">2001</td><td>ILGKIWEGIKSLF</td><td>IsCT</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Cytotoxic</td><td>30-40% hemolysis at 50µM</td><td>Sheep</td><td>CCSTHHHHTTCCC</td><td align="center"><a href="jmol.php?id=1983&amp;di=1" width="5px" height="5px"><img src="pymol_images/1983.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1983&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1983.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1984">1984</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11520071">11520071</a></td><td align="right">2001</td><td>ILGKIWEGIKSLF</td><td>IsCT</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Cytotoxic</td><td>50-60% hemolysis at 100µM</td><td>Sheep</td><td>CCSTHHHHTTCCC</td><td align="center"><a href="jmol.php?id=1984&amp;di=1" width="5px" height="5px"><img src="pymol_images/1984.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1984&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1984.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1985">1985</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11532323">11532323</a></td><td align="right">2001</td><td>WEAKLAKALAKALAKHLAKALAKALKACEA</td><td>KALA</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Cytotoxic</td><td>65-70% hemolysis at 10µg/ml</td><td>Human</td><td>CCTTTHHHHHHHHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1985&amp;di=1" width="5px" height="5px"><img src="pymol_images/1985.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1985&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1985.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1986">1986</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11532323">11532323</a></td><td align="right">2001</td><td>WEAKLAKALAKALAKHLAKALAKALKACEA</td><td>KALA</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Cytotoxic</td><td>80-85% hemolysis at 100µg/ml</td><td>Human</td><td>CCTTTHHHHHHHHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1986&amp;di=1" width="5px" height="5px"><img src="pymol_images/1986.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1986&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1986.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1987">1987</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11532323">11532323</a></td><td align="right">2001</td><td>WEAKLAKALAKALAKHLAKALAKALKACEA</td><td>PEG-KALA</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Hemolytic</td><td>50-60% hemolysis at 10µg/ml</td><td>Human</td><td>CCTTTHHHHHHHHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1987&amp;di=1" width="5px" height="5px"><img src="pymol_images/1987.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1987&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1987.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1988">1988</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11532323">11532323</a></td><td align="right">2001</td><td>WEAKLAKALAKALAKHLAKALAKALKACEA</td><td>PEG-KALA</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Hemolytic</td><td>90-95% hemolysis at 100µg/ml</td><td>Human</td><td>CCTTTHHHHHHHHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=1988&amp;di=1" width="5px" height="5px"><img src="pymol_images/1988.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1988&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1988.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1994">1994</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15003829">15003829</a></td><td align="right">2004</td><td>GLMDVFKGAAKNLLASALDKIRCKVTKC</td><td>Ranatuerin-2PRa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>HC<sub>50</sub> =150µM</td><td>Human</td><td>CCCCTTTTHHHHHHHHHHHHHHHSCCCC</td><td align="center"><a href="jmol.php?id=1994&amp;di=1" width="5px" height="5px"><img src="pymol_images/1994.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1994&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1994.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1995">1995</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15003829">15003829</a></td><td align="right">2004</td><td>FLSLALAALPKLFCLIFKKC</td><td>Brevinin-1PRa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>HC<sub>50</sub> =7µM</td><td>Human</td><td>CHHHHHTTTTTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=1995&amp;di=1" width="5px" height="5px"><img src="pymol_images/1995.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1995&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1995.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1996">1996</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15003829">15003829</a></td><td align="right">2004</td><td>ILPILGNLLNGLL</td><td>Temporin-1PRa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;300µM</td><td>Human</td><td>CCSSHHHHSCSCC</td><td align="center"><a href="jmol.php?id=1996&amp;di=1" width="5px" height="5px"><img src="pymol_images/1996.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1996&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1996.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1997">1997</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15003829">15003829</a></td><td align="right">2004</td><td>ILPILGNLLNSLL</td><td>Temporin-1PRb</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;300µM</td><td>Human</td><td>CCCSGGGGGTTCC</td><td align="center"><a href="jmol.php?id=1997&amp;di=1" width="5px" height="5px"><img src="pymol_images/1997.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1997&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1997.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=1998">1998</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21871946">21871946</a></td><td align="right">2012</td><td>FVDLKKIANIINSIF</td><td>Temporin-1CEa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>60% hemolysis at 100µM</td><td>Human</td><td>CCCTTTTHHHHTCCC</td><td align="center"><a href="jmol.php?id=1998&amp;di=1" width="5px" height="5px"><img src="pymol_images/1998.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1998&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1998.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=1999">1999</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21871946">21871946</a></td><td align="right">2012</td><td>FVDLKKIANIINSIF</td><td>Temporin-1CEa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>100% hemolysis at 200µM</td><td>Human</td><td>CCCTTTTHHHHTCCC</td><td align="center"><a href="jmol.php?id=1999&amp;di=1" width="5px" height="5px"><img src="pymol_images/1999.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=1999&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/1999.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2000">2000</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21924309">21924309</a></td><td align="right">2011</td><td>LKLKSIVSWAKKVL</td><td>MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antioxidant</td><td>Non-hemolytic</td><td>Human</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=2000&amp;di=1" width="5px" height="5px"><img src="pymol_images/2000.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2000&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2000.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2001">2001</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21924309">21924309</a></td><td align="right">2011</td><td>LKWKSIVSWAKKVL</td><td>MP-B-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antioxidant</td><td>Non-hemolytic</td><td>Human</td><td>CCSHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2001&amp;di=1" width="5px" height="5px"><img src="pymol_images/2001.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2001&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2001.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2002">2002</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21924309">21924309</a></td><td align="right">2011</td><td>LKLKNIVSWAKKVL</td><td>MP-B-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antioxidant</td><td>Non-hemolytic</td><td>Human</td><td>CCCCSTHHHHTTTC</td><td align="center"><a href="jmol.php?id=2002&amp;di=1" width="5px" height="5px"><img src="pymol_images/2002.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2002&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2002.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2003">2003</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21924309">21924309</a></td><td align="right">2011</td><td>LKLKSIVSYAKKVL</td><td>MP-B-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antioxidant</td><td>Non-hemolytic</td><td>Human</td><td>CCSGGGHHHHTTCC</td><td align="center"><a href="jmol.php?id=2003&amp;di=1" width="5px" height="5px"><img src="pymol_images/2003.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2003&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2003.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2004">2004</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21928440">21928440</a></td><td align="right">2011</td><td>CGETCVGGTCNTPGCTCSWPVCTRNGLPV</td><td>Kalata B1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Anticancer and Anti-HIV</td><td>IC<sub>50</sub> =26µM</td><td>Human</td><td>CCCGGGTTTSSCCCCCSSSCCCTTCSSCC</td><td align="center"><a href="jmol.php?id=2004&amp;di=1" width="5px" height="5px"><img src="pymol_images/2004.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2004&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2004.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2005">2005</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21928440">21928440</a></td><td align="right">2011</td><td>cgetcvggtcntpgctcswpvctrnglnpv</td><td>D-Kalata B1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>D</td><td>None</td><td align="right">29</td><td>Anticancer and Anti-HIV</td><td>IC<sub>50</sub> =77µM</td><td>Human</td><td>CCCSSSSCCCCCTTTTCCCCSSSSSCSCCC</td><td align="center"><a href="jmol.php?id=2005&amp;di=1" width="5px" height="5px"><img src="pymol_images/2005.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2005&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2005.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2008">2008</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22123629">22123629</a></td><td align="right">2012</td><td>GFGSLLGKALKIGTNLL</td><td>CPF-PG1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>LC<sub>50</sub> =145µM</td><td>Human</td><td>NA</td><td align="center"><a href="jmol.php?id=2008&amp;di=1" width="5px" height="5px"><img src="pymol_images/2008.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2008&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2008.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2009">2009</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10631295">10631295</a></td><td align="right">2000</td><td>KLKLKLKLK</td><td>(KL)<sub>4</sub>K</td><td>Amidation</td><td>Dansylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>&gt;20% hemolysis at 10µM</td><td>Human</td><td>CCCCCTTTC</td><td align="center"><a href="jmol.php?id=2009&amp;di=1" width="5px" height="5px"><img src="pymol_images/2009.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2010">2010</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10631295">10631295</a></td><td align="right">2000</td><td>KLKLKLKLKLK</td><td>(KL)<sub>5</sub>K</td><td>Amidation</td><td>Dansylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>60% hemolysis at 10µM</td><td>Human</td><td>CCCCCSSCCCC</td><td align="center"><a href="jmol.php?id=2010&amp;di=1" width="5px" height="5px"><img src="pymol_images/2010.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2011">2011</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10631295">10631295</a></td><td align="right">2000</td><td>KLKLKLKLKLKLK</td><td>(KL)<sub>6</sub>K</td><td>Amidation</td><td>Dansylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>~90% hemolysis at 10µM</td><td>Human</td><td>CCCCTTTTCSSCC</td><td align="center"><a href="jmol.php?id=2011&amp;di=1" width="5px" height="5px"><img src="pymol_images/2011.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2012">2012</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10631295">10631295</a></td><td align="right">2000</td><td>KLKLKLKLKLKLKLK</td><td>(KL)<sub>7</sub>K</td><td>Amidation</td><td>Dansylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>~90% hemolysis at 10µM</td><td>Human</td><td>CCSCCCCCTTTTTTC</td><td align="center"><a href="jmol.php?id=2012&amp;di=1" width="5px" height="5px"><img src="pymol_images/2012.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2013">2013</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10667861">10667861</a></td><td align="right">2000</td><td>INLKALAALAKKIL</td><td>MP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>68.2% hemolysis at 100µM</td><td>Human</td><td>CCHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2013&amp;di=1" width="5px" height="5px"><img src="pymol_images/2013.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2013&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2013.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2014">2014</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10667861">10667861</a></td><td align="right">2000</td><td>INLKALAALAKKIL</td><td>MP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>37.9% hemolysis at 50µM</td><td>Human</td><td>CCHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2014&amp;di=1" width="5px" height="5px"><img src="pymol_images/2014.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2014&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2014.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2015">2015</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10667861">10667861</a></td><td align="right">2000</td><td>LKAKSIVSWAKKVL</td><td>[Ala<sup>3</sup>] MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>13.8% hemolysis at 100µM</td><td>Human</td><td>CGGGSTHHHHHHHC</td><td align="center"><a href="jmol.php?id=2015&amp;di=1" width="5px" height="5px"><img src="pymol_images/2015.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2015&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2015.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2016">2016</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10667861">10667861</a></td><td align="right">2000</td><td>LKAKSIVSWAKKVL</td><td>[Ala<sup>3</sup>] MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>11.9% hemolysis at 50µM</td><td>Human</td><td>CGGGSTHHHHHHHC</td><td align="center"><a href="jmol.php?id=2016&amp;di=1" width="5px" height="5px"><img src="pymol_images/2016.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2016&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2016.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2017">2017</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10667861">10667861</a></td><td align="right">2000</td><td>LKLASIVSWAKKVL</td><td>[Ala<sup>4</sup>]MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>93.4% hemolysis at 100µM</td><td>Human</td><td>CCCCHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2017&amp;di=1" width="5px" height="5px"><img src="pymol_images/2017.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2017&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2017.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2018">2018</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10667861">10667861</a></td><td align="right">2000</td><td>LKLASIVSWAKKVL</td><td>[Ala<sup>4</sup>]MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>60.1% hemolysis at 50µM</td><td>Human</td><td>CCCCHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2018&amp;di=1" width="5px" height="5px"><img src="pymol_images/2018.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2018&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2018.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2019">2019</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10667861">10667861</a></td><td align="right">2000</td><td>LKLKSIVSAAKKVL</td><td>[Ala<sup>9</sup>]MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>6.0% hemolysis at 100µM</td><td>Human</td><td>CCCGGGHHHHTTCC</td><td align="center"><a href="jmol.php?id=2019&amp;di=1" width="5px" height="5px"><img src="pymol_images/2019.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2019&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2019.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2020">2020</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10667861">10667861</a></td><td align="right">2000</td><td>LKLKSIVSAAKKVL</td><td>[Ala<sup>9</sup>]MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>8.5% hemolysis at 50µM</td><td>Human</td><td>CCCGGGHHHHTTCC</td><td align="center"><a href="jmol.php?id=2020&amp;di=1" width="5px" height="5px"><img src="pymol_images/2020.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2020&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2020.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2021">2021</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10667861">10667861</a></td><td align="right">2000</td><td>LKLKSIVSWAKAVL</td><td>[Ala<sup>12</sup>]MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>66.2% hemolysis at 100µM</td><td>Human</td><td>CCCTTHHHHTTTTC</td><td align="center"><a href="jmol.php?id=2021&amp;di=1" width="5px" height="5px"><img src="pymol_images/2021.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2021&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2021.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2022">2022</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10667861">10667861</a></td><td align="right">2000</td><td>LKLKSIVSWAKAVL</td><td>[Ala12]MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>59.1% hemolysis at 50µM</td><td>Human</td><td>CCCTTHHHHTTTTC</td><td align="center"><a href="jmol.php?id=2022&amp;di=1" width="5px" height="5px"><img src="pymol_images/2022.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2022&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2022.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2023">2023</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10675500">10675500</a></td><td align="right">2000</td><td>KWKLFKKIGIGKFLHSAKKF</td><td>CA-MA</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCCCCCCSSGGGGCTTSCC</td><td align="center"><a href="jmol.php?id=2023&amp;di=1" width="5px" height="5px"><img src="pymol_images/2023.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2023&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2023.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2024">2024</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10675500">10675500</a></td><td align="right">2000</td><td>KWKLFKKIKFLHSAKKF</td><td>CA-MA1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCSSCCCCGGGSSCCCC</td><td align="center"><a href="jmol.php?id=2024&amp;di=1" width="5px" height="5px"><img src="pymol_images/2024.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2024&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2024.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2025">2025</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10675500">10675500</a></td><td align="right">2000</td><td>KWKLFKKIPKFLHSAKKF</td><td>CA-MA2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCCCCCCHHHHHSSSCC</td><td align="center"><a href="jmol.php?id=2025&amp;di=1" width="5px" height="5px"><img src="pymol_images/2025.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2025&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2025.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2026">2026</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10675500">10675500</a></td><td align="right">2000</td><td>KWKLFKKIGPGKFLHSAKKF</td><td>CA-MA3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCCCCCCSSCCCCCSSSCC</td><td align="center"><a href="jmol.php?id=2026&amp;di=1" width="5px" height="5px"><img src="pymol_images/2026.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2026&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2026.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2027">2027</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10817697">10817697</a></td><td align="right">2000</td><td>DSHAKRHHGYKRKFHEKHHSHRGY</td><td>Hsn-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antifungal</td><td>&lt;10% hemolysis at 200µM</td><td>Human</td><td>CCSTTTTSSCCHHHHTTTSCCSCC</td><td align="center"><a href="jmol.php?id=2027&amp;di=1" width="5px" height="5px"><img src="pymol_images/2027.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2028">2028</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10817697">10817697</a></td><td align="right">2000</td><td>DSHAKRHHGYKIKFHENHHSHRGY</td><td>R12I/K17N</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antifungal</td><td>&lt;10% hemolysis at 200µM</td><td>Human</td><td>CCCTTSCCCCCCCSSCCCCSSSCC</td><td align="center"><a href="jmol.php?id=2028&amp;di=1" width="5px" height="5px"><img src="pymol_images/2028.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2029">2029</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10817697">10817697</a></td><td align="right">2000</td><td>DSHAKRHHGYKIKFHEKHHSLRGY</td><td>R12I/H21L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antifungal</td><td>&lt;10% hemolysis at 200µM</td><td>Human</td><td>CCSSCSSCCCCSSGGGGTSCCSCC</td><td align="center"><a href="jmol.php?id=2029&amp;di=1" width="5px" height="5px"><img src="pymol_images/2029.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2030">2030</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10817697">10817697</a></td><td align="right">2000</td><td>EGRERDHELRHRRHHHQSPK</td><td>MUC7 D1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antifungal</td><td>&lt;10% hemolysis at 200µM</td><td>Human</td><td>CTTCCCTTCSSSCSSSSCCC</td><td align="center"><a href="jmol.php?id=2030&amp;di=1" width="5px" height="5px"><img src="pymol_images/2030.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2030&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2030.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2031">2031</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10819172">10819172</a></td><td align="right">2000</td><td>KLLLKLLLKLLK</td><td>S1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>100% hemolysis at 200µg/ml</td><td>Mouse</td><td>CTTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2031&amp;di=1" width="5px" height="5px"><img src="pymol_images/2031.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2031&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2031.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2032">2032</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10819172">10819172</a></td><td align="right">2000</td><td>KLL<sub>Ψ</sub>[CH<sub>2<sub>OCONH]LKLLLKLLK</sub></sub></td><td>SC1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>100% hemolysis at 200µg/ml</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2033">2033</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10819172">10819172</a></td><td align="right">2000</td><td>KLL<sub>Ψ</sub>[CH<sub>2</sub>OCONH]LKLLL<sub>Ψ</sub>[CH<sub>2</sub>OCONH]KLLK</td><td>SC2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>~90% hemolysis at 200µg/ml</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2045">2045</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>FFRRFFRR</td><td>(FFRR)<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt; 500mg/L</td><td>Human</td><td>CCTTSCCC</td><td align="center"><a href="jmol.php?id=2045&amp;di=1" width="5px" height="5px"><img src="pymol_images/2045.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2045&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2045.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2046">2046</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>LLRRLLRR</td><td>(LLRR)<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt; 500mg/L</td><td>Human</td><td>CGGGTTTC</td><td align="center"><a href="jmol.php?id=2046&amp;di=1" width="5px" height="5px"><img src="pymol_images/2046.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2046&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2046.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2047">2047</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>LLKKLLKK</td><td>(LLKK)<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt; 500mg/L</td><td>Human</td><td>CHHHHCCC</td><td align="center"><a href="jmol.php?id=2047&amp;di=1" width="5px" height="5px"><img src="pymol_images/2047.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2047&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2047.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2048">2048</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>FFRRFFRRFFRR</td><td>(FFRR)<sub>3</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt; 500mg/L</td><td>Human</td><td>CHHHHTTTSCCC</td><td align="center"><a href="jmol.php?id=2048&amp;di=1" width="5px" height="5px"><img src="pymol_images/2048.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2048&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2048.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2049">2049</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>AARRAARRAARR</td><td>(AARR)<sub>3</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt; 500mg/L</td><td>Human</td><td>CHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2049&amp;di=1" width="5px" height="5px"><img src="pymol_images/2049.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2049&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2049.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2050">2050</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>LLRRLLRRLLRR</td><td>(LLRR)<sub>3</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt; 500mg/L</td><td>Human</td><td>CHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2050&amp;di=1" width="5px" height="5px"><img src="pymol_images/2050.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2050&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2050.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2051">2051</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>LLKKLLKKLLKK</td><td>(LLKK)<sub>3</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt; 500mg/L</td><td>Human</td><td>CHHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=2051&amp;di=1" width="5px" height="5px"><img src="pymol_images/2051.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2051&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2051.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2052">2052</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>FFRRFFRRFFRRFFRR</td><td>(FFRR)<sub>4</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>HC<sub>50</sub> ~26mg/L</td><td>Human</td><td>CHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2052&amp;di=1" width="5px" height="5px"><img src="pymol_images/2052.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2052&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2052.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2053">2053</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>LLRRLLRRLLRRLLRR</td><td>(LLRR)<sub>4</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>HC<sub>50</sub> ~12mg/L</td><td>Human</td><td>CHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2053&amp;di=1" width="5px" height="5px"><img src="pymol_images/2053.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2053&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2053.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2054">2054</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21168911">21168911</a></td><td align="right">2011</td><td>LLKKLLKKLLKKLLKK</td><td>(LLKK)<sub>4</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>HC<sub>50</sub>~24mg/L</td><td>Human</td><td>CHHHHHHHHHTTTTCC</td><td align="center"><a href="jmol.php?id=2054&amp;di=1" width="5px" height="5px"><img src="pymol_images/2054.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2054&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2054.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2055">2055</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>GRILSFIKGLAEHL</td><td>OdVP1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCCCCGGGGGSCCC</td><td align="center"><a href="jmol.php?id=2055&amp;di=1" width="5px" height="5px"><img src="pymol_images/2055.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2055&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2055.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2056">2056</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>ILGIITSLLKSL</td><td>OdVP2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>EC<sub>50</sub> =31(23–41)µM</td><td>Human</td><td>CGGGHHHHTTCC</td><td align="center"><a href="jmol.php?id=2056&amp;di=1" width="5px" height="5px"><img src="pymol_images/2056.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2056&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2056.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2057">2057</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>KDLHTVVSAILQAL</td><td>OdVP3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCSHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2057&amp;di=1" width="5px" height="5px"><img src="pymol_images/2057.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2057&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2057.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2058">2058</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>LDPKVVQSLL</td><td>OdVP4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCHHHHHTTC</td><td align="center"><a href="jmol.php?id=2058&amp;di=1" width="5px" height="5px"><img src="pymol_images/2058.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2058&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2058.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2059">2059</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>INLKGLIKKVASLLT</td><td>EpVP1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCHHHHHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2059&amp;di=1" width="5px" height="5px"><img src="pymol_images/2059.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2059&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2059.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2060">2060</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>FDLLGLVKKVASAL</td><td>EpVP2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCGGGGTTHHHHTC</td><td align="center"><a href="jmol.php?id=2060&amp;di=1" width="5px" height="5px"><img src="pymol_images/2060.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2060&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2060.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2061">2061</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>FDLLGLVKSVVSAL</td><td>EpVP2b</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>EC<sub>50</sub> =64(68–70)µM</td><td>Human</td><td>CCTTSSSTTTTTTC</td><td align="center"><a href="jmol.php?id=2061&amp;di=1" width="5px" height="5px"><img src="pymol_images/2061.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2061&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2061.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2062">2062</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>AINPKSVQSLL</td><td>EpVP3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCSHHHHTTTC</td><td align="center"><a href="jmol.php?id=2062&amp;di=1" width="5px" height="5px"><img src="pymol_images/2062.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2063">2063</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>INPKSVQSLL</td><td>EpVP3S</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCCCSSCCCC</td><td align="center"><a href="jmol.php?id=2063&amp;di=1" width="5px" height="5px"><img src="pymol_images/2063.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2063&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2063.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2064">2064</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>LSPAVMASLA</td><td>EpVP4a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCGGGTSTTC</td><td align="center"><a href="jmol.php?id=2064&amp;di=1" width="5px" height="5px"><img src="pymol_images/2064.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2064&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2064.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2065">2065</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>LSPAAMASLA</td><td>EpVP4b</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCHHHHTTCC</td><td align="center"><a href="jmol.php?id=2065&amp;di=1" width="5px" height="5px"><img src="pymol_images/2065.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2065&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2065.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2066">2066</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>VHVPPICSHRECRK</td><td>EpVP5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CBTTHHHHBCCTTC</td><td align="center"><a href="jmol.php?id=2066&amp;di=1" width="5px" height="5px"><img src="pymol_images/2066.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2066&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2066.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2067">2067</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21184791">21184791</a></td><td align="right">2011</td><td>FGPVIGLLSGILKSLL</td><td>EpVP6</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CHHHHHHHHHHGGGTC</td><td align="center"><a href="jmol.php?id=2067&amp;di=1" width="5px" height="5px"><img src="pymol_images/2067.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2067&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2067.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2068">2068</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23430306">23430306</a></td><td align="right">2013</td><td>LRRLWLRANRL</td><td>LRR-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antibacterial</td><td>0% hemolysis at 128µM (non-hemolytic)</td><td>Human</td><td>CCCCCCSSCCC</td><td align="center"><a href="jmol.php?id=2068&amp;di=1" width="5px" height="5px"><img src="pymol_images/2068.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2068&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2068.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2069">2069</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23430306">23430306</a></td><td align="right">2013</td><td>LRRLWLRANRLLRRLWLRANRL</td><td>LRR-2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antibacterial</td><td>~70% hemolysis at 128µM</td><td>Human</td><td>CCTTHHHHHHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=2069&amp;di=1" width="5px" height="5px"><img src="pymol_images/2069.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2069&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2069.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2070">2070</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23594231">23594231</a></td><td align="right">2013</td><td>RKCNFLCKLKEKLRTVITSHIDKVLRPQG</td><td>Cathelicidin-PY</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>2.5% hemolysis at 12.5µg/ml</td><td>Human</td><td>CCCCSCGGGHHHHTTTSGGGSCGGGCCCC</td><td align="center"><a href="jmol.php?id=2070&amp;di=1" width="5px" height="5px"><img src="pymol_images/2070.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2070&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2070.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2071">2071</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23594231">23594231</a></td><td align="right">2013</td><td>RKCNFLCKLKEKLRTVITSHIDKVLRPQG</td><td>Cathelicidin-PY</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>3.5% hemolysis at 25µg/ml</td><td>Human</td><td>CCCCSCGGGHHHHTTTSGGGSCGGGCCCC</td><td align="center"><a href="jmol.php?id=2071&amp;di=1" width="5px" height="5px"><img src="pymol_images/2071.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2071&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2071.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2072">2072</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23594231">23594231</a></td><td align="right">2013</td><td>RKCNFLCKLKEKLRTVITSHIDKVLRPQG</td><td>Cathelicidin-PY</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>5.5% hemolysis at 50µg/ml</td><td>Human</td><td>CCCCSCGGGHHHHTTTSGGGSCGGGCCCC</td><td align="center"><a href="jmol.php?id=2072&amp;di=1" width="5px" height="5px"><img src="pymol_images/2072.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2072&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2072.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2074">2074</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16730966">16730966</a></td><td align="right">2006</td><td>QPTRRPRPGTGPGRRPRPRPRP</td><td>QPT22</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolysis at 60µM</td><td>Human</td><td>CCCCCCCCCSTTTTCCCCCCCC</td><td align="center"><a href="jmol.php?id=2074&amp;di=1" width="5px" height="5px"><img src="pymol_images/2074.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2074&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2074.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2075">2075</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16730966">16730966</a></td><td align="right">2006</td><td>KRFKQDGGWSHWSPWSS</td><td>KRF17</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 60µM</td><td>Human</td><td>CCSSCCSSSCCCCSCCC</td><td align="center"><a href="jmol.php?id=2075&amp;di=1" width="5px" height="5px"><img src="pymol_images/2075.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2075&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2075.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2076">2076</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22426384">22426384</a></td><td align="right">2012</td><td>GAFGDLLKGVAKEAGMKLLNMAQCKLSGKC</td><td>Brevinin-2LTa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>LD<sub>50</sub> =520µM</td><td>Human</td><td>CTTSCHHHHHHHHTHHHHHTTTSCCCSSCC</td><td align="center"><a href="jmol.php?id=2076&amp;di=1" width="5px" height="5px"><img src="pymol_images/2076.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2076&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2076.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2077">2077</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22426384">22426384</a></td><td align="right">2012</td><td>SILDKIKNVALGVARGAGTGILKALLCKLDKSC</td><td>Brevinin-2LTb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>LD<sub>50</sub> =480µM</td><td>Human</td><td>CCSSSTHHHHSCCSSCCSCGGGGTTTSSSCCCC</td><td align="center"><a href="jmol.php?id=2077&amp;di=1" width="5px" height="5px"><img src="pymol_images/2077.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2077&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2077.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2078">2078</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22426384">22426384</a></td><td align="right">2012</td><td>GVLDTFKDVAIGVAKGAGTGVLKALLCKLDKSC</td><td>Brevinin-2LTc</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>LD<sub>50</sub> &gt;480µM</td><td>Human</td><td>CTTTTSCSSCCSSCSTTCSCHHHHTTCSSCTTC</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2081">2081</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22426384">22426384</a></td><td align="right">2012</td><td>SLWENFKNAGKQFILNILDKIRCRVAGGCRT</td><td>Palustrin-2LTa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">31</td><td>Antimicrobial</td><td>LD<sub>50</sub> =220µM</td><td>Human</td><td>CHHHHHHHHHHHHHHHHHHHTTCSSCSSSCC</td><td align="center"><a href="jmol.php?id=2081&amp;di=1" width="5px" height="5px"><img src="pymol_images/2081.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2081&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2081.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2082">2082</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22426384">22426384</a></td><td align="right">2012</td><td>FLAGLIGGLAKMLGK</td><td>Temporin-LTe</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>LD<sub>50</sub> =40µM</td><td>Human</td><td>CCSHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2082&amp;di=1" width="5px" height="5px"><img src="pymol_images/2082.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2082&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2082.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2083">2083</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22464970">22464970</a></td><td align="right">2012</td><td>KWFRVYRGIYRRR</td><td>CDT</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolysis at 200µg/ml (non-hemolytic)</td><td>Human</td><td>CCCCCCTTTCCCC</td><td align="center"><a href="jmol.php?id=2083&amp;di=1" width="5px" height="5px"><img src="pymol_images/2083.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2083&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2083.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2084">2084</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22484288">22484288</a></td><td align="right">2012</td><td>FLFSLIPHAIGGLISAFK</td><td>AamAP1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>0%hemolysis at 10µM</td><td>Horse</td><td>CCCSSSCTTTCBTTBCCC</td><td align="center"><a href="jmol.php?id=2084&amp;di=1" width="5px" height="5px"><img src="pymol_images/2084.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2084&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2084.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2085">2085</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22484288">22484288</a></td><td align="right">2012</td><td>FLFSLIPHAIGGLISAFK</td><td>AamAP1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>100%hemolysis at 100µM</td><td>Horse</td><td>CCCSSSCTTTCBTTBCCC</td><td align="center"><a href="jmol.php?id=2085&amp;di=1" width="5px" height="5px"><img src="pymol_images/2085.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2085&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2085.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2086">2086</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22484288">22484288</a></td><td align="right">2012</td><td>FPFSLIPHAIGGLISAIK</td><td>AamAP2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>19%hemolysis at 10µM</td><td>Horse</td><td>CCSSSCCTTTCBTTBCCC</td><td align="center"><a href="jmol.php?id=2086&amp;di=1" width="5px" height="5px"><img src="pymol_images/2086.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2086&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2086.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2087">2087</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22484288">22484288</a></td><td align="right">2012</td><td>FPFSLIPHAIGGLISAIK</td><td>AamAP2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>75%hemolysis at 120µM</td><td>Horse</td><td>CCSSSCCTTTCBTTBCCC</td><td align="center"><a href="jmol.php?id=2087&amp;di=1" width="5px" height="5px"><img src="pymol_images/2087.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2087&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2087.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2088">2088</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22484288">22484288</a></td><td align="right">2012</td><td>FLFSLIPKAIGGLISAFK</td><td>AamAP-S1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>17.8%hemolysis at 10µM</td><td>Horse</td><td>CCCCSSGGGGCSSCCCCC</td><td align="center"><a href="jmol.php?id=2088&amp;di=1" width="5px" height="5px"><img src="pymol_images/2088.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2088&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2088.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2089">2089</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22484288">22484288</a></td><td align="right">2012</td><td>FLFSLIPKAIGGLISAFK</td><td>AamAP-S1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>53% hemolysis at 120µM</td><td>Horse</td><td>CCCCSSGGGGCSSCCCCC</td><td align="center"><a href="jmol.php?id=2089&amp;di=1" width="5px" height="5px"><img src="pymol_images/2089.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2089&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2089.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2090">2090</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22497805">22497805</a></td><td align="right">2012</td><td>IKLSPETKDNLKKVLKGAIKGAIAVAKMV</td><td>Hymenochirin-1B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>LC<sub>50</sub> =225µM</td><td>Human</td><td>CCCCCCCCCHHHHHHHHHHHHTTTTSCCC</td><td align="center"><a href="jmol.php?id=2090&amp;di=1" width="5px" height="5px"><img src="pymol_images/2090.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2090&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2090.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2091">2091</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22497805">22497805</a></td><td align="right">2012</td><td>LKIPGFVKDTLKKVAKGIFSAVAGAMTPS</td><td>Hymenochirin-2B</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;300µM</td><td>Human</td><td>CCCCTTTHHHHHHHHHHHHHHCGGGCCCC</td><td align="center"><a href="jmol.php?id=2091&amp;di=1" width="5px" height="5px"><img src="pymol_images/2091.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2091&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2091.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2092">2092</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22497805">22497805</a></td><td align="right">2012</td><td>IKIPAVVKDTLKKVAKGVLSAVAGALTQ</td><td>Hymenochirin-3B</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;300µM</td><td>Human</td><td>CCCTTTHHHHHHHHHHHHTTSEETTEEC</td><td align="center"><a href="jmol.php?id=2092&amp;di=1" width="5px" height="5px"><img src="pymol_images/2092.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2092&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2092.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2093">2093</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22497805">22497805</a></td><td align="right">2012</td><td>IKIPAFVKDTLKKVAKGVISAVAGALTQ</td><td>Hymenochirin-4B</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>LC<sub>50</sub> =160µM</td><td>Human</td><td>CCSGGGGHHHHHHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2093&amp;di=1" width="5px" height="5px"><img src="pymol_images/2093.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2093&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2093.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2094">2094</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22497805">22497805</a></td><td align="right">2012</td><td>IKIPPIVKDTLKKVAKGVLSTIAGALST</td><td>Hymenochirin-5B</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CCSCSSHHHHHHHHHHHHTTGGGGGTCC</td><td align="center"><a href="jmol.php?id=2094&amp;di=1" width="5px" height="5px"><img src="pymol_images/2094.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2094&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2094.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2095">2095</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168889">11168889</a></td><td align="right">2000</td><td>PKLLKTFLSKWIG</td><td>SPFK</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolysis at 30µg/mL</td><td>Rat</td><td>CCGGGTTSSTTCC</td><td align="center"><a href="jmol.php?id=2095&amp;di=1" width="5px" height="5px"><img src="pymol_images/2095.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2095&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2095.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2096">2096</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168889">11168889</a></td><td align="right">2000</td><td>GIWKSLFTKLLKG</td><td>Retro SPFK (RSac)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>95-100% hemolysis at 60µg/mL</td><td>Rat</td><td>CHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2096&amp;di=1" width="5px" height="5px"><img src="pymol_images/2096.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2096&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2096.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2097">2097</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168889">11168889</a></td><td align="right">2000</td><td>GIWKSLFTKLLKG</td><td>Retro SPFK amide (RSam)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>95-100% hemolysis at 60µg/mL</td><td>Rat</td><td>CHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2097&amp;di=1" width="5px" height="5px"><img src="pymol_images/2097.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2097&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2097.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2098">2098</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168889">11168889</a></td><td align="right">2000</td><td>PKLlKTFlSKWIG</td><td>Diastereo SPFK(DSac)</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>~10% hemolsis at 60µg/mL</td><td>Rat</td><td>CCGGGTTSSTTCC</td><td align="center"><a href="jmol.php?id=2098&amp;di=1" width="5px" height="5px"><img src="pymol_images/2098.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2098&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2098.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2099">2099</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168889">11168889</a></td><td align="right">2000</td><td>PKLlETFlSKWIG</td><td>Diastereo SPFK amide (DSam)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>~40% hemolysis at 60µg/mL</td><td>Rat</td><td>CCHHHHHTTTTCC</td><td align="center"><a href="jmol.php?id=2099&amp;di=1" width="5px" height="5px"><img src="pymol_images/2099.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2099&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2099.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2100">2100</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>GWGSFFKKAAHVGKHVGKAALTHYL</td><td>Pleurocidin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>6% hemolysis at 100µM</td><td>Rabbit</td><td>CCHHHHTTTHHHHHHHTSGGGSCCC</td><td align="center"><a href="jmol.php?id=2100&amp;di=1" width="5px" height="5px"><img src="pymol_images/2100.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2100&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2100.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2101">2101</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>GWGSFFKKAAHVGKHVGKAALTHYL</td><td>Pleurocidin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>15% hemolysis at 100µM</td><td>Human</td><td>CCHHHHTTTHHHHHHHTSGGGSCCC</td><td align="center"><a href="jmol.php?id=2101&amp;di=1" width="5px" height="5px"><img src="pymol_images/2101.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2101&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2101.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2102">2102</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>AWASFFKKAAHVGKHVGKAALTHYL</td><td>[A<sup>1,3</sup>]Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>4% hemolysis at 100µM</td><td>Rabbit</td><td>CHHHHHHHHHHHHHHHCTTSSSSCC</td><td align="center"><a href="jmol.php?id=2102&amp;di=1" width="5px" height="5px"><img src="pymol_images/2102.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2102&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2102.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2103">2103</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>AWASFFKKAAHVGKHVGKAALTHYL</td><td>[A<sup>1,3</sup>]Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>8% hemolysis at 100µM</td><td>Human</td><td>CHHHHHHHHHHHHHHHCTTSSSSCC</td><td align="center"><a href="jmol.php?id=2103&amp;di=1" width="5px" height="5px"><img src="pymol_images/2103.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2103&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2103.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2104">2104</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>GWGSFFKKAAHVAKHVAKAALTHYL</td><td>[A<sup>13,17</sup>]Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>50% hemolysis at 100µM</td><td>Rabbit</td><td>CTTTTTHHHHSTHHHHTTSSTTCCC</td><td align="center"><a href="jmol.php?id=2104&amp;di=1" width="5px" height="5px"><img src="pymol_images/2104.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2104&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2104.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2105">2105</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>GWGSFFKKAAHVAKHVAKAALTHYL</td><td>[A<sup>13,17</sup>]Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>93% hemolysis at 100µM</td><td>Human</td><td>CTTTTTHHHHSTHHHHTTSSTTCCC</td><td align="center"><a href="jmol.php?id=2105&amp;di=1" width="5px" height="5px"><img src="pymol_images/2105.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2105&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2105.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2106">2106</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>AWASFFKKAAHVAKHVAKAALTHYL</td><td>[A<sup>1,3,13,17</sup>]Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>45% hemolysis at 100µM</td><td>Rabbit</td><td>CHHHHTHHHHHHHHHHHHHHHSSCC</td><td align="center"><a href="jmol.php?id=2106&amp;di=1" width="5px" height="5px"><img src="pymol_images/2106.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2106&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2106.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2107">2107</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>AWASFFKKAAHVAKHVAKAALTHYL</td><td>[A<sup>1,3,13,17</sup>]Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>90% hemolysis at 100µM</td><td>Human</td><td>CHHHHTHHHHHHHHHHHHHHHSSCC</td><td align="center"><a href="jmol.php?id=2107&amp;di=1" width="5px" height="5px"><img src="pymol_images/2107.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2107&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2107.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2108">2108</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>GWGSFFKKAAAVGKAVGKAALTAYL</td><td>[A<sup>11,15,23</sup>]Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>15% hemolysis at 100µM</td><td>Rabbit</td><td>CTTHHHHTTHHHHHHHHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2108&amp;di=1" width="5px" height="5px"><img src="pymol_images/2108.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2108&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2108.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2109">2109</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11168895">11168895</a></td><td align="right">2000</td><td>GWGSFFKKAAAVGKAVGKAALTAYL</td><td>[A<sup>11,15,23</sup>]Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>45% hemolysis at 100µM</td><td>Human</td><td>CTTHHHHTTHHHHHHHHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2109&amp;di=1" width="5px" height="5px"><img src="pymol_images/2109.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2109&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2109.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2110">2110</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11226440">11226440</a></td><td align="right">2001</td><td>VLSAADKGNVKAAWGKVGGHAAE</td><td>α 1-23 peptide</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>0% hemolysis upto 2.013mM (non-hemolytic)</td><td>Bovine</td><td>CCCSSCCCBTTBTSSSSSSCSCC</td><td align="center"><a href="jmol.php?id=2110&amp;di=1" width="5px" height="5px"><img src="pymol_images/2110.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2110&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2110.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2111">2111</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>WLGSALKIGAKLLPSVVGLFKKKKQ</td><td>Ponericin-W1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition (2 mm) at 0.4-0.5mM</td><td>Horse</td><td>CCCCTHHHHTTHHHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=2111&amp;di=1" width="5px" height="5px"><img src="pymol_images/2111.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2111&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2111.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2112">2112</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>WLGSALKIGAKLLPSVVGLFKKKKQ</td><td>Ponericin-W1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition (1 mm) at 0.4-0.5mM</td><td>Sheep</td><td>CCCCTHHHHTTHHHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=2112&amp;di=1" width="5px" height="5px"><img src="pymol_images/2112.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2112&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2112.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2113">2113</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>GIWGTLAKIGIKAVPRVISMLKKKKQ</td><td>Ponericin-W3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition (1 mm) at 0.4-0.5mM</td><td>Horse</td><td>CCCGGGHHHHTSCCHHHHHHHTSCCC</td><td align="center"><a href="jmol.php?id=2113&amp;di=1" width="5px" height="5px"><img src="pymol_images/2113.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2113&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2113.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2114">2114</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>GIWGTLAKIGIKAVPRVISMLKKKKQ</td><td>Ponericin-W3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Sheep</td><td>CCCGGGHHHHTSCCHHHHHHHTSCCC</td><td align="center"><a href="jmol.php?id=2114&amp;di=1" width="5px" height="5px"><img src="pymol_images/2114.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2114&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2114.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2115">2115</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>GIWGTALKWGVKLLPKLVGMAQTKKQ</td><td>Ponericin-W4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition (2 mm) at 0.4-0.5mM</td><td>Horse</td><td>CTTSCSSGGGGGTHHHHTTSSCCCCC</td><td align="center"><a href="jmol.php?id=2115&amp;di=1" width="5px" height="5px"><img src="pymol_images/2115.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2115&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2115.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2116">2116</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>GIWGTALKWGVKLLPKLVGMAQTKKQ</td><td>Ponericin-W4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Sheep</td><td>CTTSCSSGGGGGTHHHHTTSSCCCCC</td><td align="center"><a href="jmol.php?id=2116&amp;di=1" width="5px" height="5px"><img src="pymol_images/2116.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2116&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2116.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2117">2117</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>FWGALIKGAAKLIPSVVGLFKKKQ</td><td>Ponericin-W5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition (4 mm) at 0.4-0.5mM</td><td>Horse</td><td>CGGGTTTTTSTTCGGGTSSCCCCC</td><td align="center"><a href="jmol.php?id=2117&amp;di=1" width="5px" height="5px"><img src="pymol_images/2117.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2117&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2117.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2118">2118</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>FWGALIKGAAKLIPSVVGLFKKKQ</td><td>Ponericin-W5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition (1.5 mm) at 0.4-0.5mM</td><td>Sheep</td><td>CGGGTTTTTSTTCGGGTSSCCCCC</td><td align="center"><a href="jmol.php?id=2118&amp;di=1" width="5px" height="5px"><img src="pymol_images/2118.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2118&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2118.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2119">2119</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>FIGTALGIASAIPAIVKLFK</td><td>Ponericin-W6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition (3 mm) at 0.4-0.5mM</td><td>Horse</td><td>CCCCSEETTEEHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2119&amp;di=1" width="5px" height="5px"><img src="pymol_images/2119.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2119&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2119.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2120">2120</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>FIGTALGIASAIPAIVKLFK</td><td>Ponericin-W6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition (1 mm) at 0.4-0.5mM</td><td>Sheep</td><td>CCCCSEETTEEHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2120&amp;di=1" width="5px" height="5px"><img src="pymol_images/2120.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2120&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2120.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2121">2121</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>GWKDWAKKAGGWLKKKGPGMAKAALKAAMQ</td><td>Ponericin-G1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Horse</td><td>CCTTHHHHSSSCCCSSCTTTTTTTGGGTCC</td><td align="center"><a href="jmol.php?id=2121&amp;di=1" width="5px" height="5px"><img src="pymol_images/2121.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2121&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2121.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2122">2122</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>GWKDWAKKAGGWLKKKGPGMAKAALKAAMQ</td><td>Ponericin-G1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Sheep</td><td>CCTTHHHHSSSCCCSSCTTTTTTTGGGTCC</td><td align="center"><a href="jmol.php?id=2122&amp;di=1" width="5px" height="5px"><img src="pymol_images/2122.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2122&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2122.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2123">2123</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>GWKDWLNKGKEWLKKKGPGIMKAALKAATQ</td><td>Ponericin-G3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Horse</td><td>CCTTTTTHHHHHHHTSSCCTHHHHTTSTTC</td><td align="center"><a href="jmol.php?id=2123&amp;di=1" width="5px" height="5px"><img src="pymol_images/2123.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2123&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2123.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2124">2124</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>GWKDWLNKGKEWLKKKGPGIMKAALKAATQ</td><td>Ponericin-G3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Sheep</td><td>CCTTTTTHHHHHHHTSSCCTHHHHTTSTTC</td><td align="center"><a href="jmol.php?id=2124&amp;di=1" width="5px" height="5px"><img src="pymol_images/2124.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2124&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2124.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2125">2125</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>DFKDWMKTAGEWLKKKGPGILKAAMAAAT</td><td>Ponericin-G4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Horse</td><td>CTTTGGGTHHHHTTTTSSCSTHHHHTSCC</td><td align="center"><a href="jmol.php?id=2125&amp;di=1" width="5px" height="5px"><img src="pymol_images/2125.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2125&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2125.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2126">2126</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>DFKDWMKTAGEWLKKKGPGILKAAMAAAT</td><td>Ponericin-G4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Sheep</td><td>CTTTGGGTHHHHTTTTSSCSTHHHHTSCC</td><td align="center"><a href="jmol.php?id=2126&amp;di=1" width="5px" height="5px"><img src="pymol_images/2126.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2126&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2126.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2127">2127</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>GLVDVLGKVGGLIKKLLP</td><td>Ponericin-G6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Horse</td><td>CCCCCCGGGSTTTTTSCC</td><td align="center"><a href="jmol.php?id=2127&amp;di=1" width="5px" height="5px"><img src="pymol_images/2127.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2127&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2127.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2128">2128</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>GLVDVLGKVGGLIKKLLP</td><td>Ponericin-G6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Sheep</td><td>CCCCCCGGGSTTTTTSCC</td><td align="center"><a href="jmol.php?id=2128&amp;di=1" width="5px" height="5px"><img src="pymol_images/2128.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2128&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2128.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2129">2129</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>LLKELWTKIKGAGKAVLGKIKGLL</td><td>Ponericin-L2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Horse</td><td>CHHHHHHHTTTTTSHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=2129&amp;di=1" width="5px" height="5px"><img src="pymol_images/2129.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2129&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2129.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2130">2130</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11279030">11279030</a></td><td align="right">2001</td><td>LLKELWTKIKGAGKAVLGKIKGLL</td><td>Ponericin-L2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial and Insecticidal</td><td>Zone of inhibition not detected at 0.4-0.5mM</td><td>Sheep</td><td>CHHHHHHHTTTTTSHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=2130&amp;di=1" width="5px" height="5px"><img src="pymol_images/2130.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2130&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2130.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2131">2131</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11557475">11557475</a></td><td align="right">2001</td><td>GFKLKGMARISCLPNGQWSNFPPKCIRECAMVSS</td><td>S1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antibacterial</td><td>0% hemolysis at 100µg/ml (non-hemolytic)</td><td>Human</td><td>CCCCCCCCCCCCCSCSSSSCCCTTTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2131&amp;di=1" width="5px" height="5px"><img src="pymol_images/2131.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2131&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2131.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2132">2132</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11557475">11557475</a></td><td align="right">2001</td><td>GFKLKGKAKISCLPNGQWSNFPPKCIRECAMVSS</td><td>S1Δ</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antibacterial</td><td>0% hemolysis at 100µg/ml (non-hemolytic)</td><td>Human</td><td>CCCCCCCCCCCSSCCSSCCCCCGGGHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2132&amp;di=1" width="5px" height="5px"><img src="pymol_images/2132.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2132&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2132.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2133">2133</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11557475">11557475</a></td><td align="right">2001</td><td>HAEHKVKIGVEQKYGQFPQGTEVTYTCSGNYFLM</td><td>S3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antibacterial</td><td>8% hemolysis at 100µg/ml</td><td>Human</td><td>CCSCSCSCSCSCCSCSSSCSSCSSCCCSSSCCCC</td><td align="center"><a href="jmol.php?id=2133&amp;di=1" width="5px" height="5px"><img src="pymol_images/2133.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2133&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2133.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2134">2134</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11557475">11557475</a></td><td align="right">2001</td><td>HAEHKVKIKVKQKYGQFPQGTEVTYTCSGNYFLM</td><td>S3Δ</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antibacterial</td><td>37% hemolysis at 100µg/ml</td><td>Human</td><td>CCSCSSCCCCBCSSSSSBCCCCTTTTCSSSCCCC</td><td align="center"><a href="jmol.php?id=2134&amp;di=1" width="5px" height="5px"><img src="pymol_images/2134.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2134&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2134.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2135">2135</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Nle-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P19(9|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">19</td><td>Antimicrobial</td><td>30% hemolysis at 10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2136">2136</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Nle-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P19(9|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">19</td><td>Antimicrobial</td><td>70% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2137">2137</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P17(9|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">17</td><td>Antimicrobial</td><td>15% hemolysis at 10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2138">2138</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P17(9|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">17</td><td>Antimicrobial</td><td>35% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2139">2139</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Aib-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P15(7|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">15</td><td>Antimicrobial</td><td>5% hemolysis at 10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2140">2140</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Aib-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P15(7|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">15</td><td>Antimicrobial</td><td>30% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2141">2141</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Aib-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-GY</td><td>P14(7|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">14</td><td>Antimicrobial</td><td>10% hemolysis at 10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2142">2142</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Aib-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-GY</td><td>P14(7|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">14</td><td>Antimicrobial</td><td>30% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2143">2143</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P13(7|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">13</td><td>Antimicrobial</td><td>5% hemolysis at 10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2144">2144</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Orn-Orn-Aib-Orn-Orn-Aib-Nle-Orn-Orn-Nle-GY</td><td>P13(7|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">13</td><td>Antimicrobial</td><td>25% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2145">2145</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Aib-Orn-Orn-Aib-Aib-Orn-Orn-Nle-Nle-GY</td><td>P13(5|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">13</td><td>Antimicrobial</td><td>12% hemolysis at 10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2146">2146</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11683882">11683882</a></td><td align="right">2001</td><td>G-Nle-Aib-Orn-Orn-Nle-Aib-Orn-Orn-Nle-Nle-Orn-GY</td><td>P14(6|B)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aib= α-aminoisobutyric acid, Nle= norleucine, Orn= ornithine</td><td align="right">14</td><td>Antimicrobial</td><td>70% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2147">2147</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11892852">11892852</a></td><td align="right">2001</td><td>FLPLLAGLAANFLPKIFCKITRKC</td><td>Brevinin 1E (B)</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">24</td><td>Antibacterial</td><td>100% hemolysis at 0.7μM</td><td>Rat</td><td>CCHHHHHHHHHHHHHHHHHHCSCC</td><td align="center"><a href="jmol.php?id=2147&amp;di=1" width="5px" height="5px"><img src="pymol_images/2147.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2147&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2147.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2148">2148</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11892852">11892852</a></td><td align="right">2001</td><td>FLPLLAGLAANFLPKIFC-Acm-KITRKC-Acm</td><td>Brevinin1E linear (BL)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>Acm= acetamidomethyl</td><td align="right">26</td><td>Antibacterial</td><td>100% hemolysisat 5μM</td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2149">2149</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11892852">11892852</a></td><td align="right">2001</td><td>FLPLLAGLCKITRKCAANFLPKIF</td><td>Analog of brevinin 1E (BA)</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">24</td><td>Antibacterial</td><td>100% hemolysis at 7μM</td><td>Rat</td><td>CCTTTSHHHHHHHHHTTCCCTTCC</td><td align="center"><a href="jmol.php?id=2149&amp;di=1" width="5px" height="5px"><img src="pymol_images/2149.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2149&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2149.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2150">2150</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11892852">11892852</a></td><td align="right">2001</td><td>FLPLLAGLC-Acm-KITRKC-Acm-AANFLPKIF</td><td>Linear analog of brevinin 1E (BAL)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>Acm=acetamidomethyl</td><td align="right">26</td><td>Antibacterial</td><td>Non-hemolytic</td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2151">2151</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11995990">11995990</a></td><td align="right">2001</td><td>FLPLIGRVLSGIL</td><td>Temporin A (TA)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antifungal</td><td>0% hemolysis at 14µM</td><td>Human</td><td>CCCCHHHHTSCCC</td><td align="center"><a href="jmol.php?id=2151&amp;di=1" width="5px" height="5px"><img src="pymol_images/2151.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2151&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2151.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2152">2152</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11995990">11995990</a></td><td align="right">2001</td><td>FLPLIGRVLSGIL</td><td>Temporin A (TA)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antifungal</td><td>100% hemolysis at 225µM</td><td>Human</td><td>CCCCHHHHTSCCC</td><td align="center"><a href="jmol.php?id=2152&amp;di=1" width="5px" height="5px"><img src="pymol_images/2152.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2152&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2152.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2153">2153</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11995990">11995990</a></td><td align="right">2001</td><td>KWKLFKKLPLIGRVL</td><td>CA(1-7)TA(2-9)NH<sub>2</sub> (CATA)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antifungal</td><td>0% hemolysis at 14µM</td><td>Human</td><td>CCCCCSSCSCCCCCC</td><td align="center"><a href="jmol.php?id=2153&amp;di=1" width="5px" height="5px"><img src="pymol_images/2153.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2153&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2153.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2154">2154</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11995990">11995990</a></td><td align="right">2001</td><td>KWKLFKKLPLIGRVL</td><td>CA(1-7)TA(2-9)NH<sub>2</sub> (CATA)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antifungal</td><td>3% hemolysis at 130µM</td><td>Human</td><td>CCCCCSSCSCCCCCC</td><td align="center"><a href="jmol.php?id=2154&amp;di=1" width="5px" height="5px"><img src="pymol_images/2154.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2154&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2154.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2155">2155</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKFKKIPKFLHLAKKF</td><td>P18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2156">2156</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKLFKKILKFLHLAKKF</td><td>[L<sup>9</sup>]-P18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>88.6% hemolysis at 100µM</td><td>Human</td><td>CCTTHHHHHHHHTTTTCC</td><td align="center"><a href="jmol.php?id=2156&amp;di=1" width="5px" height="5px"><img src="pymol_images/2156.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2156&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2156.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2157">2157</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKLFKKISKFLHLAKKF</td><td>[S<sup>9</sup>]-P18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>50.4% hemolysis at 100µM</td><td>Human</td><td>CCTTHHHHHHHHTCTTCC</td><td align="center"><a href="jmol.php?id=2157&amp;di=1" width="5px" height="5px"><img src="pymol_images/2157.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2157&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2157.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2158">2158</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>WKLFKKIPKFLHLAKKF</td><td>N-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antibacterial</td><td>10.1% hemolysis at 100µM</td><td>Human</td><td>CCSSSCCTTGGGSSSCC</td><td align="center"><a href="jmol.php?id=2158&amp;di=1" width="5px" height="5px"><img src="pymol_images/2158.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2158&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2158.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2159">2159</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>FKLFKKIPKFLHLAKKF</td><td>N-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antibacterial</td><td>0.6% hemolysis at 100µM</td><td>Human</td><td>CCCCCCCHHHHTCSSCC</td><td align="center"><a href="jmol.php?id=2159&amp;di=1" width="5px" height="5px"><img src="pymol_images/2159.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2159&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2159.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2160">2160</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWFKKIPKFLHLAKKF</td><td>N-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antibacterial</td><td>1.8% hemolysis at 100µM</td><td>Human</td><td>CCTTHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2160&amp;di=1" width="5px" height="5px"><img src="pymol_images/2160.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2161">2161</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>WFKKIPKFLHLAKKF</td><td>N-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>1.5% hemolysis at 100µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2162">2162</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>WKKIPKFLHLAKKF</td><td>N-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCCCTHHHHTTCC</td><td align="center"><a href="jmol.php?id=2162&amp;di=1" width="5px" height="5px"><img src="pymol_images/2162.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2162&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2162.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2163">2163</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWFKKIPKFLHLLKKF</td><td>N-3L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antibacterial</td><td>27.3% hemolysis at 100µM</td><td>Human</td><td>CCTTSTTHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2163&amp;di=1" width="5px" height="5px"><img src="pymol_images/2163.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2163&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2163.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2164">2164</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>WFKKIPKFLHLLKKF</td><td>N-4L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>28.8% hemolysis at 100µM</td><td>Human</td><td>CCCCCTHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2164&amp;di=1" width="5px" height="5px"><img src="pymol_images/2164.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2164&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2164.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2165">2165</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>WKKIPKFLHLLKKF</td><td>N-5L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antibacterial</td><td>9.4% hemolysis at 100µM</td><td>Human</td><td>CCCCGGGHHHHTCC</td><td align="center"><a href="jmol.php?id=2165&amp;di=1" width="5px" height="5px"><img src="pymol_images/2165.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2165&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2165.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2166">2166</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKLFKKIPFLHLAKKF</td><td>C-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antibacterial</td><td>30.3% hemolysis at 100µM</td><td>Human</td><td>CCCCTTSCCHHHHTCCC</td><td align="center"><a href="jmol.php?id=2166&amp;di=1" width="5px" height="5px"><img src="pymol_images/2166.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2167">2167</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKLFKKIPKFLHLAKK</td><td>C-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antibacterial</td><td>19.8% hemolysis at 100µM</td><td>Human</td><td>CCCCCCCCGGGTCSSCC</td><td align="center"><a href="jmol.php?id=2167&amp;di=1" width="5px" height="5px"><img src="pymol_images/2167.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2167&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2167.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2168">2168</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKLFKKIPLHLAKKF</td><td>C-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antibacterial</td><td>4.7% hemolysis at 100µM</td><td>Human</td><td>CCCCSSCCSCCGGGGC</td><td align="center"><a href="jmol.php?id=2168&amp;di=1" width="5px" height="5px"><img src="pymol_images/2168.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2168&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2168.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2169">2169</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKLFKKIPKFLHLAK</td><td>C-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTHHHHTTCCSCCCC</td><td align="center"><a href="jmol.php?id=2169&amp;di=1" width="5px" height="5px"><img src="pymol_images/2169.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2170">2170</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKLFKKIPHLAKKF</td><td>C-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCCCCCTHHHHTCC</td><td align="center"><a href="jmol.php?id=2170&amp;di=1" width="5px" height="5px"><img src="pymol_images/2170.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2170&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2170.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2171">2171</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKLFKKIPKFLHLA</td><td>C-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>6.7% hemolysis at 100µM</td><td>Human</td><td>CCSSCCCTTSCSCCC</td><td align="center"><a href="jmol.php?id=2171&amp;di=1" width="5px" height="5px"><img src="pymol_images/2171.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2171&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2171.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2172">2172</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKLFKKIPLAKKF</td><td>C-7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCCCCSSCCCCCC</td><td align="center"><a href="jmol.php?id=2172&amp;di=1" width="5px" height="5px"><img src="pymol_images/2172.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2173">2173</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKLFKKIPKFLHL</td><td>C-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCCCCCTTSCTTC</td><td align="center"><a href="jmol.php?id=2173&amp;di=1" width="5px" height="5px"><img src="pymol_images/2173.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2173&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2173.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2174">2174</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKLFKKIPLKKF</td><td>C-9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>6.2% hemolysis at 100µM</td><td>Human</td><td>CCSCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2174&amp;di=1" width="5px" height="5px"><img src="pymol_images/2174.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2175">2175</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005420">12005420</a></td><td align="right">2001</td><td>KWKLFKKIPKFLH</td><td>C-10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>7.6% hemolysis at 100µM</td><td>Human</td><td>CCSSCCCGGGTCC</td><td align="center"><a href="jmol.php?id=2175&amp;di=1" width="5px" height="5px"><img src="pymol_images/2175.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2175&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2175.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2176">2176</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11932493">11932493</a></td><td align="right">2002</td><td>KNLRRIIRKIIHIIKKYG</td><td>Ovispirin-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>70.2% hemolysis at 80μg/ml</td><td>Human</td><td>CCTHHHHTTTHHHHCCCC</td><td align="center"><a href="jmol.php?id=2176&amp;di=1" width="5px" height="5px"><img src="pymol_images/2176.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2176&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2176.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2177">2177</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11932493">11932493</a></td><td align="right">2002</td><td>KNLRRIIRKIIHIIKKYG</td><td>Ovispirin-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>55.0% hemolysis at 40μg/ml</td><td>Human</td><td>CCTHHHHTTTHHHHCCCC</td><td align="center"><a href="jmol.php?id=2177&amp;di=1" width="5px" height="5px"><img src="pymol_images/2177.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2177&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2177.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2178">2178</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11932493">11932493</a></td><td align="right">2002</td><td>KNLRRIIRKGIHIIKKYG</td><td>Novispirin G-10</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>2.5% hemolysis at 80μg/ml</td><td>Human</td><td>CTHHHHGGGGGCCCCCCC</td><td align="center"><a href="jmol.php?id=2178&amp;di=1" width="5px" height="5px"><img src="pymol_images/2178.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2178&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2178.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2179">2179</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11932493">11932493</a></td><td align="right">2002</td><td>KNLRRIIRKGIHIIKKYG</td><td>Novispirin G-10</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>1.8% hemolysis at 40µg/ml</td><td>Human</td><td>CTHHHHGGGGGCCCCCCC</td><td align="center"><a href="jmol.php?id=2179&amp;di=1" width="5px" height="5px"><img src="pymol_images/2179.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2179&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2179.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2180">2180</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11932493">11932493</a></td><td align="right">2002</td><td>KNLRRITRKIIHIIKKYG</td><td>Novispirin T-7</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>10.0% hemolysis at 80μg/ml</td><td>Human</td><td>CGGGGGGHHHHTSTTCCC</td><td align="center"><a href="jmol.php?id=2180&amp;di=1" width="5px" height="5px"><img src="pymol_images/2180.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2180&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2180.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2181">2181</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11932493">11932493</a></td><td align="right">2002</td><td>KNLRRITRKIIHIIKKYG</td><td>Novispirin T-7</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>3.0% hemolysis at 40μg/ml</td><td>Human</td><td>CGGGGGGHHHHTSTTCCC</td><td align="center"><a href="jmol.php?id=2181&amp;di=1" width="5px" height="5px"><img src="pymol_images/2181.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2181&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2181.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2191">2191</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11976325">11976325</a></td><td align="right">2002</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial and Insecticidal</td><td>~100% hemolysis at 50μM</td><td>Pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2191&amp;di=1" width="5px" height="5px"><img src="pymol_images/2191.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2191&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2191.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2192">2192</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11976325">11976325</a></td><td align="right">2002</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial and Insecticidal</td><td>~100% hemolysis at 50μM</td><td>Guinea-pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2192&amp;di=1" width="5px" height="5px"><img src="pymol_images/2192.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2192&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2192.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2193">2193</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11976325">11976325</a></td><td align="right">2002</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial and Insecticidal</td><td>~100% hemolysis at 50μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2193&amp;di=1" width="5px" height="5px"><img src="pymol_images/2193.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2193&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2193.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2194">2194</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12054688">12054688</a></td><td align="right">2002</td><td>ILGKIWEGIKSLF</td><td>IsCT</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>~30% hemolysis at 200μM</td><td>Sheep</td><td>CCSTHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2194&amp;di=1" width="5px" height="5px"><img src="pymol_images/2194.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2194&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2194.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2195">2195</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12054688">12054688</a></td><td align="right">2002</td><td>ILGKIWEGIKS</td><td>IsCTf</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antibacterial and Antifungal</td><td>~0% hemolysis at 200μM (non-hemolytic)</td><td>Sheep</td><td>CGGGGCCSSCC</td><td align="center"><a href="jmol.php?id=2195&amp;di=1" width="5px" height="5px"><img src="pymol_images/2195.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2195&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2195.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2196">2196</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12054688">12054688</a></td><td align="right">2002</td><td>IFGAIWNGIKSLF</td><td>IsCT2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>&lt;20% hemolysis at 200μM</td><td>Sheep</td><td>CTTHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2196&amp;di=1" width="5px" height="5px"><img src="pymol_images/2196.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2196&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2196.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2197">2197</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12054688">12054688</a></td><td align="right">2002</td><td>IFGAIWNGIKS</td><td>IsCT2f</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antibacterial and Antifungal</td><td>~0% hemolysis at 200μM (non-hemolytic)</td><td>Sheep</td><td>CCSSSCCSTTC</td><td align="center"><a href="jmol.php?id=2197&amp;di=1" width="5px" height="5px"><img src="pymol_images/2197.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2197&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2197.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2198">2198</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12110678">12110678</a></td><td align="right">2002</td><td>LKLLKKLLKKLLKLL</td><td>Amphipathic-1L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>100% hemolysis at 100μM</td><td>Human</td><td>CTTHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=2198&amp;di=1" width="5px" height="5px"><img src="pymol_images/2198.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2198&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2198.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2199">2199</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12110678">12110678</a></td><td align="right">2002</td><td>KLKLLKLLKLLKLLK</td><td>Scrambled-4L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>82% hemolysis at 100 µM</td><td>Human</td><td>CCCTHHHHHHTGGGC</td><td align="center"><a href="jmol.php?id=2199&amp;di=1" width="5px" height="5px"><img src="pymol_images/2199.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2199&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2199.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2200">2200</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12110678">12110678</a></td><td align="right">2002</td><td>KKKLLLLLLLLLKKK</td><td>Segregated-5L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>66% hemolysis at 100 µM</td><td>Human</td><td>CCCCTHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=2200&amp;di=1" width="5px" height="5px"><img src="pymol_images/2200.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2200&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2200.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2201">2201</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12110678">12110678</a></td><td align="right">2002</td><td>LKlLKkLlkKLLkLL</td><td>Amphipathic-1D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=2201&amp;di=1" width="5px" height="5px"><img src="pymol_images/2201.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2201&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2201.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2202">2202</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12110678">12110678</a></td><td align="right">2002</td><td>LLkKLlKllLKLlKK</td><td>Amphipathic-2D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=2202&amp;di=1" width="5px" height="5px"><img src="pymol_images/2202.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2202&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2202.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2203">2203</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12110678">12110678</a></td><td align="right">2002</td><td>LLkLLkKllKKLlKL</td><td>Amphipathic-3D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTTHHHHHTTCSCC</td><td align="center"><a href="jmol.php?id=2203&amp;di=1" width="5px" height="5px"><img src="pymol_images/2203.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2203&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2203.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2204">2204</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12110678">12110678</a></td><td align="right">2002</td><td>KLkLLkLlkLLKlLK</td><td>Scrambled-4D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCTHHHHHHTGGGC</td><td align="center"><a href="jmol.php?id=2204&amp;di=1" width="5px" height="5px"><img src="pymol_images/2204.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2204&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2204.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2205">2205</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12110678">12110678</a></td><td align="right">2002</td><td>KKkLLlLllLLLkKK</td><td>Scrambled-5D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>3% hemolysis at 100µM</td><td>Human</td><td>CCCCTHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=2205&amp;di=1" width="5px" height="5px"><img src="pymol_images/2205.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2205&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2205.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2206">2206</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12110678">12110678</a></td><td align="right">2002</td><td>LLlLLkKkkKKLlLL</td><td>Scrambled-6D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCSSCGGGTCGGGCC</td><td align="center"><a href="jmol.php?id=2206&amp;di=1" width="5px" height="5px"><img src="pymol_images/2206.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2206&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2206.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2207">2207</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12207877">12207877</a></td><td align="right">2002</td><td>VRRFPWWWPFLRR</td><td>Tritrpticin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>&gt;80% hemolysis at 200µg/ml</td><td>Human</td><td>CCCCSSCSCGGGC</td><td align="center"><a href="jmol.php?id=2207&amp;di=1" width="5px" height="5px"><img src="pymol_images/2207.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2207&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2207.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2208">2208</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12207877">12207877</a></td><td align="right">2002</td><td>VRRFGWWWGFLRR</td><td>TPG</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>&gt;60% hemolysis at 200µg/ml</td><td>Human</td><td>CTTSCCCTTTSCC</td><td align="center"><a href="jmol.php?id=2208&amp;di=1" width="5px" height="5px"><img src="pymol_images/2208.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2208&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2208.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2209">2209</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12207877">12207877</a></td><td align="right">2002</td><td>VRRFAWWWAFLRR</td><td>TPA</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>~ 90% hemolysis at 200µg/ml</td><td>Human</td><td>CCSCHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2209&amp;di=1" width="5px" height="5px"><img src="pymol_images/2209.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2209&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2209.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2210">2210</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12207877">12207877</a></td><td align="right">2002</td><td>VRRFPFFFPFLRR</td><td>TWF</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>0% hemolysis at 200µg/ml (non-hemolytic)</td><td>Human</td><td>CCTTTTCCCCTTC</td><td align="center"><a href="jmol.php?id=2210&amp;di=1" width="5px" height="5px"><img src="pymol_images/2210.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2210&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2210.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2211">2211</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12207877">12207877</a></td><td align="right">2002</td><td>VRRFPAAAPFLRR</td><td>TWA</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>0% hemolysis at 200µg/ml (non-hemolytic)</td><td>Human</td><td>CCCSCCCCGGGGC</td><td align="center"><a href="jmol.php?id=2211&amp;di=1" width="5px" height="5px"><img src="pymol_images/2211.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2211&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2211.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2212">2212</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12207877">12207877</a></td><td align="right">2002</td><td>ILPWKWPWWPWRR</td><td>Indolicidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>~ 80% hemolysis at 200µg/ml</td><td>Human</td><td>CCCCCCSSCSSCC</td><td align="center"><a href="jmol.php?id=2212&amp;di=1" width="5px" height="5px"><img src="pymol_images/2212.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2212&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2212.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2213">2213</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12297012">12297012</a></td><td align="right">2002</td><td>GSKKPVPIIYCNRRTGKCQRM</td><td>Thanatin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antibacterial</td><td>0% hemolysis at 20.8µM (non-hemolytic)</td><td>Human</td><td>CBCCSSCBCCCCCCCSSCCCC</td><td align="center"><a href="jmol.php?id=2213&amp;di=1" width="5px" height="5px"><img src="pymol_images/2213.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2213&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2213.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2214">2214</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12297012">12297012</a></td><td align="right">2002</td><td>GSKKPVPIIYCNRRGKCQRM</td><td>Th1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antibacterial</td><td>0% hemolysis at 20.8µM (non-hemolytic)</td><td>Human</td><td>CCCCSSCTTSCSCCSSCCCC</td><td align="center"><a href="jmol.php?id=2214&amp;di=1" width="5px" height="5px"><img src="pymol_images/2214.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2214&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2214.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2215">2215</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12297012">12297012</a></td><td align="right">2002</td><td>GSKKPVPIIYCNRRTKCQRM</td><td>Th2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antibacterial</td><td>0% hemolysis at 20.8µM (non-hemolytic)</td><td>Human</td><td>CCCSCCCBCCSSSSSTTBCC</td><td align="center"><a href="jmol.php?id=2215&amp;di=1" width="5px" height="5px"><img src="pymol_images/2215.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2215&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2215.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2216">2216</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12297012">12297012</a></td><td align="right">2002</td><td>GSKKPVPIIYCNRRKCQRM</td><td>Th3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>0% hemolysis at 20.8µM (non-hemolytic)</td><td>Human</td><td>CCCSCCCCSCCSTTTCCCC</td><td align="center"><a href="jmol.php?id=2216&amp;di=1" width="5px" height="5px"><img src="pymol_images/2216.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2216&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2216.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2217">2217</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12297012">12297012</a></td><td align="right">2002</td><td>GSKKPVPIIYCNRRATGKCQRM</td><td>Th4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antibacterial</td><td>0% hemolysis at 20.8µM (non-hemolytic)</td><td>Human</td><td>CBCSSSCBSCCSSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=2217&amp;di=1" width="5px" height="5px"><img src="pymol_images/2217.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2217&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2217.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2218">2218</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12297012">12297012</a></td><td align="right">2002</td><td>GSKKPVPIIYCNRRAATGKCQRM</td><td>Th5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>0% hemolysis at 20.8µM (non-hemolytic)</td><td>Human</td><td>CTTCSSCCCSSCCCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2218&amp;di=1" width="5px" height="5px"><img src="pymol_images/2218.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2218&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2218.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2219">2219</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12297012">12297012</a></td><td align="right">2002</td><td>GSKKPVPIIYANRRTGKAQRM</td><td>Th6</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antibacterial</td><td>0% hemolysis at 20.8µM (non-hemolytic)</td><td>Human</td><td>CCCCCCCSEECCCSSSCCEEC</td><td align="center"><a href="jmol.php?id=2219&amp;di=1" width="5px" height="5px"><img src="pymol_images/2219.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2219&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2219.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2222">2222</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKAAAFAAAAAFAAWAAFAAAKKKK</td><td>F25</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>4% hemolysis at 200µM</td><td>Rabbit</td><td>CTTHHHHHHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2222&amp;di=1" width="5px" height="5px"><img src="pymol_images/2222.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2222&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2222.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2223">2223</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKAAAFAAAAAFAAWAAFAAAKKKK</td><td>F25</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Rabbit</td><td>CTTHHHHHHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2223&amp;di=1" width="5px" height="5px"><img src="pymol_images/2223.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2223&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2223.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2224">2224</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKAAAFAAAAAFAAWAAFAAAKKKK</td><td>F25</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>1% hemolysis at 200µM</td><td>Human</td><td>CTTHHHHHHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2224&amp;di=1" width="5px" height="5px"><img src="pymol_images/2224.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2224&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2224.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2225">2225</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKAAAFAAAAAFAAWAAFAAAKKKK</td><td>F25</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Human</td><td>CTTHHHHHHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2225&amp;di=1" width="5px" height="5px"><img src="pymol_images/2225.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2225&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2225.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2226">2226</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAAFAAAAAFAAWAAFAAA</td><td>F25-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>Not detected hemolysis at 200µM</td><td>Rabbit</td><td>CCTTHHHHHHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2226&amp;di=1" width="5px" height="5px"><img src="pymol_images/2226.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2226&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2226.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2227">2227</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAAFAAAAAFAAWAAFAAA</td><td>F25-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>Not detected hemolysis at 50µM</td><td>Rabbit</td><td>CCTTHHHHHHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2227&amp;di=1" width="5px" height="5px"><img src="pymol_images/2227.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2227&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2227.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2228">2228</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAAFAAAAAFAAWAAFAAA</td><td>F25-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>17% hemolysis at 200µM</td><td>Human</td><td>CCTTHHHHHHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2228&amp;di=1" width="5px" height="5px"><img src="pymol_images/2228.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2228&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2228.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2229">2229</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAAFAAAAAFAAWAAFAAA</td><td>F25-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>11% hemolysis at 50µM</td><td>Human</td><td>CCTTHHHHHHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2229&amp;di=1" width="5px" height="5px"><img src="pymol_images/2229.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2229&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2229.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2230">2230</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKAAAAFAAFAAWFAAFAAAAKKKK</td><td>4F</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>34% hemolysis at 200µM</td><td>Rabbit</td><td>CCTHHHHHHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2230&amp;di=1" width="5px" height="5px"><img src="pymol_images/2230.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2230&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2230.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2231">2231</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKAAAAFAAFAAWFAAFAAAAKKKK</td><td>4F</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>14% hemolysis at 50µM</td><td>Rabbit</td><td>CCTHHHHHHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2231&amp;di=1" width="5px" height="5px"><img src="pymol_images/2231.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2231&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2231.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2232">2232</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKAAAAFAAFAAWFAAFAAAAKKKK</td><td>4F</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>42% hemolysis at 200µM</td><td>Human</td><td>CCTHHHHHHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2232&amp;di=1" width="5px" height="5px"><img src="pymol_images/2232.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2232&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2232.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2233">2233</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKAAAAFAAFAAWFAAFAAAAKKKK</td><td>4F</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>17% hemolysis at 50µM</td><td>Human</td><td>CCTHHHHHHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2233&amp;di=1" width="5px" height="5px"><img src="pymol_images/2233.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2233&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2233.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2234">2234</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKAFAAAAAFAAWAAFAKKKK</td><td>F21</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>3% hemolysis at 200µM</td><td>Rabbit</td><td>CTTHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2234&amp;di=1" width="5px" height="5px"><img src="pymol_images/2234.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2234&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2234.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2235">2235</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKAFAAAAAFAAWAAFAKKKK</td><td>F21</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>1% hemolysis at 50µM</td><td>Rabbit</td><td>CTTHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2235&amp;di=1" width="5px" height="5px"><img src="pymol_images/2235.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2235&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2235.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2236">2236</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKAFAAAAAFAAWAAFAKKKK</td><td>F21</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>0% hemolysis at 200µM</td><td>Human</td><td>CTTHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2236&amp;di=1" width="5px" height="5px"><img src="pymol_images/2236.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2236&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2236.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2237">2237</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKAFAAAAAFAAWAAFAKKKK</td><td>F21</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Human</td><td>CTTHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2237&amp;di=1" width="5px" height="5px"><img src="pymol_images/2237.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2237&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2237.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2238">2238</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKAAAFAAWAAFAKKK</td><td>F17</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>2% hemolysis at 200µM</td><td>Rabbit</td><td>CHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2238&amp;di=1" width="5px" height="5px"><img src="pymol_images/2238.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2238&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2238.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2239">2239</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKAAAFAAWAAFAKKK</td><td>F17</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Rabbit</td><td>CHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2239&amp;di=1" width="5px" height="5px"><img src="pymol_images/2239.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2239&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2239.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2240">2240</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKAAAFAAWAAFAKKK</td><td>F17</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 200µM</td><td>Human</td><td>CHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2240&amp;di=1" width="5px" height="5px"><img src="pymol_images/2240.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2240&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2240.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2241">2241</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKAAAFAAWAAFAKKK</td><td>F17</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Human</td><td>CHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2241&amp;di=1" width="5px" height="5px"><img src="pymol_images/2241.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2241&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2241.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2242">2242</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>RRRAAFAAWAAFAARRR</td><td>F17-R</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>2% hemolysis at 200µM</td><td>Rabbit</td><td>CTTHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2242&amp;di=1" width="5px" height="5px"><img src="pymol_images/2242.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2242&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2242.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2243">2243</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>RRRAAFAAWAAFAARRR</td><td>F17-R</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Rabbit</td><td>CTTHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2243&amp;di=1" width="5px" height="5px"><img src="pymol_images/2243.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2243&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2243.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2244">2244</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>RRRAAFAAWAAFAARRR</td><td>F17-R</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 200µM</td><td>Human</td><td>CTTHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2244&amp;di=1" width="5px" height="5px"><img src="pymol_images/2244.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2244&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2244.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2245">2245</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>RRRAAFAAWAAFAARRR</td><td>F17-R</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Human</td><td>CTTHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2245&amp;di=1" width="5px" height="5px"><img src="pymol_images/2245.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2245&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2245.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2246">2246</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAFAAWAAFAA</td><td>F17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>1% hemolysis at 200µM</td><td>Rabbit</td><td>CGGGGGGSTTCCSCTTC</td><td align="center"><a href="jmol.php?id=2246&amp;di=1" width="5px" height="5px"><img src="pymol_images/2246.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2246&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2246.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2247">2247</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAFAAWAAFAA</td><td>F17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Rabbit</td><td>CGGGGGGSTTCCSCTTC</td><td align="center"><a href="jmol.php?id=2247&amp;di=1" width="5px" height="5px"><img src="pymol_images/2247.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2247&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2247.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2248">2248</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAFAAWAAFAA</td><td>F17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 200µM</td><td>Human</td><td>CGGGGGGSTTCCSCTTC</td><td align="center"><a href="jmol.php?id=2248&amp;di=1" width="5px" height="5px"><img src="pymol_images/2248.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2248&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2248.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2249">2249</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAFAAWAAFAA</td><td>F17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Human</td><td>CGGGGGGSTTCCSCTTC</td><td align="center"><a href="jmol.php?id=2249&amp;di=1" width="5px" height="5px"><img src="pymol_images/2249.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2249&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2249.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2250">2250</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>kkkkkkaafaawaafaa</td><td>All-D F17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>2% hemolysis at 200µM</td><td>Rabbit</td><td>CGGGGGGSTTCCSCTTC</td><td align="center"><a href="jmol.php?id=2250&amp;di=1" width="5px" height="5px"><img src="pymol_images/2250.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2250&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2250.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2251">2251</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>kkkkkkaafaawaafaa</td><td>All-D F17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Rabbit</td><td>CGGGGGGSTTCCSCTTC</td><td align="center"><a href="jmol.php?id=2251&amp;di=1" width="5px" height="5px"><img src="pymol_images/2251.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2251&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2251.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2252">2252</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>kkkkkkaafaawaafaa</td><td>All-D F17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 200µM</td><td>Human</td><td>CGGGGGGSTTCCSCTTC</td><td align="center"><a href="jmol.php?id=2252&amp;di=1" width="5px" height="5px"><img src="pymol_images/2252.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2252&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2252.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2253">2253</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>kkkkkkaafaawaafaa</td><td>All-D F17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Human</td><td>CGGGGGGSTTCCSCTTC</td><td align="center"><a href="jmol.php?id=2253&amp;di=1" width="5px" height="5px"><img src="pymol_images/2253.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2253&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2253.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2254">2254</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>RRRRRRAAFAAWAAFAA</td><td>F17-6R</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>3% hemolysis at 200µM</td><td>Rabbit</td><td>CCSGGGTSTTHHHHTTC</td><td align="center"><a href="jmol.php?id=2254&amp;di=1" width="5px" height="5px"><img src="pymol_images/2254.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2254&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2254.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2255">2255</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>RRRRRRAAFAAWAAFAA</td><td>F17-6R</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>1% hemolysis at 50µM</td><td>Rabbit</td><td>CCSGGGTSTTHHHHTTC</td><td align="center"><a href="jmol.php?id=2255&amp;di=1" width="5px" height="5px"><img src="pymol_images/2255.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2255&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2255.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2256">2256</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>RRRRRRAAFAAWAAFAA</td><td>F17-6R</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>26% hemolysis at 200µM</td><td>Human</td><td>CCSGGGTSTTHHHHTTC</td><td align="center"><a href="jmol.php?id=2256&amp;di=1" width="5px" height="5px"><img src="pymol_images/2256.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2256&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2256.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2257">2257</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>RRRRRRAAFAAWAAFAA</td><td>F17-6R</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>14% hemolysis at 50µM</td><td>Human</td><td>CCSGGGTSTTHHHHTTC</td><td align="center"><a href="jmol.php?id=2257&amp;di=1" width="5px" height="5px"><img src="pymol_images/2257.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2257&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2257.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2258">2258</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAAAFWAAAAF</td><td>KAFW</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>1% hemolysis at 200µM</td><td>Rabbit</td><td>CCSTTTTSSSHHHHSCC</td><td align="center"><a href="jmol.php?id=2258&amp;di=1" width="5px" height="5px"><img src="pymol_images/2258.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2258&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2258.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2259">2259</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAAAFWAAAAF</td><td>KAFW</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Rabbit</td><td>CCSTTTTSSSHHHHSCC</td><td align="center"><a href="jmol.php?id=2259&amp;di=1" width="5px" height="5px"><img src="pymol_images/2259.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2259&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2259.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2260">2260</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAAAFWAAAAF</td><td>KAFW</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 200µM</td><td>Human</td><td>CCSTTTTSSSHHHHSCC</td><td align="center"><a href="jmol.php?id=2260&amp;di=1" width="5px" height="5px"><img src="pymol_images/2260.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2260&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2260.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2261">2261</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAAAFWAAAAF</td><td>KAFW</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Human</td><td>CCSTTTTSSSHHHHSCC</td><td align="center"><a href="jmol.php?id=2261&amp;di=1" width="5px" height="5px"><img src="pymol_images/2261.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2261&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2261.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2262">2262</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAFAAFAAFAA</td><td>3F17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>2% hemolysis at 200µM</td><td>Rabbit</td><td>CCCSTTHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=2262&amp;di=1" width="5px" height="5px"><img src="pymol_images/2262.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2262&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2262.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2263">2263</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAFAAFAAFAA</td><td>3F17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Rabbit</td><td>CCCSTTHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=2263&amp;di=1" width="5px" height="5px"><img src="pymol_images/2263.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2263&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2263.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2264">2264</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAFAAFAAFAA</td><td>3F17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 200µM</td><td>Human</td><td>CCCSTTHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=2264&amp;di=1" width="5px" height="5px"><img src="pymol_images/2264.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2264&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2264.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2265">2265</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAFAAFAAFAA</td><td>3F17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Human</td><td>CCCSTTHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=2265&amp;di=1" width="5px" height="5px"><img src="pymol_images/2265.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2265&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2265.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2267">2267</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAWAAWAAWAA</td><td>W17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>2% hemolysis at 200µM</td><td>Rabbit</td><td>CGGGGTTTTSCGGGGCC</td><td align="center"><a href="jmol.php?id=2267&amp;di=1" width="5px" height="5px"><img src="pymol_images/2267.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2267&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2267.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2268">2268</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAWAAWAAWAA</td><td>W17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Rabbit</td><td>CGGGGTTTTSCGGGGCC</td><td align="center"><a href="jmol.php?id=2268&amp;di=1" width="5px" height="5px"><img src="pymol_images/2268.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2268&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2268.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2269">2269</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAWAAWAAWAA</td><td>W17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 200µM</td><td>Human</td><td>CGGGGTTTTSCGGGGCC</td><td align="center"><a href="jmol.php?id=2269&amp;di=1" width="5px" height="5px"><img src="pymol_images/2269.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2269&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2269.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2270">2270</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12384369">12384369</a></td><td align="right">2002</td><td>KKKKKKAAWAAWAAWAA</td><td>W17-6K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 50µM</td><td>Human</td><td>CGGGGTTTTSCGGGGCC</td><td align="center"><a href="jmol.php?id=2270&amp;di=1" width="5px" height="5px"><img src="pymol_images/2270.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2270&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2270.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2271">2271</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12404639">12404639</a></td><td align="right">2002</td><td>LRLKYRRFKYRV</td><td>Mimetic 1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>27% hemolysis at 100µg ml-1</td><td>Human</td><td>CCCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=2271&amp;di=1" width="5px" height="5px"><img src="pymol_images/2271.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2271&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2271.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2272">2272</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12404639">12404639</a></td><td align="right">2002</td><td>LRLQYRRFQYRV</td><td>Mimetic 2</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>27% hemolysis at 100µg ml<sup>-1</sup></td><td>Human</td><td>CCCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2272&amp;di=1" width="5px" height="5px"><img src="pymol_images/2272.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2272&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2272.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2273">2273</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12404639">12404639</a></td><td align="right">2002</td><td>LRLEYRRFEYRV</td><td>Mimetic 3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>14% hemolysis at 100µg ml<sup>-1</sup></td><td>Human</td><td>CCCCCTTCCCCC</td><td align="center"><a href="jmol.php?id=2273&amp;di=1" width="5px" height="5px"><img src="pymol_images/2273.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2273&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2273.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2274">2274</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12404639">12404639</a></td><td align="right">2002</td><td>LRLKKRRWKYRV</td><td>Mimetic 4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>1% hemolysis at 100µg ml<sup>-1</sup></td><td>Human</td><td>CCSCCSSCCCCC</td><td align="center"><a href="jmol.php?id=2274&amp;di=1" width="5px" height="5px"><img src="pymol_images/2274.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2274&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2274.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2275">2275</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12404639">12404639</a></td><td align="right">2002</td><td>LRLKKRRWKYRV</td><td>Mimetic 5</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>13% hemolysis at 100µg ml<sup>-1</sup></td><td>Human</td><td>CCSCCSSCCCCC</td><td align="center"><a href="jmol.php?id=2275&amp;di=1" width="5px" height="5px"><img src="pymol_images/2275.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2275&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2275.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2276">2276</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12404639">12404639</a></td><td align="right">2002</td><td>LCLKKRRWKYCV</td><td>Mimetic 6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>3% hemolysis at 100µg ml<sup>-1</sup></td><td>Human</td><td>CCCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2276&amp;di=1" width="5px" height="5px"><img src="pymol_images/2276.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2276&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2276.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2277">2277</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12404639">12404639</a></td><td align="right">2002</td><td>LRCKKRRWKCRV</td><td>Mimetic 7</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>1% hemolysis at 100µg ml<sup>-1</sup></td><td>Human</td><td>CCBTTBCCSSCC</td><td align="center"><a href="jmol.php?id=2277&amp;di=1" width="5px" height="5px"><img src="pymol_images/2277.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2277&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2277.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2278">2278</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FFGWLIKGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>100% hemolysis at 1μM</td><td>Human</td><td>CCGGGSTTTGGGTCCCCSSCSSSCC</td><td align="center"><a href="jmol.php?id=2278&amp;di=1" width="5px" height="5px"><img src="pymol_images/2278.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2278&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2278.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2279">2279</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FFGWLIRGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>100% hemolysis at 1μM</td><td>Human</td><td>CTTTTCGGGTTSSCCSSSSSCCCCC</td><td align="center"><a href="jmol.php?id=2279&amp;di=1" width="5px" height="5px"><img src="pymol_images/2279.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2279&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2279.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2280">2280</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FIGLLISAGKAIHDLIRRRH</td><td>Chrysophsin-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antibacterial</td><td>100% hemolysis at 5μM</td><td>Human</td><td>CCSCCCCSGGGHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2280&amp;di=1" width="5px" height="5px"><img src="pymol_images/2280.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2280&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2280.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2281">2281</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12569105">12569105</a></td><td align="right">2003</td><td>FFHLHFH-dcΔDOPA</td><td>Plicatamide</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>dcΔDOPA=decarboxy-(E)α,β-dehydro3,4-dihydroxyphenylalanine</td><td align="right">8</td><td>Antibacterial</td><td>100% hemolysis at ~40μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2282">2282</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12569105">12569105</a></td><td align="right">2003</td><td>FFHLHFH-dcΔDOPA</td><td>Plicatamide</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>dcΔDOPA=decarboxy-(E)α,β-dehydro3,4-dihydroxyphenylalanine</td><td align="right">8</td><td>Antibacterial</td><td>~10% hemolysis at 80μM</td><td>Sheep</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2283">2283</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12681513">12681513</a></td><td align="right">2003</td><td>KKFPWWWPFKK</td><td>SYM11KK</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0% hemolysis at 200μg/ml (non-hemolytic)</td><td>Human</td><td>CCCTTCCSCCC</td><td align="center"><a href="jmol.php?id=2283&amp;di=1" width="5px" height="5px"><img src="pymol_images/2283.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2283&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2283.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2284">2284</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12681513">12681513</a></td><td align="right">2003</td><td>VKKFPWWWPFLKK</td><td>TRK</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>1% hemolysis at 200μg/ml</td><td>Human</td><td>CCSCSSCSSSTTC</td><td align="center"><a href="jmol.php?id=2284&amp;di=1" width="5px" height="5px"><img src="pymol_images/2284.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2284&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2284.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2285">2285</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12681513">12681513</a></td><td align="right">2003</td><td>RRFPWWWPFRR</td><td>SYM11</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>~60% hemolysis at 200μg/ml</td><td>Human</td><td>CCCCCCSCSCC</td><td align="center"><a href="jmol.php?id=2285&amp;di=1" width="5px" height="5px"><img src="pymol_images/2285.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2285&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2285.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2286">2286</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VOLfPVOLfP</td><td>GS</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine</td><td align="right">10</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 15-50µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=2286&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2286.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2288">2288</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>KyPKyP</td><td>GS6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">6</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;150µg/ml</td><td>Human</td><td>CCCCCC</td><td align="center"><a href="jmol.php?id=2288&amp;di=1" width="5px" height="5px"><img src="pymol_images/2288.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2288&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2288.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2289">2289</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKyPKLyP</td><td>GS8</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">8</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;150µg/ml</td><td>Human</td><td>CCSSSCCC</td><td align="center"><a href="jmol.php?id=2289&amp;di=1" width="5px" height="5px"><img src="pymol_images/2289.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2290">2290</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLyPVKLyP</td><td>GS10</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">10</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 51–100µg/mL</td><td>Human</td><td>CCSSCCSSCC</td><td align="center"><a href="jmol.php?id=2290&amp;di=1" width="5px" height="5px"><img src="pymol_images/2290.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2290&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2290.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2291">2291</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLKyPKVKLyP</td><td>GS12</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;150µg/ml</td><td>Human</td><td>CCTTCSSSCCCC</td><td align="center"><a href="jmol.php?id=2291&amp;di=1" width="5px" height="5px"><img src="pymol_images/2291.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2291&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2291.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2292">2292</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLKVyPLKVKLyP</td><td>GS14</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &lt;15µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2292&amp;di=1" width="5px" height="5px"><img src="pymol_images/2292.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2292&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2292.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2293">2293</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>vKLKVyPLKVKLyP</td><td>GS14V1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 15-50µg/m</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2293&amp;di=1" width="5px" height="5px"><img src="pymol_images/2293.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2293&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2293.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2294">2294</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VkLKVyPLKVKLyP</td><td>GS14K2</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 15-50µg/m</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2294&amp;di=1" width="5px" height="5px"><img src="pymol_images/2294.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2294&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2294.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2295">2295</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKlKVyPLKVKLyP</td><td>GS14L3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &lt;15µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2295&amp;di=1" width="5px" height="5px"><img src="pymol_images/2295.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2295&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2295.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2296">2296</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLkVyPLKVKLyP</td><td>GS14K4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;150µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2296&amp;di=1" width="5px" height="5px"><img src="pymol_images/2296.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2296&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2296.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2297">2297</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLKvyPLKVKLyP</td><td>GS14V5</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 100–150µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2297&amp;di=1" width="5px" height="5px"><img src="pymol_images/2297.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2297&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2297.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2298">2298</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLKVYPLKVKLyP</td><td>GS14Y6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 15-50µg/m</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2298&amp;di=1" width="5px" height="5px"><img src="pymol_images/2298.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2298&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2298.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2299">2299</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLKVypLKVKLyP</td><td>GS14P7</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &lt;15µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2299&amp;di=1" width="5px" height="5px"><img src="pymol_images/2299.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2299&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2299.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2300">2300</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLKVyPlKVKLyP</td><td>GS14L8</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 15-50µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2300&amp;di=1" width="5px" height="5px"><img src="pymol_images/2300.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2300&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2300.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2301">2301</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLKVyPLkVKLyP</td><td>GS14K9</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 51–100µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2301&amp;di=1" width="5px" height="5px"><img src="pymol_images/2301.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2301&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2301.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2302">2302</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLKVyPLKvKLyP</td><td>GS14V10</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &lt;15µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2302&amp;di=1" width="5px" height="5px"><img src="pymol_images/2302.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2302&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2302.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2303">2303</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLKVyPLKVkLyP</td><td>GS14K11</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 100–150µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2303&amp;di=1" width="5px" height="5px"><img src="pymol_images/2303.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2303&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2303.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2304">2304</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLKVyPLKVKlyP</td><td>GS14L12</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 15-50µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2304&amp;di=1" width="5px" height="5px"><img src="pymol_images/2304.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2304&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2304.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2305">2305</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLKVyPLKVKLYP</td><td>GS14Y13</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &lt;15µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2305&amp;di=1" width="5px" height="5px"><img src="pymol_images/2305.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2305&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2305.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2306">2306</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLKVyPLKVKLyp</td><td>GS14P14</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &lt;15µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2306&amp;di=1" width="5px" height="5px"><img src="pymol_images/2306.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2306&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2306.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2307">2307</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKKLVyPLKVKLyP</td><td>GS14 K3L4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 100–150µg/ml</td><td>Human</td><td>CCCSSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=2307&amp;di=1" width="5px" height="5px"><img src="pymol_images/2307.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2307&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2307.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2308">2308</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>LKLkLfPLKLKLfP</td><td>GS14K4 Y2/F2, V3/L3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &lt;15µg/ml</td><td>Human</td><td>CTTCCCCTTSCCCC</td><td align="center"><a href="jmol.php?id=2308&amp;di=1" width="5px" height="5px"><img src="pymol_images/2308.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2308&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2308.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2309">2309</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>LKLkLyPLKLKLyP</td><td>GS14K4 V3/L3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 15-50µg/ml</td><td>Human</td><td>CCBTTBSSCSCCCC</td><td align="center"><a href="jmol.php?id=2309&amp;di=1" width="5px" height="5px"><img src="pymol_images/2309.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2309&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2309.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2310">2310</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLkVfPLKVKLfP</td><td>GS14K4 Y2/F2</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 15-50µg/ml</td><td>Human</td><td>CBCTTTTCCBCCCC</td><td align="center"><a href="jmol.php?id=2310&amp;di=1" width="5px" height="5px"><img src="pymol_images/2310.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2310&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2310.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2311">2311</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKLkVyPLKVKLyP</td><td>GS14K4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;150µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2311&amp;di=1" width="5px" height="5px"><img src="pymol_images/2311.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2311&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2311.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2312">2312</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>AKLkAyPLKAKLyP</td><td>GS14K4 V3/A3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;150µg/ml</td><td>Human</td><td>CCCCCCCSSSSCCC</td><td align="center"><a href="jmol.php?id=2312&amp;di=1" width="5px" height="5px"><img src="pymol_images/2312.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2312&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2312.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2313">2313</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>VKAkVyPAKVKAyP</td><td>GS14K4 L3/A3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;150µg/ml</td><td>Human</td><td>CCSSSCSSCCCCCC</td><td align="center"><a href="jmol.php?id=2313&amp;di=1" width="5px" height="5px"><img src="pymol_images/2313.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2313&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2313.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2314">2314</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12712499">12712499</a></td><td align="right">2003</td><td>AKAkAyPAKAKAyP</td><td>GS14K4 V3L3/A6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;150µg/ml</td><td>Human</td><td>CCCCCCCTTCCCCC</td><td align="center"><a href="jmol.php?id=2314&amp;di=1" width="5px" height="5px"><img src="pymol_images/2314.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2315">2315</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12963031">12963031</a></td><td align="right">2003</td><td>HVDKKVADKVLLLKQLRIMRLLTRL</td><td>Spinigerin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCSSSSSGGGGHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2315&amp;di=1" width="5px" height="5px"><img src="pymol_images/2315.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2315&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2315.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2316">2316</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12963031">12963031</a></td><td align="right">2003</td><td>KKVADKVLLLKQLRIMRL</td><td>Spinigerin (4–21)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCSCCHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=2316&amp;di=1" width="5px" height="5px"><img src="pymol_images/2316.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2316&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2316.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2317">2317</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14514057">14514057</a></td><td align="right">2003</td><td>KWKLFKKIGIGKFLHSAKKF</td><td>CA-MA</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antibacterial and Antitumor</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCCCCCCSSGGGGCTTSCC</td><td align="center"><a href="jmol.php?id=2317&amp;di=1" width="5px" height="5px"><img src="pymol_images/2317.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2317&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2317.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2318">2318</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14514057">14514057</a></td><td align="right">2003</td><td>KWKKLLKKPLLKKLLKKL</td><td>CA-MA analogue P5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antibacterial and Antitumor</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHTHHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=2318&amp;di=1" width="5px" height="5px"><img src="pymol_images/2318.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2319">2319</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14514057">14514057</a></td><td align="right">2003</td><td>KWKLKPLLKKLLKKL</td><td>P6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial and Antitumor</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCCCGGGHHHHTTTC</td><td align="center"><a href="jmol.php?id=2319&amp;di=1" width="5px" height="5px"><img src="pymol_images/2319.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2319&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2319.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2320">2320</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14514057">14514057</a></td><td align="right">2003</td><td>KWKKLLKKPLKLKL</td><td>P7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antibacterial and Antitumor</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTHHHHTSCSCCCC</td><td align="center"><a href="jmol.php?id=2320&amp;di=1" width="5px" height="5px"><img src="pymol_images/2320.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2321">2321</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14514057">14514057</a></td><td align="right">2003</td><td>KPLLKKLLKKL</td><td>P8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antibacterial and Antitumor</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTHHHHGGGC</td><td align="center"><a href="jmol.php?id=2321&amp;di=1" width="5px" height="5px"><img src="pymol_images/2321.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2321&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2321.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2322">2322</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14514057">14514057</a></td><td align="right">2003</td><td>KLLKKPLKLKL</td><td>P9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antibacterial and Antitumor</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CGGGTGGGCCC</td><td align="center"><a href="jmol.php?id=2322&amp;di=1" width="5px" height="5px"><img src="pymol_images/2322.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2322&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2322.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2323">2323</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14531844">14531844</a></td><td align="right">2003</td><td>FLPILASLAAKFGPKLFCLVTKKC</td><td>Brevinin-1BYa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antibacterial</td><td>HC<sub>50</sub> =4µM</td><td>Human</td><td>CCHHHHHHHHHHTTCCCCCCCTTC</td><td align="center"><a href="jmol.php?id=2323&amp;di=1" width="5px" height="5px"><img src="pymol_images/2323.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2323&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2323.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2324">2324</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14531844">14531844</a></td><td align="right">2003</td><td>FLPILASLAAKLGPKLFCLVTKKC</td><td>Brevinin-1BYb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antibacterial</td><td>HC<sub>50</sub> =4µM</td><td>Human</td><td>CCHHHHHHHHHHSSSCCCCCCTTC</td><td align="center"><a href="jmol.php?id=2324&amp;di=1" width="5px" height="5px"><img src="pymol_images/2324.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2324&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2324.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2326">2326</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14531844">14531844</a></td><td align="right">2003</td><td>GILSTFKGLAKGVAKDLAGNLLDKFKCKITGC</td><td>Ranatuerin-2BYa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antibacterial</td><td>HC<sub>50</sub> =120µM</td><td>Human</td><td>CCSSSCHHHHHHHTTTCSSCSTTSCCCCCCCC</td><td align="center"><a href="jmol.php?id=2326&amp;di=1" width="5px" height="5px"><img src="pymol_images/2326.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2326&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2326.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2327">2327</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14531844">14531844</a></td><td align="right">2003</td><td>GIMDSVKGLAKNLAGKLLDSLKCKITGC</td><td>Ranatuerin-2BYb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antibacterial</td><td>HC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCSTHHHHTHHHHTTGGGSSSCSCCSCC</td><td align="center"><a href="jmol.php?id=2327&amp;di=1" width="5px" height="5px"><img src="pymol_images/2327.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2327&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2327.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2329">2329</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15019213">15019213</a></td><td align="right">2003</td><td>AKKVFKRLEKLFSKIQNDK</td><td>HP (2–20)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal</td><td>&lt;20% hemolysis at 2.15 to 215µM</td><td>Human</td><td>CHHHHTTTHHHHSGGGSCC</td><td align="center"><a href="jmol.php?id=2329&amp;di=1" width="5px" height="5px"><img src="pymol_images/2329.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2329&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2329.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2336">2336</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14637016">14637016</a></td><td align="right">2003</td><td>ILGKIWEGIKSLF</td><td>IsCT1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>~20% hemolysis at 50μM</td><td>Sheep</td><td>CCSTHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2336&amp;di=1" width="5px" height="5px"><img src="pymol_images/2336.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2336&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2336.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2337">2337</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14637016">14637016</a></td><td align="right">2003</td><td>ILGKIWEGIKSLF</td><td>IsCT1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolysis at 50μM</td><td>Pig</td><td>CCSTHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2337&amp;di=1" width="5px" height="5px"><img src="pymol_images/2337.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2337&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2337.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2338">2338</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14637016">14637016</a></td><td align="right">2003</td><td>ILGKIWEGIKSLF</td><td>IsCT1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>~100% hemolysis at 50μM</td><td>Guinea-pig</td><td>CCSTHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2338&amp;di=1" width="5px" height="5px"><img src="pymol_images/2338.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2338&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2338.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2339">2339</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14637016">14637016</a></td><td align="right">2003</td><td>IFGAIWNGIKSLF</td><td>IsCT2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>&lt;20% hemolysis at 50μM</td><td>Sheep</td><td>CTTHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2339&amp;di=1" width="5px" height="5px"><img src="pymol_images/2339.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2339&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2339.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2340">2340</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14637016">14637016</a></td><td align="right">2003</td><td>IFGAIWNGIKSLF</td><td>IsCT2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>~80% hemolysis at 50μM</td><td>Pig</td><td>CTTHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2340&amp;di=1" width="5px" height="5px"><img src="pymol_images/2340.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2340&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2340.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2341">2341</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14637016">14637016</a></td><td align="right">2003</td><td>IFGAIWNGIKSLF</td><td>IsCT2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>~80% hemolysis at 50μM</td><td>Guinea-pig</td><td>CTTHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2341&amp;di=1" width="5px" height="5px"><img src="pymol_images/2341.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2341&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2341.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2342">2342</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14637016">14637016</a></td><td align="right">2003</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>&gt;80% hemolysis at 50μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2342&amp;di=1" width="5px" height="5px"><img src="pymol_images/2342.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2342&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2342.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2343">2343</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14637016">14637016</a></td><td align="right">2003</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>100% hemolysis at 50μM</td><td>Pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2343&amp;di=1" width="5px" height="5px"><img src="pymol_images/2343.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2343&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2343.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2344">2344</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14637016">14637016</a></td><td align="right">2003</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>100% hemolysis at 50μM</td><td>Guinea-pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2344&amp;di=1" width="5px" height="5px"><img src="pymol_images/2344.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2344&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2344.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2354">2354</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antibacterial</td><td>HC<sub>10</sub> =1.78μg/ml</td><td>Human</td><td>CCCSSHHHHHHSCHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2354&amp;di=1" width="5px" height="5px"><img src="pymol_images/2354.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2354&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2354.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2355">2355</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GLPALISWIKRKRQQ</td><td>Mel(12-26)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>HC<sub>10</sub> =4500μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2356">2356</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>LLPALISWIKRKRQQ</td><td>Mel(12-26,L<sup>1</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>HC<sub>10</sub> =192μg/ml</td><td>Human</td><td>CCHHHHHHTGGGTCC</td><td align="center"><a href="jmol.php?id=2356&amp;di=1" width="5px" height="5px"><img src="pymol_images/2356.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2356&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2356.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2357">2357</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GSPALISWIKRKRQQ</td><td>Mel(12-26,S<sup>2</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>HC<sub>10</sub> =1588μg/ml</td><td>Human</td><td>CCSSHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2357&amp;di=1" width="5px" height="5px"><img src="pymol_images/2357.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2357&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2357.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2358">2358</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GLLALISWIKRKRQQ</td><td>Mel(12-26,L<sup>3</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>HC<sub>10</sub> =35.2μg/ml</td><td>Human</td><td>CGGGHHHHTHHHHCC</td><td align="center"><a href="jmol.php?id=2358&amp;di=1" width="5px" height="5px"><img src="pymol_images/2358.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2358&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2358.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2359">2359</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GLSALISWIKRKRQQ</td><td>Mel(12-26,S<sup>3</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>HC<sub>10</sub> =44μg/ml</td><td>Human</td><td>CHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2359&amp;di=1" width="5px" height="5px"><img src="pymol_images/2359.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2359&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2359.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2360">2360</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GLPLLISWIKRKRQQ</td><td>Mel(12-26,L<sup>4</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>HC<sub>10</sub> =203μg/ml</td><td>Human</td><td>CCSCCCHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2360&amp;di=1" width="5px" height="5px"><img src="pymol_images/2360.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2360&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2360.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2361">2361</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GLPASISWIKRKRQQ</td><td>Mel(12-26,S<sup>5</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>HC<sub>10</sub> =3840μg/ml</td><td>Human</td><td>CCSSSCHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2361&amp;di=1" width="5px" height="5px"><img src="pymol_images/2361.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2361&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2361.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2362">2362</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GLPALSSWIKRKRQQ</td><td>Mel(12-26,S<sup>6</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>HC<sub>10</sub> =650μg/ml</td><td>Human</td><td>CCTTSTHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2362&amp;di=1" width="5px" height="5px"><img src="pymol_images/2362.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2362&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2362.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2363">2363</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GLPALILWIKRKRQQ</td><td>Mel(12-26,L<sup>7</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>HC<sub>10</sub> =15.4μg/ml</td><td>Human</td><td>CCTTCCCHHHHHHHC</td><td align="center"><a href="jmol.php?id=2363&amp;di=1" width="5px" height="5px"><img src="pymol_images/2363.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2363&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2363.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2364">2364</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GLPALISLIKRKRQQ</td><td>Mel(12-26,L<sup>8</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>HC<sub>10</sub> =9350μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2365">2365</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GLPALISSIKRKRQQ</td><td>Mel(12-26,S<sup>8</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>HC<sub>10</sub> =13800μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2366">2366</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14596922">14596922</a></td><td align="right">2003</td><td>GLPALISWSKRKRQQ</td><td>Mel(12-26,S<sup>9</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antibacterial</td><td>HC<sub>10</sub> =6140μg/ml</td><td>Human</td><td>CCCTTCSTTGGGGTC</td><td align="center"><a href="jmol.php?id=2366&amp;di=1" width="5px" height="5px"><img src="pymol_images/2366.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2366&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2366.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2367">2367</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15752569">15752569</a></td><td align="right">2005</td><td>FLSLIPHAINAVSAIAKHN</td><td>PS-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antiprotozoa</td><td>1.98% hemolysis at 64µM</td><td>Human</td><td>CCSCCCTHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2367&amp;di=1" width="5px" height="5px"><img src="pymol_images/2367.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2367&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2367.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2368">2368</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15752569">15752569</a></td><td align="right">2005</td><td>FLSLIPHAINAVSTLVHHF</td><td>PS-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antiprotozoa</td><td>2.05% hemolysis at 64µM</td><td>Human</td><td>CCSSCCTHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2368&amp;di=1" width="5px" height="5px"><img src="pymol_images/2368.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2368&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2368.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2370">2370</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15581850">15581850</a></td><td align="right">2004</td><td>SLGGVISGAKKVAKVAIPIGKAVLPVVAKLVG</td><td>CtxC</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antibacterial</td><td>Hemolytic 645µM</td><td>Human</td><td>CTTSCCSGGGGTSCCCCTTCSTTHHHHBTTBC</td><td align="center"><a href="jmol.php?id=2370&amp;di=1" width="5px" height="5px"><img src="pymol_images/2370.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2370&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2370.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2372">2372</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15581850">15581850</a></td><td align="right">2004</td><td>SIGSALKKALPVAKKIGKIALPIAKAALP</td><td>CtxA1–29</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antibacterial</td><td>Hemolytic at960µM</td><td>Human</td><td>CTHHHHTTTCHHHHTTSBCCSSHHHHCBC</td><td align="center"><a href="jmol.php?id=2372&amp;di=1" width="5px" height="5px"><img src="pymol_images/2372.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2372&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2372.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2373">2373</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CVKVKVKVGSGVKVKVKVC</td><td>V1</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =880000nM</td><td>Human</td><td>CCCCSSCCSSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=2373&amp;di=1" width="5px" height="5px"><img src="pymol_images/2373.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2373&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2373.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2374">2374</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CVKVSVKVGSGVKVSVKVC</td><td>V2</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =5,89,000nM</td><td>Human</td><td>CCCCCEECSSSEECCCSCC</td><td align="center"><a href="jmol.php?id=2374&amp;di=1" width="5px" height="5px"><img src="pymol_images/2374.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2374&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2374.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2375">2375</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CVKVRVKVGSGVKVRVKVC</td><td>V3</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =590000nM</td><td>Human</td><td>CCCCSSCCSSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=2375&amp;di=1" width="5px" height="5px"><img src="pymol_images/2375.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2375&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2375.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2376">2376</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CVKVQVKVGSGVKVQVKVC</td><td>V4</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =3226000nM</td><td>Human</td><td>CCCCSSSSSCCSCCCCCCC</td><td align="center"><a href="jmol.php?id=2376&amp;di=1" width="5px" height="5px"><img src="pymol_images/2376.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2376&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2376.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2377">2377</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CAKAKAKAGSGAKAKAKAC</td><td>V5</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =1055000nM</td><td>Human</td><td>CBTTBCCCCSSCCCTTSCC</td><td align="center"><a href="jmol.php?id=2377&amp;di=1" width="5px" height="5px"><img src="pymol_images/2377.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2377&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2377.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2378">2378</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CFKFKFKFGSGFKFKFKFC</td><td>V6</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =35000nM</td><td>Human</td><td>CCCTTSSSSSSSCCCCSCC</td><td align="center"><a href="jmol.php?id=2378&amp;di=1" width="5px" height="5px"><img src="pymol_images/2378.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2378&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2378.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2379">2379</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328096">15328096</a></td><td align="right">2004</td><td>CWKWKWKWGSGWKWKWKWC</td><td>V7</td><td>Amidation</td><td>Acetylation</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>EC<sub>50</sub> =45000nM</td><td>Human</td><td>CCCCGGGCCSSSSSCSSCC</td><td align="center"><a href="jmol.php?id=2379&amp;di=1" width="5px" height="5px"><img src="pymol_images/2379.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2379&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2379.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2381">2381</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12147359">12147359</a></td><td align="right">2002</td><td>AKKVFKRLEKLFSKIQNDK</td><td>HP (2 – 20)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CHHHHTTTHHHHSGGGSCC</td><td align="center"><a href="jmol.php?id=2381&amp;di=1" width="5px" height="5px"><img src="pymol_images/2381.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2381&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2381.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2382">2382</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12147359">12147359</a></td><td align="right">2002</td><td>AKKVFKRLEKLFSKIQNWK</td><td>Anal 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal and Anticancer</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CGGGGGGTGGGSCTTTSCC</td><td align="center"><a href="jmol.php?id=2382&amp;di=1" width="5px" height="5px"><img src="pymol_images/2382.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2382&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2382.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2383">2383</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12147359">12147359</a></td><td align="right">2002</td><td>AKKVFKRLEKLFSKIWNDK</td><td>Anal 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal and Anticancer</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CTTTTHHHHHHHCCSSCCC</td><td align="center"><a href="jmol.php?id=2383&amp;di=1" width="5px" height="5px"><img src="pymol_images/2383.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2383&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2383.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2384">2384</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12147359">12147359</a></td><td align="right">2002</td><td>AKKVFKRLEKLFSKIWNWK</td><td>Anal 3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal and Anticancer</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CGGGGHHHHHHHHHHSCCC</td><td align="center"><a href="jmol.php?id=2384&amp;di=1" width="5px" height="5px"><img src="pymol_images/2384.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2384&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2384.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2385">2385</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12147359">12147359</a></td><td align="right">2002</td><td>AKKVFKRLEKSFSKIQNDK</td><td>Anal 4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal and Anticancer</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CHHHHTTGGGTTCTTSCCC</td><td align="center"><a href="jmol.php?id=2385&amp;di=1" width="5px" height="5px"><img src="pymol_images/2385.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2385&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2385.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2386">2386</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12147359">12147359</a></td><td align="right">2002</td><td>AKKVSKRLEKLFSKIQNDK</td><td>Anal 5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal and Anticancer</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CHHHHHHHHTTTCTTTSCC</td><td align="center"><a href="jmol.php?id=2386&amp;di=1" width="5px" height="5px"><img src="pymol_images/2386.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2386&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2386.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2387">2387</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15572194">15572194</a></td><td align="right">2004</td><td>SADLVKKIWDNPAL</td><td>Polybine-I</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antibacterial</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Rat</td><td>CHHHHHTTTCCCCC</td><td align="center"><a href="jmol.php?id=2387&amp;di=1" width="5px" height="5px"><img src="pymol_images/2387.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2387&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2387.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2388">2388</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15572194">15572194</a></td><td align="right">2004</td><td>SVDMVMKGLKIWPL</td><td>Polybine-II</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antibacterial</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Rat</td><td>CHHHHHHTTSCCCC</td><td align="center"><a href="jmol.php?id=2388&amp;di=1" width="5px" height="5px"><img src="pymol_images/2388.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2388&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2388.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2389">2389</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15546886">15546886</a></td><td align="right">2005</td><td>GCRFCCNCCPNMSGCGVCCRF</td><td>Bass Hepcidin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antibacterial and Antifungal</td><td>0% hemolysis at 100 µg/ml (non-hemolytic)</td><td>Rat</td><td>CCCCCSSSSSCCSCCSSSSCC</td><td align="center"><a href="jmol.php?id=2389&amp;di=1" width="5px" height="5px"><img src="pymol_images/2389.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2389&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2389.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2390">2390</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15345319">15345319</a></td><td align="right">2004</td><td>TFRRLKWK</td><td>LALF-10 (38-45)</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>~5% hemolysis at 100µg/ml</td><td>Human</td><td>CCCCTTCC</td><td align="center"><a href="jmol.php?id=2390&amp;di=1" width="5px" height="5px"><img src="pymol_images/2390.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2390&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2390.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2391">2391</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15345319">15345319</a></td><td align="right">2004</td><td>TFRRLKWK</td><td>LALF-11 (36/38-45)</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>~5% hemolysis at 100µg/ml</td><td>Human</td><td>CCCCTTCC</td><td align="center"><a href="jmol.php?id=2391&amp;di=1" width="5px" height="5px"><img src="pymol_images/2391.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2391&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2391.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2392">2392</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15345319">15345319</a></td><td align="right">2004</td><td>KPTFRRLKWKYK</td><td>LALF-14 (36-47)</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>~5% hemolysis at 100µg/ml</td><td>Human</td><td>CCSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=2392&amp;di=1" width="5px" height="5px"><img src="pymol_images/2392.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2393">2393</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15345319">15345319</a></td><td align="right">2004</td><td>HYRIKPTFRRLKWKYKGKFW</td><td>LALf-22 (31-52)</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>~20% hemolysis at 100µg/ml</td><td>Human</td><td>CCCCCSSSTTGGGGCCCCCC</td><td align="center"><a href="jmol.php?id=2393&amp;di=1" width="5px" height="5px"><img src="pymol_images/2393.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2393&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2393.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2394">2394</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15358153">15358153</a></td><td align="right">2004</td><td>AKKVFKRLEKLFSKIQNDK</td><td>HP (2–20)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>0% hemolysis at 140µg/ml (non-hemolytic)</td><td>Human</td><td>CHHHHTTTHHHHSGGGSCC</td><td align="center"><a href="jmol.php?id=2394&amp;di=1" width="5px" height="5px"><img src="pymol_images/2394.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2394&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2394.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2395">2395</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15369808">15369808</a></td><td align="right">2004</td><td>ILGKIWEGIKSLF</td><td>IsCT</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>80% hemolysis at 100µM</td><td>Human</td><td>CCSTHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2395&amp;di=1" width="5px" height="5px"><img src="pymol_images/2395.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2395&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2395.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2396">2396</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15369808">15369808</a></td><td align="right">2004</td><td>ILGKIAEGIKSLF</td><td>[A<sup>6</sup>]-IsCT</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CHHHHHHGGGTCC</td><td align="center"><a href="jmol.php?id=2396&amp;di=1" width="5px" height="5px"><img src="pymol_images/2396.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2396&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2396.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2397">2397</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15369808">15369808</a></td><td align="right">2004</td><td>ILGKILEGIKSLF</td><td>[L<sup>6</sup>]-IsCT</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>~20% hemolysis at 100µM</td><td>Human</td><td>CTTHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2397&amp;di=1" width="5px" height="5px"><img src="pymol_images/2397.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2397&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2397.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2398">2398</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15369808">15369808</a></td><td align="right">2004</td><td>ILGKIWKGIKSLF</td><td>[K<sup>7</sup>]-IsCT</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>~50% hemolysis at 100µM</td><td>Human</td><td>CHHHHHTTTTSCC</td><td align="center"><a href="jmol.php?id=2398&amp;di=1" width="5px" height="5px"><img src="pymol_images/2398.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2398&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2398.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2399">2399</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15369808">15369808</a></td><td align="right">2004</td><td>ILGKILKGIKKLF</td><td>[L<sup>6</sup>, K<sup>11</sup>]-IsCT</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial and Antifungal</td><td>~20% hemolysis at 100µM</td><td>Human</td><td>CHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2399&amp;di=1" width="5px" height="5px"><img src="pymol_images/2399.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2399&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2399.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2400">2400</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15369808">15369808</a></td><td align="right">2004</td><td>ILGKIWKIKKLF</td><td>[K<sup>7</sup>, P<sup>8</sup>, K<sup>11</sup>]-IsCT</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antibacterial and Antifungal</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CTTSSSCCCSCC</td><td align="center"><a href="jmol.php?id=2400&amp;di=1" width="5px" height="5px"><img src="pymol_images/2400.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2400&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2400.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2401">2401</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12878508">12878508</a></td><td align="right">2003</td><td>WLNALLHHGLNCAKGVLA</td><td>18Hc</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>~20% hemolysis at 100µg/ml</td><td>Human</td><td>CHHHHTTSSTTSSCCCCC</td><td align="center"><a href="jmol.php?id=2401&amp;di=1" width="5px" height="5px"><img src="pymol_images/2401.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2401&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2401.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2402">2402</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12878508">12878508</a></td><td align="right">2003</td><td>WLNALLKKGLNCAKGVLA</td><td>18HcKK</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>&lt;10% hemolysis at 100µg/ml</td><td>Human</td><td>CHHHHTTCCCCCTTSCCC</td><td align="center"><a href="jmol.php?id=2402&amp;di=1" width="5px" height="5px"><img src="pymol_images/2402.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2402&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2402.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2403">2403</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12878508">12878508</a></td><td align="right">2003</td><td>KWLNALLHHGLNCAKGVLA</td><td>K19Hc</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>~10% hemolysis at 100µg/ml</td><td>Human</td><td>CTHHHHTTSSCCCTTTTTC</td><td align="center"><a href="jmol.php?id=2403&amp;di=1" width="5px" height="5px"><img src="pymol_images/2403.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2403&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2403.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2404">2404</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12878508">12878508</a></td><td align="right">2003</td><td>KWLNALLKKGLNCAKGVLA</td><td>K19HcKK</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>&lt;10% hemolysis at 100µg/ml</td><td>Human</td><td>CCHHHHTTSCCCCTTSTTC</td><td align="center"><a href="jmol.php?id=2404&amp;di=1" width="5px" height="5px"><img src="pymol_images/2404.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2404&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2404.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2405">2405</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12878508">12878508</a></td><td align="right">2003</td><td>KWKLFKKIPKFLHLAKKF</td><td>P18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>&lt;0% hemolysis at 100µg/ml</td><td>Human</td><td>CCCSCSCCHHHHTCSSCC</td><td align="center"><a href="jmol.php?id=2405&amp;di=1" width="5px" height="5px"><img src="pymol_images/2405.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2405&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2405.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2407">2407</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11846794">11846794</a></td><td align="right">2002</td><td>RLYRRLYR</td><td>(RLYR)<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antibacterial and Antifungal</td><td>EC<sub>50</sub> =1950µM</td><td>Human</td><td>CGGGGTCC</td><td align="center"><a href="jmol.php?id=2407&amp;di=1" width="5px" height="5px"><img src="pymol_images/2407.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2407&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2407.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2408">2408</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11846794">11846794</a></td><td align="right">2002</td><td>RLYRRLYRRLYRRLYR</td><td>(RLYR)<sub>4</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antibacterial and Antifungal</td><td>EC<sub>50</sub> =338µM</td><td>Human</td><td>CTTHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2408&amp;di=1" width="5px" height="5px"><img src="pymol_images/2408.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2408&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2408.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2409">2409</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11846794">11846794</a></td><td align="right">2002</td><td>RLYRRLYRRLYRRLYRRLYRRLYRRLYRRLYR</td><td>(RLYR)<sub>8</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antibacterial and Antifungal</td><td>EC<sub>50</sub> =48µM</td><td>Human</td><td>CTHHHHGGGTCHHHHHHHTHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2409&amp;di=1" width="5px" height="5px"><img src="pymol_images/2409.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2409&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2409.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2410">2410</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11846794">11846794</a></td><td align="right">2002</td><td>RLYRRLYRK</td><td>(RLYR)<sub>2</sub>-[K]</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antibacterial and Antifungal</td><td>EC<sub>50</sub> =3700µM</td><td>Human</td><td>CTHHHHTTC</td><td align="center"><a href="jmol.php?id=2410&amp;di=1" width="5px" height="5px"><img src="pymol_images/2410.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2410&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2410.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2411">2411</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11846794">11846794</a></td><td align="right">2002</td><td>RLYRRLYRRLYRRLYRKKK</td><td>(RLYR)<sub>4</sub>-[K<sub>2</sub>K]</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial and Antifungal</td><td>EC<sub>50</sub> =1510µM</td><td>Human</td><td>CTTHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2411&amp;di=1" width="5px" height="5px"><img src="pymol_images/2411.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2411&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2411.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2413">2413</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11846794">11846794</a></td><td align="right">2002</td><td>RLYRKVYG</td><td>RLYRKVYG</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antibacterial and Antifungal</td><td>EC<sub>50</sub> =3950µM</td><td>Human</td><td>CCCCSSCC</td><td align="center"><a href="jmol.php?id=2413&amp;di=1" width="5px" height="5px"><img src="pymol_images/2413.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2413&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2413.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2414">2414</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11846794">11846794</a></td><td align="right">2002</td><td>RLYRKVYGRLYRKVYGK</td><td>(RLYRKVYG)<sub>2</sub>-[K]</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antibacterial and Antifungal</td><td>EC<sub>50</sub> =1420µM</td><td>Human</td><td>CTHHHHHHHHGGGGSCC</td><td align="center"><a href="jmol.php?id=2414&amp;di=1" width="5px" height="5px"><img src="pymol_images/2414.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2414&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2414.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2420">2420</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11606214">11606214</a></td><td align="right">2001</td><td>VOLfPVOLfP</td><td>GS</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>O=Ornithine</td><td align="right">10</td><td>Antimicrobial</td><td>Strong hemolysis</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=2420&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2420.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2421">2421</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11606214">11606214</a></td><td align="right">2001</td><td>VKLyPVKLyP</td><td>GS10</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>Strong hemolysis</td><td>Human</td><td>CCSSCCSSCC</td><td align="center"><a href="jmol.php?id=2421&amp;di=1" width="5px" height="5px"><img src="pymol_images/2421.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2421&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2421.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2422">2422</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11606214">11606214</a></td><td align="right">2001</td><td>VKLKyPKVKLyP</td><td>GS12</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>Weak hemolysis</td><td>Human</td><td>CCTTCSSSCCCC</td><td align="center"><a href="jmol.php?id=2422&amp;di=1" width="5px" height="5px"><img src="pymol_images/2422.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2422&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2422.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2423">2423</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11606214">11606214</a></td><td align="right">2001</td><td>VKLKVyPLKVKLyP</td><td>GS14</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>Very strong hemolysis</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2423&amp;di=1" width="5px" height="5px"><img src="pymol_images/2423.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2423&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2423.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2424">2424</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11606214">11606214</a></td><td align="right">2001</td><td>VKLkVyPLKVKLyP</td><td>[D-Lys]<sup>4</sup>GS14</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>Weak hemolysis</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2424&amp;di=1" width="5px" height="5px"><img src="pymol_images/2424.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2424&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2424.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2425">2425</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11682479">11682479</a></td><td align="right">2002</td><td>VKLKVyPLKVKLyP</td><td>GS14</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 1.5µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2425&amp;di=1" width="5px" height="5px"><img src="pymol_images/2425.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2425&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2425.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2426">2426</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11682479">11682479</a></td><td align="right">2002</td><td>LKLkLfPLKLKLfP</td><td>Y2/F2, V3/L3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 12.5µg/ml</td><td>Human</td><td>CTTCCCCTTSCCCC</td><td align="center"><a href="jmol.php?id=2426&amp;di=1" width="5px" height="5px"><img src="pymol_images/2426.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2426&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2426.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2427">2427</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11682479">11682479</a></td><td align="right">2002</td><td>LKLkLyPLKLKLyP</td><td>V3/L3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 25µg/ml</td><td>Human</td><td>CCBTTBSSCSCCCC</td><td align="center"><a href="jmol.php?id=2427&amp;di=1" width="5px" height="5px"><img src="pymol_images/2427.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2427&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2427.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2428">2428</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11682479">11682479</a></td><td align="right">2002</td><td>VKLkVfPLKVKLfP</td><td>Y2/F2</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 40µg/ml</td><td>Human</td><td>CBCTTTTCCBCCCC</td><td align="center"><a href="jmol.php?id=2428&amp;di=1" width="5px" height="5px"><img src="pymol_images/2428.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2428&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2428.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2429">2429</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11682479">11682479</a></td><td align="right">2002</td><td>VKLkVyPLKVKLyP</td><td>GS14K4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at 200µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2429&amp;di=1" width="5px" height="5px"><img src="pymol_images/2429.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2429&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2429.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2430">2430</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11682479">11682479</a></td><td align="right">2002</td><td>AKLkAyPLKAKLyP</td><td>V3/A3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;800µg/ml</td><td>Human</td><td>CCCCCCCSSSSCCC</td><td align="center"><a href="jmol.php?id=2430&amp;di=1" width="5px" height="5px"><img src="pymol_images/2430.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2430&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2430.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2431">2431</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11682479">11682479</a></td><td align="right">2002</td><td>VKAkVyPAKVKAyP</td><td>L3/A3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;800µg/ml</td><td>Human</td><td>CCSSSCSSCCCCCC</td><td align="center"><a href="jmol.php?id=2431&amp;di=1" width="5px" height="5px"><img src="pymol_images/2431.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2431&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2431.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2432">2432</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11682479">11682479</a></td><td align="right">2002</td><td>AKAkAyPAKAKAyP</td><td>V3L3/A6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>100% hemolysis at &gt;800µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2433">2433</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>CYCRIPACIAGERRYGTCIYQGRLWAFCC</td><td>HNP-2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>0.9% hemolysis at 80µg/ml</td><td>Human</td><td>CCCCCSCCCCSCTTCCEEEETTEEEECCC</td><td align="center"><a href="jmol.php?id=2433&amp;di=1" width="5px" height="5px"><img src="pymol_images/2433.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2433&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2433.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2434">2434</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>VVCACRRALCLPLERRAGFCRIRGRIHPLCCRR</td><td>NP-2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>1.9% hemolysis at 80µg/ml</td><td>Human</td><td>CCCSCTTCSSSCCCSSSSSSSCCCCCCSSSSCC</td><td align="center"><a href="jmol.php?id=2434&amp;di=1" width="5px" height="5px"><img src="pymol_images/2434.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2434&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2434.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2435">2435</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>KWCFRVCYRGICYRRCR</td><td>TP-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>66.5% hemolysis at 80µg/ml</td><td>Human</td><td>CCSCCCCSSSCCCTTCC</td><td align="center"><a href="jmol.php?id=2435&amp;di=1" width="5px" height="5px"><img src="pymol_images/2435.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2435&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2435.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2436">2436</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>RGGRLCYCRRRFCVCVGR</td><td>PG-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>70.3% hemolysis at 80µg/ml</td><td>Human</td><td>CCSCCSCCCCCSCCCCCC</td><td align="center"><a href="jmol.php?id=2436&amp;di=1" width="5px" height="5px"><img src="pymol_images/2436.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2436&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2436.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2437">2437</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>rggrlcycrrrfcvcvgr</td><td>dPG-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>90% hemolysis at 80µg/ml</td><td>Human</td><td>CCSCCSCCCCCSCCCCCC</td><td align="center"><a href="jmol.php?id=2437&amp;di=1" width="5px" height="5px"><img src="pymol_images/2437.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2437&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2437.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2438">2438</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>RGGRLCYARRRFAVCVGR</td><td>PG-1 “bullet”</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>14.7% hemolysis at 80µg/ml</td><td>Human</td><td>CTTSCCCGGGCCCCCTTC</td><td align="center"><a href="jmol.php?id=2438&amp;di=1" width="5px" height="5px"><img src="pymol_images/2438.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2438&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2438.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2439">2439</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>RGGRLAYCRRRFCVAVGR</td><td>PG-1 “kite”</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>0% hemolysis at 80µg/ml</td><td>Human</td><td>CCSSCSCGGGGSSCCSCC</td><td align="center"><a href="jmol.php?id=2439&amp;di=1" width="5px" height="5px"><img src="pymol_images/2439.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2439&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2439.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2440">2440</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>RGGRLAYARRRFAVAVGR</td><td>PG-1 “snake”</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>0.8% hemolysis at 80µg/ml</td><td>Human</td><td>CTTSCTHHHHTTSSCCCC</td><td align="center"><a href="jmol.php?id=2440&amp;di=1" width="5px" height="5px"><img src="pymol_images/2440.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2440&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2440.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2441">2441</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>94.6% hemolysis at 80µg/ml</td><td>Human</td><td>CCCSSHHHHHHSCHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2441&amp;di=1" width="5px" height="5px"><img src="pymol_images/2441.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2441&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2441.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2442">2442</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>GIGKFLHSAGKFGKAFVGEIMKS</td><td>Magainin 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>0% hemolysis at 80µg/ml</td><td>Human</td><td>CCCCCCSSGGGCCTTCHHHHTCC</td><td align="center"><a href="jmol.php?id=2442&amp;di=1" width="5px" height="5px"><img src="pymol_images/2442.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2442&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2442.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2443">2443</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>GIGKFLKKAKKFGKAFVKILKK</td><td>MSI-78</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>11.4% hemolysis at 80µg/ml</td><td>Human</td><td>CTHHHHHHHHHHTTTHHHHHHC</td><td align="center"><a href="jmol.php?id=2443&amp;di=1" width="5px" height="5px"><img src="pymol_images/2443.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2443&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2443.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2444">2444</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>SWLSKTAKKLENSAKKRISEGIAIAIQGGPR</td><td>Cecropin P1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">31</td><td>Antimicrobial</td><td>0.3% hemolysis at 80µg/ml</td><td>Human</td><td>CCGGGTCHHHHTTTTTTTSSSCCCTTSCSCC</td><td align="center"><a href="jmol.php?id=2444&amp;di=1" width="5px" height="5px"><img src="pymol_images/2444.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2444&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2444.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2446">2446</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>VFQFLGKIIHHVGNFVHGFSHVF</td><td>Clavanin A amide</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>3.7% hemolysis at 80µg/ml</td><td>Human</td><td>CTTTTTGGGGTTSSCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2446&amp;di=1" width="5px" height="5px"><img src="pymol_images/2446.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2446&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2446.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2447">2447</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>VFQFLGKIIHHVGNFVHGFSHVF</td><td>Clavanin A acid</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>0% hemolysis at 80µg/ml</td><td>Human</td><td>CTTTTTGGGGTTSSCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2447&amp;di=1" width="5px" height="5px"><img src="pymol_images/2447.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2447&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2447.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2448">2448</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>VFQFLGKIIKKVGNFVKGFSKVF</td><td>Clavanin AK acid</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>30.6% hemolysis at 80µg/ml</td><td>Human</td><td>CCCHHHHHHHTTSTTCSCCCCCC</td><td align="center"><a href="jmol.php?id=2448&amp;di=1" width="5px" height="5px"><img src="pymol_images/2448.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2448&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2448.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2449">2449</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>ILPWKWPWWPWRR</td><td>Indolicidin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>38.2% hemolysis at 80µg/ml</td><td>Human</td><td>CCCCCCSSCSSCC</td><td align="center"><a href="jmol.php?id=2449&amp;di=1" width="5px" height="5px"><img src="pymol_images/2449.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2449&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2449.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2450">2450</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>FLPVLAGIAAKVVPALFCKITKKC</td><td>Brevinin-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>89% hemolysis at 80µg/ml</td><td>Human</td><td>CCHHHHHHHHHHSHHHHCCSCCCC</td><td align="center"><a href="jmol.php?id=2450&amp;di=1" width="5px" height="5px"><img src="pymol_images/2450.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2450&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2450.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2451">2451</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10795591">10795591</a></td><td align="right">2000</td><td>FLPVLAGIAAKVVPALFCKITKKC</td><td>CAM-brevinin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>0.5% hemolysis at 80µg/ml</td><td>Human</td><td>CCHHHHHHHHHHSHHHHCCSCCCC</td><td align="center"><a href="jmol.php?id=2451&amp;di=1" width="5px" height="5px"><img src="pymol_images/2451.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2451&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2451.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2471">2471</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11779154">11779154</a></td><td align="right">2002</td><td>RIIDLLWRVRRPQKPKFVTVWVR</td><td>PMAP-23</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>0% hemolysis at 100µM (non-hemolytic)</td><td>Human</td><td>CCSSSTTTCCCCSSCSCCSCCCC</td><td align="center"><a href="jmol.php?id=2471&amp;di=1" width="5px" height="5px"><img src="pymol_images/2471.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2471&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2471.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2472">2472</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12168695">12168695</a></td><td align="right">2002</td><td>GIGAVLKVLTTGLpALISWIKRKRQQ</td><td>Melittin-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Antibacterial</td><td>50% hemolysis at &gt;200µg/ml</td><td>Human</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2472&amp;di=1" width="5px" height="5px"><img src="pymol_images/2472.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2472&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2472.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2473">2473</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12168695">12168695</a></td><td align="right">2002</td><td>GIGAVLKVLTTGLpALISWIKFKRQQ</td><td>Melittin-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Antibacterial</td><td>50% hemolysis at &gt;80µg/ml</td><td>Human</td><td>CCCCCSSCTTCCCHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2473&amp;di=1" width="5px" height="5px"><img src="pymol_images/2473.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2473&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2473.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2474">2474</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12168695">12168695</a></td><td align="right">2002</td><td>GIGAVLKVLTTGLpALISWIKXXKRQQ</td><td>Melittin-1P</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>X: Pcm arg</td><td align="right">27</td><td>Antibacterial</td><td>50% hemolysis at &gt;18µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2475">2475</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12168695">12168695</a></td><td align="right">2002</td><td>GIGAVLK*VLTTGLpALISWIK*XK*RQQ</td><td>Melittin-A1P</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>X: Pcm arg; K*: Acetylated Lysin</td><td align="right">26</td><td>Antibacterial</td><td>50% hemolysis at &gt;200µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2476">2476</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12168695">12168695</a></td><td align="right">2002</td><td>GIGAVLK*VLTTGLpALISWIK*RK*RQQ</td><td>Melittin-A1</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>K*: Acetylated Lysin</td><td align="right">26</td><td>Antibacterial</td><td>50% hemolysis at &gt;200µg/ml</td><td>Human</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2476&amp;di=1" width="5px" height="5px"><img src="pymol_images/2476.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2476&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2476.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2480">2480</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12429503">12429503</a></td><td align="right">2002</td><td>GLMDTVKNAAKNLAGQLLDTIKCKMTGC</td><td>Ranatuerin-2ARa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antibacterial</td><td>HC<sub>50</sub> =100µM</td><td>Human</td><td>CTHHHHHHHHHHHTTTGGGGGGSCCCCC</td><td align="center"><a href="jmol.php?id=2480&amp;di=1" width="5px" height="5px"><img src="pymol_images/2480.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2480&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2480.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2481">2481</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12429503">12429503</a></td><td align="right">2002</td><td>GILDTIKNAAKTVAVGLLEKIKCKMTGC</td><td>Ranatuerin-2ARb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antibacterial</td><td>HC<sub>50</sub> =150µM</td><td>Human</td><td>CCTHHHHTTGGGSHHHHHGGGGSCCSCC</td><td align="center"><a href="jmol.php?id=2481&amp;di=1" width="5px" height="5px"><img src="pymol_images/2481.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2481&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2481.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2482">2482</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12429503">12429503</a></td><td align="right">2002</td><td>GFISTVKNLATNVAGTVIDTIKCKVTGGC</td><td>Palustrin-2AR</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antibacterial</td><td>HC<sub>50</sub> &gt;300µM</td><td>Human</td><td>CCSSCHHHHTTCCCCCCCCCSSSCCSSCC</td><td align="center"><a href="jmol.php?id=2482&amp;di=1" width="5px" height="5px"><img src="pymol_images/2482.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2482&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2482.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2483">2483</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12429503">12429503</a></td><td align="right">2002</td><td>FLPIVGRLISGLL</td><td>Temporin-1AR</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>HC<sub>50</sub> =210µM</td><td>Human</td><td>CCCCHHHHHTCCC</td><td align="center"><a href="jmol.php?id=2483&amp;di=1" width="5px" height="5px"><img src="pymol_images/2483.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2483&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2483.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2484">2484</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10747913">10747913</a></td><td align="right">2000</td><td>CGETCVGGTCNTPGCTCSWPVCTRNGLPV</td><td>Kalata B1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Anti-HIV</td><td>50% hemolytic at 50μM</td><td>Human</td><td>CCCGGGTTTSSCCCCCSSSCCCTTCSSCC</td><td align="center"><a href="jmol.php?id=2484&amp;di=1" width="5px" height="5px"><img src="pymol_images/2484.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2484&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2484.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2486">2486</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10799508">10799508</a></td><td align="right">2000</td><td>VKLKVyPLKVKLyP</td><td>GS14</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 10μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2486&amp;di=1" width="5px" height="5px"><img src="pymol_images/2486.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2486&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2486.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2487">2487</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10799508">10799508</a></td><td align="right">2000</td><td>VKLKVyPLKVKLyP</td><td>GS14</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 100μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2487&amp;di=1" width="5px" height="5px"><img src="pymol_images/2487.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2487&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2487.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2488">2488</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10799508">10799508</a></td><td align="right">2000</td><td>VKLkVyPLKVKLyP</td><td>GS14K4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>1% hemolytic at 10μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2488&amp;di=1" width="5px" height="5px"><img src="pymol_images/2488.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2488&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2488.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2489">2489</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10799508">10799508</a></td><td align="right">2000</td><td>VKLkVyPLKVKLyP</td><td>GS14K4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>8% hemolytic at 100μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2489&amp;di=1" width="5px" height="5px"><img src="pymol_images/2489.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2489&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2489.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2490">2490</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10799508">10799508</a></td><td align="right">2000</td><td>VKKLVyPLKVKLyP</td><td>GS14K3L4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>3% hemolytic at 10μg/ml</td><td>Human</td><td>CCCSSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=2490&amp;di=1" width="5px" height="5px"><img src="pymol_images/2490.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2490&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2490.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2491">2491</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10799508">10799508</a></td><td align="right">2000</td><td>VKKLVyPLKVKLyP</td><td>GS14K3L4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>37% hemolytic at 100μg/ml</td><td>Human</td><td>CCCSSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=2491&amp;di=1" width="5px" height="5px"><img src="pymol_images/2491.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2491&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2491.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2492">2492</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15222751">15222751</a></td><td align="right">2004</td><td>GLVTSLIKGAGKLLGGLFGSVTGGQS</td><td>DRP-PBN2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytotoxic</td><td>42% hemolytic at 50μM</td><td>Rat</td><td>CCHHHHHTTSSTTTSSSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=2492&amp;di=1" width="5px" height="5px"><img src="pymol_images/2492.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2492&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2492.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2493">2493</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15222751">15222751</a></td><td align="right">2004</td><td>GLVTSLIKGAGKLLGGLFGSVTGGQS</td><td>DRP-PBN2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytotoxic</td><td>75% hemolytic at 100μM</td><td>Rat</td><td>CCHHHHHTTSSTTTSSSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=2493&amp;di=1" width="5px" height="5px"><img src="pymol_images/2493.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2493&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2493.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2494">2494</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15222751">15222751</a></td><td align="right">2004</td><td>GVVTDLLNTAGGLLGNLVGSLSG</td><td>DRP-PD 3-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Cytotoxic</td><td>43% hemolytic at 50μM</td><td>Rat</td><td>CCSSCSSHHHHHHHHHHCSSSCC</td><td align="center"><a href="jmol.php?id=2494&amp;di=1" width="5px" height="5px"><img src="pymol_images/2494.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2494&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2494.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2495">2495</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15222751">15222751</a></td><td align="right">2004</td><td>GVVTDLLNTAGGLLGNLVGSLSG</td><td>DRP-PD 3-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Cytotoxic</td><td>65% hemolytic at 100μM</td><td>Rat</td><td>CCSSCSSHHHHHHHHHHCSSSCC</td><td align="center"><a href="jmol.php?id=2495&amp;di=1" width="5px" height="5px"><img src="pymol_images/2495.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2495&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2495.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2496">2496</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15222751">15222751</a></td><td align="right">2004</td><td>GLVTGLLKTAGKLLGDLFGSLSG</td><td>ANC [K<sup>8,12</sup>] - ANC</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Cytotoxic</td><td>22% hemolytic at 50μM</td><td>Rat</td><td>CCSSHHHHHHTTHHHHTTTCCCC</td><td align="center"><a href="jmol.php?id=2496&amp;di=1" width="5px" height="5px"><img src="pymol_images/2496.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2496&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2496.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2497">2497</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15222751">15222751</a></td><td align="right">2004</td><td>GLVTGLLKTAGKLLGDLFGSLSG</td><td>ANC [K<sup>8,12</sup>] - ANC</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Cytotoxic</td><td>48% hemolytic at 100μM</td><td>Rat</td><td>CCSSHHHHHHTTHHHHTTTCCCC</td><td align="center"><a href="jmol.php?id=2497&amp;di=1" width="5px" height="5px"><img src="pymol_images/2497.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2497&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2497.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2498">2498</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15222751">15222751</a></td><td align="right">2004</td><td>GVVTDLLKTAGKLLGNLVGSLSG</td><td>[K<sup>8,12</sup>] - DRP-PD 3-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Cytotoxic</td><td>35% hemolytic at 50μM</td><td>Rat</td><td>CCHHHHTTTHHHHHTTTSCCCCC</td><td align="center"><a href="jmol.php?id=2498&amp;di=1" width="5px" height="5px"><img src="pymol_images/2498.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2498&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2498.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2499">2499</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15222751">15222751</a></td><td align="right">2004</td><td>GVVTDLLKTAGKLLGNLVGSLSG</td><td>[K<sup>8,12</sup>] – DRP-PD 3-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Cytotoxic</td><td>71% hemolytic at 100μM</td><td>Rat</td><td>CCHHHHTTTHHHHHTTTSCCCCC</td><td align="center"><a href="jmol.php?id=2499&amp;di=1" width="5px" height="5px"><img src="pymol_images/2499.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2499&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2499.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2500">2500</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15195974">15195974</a></td><td align="right">2004</td><td>KWKKLLKKLLKLL</td><td>K<sub>6</sub>L<sub>6</sub>W</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>75% hemolytic at 100μM</td><td>Human</td><td>CHHHHGGGGGGTC</td><td align="center"><a href="jmol.php?id=2500&amp;di=1" width="5px" height="5px"><img src="pymol_images/2500.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2500&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2500.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2501">2501</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15195974">15195974</a></td><td align="right">2004</td><td>KWKKLPKKLLKLL</td><td>K<sub>6</sub>L<sub>5</sub>WP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CCSHHHHHTTTTC</td><td align="center"><a href="jmol.php?id=2501&amp;di=1" width="5px" height="5px"><img src="pymol_images/2501.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2502">2502</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14965277">14965277</a></td><td align="right">2004</td><td>GWGSFFKKAAHVGKHVGKAALTHYL</td><td>Ple</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>2.5% hemolytic at 100μM</td><td>Human</td><td>CCHHHHTTTHHHHHHHTSGGGSCCC</td><td align="center"><a href="jmol.php?id=2502&amp;di=1" width="5px" height="5px"><img src="pymol_images/2502.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2502&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2502.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2503">2503</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14965277">14965277</a></td><td align="right">2004</td><td>GWGSFFKKAAHVAKHVGKAALTHYL</td><td>Ple-AG</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>16.6% hemolytic at 100μM</td><td>Human</td><td>CCTTTHHHHHHHHHHHGGGGTCSCC</td><td align="center"><a href="jmol.php?id=2503&amp;di=1" width="5px" height="5px"><img src="pymol_images/2503.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2503&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2503.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2504">2504</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14965277">14965277</a></td><td align="right">2004</td><td>GWGSFFKKAAHVGKHVAKAALTHYL</td><td>Ple-GA</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>15.4% hemolytic at 100μM</td><td>Human</td><td>CCSSHHHHHHHHHHHHTTTTTTSCC</td><td align="center"><a href="jmol.php?id=2504&amp;di=1" width="5px" height="5px"><img src="pymol_images/2504.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2504&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2504.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2505">2505</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14965277">14965277</a></td><td align="right">2004</td><td>GWGSFFKKAAHVAKHVAKAALTHYL</td><td>Ple-AA</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>37.4% hemolytic at 100μM</td><td>Human</td><td>CTTTTTHHHHSTHHHHTTSSTTCCC</td><td align="center"><a href="jmol.php?id=2505&amp;di=1" width="5px" height="5px"><img src="pymol_images/2505.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2505&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2505.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2506">2506</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12931994">12931994</a></td><td align="right">2003</td><td>RGGRLCYCRRRFCVCVGR</td><td>Protegrin-I (2)</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>37% hemolytic at 100μg/ml</td><td>Human</td><td>CCSCCSCCCCCSCCCCCC</td><td align="center"><a href="jmol.php?id=2506&amp;di=1" width="5px" height="5px"><img src="pymol_images/2506.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2507">2507</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12931994">12931994</a></td><td align="right">2003</td><td>LRLKKRRWKYRVp</td><td>Peptide 5</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>1.4% hemolytic at 100μg/ml</td><td>Human</td><td>CCSCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2507&amp;di=1" width="5px" height="5px"><img src="pymol_images/2507.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2507&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2507.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2508">2508</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12071741">12071741</a></td><td align="right">2002</td><td>GIGKFLHAAKKFAKAFVAEIMNS</td><td>Ala8,13,18-magainin II</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>100% hemolytic at 100μg/ml</td><td>Human</td><td>CCSCHHHHHHTTTTTTTTTTTTC</td><td align="center"><a href="jmol.php?id=2508&amp;di=1" width="5px" height="5px"><img src="pymol_images/2508.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2508&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2508.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2515">2515</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10973820">10973820</a></td><td align="right">2000</td><td>GEKLKKIGQKIKNFFQKL</td><td>CRAMP-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and Anticancer</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CCHHHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2515&amp;di=1" width="5px" height="5px"><img src="pymol_images/2515.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2515&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2515.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2516">2516</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10973820">10973820</a></td><td align="right">2000</td><td>GKKLKKIGQKIKNFFQKL</td><td>K2-CRAMP-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and Anticancer</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CTTHHHHTTTHHHHTTTC</td><td align="center"><a href="jmol.php?id=2516&amp;di=1" width="5px" height="5px"><img src="pymol_images/2516.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2516&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2516.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2517">2517</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10973820">10973820</a></td><td align="right">2000</td><td>GEKLKKIGKKIKNFFQKL</td><td>K9-CRAMP-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and Anticancer</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CCSHHHHHHHHTTTTTCC</td><td align="center"><a href="jmol.php?id=2517&amp;di=1" width="5px" height="5px"><img src="pymol_images/2517.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2517&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2517.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2518">2518</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10973820">10973820</a></td><td align="right">2000</td><td>GEKLKKIGQKIKKFFQKL</td><td>K13-CRAMP-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and Anticancer</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CCHHHHHHHHHHGGGGGC</td><td align="center"><a href="jmol.php?id=2518&amp;di=1" width="5px" height="5px"><img src="pymol_images/2518.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2518&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2518.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2519">2519</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10973820">10973820</a></td><td align="right">2000</td><td>GEKLKKIGQKIKNFFKKL</td><td>K16-CRAMP-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and Anticancer</td><td>0.2% hemolytic at 100μM</td><td>Human</td><td>CCSSTTHHHHHHHHCCCC</td><td align="center"><a href="jmol.php?id=2519&amp;di=1" width="5px" height="5px"><img src="pymol_images/2519.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2519&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2519.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2520">2520</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10879468">10879468</a></td><td align="right">2000</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>Magainin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimcrobial and Hemolytic</td><td>4.5% hemolytic at 10μM</td><td>Human</td><td>CCSSHHHHHHHHHHHHTTTTTTSCC</td><td align="center"><a href="jmol.php?id=2520&amp;di=1" width="5px" height="5px"><img src="pymol_images/2520.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2520&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2520.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2521">2521</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10879468">10879468</a></td><td align="right">2000</td><td>NWKGIAAMAKKLL</td><td>Mastroparan X</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimcrobial and Hemolytic</td><td>24% hemolytic at 10μM</td><td>Human</td><td>CCHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2521&amp;di=1" width="5px" height="5px"><img src="pymol_images/2521.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2521&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2521.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2522">2522</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10879468">10879468</a></td><td align="right">2000</td><td>KNWKGIAGMAKKLLGKNWKLM</td><td>Mast 21</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimcrobial and Hemolytic</td><td>5.4% hemolytic at 10μM</td><td>Human</td><td>CCCTTHHHHHHTTCSSCCCCC</td><td align="center"><a href="jmol.php?id=2522&amp;di=1" width="5px" height="5px"><img src="pymol_images/2522.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2522&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2522.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2523">2523</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10879468">10879468</a></td><td align="right">2000</td><td>KNWKGIAGMAKKLLGENWELM</td><td>Mast 21(+2)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimcrobial and Hemolytic</td><td>5.3% hemolytic at 10μM</td><td>Human</td><td>CCCCCGGGTTGGGGCCSSCCC</td><td align="center"><a href="jmol.php?id=2523&amp;di=1" width="5px" height="5px"><img src="pymol_images/2523.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2523&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2523.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2524">2524</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10879468">10879468</a></td><td align="right">2000</td><td>KNWKGIAGMAKKLLGENWKLM</td><td>Mast 21(+4)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimcrobial and Hemolytic</td><td>7.1% hemolytic at 10μM</td><td>Human</td><td>CCCCTHHHHHTTTTTSCCCCC</td><td align="center"><a href="jmol.php?id=2524&amp;di=1" width="5px" height="5px"><img src="pymol_images/2524.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2524&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2524.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2525">2525</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10879468">10879468</a></td><td align="right">2000</td><td>KNWKKIAGMAKKLLKKNWKLM</td><td>Mast 21(+8)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimcrobial and Hemolytic</td><td>6.3% hemolytic at 10μM</td><td>Human</td><td>CCHHHHHHHTHHHHTTTCCCC</td><td align="center"><a href="jmol.php?id=2525&amp;di=1" width="5px" height="5px"><img src="pymol_images/2525.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2525&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2525.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2526">2526</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10879468">10879468</a></td><td align="right">2000</td><td>KNWKGIAGMAKKLLGKNW</td><td>Mast 18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimcrobial and Hemolytic</td><td>2.7% hemolytic at 10μM</td><td>Human</td><td>CCCSSHHHHTTTCSSSCC</td><td align="center"><a href="jmol.php?id=2526&amp;di=1" width="5px" height="5px"><img src="pymol_images/2526.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2526&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2526.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2527">2527</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10879468">10879468</a></td><td align="right">2000</td><td>KNWKGIAGMAKKLLGKNW</td><td>Mast 19</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimcrobial and Hemolytic</td><td>5.1% hemolytic at 10μM</td><td>Human</td><td>CCCSSHHHHTTTCSSSCC</td><td align="center"><a href="jmol.php?id=2527&amp;di=1" width="5px" height="5px"><img src="pymol_images/2527.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2527&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2527.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2528">2528</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10879468">10879468</a></td><td align="right">2000</td><td>KNWKGIAGMAKKLLGKNWELMEK</td><td>Mast 23(+4)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimcrobial and Hemolytic</td><td>6.3% hemolytic at 10μM</td><td>Human</td><td>CCCTTHHHHHHHHTSSCTTTCCC</td><td align="center"><a href="jmol.php?id=2528&amp;di=1" width="5px" height="5px"><img src="pymol_images/2528.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2528&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2528.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2529">2529</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10879468">10879468</a></td><td align="right">2000</td><td>KNWKGIAGMAKKLLGKNWELMKK</td><td>Mast 23(+6)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimcrobial and Hemolytic</td><td>7.2% hemolytic at 10μM</td><td>Human</td><td>CCSSSHHHHHHHHHTTSGGGTCC</td><td align="center"><a href="jmol.php?id=2529&amp;di=1" width="5px" height="5px"><img src="pymol_images/2529.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2529&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2529.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2530">2530</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10879468">10879468</a></td><td align="right">2000</td><td>KNWKGIAGMAKKLLGKNWKLMKK</td><td>Mast 23(+8)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimcrobial and Hemolytic</td><td>7.2% hemolytic at 10μM</td><td>Human</td><td>CCCSSHHHHHHHHTTTCTTTCCC</td><td align="center"><a href="jmol.php?id=2530&amp;di=1" width="5px" height="5px"><img src="pymol_images/2530.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2530&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2530.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2531">2531</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10879468">10879468</a></td><td align="right">2000</td><td>KNWKGIAGMAKKLLGKNWKLM</td><td>Mast 21N</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimcrobial and Hemolytic</td><td>12.3% hemolytic at 10μM</td><td>Human</td><td>CCCTTHHHHHHTTCSSCCCCC</td><td align="center"><a href="jmol.php?id=2531&amp;di=1" width="5px" height="5px"><img src="pymol_images/2531.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2532">2532</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10821683">10821683</a></td><td align="right">2000</td><td>CKLLLKWLLKLLKC</td><td>Cyclic-K<sup>4</sup>L<sup>7</sup>W</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">14</td><td>Cytolytic</td><td>50% hemolytic at 50μM</td><td>Human</td><td>CTTHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2532&amp;di=1" width="5px" height="5px"><img src="pymol_images/2532.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2532&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2532.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2533">2533</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10821683">10821683</a></td><td align="right">2000</td><td>CH3-S-CKLLLKWLLKLLKC-S-CH3</td><td>Linear-K<sup>4</sup>L<sup>7</sup>W</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Cytolytic</td><td>100% hemolytic at 5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2534">2534</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10821683">10821683</a></td><td align="right">2000</td><td>cklllkwllkllkc</td><td>cyclic-[D]-L<sup>3,4,8,10</sup>-K<sup>4</sup>L<sup>7</sup>W</td><td>Free</td><td>Free</td><td>Cyclic</td><td>D</td><td>None</td><td align="right">14</td><td>Cytolytic</td><td>90% hemolytic at 50μM</td><td>Human</td><td>CTTHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2534&amp;di=1" width="5px" height="5px"><img src="pymol_images/2534.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2534&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2534.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2535">2535</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10821683">10821683</a></td><td align="right">2000</td><td>CH3-S-cklllkwllkllkc-S-CH3</td><td>linear-[D]-L<sup>3,4,8,10</sup>-K<sup>4</sup>L<sup>7</sup>W</td><td>Free</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">14</td><td>Cytolytic</td><td>5% hemolytic at 50μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2536">2536</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10673369">10673369</a></td><td align="right">1999</td><td>KWCFRVCYRGICYRRCR-X</td><td>TP-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>EC<sub>25</sub> =29μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2537">2537</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10673369">10673369</a></td><td align="right">1999</td><td>KWCFCVCYRGICYCRCRG</td><td>cc TP</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>EC<sub>25</sub> =590μM</td><td>Human</td><td>CCCCCBCTTSSBSSCCCC</td><td align="center"><a href="jmol.php?id=2537&amp;di=1" width="5px" height="5px"><img src="pymol_images/2537.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2537&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2537.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2538">2538</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10673369">10673369</a></td><td align="right">1999</td><td>GFCRCLCRRGVCRCLCTK</td><td>RTD</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>EC<sub>25</sub> =2350μM</td><td>Human</td><td>CCTTSSCCCCCCCCCTTC</td><td align="center"><a href="jmol.php?id=2538&amp;di=1" width="5px" height="5px"><img src="pymol_images/2538.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2538&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2538.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2539">2539</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10669014">10669014</a></td><td align="right">1999</td><td>LFGFLIKLIPSLFGALSNIGRNRNQ</td><td>Gs1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Hemolytic and Ichthyotoxic</td><td>LC<sub>50</sub> =1.9μM</td><td>Horse</td><td>CCCSCTTTHHHHHHHHHTTSTTTCC</td><td align="center"><a href="jmol.php?id=2539&amp;di=1" width="5px" height="5px"><img src="pymol_images/2539.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2539&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2539.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2540">2540</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10669014">10669014</a></td><td align="right">1999</td><td>LFGFLIPLLPHIIGAIPQVIGAIR</td><td>Gs2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Hemolytic and Ichthyotoxic</td><td>LC<sub>50</sub> =0.19μM</td><td>Horse</td><td>CCSSSTTHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2540&amp;di=1" width="5px" height="5px"><img src="pymol_images/2540.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2540&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2540.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2541">2541</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18501981">18501981</a></td><td align="right">2008</td><td>GVIIDTLKGAAKTVAAELLRKAHCKLTNSC</td><td>Brevinin-2GUb</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Cytolytic</td><td>LC<sub>50</sub> =700μM</td><td>Human</td><td>CCCCCSCCSSHHHHHHHHHHHHSGGGSCCC</td><td align="center"><a href="jmol.php?id=2541&amp;di=1" width="5px" height="5px"><img src="pymol_images/2541.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2541&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2541.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2542">2542</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18325325">18325325</a></td><td align="right">2008</td><td>GWGSFFKKAAHVGKHVGKAALTHYL</td><td>Ple</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>77% hemolysis at 100μM</td><td>Human</td><td>CCHHHHTTTHHHHHHHTSGGGSCCC</td><td align="center"><a href="jmol.php?id=2542&amp;di=1" width="5px" height="5px"><img src="pymol_images/2542.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2543">2543</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18325325">18325325</a></td><td align="right">2008</td><td>GWGSFFKKAAHVGKHVGKAALTHYL</td><td>Ple</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>29% hemolysis at 50μM</td><td>Human</td><td>CCHHHHTTTHHHHHHHTSGGGSCCC</td><td align="center"><a href="jmol.php?id=2543&amp;di=1" width="5px" height="5px"><img src="pymol_images/2543.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2544">2544</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18325325">18325325</a></td><td align="right">2008</td><td>GWGSFFKKAAHVGKHVGKAALTHYL</td><td>Ple</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>6% hemolysis at 25μM</td><td>Human</td><td>CCHHHHTTTHHHHHHHTSGGGSCCC</td><td align="center"><a href="jmol.php?id=2544&amp;di=1" width="5px" height="5px"><img src="pymol_images/2544.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2545">2545</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18325325">18325325</a></td><td align="right">2008</td><td>GWGSFFKKAAHVGKHVGKAALTHYL</td><td>Ple</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>0% hemolytic at 12.5μM</td><td>Human</td><td>CCHHHHTTTHHHHHHHTSGGGSCCC</td><td align="center"><a href="jmol.php?id=2545&amp;di=1" width="5px" height="5px"><img src="pymol_images/2545.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2546">2546</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18325325">18325325</a></td><td align="right">2008</td><td>RWRSFFKKAAHRGKHVGKRARTHYL</td><td>Anal-R</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CCTTHHHHSSCSSCCCCSBTTBCCC</td><td align="center"><a href="jmol.php?id=2546&amp;di=1" width="5px" height="5px"><img src="pymol_images/2546.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2546&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2546.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2547">2547</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18325325">18325325</a></td><td align="right">2008</td><td>SWSSFFKKAAHSGKHVGKSASTHYL</td><td>Anal-S</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CCHHHHTTTSTTCCCSSCSSCSSCC</td><td align="center"><a href="jmol.php?id=2547&amp;di=1" width="5px" height="5px"><img src="pymol_images/2547.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2547&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2547.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2548">2548</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18068868">18068868</a></td><td align="right">2007</td><td>GFKDLLKGAAKALVKTVLF</td><td>Ascaphin-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =55μM</td><td>Human</td><td>CHHHHHHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=2548&amp;di=1" width="5px" height="5px"><img src="pymol_images/2548.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2548&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2548.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2549">2549</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18068868">18068868</a></td><td align="right">2007</td><td>GFKKLLKGAAKALVKTVLF</td><td>Lys<sup>4</sup>-Ascaphin-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =24μM</td><td>Human</td><td>CHHHHHHHHHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=2549&amp;di=1" width="5px" height="5px"><img src="pymol_images/2549.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2549&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2549.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2550">2550</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18068868">18068868</a></td><td align="right">2007</td><td>GFKDLLKKAAKALVKTVLF</td><td>Lys<sup>8</sup>-Ascaphin-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>LC<sub>50</sub> =11μM</td><td>Human</td><td>CHHHHHHHHHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=2550&amp;di=1" width="5px" height="5px"><img src="pymol_images/2550.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2550&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2550.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2551">2551</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VOLfPVOLfP</td><td>GS</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine</td><td align="right">10</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 12.5μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=2551&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2551.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2552">2552</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLKVyPLKVKLyP</td><td>GS14</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 1.5μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2552&amp;di=1" width="5px" height="5px"><img src="pymol_images/2552.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2552&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2552.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2553">2553</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLKVyPLKvKLyP</td><td>GS14V10</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 6.2μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2553&amp;di=1" width="5px" height="5px"><img src="pymol_images/2553.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2553&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2553.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2554">2554</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKlKVyPLKVKLyP</td><td>GS14L3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 12.5μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2554&amp;di=1" width="5px" height="5px"><img src="pymol_images/2554.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2554&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2554.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2555">2555</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLKVyPLKVKLyp</td><td>GS14P14</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 6.2μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2555&amp;di=1" width="5px" height="5px"><img src="pymol_images/2555.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2555&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2555.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2556">2556</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLKVypLKVKLyP</td><td>GS14P7</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 12.5μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2556&amp;di=1" width="5px" height="5px"><img src="pymol_images/2556.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2556&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2556.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2557">2557</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLKVyPLKVKLyP</td><td>GS14V1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 40μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2557&amp;di=1" width="5px" height="5px"><img src="pymol_images/2557.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2557&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2557.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2558">2558</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLKVyPLKVKlyP</td><td>GS14L12</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 25μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2558&amp;di=1" width="5px" height="5px"><img src="pymol_images/2558.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2558&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2558.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2559">2559</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLKvyPLKVKLyP</td><td>GS14V5</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 150μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2559&amp;di=1" width="5px" height="5px"><img src="pymol_images/2559.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2559&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2559.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2560">2560</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLKVyPLKVKLYP</td><td>GS14Y13</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 12.5μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2560&amp;di=1" width="5px" height="5px"><img src="pymol_images/2560.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2560&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2560.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2561">2561</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLKVyPlKVKLyP</td><td>GS14L8</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 50μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2561&amp;di=1" width="5px" height="5px"><img src="pymol_images/2561.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2561&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2561.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2562">2562</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLKVYPLKVKLyP</td><td>GS14Y6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 25μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2562&amp;di=1" width="5px" height="5px"><img src="pymol_images/2562.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2562&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2562.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2563">2563</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VkLKVyPLKVKLyP</td><td>GS14K2</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 50μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2563&amp;di=1" width="5px" height="5px"><img src="pymol_images/2563.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2563&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2563.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2564">2564</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLKVyPLkVKLyP</td><td>GS14K9</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 100μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2564&amp;di=1" width="5px" height="5px"><img src="pymol_images/2564.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2564&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2564.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2565">2565</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLkVyPLKVKLyP</td><td>GS14K4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 200μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2565&amp;di=1" width="5px" height="5px"><img src="pymol_images/2565.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2565&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2565.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2566">2566</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VKLKVyPLKVkLyP</td><td>GS14K11</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at 150μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2566&amp;di=1" width="5px" height="5px"><img src="pymol_images/2566.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2566&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2566.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2567">2567</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VkLkVyPLKVKLyP</td><td>GS14K2K4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at &gt;200μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2567&amp;di=1" width="5px" height="5px"><img src="pymol_images/2567.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2567&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2567.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2568">2568</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10224074">10224074</a></td><td align="right">1999</td><td>VkLkVyPLkVkLyP</td><td>GS14K2K4K9K11</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial and Hemolytic</td><td>100% hemolytic at &gt;200μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2568&amp;di=1" width="5px" height="5px"><img src="pymol_images/2568.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2568&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2568.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2570">2570</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FVDLKKIANIINSIFGK</td><td>Temporin-1CEa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial and Anticancer and moderate Hemolytic</td><td>LC<sub>50</sub> =95.7μM</td><td>Human</td><td>CCCHHHHHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=2570&amp;di=1" width="5px" height="5px"><img src="pymol_images/2570.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2570&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2570.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2571">2571</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FVDLKKIANIINSIFKK</td><td>LK1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Anticancer</td><td>LC<sub>50</sub> =1,467μM</td><td>Human</td><td>CCCHHHHHHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2571&amp;di=1" width="5px" height="5px"><img src="pymol_images/2571.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2571&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2571.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2572">2572</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FKDLKKIANIINSIFKK</td><td>LK2(5)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Anticancer</td><td>LC<sub>50</sub> =1,186μM</td><td>Human</td><td>CCCSIIIIIGGGTTSCC</td><td align="center"><a href="jmol.php?id=2572&amp;di=1" width="5px" height="5px"><img src="pymol_images/2572.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2572&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2572.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2573">2573</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FVKLKKIANIINSIFKK</td><td>LK2(6)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Anticancer</td><td>LC<sub>50</sub> =1,993μM</td><td>Human</td><td>CCCTHHHHHHHTTTTCC</td><td align="center"><a href="jmol.php?id=2573&amp;di=1" width="5px" height="5px"><img src="pymol_images/2573.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2573&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2573.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2574">2574</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FKKLKKIANIINSIFKK</td><td>LK3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Anticancer</td><td>LC<sub>50</sub> =4,649μM</td><td>Human</td><td>CCGGGGHHHHHTTTTCC</td><td align="center"><a href="jmol.php?id=2574&amp;di=1" width="5px" height="5px"><img src="pymol_images/2574.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2574&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2574.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2575">2575</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FVKLKKILNIINSIFKK</td><td>LK2(6)A(L)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Anticancer</td><td>LC<sub>50</sub> =86.99μM</td><td>Human</td><td>CCSTHHHHHHHTTTTCC</td><td align="center"><a href="jmol.php?id=2575&amp;di=1" width="5px" height="5px"><img src="pymol_images/2575.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2575&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2575.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2576">2576</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23609760">23609760</a></td><td align="right">2013</td><td>FVKLKKILNIILSIFKK</td><td>LK2(6)AN(2L)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Anticancer</td><td>LC<sub>50</sub> =28.34μM</td><td>Human</td><td>CCCTHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2576&amp;di=1" width="5px" height="5px"><img src="pymol_images/2576.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2576&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2576.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2577">2577</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23570790">23570790</a></td><td align="right">2013</td><td>WKSDVRRWRSRY</td><td>TSG-6</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0.3 ±0.9% hemolytic at 100μM</td><td>Human</td><td>CCSCHHHHTSCC</td><td align="center"><a href="jmol.php?id=2577&amp;di=1" width="5px" height="5px"><img src="pymol_images/2577.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2577&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2577.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2578">2578</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23570790">23570790</a></td><td align="right">2013</td><td>WKSYVRRWRSRY</td><td>TSG-7</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>1.5 ±1.4%hemolytic at 100μM</td><td>Human</td><td>CGGGGGTBTTBC</td><td align="center"><a href="jmol.php?id=2578&amp;di=1" width="5px" height="5px"><img src="pymol_images/2578.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2578&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2578.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2579">2579</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23570790">23570790</a></td><td align="right">2013</td><td>WKSYVRRWRSR</td><td>TSG-8</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>-1.2 ± 2.6%hemolytic at 100μM</td><td>Human</td><td>CGGGHHHHCCC</td><td align="center"><a href="jmol.php?id=2579&amp;di=1" width="5px" height="5px"><img src="pymol_images/2579.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2579&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2579.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2580">2580</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23570790">23570790</a></td><td align="right">2013</td><td>WWSYVRRWRSR</td><td>TSG-8-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>3.4 ±1.3% hemolytic at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2581">2581</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23570790">23570790</a></td><td align="right">2013</td><td>WKSYVRRWRS</td><td>TSG-9</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>-0.4 ± 0.5%hemolytic at 100μM</td><td>Human</td><td>CGGGGTCCCC</td><td align="center"><a href="jmol.php?id=2581&amp;di=1" width="5px" height="5px"><img src="pymol_images/2581.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2581&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2581.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2582">2582</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23570790">23570790</a></td><td align="right">2013</td><td>KKSYVRRWRS</td><td>TSG-9-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>-0.6 ± 1.6% hemolytic at 100μM</td><td>Human</td><td>CCSSCCCCCC</td><td align="center"><a href="jmol.php?id=2582&amp;di=1" width="5px" height="5px"><img src="pymol_images/2582.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2582&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2582.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2583">2583</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23570790">23570790</a></td><td align="right">2013</td><td>WKSYVRRWR</td><td>TSG-10</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>1.1 ±1.3% hemolytic at 100μM</td><td>Human</td><td>CCSCCCCCC</td><td align="center"><a href="jmol.php?id=2583&amp;di=1" width="5px" height="5px"><img src="pymol_images/2583.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2583&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2583.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2584">2584</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23570790">23570790</a></td><td align="right">2013</td><td>WKSYVRRW</td><td>TSG-11</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>2.0 ±0.3% hemolytic at 100μM</td><td>Human</td><td>CCCCCSCC</td><td align="center"><a href="jmol.php?id=2584&amp;di=1" width="5px" height="5px"><img src="pymol_images/2584.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2584&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2584.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2586">2586</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23519707">23519707</a></td><td align="right">2013</td><td>GLFDIIKKIAESF</td><td>AU</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>50% hemolytic at 128μmol/L</td><td>Human</td><td>CTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2586&amp;di=1" width="5px" height="5px"><img src="pymol_images/2586.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2586&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2586.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2587">2587</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23519707">23519707</a></td><td align="right">2013</td><td>GLFDIIKKIAESF</td><td>AU</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 4-16μmol/L</td><td>Human</td><td>CTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2587&amp;di=1" width="5px" height="5px"><img src="pymol_images/2587.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2587&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2587.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2588">2588</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23519707">23519707</a></td><td align="right">2013</td><td>FSEAIKKIIDFLGEGLFDIIKKIAESF</td><td>E(AU)2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>~5% hemolytic at 128μmol/L</td><td>Human</td><td>CTHHHHTTTTTTCCCTTGGGGTTTSCC</td><td align="center"><a href="jmol.php?id=2588&amp;di=1" width="5px" height="5px"><img src="pymol_images/2588.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2588&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2588.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2589">2589</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23519707">23519707</a></td><td align="right">2013</td><td>FSEAIKKIIDFLGEGLFDIIKKIAESF</td><td>E(AU)2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>0% hemolytic at 4-32μmol/L</td><td>Human</td><td>CTHHHHTTTTTTCCCTTGGGGTTTSCC</td><td align="center"><a href="jmol.php?id=2589&amp;di=1" width="5px" height="5px"><img src="pymol_images/2589.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2589&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2589.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2590">2590</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23519707">23519707</a></td><td align="right">2013</td><td>GLFDIIKKIAESFKFSEAIKKIIDFLG</td><td>(AU)2K</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>~30% hemolytic at 32μmol/L</td><td>Human</td><td>CGGGHHHHHHHHCSSTHHHHGGGTTCC</td><td align="center"><a href="jmol.php?id=2590&amp;di=1" width="5px" height="5px"><img src="pymol_images/2590.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2590&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2590.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2591">2591</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23519707">23519707</a></td><td align="right">2013</td><td>GLFDIIKKIAESFKFSEAIKKIIDFLG</td><td>(AU)2K</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>10% hemolytic at 4μmol/L</td><td>Human</td><td>CGGGHHHHHHHHCSSTHHHHGGGTTCC</td><td align="center"><a href="jmol.php?id=2591&amp;di=1" width="5px" height="5px"><img src="pymol_images/2591.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2591&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2591.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2592">2592</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23344198">23344198</a></td><td align="right">2013</td><td>WLKKLLKKLLK</td><td>L<sub>5</sub>K<sub>5<sub>W <sup>1</sup></sub></sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial and Hemolytic</td><td>MHC =32μg/mL</td><td>Human</td><td>CHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2592&amp;di=1" width="5px" height="5px"><img src="pymol_images/2592.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2592&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2592.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2593">2593</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23344198">23344198</a></td><td align="right">2013</td><td>KWLKKLLKKLL</td><td>L<sub>5</sub>K<sub>5<sub>W <sup>2</sup></sub></sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial and Hemolytic</td><td>MHC =16μg/mL</td><td>Human</td><td>CCHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2593&amp;di=1" width="5px" height="5px"><img src="pymol_images/2593.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2593&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2593.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2594">2594</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23344198">23344198</a></td><td align="right">2013</td><td>LKWLKKLLKKL</td><td>L<sub>5</sub>K<sub>5</sub>W <sup>3</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial and Hemolytic</td><td>MHC =64μg/mL</td><td>Human</td><td>CHHHHTTGGGC</td><td align="center"><a href="jmol.php?id=2594&amp;di=1" width="5px" height="5px"><img src="pymol_images/2594.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2594&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2594.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2595">2595</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23344198">23344198</a></td><td align="right">2013</td><td>LLKWLKKLLKK</td><td>L<sub>5</sub>K<sub>5</sub>W <sup>4</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial and Hemolytic</td><td>MHC =128μg/mL</td><td>Human</td><td>CHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2595&amp;di=1" width="5px" height="5px"><img src="pymol_images/2595.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2595&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2595.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2596">2596</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23344198">23344198</a></td><td align="right">2013</td><td>KLLKWLKKLLK</td><td>L<sub>5</sub>K<sub>5</sub>W <sup>5</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial and Hemolytic</td><td>MHC =64μg/mL</td><td>Human</td><td>CTHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2596&amp;di=1" width="5px" height="5px"><img src="pymol_images/2596.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2596&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2596.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2597">2597</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23344198">23344198</a></td><td align="right">2013</td><td>KKLLKWLKKLL</td><td>L<sub>5</sub>K<sub>5</sub>W <sup>6</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial and Hemolytic</td><td>MHC =32μg/mL</td><td>Human</td><td>CTTHHHHTTTC</td><td align="center"><a href="jmol.php?id=2597&amp;di=1" width="5px" height="5px"><img src="pymol_images/2597.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2597&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2597.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2598">2598</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23344198">23344198</a></td><td align="right">2013</td><td>LKKLLKWLKKL</td><td>L<sub>5</sub>K<sub>5</sub>W <sup>7</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial and Hemolytic</td><td>MHC =128μg/mL</td><td>Human</td><td>CHHHHTTTTTC</td><td align="center"><a href="jmol.php?id=2598&amp;di=1" width="5px" height="5px"><img src="pymol_images/2598.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2598&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2598.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2599">2599</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23344198">23344198</a></td><td align="right">2013</td><td>LLKKLLKWLKK</td><td>L<sub>5</sub>K<sub>5</sub>W <sup>8</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial and Hemolytic</td><td>MHC =128μg/mL</td><td>Human</td><td>CHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2599&amp;di=1" width="5px" height="5px"><img src="pymol_images/2599.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2599&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2599.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2600">2600</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23344198">23344198</a></td><td align="right">2013</td><td>KLLKKLLKWLK</td><td>L<sub>5</sub>K<sub>5</sub>W <sup>9</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial and Hemolytic</td><td>MHC =64μg/mL</td><td>Human</td><td>CTHHHHTTTTC</td><td align="center"><a href="jmol.php?id=2600&amp;di=1" width="5px" height="5px"><img src="pymol_images/2600.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2600&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2600.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2601">2601</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23344198">23344198</a></td><td align="right">2013</td><td>KKLLKKLLKWL</td><td>L<sub>5</sub>K<sub>5</sub>W <sup>10</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial and Hemolytic</td><td>MHC =32μg/mL</td><td>Human</td><td>CTTHHHHTTCC</td><td align="center"><a href="jmol.php?id=2601&amp;di=1" width="5px" height="5px"><img src="pymol_images/2601.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2601&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2601.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2602">2602</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23344198">23344198</a></td><td align="right">2013</td><td>LKKLLKKLLKW</td><td>L<sub>5</sub>K<sub>5</sub>W <sup>11</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial and Hemolytic</td><td>MHC =128μg/mL</td><td>Human</td><td>CHHHHTTTTTC</td><td align="center"><a href="jmol.php?id=2602&amp;di=1" width="5px" height="5px"><img src="pymol_images/2602.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2602&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2602.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2604">2604</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22914980">22914980</a></td><td align="right">2013</td><td>KWKWKW</td><td>(KW)<sub>3</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">6</td><td>Antimicrobial</td><td>0% hemolytic at 200μM</td><td>Human</td><td>CCCCCC</td><td align="center"><a href="jmol.php?id=2604&amp;di=1" width="5px" height="5px"><img src="pymol_images/2604.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2604&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2604.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2605">2605</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22914980">22914980</a></td><td align="right">2013</td><td>KWKWKWKW</td><td>(KW)<sub>4</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>8% hemolytic at 200μM</td><td>Human</td><td>CCSSSSCC</td><td align="center"><a href="jmol.php?id=2605&amp;di=1" width="5px" height="5px"><img src="pymol_images/2605.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2605&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2605.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2606">2606</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22914980">22914980</a></td><td align="right">2013</td><td>KWKWKWKWKW</td><td>(KW)<sub>5</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>71% hemolytic at 200μM</td><td>Human</td><td>CCCCCCSSCC</td><td align="center"><a href="jmol.php?id=2606&amp;di=1" width="5px" height="5px"><img src="pymol_images/2606.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2606&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2606.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2607">2607</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23226256">23226256</a></td><td align="right">2013</td><td>FDWDSVLKGVEGFVRGYF</td><td>TP1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>0% hemolytic at 100 µg/ml (non-hemolytic)</td><td>Fish</td><td>CCSHHHHHHHHTHHHHCC</td><td align="center"><a href="jmol.php?id=2607&amp;di=1" width="5px" height="5px"><img src="pymol_images/2607.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2607&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2607.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2608">2608</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23226256">23226256</a></td><td align="right">2013</td><td>GECIWDAIFHGAKHFLHRLVNP</td><td>TP2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>60% hemolytic at 100µg/ml</td><td>Fish</td><td>CCSSHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2608&amp;di=1" width="5px" height="5px"><img src="pymol_images/2608.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2608&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2608.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2609">2609</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23226256">23226256</a></td><td align="right">2013</td><td>FIHHIIGGLFSVGKHIHSLIHGH</td><td>TP3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>42% hemolytic at 100 µg/ml</td><td>Fish</td><td>CTTTTTTTTSSTTTTHHHHSCCC</td><td align="center"><a href="jmol.php?id=2609&amp;di=1" width="5px" height="5px"><img src="pymol_images/2609.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2609&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2609.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2610">2610</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23226256">23226256</a></td><td align="right">2013</td><td>FIHHIIGGLFSAGKAIHRLIRRRRR</td><td>TP4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>100% hemolytic at 100 µg/ml</td><td>Fish</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2611">2611</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23226256">23226256</a></td><td align="right">2013</td><td>QLQGKQVSGEVVQKVLQELIQSVAKP</td><td>TP5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>0% hemolytic at 100μg/ml (non-hemolytic)</td><td>Fish</td><td>CTTSCCSSSHHHHHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=2611&amp;di=1" width="5px" height="5px"><img src="pymol_images/2611.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2611&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2611.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2613">2613</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23124812">23124812</a></td><td align="right">2013</td><td>FSGGNCRGFRRRCFCTK</td><td>SolyC</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>&lt;5% hemolytic at 50mg/ml</td><td>Mouse</td><td>CCSSCCCSSSSCCSSCC</td><td align="center"><a href="jmol.php?id=2613&amp;di=1" width="5px" height="5px"><img src="pymol_images/2613.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2613&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2613.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2614">2614</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23093034">23093034</a></td><td align="right">2013</td><td>SWKSMAKKLKEYMEKLKQRA</td><td>La47</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>2% hemolytic at 25mM</td><td>Human</td><td>CHHHHHHHHHHHHTTTTTCC</td><td align="center"><a href="jmol.php?id=2614&amp;di=1" width="5px" height="5px"><img src="pymol_images/2614.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2614&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2614.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2615">2615</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23093034">23093034</a></td><td align="right">2013</td><td>FFGSLLSLGSKLLPSVFKLFQRKKE</td><td>Css54</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>83% hemolytic at 25mM</td><td>Human</td><td>CTTSSSSSCCCSCHHHHHHHSSSCC</td><td align="center"><a href="jmol.php?id=2615&amp;di=1" width="5px" height="5px"><img src="pymol_images/2615.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2615&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2615.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2616">2616</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23019445">23019445</a></td><td align="right">2012</td><td>KWASLWNWFNITNWLWYIK</td><td>gp41w</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>EC<sub>50</sub> =550μg/ml</td><td>Human</td><td>NA</td><td align="center"><a href="jmol.php?id=2616&amp;di=1" width="5px" height="5px"><img src="pymol_images/2616.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2616&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2616.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2617">2617</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23019445">23019445</a></td><td align="right">2012</td><td>KWARLWRWFRITRWLWYIK</td><td>gp41w-4R</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>EC<sub>50</sub> =7-14μg/ml</td><td>Human</td><td>CTHHHHHHHHHHTTSCSCC</td><td align="center"><a href="jmol.php?id=2617&amp;di=1" width="5px" height="5px"><img src="pymol_images/2617.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2617&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2617.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2618">2618</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23019445">23019445</a></td><td align="right">2012</td><td>KWAKKWKWFAKAAWKWYKK</td><td>gp41w-KA</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>EC<sub>50</sub> =42μg/ml</td><td>Human</td><td>CTHHHHTTHHHHTHHHHTC</td><td align="center"><a href="jmol.php?id=2618&amp;di=1" width="5px" height="5px"><img src="pymol_images/2618.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2618&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2618.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2619">2619</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23019445">23019445</a></td><td align="right">2012</td><td>KFAKKFKWFAKAAFKFFKK</td><td>gp41w-FKA</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>EC<sub>50</sub> =195μg/ml</td><td>Human</td><td>CCSHHHHHTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2619&amp;di=1" width="5px" height="5px"><img src="pymol_images/2619.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2619&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2619.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2621">2621</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22865764">22865764</a></td><td align="right">2012</td><td>QCRRLCYKQRCVTYCRGR</td><td>c(1-18) [Gln<sup>1</sup>]-Gm</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>35.5% hemolytic at 100µM</td><td>Human</td><td>NA</td><td align="center"><a href="jmol.php?id=2621&amp;di=1" width="5px" height="5px"><img src="pymol_images/2621.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2621&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2621.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2622">2622</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22865764">22865764</a></td><td align="right">2012</td><td>QC*RRLCYKQRCVTYC*RGR</td><td>c(1-18) [Gln<sup>1</sup>,Cys(Acm)<sup>2,15</sup>]-Gm</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>C*: Cystein is protected by Acm</td><td align="right">18</td><td>Antimicrobial</td><td>63% hemolytic at 100µM</td><td>Human</td><td>NA</td><td align="center"><a href="jmol.php?id=2622&amp;di=1" width="5px" height="5px"><img src="pymol_images/2622.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2622&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2622.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2623">2623</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22865764">22865764</a></td><td align="right">2012</td><td>QTRRLTYKpRTVTYTRGR</td><td>c(1-18) [Gln<sup>1</sup>,Thr<sup>2,6,11,15</sup>D-Pro<sup>9</sup>]-Gm</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>26.5% hemolytic at 100µM</td><td>Human</td><td>CTTSSCSSSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=2623&amp;di=1" width="5px" height="5px"><img src="pymol_images/2623.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2623&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2623.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2624">2624</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22869378">22869378</a></td><td align="right">2012</td><td>kklallalkkwllalkklallalkk</td><td>D-LAK120</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">25</td><td>Antimicrobial and Antiplasmodial and Anticancer</td><td>HC<sub>50</sub> =4.06±1.30μM</td><td>Human</td><td>CCCHHHHHHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2624&amp;di=1" width="5px" height="5px"><img src="pymol_images/2624.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2624&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2624.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2625">2625</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22869378">22869378</a></td><td align="right">2012</td><td>kklallalkkwlpalkklallalkk</td><td>D-LAK120-P13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">25</td><td>Antimicrobial and Antiplasmodial and Anticancer</td><td>HC<sub>50</sub> =11.66±0.03μM</td><td>Human</td><td>CCSHHHHHHHHHHHHHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=2625&amp;di=1" width="5px" height="5px"><img src="pymol_images/2625.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2625&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2625.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2626">2626</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22869378">22869378</a></td><td align="right">2012</td><td>kklalalakkwlalakklalalakk</td><td>D-LAK120-A</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">25</td><td>Antimicrobial and Antiplasmodial and Anticancer</td><td>HC<sub>50</sub> =17.42±2.17μM</td><td>Human</td><td>CHHHHHHHHHHHHHHHHHHHTGGGC</td><td align="center"><a href="jmol.php?id=2626&amp;di=1" width="5px" height="5px"><img src="pymol_images/2626.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2626&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2626.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2627">2627</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22869378">22869378</a></td><td align="right">2012</td><td>kklalalakkwlplakklalalakk</td><td>D-LAK120-AP13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">25</td><td>Antimicrobial and Antiplasmodial and Anticancer</td><td>HC<sub>50</sub> =58.61±10.41μMc</td><td>Human</td><td>CHHHHHTGGGGHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2627&amp;di=1" width="5px" height="5px"><img src="pymol_images/2627.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2627&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2627.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2628">2628</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22869378">22869378</a></td><td align="right">2012</td><td>kklalhalkkwlhalkklahlalkk</td><td>D-LAK120-H</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">25</td><td>Antimicrobial and Antiplasmodial and Anticancer</td><td>HC<sub>50</sub> =20.29±2.42μM</td><td>Human</td><td>CCSHHHHHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2628&amp;di=1" width="5px" height="5px"><img src="pymol_images/2628.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2628&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2628.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2629">2629</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22869378">22869378</a></td><td align="right">2012</td><td>kkalahalkkwlpalkklahalakk</td><td>D-LAK120-HP13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">25</td><td>Antimicrobial and Antiplasmodial and Anticancer</td><td>HC<sub>50</sub> =79.61±2.80μM</td><td>Human</td><td>CTTHHHHTGGGHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2629&amp;di=1" width="5px" height="5px"><img src="pymol_images/2629.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2629&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2629.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2630">2630</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22869378">22869378</a></td><td align="right">2012</td><td>kkalakalkhwlpalhklakalakk</td><td>D-LAK80-HP13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">25</td><td>Antimicrobial and Antiplasmodial and Anticancer</td><td>HC<sub>50</sub> =8.61±0.35μM</td><td>Human</td><td>CHHHHHHHTTHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2630&amp;di=1" width="5px" height="5px"><img src="pymol_images/2630.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2630&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2630.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2631">2631</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22869378">22869378</a></td><td align="right">2012</td><td>kkalkhalakwlpalkalahklakk</td><td>D-LAK160-HP13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">25</td><td>Antimicrobial and Antiplasmodial and Anticancer</td><td>HC<sub>50</sub> =185.0±18.65μM</td><td>Human</td><td>CTTHHHHGGGTHHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=2631&amp;di=1" width="5px" height="5px"><img src="pymol_images/2631.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2631&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2631.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2634">2634</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFLKTFKSLKKTVLHTLLKAISS</td><td>P</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =5.20±0.02μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2634&amp;di=1" width="5px" height="5px"><img src="pymol_images/2634.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2634&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2634.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2635">2635</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFLkTFKSLKKTVLHTLLKAISS</td><td>K7<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =10.4±0.04μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2635&amp;di=1" width="5px" height="5px"><img src="pymol_images/2635.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2635&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2635.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2636">2636</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFLKTFKSLKkTVLHTLLKAISS</td><td>K14<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =5.20±0.02μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2636&amp;di=1" width="5px" height="5px"><img src="pymol_images/2636.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2636&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2636.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2637">2637</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFLKTFKSLKKTVLHTLLkAISS</td><td>K22<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =20.81±0.10 μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2637&amp;di=1" width="5px" height="5px"><img src="pymol_images/2637.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2637&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2637.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2638">2638</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFLkTFKSLKkTVLHTLLKAISS</td><td>K7<sub>D</sub>K14 <sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =20.81±0.15μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2638&amp;di=1" width="5px" height="5px"><img src="pymol_images/2638.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2638&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2638.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2639">2639</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFLKTFKSLKkTVLHTLLkAISS</td><td>K14<sub>D</sub>/K22<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =20.81±0.06μmol/L </td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2639&amp;di=1" width="5px" height="5px"><img src="pymol_images/2639.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2639&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2639.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2640">2640</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFLkTFKSLKkTVLHTLLkAISS</td><td>K7<sub>D</sub>/K14<sub>D</sub>/K22<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =81.31±0.17μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2640&amp;di=1" width="5px" height="5px"><img src="pymol_images/2640.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2640&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2640.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2641">2641</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFLkTFkSLKkTVLHTLLkAISS</td><td>K7<sub>D</sub>/K10<sub>D</sub>/K14<sub>D</sub>/K22<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =325.20±0.82μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2641&amp;di=1" width="5px" height="5px"><img src="pymol_images/2641.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2641&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2641.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2642">2642</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWkSFLkTFkSLKkTVLHTLLkAISS</td><td>K3<sub>D</sub>/K7<sub>D</sub>/K10<sub>D</sub>/K14<sub>D</sub>/K22<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC &gt;325.20μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2642&amp;di=1" width="5px" height="5px"><img src="pymol_images/2642.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2642&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2642.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2643">2643</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>kWkSFLkTFkSLKkTVLHTLLkAISS</td><td>K1<sub>D</sub>/K3<sub>D</sub>/K7<sub>D</sub>/K10<sub>D</sub>/K14<sub>D</sub>/K22<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC &gt;325.20μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2643&amp;di=1" width="5px" height="5px"><img src="pymol_images/2643.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2643&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2643.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2644">2644</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFlKTFKSLKKTVLHTLLKAISS</td><td>L6<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =10.40±0.08μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2644&amp;di=1" width="5px" height="5px"><img src="pymol_images/2644.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2644&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2644.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2645">2645</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFLKTFKSlKKTVLHTLLKAISS</td><td>L12<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =20.81±0.03μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2645&amp;di=1" width="5px" height="5px"><img src="pymol_images/2645.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2645&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2645.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2646">2646</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFLKTFKSLKKTVLHTlLKAISS</td><td>L20<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =20.81±0.13μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2646&amp;di=1" width="5px" height="5px"><img src="pymol_images/2646.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2646&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2646.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2647">2647</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFlKTFKSlKKTVLHTLLKAISS</td><td>L6<sub>D</sub>/L12<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =20.81±0.10 μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2647&amp;di=1" width="5px" height="5px"><img src="pymol_images/2647.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2647&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2647.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2648">2648</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFLKTFKSlKKTVLHTlLKAISS</td><td>L12<sub>D</sub>/L20<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =81.31±0.43μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2648&amp;di=1" width="5px" height="5px"><img src="pymol_images/2648.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2648&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2648.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2649">2649</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFlKTFKSlKKTVLHTlLKAISS</td><td>L6<sub>D</sub>/L12<sub>D</sub>/L20<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =162.61±0.19μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2649&amp;di=1" width="5px" height="5px"><img src="pymol_images/2649.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2649&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2649.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2650">2650</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFlKTFKSlKKTVLHTlLKAISS</td><td>L6<sub>D</sub>/L12<sub>D</sub>/L17<sub>D</sub>/L20<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC =81.31±1.05μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2650&amp;di=1" width="5px" height="5px"><img src="pymol_images/2650.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2650&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2650.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2651">2651</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22837667">22837667</a></td><td align="right">2012</td><td>KWKSFlKTFKSlKKTVLHTllKAISS</td><td>L6<sub>D</sub>/L12<sub>D</sub>/L17<sub>D</sub>/L20<sub>D</sub>/L21<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>MHC &gt;325.20μmol/L</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2651&amp;di=1" width="5px" height="5px"><img src="pymol_images/2651.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2651&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2651.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2652">2652</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22771964">22771964</a></td><td align="right">2013</td><td>VKVGINGFGRIGRLVTRAAFHGKKVEVVAIND</td><td>YFGAP-OH</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>0% hemolytic at 3.13μg/ml</td><td>Human</td><td>CCCSSSSCTTSTTGGGHHHHTSCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=2652&amp;di=1" width="5px" height="5px"><img src="pymol_images/2652.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2652&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2652.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2653">2653</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22771964">22771964</a></td><td align="right">2013</td><td>VKVGINGFGRIGRLVTRAAFHGKKVEVVAIND</td><td>YFGAP-OH</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>0% hemolytic at 6.25μg/ml</td><td>Human</td><td>CCCSSSSCTTSTTGGGHHHHTSCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=2653&amp;di=1" width="5px" height="5px"><img src="pymol_images/2653.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2653&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2653.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2654">2654</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22771964">22771964</a></td><td align="right">2013</td><td>VKVGINGFGRIGRLVTRAAFHGKKVEVVAIND</td><td>YFGAP-OH</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>0% hemolytic at 12.5μg/ml</td><td>Human</td><td>CCCSSSSCTTSTTGGGHHHHTSCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=2654&amp;di=1" width="5px" height="5px"><img src="pymol_images/2654.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2654&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2654.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2655">2655</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22771964">22771964</a></td><td align="right">2013</td><td>VKVGINGFGRIGRLVTRAAFHGKKVEVVAIND</td><td>YFGAP-OH</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>0.1% hemolytic at 25.0μg/ml</td><td>Human</td><td>CCCSSSSCTTSTTGGGHHHHTSCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=2655&amp;di=1" width="5px" height="5px"><img src="pymol_images/2655.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2655&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2655.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2656">2656</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22771964">22771964</a></td><td align="right">2013</td><td>VKVGINGFGRIGRLVTRAAFHGKKVEVVAIND</td><td>YFGAP-OH</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>0.1 hemolytic at 50.0μg/ml</td><td>Human</td><td>CCCSSSSCTTSTTGGGHHHHTSCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=2656&amp;di=1" width="5px" height="5px"><img src="pymol_images/2656.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2656&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2656.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2657">2657</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22771964">22771964</a></td><td align="right">2013</td><td>VKVGINGFGRIGRLVTRAAFHGKKVEVVAIND</td><td>YFGAP-OH</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>0.4% hemolytic at 100mg/mL</td><td>Human</td><td>CCCSSSSCTTSTTGGGHHHHTSCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=2657&amp;di=1" width="5px" height="5px"><img src="pymol_images/2657.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2657&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2657.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2658">2658</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22771964">22771964</a></td><td align="right">2013</td><td>VKVGINGFGRIGRLVTRAAFHGKKVEVVAIND</td><td>YFGAP-NH2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>0% hemolytic at 3.13μg/ml</td><td>Human</td><td>CCSSSCCSSCCCTTHHHHSSSSCSCSBTTBCC</td><td align="center"><a href="jmol.php?id=2658&amp;di=1" width="5px" height="5px"><img src="pymol_images/2658.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2658&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2658.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2659">2659</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22771964">22771964</a></td><td align="right">2013</td><td>VKVGINGFGRIGRLVTRAAFHGKKVEVVAIND</td><td>YFGAP-NH2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>0% hemolytic at 6.25μg/ml</td><td>Human</td><td>CCSSSCCSSCCCTTHHHHSSSSCSCSBTTBCC</td><td align="center"><a href="jmol.php?id=2659&amp;di=1" width="5px" height="5px"><img src="pymol_images/2659.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2659&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2659.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2660">2660</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22771964">22771964</a></td><td align="right">2013</td><td>VKVGINGFGRIGRLVTRAAFHGKKVEVVAIND</td><td>YFGAP-NH2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>2.3% hemolytic at 12.5μg/ml</td><td>Human</td><td>CCSSSCCSSCCCTTHHHHSSSSCSCSBTTBCC</td><td align="center"><a href="jmol.php?id=2660&amp;di=1" width="5px" height="5px"><img src="pymol_images/2660.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2660&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2660.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2661">2661</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22771964">22771964</a></td><td align="right">2013</td><td>VKVGINGFGRIGRLVTRAAFHGKKVEVVAIND</td><td>YFGAP-NH2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>2.5% hemolytic at 25μg/ml</td><td>Human</td><td>CCSSSCCSSCCCTTHHHHSSSSCSCSBTTBCC</td><td align="center"><a href="jmol.php?id=2661&amp;di=1" width="5px" height="5px"><img src="pymol_images/2661.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2661&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2661.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2662">2662</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22771964">22771964</a></td><td align="right">2013</td><td>VKVGINGFGRIGRLVTRAAFHGKKVEVVAIND</td><td>YFGAP-NH2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>2.6% hemolytic at 50μg/ml</td><td>Human</td><td>CCSSSCCSSCCCTTHHHHSSSSCSCSBTTBCC</td><td align="center"><a href="jmol.php?id=2662&amp;di=1" width="5px" height="5px"><img src="pymol_images/2662.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2662&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2662.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2663">2663</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22771964">22771964</a></td><td align="right">2013</td><td>VKVGINGFGRIGRLVTRAAFHGKKVEVVAIND</td><td>YFGAP-NH2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>2.7% hemolytic at 100mg/mL</td><td>Human</td><td>CCCSSSSCTTSTTGGGHHHHTSCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=2663&amp;di=1" width="5px" height="5px"><img src="pymol_images/2663.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2663&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2663.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2664">2664</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22699557">22699557</a></td><td align="right">2012</td><td>IWRIFRRIFRIF</td><td>HFU4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>MHC =19.6μM</td><td>Human</td><td>CHHHHTHHHHCC</td><td align="center"><a href="jmol.php?id=2664&amp;di=1" width="5px" height="5px"><img src="pymol_images/2664.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2664&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2664.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2665">2665</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22699557">22699557</a></td><td align="right">2012</td><td>IWRIFRRIFRIFIRF</td><td>HFU5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>MHC =15μM</td><td>Human</td><td>CHHHHTTTTTTSCCC</td><td align="center"><a href="jmol.php?id=2665&amp;di=1" width="5px" height="5px"><img src="pymol_images/2665.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2665&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2665.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2666">2666</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22670762">22670762</a></td><td align="right">2013</td><td>GLWDTIKQAGKKFFLNVLDKIRCKVAGGCRT</td><td>OG1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">31</td><td>Antimicrobial</td><td>80% hemolytic at 256μg/mL</td><td>Porcine</td><td>CCTTHHHHHHHHCHHHHHHHTTTTSSCCCCC</td><td align="center"><a href="jmol.php?id=2666&amp;di=1" width="5px" height="5px"><img src="pymol_images/2666.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2666&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2666.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2667">2667</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22670762">22670762</a></td><td align="right">2013</td><td>GLWDTIKQAGKKFFLNVLDKIRCKVAGGCRT</td><td>OG1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">31</td><td>Antimicrobial</td><td>10% hemolytic at 4μg/mL</td><td>Porcine</td><td>CCTTHHHHHHHHCHHHHHHHTTTTSSCCCCC</td><td align="center"><a href="jmol.php?id=2667&amp;di=1" width="5px" height="5px"><img src="pymol_images/2667.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2667&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2667.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2668">2668</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22670762">22670762</a></td><td align="right">2013</td><td>KKFFLKVLTKIRCKVAGGCRT</td><td>OG2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>10% hemolytic at 256μg/mL</td><td>Porcine</td><td>CGGGHHHHHHHTSSCSTTSCC</td><td align="center"><a href="jmol.php?id=2668&amp;di=1" width="5px" height="5px"><img src="pymol_images/2668.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2668&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2668.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2671">2671</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22792229">22792229</a></td><td align="right">2012</td><td>FIKRIARLLRKIF</td><td>Kn2-7</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>HC<sub>50</sub> =90.27µg/mL</td><td>Human</td><td>CHHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=2671&amp;di=1" width="5px" height="5px"><img src="pymol_images/2671.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2671&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2671.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2672">2672</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22792229">22792229</a></td><td align="right">2012</td><td>FIGAIARLLSKIF</td><td>BmKn2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>HC<sub>50</sub> =17.13µg/mL</td><td>Human</td><td>CHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2672&amp;di=1" width="5px" height="5px"><img src="pymol_images/2672.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2672&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2672.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2673">2673</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKIKIGAKIKIGAKI</td><td>KIGAKI-wt</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>~14% hemolytic at 7.5μM</td><td>Human</td><td>CCSEESSSSEEECSSSCC</td><td align="center"><a href="jmol.php?id=2673&amp;di=1" width="5px" height="5px"><img src="pymol_images/2673.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2673&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2673.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2674">2674</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKIKIGAKIKIGAKI</td><td>KIGAKI-wt</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>~54% hemolytic at 60μM</td><td>Human</td><td>CCSEESSSSEEECSSSCC</td><td align="center"><a href="jmol.php?id=2674&amp;di=1" width="5px" height="5px"><img src="pymol_images/2674.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2674&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2674.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2675">2675</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAK-<sub>L</sub>-CF3-Bpg-KIGAKIKIGAKI</td><td>KIGAKI-6L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~23% hemolytic at 7.5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2676">2676</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAK-<sub>L</sub>-CF<sub>3</sub>-Bpg-KIGAKIKIGAKI</td><td>KIGAKI-6L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~50% hemolytic at 60μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2677">2677</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAK-<sub>D</sub>-CF<sub>3</sub>-Bpg-KIGAKIKIGAKI</td><td>KIGAKI-6D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~7% hemolysis at 7.5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2678">2678</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAK-<sub>D</sub>-CF<sub>3</sub>-Bpg-KIGAKIKIGAKI</td><td>KIGAKI-6D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~10% hemolytic at 60μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2679">2679</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKIK-<sub>L</sub>-CF<sub>3</sub>-Bpg-GAKIKIGAKI</td><td>KIGAKI-8L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~13% hemolytic at 7.5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2680">2680</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKIK-<sub>L</sub>-CF<sub>3</sub>-Bpg-GAKIKIGAKI</td><td>KIGAKI-8L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~38% hemolytic at 60μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2681">2681</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKIK-<sub>D</sub>-CF<sub>3</sub>-Bpg-GAKIKIGAKI</td><td>KIGAKI-8D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~5% hemolytic at 7.5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2682">2682</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKIK-<sub>D</sub>-CF<sub>3</sub>-Bpg-GAKIKIGAKI</td><td>KIGAKI-8D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~8% hemolytic at 60μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2683">2683</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKIKIGAK-<sub>L</sub>-CF<sub>3</sub>-Bpg-KIGAKI</td><td>KIGAKI-12L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~17% hemolytic at 7.5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2684">2684</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKIKIGAK-<sub>L</sub>-CF<sub>3</sub>-Bpg-KIGAKI</td><td>KIGAKI-12L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~38% hemolytic at 60μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2685">2685</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKI-12DKIGAKIKIGAK-<sub>D</sub>-CF<sub>3</sub>-Bpg-KIGAKI</td><td>KIGAKI-12D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~5% hemolysis at 7.5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2686">2686</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKI-12DKIGAKIKIGAK-<sub>D</sub>-CF<sub>3</sub>-Bpg-KIGAKI</td><td>KIGAKI-12D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~7% hemolysis at 60μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2687">2687</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKIKIGAKIK-<sub>L</sub>-CF<sub>3</sub>-Bpg-GAKI</td><td>KIGAKI-14L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~13% hemolysis at 7.5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2688">2688</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKIKIGAKIK-<sub>L</sub>-CF<sub>3</sub>-Bpg-GAKI</td><td>KIGAKI-14L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~34% hemolytic at 60μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2689">2689</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKIKIGAKIK-<sub>D</sub>-CF<sub>3</sub>-Bpg-GAKI</td><td>KIGAKI-14D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~6% hemolysis at 7.5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2690">2690</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23652359">23652359</a></td><td align="right">2013</td><td>KIGAKIKIGAKIK-<sub>D</sub>-CF<sub>3</sub>-Bpg-GAKI</td><td>KIGAKI-14D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>CF3-Bpg ie [3-(trifluoromethyl)-bicyclopent-[1.1.1]-1-ylglycine]</td><td align="right">17</td><td>Antimicrobial</td><td>~7% hemolysis at 60μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2691">2691</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22029824">22029824</a></td><td align="right">2012</td><td>TSRCIFYRRKKCS</td><td>Andersonin-C1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>15.2±2.1% hemolytic at 50μg/mL</td><td>Human</td><td>CCSCCCCTTSCCC</td><td align="center"><a href="jmol.php?id=2691&amp;di=1" width="5px" height="5px"><img src="pymol_images/2691.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2691&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2691.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2692">2692</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22083756">22083756</a></td><td align="right">2012</td><td>CLRIGMRGRELMGGIGKTM</td><td>pCM19</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>0.109% hemolytic at 100μg/mL</td><td>Human</td><td>CCCSCCEESSSSEECCCCC</td><td align="center"><a href="jmol.php?id=2692&amp;di=1" width="5px" height="5px"><img src="pymol_images/2692.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2692&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2692.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2693">2693</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22083756">22083756</a></td><td align="right">2012</td><td>CLRIGMRGRELMGGIGKTM</td><td>pCM19</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>0.119% hemolytic at 3000μg/mL</td><td>Human</td><td>CCCSCCEESSSSEECCCCC</td><td align="center"><a href="jmol.php?id=2693&amp;di=1" width="5px" height="5px"><img src="pymol_images/2693.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2693&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2693.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2694">2694</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22249992">22249992</a></td><td align="right">2012</td><td>NWCKRGRKQCKTHPH</td><td>NWC15l</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>~2% hemolytic at 60μM and 6μM</td><td>Human</td><td>CCSTTTTTTCCCCCC</td><td align="center"><a href="jmol.php?id=2694&amp;di=1" width="5px" height="5px"><img src="pymol_images/2694.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2694&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2694.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2695">2695</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22249992">22249992</a></td><td align="right">2012</td><td>NWCKRGRKQCKTHPH</td><td>NWC15c</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>~2% hemolytic at 6μM</td><td>Human</td><td>CCSTTTTTTCCCCCC</td><td align="center"><a href="jmol.php?id=2695&amp;di=1" width="5px" height="5px"><img src="pymol_images/2695.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2696">2696</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22249992">22249992</a></td><td align="right">2012</td><td>NWCKRGRKQCKTHPH</td><td>NWC15c</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>~3% hemolytic at 60μM</td><td>Human</td><td>CCSTTTTTTCCCCCC</td><td align="center"><a href="jmol.php?id=2696&amp;di=1" width="5px" height="5px"><img src="pymol_images/2696.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2697">2697</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22249992">22249992</a></td><td align="right">2012</td><td>NWCKRGRKQCKTHPHFVIPY</td><td>NWC20c</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>~4% hemolytic at 6μM</td><td>Human</td><td>CCSSCSTTTSSSSSSCSSCC</td><td align="center"><a href="jmol.php?id=2697&amp;di=1" width="5px" height="5px"><img src="pymol_images/2697.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2698">2698</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22249992">22249992</a></td><td align="right">2012</td><td>NWCKRGRKQCKTHPHFVIPY</td><td>NWC20c</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>~3% hemolytic at 60μM</td><td>Human</td><td>CCSSCSTTTSSSSSSCSSCC</td><td align="center"><a href="jmol.php?id=2698&amp;di=1" width="5px" height="5px"><img src="pymol_images/2698.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2700">2700</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>CPP</td><td>&gt;80% hemolytic at 25μM at pH 8</td><td>Human</td><td>CCCSSHHHHHHSCHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2700&amp;di=1" width="5px" height="5px"><img src="pymol_images/2700.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2700&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2700.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2701">2701</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>CPP</td><td>&lt;60% hemolytic at 25μM at pH 6</td><td>Human</td><td>CCCSSHHHHHHSCHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2701&amp;di=1" width="5px" height="5px"><img src="pymol_images/2701.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2701&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2701.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2702">2702</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAVLKVLTTGLPALISWI</td><td>MT20</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>CPP</td><td>&lt;80% hemolytic at 25μM at pH 8</td><td>Human</td><td>CCSBGGGCSSCCCSSSSSBC</td><td align="center"><a href="jmol.php?id=2702&amp;di=1" width="5px" height="5px"><img src="pymol_images/2702.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2702&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2702.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2703">2703</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAVLKVLTTGLPALISWI</td><td>MT20</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>CPP</td><td>40% hemolytic at 25μM at pH 5</td><td>Human</td><td>CCSBGGGCSSCCCSSSSSBC</td><td align="center"><a href="jmol.php?id=2703&amp;di=1" width="5px" height="5px"><img src="pymol_images/2703.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2703&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2703.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2704">2704</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAILKVLATGLPTLISWIKNKRKQ</td><td>Florae</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>CPP</td><td>&lt;70% hemolytic at 25μM at pH 6 </td><td>Human</td><td>CCTTSHHHHTTTSHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2704&amp;di=1" width="5px" height="5px"><img src="pymol_images/2704.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2704&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2704.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2705">2705</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAILKVLATGLPTLISWIKNKRKQ</td><td>Florae</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>CPP</td><td>~80% hemolytic at 25μM at pH 8</td><td>Human</td><td>CCTTSHHHHTTTSHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2705&amp;di=1" width="5px" height="5px"><img src="pymol_images/2705.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2705&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2705.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2706">2706</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAILKVLATGLPTLISWI</td><td>FL20</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>CPP</td><td>&lt;60% hemolytic at 25μM at pH 5 </td><td>Human</td><td>CCTHHHHGGGSCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2706&amp;di=1" width="5px" height="5px"><img src="pymol_images/2706.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2706&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2706.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2707">2707</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22328546">22328546</a></td><td align="right">2012</td><td>GIGAILKVLATGLPTLISWI</td><td>FL20</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>CPP</td><td>~80% hemolytic at 25μM at pH 8</td><td>Human</td><td>CCTHHHHGGGSCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2707&amp;di=1" width="5px" height="5px"><img src="pymol_images/2707.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2707&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2707.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2708">2708</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22371493">22371493</a></td><td align="right">2012</td><td>MGIIAGIIKVIKSLIEQFTGK</td><td>PSMα1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Immunosuppressive and Antimicrobial</td><td>58% hemolytic at 10μg/mL</td><td>Human</td><td>CTHHHHHHHHHHTTGGGTTTC</td><td align="center"><a href="jmol.php?id=2708&amp;di=1" width="5px" height="5px"><img src="pymol_images/2708.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2708&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2708.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2709">2709</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22371493">22371493</a></td><td align="right">2012</td><td>GIIKVIKSLIEQFTGK</td><td>dPSMα1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Immunosuppressive and Antimicrobial</td><td>Non-hemolytic at 10μg/mL</td><td>Human</td><td>CHHHHHTHHHHGGGCC</td><td align="center"><a href="jmol.php?id=2709&amp;di=1" width="5px" height="5px"><img src="pymol_images/2709.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2709&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2709.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2710">2710</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22371493">22371493</a></td><td align="right">2012</td><td>MAIVGTIIKIIKAIIDIFAK</td><td>PSMα4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Immunosuppressive and Antimicrobial</td><td>19% hemolytic at 10μg/mL</td><td>Human</td><td>CTTTTTTHHHHHHHTTTTTC</td><td align="center"><a href="jmol.php?id=2710&amp;di=1" width="5px" height="5px"><img src="pymol_images/2710.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2710&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2710.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2711">2711</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22371493">22371493</a></td><td align="right">2012</td><td>VGTIIKIIKAIIDIFAK</td><td>dPSMα4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Immunosuppressive and Antimicrobial</td><td>52% hemolytic at 10μg/mL</td><td>Human</td><td>CCCTHHHHHHHTTTTCC</td><td align="center"><a href="jmol.php?id=2711&amp;di=1" width="5px" height="5px"><img src="pymol_images/2711.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2711&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2711.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2712">2712</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22391524">22391524</a></td><td align="right">2012</td><td>GWLDVAKKIGKAAFNVAKNFL</td><td>MON</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>HC<sub>50</sub> =37.3±1.4μmol/L</td><td>Human</td><td>CCHHHHHHHHHHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=2712&amp;di=1" width="5px" height="5px"><img src="pymol_images/2712.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2712&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2712.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2716">2716</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKLFKKIPKFLHLAKKF</td><td>P18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>MHC =250μg/mL</td><td>Human</td><td>CCCSCSCCHHHHTCSSCC</td><td align="center"><a href="jmol.php?id=2716&amp;di=1" width="5px" height="5px"><img src="pymol_images/2716.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2716&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2716.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2717">2717</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKLTSKFLHLAKKF</td><td>CP-P</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC =250μg/mL</td><td>Human</td><td>CHHHHHHHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=2717&amp;di=1" width="5px" height="5px"><img src="pymol_images/2717.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2717&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2717.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2718">2718</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKLTSKFLHSAKKF</td><td>S16</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &gt;500μg/mL</td><td>Human</td><td>CHHHHHHHHHHHHHHHSSCC</td><td align="center"><a href="jmol.php?id=2718&amp;di=1" width="5px" height="5px"><img src="pymol_images/2718.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2718&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2718.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2719">2719</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KFKSFIKKLTSKFLHSAKKF</td><td>F2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC =250μg/mL</td><td>Human</td><td>CCGGGHHHHHHHHHHHCCCC</td><td align="center"><a href="jmol.php?id=2719&amp;di=1" width="5px" height="5px"><img src="pymol_images/2719.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2719&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2719.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2720">2720</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFINKLTSKFLHSAKKF</td><td>N7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &gt;500μg/mL</td><td>Human</td><td>CCHHHHHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=2720&amp;di=1" width="5px" height="5px"><img src="pymol_images/2720.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2720&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2720.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2721">2721</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKSTSKFLHSAKKF</td><td>S9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &gt;500μg/mL</td><td>Human</td><td>CCGGGHHHHHHHHHHHCSCC</td><td align="center"><a href="jmol.php?id=2721&amp;di=1" width="5px" height="5px"><img src="pymol_images/2721.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2721&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2721.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2722">2722</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKAKTSFLHSAKKF</td><td>A9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &gt;500μg/mL </td><td>Human</td><td>CCGGGHHHHHHHTTSSCCCC</td><td align="center"><a href="jmol.php?id=2722&amp;di=1" width="5px" height="5px"><img src="pymol_images/2722.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2722&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2722.png" style="width:60px;height:60px;border:1;"></a></td></tr>'
test2 = '<tr class="even"><td align="right"><a href="display_sub.php?details=2723">2723</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKLLSKFLHSAKKF</td><td>L10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &lt;125μg/mL</td><td>Human</td><td>CHHHHHHHHHHHHHHHSSCC</td><td align="center"><a href="jmol.php?id=2723&amp;di=1" width="5px" height="5px"><img src="pymol_images/2723.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2723&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2723.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2724">2724</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKLASKFLHSAKKF</td><td>A10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC =250μg/mL</td><td>Human</td><td>CHHHHHHHHHHHHHHHCTTC</td><td align="center"><a href="jmol.php?id=2724&amp;di=1" width="5px" height="5px"><img src="pymol_images/2724.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2724&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2724.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2725">2725</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKLTDKFLHSAKKF</td><td>D11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &gt;500μg/mL</td><td>Human</td><td>CHHHHHHTTTTTTTTTSSCC</td><td align="center"><a href="jmol.php?id=2725&amp;di=1" width="5px" height="5px"><img src="pymol_images/2725.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2725&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2725.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2726">2726</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKLTKKFLHSAKKF</td><td>K11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &gt;500μg/mL</td><td>Human</td><td>CHHHHGGGTHHHHTTTCCCC</td><td align="center"><a href="jmol.php?id=2726&amp;di=1" width="5px" height="5px"><img src="pymol_images/2726.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2726&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2726.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2727">2727</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKLTSKALHSAKKF</td><td>A13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &gt;500μg/mL</td><td>Human</td><td>CHHHHHHHHHHHHHHHCCCC</td><td align="center"><a href="jmol.php?id=2727&amp;di=1" width="5px" height="5px"><img src="pymol_images/2727.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2727&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2727.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2728">2728</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKLTSKFLHSKKKF</td><td>K17</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &gt;500μg/mL</td><td>Human</td><td>CHHHHHHHHHHHHHSSSTTC</td><td align="center"><a href="jmol.php?id=2728&amp;di=1" width="5px" height="5px"><img src="pymol_images/2728.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2728&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2728.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2729">2729</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKLTSKFLHSADKF</td><td>D18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &gt;500μg/mL</td><td>Human</td><td>CTHHHHHHHHTTTCCSSCCC</td><td align="center"><a href="jmol.php?id=2729&amp;di=1" width="5px" height="5px"><img src="pymol_images/2729.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2729&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2729.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2730">2730</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKLTSKFLHSANKF</td><td>N18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &gt;500μg/mL</td><td>Human</td><td>CHHHHHHHHHHHHTTTSSCC</td><td align="center"><a href="jmol.php?id=2730&amp;di=1" width="5px" height="5px"><img src="pymol_images/2730.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2730&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2730.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2731">2731</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22518150">22518150</a></td><td align="right">2012</td><td>KWKSFIKKLTSKFLHSAKKN</td><td>N20</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &gt;500μg/mL</td><td>Human</td><td>CCGGGHHHHHHHHHHHCSCC</td><td align="center"><a href="jmol.php?id=2731&amp;di=1" width="5px" height="5px"><img src="pymol_images/2731.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2731&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2731.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2736">2736</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22038181">22038181</a></td><td align="right">2012</td><td>GLPRKILCAIAKKKGKCKGPLKLVCKC</td><td>Lasiocepsin</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>&lt;1% hemolytic at 200μM</td><td>Human</td><td>CCTTSCCGGGGSCCCSSCSSCCCCCCC</td><td align="center"><a href="jmol.php?id=2736&amp;di=1" width="5px" height="5px"><img src="pymol_images/2736.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2736&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2736.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2737">2737</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22038181">22038181</a></td><td align="right">2012</td><td>GLPRKILCAIAKKKGKCKGPLKLVCKC</td><td>Lasiocepsin-NH2</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>&lt;1% hemolytic at 200μM</td><td>Human</td><td>CCTTSCCGGGGSCCCSSCSSCCCCCCC</td><td align="center"><a href="jmol.php?id=2737&amp;di=1" width="5px" height="5px"><img src="pymol_images/2737.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2738">2738</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22038181">22038181</a></td><td align="right">2012</td><td>GLPRKILCAIAKKKGKCKGPLKLVCKC</td><td>Las[C8-C27, C17-C25]</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>&lt;1% hemolytic at 200μM</td><td>Human</td><td>CCTTSCCGGGGSCCCSSCSSCCCCCCC</td><td align="center"><a href="jmol.php?id=2738&amp;di=1" width="5px" height="5px"><img src="pymol_images/2738.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2738&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2738.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2739">2739</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22038181">22038181</a></td><td align="right">2012</td><td>GLPRKILCAIAKKKGKCKGPLKLVCKC</td><td>Las[C8-C17, C25-C27]</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>&lt;1% hemolytic at 200μM</td><td>Human</td><td>CCTTSCCGGGGSCCCSSCSSCCCCCCC</td><td align="center"><a href="jmol.php?id=2739&amp;di=1" width="5px" height="5px"><img src="pymol_images/2739.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2739&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2739.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2740">2740</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22038181">22038181</a></td><td align="right">2012</td><td>GLPRKILCAIAKKKGKAKGPLKLVCKA</td><td>Las[C8-C25, Ala17,27]</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>&lt;1% hemolytic at 200μM</td><td>Human</td><td>CCTTCCCHHHHTTCCCCCCSCCSSCCC</td><td align="center"><a href="jmol.php?id=2740&amp;di=1" width="5px" height="5px"><img src="pymol_images/2740.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2740&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2740.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2741">2741</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22038181">22038181</a></td><td align="right">2012</td><td>GLPRKILAAIAKKKGKCKGPLKLVAKC</td><td>Las[C17-C27, Ala8,25]</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>&lt;1% hemolytic at 200μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2742">2742</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22038181">22038181</a></td><td align="right">2012</td><td>GLPRKILAAIAKKKGKAKGPLKLVAKA</td><td>Las[Ala8,17,25,27]</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>&lt;1% hemolytic at 200μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2743">2743</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22038181">22038181</a></td><td align="right">2012</td><td>LCAIAKKKGKCKGPLKLVCKC</td><td>Las[des1-6]-NH<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>&lt;1% hemolytic at 200μM</td><td>Human</td><td>CCTTTTSCCSSCCSCCSSCCC</td><td align="center"><a href="jmol.php?id=2743&amp;di=1" width="5px" height="5px"><img src="pymol_images/2743.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2744">2744</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22038181">22038181</a></td><td align="right">2012</td><td>AKKKGKCKGPLKLVAKC</td><td>Las[des1-10, Ala25]-NH<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>&lt;1% hemolytic at 200μM</td><td>Human</td><td>CCSTTTTCCSCCCCCCC</td><td align="center"><a href="jmol.php?id=2744&amp;di=1" width="5px" height="5px"><img src="pymol_images/2744.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2745">2745</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22038181">22038181</a></td><td align="right">2012</td><td>GLPRKILAAIAKKK</td><td>Las[des15-27, Ala8]-NH<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>&lt;1% hemolytic at 200μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2746">2746</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSILKKVLPKVMAHMK</td><td>MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100μM</td><td>Human</td><td>CCHHHHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=2746&amp;di=1" width="5px" height="5px"><img src="pymol_images/2746.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2746&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2746.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2747">2747</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSILKKVLPKVXAHXK</td><td>MEP-N</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =50μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2748">2748</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSILKKVLPKSXAHSK</td><td>MEP-Ns-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =18.1μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2749">2749</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSSLKKSLPKVXAHXK</td><td>MEP-Ns-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =14.7μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2750">2750</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSSLKKSLPKSXAHSK</td><td>MEP-Ns-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =13.9μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2751">2751</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFRSILKKVSPKVXAHXK</td><td>MEP-Ns-4 cis</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =11μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2752">2752</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFRSILKKVSPKVXAHXK</td><td>MEP-Ns-4 trans</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =20μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2753">2753</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSSLKKSLGKSXAHSK</td><td>MEP-Ns-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =29μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2754">2754</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>GFLSSLKKSLAKSXAHSK</td><td>MEP-Ns-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>X; Nle</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =39.5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2755">2755</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>VNWKKILGKIIKVVK</td><td>LL-III</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200μM</td><td>Human</td><td>CCHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2755&amp;di=1" width="5px" height="5px"><img src="pymol_images/2755.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2755&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2755.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2756">2756</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>VNWKKSLGKSIKVVK</td><td>LL-IIIs-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>LC<sub>50</sub> =31.3μM</td><td>Human</td><td>CCCCCCCCCCSSSCC</td><td align="center"><a href="jmol.php?id=2756&amp;di=1" width="5px" height="5px"><img src="pymol_images/2756.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2756&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2756.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2757">2757</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>VNWKKILGKSIKVSK</td><td>LL-IIIs-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>LC<sub>50</sub> =69μM</td><td>Human</td><td>CCGGGTTTCCSSCCC</td><td align="center"><a href="jmol.php?id=2757&amp;di=1" width="5px" height="5px"><img src="pymol_images/2757.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2757&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2757.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2758">2758</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>VSWKKSLGKIIKVVK</td><td>LL-IIIs-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>LC<sub>50</sub> =30μM</td><td>Human</td><td>CCGGGTTTSCCSSCC</td><td align="center"><a href="jmol.php?id=2758&amp;di=1" width="5px" height="5px"><img src="pymol_images/2758.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2758&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2758.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2759">2759</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>VNSKKISGKSIKVSK</td><td>LL-IIIs-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100μM</td><td>Human</td><td>CCSSSSSSCCCSSCC</td><td align="center"><a href="jmol.php?id=2759&amp;di=1" width="5px" height="5px"><img src="pymol_images/2759.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2759&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2759.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2760">2760</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>VNRKKILGKSIKVVK</td><td>LL-IIIs-5 cis</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>LC<sub>50</sub> =100μM</td><td>Human</td><td>CCGGGTSBTTBCCCC</td><td align="center"><a href="jmol.php?id=2760&amp;di=1" width="5px" height="5px"><img src="pymol_images/2760.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2761">2761</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>VNRKKILGKSIKVVK</td><td>LL-IIIs-5 trans</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100μM</td><td>Human</td><td>CCGGGTSBTTBCCCC</td><td align="center"><a href="jmol.php?id=2761&amp;di=1" width="5px" height="5px"><img src="pymol_images/2761.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2762">2762</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>VNSKKISPKSIKVSK</td><td>LL-IIIs-6a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100μM</td><td>Human</td><td>CCSSCCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2762&amp;di=1" width="5px" height="5px"><img src="pymol_images/2762.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2762&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2762.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2763">2763</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22526241">22526241</a></td><td align="right">2012</td><td>VNSKKISPKSIKVSK</td><td>LL-IIIs-6b</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>LC<sub>50</sub> =82μM</td><td>Human</td><td>CCSSCCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2763&amp;di=1" width="5px" height="5px"><img src="pymol_images/2763.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2763&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2763.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2764">2764</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22578463">22578463</a></td><td align="right">2012</td><td>QLKVDLWGTRSGIQPEQHSSGKSDVRRWRSRY</td><td>Ixosin-B-amide</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CCTTCBTTBCCSSCCSCCCSTTCTHHHHTCCC</td><td align="center"><a href="jmol.php?id=2764&amp;di=1" width="5px" height="5px"><img src="pymol_images/2764.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2764&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2764.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2765">2765</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22578463">22578463</a></td><td align="right">2012</td><td>KSLVRRWRSRW</td><td>MAP-04-01</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CGGGGTCSSCC</td><td align="center"><a href="jmol.php?id=2765&amp;di=1" width="5px" height="5px"><img src="pymol_images/2765.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2765&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2765.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2766">2766</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22578463">22578463</a></td><td align="right">2012</td><td>KSLRRVWRSWR</td><td>MAP-04-02</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2766&amp;di=1" width="5px" height="5px"><img src="pymol_images/2766.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2766&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2766.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2767">2767</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22578463">22578463</a></td><td align="right">2012</td><td>KWLRRVWRWWR</td><td>MAP-04-03</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>100% hemolytic at 100μM</td><td>Human</td><td>CTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2767&amp;di=1" width="5px" height="5px"><img src="pymol_images/2767.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2767&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2767.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2768">2768</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22578463">22578463</a></td><td align="right">2012</td><td>KRLRRVWRRWR</td><td>MAP-04-04</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>3% hemolytic at 100μM</td><td>Human</td><td>CCGGGTTTTCC</td><td align="center"><a href="jmol.php?id=2768&amp;di=1" width="5px" height="5px"><img src="pymol_images/2768.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2768&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2768.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2769">2769</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22581068">22581068</a></td><td align="right">2012</td><td>SAVGRHGRRFGLRKHRKH</td><td>Chensinin-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LD<sub>50</sub> &gt;500μM</td><td>Human</td><td>CCSSCCSSCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2769&amp;di=1" width="5px" height="5px"><img src="pymol_images/2769.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2769&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2769.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2770">2770</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19635451">19635451</a></td><td align="right">2010</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>100% hemolytic at 0.4μM &amp; 23μM</td><td>Human</td><td>CCCSSHHHHHHSCHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=2770&amp;di=1" width="5px" height="5px"><img src="pymol_images/2770.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2770&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2770.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2775">2775</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19960443">19960443</a></td><td align="right">2010</td><td>pslvslvVqLvΔ-But-allothr-I-Hse-dab-K</td><td>Tolaasin</td><td>Free</td><td>β-hydroxyoctanoyl-Δ-But</td><td>Linear</td><td>Mix</td><td>Δ-But= 2,3-dehydro-2-aminobutyric acid; Hse= L-homoserine; dab= D-2,4-diaminobutyric acid</td><td align="right">18</td><td>Hemolytic</td><td>100% hemolysis at 0.53μg/mL</td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2777">2777</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20058937">20058937</a></td><td align="right">2010</td><td>KKllKLLlKlLK</td><td>K<sub>5</sub>L<sub>7</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CTTHHHHTTTTC</td><td align="center"><a href="jmol.php?id=2777&amp;di=1" width="5px" height="5px"><img src="pymol_images/2777.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2777&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2777.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2778">2778</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20058937">20058937</a></td><td align="right">2010</td><td>KKllKLLlKlLK</td><td>C6-K<sub>5</sub>L<sub>7</sub></td><td>Amidation</td><td>Acetylation (CH<sub>3</sub>(CH<sub>2</sub>)<sub>4</sub>CO)</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CTTHHHHTTTTC</td><td align="center"><a href="jmol.php?id=2778&amp;di=1" width="5px" height="5px"><img src="pymol_images/2778.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2778&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2778.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2779">2779</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20058937">20058937</a></td><td align="right">2010</td><td>KKllKLLlKlLK</td><td>C8-K<sub>5</sub>L<sub>7</sub></td><td>Amidation</td><td>Acetylation (CH<sub>3</sub>(CH<sub>2</sub>)<sub>6</sub>CO)</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>19% hemolytic at 100μM</td><td>Human</td><td>CTTHHHHTTTTC</td><td align="center"><a href="jmol.php?id=2779&amp;di=1" width="5px" height="5px"><img src="pymol_images/2779.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2779&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2779.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2780">2780</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20058937">20058937</a></td><td align="right">2010</td><td>KKllKKLkKlLK</td><td>K<sub>7</sub>L<sub>5</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CTTHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2780&amp;di=1" width="5px" height="5px"><img src="pymol_images/2780.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2780&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2780.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2781">2781</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20058937">20058937</a></td><td align="right">2010</td><td>KKllKKLkKlLK</td><td>C<sub>6</sub>-K<sub>7</sub>L<sub>5</sub></td><td>Amidation</td><td>Acetylation (CH<sub>3</sub>(CH<sub>2</sub>)<sub>4</sub>CO)</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CTTHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2781&amp;di=1" width="5px" height="5px"><img src="pymol_images/2781.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2781&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2781.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2782">2782</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20058937">20058937</a></td><td align="right">2010</td><td>KKllKKLkKlLK</td><td>C<sub>8</sub>-K<sub>7</sub>L<sub>5</sub></td><td>Amidation</td><td>Acetylation (CH<sub>3</sub>(CH<sub>2</sub>)<sub>6</sub>CO)</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CTTHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2782&amp;di=1" width="5px" height="5px"><img src="pymol_images/2782.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2782&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2782.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2783">2783</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20058937">20058937</a></td><td align="right">2010</td><td>KKklKKLkKkLK</td><td>K<sub>9</sub>L<sub>3</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CTTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2783&amp;di=1" width="5px" height="5px"><img src="pymol_images/2783.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2783&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2783.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2784">2784</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20058937">20058937</a></td><td align="right">2010</td><td>KKklKKLkKkLK</td><td>C<sub>6</sub>-K<sub>9</sub>L<sub>3</sub></td><td>Amidation</td><td>Acetylation (CH<sub>3</sub>(CH<sub>2</sub>)<sub>4</sub>CO)</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CTTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2784&amp;di=1" width="5px" height="5px"><img src="pymol_images/2784.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2784&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2784.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2785">2785</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20058937">20058937</a></td><td align="right">2010</td><td>KKklKKLkKkLK</td><td>C<sub>8</sub>-K<sub>9</sub>L<sub>3</sub></td><td>Amidation</td><td>Acetylation (CH3(CH<sub>2</sub>)<sub>6</sub>CO)</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CTTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2785&amp;di=1" width="5px" height="5px"><img src="pymol_images/2785.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2785&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2785.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2786">2786</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20798581">20798581</a></td><td align="right">2010</td><td>GWGSFFKKAAHVGKHVGKAALTHYL</td><td>Ple</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>7% hemolytic at 100μM</td><td>Human</td><td>CCHHHHTTTHHHHHHHTSGGGSCCC</td><td align="center"><a href="jmol.php?id=2786&amp;di=1" width="5px" height="5px"><img src="pymol_images/2786.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2787">2787</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20798581">20798581</a></td><td align="right">2010</td><td>SFFKKAAHVGKHVGKAALTHYL</td><td>Ple (4-25)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolytoc at 100μM (non-hemolytic)</td><td>Human</td><td>CCSTTSSSCSSSSCGGGTSCCC</td><td align="center"><a href="jmol.php?id=2787&amp;di=1" width="5px" height="5px"><img src="pymol_images/2787.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2787&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2787.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2788">2788</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20798581">20798581</a></td><td align="right">2010</td><td>KKAAHVGKHVGKAALTHYL</td><td>Ple (7-25)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CCSSCCCCCSCGGGTSCCC</td><td align="center"><a href="jmol.php?id=2788&amp;di=1" width="5px" height="5px"><img src="pymol_images/2788.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2788&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2788.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2789">2789</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20798581">20798581</a></td><td align="right">2010</td><td>GWGSFFKKAAHVGKHVGKAALT</td><td>Ple (1-22)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolysis at 100μM (non-hemolytic)</td><td>Human</td><td>CCHHHHHHHHHHHHHEETTEEC</td><td align="center"><a href="jmol.php?id=2789&amp;di=1" width="5px" height="5px"><img src="pymol_images/2789.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2790">2790</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20798581">20798581</a></td><td align="right">2010</td><td>GWGSFFKKAAHVGKHVGKA</td><td>Ple (1-19)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolytic at 100μM (non-hemolytic)</td><td>Human</td><td>CHHHHHHHHHHTTTTTTCC</td><td align="center"><a href="jmol.php?id=2790&amp;di=1" width="5px" height="5px"><img src="pymol_images/2790.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2791">2791</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fPVOLfPVOL</td><td>GS</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =17.5±1.07μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=2791&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2791.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2792">2792</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fPVOLfP-AdaG-OL</td><td>GS2</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, Ada=Adamantylglycine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =10.1±1.11μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2793">2793</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fPVOLfPVO-AdaA</td><td>GS4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, Ada=Adamantylalanine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =5.20±1.13μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2794">2794</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fPVO-Ada-AfPVO-AdaA</td><td>GS5</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, Ada=Adamantylalanine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =10.3±1.07μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2795">2795</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fP-Ada-GO-Ada-AfP-Ada-GO-AdaA</td><td>GS6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, Ada=Adamantylalanine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =7.55±1.10μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2796">2796</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>FP-tBuGly-O-tBuAla-fP-tBuGly-O-tBuAla</td><td>GS7</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, tBuGly= tert-butylglycine, tBuAla=tert-butylalanine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =9.30±1.07μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2797">2797</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fP-Chg-O-Cha-fP-Chg-O-Cha</td><td>GS8</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, Chg=Cyclohexylglycine, Cha= Cyclohexylalanine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =9.12±1.13μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2798">2798</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fPOVO</td><td>GS9</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine</td><td align="right">10</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=2798&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2798.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2799">2799</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20848624">20848624</a></td><td align="right">2010</td><td>fPO-AdaGly-OfPO-Ada-Gly-O</td><td>GS10</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine, Ada=Adamantylglycine</td><td align="right">10</td><td>Antimicrobial</td><td>HC<sub>50</sub> =107±1.07μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2800">2800</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20858205">20858205</a></td><td align="right">2011</td><td>kwksflktfksakktvlhtalkaiss</td><td>D1</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>HC<sub>50</sub> =421.5μg/ml</td><td>Human</td><td>CCHHHHHHHHHCGGGGHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=2800&amp;di=1" width="5px" height="5px"><img src="pymol_images/2800.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2800&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2800.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2801">2801</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20858205">20858205</a></td><td align="right">2011</td><td>kwksflktfksakktvlhtllkaiss</td><td>D2</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>HC<sub>50</sub> =83μg/ml</td><td>Human</td><td>CCHHHHHHTTSSTTGGGHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2801&amp;di=1" width="5px" height="5px"><img src="pymol_images/2801.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2801&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2801.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2802">2802</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20858205">20858205</a></td><td align="right">2011</td><td>kwksflktfkslkktvlhtllkaiss</td><td>D3</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>HC<sub>50</sub> =14μg/ml</td><td>Human</td><td>CCGGGHHHHHHHTTSHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2802&amp;di=1" width="5px" height="5px"><img src="pymol_images/2802.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2802&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2802.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2803">2803</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20858205">20858205</a></td><td align="right">2011</td><td>kwksflktfkslkktvlhtllkliss</td><td>D4</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>HC<sub>50</sub> =3.5μg/ml</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2803&amp;di=1" width="5px" height="5px"><img src="pymol_images/2803.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2803&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2803.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2804">2804</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20858205">20858205</a></td><td align="right">2011</td><td>kwksflktfkslkktklhtllkliss</td><td>D5</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>HC<sub>50</sub> =47μg/ml</td><td>Human</td><td>CCHHHHHHHHHHTTSCSGGGTTTTCC</td><td align="center"><a href="jmol.php?id=2804&amp;di=1" width="5px" height="5px"><img src="pymol_images/2804.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2804&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2804.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2807">2807</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20955730">20955730</a></td><td align="right">2011</td><td>RIKRFWPVVIRTVVAGYNLYRAIKKK</td><td>Pc-CATH1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>13.2% hemolytic at 31.5μM</td><td>Human</td><td>CCCCCCGGGCSSTTCCSGGGGTTTCC</td><td align="center"><a href="jmol.php?id=2807&amp;di=1" width="5px" height="5px"><img src="pymol_images/2807.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2807&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2807.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2808">2808</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20955730">20955730</a></td><td align="right">2011</td><td>RIKRFWPVVIRTVVAGYNLYRAIKKK</td><td>Pc-CATH1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>9.1% hemolytic at 15.7μM</td><td>Human</td><td>CCCCCCGGGCSSTTCCSGGGGTTTCC</td><td align="center"><a href="jmol.php?id=2808&amp;di=1" width="5px" height="5px"><img src="pymol_images/2808.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2808&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2808.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2809">2809</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20955730">20955730</a></td><td align="right">2011</td><td>RIKRFWPVVIRTVVAGYNLYRAIKKK</td><td>Pc-CATH1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>6.3% hemolytic at 6.30μM</td><td>Human</td><td>CCCCCCGGGCSSTTCCSGGGGTTTCC</td><td align="center"><a href="jmol.php?id=2809&amp;di=1" width="5px" height="5px"><img src="pymol_images/2809.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2809&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2809.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2810">2810</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20955730">20955730</a></td><td align="right">2011</td><td>RIKRFWPVVIRTVVAGYNLYRAIKKK</td><td>Pc-CATH1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>3.6% hemolytic at 3.15μM</td><td>Human</td><td>CCCCCCGGGCSSTTCCSGGGGTTTCC</td><td align="center"><a href="jmol.php?id=2810&amp;di=1" width="5px" height="5px"><img src="pymol_images/2810.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2810&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2810.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2819">2819</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21219588">21219588</a></td><td align="right">2011</td><td>kwksflktfksavktvlhtalkaiss</td><td>D1 (V13)</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>HC<sub>50</sub> =1.8μM</td><td>Human</td><td>CCTTTTTHHHHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=2819&amp;di=1" width="5px" height="5px"><img src="pymol_images/2819.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2819&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2819.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2820">2820</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21219588">21219588</a></td><td align="right">2011</td><td>kwksflktfksakktvlhtalkaiss</td><td>D1 (K13)</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>HC<sub>50</sub> =140.9μM</td><td>Human</td><td>CCHHHHHHHHHCGGGGHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=2820&amp;di=1" width="5px" height="5px"><img src="pymol_images/2820.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2820&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2820.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2821">2821</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21219588">21219588</a></td><td align="right">2011</td><td>kwksflktfskakkkvlktalkaisk</td><td>D11</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>HC<sub>50</sub> =254.1μM</td><td>Human</td><td>CCTTTHHHHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2821&amp;di=1" width="5px" height="5px"><img src="pymol_images/2821.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2821&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2821.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2822">2822</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21219588">21219588</a></td><td align="right">2011</td><td>kwksflktfskakkkalktllkaisk</td><td>D22</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>HC<sub>50</sub> =81.3μM</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHHGGGGCC</td><td align="center"><a href="jmol.php?id=2822&amp;di=1" width="5px" height="5px"><img src="pymol_images/2822.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2822&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2822.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2823">2823</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21219588">21219588</a></td><td align="right">2011</td><td>kwksflktfskakkkklktllkaisk</td><td>D14</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>HC<sub>50</sub> =351.5μM</td><td>Human</td><td>CCHHHHHHTHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2823&amp;di=1" width="5px" height="5px"><img src="pymol_images/2823.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2823&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2823.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2824">2824</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21219588">21219588</a></td><td align="right">2011</td><td>klksllktlskakkkllktalkalsk</td><td>D15</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>HC<sub>50</sub> =169.6μM</td><td>Human</td><td>CHHHHHHHHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2824&amp;di=1" width="5px" height="5px"><img src="pymol_images/2824.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2824&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2824.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2825">2825</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21219588">21219588</a></td><td align="right">2011</td><td>klksllktlskakkkklktllkalsk</td><td>D16</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>HC<sub>50</sub> =1342.0μM</td><td>Human</td><td>CGGGHHHHHHHHHHHHGGGTGGGTTC</td><td align="center"><a href="jmol.php?id=2825&amp;di=1" width="5px" height="5px"><img src="pymol_images/2825.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2825&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2825.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2826">2826</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21300831">21300831</a></td><td align="right">2011</td><td>GKKLLKKLKKLLKKG</td><td>Pep-1</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>0.3% hemolytic at 3.4mg/ml</td><td>Human</td><td>CTTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=2826&amp;di=1" width="5px" height="5px"><img src="pymol_images/2826.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2826&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2826.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2827">2827</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21300831">21300831</a></td><td align="right">2011</td><td>GKKllKKlKKllKKG</td><td>MA-d</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>1% hemolytic at 3.4mg/ml</td><td>Human</td><td>CTTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=2827&amp;di=1" width="5px" height="5px"><img src="pymol_images/2827.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2827&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2827.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2828">2828</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21300831">21300831</a></td><td align="right">2011</td><td>GKKLLKKLKKLLKKW</td><td>MAw-1</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>17% hemolytic at 3.4mg/ml</td><td>Human</td><td>CTTTHHHHHHHCCCC</td><td align="center"><a href="jmol.php?id=2828&amp;di=1" width="5px" height="5px"><img src="pymol_images/2828.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2828&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2828.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2829">2829</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21300831">21300831</a></td><td align="right">2011</td><td>GKKLLKKLKKLWKKW</td><td>Maw-2</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>~3% hemolytic at 3.4mg/ml</td><td>Human</td><td>CTTTHHHHTTTTTCC</td><td align="center"><a href="jmol.php?id=2829&amp;di=1" width="5px" height="5px"><img src="pymol_images/2829.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2829&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2829.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2830">2830</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21300831">21300831</a></td><td align="right">2011</td><td>GKKLLKKWKKLWKKW</td><td>Maw-3</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>18% hemolytic at 3.4mg/ml</td><td>Human</td><td>CCTTHHHHTTTTTCC</td><td align="center"><a href="jmol.php?id=2830&amp;di=1" width="5px" height="5px"><img src="pymol_images/2830.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2830&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2830.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2831">2831</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>48% hemolytic at 6μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=2831&amp;di=1" width="5px" height="5px"><img src="pymol_images/2831.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2831&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2831.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2832">2832</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>LLQWLSKLLGRLL</td><td>TL-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>25% hemolytic at 6μM</td><td>Human</td><td>CHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=2832&amp;di=1" width="5px" height="5px"><img src="pymol_images/2832.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2832&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2832.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2833">2833</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>LLQWLSKLLGRWL</td><td>TL-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>78% hemolytic at 6μM</td><td>Human</td><td>CHHHHHTTTCCCC</td><td align="center"><a href="jmol.php?id=2833&amp;di=1" width="5px" height="5px"><img src="pymol_images/2833.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2833&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2833.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2834">2834</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVQWFSKFLLRIL</td><td>[Leu<sup>10</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>80% hemolytic at 6μM</td><td>Human</td><td>CHHHHHHHTTTTC</td><td align="center"><a href="jmol.php?id=2834&amp;di=1" width="5px" height="5px"><img src="pymol_images/2834.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2834&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2834.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2835">2835</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL<sup>b</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>10% hemolytic at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=2835&amp;di=1" width="5px" height="5px"><img src="pymol_images/2835.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2835&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2835.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2836">2836</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FLPWFSKFLGRIL</td><td>[Leu<sup>2</sup>, Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>19% hemolytic at 6μM</td><td>Human</td><td>CCCTTTTTTCCCC</td><td align="center"><a href="jmol.php?id=2836&amp;di=1" width="5px" height="5px"><img src="pymol_images/2836.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2836&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2836.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2837">2837</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVPWFSKFLPRIL</td><td>[Pro<sup>3</sup>, Pro<sup>10</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>4.5% hemolytic at 6μM</td><td>Human</td><td>CCCCGGGTSSSCC</td><td align="center"><a href="jmol.php?id=2837&amp;di=1" width="5px" height="5px"><img src="pymol_images/2837.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2837&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2837.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2838">2838</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVPWFSKFLpRIL</td><td>[Pro<sup>3</sup>, DPro<sup>10</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0.6% hemolytic at 6μM</td><td>Human</td><td>CCCCGGGTSSSCC</td><td align="center"><a href="jmol.php?id=2838&amp;di=1" width="5px" height="5px"><img src="pymol_images/2838.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2838&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2838.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2839">2839</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVQWFSKFLGKIL</td><td>[Lys<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>14% hemolytic at 6μM</td><td>Human</td><td>CHHHHHHHHTCCC</td><td align="center"><a href="jmol.php?id=2839&amp;di=1" width="5px" height="5px"><img src="pymol_images/2839.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2839&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2839.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2840">2840</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVQWFSKFLGOIL</td><td>[Orn<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Ornithine</td><td align="right">13</td><td>Antimicrobial</td><td>22% hemolytic at 6μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=2840&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2840.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2841">2841</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVQWFSRFLGRIL</td><td>[Arg<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>29% hemolytic at 6μM</td><td>Human</td><td>CHHHHHHHTTTCC</td><td align="center"><a href="jmol.php?id=2841&amp;di=1" width="5px" height="5px"><img src="pymol_images/2841.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2841&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2841.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2842">2842</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVRWFSKFLGRIL</td><td>[Arg<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>52% hemolytic at 6μM</td><td>Human</td><td>CHHHHHTTTTCCC</td><td align="center"><a href="jmol.php?id=2842&amp;di=1" width="5px" height="5px"><img src="pymol_images/2842.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2842&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2842.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2843">2843</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVRWFSRFLGRIL</td><td>[Arg<sup>3</sup>, Arg<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>43% hemolytic at 6μM</td><td>Human</td><td>CCGGGHHHHTTCC</td><td align="center"><a href="jmol.php?id=2843&amp;di=1" width="5px" height="5px"><img src="pymol_images/2843.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2843&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2843.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2844">2844</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>FVPWFSKFLGOIL</td><td>[Pro<sup>3</sup>, Orn<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolytic at 6μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2845">2845</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>fvqwfskflGril</td><td>D-TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>46% hemolytic at 6μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=2845&amp;di=1" width="5px" height="5px"><img src="pymol_images/2845.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2845&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2845.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2846">2846</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21319749">21319749</a></td><td align="right">2011</td><td>lirGlfksfwqvf</td><td>RI-TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>15% hemolytic at 6μM</td><td>Human</td><td>CTHHHHHHHSCCC</td><td align="center"><a href="jmol.php?id=2846&amp;di=1" width="5px" height="5px"><img src="pymol_images/2846.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2846&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2846.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2847">2847</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFKKILKYL</td><td>BP100</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>54 ±0.1% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2847&amp;di=1" width="5px" height="5px"><img src="pymol_images/2847.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2847&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2847.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2848">2848</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFKKILKyL</td><td>BP138</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>7 ±0.1% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2848&amp;di=1" width="5px" height="5px"><img src="pymol_images/2848.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2848&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2848.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2849">2849</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFKKILKYl</td><td>BP139</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>23 ±0.9% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2849&amp;di=1" width="5px" height="5px"><img src="pymol_images/2849.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2849&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2849.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2850">2850</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFKKILkYL</td><td>BP140</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0± 0% hemolytic at 250μM (non-hemolytic)</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2850&amp;di=1" width="5px" height="5px"><img src="pymol_images/2850.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2850&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2850.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2851">2851</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFKKIlKYL</td><td>BP141</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>4 ±0.3% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2851&amp;di=1" width="5px" height="5px"><img src="pymol_images/2851.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2851&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2851.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2852">2852</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFkKILKYL</td><td>BP142</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>3± 0.6% hemolysis at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2852&amp;di=1" width="5px" height="5px"><img src="pymol_images/2852.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2852&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2852.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2853">2853</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLfKKILKYL</td><td>BP143</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>5± 0.8% hemolysis at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2853&amp;di=1" width="5px" height="5px"><img src="pymol_images/2853.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2853&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2853.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2854">2854</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKlFKKILKYL</td><td>BP144</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>7±0.4% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2854&amp;di=1" width="5px" height="5px"><img src="pymol_images/2854.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2854&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2854.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2855">2855</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KkLFKKILKYL</td><td>BP145</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>51±0.5% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2855&amp;di=1" width="5px" height="5px"><img src="pymol_images/2855.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2855&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2855.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2856">2856</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFKkILKYL</td><td>BP146</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>53 ±1.1% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2856&amp;di=1" width="5px" height="5px"><img src="pymol_images/2856.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2856&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2856.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2857">2857</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>kKLFKKILKYL</td><td>BP147</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>71±0.4% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2857&amp;di=1" width="5px" height="5px"><img src="pymol_images/2857.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2857&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2857.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2858">2858</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFKKiLKYL</td><td>BP148</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0±0% hemolytic at 250μM (non-hemolytic)</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2858&amp;di=1" width="5px" height="5px"><img src="pymol_images/2858.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2858&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2858.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2859">2859</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKlFKKILkYL</td><td>BP149</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0±0% hemolytic at 250μM (non-hemolytic)</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2859&amp;di=1" width="5px" height="5px"><img src="pymol_images/2859.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2859&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2859.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2860">2860</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFKKIlkYL</td><td>BP150</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>1±0.1% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2860&amp;di=1" width="5px" height="5px"><img src="pymol_images/2860.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2860&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2860.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2861">2861</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KklFKkILkYL</td><td>BP151</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>1±0.3% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2861&amp;di=1" width="5px" height="5px"><img src="pymol_images/2861.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2861&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2861.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2862">2862</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFKKilkYL</td><td>BP152</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>5±1.6% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2862&amp;di=1" width="5px" height="5px"><img src="pymol_images/2862.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2862&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2862.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2863">2863</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>kkLfKKILKYL</td><td>BP154</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>41±6.2% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2863&amp;di=1" width="5px" height="5px"><img src="pymol_images/2863.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2863&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2863.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2864">2864</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFKKIlkyl</td><td>BP155</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>3±0.1% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2864&amp;di=1" width="5px" height="5px"><img src="pymol_images/2864.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2864&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2864.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2865">2865</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFKKllkyl</td><td>BP156</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>49±1.2% hemolytic at 250μM</td><td>Human</td><td>CTTHHHHTTCC</td><td align="center"><a href="jmol.php?id=2865&amp;di=1" width="5px" height="5px"><img src="pymol_images/2865.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2865&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2865.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2866">2866</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFKkilkyl</td><td>BP157</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>3±1.0% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2866&amp;di=1" width="5px" height="5px"><img src="pymol_images/2866.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2866&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2866.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2867">2867</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLFkkilkyl</td><td>BP158</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>28±2.1% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2867&amp;di=1" width="5px" height="5px"><img src="pymol_images/2867.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2867&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2867.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2868">2868</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKLfkkilkyl</td><td>BP159</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>58±5.7% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2868&amp;di=1" width="5px" height="5px"><img src="pymol_images/2868.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2868&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2868.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2869">2869</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>KKlfkkilkyl</td><td>BP160</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>65±1.9% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2869&amp;di=1" width="5px" height="5px"><img src="pymol_images/2869.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2869&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2869.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2870">2870</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>Kklfkkilkyl</td><td>BP161</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>66±2.9% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2870&amp;di=1" width="5px" height="5px"><img src="pymol_images/2870.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2870&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2870.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2871">2871</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>kklfKKILKYL</td><td>BP162</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>17±0.7% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2871&amp;di=1" width="5px" height="5px"><img src="pymol_images/2871.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2871&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2871.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2872">2872</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>kklfkKILKYL</td><td>BP163</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>8±5.0% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2872&amp;di=1" width="5px" height="5px"><img src="pymol_images/2872.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2872&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2872.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2873">2873</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>kklfkkILKYL</td><td>BP164</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0±0% hemolytic at 250μM (non-hemolytic)</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2873&amp;di=1" width="5px" height="5px"><img src="pymol_images/2873.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2873&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2873.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2874">2874</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>kklfkkiLKYL</td><td>BP165</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>6 ±0.2% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2874&amp;di=1" width="5px" height="5px"><img src="pymol_images/2874.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2874&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2874.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2875">2875</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>kklfkkilKYL</td><td>BP166</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>1±0.2% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2875&amp;di=1" width="5px" height="5px"><img src="pymol_images/2875.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2875&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2875.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2876">2876</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>kklfkkilkYL</td><td>BP167</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>21±7.4% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2876&amp;di=1" width="5px" height="5px"><img src="pymol_images/2876.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2876&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2876.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2877">2877</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>kklfkkilkyL</td><td>BP168</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>3±2.7% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2877&amp;di=1" width="5px" height="5px"><img src="pymol_images/2877.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2877&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2877.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2878">2878</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21335383">21335383</a></td><td align="right">2011</td><td>kklfkkilkyl</td><td>BP153</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>50±2.1% hemolytic at 250μM</td><td>Human</td><td>CTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=2878&amp;di=1" width="5px" height="5px"><img src="pymol_images/2878.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2878&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2878.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2879">2879</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSGIL</td><td>1Ta</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>27.5% hemolytic at 40μM</td><td>Human</td><td>CCCCHHHHTSCCC</td><td align="center"><a href="jmol.php?id=2879&amp;di=1" width="5px" height="5px"><img src="pymol_images/2879.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2879&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2879.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2880">2880</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSGIL</td><td>1Ta</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0.5% hemolytic at 1.25μM</td><td>Human</td><td>CCCCHHHHTSCCC</td><td align="center"><a href="jmol.php?id=2880&amp;di=1" width="5px" height="5px"><img src="pymol_images/2880.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2880&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2880.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2881">2881</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>ALPLIGRVLSGIL</td><td>Ta A1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>62.5% hemolytic at 40μM</td><td>Human</td><td>CCCCGGGTTSCCC</td><td align="center"><a href="jmol.php?id=2881&amp;di=1" width="5px" height="5px"><img src="pymol_images/2881.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2881&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2881.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2882">2882</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>ALPLIGRVLSGIL</td><td>Ta A1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolytic at 1.25μM</td><td>Human</td><td>CCCCGGGTTSCCC</td><td align="center"><a href="jmol.php?id=2882&amp;di=1" width="5px" height="5px"><img src="pymol_images/2882.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2882&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2882.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2883">2883</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FAPLIGRVLSGIL</td><td>Ta A2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>15% hemolytic at 40μM</td><td>Human</td><td>CCCCHHHHTSSCC</td><td align="center"><a href="jmol.php?id=2883&amp;di=1" width="5px" height="5px"><img src="pymol_images/2883.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2883&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2883.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2884">2884</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FAPLIGRVLSGIL</td><td>Ta A2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CCCCHHHHTSSCC</td><td align="center"><a href="jmol.php?id=2884&amp;di=1" width="5px" height="5px"><img src="pymol_images/2884.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2884&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2884.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2885">2885</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLALIGRVLSGIL</td><td>Ta A3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolytic at 40μM</td><td>Human</td><td>CHHHHTTTTSSCC</td><td align="center"><a href="jmol.php?id=2885&amp;di=1" width="5px" height="5px"><img src="pymol_images/2885.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2885&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2885.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2886">2886</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLALIGRVLSGIL</td><td>Ta A3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CHHHHTTTTSSCC</td><td align="center"><a href="jmol.php?id=2886&amp;di=1" width="5px" height="5px"><img src="pymol_images/2886.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2886&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2886.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2887">2887</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPAIGRVLSGIL</td><td>Ta A4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>11.5% hemolytic at 40μM</td><td>Human</td><td>CCCCTHHHHCCCC</td><td align="center"><a href="jmol.php?id=2887&amp;di=1" width="5px" height="5px"><img src="pymol_images/2887.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2887&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2887.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2888">2888</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPAIGRVLSGIL</td><td>Ta A4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolytic at 1.25μM</td><td>Human</td><td>CCCCTHHHHCCCC</td><td align="center"><a href="jmol.php?id=2888&amp;di=1" width="5px" height="5px"><img src="pymol_images/2888.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2888&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2888.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2889">2889</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLAGRVLSGIL</td><td>Ta A5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3.5% hemolytic at 40μM</td><td>Human</td><td>CCGGGSTTSCSCC</td><td align="center"><a href="jmol.php?id=2889&amp;di=1" width="5px" height="5px"><img src="pymol_images/2889.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2889&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2889.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2890">2890</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLAGRVLSGIL</td><td>Ta A5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CCGGGSTTSCSCC</td><td align="center"><a href="jmol.php?id=2890&amp;di=1" width="5px" height="5px"><img src="pymol_images/2890.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2890&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2890.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2891">2891</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIARVLSGIL</td><td>Ta A6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>97% hemolytic at 40μM</td><td>Human</td><td>CCCHHHHTTSSCC</td><td align="center"><a href="jmol.php?id=2891&amp;di=1" width="5px" height="5px"><img src="pymol_images/2891.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2891&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2891.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2892">2892</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIARVLSGIL</td><td>Ta A6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>1% hemolytic at 1.25μM</td><td>Human</td><td>CCCHHHHTTSSCC</td><td align="center"><a href="jmol.php?id=2892&amp;di=1" width="5px" height="5px"><img src="pymol_images/2892.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2892&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2892.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2893">2893</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGAVLSGIL</td><td>Ta A7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>39% hemolytic at 40μM</td><td>Human</td><td>CCGGGGGGTSSCC</td><td align="center"><a href="jmol.php?id=2893&amp;di=1" width="5px" height="5px"><img src="pymol_images/2893.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2893&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2893.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2894">2894</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGAVLSGIL</td><td>Ta A7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>6% hemolytic at 1.25μM</td><td>Human</td><td>CCGGGGGGTSSCC</td><td align="center"><a href="jmol.php?id=2894&amp;di=1" width="5px" height="5px"><img src="pymol_images/2894.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2894&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2894.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2895">2895</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRALSGIL</td><td>Ta A8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>10.5% hemolytic at 40μM</td><td>Human</td><td>CCCCGGGTSCCCC</td><td align="center"><a href="jmol.php?id=2895&amp;di=1" width="5px" height="5px"><img src="pymol_images/2895.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2895&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2895.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2896">2896</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRALSGIL</td><td>Ta A8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>1.5% hemolytic at 1.25μM</td><td>Human</td><td>CCCCGGGTSCCCC</td><td align="center"><a href="jmol.php?id=2896&amp;di=1" width="5px" height="5px"><img src="pymol_images/2896.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2896&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2896.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2897">2897</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVASGIL</td><td>Ta A9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3.5% hemolytic at 40μM</td><td>Human</td><td>CCCCGGGTSSSCC</td><td align="center"><a href="jmol.php?id=2897&amp;di=1" width="5px" height="5px"><img src="pymol_images/2897.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2897&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2897.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2898">2898</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVASGIL</td><td>Ta A9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CCCCGGGTSSSCC</td><td align="center"><a href="jmol.php?id=2898&amp;di=1" width="5px" height="5px"><img src="pymol_images/2898.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2898&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2898.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2899">2899</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLAGIL</td><td>Ta A10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>70% hemolytic at 40μM</td><td>Human</td><td>CCCCTTBTTSBCC</td><td align="center"><a href="jmol.php?id=2899&amp;di=1" width="5px" height="5px"><img src="pymol_images/2899.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2899&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2899.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2900">2900</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLAGIL</td><td>Ta A10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CCCCTTBTTSBCC</td><td align="center"><a href="jmol.php?id=2900&amp;di=1" width="5px" height="5px"><img src="pymol_images/2900.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2900&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2900.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2901">2901</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSAIL</td><td>Ta A11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolytic at 40μM</td><td>Human</td><td>CCSSTHHHHTTTC</td><td align="center"><a href="jmol.php?id=2901&amp;di=1" width="5px" height="5px"><img src="pymol_images/2901.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2901&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2901.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2902">2902</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSAIL</td><td>Ta A11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3.5% hemolytic at 1.25μM</td><td>Human</td><td>CCSSTHHHHTTTC</td><td align="center"><a href="jmol.php?id=2902&amp;di=1" width="5px" height="5px"><img src="pymol_images/2902.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2902&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2902.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2903">2903</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSGAL</td><td>Ta A12</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>1% hemolytic at 40μM</td><td>Human</td><td>CCSSGGGTTSCCC</td><td align="center"><a href="jmol.php?id=2903&amp;di=1" width="5px" height="5px"><img src="pymol_images/2903.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2903&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2903.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2904">2904</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSGAL</td><td>Ta A12</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CCSSGGGTTSCCC</td><td align="center"><a href="jmol.php?id=2904&amp;di=1" width="5px" height="5px"><img src="pymol_images/2904.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2904&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2904.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2905">2905</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSGIA</td><td>Ta A13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3.5% hemolytic at 40μM</td><td>Human</td><td>CCCSGGGTSTTTC</td><td align="center"><a href="jmol.php?id=2905&amp;di=1" width="5px" height="5px"><img src="pymol_images/2905.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2905&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2905.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2906">2906</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21337476">21337476</a></td><td align="right">2011</td><td>FLPLIGRVLSGIA</td><td>Ta A13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolytic at 1.25μM</td><td>Human</td><td>CCCSGGGTSTTTC</td><td align="center"><a href="jmol.php?id=2906&amp;di=1" width="5px" height="5px"><img src="pymol_images/2906.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2906&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2906.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2907">2907</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21343499">21343499</a></td><td align="right">2011</td><td>kwklfkkipkflhlakkf</td><td>HDP D-P18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">18</td><td>Antimcrobial and Hemolytic</td><td>~94% hemolytic at 500μM</td><td>Human</td><td>CCCSCSCCHHHHTCSSCC</td><td align="center"><a href="jmol.php?id=2907&amp;di=1" width="5px" height="5px"><img src="pymol_images/2907.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2907&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2907.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2908">2908</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21343499">21343499</a></td><td align="right">2011</td><td>kwklfkkipkflhlakkf</td><td>HDP D-P18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">18</td><td>Antimcrobial and Hemolytic</td><td>~10% hemolytic at 50μM</td><td>Human</td><td>CCCSCSCCHHHHTCSSCC</td><td align="center"><a href="jmol.php?id=2908&amp;di=1" width="5px" height="5px"><img src="pymol_images/2908.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2908&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2908.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2909">2909</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21343499">21343499</a></td><td align="right">2011</td><td>EEEEAAAkwklfkkipkflhlakkf</td><td>Ac-E4-A3–D-P18</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">25</td><td>Antimcrobial and Hemolytic</td><td>~3.5% hemolytic upto 500μM</td><td>Human</td><td>CTTTTTHHHHTTTTCSSCSSCTTCC</td><td align="center"><a href="jmol.php?id=2909&amp;di=1" width="5px" height="5px"><img src="pymol_images/2909.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2909&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2909.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2911">2911</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21607695">21607695</a></td><td align="right">2011</td><td>FKCRRWQWRMKKLGAPSITCVRRAF</td><td>LfcinB (C–C)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>~7% hemolytic at 256mg/ml</td><td>Pig</td><td>CCCCCCHHHHTTSSCCCCGGGGGCC</td><td align="center"><a href="jmol.php?id=2911&amp;di=1" width="5px" height="5px"><img src="pymol_images/2911.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2911&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2911.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2912">2912</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21607695">21607695</a></td><td align="right">2011</td><td>FKCRRWQWRMKKLGAPSITCVRRAF</td><td>LfcinB</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>&lt;10% hemolytic at 256mg/ml</td><td>Pig</td><td>CCCCCCHHHHTTSSCCCCGGGGGCC</td><td align="center"><a href="jmol.php?id=2912&amp;di=1" width="5px" height="5px"><img src="pymol_images/2912.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2912&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2912.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2913">2913</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21607695">21607695</a></td><td align="right">2011</td><td>FKCRRWQWRMKKLGA</td><td>LfcinB15</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>&gt;7% hemolytic at 256mg/ml</td><td>Pig</td><td>CCCCSSHHHHTSSCC</td><td align="center"><a href="jmol.php?id=2913&amp;di=1" width="5px" height="5px"><img src="pymol_images/2913.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2913&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2913.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2914">2914</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21607695">21607695</a></td><td align="right">2011</td><td>RRWQWRMKKLG</td><td>LfcinB11</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>~6% hemolytic at 256mg/ml</td><td>Pig</td><td>CCCHHHHTTCC</td><td align="center"><a href="jmol.php?id=2914&amp;di=1" width="5px" height="5px"><img src="pymol_images/2914.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2914&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2914.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2915">2915</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21607695">21607695</a></td><td align="right">2011</td><td>RRWQWRMKK</td><td>LfcinB9</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>&lt;5% hemolytic at 256mg/ml</td><td>Pig</td><td>CCCCSSSCC</td><td align="center"><a href="jmol.php?id=2915&amp;di=1" width="5px" height="5px"><img src="pymol_images/2915.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2915&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2915.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2916">2916</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GKGRWLERIGKAGGIIIGGALDHL</td><td>NRC-01</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCCCTTTHHHHSSSCCCTTSCTTC</td><td align="center"><a href="jmol.php?id=2916&amp;di=1" width="5px" height="5px"><img src="pymol_images/2916.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2916&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2916.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2917">2917</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>WLRRIGKGVKIIGGAALDHL</td><td>NRC-02</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CTTCCSSSCCCSSSSSCCCC</td><td align="center"><a href="jmol.php?id=2917&amp;di=1" width="5px" height="5px"><img src="pymol_images/2917.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2917&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2917.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2918">2918</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GRRKRKWLRRIGKGVKIIGGAALDHL</td><td>NRC-03</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCCSCCCGGGCSCCSCCSSSSSCCCC</td><td align="center"><a href="jmol.php?id=2918&amp;di=1" width="5px" height="5px"><img src="pymol_images/2918.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2919">2919</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GWGSFFKKAAHVGKHVGKAALTHYL</td><td>NRC-04</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCHHHHTTTHHHHHHHTSGGGSCCC</td><td align="center"><a href="jmol.php?id=2919&amp;di=1" width="5px" height="5px"><img src="pymol_images/2919.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2920">2920</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>FLGALIKGAIHGGRFIHGMIQNHH</td><td>NRC-05</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CHHHHCCSSCCSTTCCCCTTTCCC</td><td align="center"><a href="jmol.php?id=2920&amp;di=1" width="5px" height="5px"><img src="pymol_images/2920.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2920&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2920.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2921">2921</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GWGSIFKHGRHAAKHIGHAAVNHYL</td><td>NRC-06</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCTTTCSSSHHHHTTTCHHHHHCCC</td><td align="center"><a href="jmol.php?id=2921&amp;di=1" width="5px" height="5px"><img src="pymol_images/2921.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2921&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2921.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2922">2922</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>RWGKWFKKATHVGKHVGKAALTAYL</td><td>NRC-07</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCHHHHTTTTCSSSCCCTTSTTTCC</td><td align="center"><a href="jmol.php?id=2922&amp;di=1" width="5px" height="5px"><img src="pymol_images/2922.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2922&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2922.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2923">2923</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>RSTEDIIKSISGGGFLNAMNA</td><td>NRC-08</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCTTHHHHTTSSCCCCTTSCC</td><td align="center"><a href="jmol.php?id=2923&amp;di=1" width="5px" height="5px"><img src="pymol_images/2923.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2923&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2923.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2924">2924</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>FFRLLFHGVHHGGGYLNAA</td><td>NRC-09</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CGGGHHHHSEESSSCEECC</td><td align="center"><a href="jmol.php?id=2924&amp;di=1" width="5px" height="5px"><img src="pymol_images/2924.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2925">2925</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>FFRLLFHGVHHVGKIKPRA</td><td>NRC-10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCCCCCCCCCSGGGCCCCC</td><td align="center"><a href="jmol.php?id=2925&amp;di=1" width="5px" height="5px"><img src="pymol_images/2925.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2925&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2925.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2926">2926</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GWKSVFRKAKKVGKTVGGLALDHYL</td><td>NRC-11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCGGGHHHHCCCCCCSSSSCSSCCC</td><td align="center"><a href="jmol.php?id=2926&amp;di=1" width="5px" height="5px"><img src="pymol_images/2926.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2926&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2926.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2927">2927</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GWKKWFNRAKKVGKTVGGLAVDHYL</td><td>NRC-12</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CGGGTHHHHGGGCSCCSSCSSSTTC</td><td align="center"><a href="jmol.php?id=2927&amp;di=1" width="5px" height="5px"><img src="pymol_images/2927.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2927&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2927.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2928">2928</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GWRTLLKKAEVKTVGKLALKHYL</td><td>NRC-13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CTHHHHTTTSTHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2928&amp;di=1" width="5px" height="5px"><img src="pymol_images/2928.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2928&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2928.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2929">2929</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>AGWGSIFKHIFKAGKFIHGAIQAHND</td><td>NRC-14</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCSCCTHHHHCSSCCCCCGGGTTCCC</td><td align="center"><a href="jmol.php?id=2929&amp;di=1" width="5px" height="5px"><img src="pymol_images/2929.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2929&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2929.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2930">2930</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GFWGKLFKLGLHGIGLLHLHL</td><td>NRC-15</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Anticancer</td><td>50% hemolytic at 64μg/ml</td><td>Human</td><td>CCTHHHHTTSCCCTTCCCSCC</td><td align="center"><a href="jmol.php?id=2930&amp;di=1" width="5px" height="5px"><img src="pymol_images/2930.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2930&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2930.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2931">2931</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GWKKWLRKGAKHLGQAAIK</td><td>NRC-16</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CGGGTHHHHTTSSSSSTTC</td><td align="center"><a href="jmol.php?id=2931&amp;di=1" width="5px" height="5px"><img src="pymol_images/2931.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2931&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2931.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2932">2932</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GWKKWLRKGAKHLGQAAIKGLAS</td><td>NRC-17</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CHHHHGGGGHHHHHHHHHTCCCC</td><td align="center"><a href="jmol.php?id=2932&amp;di=1" width="5px" height="5px"><img src="pymol_images/2932.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2932&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2932.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2933">2933</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GWKKWFTKGERLSQRHFA</td><td>NRC-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCCCCCSCSSSCSCCSCC</td><td align="center"><a href="jmol.php?id=2933&amp;di=1" width="5px" height="5px"><img src="pymol_images/2933.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2933&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2933.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2934">2934</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>FLGLLFHGVHHVGKWIHGLIHGHH</td><td>NRC-19</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Anticancer</td><td>50% hemolytic at 16μg/ml</td><td>Human</td><td>CTTSCCCSCCCTTBTTBCCCSSCC</td><td align="center"><a href="jmol.php?id=2934&amp;di=1" width="5px" height="5px"><img src="pymol_images/2934.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2934&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2934.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2935">2935</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GFLGILFHGVHHGRKKALHMNSERRS</td><td>NRC-20</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCCSSCCBSSSSSBSSCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=2935&amp;di=1" width="5px" height="5px"><img src="pymol_images/2935.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2935&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2935.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2936">2936</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GWKDWFRKAKKVGKTVGGLALNHYL</td><td>NRC-123</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CTHHHHHHHTTSCCTTTTCCTTSCC</td><td align="center"><a href="jmol.php?id=2936&amp;di=1" width="5px" height="5px"><img src="pymol_images/2936.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2937">2937</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GIRKWFKKAAHVGKEVGKVALNACL</td><td>NRC-124</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CHHHHHHHHTTSSTTTSTTCTTSCC</td><td align="center"><a href="jmol.php?id=2937&amp;di=1" width="5px" height="5px"><img src="pymol_images/2937.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2937&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2937.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2938">2938</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GLKKWFKKAVHVGKKVGKVALNAYL</td><td>NRC-125</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2939">2939</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GWRKWIKKATHVGKHIGKAALDAYI</td><td>NRC-126</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CHHHHTTTTTSSCCCHHHHTSCCCC</td><td align="center"><a href="jmol.php?id=2939&amp;di=1" width="5px" height="5px"><img src="pymol_images/2939.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2939&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2939.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2940">2940</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GCKKWFKKAAHVGKNVGKVALNAYL</td><td>NRC-127</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCTTTHHHHTCSCCCSSSCCTTSCC</td><td align="center"><a href="jmol.php?id=2940&amp;di=1" width="5px" height="5px"><img src="pymol_images/2940.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2940&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2940.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2941">2941</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GIRKWFKKAAHVGKKVGKVALNAYL</td><td>NRC-128</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CHHHHHHHHHHHCTTTSSSCSSSCC</td><td align="center"><a href="jmol.php?id=2941&amp;di=1" width="5px" height="5px"><img src="pymol_images/2941.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2941&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2941.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2942">2942</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GrrkrkWLrrIGkGVkIIGGAALDHL</td><td>NRC-03D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2943">2943</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GRRKRKWLRRIGKGVKIIGGAALDHL</td><td>NRC-03B</td><td>Amidation</td><td>Biotin</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCCSCCCGGGCSCCSCCSSSSSCCCC</td><td align="center"><a href="jmol.php?id=2943&amp;di=1" width="5px" height="5px"><img src="pymol_images/2943.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2944">2944</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21729875">21729875</a></td><td align="right">2011</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>Mag2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Anticancer</td><td>50% hemolytic at &gt;256μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=2944&amp;di=1" width="5px" height="5px"><img src="pymol_images/2944.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2945">2945</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDIWDWICEVLSDFK</td><td>C5A</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =3.46μM</td><td>Human</td><td>CCTTCSSSTTTGGGGCCC</td><td align="center"><a href="jmol.php?id=2945&amp;di=1" width="5px" height="5px"><img src="pymol_images/2945.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2945&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2945.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2946">2946</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDLWDWICEVLSDFK</td><td>I6L</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =13.84μM</td><td>Human</td><td>CCTTTTTGGGGGTTSCCC</td><td align="center"><a href="jmol.php?id=2946&amp;di=1" width="5px" height="5px"><img src="pymol_images/2946.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2946&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2946.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2947">2947</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDIWDWLCEVLSDFK</td><td>I10L</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =6.92μM</td><td>Human</td><td>CCTTHHHHHHTTTTSCCC</td><td align="center"><a href="jmol.php?id=2947&amp;di=1" width="5px" height="5px"><img src="pymol_images/2947.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2947&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2947.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2948">2948</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDIWDWICELLSDFK</td><td>V13L</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =3.44μM</td><td>Human</td><td>CCSTTTTTTTHHHHTCCC</td><td align="center"><a href="jmol.php?id=2948&amp;di=1" width="5px" height="5px"><img src="pymol_images/2948.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2948&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2948.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2949">2949</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDLWDWLCEVLSDFK</td><td>I6L/I10L</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =6.92μM</td><td>Human</td><td>CCSSTTTTTTGGGTSCCC</td><td align="center"><a href="jmol.php?id=2949&amp;di=1" width="5px" height="5px"><img src="pymol_images/2949.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2949&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2949.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2950">2950</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDLWDWICELLSDFK</td><td>I6L/V13L</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =53.75μM</td><td>Human</td><td>CCTTHHHHHHHGGGGCCC</td><td align="center"><a href="jmol.php?id=2950&amp;di=1" width="5px" height="5px"><img src="pymol_images/2950.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2950&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2950.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2951">2951</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDIWDWLCELLSDFK</td><td>I10L/V13L</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =3.44μM</td><td>Human</td><td>CCSSGGGGGGCCSSSCCC</td><td align="center"><a href="jmol.php?id=2951&amp;di=1" width="5px" height="5px"><img src="pymol_images/2951.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2951&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2951.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2952">2952</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDLWDWLCELLSDFK</td><td>I6L/I10L/V13L</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =107.50μM</td><td>Human</td><td>CCSSTTTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=2952&amp;di=1" width="5px" height="5px"><img src="pymol_images/2952.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2952&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2952.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2954">2954</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDIWDWVCEVLSDFK</td><td>I10V</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =13.93μM</td><td>Human</td><td>CCSSTTTTHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=2954&amp;di=1" width="5px" height="5px"><img src="pymol_images/2954.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2954&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2954.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2955">2955</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDIWDWACEVLSDFK</td><td>I10A</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =14.10μM</td><td>Human</td><td>CCTTHHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=2955&amp;di=1" width="5px" height="5px"><img src="pymol_images/2955.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2955&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2955.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2956">2956</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDIWDWGCEVLSDFK</td><td>I10G</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =28.38μM </td><td>Human</td><td>CCTTGGGTCTTTTTTCCC</td><td align="center"><a href="jmol.php?id=2956&amp;di=1" width="5px" height="5px"><img src="pymol_images/2956.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2956&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2956.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2957">2957</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDIWDWSCEVLSDFK</td><td>I10S</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =28.00μM</td><td>Human</td><td>CCSSCTTTHHHHTTTCCC</td><td align="center"><a href="jmol.php?id=2957&amp;di=1" width="5px" height="5px"><img src="pymol_images/2957.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2957&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2957.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2958">2958</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDIWDWECEVLSDFK</td><td>I10E</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =214.91μM</td><td>Human</td><td>CCSCCSSSCHHHHTTTTC</td><td align="center"><a href="jmol.php?id=2958&amp;di=1" width="5px" height="5px"><img src="pymol_images/2958.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2958&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2958.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2959">2959</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21801309">21801309</a></td><td align="right">2011</td><td>SWLRDIWDWKCEVLSDFK</td><td>I10K</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Anti-HCV/HIV</td><td>MHC =214.81μM</td><td>Human</td><td>CCTTTTTTTTTBTTTBCC</td><td align="center"><a href="jmol.php?id=2959&amp;di=1" width="5px" height="5px"><img src="pymol_images/2959.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2959&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2959.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2960">2960</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21820306">21820306</a></td><td align="right">2011</td><td>tle-K-tle-A-tle-A-tle-A</td><td>A</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Tle=Tert-leucine</td><td align="right">8</td><td>Antimicrobial</td><td>No hemolytic at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2961">2961</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21820306">21820306</a></td><td align="right">2011</td><td>Tle-k-Tle-a-Tle-a-Tle-a</td><td>B</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Tle=Tert-leucine</td><td align="right">8</td><td>Antimicrobial</td><td>No hemolytic at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2962">2962</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21820306">21820306</a></td><td align="right">2011</td><td>tle-k-tle-A-tle-A-tle-A</td><td>C</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Tle=Tert-leucine</td><td align="right">8</td><td>Antimicrobial</td><td>No hemolytic at 1000μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2963">2963</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21820306">21820306</a></td><td align="right">2011</td><td>Tle-K-Tle-a-Tle-a-Tle-a</td><td>D</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Tle=Tert-leucine</td><td align="right">8</td><td>Antimicrobial</td><td>No hemolytic at 1000μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2964">2964</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21955251">21955251</a></td><td align="right">2011</td><td>GIIKKI</td><td>G(IIKK)I</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">6</td><td>Antimicrobial and Antitumor</td><td>Non-hemolytic up to 250μM</td><td>Human</td><td>CCCCCC</td><td align="center"><a href="jmol.php?id=2964&amp;di=1" width="5px" height="5px"><img src="pymol_images/2964.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2964&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2964.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2965">2965</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21955251">21955251</a></td><td align="right">2011</td><td>GIIKKIIKKI</td><td>G(IIKK)<sub>2</sub>I</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial and Antitumor</td><td>Non-hemolytic up to 250μM</td><td>Human</td><td>CTHHHHTTTC</td><td align="center"><a href="jmol.php?id=2965&amp;di=1" width="5px" height="5px"><img src="pymol_images/2965.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2965&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2965.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2966">2966</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21955251">21955251</a></td><td align="right">2011</td><td>GIIKKIIKKIIKKI</td><td>G(IIKK)<sub>3</sub>I</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial and Antitumor</td><td>EC<sub>50</sub> &gt;250μM</td><td>Human</td><td>CGGGGTGGGTTTTC</td><td align="center"><a href="jmol.php?id=2966&amp;di=1" width="5px" height="5px"><img src="pymol_images/2966.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2966&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2966.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2967">2967</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21955251">21955251</a></td><td align="right">2011</td><td>GIIKKIIKKIIKKIIKKI</td><td>G(IIKK)<sub>4</sub>I</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and Antitumor</td><td>EC<sub>50</sub> = 41±2μM</td><td>Human</td><td>CGGGGTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=2967&amp;di=1" width="5px" height="5px"><img src="pymol_images/2967.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2967&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2967.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2968">2968</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21956793">21956793</a></td><td align="right">2011</td><td>LLIILRRRIRKQAHAHSK</td><td>pVEC</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and CPP</td><td>&gt;10% hemolytic at 200μM</td><td>Human</td><td>CTTTCGGGHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2968&amp;di=1" width="5px" height="5px"><img src="pymol_images/2968.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2968&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2968.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2969">2969</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21956793">21956793</a></td><td align="right">2011</td><td>LLIILRRRWRKQARARSK</td><td>pVEC-a1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and CPP</td><td>&gt;10% hemolytic at 200μM</td><td>Human</td><td>CGGGCGGGHHHHTTSCCC</td><td align="center"><a href="jmol.php?id=2969&amp;di=1" width="5px" height="5px"><img src="pymol_images/2969.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2969&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2969.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2970">2970</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21956793">21956793</a></td><td align="right">2011</td><td>LLIILRRRWRKQAKAKSK</td><td>pVEC-a2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and CPP</td><td>&gt;10% hemolytic at 200μM</td><td>Human</td><td>CGGGCGGGHHHHGGGTCC</td><td align="center"><a href="jmol.php?id=2970&amp;di=1" width="5px" height="5px"><img src="pymol_images/2970.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2970&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2970.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2971">2971</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21956793">21956793</a></td><td align="right">2011</td><td>LLIILRRRWRRQARARSR</td><td>pVEC-a3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and CPP</td><td>&gt;10% hemolytic at 94μM</td><td>Human</td><td>CCCCCHHHHTTTGGGGCC</td><td align="center"><a href="jmol.php?id=2971&amp;di=1" width="5px" height="5px"><img src="pymol_images/2971.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2971&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2971.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2972">2972</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21956793">21956793</a></td><td align="right">2011</td><td>LLIILKKKWKKQAKAKSK</td><td>pVEC-a4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial and CPP</td><td>&gt;10% hemolytic at 200μM</td><td>Human</td><td>CCSCCGGGHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=2972&amp;di=1" width="5px" height="5px"><img src="pymol_images/2972.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2972&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2972.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2973">2973</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21968924">21968924</a></td><td align="right">2011</td><td>GRPNPVNNKPT*PHPRL</td><td>formaecin I</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>*=α-GalNAc</td><td align="right">16</td><td>Antimicrobial</td><td>&lt;5% hemolytic at 100μM</td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2974">2974</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21968924">21968924</a></td><td align="right">2011</td><td>GRPNPVNNKPTPHPRL</td><td>Non-glycosylated formaecin I</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>&lt;5% hemolytic at 100μM</td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2975">2975</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21968924">21968924</a></td><td align="right">2011</td><td>GKPRPYSPRPT*SHPRPIRV</td><td>M-drosocin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>*=α-GalNAc</td><td align="right">19</td><td>Antimicrobial</td><td>&lt;5% hemolytic at 100μM</td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2976">2976</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21968924">21968924</a></td><td align="right">2011</td><td>GKPRPYSPRPTSHPRPIRV</td><td>Non-glycosylated M-drosocin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>&lt;5% hemolytic at 100μM</td><td>Rat</td><td>CCSSCCCSSCSCCCCCSCC</td><td align="center"><a href="jmol.php?id=2976&amp;di=1" width="5px" height="5px"><img src="pymol_images/2976.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2976&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2976.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2977">2977</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>GLLCYCRKGHCKRGERVRGTCGIRFLYCCPRR</td><td>Crp4-wt</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>4.2% hemolytic at 0μg/ml</td><td>Human</td><td>CCCCCCSSCCCCCCCSSCCCCTTSCSCCCCCC</td><td align="center"><a href="jmol.php?id=2977&amp;di=1" width="5px" height="5px"><img src="pymol_images/2977.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2977&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2977.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2978">2978</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>GLLCYCRKGHCKRGERVRGTCGIRFLYCCPRR</td><td>Crp4-wt</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>4.1% hemolytic at 12.5μg/ml</td><td>Human</td><td>CCCCCCSSCCCCCCCSSCCCCTTSCSCCCCCC</td><td align="center"><a href="jmol.php?id=2978&amp;di=1" width="5px" height="5px"><img src="pymol_images/2978.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2978&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2978.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2979">2979</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>GLLCYCRKGHCKRGERVRGTCGIRFLYCCPRR</td><td>Crp4-wt</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>3.4% hemolytic at 25μg/ml</td><td>Human</td><td>CCCCCCSSCCCCCCCSSCCCCTTSCSCCCCCC</td><td align="center"><a href="jmol.php?id=2979&amp;di=1" width="5px" height="5px"><img src="pymol_images/2979.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2979&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2979.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2980">2980</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>GLLCYCRKGHCKRGERVRGTCGIRFLYCCPRR</td><td>Crp4-wt</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>3.0% hemolytic at 50μg/ml</td><td>Human</td><td>CCCCCCSSCCCCCCCSSCCCCTTSCSCCCCCC</td><td align="center"><a href="jmol.php?id=2980&amp;di=1" width="5px" height="5px"><img src="pymol_images/2980.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2980&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2980.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2981">2981</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>GLLCYCRKGHCKRGERVRGTCGIRFLYCCPRR</td><td>Crp4-wt</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>3.1% hemolytic at 100μg/mL</td><td>Human</td><td>CCCCCCSSCCCCCCCSSCCCCTTSCSCCCCCC</td><td align="center"><a href="jmol.php?id=2981&amp;di=1" width="5px" height="5px"><img src="pymol_images/2981.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2981&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2981.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2982">2982</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>CYCRKGHCKRGERVRGTCGIRFLYCCPRRGLL</td><td>Crp4-1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>4.2% hemolytic at 0μg/ml</td><td>Human</td><td>CCCSSCCCSCCCCCCCCCCCCCSCCCTTSSCC</td><td align="center"><a href="jmol.php?id=2982&amp;di=1" width="5px" height="5px"><img src="pymol_images/2982.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2983">2983</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>CYCRKGHCKRGERVRGTCGIRFLYCCPRRGLL</td><td>Crp4-1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>5.4% hemolytic at 12.5μg/ml</td><td>Human</td><td>CCCSSCCCSCCCCCCCCCCCCCSCCCTTSSCC</td><td align="center"><a href="jmol.php?id=2983&amp;di=1" width="5px" height="5px"><img src="pymol_images/2983.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2984">2984</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>CYCRKGHCKRGERVRGTCGIRFLYCCPRRGLL</td><td>Crp4-1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>5.8% hemolytic at 25μg/ml</td><td>Human</td><td>CCCSSCCCSCCCCCCCCCCCCCSCCCTTSSCC</td><td align="center"><a href="jmol.php?id=2984&amp;di=1" width="5px" height="5px"><img src="pymol_images/2984.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2985">2985</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>CYCRKGHCKRGERVRGTCGIRFLYCCPRRGLL</td><td>Crp4-1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>5.9% hemolytic at 50μg/ml</td><td>Human</td><td>CCCSSCCCSCCCCCCCCCCCCCSCCCTTSSCC</td><td align="center"><a href="jmol.php?id=2985&amp;di=1" width="5px" height="5px"><img src="pymol_images/2985.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2986">2986</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22040603">22040603</a></td><td align="right">2011</td><td>CYCRKGHCKRGERVRGTCGIRFLYCCPRRGLL</td><td>Crp4-1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>4.9% hemolytic at 100μg/mL</td><td>Human</td><td>CCCSSCCCSCCCCCCCCCCCCCSCCCTTSSCC</td><td align="center"><a href="jmol.php?id=2986&amp;di=1" width="5px" height="5px"><img src="pymol_images/2986.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2987">2987</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22082130">22082130</a></td><td align="right">2011</td><td>KLAKKLAKLAKLAKAL</td><td>Modelin-5-COOH</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>~1% hemolytic at 300μM</td><td>Pig</td><td>CTHHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=2987&amp;di=1" width="5px" height="5px"><img src="pymol_images/2987.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2987&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2987.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2988">2988</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22082130">22082130</a></td><td align="right">2011</td><td>KLAKKLAKLAKLAKAL</td><td>Modelin-5-CONH<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>&lt;2% hemolytic at 300μM</td><td>Pig</td><td>CTHHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=2988&amp;di=1" width="5px" height="5px"><img src="pymol_images/2988.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2988&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2988.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=2989">2989</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22189867">22189867</a></td><td align="right">2012</td><td>FKCRRWQWRWKKLGAKPVPIIYCNRRTGKCQRM</td><td>LFT33</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>Non-hemolytic upto 256μg/ml</td><td>Human</td><td>CCCSCCHHHHTTCCSSSCCCCCCCCSCSCCCCC</td><td align="center"><a href="jmol.php?id=2989&amp;di=1" width="5px" height="5px"><img src="pymol_images/2989.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2989&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2989.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=2999">2999</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21762675">21762675</a></td><td align="right">2011</td><td>ELLKAVRLIK</td><td>Rev–NIS</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>0% hemolytic upto 100μM(non-hemolytic)</td><td>Human</td><td>CCSSCSCCCC</td><td align="center"><a href="jmol.php?id=2999&amp;di=1" width="5px" height="5px"><img src="pymol_images/2999.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=2999&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/2999.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3000">3000</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21762675">21762675</a></td><td align="right">2011</td><td>RLRKAVRLIK</td><td>Anal R</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>0% hemolytic upto 100μM (non-hemolytic)</td><td>Human</td><td>CCTTTSSCCC</td><td align="center"><a href="jmol.php?id=3000&amp;di=1" width="5px" height="5px"><img src="pymol_images/3000.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3000&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3000.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3001">3001</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21762675">21762675</a></td><td align="right">2011</td><td>ELSKAVRLIK</td><td>Anal S</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>0% hemolytic upto 100μM (non-hemolytic)</td><td>Human</td><td>CTTTSSCCCC</td><td align="center"><a href="jmol.php?id=3001&amp;di=1" width="5px" height="5px"><img src="pymol_images/3001.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3001&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3001.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3002">3002</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGETCVGGTCNTPGCTCSWPVCTRN</td><td>kB1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>41.2 ±1.5% hemolytic at 25μM</td><td>Human</td><td>CCCCCSSHHHHCCCCCSSCCCCSSCCSCC</td><td align="center"><a href="jmol.php?id=3002&amp;di=1" width="5px" height="5px"><img src="pymol_images/3002.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3002&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3002.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3003">3003</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPKCGETCVGGTCNTPGCTCSWPVCTRN</td><td>V4K</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>2.9±0.5% hemolytic at 25μM</td><td>Human</td><td>CCCCSSSSSSCCCCCCCCCSSSCCTTSCC</td><td align="center"><a href="jmol.php?id=3003&amp;di=1" width="5px" height="5px"><img src="pymol_images/3003.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3003&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3003.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3004">3004</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGDTCVGGTCNTPGCTCSWPVCTRN</td><td>E7D</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>3.8±0.4% hemolytic at 25μM</td><td>Human</td><td>CCCCCCCTTTGGGSCSCCTTSCCCCCSCC</td><td align="center"><a href="jmol.php?id=3004&amp;di=1" width="5px" height="5px"><img src="pymol_images/3004.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3004&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3004.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3005">3005</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGEKCVGGTCNTPGCTCSWPVCTRN</td><td>T8K</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>3.4±0.7% hemolytic at 25μM</td><td>Human</td><td>CCSSSSCCSSSTTSCCSSSSSCCCTTSCC</td><td align="center"><a href="jmol.php?id=3005&amp;di=1" width="5px" height="5px"><img src="pymol_images/3005.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3005&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3005.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3006">3006</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGETCKGGTCNTPGCTCSWPVCTRN</td><td>V10K</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>0±1.7% hemolytic at 25μM</td><td>Human</td><td>CCSSSSSSSCSSSCCCSSCSSCSGGGTCC</td><td align="center"><a href="jmol.php?id=3006&amp;di=1" width="5px" height="5px"><img src="pymol_images/3006.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3006&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3006.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3007">3007</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGETCVGGTCKTPGCTCSWPVCTRN</td><td>N15K</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>2.1±0.4% hemolytic at 25μM</td><td>Human</td><td>CCBSSSBSSSSBSCCCSCCCSCCCSSSCC</td><td align="center"><a href="jmol.php?id=3007&amp;di=1" width="5px" height="5px"><img src="pymol_images/3007.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3007&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3007.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3008">3008</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGETCVGGTCNKPGCTCSWPVCTRN</td><td>T16K</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>4.3±0.1% hemolytic at 25μM</td><td>Human</td><td>CCSSTTSCSHHHHSSSTTCSCCCCGGGGC</td><td align="center"><a href="jmol.php?id=3008&amp;di=1" width="5px" height="5px"><img src="pymol_images/3008.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3008&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3008.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3009">3009</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGETCVGGTCNTPGCKCSWPVCTRN</td><td>T20K</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>51.3±1.6% hemolytic at 25μM</td><td>Human</td><td>CCCCCSCTTTTTTCCCSSCCCCCTTCSCC</td><td align="center"><a href="jmol.php?id=3009&amp;di=1" width="5px" height="5px"><img src="pymol_images/3009.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3009&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3009.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3010">3010</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGETCVGGTCNTPGCTCSWPKCTRN</td><td>V25K</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>1.1±1.0% hemolytic at 25μM</td><td>Human</td><td>CCSSCSCTTTTTTSCCCCCSSCCSSCCCC</td><td align="center"><a href="jmol.php?id=3010&amp;di=1" width="5px" height="5px"><img src="pymol_images/3010.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3010&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3010.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3011">3011</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGETCVGGTCNTPGCTCSWPVCTRK</td><td>N29K</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>54.5±0.9% hemolytic at 25μM</td><td>Human</td><td>CCSSTTSCSHHHHSCCSCCCCSCSSSSCC</td><td align="center"><a href="jmol.php?id=3011&amp;di=1" width="5px" height="5px"><img src="pymol_images/3011.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3011&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3011.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3012">3012</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGETCVGGTCNTPGCTCSKPVCTRN</td><td>W23K</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>2.4±0.7% hemolytic at 25μM</td><td>Human</td><td>CCSCSSSTTCCCCCSCSSCCSCCCCCCCC</td><td align="center"><a href="jmol.php?id=3012&amp;di=1" width="5px" height="5px"><img src="pymol_images/3012.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3012&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3012.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3013">3013</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGETCVGGTCNTPGCNCSWPVCTRK</td><td>T20N29K</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>64.4±2.2% hemolytic at 25μM</td><td>Human</td><td>CBCCSSCTTTBCCCCSTTCCCCSSSTTCC</td><td align="center"><a href="jmol.php?id=3013&amp;di=1" width="5px" height="5px"><img src="pymol_images/3013.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3013&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3013.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3014">3014</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGETCVGGTCNTPGCTCSWPVCTKN</td><td>R28K</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>12.0±1.8% hemolytic at 25μM</td><td>Human</td><td>CCCCCCCTTCSSCCCCSSCSSCCSSSCCC</td><td align="center"><a href="jmol.php?id=3014&amp;di=1" width="5px" height="5px"><img src="pymol_images/3014.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3014&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3014.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3015">3015</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21576247">21576247</a></td><td align="right">2011</td><td>GLPVCGETCVGGTCNTPGCTCSWPVCTRN</td><td>acyclic</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Hemolytic and Anti-HIV</td><td>2.8±0.3% hemolytic at 25μM</td><td>Human</td><td>CCCCCSSHHHHCCCCCSSCCCCSSCCSCC</td><td align="center"><a href="jmol.php?id=3015&amp;di=1" width="5px" height="5px"><img src="pymol_images/3015.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3015&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3015.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3016">3016</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21462284">21462284</a></td><td align="right">2011</td><td>WKWKWK</td><td>Normal-(WK)<sub>3</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">6</td><td>Antimicrobial</td><td>0% hemolytic upto 400μM</td><td>Human</td><td>CCSTTC</td><td align="center"><a href="jmol.php?id=3016&amp;di=1" width="5px" height="5px"><img src="pymol_images/3016.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3016&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3016.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3017">3017</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21462284">21462284</a></td><td align="right">2011</td><td>WKWKWK</td><td>Normal-(WK)<sub>3</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">6</td><td>Antimicrobial</td><td>~10% hemolytic at 600μM</td><td>Human</td><td>CCSTTC</td><td align="center"><a href="jmol.php?id=3017&amp;di=1" width="5px" height="5px"><img src="pymol_images/3017.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3017&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3017.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3018">3018</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21462284">21462284</a></td><td align="right">2011</td><td>KWKWKW</td><td>Reversed-(KW)<sub>3</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">6</td><td>Antimicrobial</td><td>0% hemolytic upto 400μM</td><td>Human</td><td>CCCCCC</td><td align="center"><a href="jmol.php?id=3018&amp;di=1" width="5px" height="5px"><img src="pymol_images/3018.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3018&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3018.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3019">3019</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21462284">21462284</a></td><td align="right">2011</td><td>KWKWKW</td><td>Reversed-(KW)<sub>3</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">6</td><td>Antimicrobial</td><td>~10% hemolytic at 600μM</td><td>Human</td><td>CCCCCC</td><td align="center"><a href="jmol.php?id=3019&amp;di=1" width="5px" height="5px"><img src="pymol_images/3019.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3019&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3019.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3020">3020</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22029824">22029824</a></td><td align="right">2012</td><td>FIFPKKNIINSLFGR</td><td>Andersonin-D1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>Non-hemolytic upto 50μg/mL</td><td>Human</td><td>CCSSSSCGGGTCSCC</td><td align="center"><a href="jmol.php?id=3020&amp;di=1" width="5px" height="5px"><img src="pymol_images/3020.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3020&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3020.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3021">3021</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22029824">22029824</a></td><td align="right">2012</td><td>KEKLKLKCKAPKCYNDKLACT</td><td>Andersonin-G1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>Non-hemolytic upto 50μg/mL</td><td>Human</td><td>CCCCCCTTCCSCCSTTTCSCC</td><td align="center"><a href="jmol.php?id=3021&amp;di=1" width="5px" height="5px"><img src="pymol_images/3021.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3021&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3021.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3022">3022</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22029824">22029824</a></td><td align="right">2012</td><td>ENMFNIKSSVESDSFWG</td><td>Andersonin-N1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>Non-hemolytic upto 50μg/mL</td><td>Human</td><td>CCCCSSCBTTBCHHHHC</td><td align="center"><a href="jmol.php?id=3022&amp;di=1" width="5px" height="5px"><img src="pymol_images/3022.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3022&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3022.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3023">3023</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22029824">22029824</a></td><td align="right">2012</td><td>EMLKKKEVKMERKT</td><td>Andersonin-Q1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>Non-hemolytic upto 50μg/mL</td><td>Human</td><td>CCCSSSCTTTCTTC</td><td align="center"><a href="jmol.php?id=3023&amp;di=1" width="5px" height="5px"><img src="pymol_images/3023.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3023&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3023.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3024">3024</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22029824">22029824</a></td><td align="right">2012</td><td>ENAEEDIVLMENLFCSYIVGSADSFWT</td><td>Andersonin-R1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>Non-hemolytic upto 50μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3025">3025</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22029824">22029824</a></td><td align="right">2012</td><td>DANVENGEDAEDLTDKFIGLMG</td><td>Andersonin-S1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>Non-hemolytic upto 50μg/mL</td><td>Human</td><td>CCCSCCSCCSCCCTHHHHSTTC</td><td align="center"><a href="jmol.php?id=3025&amp;di=1" width="5px" height="5px"><img src="pymol_images/3025.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3025&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3025.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3026">3026</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>ILPWKWPWWPWRR</td><td>ID</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>5-10% hemolysis at 25μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3027">3027</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>I*LPWKWPWWPWRR</td><td>ID-I</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>*=Reduced amide bond ψ[CH2NH]</td><td align="right">13</td><td>Antimicrobial</td><td>5-10% hemolysis at 25μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3028">3028</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>ILPWKWPW*WPWRR</td><td>ID-W</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>*=Reduced amide bond ψ[CH2NH]</td><td align="right">13</td><td>Antimicrobial</td><td>5-10% hemolysis at 25μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3029">3029</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>I*LPWKWPW*WPWRR</td><td>ID-IW</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>*=Reduced amide bond ψ[CH2NH]</td><td align="right">13</td><td>Antimicrobial</td><td>5% hemolysis at 25μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3030">3030</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>ILPWKWPWWPWRR</td><td>ID</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>10% hemolysis at 50μg/mL</td><td>Human</td><td>CCCCCCSSCSSCC</td><td align="center"><a href="jmol.php?id=3030&amp;di=1" width="5px" height="5px"><img src="pymol_images/3030.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3031">3031</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>I*LPWKWPWWPWRR</td><td>ID-I</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>*=Reduced amide bond ψ[CH2NH]</td><td align="right">13</td><td>Antimicrobial</td><td>10% hemolysis at 50μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3032">3032</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>ILPWKWPW*WPWRR</td><td>ID-W</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>*=Reduced amide bond ψ[CH2NH]</td><td align="right">13</td><td>Antimicrobial</td><td>10% hemolysis at 50μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3033">3033</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>I*LPWKWPW*WPWRR</td><td>ID-IW</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>*=Reduced amide bond ψ[CH2NH]</td><td align="right">13</td><td>Antimicrobial</td><td>5% hemolysis at 50μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3034">3034</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>ILPWKWPWWPWRR</td><td>ID</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>10% hemolysis at 100μg/mL</td><td>Human</td><td>CCCCCCSSCSSCC</td><td align="center"><a href="jmol.php?id=3034&amp;di=1" width="5px" height="5px"><img src="pymol_images/3034.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3035">3035</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>I*LPWKWPWWPWRR</td><td>ID-I</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>*=Reduced amide bond ψ[CH2NH]</td><td align="right">13</td><td>Antimicrobial</td><td>10-15% hemolysis at 100μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3036">3036</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>ILPWKWPW*WPWRR</td><td>ID-W</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>*=Reduced amide bond ψ[CH2NH]</td><td align="right">13</td><td>Antimicrobial</td><td>10-15% hemolysis at 100μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3037">3037</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>I*LPWKWPW*WPWRR</td><td>ID-IW</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>*=Reduced amide bond ψ[CH2NH]</td><td align="right">13</td><td>Antimicrobial</td><td>10% hemolysis at 100μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3038">3038</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>ILPWKWPWWPWRR</td><td>ID</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>30% hemolysis at 200μg/mL</td><td>Human</td><td>CCCCCCSSCSSCC</td><td align="center"><a href="jmol.php?id=3038&amp;di=1" width="5px" height="5px"><img src="pymol_images/3038.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3039">3039</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>I*LPWKWPWWPWRR</td><td>ID-I</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>*=Reduced amide bond ψ[CH2NH]</td><td align="right">13</td><td>Antimicrobial</td><td>30% hemolysis at 200μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3040">3040</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>ILPWKWPW*WPWRR</td><td>ID-W</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>*=Reduced amide bond ψ[CH2NH]</td><td align="right">13</td><td>Antimicrobial</td><td>30% hemolysis at 200μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3041">3041</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19038369">19038369</a></td><td align="right">2009</td><td>I*LPWKWPW*WPWRR</td><td>ID-IW</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>*=Reduced amide bond ψ[CH2NH]</td><td align="right">13</td><td>Antimicrobial</td><td>~10% hemolysis at 200μg/mL</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3042">3042</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19961432">19961432</a></td><td align="right">2010</td><td>SVSNAATRVCRTGRSRWRDVCRNFMR</td><td>Gr-1L(25-50)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>20-30% hemolysis at 50μM</td><td>Human</td><td>CCCSSSSSCSCCCSSSSGGGHHHHCC</td><td align="center"><a href="jmol.php?id=3042&amp;di=1" width="5px" height="5px"><img src="pymol_images/3042.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3042&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3042.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3043">3043</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19961432">19961432</a></td><td align="right">2010</td><td>SVSNAATRVCRTGRSRWRDVCRNFMR</td><td>Gr-1C(25-50)</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>20-30% hemolysis at 50μM</td><td>Human</td><td>CCCSSSSSCSCCCSSSSGGGHHHHCC</td><td align="center"><a href="jmol.php?id=3043&amp;di=1" width="5px" height="5px"><img src="pymol_images/3043.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3043&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3043.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3044">3044</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19961432">19961432</a></td><td align="right">2010</td><td>SVSNAATRVARTGRSRWRDVARNFMR</td><td>Gr-2 (25-50)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>20-30% hemolysis at 50μM</td><td>Human</td><td>CCSSHHHHHHTSSCCSSHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3044&amp;di=1" width="5px" height="5px"><img src="pymol_images/3044.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3044&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3044.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3045">3045</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19961432">19961432</a></td><td align="right">2010</td><td>SRWRDVARNFMRRYQSRVIQGLVA</td><td>Gr-3 (39-62)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>90% hemolysis at 50μM</td><td>Human</td><td>CCSHHHHHHHHHHHHHHTHHHHTC</td><td align="center"><a href="jmol.php?id=3045&amp;di=1" width="5px" height="5px"><img src="pymol_images/3045.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3045&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3045.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3046">3046</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19961432">19961432</a></td><td align="right">2010</td><td>SVSNAATRVCRTGRSRWRDVCRNFMR</td><td>Gr-1L(25-50)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>40-50% hemolysis at 100μM</td><td>Human</td><td>CCCSSSSSCSCCCSSSSGGGHHHHCC</td><td align="center"><a href="jmol.php?id=3046&amp;di=1" width="5px" height="5px"><img src="pymol_images/3046.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3046&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3046.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3047">3047</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19961432">19961432</a></td><td align="right">2010</td><td>SVSNAATRVCRTGRSRWRDVCRNFMR</td><td>Gr-1C(25-50)</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>40-50% hemolysis at 100μM</td><td>Human</td><td>CCCSSSSSCSCCCSSSSGGGHHHHCC</td><td align="center"><a href="jmol.php?id=3047&amp;di=1" width="5px" height="5px"><img src="pymol_images/3047.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3047&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3047.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3048">3048</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19961432">19961432</a></td><td align="right">2010</td><td>SVSNAATRVARTGRSRWRDVARNFMR</td><td>Gr-2 (25-50)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>40-50% hemolysis at 100μM</td><td>Human</td><td>CCSSHHHHHHTSSCCSSHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3048&amp;di=1" width="5px" height="5px"><img src="pymol_images/3048.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3048&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3048.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3049">3049</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19961432">19961432</a></td><td align="right">2010</td><td>SRWRDVARNFMRRYQSRVIQGLVA</td><td>Gr-3 (39-62)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>100% hemolysis at 100μM</td><td>Human</td><td>CCSHHHHHHHHHHHHHHTHHHHTC</td><td align="center"><a href="jmol.php?id=3049&amp;di=1" width="5px" height="5px"><img src="pymol_images/3049.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3049&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3049.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3050">3050</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19961432">19961432</a></td><td align="right">2010</td><td>SVSNAATRVCRTGRSRWRDVCRNFMR</td><td>Gr-1L(25-50)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>80-85% hemolysis at 200μM</td><td>Human</td><td>CCCSSSSSCSCCCSSSSGGGHHHHCC</td><td align="center"><a href="jmol.php?id=3050&amp;di=1" width="5px" height="5px"><img src="pymol_images/3050.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3050&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3050.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3051">3051</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19961432">19961432</a></td><td align="right">2010</td><td>SVSNAATRVCRTGRSRWRDVCRNFMR</td><td>Gr-1C(25-50)</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>70% hemolysis at 200μM</td><td>Human</td><td>CCCSSSSSCSCCCSSSSGGGHHHHCC</td><td align="center"><a href="jmol.php?id=3051&amp;di=1" width="5px" height="5px"><img src="pymol_images/3051.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3051&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3051.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3052">3052</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19961432">19961432</a></td><td align="right">2010</td><td>SVSNAATRVARTGRSRWRDVARNFMR</td><td>Gr-2 (25-50)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>90% hemolysis at 200μM</td><td>Human</td><td>CCSSHHHHHHTSSCCSSHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3052&amp;di=1" width="5px" height="5px"><img src="pymol_images/3052.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3052&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3052.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3053">3053</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19961432">19961432</a></td><td align="right">2010</td><td>SRWRDVARNFMRRYQSRVIQGLVA</td><td>Gr-3 (39-62)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>100% hemolysis at 200μM</td><td>Human</td><td>CCSHHHHHHHHHHHHHHTHHHHTC</td><td align="center"><a href="jmol.php?id=3053&amp;di=1" width="5px" height="5px"><img src="pymol_images/3053.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3053&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3053.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3054">3054</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20363271">20363271</a></td><td align="right">2010</td><td>KLKKLLKKWLKLLKKLLK</td><td>K<sub>9</sub>L<sub>8</sub>W</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HC<sub>50</sub> =14μM</td><td>Human</td><td>CGGGSHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3054&amp;di=1" width="5px" height="5px"><img src="pymol_images/3054.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3054&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3054.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3055">3055</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20363271">20363271</a></td><td align="right">2010</td><td>KLKKLlKKWLKlLKKLLk</td><td>D<sub>3</sub>-K<sub>9</sub>L<sub>8</sub>W-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HC<sub>50</sub> =150μM</td><td>Human</td><td>CGGGSHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3055&amp;di=1" width="5px" height="5px"><img src="pymol_images/3055.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3055&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3055.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3056">3056</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20363271">20363271</a></td><td align="right">2010</td><td>KLKKlLKKWlKLLKkLLK</td><td>D<sub>3</sub>-K<sub>9</sub>L<sub>8</sub>W-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HC<sub>50</sub> =116μM</td><td>Human</td><td>CGGGSHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3056&amp;di=1" width="5px" height="5px"><img src="pymol_images/3056.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3056&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3056.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3057">3057</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20363271">20363271</a></td><td align="right">2010</td><td>KLKkLLKkWLKlLKKlLK</td><td>D<sub>4</sub>-K<sub>9</sub>L<sub>8</sub>W</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HC<sub>50</sub> =800μM</td><td>Human</td><td>CGGGSHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3057&amp;di=1" width="5px" height="5px"><img src="pymol_images/3057.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3057&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3057.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3058">3058</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20363271">20363271</a></td><td align="right">2010</td><td>KLkKLlKKwLKlLKkLLk</td><td>D<sub>6</sub>-K<sub>9</sub>L<sub>8</sub>W</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HC<sub>50</sub> =800μM</td><td>Human</td><td>CGGGSHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3058&amp;di=1" width="5px" height="5px"><img src="pymol_images/3058.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3058&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3058.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3059">3059</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20363271">20363271</a></td><td align="right">2010</td><td>KlKkLlKkWlKlLkKlLk</td><td>D<sub>9</sub>-K<sub>9</sub>L<sub>8</sub>W-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HC<sub>50</sub> =800μM</td><td>Human</td><td>CGGGSHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3059&amp;di=1" width="5px" height="5px"><img src="pymol_images/3059.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3059&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3059.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3060">3060</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20363271">20363271</a></td><td align="right">2010</td><td>KLKKLLKKWlkllkkllk</td><td>D<sub>9</sub>-K<sub>9</sub>L<sub>8</sub>W-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HC<sub>50</sub> =154μM</td><td>Human</td><td>CGGGSHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3060&amp;di=1" width="5px" height="5px"><img src="pymol_images/3060.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3060&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3060.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3061">3061</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20399752">20399752</a></td><td align="right">2010</td><td>KRFKKFFKKLKNSVKKRAKKFFKKPKVIGVTFPF</td><td>NA-CATH</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antimicrobial</td><td>EC<sub>50</sub> =0.37μM</td><td>Human</td><td>CCTTTTHHHHGGGGHHHHTTTTTSTTCSSSCCCC</td><td align="center"><a href="jmol.php?id=3061&amp;di=1" width="5px" height="5px"><img src="pymol_images/3061.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3062">3062</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20399752">20399752</a></td><td align="right">2010</td><td>KRFKKFFKKLK</td><td>ATRA-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>EC<sub>50</sub> =5.98μM</td><td>Human</td><td>CGGGGTTTTCC</td><td align="center"><a href="jmol.php?id=3062&amp;di=1" width="5px" height="5px"><img src="pymol_images/3062.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3062&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3062.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3063">3063</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20399752">20399752</a></td><td align="right">2010</td><td>KRAKKFFKKPK</td><td>ATRA-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>EC<sub>50</sub> =107μM</td><td>Human</td><td>CGGGTSSSTTC</td><td align="center"><a href="jmol.php?id=3063&amp;di=1" width="5px" height="5px"><img src="pymol_images/3063.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3063&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3063.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3064">3064</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20399752">20399752</a></td><td align="right">2010</td><td>KRAKKFFKKLK</td><td>ATRA-1A</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>EC<sub>50</sub> =7.98μM</td><td>Human</td><td>CCHHHHTTTCC</td><td align="center"><a href="jmol.php?id=3064&amp;di=1" width="5px" height="5px"><img src="pymol_images/3064.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3064&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3064.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3065">3065</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20399752">20399752</a></td><td align="right">2010</td><td>KRFKKFFKKPK</td><td>ATRA-1P</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>EC<sub>50</sub> =95.92μM</td><td>Human</td><td>CGGGGTCCTTC</td><td align="center"><a href="jmol.php?id=3065&amp;di=1" width="5px" height="5px"><img src="pymol_images/3065.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3065&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3065.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3067">3067</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20457198">20457198</a></td><td align="right">2010</td><td>DVNDLKNLCAKTHNLLPMCAMFGKK</td><td>DK25</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>No hemolysis upto &lt;400μM</td><td>Human</td><td>CCCTTGGGGGTSSCCCHHHHHTCCC</td><td align="center"><a href="jmol.php?id=3067&amp;di=1" width="5px" height="5px"><img src="pymol_images/3067.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3067&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3067.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3068">3068</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20457198">20457198</a></td><td align="right">2010</td><td>DVNDLKNLCAKTHNLLPMCAMF</td><td>DF-22-amide</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>No hemolysis upto &lt;400μM</td><td>Human</td><td>CCSTHHHHHHHSSTTSSSSTTC</td><td align="center"><a href="jmol.php?id=3068&amp;di=1" width="5px" height="5px"><img src="pymol_images/3068.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3068&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3068.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3069">3069</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20466006">20466006</a></td><td align="right">2010</td><td>YVPPVQKPHPNGPKFPTFP</td><td>PP30</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 125μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3070">3070</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20466006">20466006</a></td><td align="right">2010</td><td>YVPPVQKPHPNGPKFPTFP</td><td>PP30</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>3.8% hemolysis at 250μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3071">3071</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20472009">20472009</a></td><td align="right">2010</td><td>GIGAVLKVLSTGLPALISWIKRKRQQ</td><td>melittin-S</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>35% hemolysis at 3μM</td><td>Human</td><td>CCSCCCCTTTSCCHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3071&amp;di=1" width="5px" height="5px"><img src="pymol_images/3071.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3071&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3071.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3074">3074</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20600431">20600431</a></td><td align="right">2010</td><td>GSKKPVPIIYCNRRSGKCQRM</td><td>S-thanatin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>Low hemolytic activity up to 400μg/ml</td><td>Human</td><td>CTTSSSCCCCCCCCSSCCCCC</td><td align="center"><a href="jmol.php?id=3074&amp;di=1" width="5px" height="5px"><img src="pymol_images/3074.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3074&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3074.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3075">3075</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20603168">20603168</a></td><td align="right">2010</td><td>GLKEIFKAGLGSLVKGIAAHVAS</td><td>Alyteserin-1c</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>LC<sub>50</sub> =220μM</td><td>Human</td><td>CTHHHHTSSSTTHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3075&amp;di=1" width="5px" height="5px"><img src="pymol_images/3075.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3075&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3075.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3076">3076</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20603168">20603168</a></td><td align="right">2010</td><td>GLKKIFKAGLGSLVKGIAAHVAS</td><td>E4K</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;400μM</td><td>Human</td><td>CGGGGHHHHTSSSTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3076&amp;di=1" width="5px" height="5px"><img src="pymol_images/3076.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3076&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3076.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3077">3077</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20603168">20603168</a></td><td align="right">2010</td><td>GLKkIFKAGLGSLVKGIAAHVAS</td><td>E4k</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;400μM</td><td>Human</td><td>CGGGGHHHHTSSSTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3077&amp;di=1" width="5px" height="5px"><img src="pymol_images/3077.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3077&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3077.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3078">3078</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20603168">20603168</a></td><td align="right">2010</td><td>GLKKIFKKGLGSLVKGIAAHVAS</td><td>E4K,A8K</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;400μM</td><td>Human</td><td>CCHHHHTTCSSGGGGGTHHHHTC</td><td align="center"><a href="jmol.php?id=3078&amp;di=1" width="5px" height="5px"><img src="pymol_images/3078.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3078&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3078.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3079">3079</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20603168">20603168</a></td><td align="right">2010</td><td>GLKkIFKKGLGSLVKGIAAHVAS</td><td>E4k,A8K</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;400μM</td><td>Human</td><td>CCHHHHTTCSSGGGGGTHHHHTC</td><td align="center"><a href="jmol.php?id=3079&amp;di=1" width="5px" height="5px"><img src="pymol_images/3079.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3079&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3079.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3080">3080</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20603168">20603168</a></td><td align="right">2010</td><td>GLKKIFKAGLGSLKKGIAAHVAS</td><td>E4K,V14K</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;400μM</td><td>Human</td><td>CCHHHHTTSSCCGGGGGGGTSCC</td><td align="center"><a href="jmol.php?id=3080&amp;di=1" width="5px" height="5px"><img src="pymol_images/3080.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3080&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3080.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3081">3081</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20603168">20603168</a></td><td align="right">2010</td><td>GLKkIFKAGLGSLKKGIAAHVAS</td><td>E4k,V14K</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;400μM</td><td>Human</td><td>CCHHHHTTSSCCGGGGGGGTSCC</td><td align="center"><a href="jmol.php?id=3081&amp;di=1" width="5px" height="5px"><img src="pymol_images/3081.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3081&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3081.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3082">3082</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20603168">20603168</a></td><td align="right">2010</td><td>GLKKIFKAGLGSLVKGIKAHVAS</td><td>E4K,A18K</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;400μM</td><td>Human</td><td>CCTTCGGGTTCTHHHHTGGGCCC</td><td align="center"><a href="jmol.php?id=3082&amp;di=1" width="5px" height="5px"><img src="pymol_images/3082.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3082&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3082.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3083">3083</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20603168">20603168</a></td><td align="right">2010</td><td>GLKkIFKAGLGSLVKGIKAHVAS</td><td>E4k,A18K</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;400μM</td><td>Human</td><td>CCTTCGGGTTCTHHHHTGGGCCC</td><td align="center"><a href="jmol.php?id=3083&amp;di=1" width="5px" height="5px"><img src="pymol_images/3083.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3083&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3083.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3084">3084</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20603168">20603168</a></td><td align="right">2010</td><td>GLKEIFKAGLGSLVKGIAAHVAS</td><td>N-Pal,E4K</td><td>Free</td><td>palmitate(pal)</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>LC<sub>50</sub> =10μM</td><td>Human</td><td>CTHHHHTSSSTTHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3084&amp;di=1" width="5px" height="5px"><img src="pymol_images/3084.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3085">3085</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20603168">20603168</a></td><td align="right">2010</td><td>GLKKIFKAGLGSLVKGIK*AHVAS</td><td>E4K,Pal-A18K</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>*=palmitate(PAL)</td><td align="right">23</td><td>Antimicrobial</td><td>LC<sub>50</sub> =30μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3086">3086</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20637182">20637182</a></td><td align="right">2010</td><td>FSPQMLQDIIEAATAIL</td><td>A17</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 200μg/ml</td><td>Mouse</td><td>CCCCTTTTTTHHHHHTC</td><td align="center"><a href="jmol.php?id=3086&amp;di=1" width="5px" height="5px"><img src="pymol_images/3086.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3086&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3086.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3087">3087</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20637182">20637182</a></td><td align="right">2010</td><td>FSPQMLQDIIEKKTKIL</td><td>K17</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0% hemolysis at 200μg/ml</td><td>Mouse</td><td>CCCHHHHHHTGGGTTCC</td><td align="center"><a href="jmol.php?id=3087&amp;di=1" width="5px" height="5px"><img src="pymol_images/3087.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3087&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3087.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3088">3088</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20656059">20656059</a></td><td align="right">2010</td><td>GIGKFLHSAGKFGKAFLGEVMKS</td><td>Magainin-B2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>0% hemolysis at 200μM</td><td>Human</td><td>CTTSCCGGGGCSSGGGGGGGTCC</td><td align="center"><a href="jmol.php?id=3088&amp;di=1" width="5px" height="5px"><img src="pymol_images/3088.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3088&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3088.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3091">3091</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20735987">20735987</a></td><td align="right">2010</td><td>CIAKGNGCQPSGVQGNCCSGHCHKEPGWVAGYCK</td><td>Psacotheasin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antifungal</td><td>0% hemolysis at 100μM</td><td>Human</td><td>CCSCSSSCCSSCSSCSSSSCCCSSSSCCCTTTTC</td><td align="center"><a href="jmol.php?id=3091&amp;di=1" width="5px" height="5px"><img src="pymol_images/3091.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3091&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3091.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3094">3094</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>KWLKKWL</td><td>L<sub>2</sub>K<sub>3</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">7</td><td>Antimicrobial</td><td>Negligible hemolysis at 320μg/ml</td><td>Human</td><td>CCSCCCC</td><td align="center"><a href="jmol.php?id=3094&amp;di=1" width="5px" height="5px"><img src="pymol_images/3094.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3094&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3094.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3095">3095</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>KWLLKWL</td><td>L<sub>3</sub>K<sub>2</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">7</td><td>Antimicrobial</td><td>0-5% hemolysis at 160μg/ml</td><td>Human</td><td>CCHHHHC</td><td align="center"><a href="jmol.php?id=3095&amp;di=1" width="5px" height="5px"><img src="pymol_images/3095.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3095&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3095.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3096">3096</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>KKWLKKWLK</td><td>L<sub>2</sub>K<sub>5</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>Negligible hemolysis at 320μg/ml</td><td>Human</td><td>CTTHHHHCC</td><td align="center"><a href="jmol.php?id=3096&amp;di=1" width="5px" height="5px"><img src="pymol_images/3096.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3096&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3096.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3097">3097</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LKWLKKWLK</td><td>L<sub>3</sub>K<sub>4</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>0% hemolysis at 160μg/ml</td><td>Human</td><td>CHHHHTTCC</td><td align="center"><a href="jmol.php?id=3097&amp;di=1" width="5px" height="5px"><img src="pymol_images/3097.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3097&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3097.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3098">3098</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LKWLLKWLK</td><td>L<sub>4</sub>K<sub>3</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>10% hemolysis at 40μg/ml</td><td>Human</td><td>CGGGGTTTC</td><td align="center"><a href="jmol.php?id=3098&amp;di=1" width="5px" height="5px"><img src="pymol_images/3098.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3098&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3098.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3099">3099</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LKWLLKWLL</td><td>L<sub>5</sub>K<sub>2</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>60% hemolysis at 40μg/ml</td><td>Human</td><td>CTTHHHHHC</td><td align="center"><a href="jmol.php?id=3099&amp;di=1" width="5px" height="5px"><img src="pymol_images/3099.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3099&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3099.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3100">3100</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LKKWLKKWLKK</td><td>L<sub>3</sub>K<sub>6</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>Negligible hemolysis at 320μg/ml</td><td>Human</td><td>CGGGGGGTTTC</td><td align="center"><a href="jmol.php?id=3100&amp;di=1" width="5px" height="5px"><img src="pymol_images/3100.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3100&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3100.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3101">3101</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LLKWLKKWLKK</td><td>L<sub>4</sub>K<sub>5</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>0-5% hemolysis at 40μg/ml</td><td>Human</td><td>CHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3101&amp;di=1" width="5px" height="5px"><img src="pymol_images/3101.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3101&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3101.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3102">3102</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LLKWLLKWLKK</td><td>L<sub>5</sub>K<sub>4</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>40-50% hemolysis at 40μg/ml</td><td>Human</td><td>CHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3102&amp;di=1" width="5px" height="5px"><img src="pymol_images/3102.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3102&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3102.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3103">3103</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LLKWLLKWLLK</td><td>L<sub>6</sub>K<sub>3</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>55-60% hemolysis at 40μg/ml</td><td>Human</td><td>CHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3103&amp;di=1" width="5px" height="5px"><img src="pymol_images/3103.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3103&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3103.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3104">3104</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>KWLLKWL</td><td>L<sub>3</sub>K<sub>2</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">7</td><td>Antimicrobial</td><td>10% hemolysis at 320μg/ml</td><td>Human</td><td>CCHHHHC</td><td align="center"><a href="jmol.php?id=3104&amp;di=1" width="5px" height="5px"><img src="pymol_images/3104.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3104&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3104.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3105">3105</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LKWLKKWLK</td><td>L<sub>3</sub>K<sub>4</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>0-5% hemolysis at 320μg/ml</td><td>Human</td><td>CHHHHTTCC</td><td align="center"><a href="jmol.php?id=3105&amp;di=1" width="5px" height="5px"><img src="pymol_images/3105.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3105&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3105.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3106">3106</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LKWLLKWLK</td><td>L<sub>4</sub>K<sub>3</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>25% hemolysis at 80μg/ml</td><td>Human</td><td>CGGGGTTTC</td><td align="center"><a href="jmol.php?id=3106&amp;di=1" width="5px" height="5px"><img src="pymol_images/3106.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3106&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3106.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3107">3107</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LKWLLKWLK</td><td>L<sub>4</sub>K<sub>3</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>40-50% hemolysis at 160μg/ml</td><td>Human</td><td>CGGGGTTTC</td><td align="center"><a href="jmol.php?id=3107&amp;di=1" width="5px" height="5px"><img src="pymol_images/3107.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3107&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3107.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3108">3108</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LKWLLKWLL</td><td>L<sub>5</sub>K<sub>2</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>65% hemolysis at 80μg/ml</td><td>Human</td><td>CTTHHHHHC</td><td align="center"><a href="jmol.php?id=3108&amp;di=1" width="5px" height="5px"><img src="pymol_images/3108.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3108&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3108.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3109">3109</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LKWLLKWLL</td><td>L<sub>5</sub>K<sub>2</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>70% hemolysis at 160μg/ml</td><td>Human</td><td>CTTHHHHHC</td><td align="center"><a href="jmol.php?id=3109&amp;di=1" width="5px" height="5px"><img src="pymol_images/3109.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3109&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3109.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3110">3110</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LLKWLKKWLKK</td><td>L<sub>4</sub>K<sub>5</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>5% hemolysis at 80μg/ml</td><td>Human</td><td>CHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3110&amp;di=1" width="5px" height="5px"><img src="pymol_images/3110.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3110&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3110.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3111">3111</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LLKWLKKWLKK</td><td>L<sub>4</sub>K<sub>5</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>10% hemolysis at 160μg/ml</td><td>Human</td><td>CHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3111&amp;di=1" width="5px" height="5px"><img src="pymol_images/3111.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3111&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3111.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3112">3112</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LLKWLLKWLKK</td><td>L<sub>5</sub>K<sub>4</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>50-60% hemolysis at 80μg/ml</td><td>Human</td><td>CHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3112&amp;di=1" width="5px" height="5px"><img src="pymol_images/3112.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3112&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3112.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3113">3113</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LLKWLLKWLKK</td><td>L<sub>5</sub>K<sub>4</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>60-65% hemolysis at 160μg/ml</td><td>Human</td><td>CHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3113&amp;di=1" width="5px" height="5px"><img src="pymol_images/3113.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3113&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3113.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3114">3114</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LLKWLLKWLLK</td><td>L<sub>6</sub>K<sub>3</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>60-70% hemolysis at 80μg/ml</td><td>Human</td><td>CHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3114&amp;di=1" width="5px" height="5px"><img src="pymol_images/3114.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3114&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3114.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3115">3115</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20736034">20736034</a></td><td align="right">2011</td><td>LLKWLLKWLLK</td><td>L<sub>6</sub>K<sub>3</sub>W<sub>2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>70-75% hemolysis at 160μg/ml</td><td>Human</td><td>CHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3115&amp;di=1" width="5px" height="5px"><img src="pymol_images/3115.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3115&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3115.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3116">3116</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21073979">21073979</a></td><td align="right">2011</td><td>GKLNLFLSRLEILKLFVGAL</td><td>Pelteobagrin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>No hemolysis upto 400μg/ml</td><td>Rabbit</td><td>CCCSSHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3116&amp;di=1" width="5px" height="5px"><img src="pymol_images/3116.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3116&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3116.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3117">3117</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21241661">21241661</a></td><td align="right">2011</td><td>RWCVYAYVRVRGVLVRYRRCW</td><td>Arenicin-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>0% hemolysis at 12.5μM</td><td>Human</td><td>CCSCCTTSCCCCCSCCCCCCC</td><td align="center"><a href="jmol.php?id=3117&amp;di=1" width="5px" height="5px"><img src="pymol_images/3117.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3117&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3117.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3118">3118</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21241661">21241661</a></td><td align="right">2011</td><td>RWCVYAYVRVRGVLVRYRRCW</td><td>Arenicin-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>12% hemolysis at 25μM</td><td>Human</td><td>CCSCCTTSCCCCCSCCCCCCC</td><td align="center"><a href="jmol.php?id=3118&amp;di=1" width="5px" height="5px"><img src="pymol_images/3118.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3118&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3118.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3119">3119</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21241661">21241661</a></td><td align="right">2011</td><td>RWCVYAYVRVRGVLVRYRRCW</td><td>Arenicin-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>35% hemolysis at 50μM</td><td>Human</td><td>CCSCCTTSCCCCCSCCCCCCC</td><td align="center"><a href="jmol.php?id=3119&amp;di=1" width="5px" height="5px"><img src="pymol_images/3119.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3119&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3119.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3120">3120</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21241661">21241661</a></td><td align="right">2011</td><td>RWCVYAYVRVRGVLVRYRRCW</td><td>Arenicin-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>51% hemolysis at 100μM</td><td>Human</td><td>CCSCCTTSCCCCCSCCCCCCC</td><td align="center"><a href="jmol.php?id=3120&amp;di=1" width="5px" height="5px"><img src="pymol_images/3120.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3120&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3120.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3121">3121</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21241661">21241661</a></td><td align="right">2011</td><td>CVYAYVRVRGVLVRYRRCW</td><td>Arenicin-1(analog-1)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 50μM</td><td>Human</td><td>CCSTTSCCCCCCCCCCSCC</td><td align="center"><a href="jmol.php?id=3121&amp;di=1" width="5px" height="5px"><img src="pymol_images/3121.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3121&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3121.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3122">3122</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21241661">21241661</a></td><td align="right">2011</td><td>CVYAYVRVRGVLVRYRRCW</td><td>Arenicin-1(analog-1)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>4% hemolysis at 100μM</td><td>Human</td><td>CCSTTSCCCCCCCCCCSCC</td><td align="center"><a href="jmol.php?id=3122&amp;di=1" width="5px" height="5px"><img src="pymol_images/3122.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3122&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3122.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3123">3123</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21241661">21241661</a></td><td align="right">2011</td><td>RWCVYAYVRVAGVLVRYRRCW</td><td>Arenicin-1(analog-2)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>0% hemolysis at 12.5μM</td><td>Human</td><td>CCCSSCCCSSCSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3123&amp;di=1" width="5px" height="5px"><img src="pymol_images/3123.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3123&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3123.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3124">3124</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21241661">21241661</a></td><td align="right">2011</td><td>RWCVYAYVRVAGVLVRYRRCW</td><td>Arenicin-1(analog-2)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>7% hemolysis at 25μM</td><td>Human</td><td>CCCSSCCCSSCSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3124&amp;di=1" width="5px" height="5px"><img src="pymol_images/3124.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3124&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3124.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3125">3125</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21241661">21241661</a></td><td align="right">2011</td><td>RWCVYAYVRVAGVLVRYRRCW</td><td>Arenicin-1(analog-2)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>32% hemolysis at 50μM</td><td>Human</td><td>CCCSSCCCSSCSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3125&amp;di=1" width="5px" height="5px"><img src="pymol_images/3125.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3125&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3125.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3126">3126</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21241661">21241661</a></td><td align="right">2011</td><td>RWCVYAYVRVAGVLVRYRRCW</td><td>Arenicin-1(analog-2)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>45% hemolysis at 100μM</td><td>Human</td><td>CCCSSCCCSSCSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3126&amp;di=1" width="5px" height="5px"><img src="pymol_images/3126.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3126&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3126.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3127">3127</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21497177">21497177</a></td><td align="right">2011</td><td>GRFKRFRKKFKKLFKKLSPVIPLLHL</td><td>BMAP -27</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>10% hemolysis at 6.2μM</td><td>Human</td><td>CTTTTTHHHHHHHHHHHSSCCTTCCC</td><td align="center"><a href="jmol.php?id=3127&amp;di=1" width="5px" height="5px"><img src="pymol_images/3127.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3127&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3127.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3128">3128</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21497177">21497177</a></td><td align="right">2011</td><td>GRFKRFRKKFKKLFKKLS</td><td>BMAP -18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>10% hemolysis at &gt;400μM</td><td>Human</td><td>CTTTTTHHHHTTGGGTCC</td><td align="center"><a href="jmol.php?id=3128&amp;di=1" width="5px" height="5px"><img src="pymol_images/3128.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3128&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3128.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3129">3129</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21497177">21497177</a></td><td align="right">2011</td><td>GRWKRWRKKWKKLWKKLS</td><td>BMAP- 18-W</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>10% hemolysis at 38μM</td><td>Human</td><td>CTTTTTTTTCSTTTTTCC</td><td align="center"><a href="jmol.php?id=3129&amp;di=1" width="5px" height="5px"><img src="pymol_images/3129.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3129&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3129.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3130">3130</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21497177">21497177</a></td><td align="right">2011</td><td>GRLKRLRKKLKKLLKKLS</td><td>BMAP -18-L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>10% hemolysis at 300μM</td><td>Human</td><td>CHHHHTHHHHHHHGGGCC</td><td align="center"><a href="jmol.php?id=3130&amp;di=1" width="5px" height="5px"><img src="pymol_images/3130.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3130&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3130.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3131">3131</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21497177">21497177</a></td><td align="right">2011</td><td>GRIKRIRKKIKKLIKKLS</td><td>BMAP -18-I</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>10% hemolysis at 265μM</td><td>Human</td><td>CTTTTTGGGCGGGGGGTC</td><td align="center"><a href="jmol.php?id=3131&amp;di=1" width="5px" height="5px"><img src="pymol_images/3131.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3131&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3131.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3132">3132</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21497177">21497177</a></td><td align="right">2011</td><td>GRfKRfRKKfKKLfKKLS</td><td>BMAP -18-f</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>10% hemolysis at &gt;400μM</td><td>Human</td><td>CTTTTTHHHHTTGGGTCC</td><td align="center"><a href="jmol.php?id=3132&amp;di=1" width="5px" height="5px"><img src="pymol_images/3132.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3132&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3132.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3133">3133</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21664395">21664395</a></td><td align="right">2011</td><td>GIGKFLHSAGKFGKAFIGEIMKS</td><td>Magainin-M1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>LC<sub>50</sub> =180μM</td><td>Human</td><td>CCSSCSGGGSCCSTTTTTTTTTC</td><td align="center"><a href="jmol.php?id=3133&amp;di=1" width="5px" height="5px"><img src="pymol_images/3133.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3133&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3133.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3134">3134</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21664395">21664395</a></td><td align="right">2011</td><td>WCPPMIPLCSRF</td><td>Tigerinin-M1</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>LC<sub>50</sub> &gt;500μM</td><td>Human</td><td>CCCCCCSCSTTC</td><td align="center"><a href="jmol.php?id=3134&amp;di=1" width="5px" height="5px"><img src="pymol_images/3134.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3135">3135</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21664395">21664395</a></td><td align="right">2011</td><td>GIGKFLHSAGKFGKAFLGEVMKS</td><td>Magainin-MW1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>LC<sub>50</sub> &gt;200μM</td><td>Human</td><td>CTTSCCGGGGCSSGGGGGGGTCC</td><td align="center"><a href="jmol.php?id=3135&amp;di=1" width="5px" height="5px"><img src="pymol_images/3135.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3135&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3135.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3136">3136</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21664395">21664395</a></td><td align="right">2011</td><td>GMASKAGSVLGKITKIALGAL</td><td>PGLa-MW1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antibacterial</td><td>LC<sub>50</sub> &gt;200μM</td><td>Human</td><td>CCEETTEEGGGGGGGTTSCCC</td><td align="center"><a href="jmol.php?id=3136&amp;di=1" width="5px" height="5px"><img src="pymol_images/3136.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3136&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3136.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3137">3137</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21664395">21664395</a></td><td align="right">2011</td><td>GLGSLLGKAFKFGLKTVGKMMGGAPREQ</td><td>CPF-MW1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antibacterial</td><td>LC<sub>50</sub> =70μM</td><td>Human</td><td>CHHHHHHHHHHHHGGGHHHHHTTSCCCC</td><td align="center"><a href="jmol.php?id=3137&amp;di=1" width="5px" height="5px"><img src="pymol_images/3137.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3138">3138</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21664395">21664395</a></td><td align="right">2011</td><td>GLGSLLGKAFKFGLKTVGKMMGGAPREE</td><td>CPF-MW2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antibacterial</td><td>LC<sub>50</sub> =105μM</td><td>Human</td><td>CHHHHHHHHHHHHHTTHHHHSCCSCCCC</td><td align="center"><a href="jmol.php?id=3138&amp;di=1" width="5px" height="5px"><img src="pymol_images/3138.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3139">3139</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>LGRVDIHVWDGVYIRGR</td><td>KK42-63-3H</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Cytotoxic</td><td>5.9% hemolysis at 100μM</td><td>Human</td><td>CCCCCCSSSSSSCTTCC</td><td align="center"><a href="jmol.php?id=3139&amp;di=1" width="5px" height="5px"><img src="pymol_images/3139.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3139&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3139.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3140">3140</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>LGRVDIHVWDGVYI</td><td>PIP56-69</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Cytotoxic</td><td>2.48% hemolysis at 100μM</td><td>Human</td><td>CCCCCEEETTEEEC</td><td align="center"><a href="jmol.php?id=3140&amp;di=1" width="5px" height="5px"><img src="pymol_images/3140.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3140&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3140.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3141">3141</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>VDIHVWDGVVDIHVWDGV</td><td>PIP-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Cytotoxic</td><td>1.16% hemolysis at 100μM</td><td>Human</td><td>CCSSSTTTTCCSCCSCCC</td><td align="center"><a href="jmol.php?id=3141&amp;di=1" width="5px" height="5px"><img src="pymol_images/3141.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3141&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3141.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3142">3142</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>VDIHVWDGV</td><td>PIP[59-67]</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Cytotoxic</td><td>3.2% hemolysis at 100μM</td><td>Human</td><td>CCCCCCTTC</td><td align="center"><a href="jmol.php?id=3142&amp;di=1" width="5px" height="5px"><img src="pymol_images/3142.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3142&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3142.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3143">3143</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>LGRVDIHVWDGVYIRGR</td><td>P-NT.II</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Cytotoxic</td><td>9.5% hemolysis at 100μM</td><td>Human</td><td>CCCCCCSSSSSSCTTCC</td><td align="center"><a href="jmol.php?id=3143&amp;di=1" width="5px" height="5px"><img src="pymol_images/3143.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3143&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3143.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3144">3144</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>LGRVDIHVWDAVYIRGR</td><td>[D-Ala66]-PNT.II</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Cytotoxic</td><td>3% hemolysis at 100μM</td><td>Human</td><td>CCCCTTSCCCSSGGGTC</td><td align="center"><a href="jmol.php?id=3144&amp;di=1" width="5px" height="5px"><img src="pymol_images/3144.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3144&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3144.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3145">3145</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>VDIHVWD*GV</td><td>β-Asp65-PIP[59-67]</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>D*=β-aspartic acid</td><td align="right">9</td><td>Cytotoxic</td><td>1% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3146">3146</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>VDIHVWAGV</td><td>D65A-PIP[59-67]</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Cytotoxic</td><td>7.5% hemolysis at 100μM</td><td>Human</td><td>CCTTTTTCC</td><td align="center"><a href="jmol.php?id=3146&amp;di=1" width="5px" height="5px"><img src="pymol_images/3146.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3146&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3146.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3147">3147</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>VDIHVWSGV</td><td>D65S-PIP[59-67]</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Cytotoxic</td><td>1.03% hemolysis at 100μM</td><td>Human</td><td>CCSSSSCCC</td><td align="center"><a href="jmol.php?id=3147&amp;di=1" width="5px" height="5px"><img src="pymol_images/3147.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3147&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3147.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3148">3148</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>VDIHVWEGV</td><td>D65E-PIP[59-67]</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Cytotoxic</td><td>9.9% hemolysis at 100μM</td><td>Human</td><td>CCSSCSCCC</td><td align="center"><a href="jmol.php?id=3148&amp;di=1" width="5px" height="5px"><img src="pymol_images/3148.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3148&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3148.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3149">3149</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>VEIHVWEGV</td><td>D60,65E-PIP[59-67]</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Cytotoxic</td><td>2.24% hemolysis at 100μM</td><td>Human</td><td>CCCCCCCCC</td><td align="center"><a href="jmol.php?id=3149&amp;di=1" width="5px" height="5px"><img src="pymol_images/3149.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3149&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3149.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3150">3150</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21682682">21682682</a></td><td align="right">2011</td><td>CVDIHVWDGVC</td><td>Cyclic C-PIP[59-67]C</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">11</td><td>Cytotoxic</td><td>7.2% hemolysis at 100μM</td><td>Human</td><td>CCCSSCCCTTC</td><td align="center"><a href="jmol.php?id=3150&amp;di=1" width="5px" height="5px"><img src="pymol_images/3150.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3150&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3150.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3151">3151</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GKLEVLHSTKKFAKGFITGLTGQ</td><td>Magainin-SE1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;160μM</td><td>Human</td><td>CCCTTTSCTTTCTTCSSCCCTTC</td><td align="center"><a href="jmol.php?id=3151&amp;di=1" width="5px" height="5px"><img src="pymol_images/3151.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3151&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3151.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3152">3152</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GMATKAGTALGKVAKAVIGAAL</td><td>PGLa-SE1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;160μM</td><td>Human</td><td>CTTSSSSCTTHHHHTCHHHHCC</td><td align="center"><a href="jmol.php?id=3152&amp;di=1" width="5px" height="5px"><img src="pymol_images/3152.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3152&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3152.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3153">3153</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GMATAAGTTLGKLAKFVIGAV</td><td>PGLa-SE2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;160μM</td><td>Human</td><td>CCCCCCCSHHHHHHHHHCCCC</td><td align="center"><a href="jmol.php?id=3153&amp;di=1" width="5px" height="5px"><img src="pymol_images/3153.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3153&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3153.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3154">3154</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GLASTIGSLLGKFAKGGAQAFLQPK</td><td>XPF-SE2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;160μM</td><td>Human</td><td>CCTHHHHHHHHHHHSCSSSCTTSCC</td><td align="center"><a href="jmol.php?id=3154&amp;di=1" width="5px" height="5px"><img src="pymol_images/3154.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3155">3155</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GFWTTAAEGLKKFAKAGLASILNPK</td><td>XPF-SE3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>LC<sub>50</sub> =105μM</td><td>Human</td><td>CCCCHHHHHHTTGGGCSTTTSSSCC</td><td align="center"><a href="jmol.php?id=3155&amp;di=1" width="5px" height="5px"><img src="pymol_images/3155.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3155&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3155.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3156">3156</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GVWTTILGGLKKFAKGGLEALTNPK</td><td>XPF-SE4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>LC<sub>50</sub> =60μM</td><td>Human</td><td>CGGGSTTCSTHHHHHTSSTTCCSCC</td><td align="center"><a href="jmol.php?id=3156&amp;di=1" width="5px" height="5px"><img src="pymol_images/3156.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3156&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3156.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3157">3157</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GFLGPLLKLGLKGVAKVIPHLIPSRQQ</td><td>CPF-SE1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>LC<sub>50</sub> =50μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3158">3158</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GFLGPLLKLGLKGAAKLLPQLLPSRQQ</td><td>CPF-SE2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>LC<sub>50</sub> =50μM</td><td>Human</td><td>CCCGGGGGGCTTGGGGTHHHHSSSCCC</td><td align="center"><a href="jmol.php?id=3158&amp;di=1" width="5px" height="5px"><img src="pymol_images/3158.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3159">3159</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>GFLGSLLKTGLKVGSNLL</td><td>CPF-SE3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =220μM</td><td>Human</td><td>CCCHHHHTTTTTCSSCCC</td><td align="center"><a href="jmol.php?id=3159&amp;di=1" width="5px" height="5px"><img src="pymol_images/3159.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3159&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3159.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3160">3160</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22800690">22800690</a></td><td align="right">2012</td><td>FLGALLGPLMNLLQ</td><td>PFQa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>LC<sub>50</sub> =105μM</td><td>Human</td><td>CCTTSHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3160&amp;di=1" width="5px" height="5px"><img src="pymol_images/3160.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3160&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3160.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3161">3161</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22943778">22943778</a></td><td align="right">2012</td><td>FLPFLLSALPKVFCFFSKKC</td><td>Brevinin-1ZHa</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>HC<sub>50</sub> =3μM</td><td>Human</td><td>CCCTTTCSSCCSCCCSSCCC</td><td align="center"><a href="jmol.php?id=3161&amp;di=1" width="5px" height="5px"><img src="pymol_images/3161.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3161&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3161.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3162">3162</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22943778">22943778</a></td><td align="right">2012</td><td>FLPLLLSALPSFLCLVFKKC</td><td>Brevinin-1ZHb</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>HC<sub>50</sub> =5μM</td><td>Human</td><td>CHHHHHHHTTTTTSSGGGCC</td><td align="center"><a href="jmol.php?id=3162&amp;di=1" width="5px" height="5px"><img src="pymol_images/3162.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3162&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3162.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3164">3164</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22943778">22943778</a></td><td align="right">2012</td><td>GLADYWRTAFRANFANLGPGIRCKSARC</td><td>Ranatuerin-2ZHa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;64μM</td><td>Human</td><td>CCCHHHHTTGGGGSSSSSCCTTTSCCCC</td><td align="center"><a href="jmol.php?id=3164&amp;di=1" width="5px" height="5px"><img src="pymol_images/3164.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3164&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3164.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3165">3165</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22943778">22943778</a></td><td align="right">2012</td><td>LALKSGGWLRLFGLKDKKH</td><td>Chensinin-1ZHa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;92μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3166">3166</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22960382">22960382</a></td><td align="right">2012</td><td>KIKIPWGKVKDFLVGGMKAV</td><td>Bicarinalin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>50% hemolysis at 0.72mg/ml</td><td>Human</td><td>CCCCSSSBCBTTBCTBCCCC</td><td align="center"><a href="jmol.php?id=3166&amp;di=1" width="5px" height="5px"><img src="pymol_images/3166.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3166&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3166.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3167">3167</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22960382">22960382</a></td><td align="right">2012</td><td>LFKEILEKIKAKL</td><td>P-17</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Human</td><td>CHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3167&amp;di=1" width="5px" height="5px"><img src="pymol_images/3167.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3167&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3167.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3168">3168</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTAAGLLSNL</td><td>Alyteserin-2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =140μM</td><td>Human</td><td>CCSCGGGTCGGGTSCC</td><td align="center"><a href="jmol.php?id=3168&amp;di=1" width="5px" height="5px"><img src="pymol_images/3168.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3168&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3168.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3169">3169</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILKKLLSTAAGLLSNL</td><td>G3K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =105μM</td><td>Human</td><td>CHHHHHHHHHTTCCCC</td><td align="center"><a href="jmol.php?id=3169&amp;di=1" width="5px" height="5px"><img src="pymol_images/3169.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3169&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3169.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3170">3170</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLKTAAGLLSNL</td><td>S7K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =28μM</td><td>Human</td><td>CHHHHHTTTTTTCCCC</td><td align="center"><a href="jmol.php?id=3170&amp;di=1" width="5px" height="5px"><img src="pymol_images/3170.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3170&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3170.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3171">3171</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLkTAAGLLSNL</td><td>S7k</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =105μM</td><td>Human</td><td>CHHHHHTTTTTTCCCC</td><td align="center"><a href="jmol.php?id=3171&amp;di=1" width="5px" height="5px"><img src="pymol_images/3171.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3171&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3171.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3172">3172</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTAAKLLSNL</td><td>G11K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =24μM</td><td>Human</td><td>CHHHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=3172&amp;di=1" width="5px" height="5px"><img src="pymol_images/3172.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3172&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3172.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3173">3173</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTAAkLLSNL</td><td>G11k</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =42μM</td><td>Human</td><td>CHHHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=3173&amp;di=1" width="5px" height="5px"><img src="pymol_images/3173.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3173&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3173.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3174">3174</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTAAGLLKNL</td><td>S14K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =55μM</td><td>Human</td><td>CCCCTHHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=3174&amp;di=1" width="5px" height="5px"><img src="pymol_images/3174.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3174&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3174.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3175">3175</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTAAGLLSKL</td><td>N15K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =50μM</td><td>Human</td><td>CCCSGGGGTEETTEEC</td><td align="center"><a href="jmol.php?id=3175&amp;di=1" width="5px" height="5px"><img src="pymol_images/3175.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3175&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3175.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3176">3176</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTAAGLLSkL</td><td>N15k</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =310μM</td><td>Human</td><td>CCCSGGGGTEETTEEC</td><td align="center"><a href="jmol.php?id=3176&amp;di=1" width="5px" height="5px"><img src="pymol_images/3176.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3176&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3176.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3177">3177</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLKTAAKLLSNL</td><td>S7K,G11K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =38μM</td><td>Human</td><td>CHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3177&amp;di=1" width="5px" height="5px"><img src="pymol_images/3177.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3177&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3177.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3178">3178</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLkTAAkLLSNL</td><td>S7k,G11k</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =185μM</td><td>Human</td><td>CHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3178&amp;di=1" width="5px" height="5px"><img src="pymol_images/3178.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3178&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3178.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3179">3179</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTAAKLLSKL</td><td>G11K,N15K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =16μM</td><td>Human</td><td>CGGGGGHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3179&amp;di=1" width="5px" height="5px"><img src="pymol_images/3179.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3179&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3179.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3180">3180</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTAAkLLSKL</td><td>G11k,N15K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =195μM</td><td>Human</td><td>CGGGGGHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3180&amp;di=1" width="5px" height="5px"><img src="pymol_images/3180.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3180&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3180.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3181">3181</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSWAAGLLSNL</td><td>T8W</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =13μM</td><td>Human</td><td>CTTSGGGGGGGTCSCC</td><td align="center"><a href="jmol.php?id=3181&amp;di=1" width="5px" height="5px"><img src="pymol_images/3181.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3181&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3181.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3182">3182</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTWAGLLSNL</td><td>A9W</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =39μM</td><td>Human</td><td>CCCCGGGGGTTTTSCC</td><td align="center"><a href="jmol.php?id=3182&amp;di=1" width="5px" height="5px"><img src="pymol_images/3182.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3182&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3182.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3183">3183</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTAWGLLSNL</td><td>A10W</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =11μM</td><td>Human</td><td>CHHHHHTTGGGTSCCC</td><td align="center"><a href="jmol.php?id=3183&amp;di=1" width="5px" height="5px"><img src="pymol_images/3183.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3183&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3183.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3184">3184</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTAWkLLSNL</td><td>A10W,G11k</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =21μM</td><td>Human</td><td>CTTHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3184&amp;di=1" width="5px" height="5px"><img src="pymol_images/3184.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3184&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3184.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3185">3185</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTAWGLLSKL</td><td>A10W,N15K</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =6μM</td><td>Human</td><td>CCSCTHHHHTTTSSCC</td><td align="center"><a href="jmol.php?id=3185&amp;di=1" width="5px" height="5px"><img src="pymol_images/3185.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3185&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3185.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3186">3186</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22965637">22965637</a></td><td align="right">2013</td><td>ILGKLLSTAWALLSkL</td><td>A10W,N15k</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">16</td><td>CPP</td><td>LC<sub>50</sub> =27μM</td><td>Human</td><td>CTTHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3186&amp;di=1" width="5px" height="5px"><img src="pymol_images/3186.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3186&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3186.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3188">3188</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolysis at 48μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3188&amp;di=1" width="5px" height="5px"><img src="pymol_images/3188.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3188&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3188.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3189">3189</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>94% hemolysis at 24μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3189&amp;di=1" width="5px" height="5px"><img src="pymol_images/3189.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3189&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3189.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3190">3190</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>92% hemolysis at 12μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3190&amp;di=1" width="5px" height="5px"><img src="pymol_images/3190.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3190&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3190.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3191">3191</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>48% hemolysis at 6μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3191&amp;di=1" width="5px" height="5px"><img src="pymol_images/3191.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3191&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3191.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3192">3192</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>13% hemolysis at 3μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3192&amp;di=1" width="5px" height="5px"><img src="pymol_images/3192.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3192&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3192.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3193">3193</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVQWFSKFLGRIL</td><td>TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3% hemolysis at 1.5μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=3193&amp;di=1" width="5px" height="5px"><img src="pymol_images/3193.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3193&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3193.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3194">3194</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3194&amp;di=1" width="5px" height="5px"><img src="pymol_images/3194.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3194&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3194.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3195">3195</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>92% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3195&amp;di=1" width="5px" height="5px"><img src="pymol_images/3195.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3195&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3195.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3196">3196</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>42% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3196&amp;di=1" width="5px" height="5px"><img src="pymol_images/3196.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3196&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3196.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3197">3197</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>10% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3197&amp;di=1" width="5px" height="5px"><img src="pymol_images/3197.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3197&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3197.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3198">3198</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>6% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3198&amp;di=1" width="5px" height="5px"><img src="pymol_images/3198.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3198&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3198.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3199">3199</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRIL</td><td>[Pro<sup>3</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>4.5% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3199&amp;di=1" width="5px" height="5px"><img src="pymol_images/3199.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3199&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3199.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3200">3200</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSkFLGRIL</td><td>[Pro<sup>3</sup>, DLys<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>18% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3200&amp;di=1" width="5px" height="5px"><img src="pymol_images/3200.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3200&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3200.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3201">3201</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSkFLGRIL</td><td>[Pro<sup>3</sup>, DLys<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>8% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3201&amp;di=1" width="5px" height="5px"><img src="pymol_images/3201.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3201&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3201.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3202">3202</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSkFLGRIL</td><td>[Pro<sup>3</sup>, DLys<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>5% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3202&amp;di=1" width="5px" height="5px"><img src="pymol_images/3202.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3202&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3202.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3203">3203</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSkFLGRIL</td><td>[Pro<sup>3</sup>, DLys<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>7% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3203&amp;di=1" width="5px" height="5px"><img src="pymol_images/3203.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3203&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3203.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3204">3204</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSkFLGRIL</td><td>[Pro<sup>3</sup>, DLys<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>4% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3204&amp;di=1" width="5px" height="5px"><img src="pymol_images/3204.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3204&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3204.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3205">3205</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSkFLGRIL</td><td>[Pro<sup>3</sup>, DLys<sup>7</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3205&amp;di=1" width="5px" height="5px"><img src="pymol_images/3205.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3205&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3205.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3206">3206</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKfLGRIL</td><td>[Pro<sup>3</sup>, DPhe<sup>8</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>94% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3206&amp;di=1" width="5px" height="5px"><img src="pymol_images/3206.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3206&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3206.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3207">3207</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKfLGRIL</td><td>[Pro<sup>3</sup>, DPhe<sup>8</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>19% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3207&amp;di=1" width="5px" height="5px"><img src="pymol_images/3207.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3207&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3207.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3208">3208</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKfLGRIL</td><td>[Pro<sup>3</sup>, DPhe<sup>8</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>19% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3208&amp;di=1" width="5px" height="5px"><img src="pymol_images/3208.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3208&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3208.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3209">3209</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKfLGRIL</td><td>[Pro<sup>3</sup>, DPhe<sup>8</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>17% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3209&amp;di=1" width="5px" height="5px"><img src="pymol_images/3209.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3209&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3209.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3210">3210</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKfLGRIL</td><td>[Pro<sup>3</sup>, DPhe<sup>8</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>5% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3210&amp;di=1" width="5px" height="5px"><img src="pymol_images/3210.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3210&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3210.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3211">3211</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKfLGRIL</td><td>[Pro<sup>3</sup>, DPhe<sup>8</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>4% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3211&amp;di=1" width="5px" height="5px"><img src="pymol_images/3211.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3211&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3211.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3212">3212</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIL</td><td>[Pro<sup>3</sup>, DLeu<sup>9</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>9% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3212&amp;di=1" width="5px" height="5px"><img src="pymol_images/3212.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3212&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3212.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3213">3213</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIL</td><td>[Pro<sup>3</sup>, DLeu<sup>9</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>8% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3213&amp;di=1" width="5px" height="5px"><img src="pymol_images/3213.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3213&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3213.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3214">3214</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIL</td><td>[Pro<sup>3</sup>, DLeu<sup>9</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>7% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3214&amp;di=1" width="5px" height="5px"><img src="pymol_images/3214.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3214&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3214.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3215">3215</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIL</td><td>[Pro<sup>3</sup>, DLeu<sup>9</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3215&amp;di=1" width="5px" height="5px"><img src="pymol_images/3215.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3215&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3215.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3216">3216</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIL</td><td>[Pro<sup>3</sup>, DLeu<sup>9</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3216&amp;di=1" width="5px" height="5px"><img src="pymol_images/3216.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3216&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3216.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3217">3217</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIL</td><td>[Pro<sup>3</sup>, DLeu<sup>9</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3217&amp;di=1" width="5px" height="5px"><img src="pymol_images/3217.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3217&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3217.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3218">3218</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGrIL</td><td>[Pro<sup>3</sup>, DArg<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>22% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3218&amp;di=1" width="5px" height="5px"><img src="pymol_images/3218.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3218&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3218.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3219">3219</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGrIL</td><td>[Pro<sup>3</sup>, DArg<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>16% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3219&amp;di=1" width="5px" height="5px"><img src="pymol_images/3219.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3219&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3219.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3220">3220</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGrIL</td><td>[Pro<sup>3</sup>, DArg<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>8% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3220&amp;di=1" width="5px" height="5px"><img src="pymol_images/3220.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3220&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3220.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3221">3221</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGrIL</td><td>[Pro<sup>3</sup>, DArg<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>6% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3221&amp;di=1" width="5px" height="5px"><img src="pymol_images/3221.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3221&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3221.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3222">3222</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGrIL</td><td>[Pro<sup>3</sup>, DArg<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>4% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3222&amp;di=1" width="5px" height="5px"><img src="pymol_images/3222.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3222&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3222.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3223">3223</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGrIL</td><td>[Pro<sup>3</sup>, DArg<sup>11</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3223&amp;di=1" width="5px" height="5px"><img src="pymol_images/3223.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3223&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3223.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3224">3224</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRiL</td><td>[Pro<sup>3</sup>, DIle<sup>12</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>17% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3224&amp;di=1" width="5px" height="5px"><img src="pymol_images/3224.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3224&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3224.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3225">3225</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRiL</td><td>[Pro<sup>3</sup>, DIle<sup>12</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>14% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3225&amp;di=1" width="5px" height="5px"><img src="pymol_images/3225.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3225&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3225.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3226">3226</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRiL</td><td>[Pro<sup>3</sup>, DIle<sup>12</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>10% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3226&amp;di=1" width="5px" height="5px"><img src="pymol_images/3226.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3226&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3226.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3227">3227</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRiL</td><td>[Pro<sup>3</sup>, DIle<sup>12</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>5% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3227&amp;di=1" width="5px" height="5px"><img src="pymol_images/3227.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3227&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3227.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3228">3228</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRiL</td><td>[Pro<sup>3</sup>, DIle<sup>12</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>4% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3228&amp;di=1" width="5px" height="5px"><img src="pymol_images/3228.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3228&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3228.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3229">3229</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFLGRiL</td><td>[Pro<sup>3</sup>, DIle<sup>12</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>2% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3229&amp;di=1" width="5px" height="5px"><img src="pymol_images/3229.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3229&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3229.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3230">3230</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIl</td><td>[Pro<sup>3</sup>, DLeu<sup>13</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>15% hemolysis at 48μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3230&amp;di=1" width="5px" height="5px"><img src="pymol_images/3230.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3230&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3230.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3231">3231</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIl</td><td>[Pro<sup>3</sup>, DLeu<sup>13</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>17% hemolysis at 24μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3231&amp;di=1" width="5px" height="5px"><img src="pymol_images/3231.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3231&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3231.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3232">3232</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIl</td><td>[Pro<sup>3</sup>, DLeu<sup>13</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>17% hemolysis at 12μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3232&amp;di=1" width="5px" height="5px"><img src="pymol_images/3232.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3232&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3232.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3233">3233</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIl</td><td>[Pro<sup>3</sup>, DLeu<sup>13</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>8% hemolysis at 6μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3233&amp;di=1" width="5px" height="5px"><img src="pymol_images/3233.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3233&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3233.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3234">3234</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIl</td><td>[Pro<sup>3</sup>, DLeu<sup>13</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>6% hemolysis at 3μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3234&amp;di=1" width="5px" height="5px"><img src="pymol_images/3234.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3234&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3234.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3235">3235</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22974815">22974815</a></td><td align="right">2013</td><td>FVPWFSKFlGRIl</td><td>[Pro<sup>3</sup>, DLeu<sup>13</sup>]TL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3% hemolysis at 1.5μM</td><td>Human</td><td>CCCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3235&amp;di=1" width="5px" height="5px"><img src="pymol_images/3235.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3235&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3235.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3237">3237</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23026014">23026014</a></td><td align="right">2013</td><td>WFRKQLKW</td><td>L10</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antibacterial and Antifungal</td><td>No hemolysis upto 800μg/ml</td><td>Human</td><td>CTTSCCCC</td><td align="center"><a href="jmol.php?id=3237&amp;di=1" width="5px" height="5px"><img src="pymol_images/3237.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3237&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3237.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3246">3246</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23262358">23262358</a></td><td align="right">2013</td><td>DLLGTLGNLPLPFI</td><td>frenatin 2D</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Immunomodulatory</td><td>No hemolysis upto 500μM</td><td>Human</td><td>CCSSSTTSCCCSCC</td><td align="center"><a href="jmol.php?id=3246&amp;di=1" width="5px" height="5px"><img src="pymol_images/3246.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3247">3247</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23270672">23270672</a></td><td align="right">2013</td><td>RRLMAAKAESRK</td><td>CL(14-25)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>No or little hemolysis upto 800μM</td><td>Sheep</td><td>CTTTTHHHHHCC</td><td align="center"><a href="jmol.php?id=3247&amp;di=1" width="5px" height="5px"><img src="pymol_images/3247.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3247&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3247.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3248">3248</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23270672">23270672</a></td><td align="right">2013</td><td>RLMAAKAESRK</td><td>CL(15-25)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>No or little hemolysis upto 800μM</td><td>Sheep</td><td>CCCSCSSCTTC</td><td align="center"><a href="jmol.php?id=3248&amp;di=1" width="5px" height="5px"><img src="pymol_images/3248.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3248&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3248.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3249">3249</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23270672">23270672</a></td><td align="right">2013</td><td>LMAAKAESRK</td><td>CL(16-25)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>No or little hemolysis upto 800μM</td><td>Sheep</td><td>CCCBTTTBTC</td><td align="center"><a href="jmol.php?id=3249&amp;di=1" width="5px" height="5px"><img src="pymol_images/3249.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3249&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3249.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3250">3250</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23270672">23270672</a></td><td align="right">2013</td><td>RRLMAAKAESR</td><td>CL(14-24)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>No or little hemolysis upto 800μM</td><td>Sheep</td><td>CCCCBTTTBCC</td><td align="center"><a href="jmol.php?id=3250&amp;di=1" width="5px" height="5px"><img src="pymol_images/3250.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3250&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3250.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3251">3251</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23270672">23270672</a></td><td align="right">2013</td><td>RRLMAAKAES</td><td>CL(14-23)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>No or little hemolysis upto 800μM</td><td>Sheep</td><td>CCSSTTTTCC</td><td align="center"><a href="jmol.php?id=3251&amp;di=1" width="5px" height="5px"><img src="pymol_images/3251.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3251&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3251.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3252">3252</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23328867">23328867</a></td><td align="right">2013</td><td>THRPPMWSPVWPGGGKLlLKlLkkLLKlLKKK</td><td>TfR-lytic hybrid peptide</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">32</td><td>Anticancer</td><td>17% hemolysis at 100μM</td><td>Mouse</td><td>CCBTTBSSSSSCCCTHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3252&amp;di=1" width="5px" height="5px"><img src="pymol_images/3252.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3252&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3252.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3253">3253</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23403131">23403131</a></td><td align="right">2013</td><td>GILKTIKSIASKVANTVQKLKRKAKNAVA</td><td>Peptide</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>EC<sub>50</sub> =46μM</td><td>Human</td><td>CTTTTHHHHHHHHHHHTHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=3253&amp;di=1" width="5px" height="5px"><img src="pymol_images/3253.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3253&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3253.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3254">3254</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23403131">23403131</a></td><td align="right">2013</td><td>GILKTIKSIASKVANTVQKLKRKAKNAV</td><td>Δ(Α29)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>EC<sub>50</sub> =47μM</td><td>Human</td><td>CCTTTHHHHHHHHHHHHHHHHHTCSSCC</td><td align="center"><a href="jmol.php?id=3254&amp;di=1" width="5px" height="5px"><img src="pymol_images/3254.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3254&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3254.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3255">3255</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23403131">23403131</a></td><td align="right">2013</td><td>GILKTIKSIASKLKRKAK</td><td>Δ(K12-Q18;Ν26−Α29)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>EC<sub>50</sub> =80μM</td><td>Human</td><td>CCHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3255&amp;di=1" width="5px" height="5px"><img src="pymol_images/3255.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3255&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3255.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3256">3256</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23403131">23403131</a></td><td align="right">2013</td><td>GILNTIKSIASKLKRKAK</td><td>K4N Δ(K12-Q18; Ν26−Α29)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>EC<sub>50</sub> =139μM</td><td>Human</td><td>CCCCHHHHHHHHHGGGCC</td><td align="center"><a href="jmol.php?id=3256&amp;di=1" width="5px" height="5px"><img src="pymol_images/3256.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3256&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3256.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3257">3257</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23403136">23403136</a></td><td align="right">2013</td><td>KKLLPIVANLLKSLL</td><td>TB_KKG6A</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>Non-hemolytic</td><td>Mouse</td><td>CCSCGGGHHHHTTTC</td><td align="center"><a href="jmol.php?id=3257&amp;di=1" width="5px" height="5px"><img src="pymol_images/3257.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3257&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3257.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3258">3258</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23415044">23415044</a></td><td align="right">2013</td><td>FLGFLKNLF</td><td>WT</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antiviral</td><td>HC<sub>50</sub> =137.9μg/ml</td><td>Human</td><td>CCSSCCCCC</td><td align="center"><a href="jmol.php?id=3258&amp;di=1" width="5px" height="5px"><img src="pymol_images/3258.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3258&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3258.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3259">3259</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23415044">23415044</a></td><td align="right">2013</td><td>FLGFLHHLF</td><td>H2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antiviral</td><td>HC<sub>50</sub> =203.3μg/ml</td><td>Human</td><td>CCSCCCCCC</td><td align="center"><a href="jmol.php?id=3259&amp;di=1" width="5px" height="5px"><img src="pymol_images/3259.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3259&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3259.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3260">3260</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23415044">23415044</a></td><td align="right">2013</td><td>FLHFLHHLF</td><td>H3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antiviral</td><td>HC<sub>50</sub> =416.4μg/ml</td><td>Human</td><td>CCCCCCCCC</td><td align="center"><a href="jmol.php?id=3260&amp;di=1" width="5px" height="5px"><img src="pymol_images/3260.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3260&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3260.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3265">3265</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23624072">23624072</a></td><td align="right">2013</td><td>GILGKLWEGFKSIV</td><td>Pantinin-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>0% hemolysis up to 32μM</td><td>Human</td><td>CCSTTCCSSCCCCC</td><td align="center"><a href="jmol.php?id=3265&amp;di=1" width="5px" height="5px"><img src="pymol_images/3265.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3265&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3265.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3266">3266</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23624072">23624072</a></td><td align="right">2013</td><td>IFGAIWKGISSLL</td><td>Pantinin-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolysis up to 8μM</td><td>Human</td><td>CCSCSCTTTTTCC</td><td align="center"><a href="jmol.php?id=3266&amp;di=1" width="5px" height="5px"><img src="pymol_images/3266.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3266&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3266.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3267">3267</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23624072">23624072</a></td><td align="right">2013</td><td>FLSTIWNGIKSLL</td><td>Pantinin-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>0% hemolysis upto 8μM</td><td>Human</td><td>CCHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3267&amp;di=1" width="5px" height="5px"><img src="pymol_images/3267.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3267&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3267.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3268">3268</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23624072">23624072</a></td><td align="right">2013</td><td>GILGKLWEGFKSIV</td><td>Pantinin-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>21% hemolysis at 64μM</td><td>Human</td><td>CCSTTCCSSCCCCC</td><td align="center"><a href="jmol.php?id=3268&amp;di=1" width="5px" height="5px"><img src="pymol_images/3268.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3268&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3268.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3269">3269</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23624072">23624072</a></td><td align="right">2013</td><td>IFGAIWKGISSLL</td><td>Pantinin-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>8% hemolysis at 16μM</td><td>Human</td><td>CCSCSCTTTTTCC</td><td align="center"><a href="jmol.php?id=3269&amp;di=1" width="5px" height="5px"><img src="pymol_images/3269.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3269&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3269.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3270">3270</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23624072">23624072</a></td><td align="right">2013</td><td>FLSTIWNGIKSLL</td><td>Pantinin-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>70% hemolysis at 16μM</td><td>Human</td><td>CCHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3270&amp;di=1" width="5px" height="5px"><img src="pymol_images/3270.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3270&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3270.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3271">3271</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23624072">23624072</a></td><td align="right">2013</td><td>IFGAIWKGISSLL</td><td>Pantinin-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>81% hemolysis at 32μM</td><td>Human</td><td>CCSCSCTTTTTCC</td><td align="center"><a href="jmol.php?id=3271&amp;di=1" width="5px" height="5px"><img src="pymol_images/3271.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3271&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3271.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3272">3272</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23624072">23624072</a></td><td align="right">2013</td><td>FLSTIWNGIKSLL</td><td>Pantinin-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolysis at 32μM</td><td>Human</td><td>CCHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3272&amp;di=1" width="5px" height="5px"><img src="pymol_images/3272.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3272&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3272.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3273">3273</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23624072">23624072</a></td><td align="right">2013</td><td>IFGAIWKGISSLL</td><td>Pantinin-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolysis at 64μM</td><td>Human</td><td>CCSCSCTTTTTCC</td><td align="center"><a href="jmol.php?id=3273&amp;di=1" width="5px" height="5px"><img src="pymol_images/3273.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3273&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3273.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3274">3274</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23776609">23776609</a></td><td align="right">2013</td><td>RAGLQFPVGRLLR</td><td>BR1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>CPP</td><td>Non-hemolytic</td><td>Human</td><td>CCSCCCCCSSCCC</td><td align="center"><a href="jmol.php?id=3274&amp;di=1" width="5px" height="5px"><img src="pymol_images/3274.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3274&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3274.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3275">3275</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23776609">23776609</a></td><td align="right">2013</td><td>RAGLQFPVGRLLRRLLR</td><td>BR2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>CPP</td><td>Non-hemolytic</td><td>Human</td><td>CCCCCCCTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3275&amp;di=1" width="5px" height="5px"><img src="pymol_images/3275.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3275&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3275.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3276">3276</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23776609">23776609</a></td><td align="right">2013</td><td>RAGLQFPVGRLLRRLLRRLLR</td><td>BR3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>CPP</td><td>5% hemolysis at 25μM</td><td>Human</td><td>CCSSCCCHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3276&amp;di=1" width="5px" height="5px"><img src="pymol_images/3276.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3276&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3276.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3277">3277</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23776609">23776609</a></td><td align="right">2013</td><td>RAGLQFPVGRLLRRLLRRLLR</td><td>BR3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>CPP</td><td>34.3% hemolysis at 200μM</td><td>Human</td><td>CCSSCCCHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3277&amp;di=1" width="5px" height="5px"><img src="pymol_images/3277.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3277&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3277.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3278">3278</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23776609">23776609</a></td><td align="right">2013</td><td>RKKRRQRRR</td><td>Tat (49–57)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>CPP</td><td>Non-hemolytic</td><td>Human</td><td>CGGGTSCCC</td><td align="center"><a href="jmol.php?id=3278&amp;di=1" width="5px" height="5px"><img src="pymol_images/3278.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3278&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3278.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3279">3279</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9675125">9675125</a></td><td align="right">1998</td><td>MASRAAGLAARLARLALRAL</td><td>ESF1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0.1% hemolysis at 50μg/ml</td><td>Human</td><td>CCSTTSTTTTHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3279&amp;di=1" width="5px" height="5px"><img src="pymol_images/3279.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3279&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3279.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3280">3280</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9675125">9675125</a></td><td align="right">1998</td><td>MASRAARLAARLARLALRAL</td><td>GR7</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0.4% hemolysis at 50μg/ml</td><td>Human</td><td>CCSHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3280&amp;di=1" width="5px" height="5px"><img src="pymol_images/3280.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3280&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3280.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3281">3281</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9675125">9675125</a></td><td align="right">1998</td><td>MAARAARLAARLARLALRAL</td><td>SA3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0.4% hemolysis at 50μg/ml</td><td>Human</td><td>CHHHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3281&amp;di=1" width="5px" height="5px"><img src="pymol_images/3281.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3281&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3281.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3282">3282</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9675125">9675125</a></td><td align="right">1998</td><td>MASRAAGLAARLARLALRAL</td><td>ESF1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0.5% hemolysis at 200μg/ml</td><td>Human</td><td>CCSTTSTTTTHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3282&amp;di=1" width="5px" height="5px"><img src="pymol_images/3282.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3282&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3282.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3283">3283</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9675125">9675125</a></td><td align="right">1998</td><td>MASRAARLAARLARLALRAL</td><td>GR7</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0.9% hemolysis at 200μg/ml</td><td>Human</td><td>CCSHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3283&amp;di=1" width="5px" height="5px"><img src="pymol_images/3283.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3283&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3283.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3284">3284</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9675125">9675125</a></td><td align="right">1998</td><td>MAARAARLAARLARLALRAL</td><td>SA3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0.8% hemolysis at 200μg/ml</td><td>Human</td><td>CHHHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3284&amp;di=1" width="5px" height="5px"><img src="pymol_images/3284.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3284&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3284.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3285">3285</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9748603">9748603</a></td><td align="right">1998</td><td>FLPLLAGLAANFLPKIFCKITRKC</td><td>Brevinin 1E</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>HD<sub>50</sub> =1.1μg/ml</td><td>Mouse</td><td>NA</td><td align="center"><a href="jmol.php?id=3285&amp;di=1" width="5px" height="5px"><img src="pymol_images/3285.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3286">3286</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9748603">9748603</a></td><td align="right">1998</td><td>FLPLLAGLAANFLPKIFCKITRKC</td><td>Brevinin 1E (red)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>HD<sub>50</sub> =1.6μg/ml</td><td>Mouse</td><td>NA</td><td align="center"><a href="jmol.php?id=3286&amp;di=1" width="5px" height="5px"><img src="pymol_images/3286.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3287">3287</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9748603">9748603</a></td><td align="right">1998</td><td>LLAGLAANFLPKIFCKITRKC</td><td>Brevinin 21 (oxi)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>HD<sub>50</sub> =77μg/ml</td><td>Mouse</td><td>NA</td><td align="center"><a href="jmol.php?id=3287&amp;di=1" width="5px" height="5px"><img src="pymol_images/3287.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3288">3288</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9748603">9748603</a></td><td align="right">1998</td><td>LLAGLAANFLPKIFCKITRKC</td><td>Brevinin 21 (red)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>HD<sub>50</sub> =110μg/ml</td><td>Mouse</td><td>NA</td><td align="center"><a href="jmol.php?id=3288&amp;di=1" width="5px" height="5px"><img src="pymol_images/3288.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3289">3289</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9748603">9748603</a></td><td align="right">1998</td><td>GLAANFLPKIFCKITRKC</td><td>Brevinin 18 (oxi)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HD<sub>50</sub> =146μg/ml</td><td>Mouse</td><td>CCCCCCCCHHHHHTCCCC</td><td align="center"><a href="jmol.php?id=3289&amp;di=1" width="5px" height="5px"><img src="pymol_images/3289.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3289&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3289.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3290">3290</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9748603">9748603</a></td><td align="right">1998</td><td>GLAANFLPKIFCKITRKC</td><td>Brevinin 18 (red)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HD<sub>50</sub> =230μg/ml</td><td>Mouse</td><td>CCCCCCCCHHHHHTCCCC</td><td align="center"><a href="jmol.php?id=3290&amp;di=1" width="5px" height="5px"><img src="pymol_images/3290.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3290&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3290.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3291">3291</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/6325405">6325405</a></td><td align="right">1984</td><td>KFTIVFPHNQKGNWKNVPSNYHYCP</td><td>VSV G</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Hemolysin</td><td>400μg of peptide can totally lyse 7x10<sup>8</sup> erythrocytes in 10 min at pH 5.0</td><td>Sheep</td><td>CCSSCCSSTTSCCSCSSSSCCCCCC</td><td align="center"><a href="jmol.php?id=3291&amp;di=1" width="5px" height="5px"><img src="pymol_images/3291.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3291&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3291.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3292">3292</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/6325405">6325405</a></td><td align="right">1984</td><td>CKTRTSTNTDHLWIAKSKARLSET</td><td>Synthetic peptide-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Not mentioned</td><td>0% hemolysis upto 500μg/ml (non hemolytic)</td><td>Sheep</td><td>CCSSSCCSSCCTTTSCCTTSCSCC</td><td align="center"><a href="jmol.php?id=3292&amp;di=1" width="5px" height="5px"><img src="pymol_images/3292.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3292&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3292.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3293">3293</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/6325405">6325405</a></td><td align="right">1984</td><td>PKDLKETLQEKKP</td><td>Synthetic peptide-2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Not mentioned</td><td>0% hemolysis upto 500μg/ml (non hemolytic)</td><td>Sheep</td><td>CCCHHHHTTTTCC</td><td align="center"><a href="jmol.php?id=3293&amp;di=1" width="5px" height="5px"><img src="pymol_images/3293.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3293&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3293.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3294">3294</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8027544">8027544</a></td><td align="right">1994</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>100% hemolysis at 120μM</td><td>Mouse</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3294&amp;di=1" width="5px" height="5px"><img src="pymol_images/3294.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3294&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3294.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3295">3295</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8027544">8027544</a></td><td align="right">1994</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Ac-Melittin</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>100% hemolysis at 120μM compared with melittin</td><td>Mouse</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3295&amp;di=1" width="5px" height="5px"><img src="pymol_images/3295.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3295&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3295.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3296">3296</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8027544">8027544</a></td><td align="right">1994</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin-Lac</td><td>Lac=p-amidophenyl-β-D-lactopyranoside</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>120% hemolysis at 120μM compared with melittin</td><td>Mouse</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3296&amp;di=1" width="5px" height="5px"><img src="pymol_images/3296.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3296&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3296.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3297">3297</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8027544">8027544</a></td><td align="right">1994</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>LacSuc-Melittin</td><td>Amidation</td><td>Lac=p-amidophenyl-β-D-lactopyranoside and Suc=Succinyl</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>43% hemolysis at 120μM compared with melittin</td><td>Mouse</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3297&amp;di=1" width="5px" height="5px"><img src="pymol_images/3297.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3297&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3297.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3298">3298</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8027544">8027544</a></td><td align="right">1994</td><td>KRKRQQGIGAVLKVLTTGLPALISWI</td><td>Melittin-(21-26) (1-20)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>20% hemolysis at 120μM compared with melittin</td><td>Mouse</td><td>CCCTTTTTHHHHHHHHHHSGGGTTCC</td><td align="center"><a href="jmol.php?id=3298&amp;di=1" width="5px" height="5px"><img src="pymol_images/3298.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3299">3299</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8027544">8027544</a></td><td align="right">1994</td><td>KRKRQQGIGAVLKVLTTGLPALISWI</td><td>Ac-Melttin-(21-26) (1-20)</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>20% hemolysis at 120μM compared with melittin</td><td>Mouse</td><td>CCCTTTTTHHHHHHHHHHSGGGTTCC</td><td align="center"><a href="jmol.php?id=3299&amp;di=1" width="5px" height="5px"><img src="pymol_images/3299.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3300">3300</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8027544">8027544</a></td><td align="right">1994</td><td>QQRKRKGIGAVLKVLTTGLPALISWI</td><td>Melttin-(26-21) (1-20)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>15% hemolysis at 120μM compared with melittin</td><td>Mouse</td><td>CCSSCSCSHHHHHHHHHHSCTTTTCC</td><td align="center"><a href="jmol.php?id=3300&amp;di=1" width="5px" height="5px"><img src="pymol_images/3300.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3301">3301</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8027544">8027544</a></td><td align="right">1994</td><td>QQRKRKGIGAVLKVLTTGLPALISWI</td><td>Ac-Melittin-(26-21) (1-20)</td><td>Free</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>15% hemolysis at 120μM compared with melittin</td><td>Mouse</td><td>CCSSCSCSHHHHHHHHHHSCTTTTCC</td><td align="center"><a href="jmol.php?id=3301&amp;di=1" width="5px" height="5px"><img src="pymol_images/3301.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3302">3302</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8163497">8163497</a></td><td align="right">1994</td><td>FLPLLAGLAANFLPKIFCKITRKC</td><td>Brevinin-1E</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>Lethal concentration =0.5μM</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHHCSCC</td><td align="center"><a href="jmol.php?id=3302&amp;di=1" width="5px" height="5px"><img src="pymol_images/3302.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3303">3303</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8163497">8163497</a></td><td align="right">1994</td><td>GIMDTLKNLAKTAGKGALQSLLNKASCKLSGQC</td><td>Brevinin-2E</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>Lethal concentration &gt;100μM</td><td>Human</td><td>CCTTHHHHHHHTTCCHHHHHHHTTSCSTTSSCC</td><td align="center"><a href="jmol.php?id=3303&amp;di=1" width="5px" height="5px"><img src="pymol_images/3303.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3303&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3303.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3305">3305</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1362637">1362637</a></td><td align="right">1992</td><td>PKLLETFLSKWIG</td><td>SPF</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>10% hemolysis at 9μM</td><td>Rat</td><td>CCHHHHHTTTTCC</td><td align="center"><a href="jmol.php?id=3305&amp;di=1" width="5px" height="5px"><img src="pymol_images/3305.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3305&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3305.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3306">3306</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1362637">1362637</a></td><td align="right">1992</td><td>PKLLKTFLSKWIG</td><td>SPFK</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>10% hemolysis at 9μM</td><td>Rat</td><td>CCGGGTTSSTTCC</td><td align="center"><a href="jmol.php?id=3306&amp;di=1" width="5px" height="5px"><img src="pymol_images/3306.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3306&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3306.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3307">3307</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2253768">2253768</a></td><td align="right">1990</td><td>gigkflhsakkfgkafvgeimns</td><td>All-D-Magainin-2</td><td>Free</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">23</td><td>Antibacterial and Antifungal</td><td>0% hemolysis upto 80μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=3307&amp;di=1" width="5px" height="5px"><img src="pymol_images/3307.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3308">3308</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2253768">2253768</a></td><td align="right">1990</td><td>gigkflhsakkfgkafvgeimns</td><td>All-D-Magainin-2</td><td>Free</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">23</td><td>Antibacterial and Antifungal</td><td>1% hemolysis at 100μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=3308&amp;di=1" width="5px" height="5px"><img src="pymol_images/3308.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3309">3309</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2253768">2253768</a></td><td align="right">1990</td><td>gigkflhsakkfgkafvgeimns</td><td>All-D-Magainin-2</td><td>Free</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">23</td><td>Antibacterial and Antifungal</td><td>2% hemolysis at 200μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=3309&amp;di=1" width="5px" height="5px"><img src="pymol_images/3309.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3310">3310</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2253768">2253768</a></td><td align="right">1990</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>All-L-Magainin-2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial and Antifungal</td><td>0% hemolysis at 25μg/ml (non hemolytic)</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=3310&amp;di=1" width="5px" height="5px"><img src="pymol_images/3310.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3311">3311</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2253768">2253768</a></td><td align="right">1990</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>All-L-Magainin-2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial and Antifungal</td><td>1% hemolysis at 80μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=3311&amp;di=1" width="5px" height="5px"><img src="pymol_images/3311.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3312">3312</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2253768">2253768</a></td><td align="right">1990</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>All-L-Magainin-2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial and Antifungal</td><td>2% hemolysis at 200μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=3312&amp;di=1" width="5px" height="5px"><img src="pymol_images/3312.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3313">3313</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-A</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>82% hemolysis at 25μg</td><td>Human</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3313&amp;di=1" width="5px" height="5px"><img src="pymol_images/3313.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3313&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3313.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3314">3314</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-A</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>100% hemolysis at 75μg</td><td>Human</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3314&amp;di=1" width="5px" height="5px"><img src="pymol_images/3314.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3314&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3314.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3315">3315</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-A</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>73% hemolysis at 25μg</td><td>Horse</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3315&amp;di=1" width="5px" height="5px"><img src="pymol_images/3315.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3315&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3315.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3316">3316</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-A</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>85% hemolysis at 75μg</td><td>Horse</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3316&amp;di=1" width="5px" height="5px"><img src="pymol_images/3316.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3316&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3316.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3317">3317</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-A</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>92% hemolysis at 25μg</td><td>Guinea-pig</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3317&amp;di=1" width="5px" height="5px"><img src="pymol_images/3317.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3317&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3317.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3318">3318</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-A</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>100% hemolysis at 75μg</td><td>Guinea-pig</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3318&amp;di=1" width="5px" height="5px"><img src="pymol_images/3318.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3318&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3318.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3319">3319</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-A</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>87% hemolysis at 25μg</td><td>Rabbit</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3319&amp;di=1" width="5px" height="5px"><img src="pymol_images/3319.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3319&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3319.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3320">3320</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-A</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>100% hemolysis at 75μg</td><td>Rabbit</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3320&amp;di=1" width="5px" height="5px"><img src="pymol_images/3320.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3320&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3320.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3321">3321</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-A</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>38% hemolysis at 25μg</td><td>Sheep</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3321&amp;di=1" width="5px" height="5px"><img src="pymol_images/3321.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3321&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3321.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3322">3322</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-A</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>80% hemolysis at 75μg</td><td>Sheep</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3322&amp;di=1" width="5px" height="5px"><img src="pymol_images/3322.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3322&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3322.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3323">3323</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-B</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>47% hemolysis at 25μg</td><td>Human</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3323&amp;di=1" width="5px" height="5px"><img src="pymol_images/3323.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3323&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3323.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3324">3324</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-B</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>55% hemolysis at 75μg</td><td>Human</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3324&amp;di=1" width="5px" height="5px"><img src="pymol_images/3324.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3324&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3324.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3325">3325</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-B</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>34% hemolysis at 25μg</td><td>Horse</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3325&amp;di=1" width="5px" height="5px"><img src="pymol_images/3325.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3325&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3325.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3326">3326</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-B</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>43% hemolysis at 75μg</td><td>Horse</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3326&amp;di=1" width="5px" height="5px"><img src="pymol_images/3326.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3326&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3326.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3327">3327</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-B</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>32% hemolysis at 25μg</td><td>Guinea-pig</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3327&amp;di=1" width="5px" height="5px"><img src="pymol_images/3327.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3327&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3327.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3328">3328</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-B</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>55% hemolysis at 75μg</td><td>Guinea-pig</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3328&amp;di=1" width="5px" height="5px"><img src="pymol_images/3328.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3328&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3328.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3329">3329</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-B</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>20% hemolysis at 25μg</td><td>Rabbit</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3329&amp;di=1" width="5px" height="5px"><img src="pymol_images/3329.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3329&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3329.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3330">3330</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-B</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>22% hemolysis at 75μg</td><td>Rabbit</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3330&amp;di=1" width="5px" height="5px"><img src="pymol_images/3330.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3330&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3330.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3331">3331</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-B</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>2% hemolysis at 75μg</td><td>Sheep</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3331&amp;di=1" width="5px" height="5px"><img src="pymol_images/3331.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3331&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3331.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3332">3332</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-C</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>60% hemolysis at 75μg</td><td>Human</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3332&amp;di=1" width="5px" height="5px"><img src="pymol_images/3332.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3332&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3332.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3333">3333</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-C</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>50% hemolysis at 75μg</td><td>Horse</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3333&amp;di=1" width="5px" height="5px"><img src="pymol_images/3333.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3333&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3333.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3334">3334</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-C</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>58% hemolysis at 75μg</td><td>Guinea-pig</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3334&amp;di=1" width="5px" height="5px"><img src="pymol_images/3334.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3334&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3334.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3335">3335</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-C</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>25% hemolysis at 75μg</td><td>Rabbit</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3335&amp;di=1" width="5px" height="5px"><img src="pymol_images/3335.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3335&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3335.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3336">3336</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGDLVKWIIDTVNKFPKK</td><td>Peptide-C</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>8% hemolysis at 75μg</td><td>Sheep</td><td>CTTGGGTHHHHHHHTTTTTSCCSCCC</td><td align="center"><a href="jmol.php?id=3336&amp;di=1" width="5px" height="5px"><img src="pymol_images/3336.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3336&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3336.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3337">3337</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>0% hemolysis at 75μg</td><td>Human</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3337&amp;di=1" width="5px" height="5px"><img src="pymol_images/3337.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3337&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3337.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3338">3338</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>12% hemolysis at 250μg</td><td>Human</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3338&amp;di=1" width="5px" height="5px"><img src="pymol_images/3338.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3338&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3338.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3339">3339</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>18% hemolysis at 500μg</td><td>Human</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3339&amp;di=1" width="5px" height="5px"><img src="pymol_images/3339.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3339&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3339.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3340">3340</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>0% hemolysis at 75μg</td><td>Horse</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3340&amp;di=1" width="5px" height="5px"><img src="pymol_images/3340.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3340&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3340.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3341">3341</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>20% hemolysis at 250μg</td><td>Horse</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3341&amp;di=1" width="5px" height="5px"><img src="pymol_images/3341.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3341&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3341.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3342">3342</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>19% hemolysis at 500μg</td><td>Horse</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3342&amp;di=1" width="5px" height="5px"><img src="pymol_images/3342.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3342&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3342.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3343">3343</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>0% hemolysis at 75μg</td><td>Guinea-pig</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3343&amp;di=1" width="5px" height="5px"><img src="pymol_images/3343.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3343&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3343.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3344">3344</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>23% hemolysis at 250μg</td><td>Guinea-pig</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3344&amp;di=1" width="5px" height="5px"><img src="pymol_images/3344.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3344&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3344.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3345">3345</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>51% hemolysis at 500μg</td><td>Guinea-pig</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3345&amp;di=1" width="5px" height="5px"><img src="pymol_images/3345.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3345&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3345.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3346">3346</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>0% hemolysis at 250μg</td><td>Rabbit</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3346&amp;di=1" width="5px" height="5px"><img src="pymol_images/3346.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3346&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3346.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3347">3347</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>14% hemolysis at 500μg</td><td>Rabbit</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3347&amp;di=1" width="5px" height="5px"><img src="pymol_images/3347.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3347&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3347.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3348">3348</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>0% hemolysis at 250μg</td><td>Sheep</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3348&amp;di=1" width="5px" height="5px"><img src="pymol_images/3348.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3348&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3348.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3349">3349</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>DLVKWIIDTVNKFTKK</td><td>Peptide-D</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Cytolytic</td><td>17% hemolysis at 500μg</td><td>Sheep</td><td>CCSSSTTTTHHHHTCC</td><td align="center"><a href="jmol.php?id=3349&amp;di=1" width="5px" height="5px"><img src="pymol_images/3349.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3349&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3349.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3350">3350</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGD</td><td>Peptide-E</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Cytolytic</td><td>0% hemolysis at 1000μg (non hemolytic)</td><td>Human</td><td>CCHHHHTTCCC</td><td align="center"><a href="jmol.php?id=3350&amp;di=1" width="5px" height="5px"><img src="pymol_images/3350.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3350&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3350.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3351">3351</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGD</td><td>Peptide-E</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Cytolytic</td><td>0% hemolysis at 1000μg (non hemolytic)</td><td>Horse</td><td>CCHHHHTTCCC</td><td align="center"><a href="jmol.php?id=3351&amp;di=1" width="5px" height="5px"><img src="pymol_images/3351.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3351&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3351.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3352">3352</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGD</td><td>Peptide-E</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Cytolytic</td><td>0% hemolysis at 1000μg (non hemolytic)</td><td>Guinea-pig</td><td>CCHHHHTTCCC</td><td align="center"><a href="jmol.php?id=3352&amp;di=1" width="5px" height="5px"><img src="pymol_images/3352.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3352&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3352.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3353">3353</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGD</td><td>Peptide-E</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Cytolytic</td><td>0% hemolysis at 1000μg (non hemolytic)</td><td>Rabbit</td><td>CCHHHHTTCCC</td><td align="center"><a href="jmol.php?id=3353&amp;di=1" width="5px" height="5px"><img src="pymol_images/3353.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3353&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3353.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3354">3354</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MAQDIISTIGD</td><td>Peptide-E</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Cytolytic</td><td>0% hemolysis at 1000μg (non hemolytic)</td><td>Sheep</td><td>CCHHHHTTCCC</td><td align="center"><a href="jmol.php?id=3354&amp;di=1" width="5px" height="5px"><img src="pymol_images/3354.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3354&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3354.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3355">3355</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDLLSSLGDLLKSWLDTLNKFTKK</td><td>Peptide-F</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>4% hemolysis at 75μg</td><td>Human</td><td>CTTTHHHHTTTHHHHHHHTTGGGGTC</td><td align="center"><a href="jmol.php?id=3355&amp;di=1" width="5px" height="5px"><img src="pymol_images/3355.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3355&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3355.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3356">3356</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDLLSSLGDLLKSWLDTLNKFTKK</td><td>Peptide-F</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>12% hemolysis at 75μg</td><td>Horse</td><td>CTTTHHHHTTTHHHHHHHTTGGGGTC</td><td align="center"><a href="jmol.php?id=3356&amp;di=1" width="5px" height="5px"><img src="pymol_images/3356.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3356&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3356.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3357">3357</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDLLSSLGDLLKSWLDTLNKFTKK</td><td>Peptide-F</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>36% hemolysis at 75μg</td><td>Guinea-pig</td><td>CTTTHHHHTTTHHHHHHHTTGGGGTC</td><td align="center"><a href="jmol.php?id=3357&amp;di=1" width="5px" height="5px"><img src="pymol_images/3357.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3357&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3357.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3358">3358</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDLLSSLGDLLKSWLDTLNKFTKK</td><td>Peptide-F</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>0% hemolysis at 75μg (non hemolytic)</td><td>Rabbit</td><td>CTTTHHHHTTTHHHHHHHTTGGGGTC</td><td align="center"><a href="jmol.php?id=3358&amp;di=1" width="5px" height="5px"><img src="pymol_images/3358.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3358&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3358.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3359">3359</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDLLSSLGDLLKSWLDTLNKFTKK</td><td>Peptide-F</td><td>Free</td><td>Formylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>0% hemolysis at 75μg (non hemolytic)</td><td>Sheep</td><td>CTTTHHHHTTTHHHHHHHTTGGGGTC</td><td align="center"><a href="jmol.php?id=3359&amp;di=1" width="5px" height="5px"><img src="pymol_images/3359.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3359&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3359.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3360">3360</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDLLSSLGDLLKSWLDTLNKFTKK</td><td>Peptide-G</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>13% hemolysis at 75μg</td><td>Human</td><td>CTTTHHHHTTTHHHHHHHTTGGGGTC</td><td align="center"><a href="jmol.php?id=3360&amp;di=1" width="5px" height="5px"><img src="pymol_images/3360.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3360&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3360.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3361">3361</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDLLSSLGDLLKSWLDTLNKFTKK</td><td>Peptide-G</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>31% hemolysis at 75μg</td><td>Horse</td><td>CTTTHHHHTTTHHHHHHHTTGGGGTC</td><td align="center"><a href="jmol.php?id=3361&amp;di=1" width="5px" height="5px"><img src="pymol_images/3361.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3361&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3361.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3362">3362</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDLLSSLGDLLKSWLDTLNKFTKK</td><td>Peptide-G</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>34% hemolysis at 75μg</td><td>Guinea-pig</td><td>CTTTHHHHTTTHHHHHHHTTGGGGTC</td><td align="center"><a href="jmol.php?id=3362&amp;di=1" width="5px" height="5px"><img src="pymol_images/3362.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3362&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3362.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3363">3363</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDLLSSLGDLLKSWLDTLNKFTKK</td><td>Peptide-G</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>35% hemolysis at 75μg</td><td>Rabbit</td><td>CTTTHHHHTTTHHHHHHHTTGGGGTC</td><td align="center"><a href="jmol.php?id=3363&amp;di=1" width="5px" height="5px"><img src="pymol_images/3363.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3363&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3363.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3364">3364</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDLLSSLGDLLKSWLDTLNKFTKK</td><td>Peptide-G</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>13% hemolysis at 75μg</td><td>Sheep</td><td>CTTTHHHHTTTHHHHHHHTTGGGGTC</td><td align="center"><a href="jmol.php?id=3364&amp;di=1" width="5px" height="5px"><img src="pymol_images/3364.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3364&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3364.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3365">3365</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDWLSSLGDLLKSLLDTVNKFTKK</td><td>Peptide-H</td><td>Free</td><td>Fmoc</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>59% hemolysis at 75μg</td><td>Human</td><td>CTTTHHHHHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3365&amp;di=1" width="5px" height="5px"><img src="pymol_images/3365.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3365&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3365.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3366">3366</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDWLSSLGDLLKSLLDTVNKFTKK</td><td>Peptide-H</td><td>Free</td><td>Fmoc</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>25% hemolysis at 75μg</td><td>Horse</td><td>CTTTHHHHHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3366&amp;di=1" width="5px" height="5px"><img src="pymol_images/3366.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3366&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3366.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3367">3367</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDWLSSLGDLLKSLLDTVNKFTKK</td><td>Peptide-H</td><td>Free</td><td>Fmoc</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>91% hemolysis at 75μg</td><td>Guinea-pig</td><td>CTTTHHHHHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3367&amp;di=1" width="5px" height="5px"><img src="pymol_images/3367.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3367&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3367.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3368">3368</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDWLSSLGDLLKSLLDTVNKFTKK</td><td>Peptide-H</td><td>Free</td><td>Fmoc</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>43% hemolysis at 75μg</td><td>Sheep</td><td>CTTTHHHHHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3368&amp;di=1" width="5px" height="5px"><img src="pymol_images/3368.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3368&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3368.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3369">3369</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDWLSSLGDLLKSLLDTVNKFTKK</td><td>Peptide-I</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>40% hemolysis at 75μg</td><td>Human</td><td>CTTTHHHHHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3369&amp;di=1" width="5px" height="5px"><img src="pymol_images/3369.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3369&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3369.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3370">3370</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDWLSSLGDLLKSLLDTVNKFTKK</td><td>Peptide-I</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>11% hemolysis at 75μg</td><td>Horse</td><td>CTTTHHHHHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3370&amp;di=1" width="5px" height="5px"><img src="pymol_images/3370.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3370&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3370.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3371">3371</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDWLSSLGDLLKSLLDTVNKFTKK</td><td>Peptide-I</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>66% hemolysis at 75μg</td><td>Guinea-pig</td><td>CTTTHHHHHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3371&amp;di=1" width="5px" height="5px"><img src="pymol_images/3371.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3371&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3371.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3372">3372</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2474443">2474443</a></td><td align="right">1989</td><td>MLQDWLSSLGDLLKSLLDTVNKFTKK</td><td>Peptide-I</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>24% hemolysis at 75μg</td><td>Sheep</td><td>CTTTHHHHHHHHHHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3372&amp;di=1" width="5px" height="5px"><img src="pymol_images/3372.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3372&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3372.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3373">3373</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9022710">9022710</a></td><td align="right">1996</td><td>FLPLIGRVLSGIL</td><td>Temporin A</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>Lethal concentration &gt;120μM</td><td>Human</td><td>CCCCHHHHTSCCC</td><td align="center"><a href="jmol.php?id=3373&amp;di=1" width="5px" height="5px"><img src="pymol_images/3373.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3373&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3373.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3374">3374</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9022710">9022710</a></td><td align="right">1996</td><td>LLPIVGNLLKSLL</td><td>Temporin B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>Lethal concentration &gt;120μM</td><td>Human</td><td>CCHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=3374&amp;di=1" width="5px" height="5px"><img src="pymol_images/3374.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3374&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3374.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3375">3375</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9022710">9022710</a></td><td align="right">1996</td><td>FIGSALKVLAGVLPSVISWVKQ</td><td>MLP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>Lethal concentration =0.5μM</td><td>Human</td><td>CCCCHHHHHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3375&amp;di=1" width="5px" height="5px"><img src="pymol_images/3375.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3378">3378</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9273892">9273892</a></td><td align="right">1997</td><td>LALILRKIVTAL</td><td>FCR(3A)</td><td>Amidation</td><td>Fmoc</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>100% hemolysis at 8μg</td><td>Rat</td><td>CHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3378&amp;di=1" width="5px" height="5px"><img src="pymol_images/3378.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3378&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3378.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3379">3379</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9273892">9273892</a></td><td align="right">1997</td><td>LALILRKIVTAL</td><td>CR(3A)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>~90% hemolysis at 80μg</td><td>Rat</td><td>CHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3379&amp;di=1" width="5px" height="5px"><img src="pymol_images/3379.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3379&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3379.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3380">3380</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9273892">9273892</a></td><td align="right">1997</td><td>LKLIPRKIVTAL</td><td>FCR(3K,6P)</td><td>Amidation</td><td>Fmoc</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>~60% hemolysis at 80μg</td><td>Rat</td><td>CCCCCTTGGGTC</td><td align="center"><a href="jmol.php?id=3380&amp;di=1" width="5px" height="5px"><img src="pymol_images/3380.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3380&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3380.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3381">3381</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9273892">9273892</a></td><td align="right">1997</td><td>LKLIPRKIVTAL</td><td>CR(3K,6P)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>Non-hemolytic</td><td>Rat</td><td>CCCCCTTGGGTC</td><td align="center"><a href="jmol.php?id=3381&amp;di=1" width="5px" height="5px"><img src="pymol_images/3381.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3381&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3381.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3382">3382</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9273892">9273892</a></td><td align="right">1997</td><td>LPLILRKIVTAL</td><td>FCR</td><td>Amidation</td><td>Fmoc</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>~80% hemolysis at 8μg</td><td>Rat</td><td>CCCHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3382&amp;di=1" width="5px" height="5px"><img src="pymol_images/3382.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3382&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3382.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3383">3383</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9273892">9273892</a></td><td align="right">1997</td><td>LPLILRKIVTAL</td><td>CR</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>Non-hemolytic</td><td>Rat</td><td>CCCHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3383&amp;di=1" width="5px" height="5px"><img src="pymol_images/3383.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3383&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3383.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3384">3384</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9442044">9442044</a></td><td align="right">1998</td><td>IWLTALKFLGKHAAKHLAKQQLSKL</td><td>Lycotoxin I</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial and Neuroactive</td><td>55% hemolysis at 200μM</td><td>Rabbit</td><td>CTTTHHHHTHHHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3384&amp;di=1" width="5px" height="5px"><img src="pymol_images/3384.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3385">3385</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9442044">9442044</a></td><td align="right">1998</td><td>GIGKFLHAAKKFAKAFVAEIMNS</td><td>Magainin-B</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>35% hemolysis at 200μM</td><td>Rabbit</td><td>CCSCHHHHHHTTTTTTTTTTTTC</td><td align="center"><a href="jmol.php?id=3385&amp;di=1" width="5px" height="5px"><img src="pymol_images/3385.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3385&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3385.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3391">3391</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9784609">9784609</a></td><td align="right">1998</td><td>RQKDKRPYSERKNQYTGPQFLYPPERIPP</td><td>Lumbricin I(6-34)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>0.03% hemolysis at 5μg/ml</td><td>Human</td><td>CCCSCCCCSGGGCCCCSSSSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3391&amp;di=1" width="5px" height="5px"><img src="pymol_images/3391.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3391&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3391.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3392">3392</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9784609">9784609</a></td><td align="right">1998</td><td>RQKDKRPYSERKNQYTGPQFLYPPERIPP</td><td>Lumbricin I(6-34)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>0.07% hemolysis at 10μg/ml</td><td>Human</td><td>CCCSCCCCSGGGCCCCSSSSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3392&amp;di=1" width="5px" height="5px"><img src="pymol_images/3392.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3392&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3392.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3393">3393</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9784609">9784609</a></td><td align="right">1998</td><td>RQKDKRPYSERKNQYTGPQFLYPPERIPP</td><td>Lumbricin I(6-34)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>0.13% hemolysis at 25μg/ml</td><td>Human</td><td>CCCSCCCCSGGGCCCCSSSSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3393&amp;di=1" width="5px" height="5px"><img src="pymol_images/3393.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3393&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3393.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3394">3394</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9784609">9784609</a></td><td align="right">1998</td><td>RQKDKRPYSERKNQYTGPQFLYPPERIPP</td><td>Lumbricin I(6-34)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>0.17% hemolysis at 50μg/ml</td><td>Human</td><td>CCCSCCCCSGGGCCCCSSSSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3394&amp;di=1" width="5px" height="5px"><img src="pymol_images/3394.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3394&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3394.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3395">3395</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9784609">9784609</a></td><td align="right">1998</td><td>RQKDKRPYSERKNQYTGPQFLYPPERIPP</td><td>Lumbricin I(6-34)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>0.21% hemolysis at 100μg/ml</td><td>Human</td><td>CCCSCCCCSGGGCCCCSSSSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3395&amp;di=1" width="5px" height="5px"><img src="pymol_images/3395.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3395&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3395.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3396">3396</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10430870">10430870</a></td><td align="right">1999</td><td>CTCSWPVCTRNGLPVCGETCVGGTCNTPG</td><td>Kalata B1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>50% hemolysis at &gt;400μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3397">3397</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10430870">10430870</a></td><td align="right">1999</td><td>CSCKNKVCYRNGIPCGESCVWIPCISAALG</td><td>Cir A</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>50% hemolysis at &gt;400μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3398">3398</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10430870">10430870</a></td><td align="right">1999</td><td>CSCKNKVCYRNGVIPCGESCVFIPCISTLLG</td><td>Cir B</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>50% hemolysis at &gt;400μM</td><td>Human</td><td>CCCCCCCSSSSSSSCSSCCSSCCCSSSCCCC</td><td align="center"><a href="jmol.php?id=3398&amp;di=1" width="5px" height="5px"><img src="pymol_images/3398.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3398&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3398.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3399">3399</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10430870">10430870</a></td><td align="right">1999</td><td>CSCKSKVCYKNSIPCGESCVFIPCTVTALLG</td><td>CPT</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>50% hemolysis at &gt;400μM</td><td>Human</td><td>CCSSSSSSSSSSSCSSSCCCCCSSSSHHHHC</td><td align="center"><a href="jmol.php?id=3399&amp;di=1" width="5px" height="5px"><img src="pymol_images/3399.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3399&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3399.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3404">3404</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10942757">10942757</a></td><td align="right">2000</td><td>ZCRRLCYKQRCVTYCRGR</td><td>Gomesin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>16% hemolysis at 1μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3405">3405</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10942757">10942757</a></td><td align="right">2000</td><td>ZCRRLCYKQRCVTYCRGR</td><td>Gomesin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>~23% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3406">3406</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10942757">10942757</a></td><td align="right">2000</td><td>ZCRRLCYKQRCVTYCRGR</td><td>Gomesin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>11% hemolysis at 1μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3407">3407</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10942757">10942757</a></td><td align="right">2000</td><td>ZCRRLCYKQRCVTYCRGR</td><td>Gomesin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>20% hemolysis at 100μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3409">3409</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11713517">11713517</a></td><td align="right">2001</td><td>FFHHIFRGIVHVGKTIHRLVTG</td><td>Piscidin 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolysis at 10μg/ml</td><td>Human</td><td>CHHHHHHHHHHHTTHHHHHHTC</td><td align="center"><a href="jmol.php?id=3409&amp;di=1" width="5px" height="5px"><img src="pymol_images/3409.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3410">3410</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11713517">11713517</a></td><td align="right">2001</td><td>FFHHIFRGIVHVGKTIHRLVTG</td><td>Piscidin 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>~90% hemolysis at 100μg/ml</td><td>Human</td><td>CHHHHHHHHHHHTTHHHHHHTC</td><td align="center"><a href="jmol.php?id=3410&amp;di=1" width="5px" height="5px"><img src="pymol_images/3410.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3411">3411</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11713517">11713517</a></td><td align="right">2001</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Piscidin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolysis at 10μg/ml</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3411&amp;di=1" width="5px" height="5px"><img src="pymol_images/3411.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3412">3412</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11713517">11713517</a></td><td align="right">2001</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Piscidin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>~90% hemolysis at 100μg/ml</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3412&amp;di=1" width="5px" height="5px"><img src="pymol_images/3412.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3413">3413</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11713517">11713517</a></td><td align="right">2001</td><td>FIHHIFRGIVHAGRSIGRFLTG</td><td>Piscidin 3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolysis at 10μg/ml</td><td>Human</td><td>CHHHHHHHHHHHTTSHHHHTTC</td><td align="center"><a href="jmol.php?id=3413&amp;di=1" width="5px" height="5px"><img src="pymol_images/3413.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3414">3414</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11713517">11713517</a></td><td align="right">2001</td><td>FIHHIFRGIVHAGRSIGRFLTG</td><td>Piscidin 3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>~5% hemolysis at 100μg/ml</td><td>Human</td><td>CHHHHHHHHHHHTTSHHHHTTC</td><td align="center"><a href="jmol.php?id=3414&amp;di=1" width="5px" height="5px"><img src="pymol_images/3414.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3415">3415</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolysis upto 1.25μM</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3415&amp;di=1" width="5px" height="5px"><img src="pymol_images/3415.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3416">3416</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>1% hemolysis upto 2.5μM</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3416&amp;di=1" width="5px" height="5px"><img src="pymol_images/3416.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3417">3417</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>0% hemolysis upto 5μM</td><td>Sheep</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3417&amp;di=1" width="5px" height="5px"><img src="pymol_images/3417.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3418">3418</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>19% hemolysis at 5μM</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3418&amp;di=1" width="5px" height="5px"><img src="pymol_images/3418.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3419">3419</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>11% hemolysis at 10μM</td><td>Sheep</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3419&amp;di=1" width="5px" height="5px"><img src="pymol_images/3419.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3420">3420</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>55% hemolysis at 10μM</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3420&amp;di=1" width="5px" height="5px"><img src="pymol_images/3420.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3421">3421</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>51% hemolysis at 20μM</td><td>Sheep</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3421&amp;di=1" width="5px" height="5px"><img src="pymol_images/3421.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3422">3422</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>100% hemolysis at 80μM</td><td>Human</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3422&amp;di=1" width="5px" height="5px"><img src="pymol_images/3422.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3423">3423</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>80% hemolysis at 40μM</td><td>Sheep</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3423&amp;di=1" width="5px" height="5px"><img src="pymol_images/3423.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3424">3424</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11739390">11739390</a></td><td align="right">2002</td><td>FFHHIFRGIVHVGKTIHKLVTG</td><td>Wb-moronecidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>100% hemolysis at 80μM</td><td>Sheep</td><td>CHHHHHHHHTTTHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3424&amp;di=1" width="5px" height="5px"><img src="pymol_images/3424.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3428">3428</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11835991">11835991</a></td><td align="right">2002</td><td>ILGPVISTIGGVLGGLLKNL</td><td>Maximin H1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial and Cytotoxic and Anti-HIV</td><td>90-100% hemolysis at 50μg/ml</td><td>Rabbit</td><td>CCCCCCHHHHHHHHHHCCCC</td><td align="center"><a href="jmol.php?id=3428&amp;di=1" width="5px" height="5px"><img src="pymol_images/3428.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3428&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3428.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3429">3429</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11835991">11835991</a></td><td align="right">2002</td><td>ILGPVLSMVGSALGGLIKKI</td><td>Maximin H2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial and Cytotoxic and Anti-HIV</td><td>90-100% hemolysis at 50μg/ml</td><td>Rabbit</td><td>CCCSCHHHHHHTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3429&amp;di=1" width="5px" height="5px"><img src="pymol_images/3429.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3429&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3429.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3430">3430</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11835991">11835991</a></td><td align="right">2002</td><td>ILGPVLGLVGNALGGLIKKI</td><td>Maximin H3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial and Cytotoxic and Anti-HIV</td><td>90-100% hemolysis at 50μg/ml</td><td>Rabbit</td><td>CCCHHHHHHHHHTSSSCCCC</td><td align="center"><a href="jmol.php?id=3430&amp;di=1" width="5px" height="5px"><img src="pymol_images/3430.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3430&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3430.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3431">3431</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11835991">11835991</a></td><td align="right">2002</td><td>ILGPVISKIGGVLGGLLKNL</td><td>Maximin H4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial and Cytotoxic and Anti-HIV</td><td>90-100% hemolysis at 50μg/ml</td><td>Rabbit</td><td>CTTHHHHHHHHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=3431&amp;di=1" width="5px" height="5px"><img src="pymol_images/3431.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3432">3432</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>Peptide-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>0% hemolysis upto 100μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=3432&amp;di=1" width="5px" height="5px"><img src="pymol_images/3432.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3433">3433</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>Peptide-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>1% hemolysis upto 200μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=3433&amp;di=1" width="5px" height="5px"><img src="pymol_images/3433.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3434">3434</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>KKKKKKKKKKGIGKFLHSAKKFGKAFVGEIMNS</td><td>Peptide-3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>0% hemolysis upto 200μg/ml (non hemolytic)</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3435">3435</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS</td><td>Peptide-4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>2% hemolysis upto 100μg/ml</td><td>Human</td><td>CTTSTTSHHHHHHHHHHHHHSSTTSSTTTTTTC</td><td align="center"><a href="jmol.php?id=3435&amp;di=1" width="5px" height="5px"><img src="pymol_images/3435.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3436">3436</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS</td><td>Peptide-4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>9% hemolysis upto 200μg/ml</td><td>Human</td><td>CTTSTTSHHHHHHHHHHHHHSSTTSSTTTTTTC</td><td align="center"><a href="jmol.php?id=3436&amp;di=1" width="5px" height="5px"><img src="pymol_images/3436.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3437">3437</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>RRRRRRRRRRGIGKFLHSAKKFGKAFVGEIMNS</td><td>Peptide-5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>0% hemolysis at 25μg/ml</td><td>Human</td><td>CCCCGGGGGTTTTHHHHHTTSSSSSSTTTGGGC</td><td align="center"><a href="jmol.php?id=3437&amp;di=1" width="5px" height="5px"><img src="pymol_images/3437.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3438">3438</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>RRRRRRRRRRGIGKFLHSAKKFGKAFVGEIMNS</td><td>Peptide-5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>1% hemolysis at 50μg/ml</td><td>Human</td><td>CCCCGGGGGTTTTHHHHHTTSSSSSSTTTGGGC</td><td align="center"><a href="jmol.php?id=3438&amp;di=1" width="5px" height="5px"><img src="pymol_images/3438.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3439">3439</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>RRRRRRRRRRGIGKFLHSAKKFGKAFVGEIMNS</td><td>Peptide-5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>2% hemolysis at 100μg/ml</td><td>Human</td><td>CCCCGGGGGTTTTHHHHHTTSSSSSSTTTGGGC</td><td align="center"><a href="jmol.php?id=3439&amp;di=1" width="5px" height="5px"><img src="pymol_images/3439.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3440">3440</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>RRRRRRRRRRGIGKFLHSAKKFGKAFVGEIMNS</td><td>Peptide-5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>4% hemolysis at 200μg/ml</td><td>Human</td><td>CCCCGGGGGTTTTHHHHHTTSSSSSSTTTGGGC</td><td align="center"><a href="jmol.php?id=3440&amp;di=1" width="5px" height="5px"><img src="pymol_images/3440.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3445">3445</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>GIGKFLHSAKKFGKAFVGEIMNSKKKKKKKKKK</td><td>Peptide-7</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>0% hemolysis upto 100μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3446">3446</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>GIGKFLHSAKKFGKAFVGEIMNSKKKKKKKKKK</td><td>Peptide-7</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>1% hemolysis at 200μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3447">3447</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>KKFLHSAKKWLKAFVKLKKNW</td><td>Peptide-8</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antibacterial</td><td>3% hemolysis at 25μg/ml</td><td>Human</td><td>CCCHHHHHHHHHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=3447&amp;di=1" width="5px" height="5px"><img src="pymol_images/3447.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3447&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3447.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3448">3448</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>KKFLHSAKKWLKAFVKLKKNW</td><td>Peptide-8</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antibacterial</td><td>16% hemolysis at 50μg/ml</td><td>Human</td><td>CCCHHHHHHHHHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=3448&amp;di=1" width="5px" height="5px"><img src="pymol_images/3448.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3448&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3448.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3449">3449</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>KKFLHSAKKWLKAFVKLKKNW</td><td>Peptide-8</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antibacterial</td><td>29% hemolysis at 100μg/ml</td><td>Human</td><td>CCCHHHHHHHHHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=3449&amp;di=1" width="5px" height="5px"><img src="pymol_images/3449.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3449&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3449.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3450">3450</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>KKFLHSAKKWLKAFVKLKKNW</td><td>Peptide-8</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antibacterial</td><td>41% hemolysis at 200μg/ml</td><td>Human</td><td>CCCHHHHHHHHHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=3450&amp;di=1" width="5px" height="5px"><img src="pymol_images/3450.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3450&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3450.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3451">3451</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>GIGKLFLHAAKKFAKAFVAEKMNS</td><td>Peptide-9</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antibacterial</td><td>0% hemolysis upto 200μg/ml (non hemolytic)</td><td>Human</td><td>CTTTTGGGTTHHHHGGGGTGGGTC</td><td align="center"><a href="jmol.php?id=3451&amp;di=1" width="5px" height="5px"><img src="pymol_images/3451.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3451&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3451.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3452">3452</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>KKKKKKKKKKGIGKLFLHAAKKFAKAFVAEKMNS</td><td>Peptide-10</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antibacterial</td><td>0% hemolysis at 100μg/ml</td><td>Human</td><td>CCGGGGCCCCCCCSSHHHHHHHHHHHHTTTTSCC</td><td align="center"><a href="jmol.php?id=3452&amp;di=1" width="5px" height="5px"><img src="pymol_images/3452.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3452&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3452.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3453">3453</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/1605597">1605597</a></td><td align="right">1992</td><td>KKKKKKKKKKGIGKLFLHAAKKFAKAFVAEKMNS</td><td>Peptide-10</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antibacterial</td><td>2% hemolysis at 200μg/ml</td><td>Human</td><td>CCGGGGCCCCCCCSSHHHHHHHHHHHHTTTTSCC</td><td align="center"><a href="jmol.php?id=3453&amp;di=1" width="5px" height="5px"><img src="pymol_images/3453.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3453&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3453.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3454">3454</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10604598">10604598</a></td><td align="right">1999</td><td>PKLLKTFLSKWIG</td><td>SPFK</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>100% hemolysis at 65μg/ml</td><td>Rat</td><td>CCGGGTTSSTTCC</td><td align="center"><a href="jmol.php?id=3454&amp;di=1" width="5px" height="5px"><img src="pymol_images/3454.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3454&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3454.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3455">3455</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10604598">10604598</a></td><td align="right">1999</td><td>CKLLKTFLSKWIC</td><td>SC</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>100% hemolysis at 40μg/ml</td><td>Rat</td><td>CCHHHHTGGGTTC</td><td align="center"><a href="jmol.php?id=3455&amp;di=1" width="5px" height="5px"><img src="pymol_images/3455.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3455&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3455.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3456">3456</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10604598">10604598</a></td><td align="right">1999</td><td>CKLLKTFLSKWIC</td><td>SAC</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>100% hemolysis at 5μg/ml</td><td>Rat</td><td>CCHHHHTGGGTTC</td><td align="center"><a href="jmol.php?id=3456&amp;di=1" width="5px" height="5px"><img src="pymol_images/3456.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3456&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3456.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3457">3457</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8823199">8823199</a></td><td align="right">1996</td><td>KLALKLALKALKAALKLA</td><td>KLAL</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>EC<sub>50</sub> =10μM</td><td>Human</td><td>CGGGTTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3457&amp;di=1" width="5px" height="5px"><img src="pymol_images/3457.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3457&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3457.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3458">3458</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8823199">8823199</a></td><td align="right">1996</td><td>klALKLALKALKAALKLA</td><td>k1/l2</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>EC<sub>50</sub> =7μM</td><td>Human</td><td>CGGGTTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3458&amp;di=1" width="5px" height="5px"><img src="pymol_images/3458.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3458&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3458.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3459">3459</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8823199">8823199</a></td><td align="right">1996</td><td>KLalKLALKALKAALKLA</td><td>a3/l4</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>EC<sub>50</sub> =33μM</td><td>Human</td><td>CGGGTTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3459&amp;di=1" width="5px" height="5px"><img src="pymol_images/3459.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3459&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3459.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3460">3460</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8823199">8823199</a></td><td align="right">1996</td><td>KLALklALKALKAALKLA</td><td>k5/l6</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>EC<sub>50</sub> =160μM</td><td>Human</td><td>CGGGTTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3460&amp;di=1" width="5px" height="5px"><img src="pymol_images/3460.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3460&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3460.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3461">3461</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8823199">8823199</a></td><td align="right">1996</td><td>KLALKLalKALKAALKLA</td><td>a7/l8</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>EC<sub>50</sub> =180μM</td><td>Human</td><td>CGGGTTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3461&amp;di=1" width="5px" height="5px"><img src="pymol_images/3461.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3461&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3461.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3462">3462</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8823199">8823199</a></td><td align="right">1996</td><td>KLALKLALkaLKAALKLA</td><td>k9/a10</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>EC<sub>50</sub> =56μM</td><td>Human</td><td>CGGGTTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3462&amp;di=1" width="5px" height="5px"><img src="pymol_images/3462.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3462&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3462.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3463">3463</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8823199">8823199</a></td><td align="right">1996</td><td>KLALKLALKAlkAALKLA</td><td>l11/k12</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>EC<sub>50</sub> =540μM</td><td>Human</td><td>CGGGTTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3463&amp;di=1" width="5px" height="5px"><img src="pymol_images/3463.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3463&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3463.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3464">3464</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8823199">8823199</a></td><td align="right">1996</td><td>KLALKLALKALKaaLKLA</td><td>a13/a14</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>EC<sub>50</sub> =260μM</td><td>Human</td><td>CGGGTTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3464&amp;di=1" width="5px" height="5px"><img src="pymol_images/3464.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3464&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3464.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3465">3465</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8823199">8823199</a></td><td align="right">1996</td><td>KLALKLALKALKAAlkLA</td><td>l15/k16</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>EC<sub>50</sub> =210μM</td><td>Human</td><td>CGGGTTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3465&amp;di=1" width="5px" height="5px"><img src="pymol_images/3465.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3465&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3465.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3466">3466</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8823199">8823199</a></td><td align="right">1996</td><td>KLALKLALKALKAALKla</td><td>l17/a18</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>EC<sub>50</sub> =70μM</td><td>Human</td><td>CGGGTTTHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3466&amp;di=1" width="5px" height="5px"><img src="pymol_images/3466.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3466&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3466.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3467">3467</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8855958">8855958</a></td><td align="right">1996</td><td>KKLKKLKKKWKKLKKKLK</td><td>Hel 5-13</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>0% hemolysis at 100μM</td><td>Human</td><td>CTHHHHGGGGTTGGGTCC</td><td align="center"><a href="jmol.php?id=3467&amp;di=1" width="5px" height="5px"><img src="pymol_images/3467.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3467&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3467.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3468">3468</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8855958">8855958</a></td><td align="right">1996</td><td>KKLKKLLKKWKKLLKKLK</td><td>Hel 7-11</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>~5% hemolysis upto 100μM (non hemolytic)</td><td>Human</td><td>CCGGGTHHHHTTTTTTCC</td><td align="center"><a href="jmol.php?id=3468&amp;di=1" width="5px" height="5px"><img src="pymol_images/3468.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3468&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3468.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3469">3469</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8855958">8855958</a></td><td align="right">1996</td><td>KLLKKLLKLWKKLLKKLK</td><td>Hel 9-9</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>~45% hemolysis at 10μM</td><td>Human</td><td>CTHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3469&amp;di=1" width="5px" height="5px"><img src="pymol_images/3469.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3469&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3469.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3470">3470</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8855958">8855958</a></td><td align="right">1996</td><td>KLLKKLLKLWKKLLKKLK</td><td>Hel 9-9</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>~60% hemolysis at 100μM</td><td>Human</td><td>CTHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3470&amp;di=1" width="5px" height="5px"><img src="pymol_images/3470.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3470&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3470.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3471">3471</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8855958">8855958</a></td><td align="right">1996</td><td>KLLKLLLKLWKKLLKLLK</td><td>Hel 11-7</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>~45% hemolysis at 1μM</td><td>Human</td><td>CTHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3471&amp;di=1" width="5px" height="5px"><img src="pymol_images/3471.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3471&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3471.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3472">3472</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8855958">8855958</a></td><td align="right">1996</td><td>KLLKLLLKLWKKLLKLLK</td><td>Hel 11-7</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>100% hemolysis at 10μM</td><td>Human</td><td>CTHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3472&amp;di=1" width="5px" height="5px"><img src="pymol_images/3472.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3472&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3472.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3473">3473</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8855958">8855958</a></td><td align="right">1996</td><td>KLLKLLLKLWLKLLKLLL</td><td>Hel 13-5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>100% hemolysis at 1μM</td><td>Human</td><td>CTHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3473&amp;di=1" width="5px" height="5px"><img src="pymol_images/3473.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3473&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3473.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3474">3474</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11738089">11738089</a></td><td align="right">2001</td><td>GLLKRIKTLL</td><td>Anoplin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial and Mast cell degranulating</td><td>Low hemolytic activity</td><td>Human</td><td>CGGGTGGGTC</td><td align="center"><a href="jmol.php?id=3474&amp;di=1" width="5px" height="5px"><img src="pymol_images/3474.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3474&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3474.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3475">3475</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11738089">11738089</a></td><td align="right">2001</td><td>FLPLILRKIVTAL</td><td>Crabrolin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial and Mast cell degranulating</td><td>No hemolysis (non hemolytic)</td><td>Rat</td><td>CCHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3475&amp;di=1" width="5px" height="5px"><img src="pymol_images/3475.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3475&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3475.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3476">3476</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11738090">11738090</a></td><td align="right">2001</td><td>GFLGPLLKLAAKGVAKVIPHLIPSRQQ</td><td>XT-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>HC<sub>50</sub> =90μM</td><td>Human</td><td>CCTTTTGGGGTCCCTTTSCCSSCCCCC</td><td align="center"><a href="jmol.php?id=3476&amp;di=1" width="5px" height="5px"><img src="pymol_images/3476.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3476&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3476.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3477">3477</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11738090">11738090</a></td><td align="right">2001</td><td>GVFLDALKKFAKGGMNAVLNPK</td><td>XT-4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;150μM</td><td>Human</td><td>CCCHHHHHHHTCCSCCCCCCCC</td><td align="center"><a href="jmol.php?id=3477&amp;di=1" width="5px" height="5px"><img src="pymol_images/3477.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3477&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3477.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3478">3478</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11738090">11738090</a></td><td align="right">2001</td><td>GMATKAGTALGKVAKAVIGAAL</td><td>XT-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;150μM</td><td>Human</td><td>CTTSSSSCTTHHHHTCHHHHCC</td><td align="center"><a href="jmol.php?id=3478&amp;di=1" width="5px" height="5px"><img src="pymol_images/3478.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3478&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3478.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3479">3479</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11738090">11738090</a></td><td align="right">2001</td><td>GFLGSLLKTGLKVGSNLL</td><td>XT-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;150μM</td><td>Human</td><td>CCCHHHHTTTTTCSSCCC</td><td align="center"><a href="jmol.php?id=3479&amp;di=1" width="5px" height="5px"><img src="pymol_images/3479.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3479&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3479.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3480">3480</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11738090">11738090</a></td><td align="right">2001</td><td>GLLGPLLKIAAKVGSNLL</td><td>XT-7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HC<sub>50</sub> =70μM</td><td>Human</td><td>CCSGGGHHHHHHHTCCCC</td><td align="center"><a href="jmol.php?id=3480&amp;di=1" width="5px" height="5px"><img src="pymol_images/3480.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3480&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3480.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3484">3484</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19427345">19427345</a></td><td align="right">2009</td><td>FLGAIAAALPHVINAVTNAL</td><td>Kassinatuerin-2Ma</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>45-50% hemolysis at 120μM</td><td>Horse</td><td>CCTTHHHHHHHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=3484&amp;di=1" width="5px" height="5px"><img src="pymol_images/3484.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3485">3485</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19427345">19427345</a></td><td align="right">2009</td><td>FFGAIAAALPHVISAIKNAL</td><td>Kassinatuerin-2Mb</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>45-50% hemolysis at 120μM</td><td>Horse</td><td>CCCTHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3485&amp;di=1" width="5px" height="5px"><img src="pymol_images/3485.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3486">3486</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19427345">19427345</a></td><td align="right">2009</td><td>FVGAIAAALPHVISAIKNAL</td><td>Kassinatuerin-2Mc</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>45-50% hemolysis at 120μM</td><td>Horse</td><td>CCSTHHHHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=3486&amp;di=1" width="5px" height="5px"><img src="pymol_images/3486.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3487">3487</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19427345">19427345</a></td><td align="right">2009</td><td>IIGAIAAALPHVINAIKNTF</td><td>Kassinatuerin-2Md</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>45-50% hemolysis at 120μM</td><td>Horse</td><td>CTTTHHHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3487&amp;di=1" width="5px" height="5px"><img src="pymol_images/3487.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3488">3488</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19427345">19427345</a></td><td align="right">2009</td><td>FIQYLAPLIPHAVKAISDLI</td><td>Kassinatuerin-2S</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>45-50% hemolysis at 120μM</td><td>Horse</td><td>CCTTCTTTSGGGGGGGSCCC</td><td align="center"><a href="jmol.php?id=3488&amp;di=1" width="5px" height="5px"><img src="pymol_images/3488.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3489">3489</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19427345">19427345</a></td><td align="right">2009</td><td>GFMKYIGPLIPHAVKAISDLI</td><td>Kassinatuerin-1S</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>45-50% hemolysis at 120μM</td><td>Horse</td><td>CCCCCCSSSSTTTGGGTCCCC</td><td align="center"><a href="jmol.php?id=3489&amp;di=1" width="5px" height="5px"><img src="pymol_images/3489.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3489&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3489.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3490">3490</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19539775">19539775</a></td><td align="right">2009</td><td>FLSLALAALPKLFCLIFKKC</td><td>Brevinin-1CDYa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>HC<sub>50</sub> =125μM</td><td>Human</td><td>CHHHHHTTTTTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3490&amp;di=1" width="5px" height="5px"><img src="pymol_images/3490.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3490&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3490.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3491">3491</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19539775">19539775</a></td><td align="right">2009</td><td>FFPLALLCKVFKKC</td><td>Japonicin-1CDYa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;300μM</td><td>Human</td><td>CCHHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3491&amp;di=1" width="5px" height="5px"><img src="pymol_images/3491.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3491&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3491.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3492">3492</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19539775">19539775</a></td><td align="right">2009</td><td>ILPILSLIGGLL</td><td>Temporin-CDYb</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>HC<sub>50</sub> =180μM</td><td>Human</td><td>CCGGGGGTTCCC</td><td align="center"><a href="jmol.php?id=3492&amp;di=1" width="5px" height="5px"><img src="pymol_images/3492.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3492&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3492.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3493">3493</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19539775">19539775</a></td><td align="right">2009</td><td>IIPLPLGYFAKKT</td><td>Dybowskin-1CDYa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;300μM</td><td>Human</td><td>CCSSGGGGSSCCC</td><td align="center"><a href="jmol.php?id=3493&amp;di=1" width="5px" height="5px"><img src="pymol_images/3493.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3493&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3493.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3494">3494</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19539775">19539775</a></td><td align="right">2009</td><td>SAVGRHGRRFGLRKHRKH</td><td>Dybowskin-2CDYa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>HC<sub>50</sub> &gt;300μM</td><td>Human</td><td>CCSSCCSSCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=3494&amp;di=1" width="5px" height="5px"><img src="pymol_images/3494.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3494&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3494.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3495">3495</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19109988">19109988</a></td><td align="right">2010</td><td>GFLSALKKYLPIVLKHV</td><td>Osmin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antibacterial and Antifungal</td><td>Weak hemolytic activity <i>i.e.</i> 5.0HU/mg</td><td>Human</td><td>CTHHHHTTTTTSSCCCC</td><td align="center"><a href="jmol.php?id=3495&amp;di=1" width="5px" height="5px"><img src="pymol_images/3495.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3495&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3495.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3496">3496</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19109988">19109988</a></td><td align="right">2010</td><td>GFLSALKKYLPIVLKHV</td><td>Osmin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antibacterial and Antifungal</td><td>Hemolytic activity <i>i.e.</i> 16.6HU/mg</td><td>Human</td><td>CTHHHHTTTTTSSCCCC</td><td align="center"><a href="jmol.php?id=3496&amp;di=1" width="5px" height="5px"><img src="pymol_images/3496.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3496&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3496.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3497">3497</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLKSIVSWAKKVL</td><td>MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>&gt;95% hemolysis at 30μM</td><td>Guinea-pig</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=3497&amp;di=1" width="5px" height="5px"><img src="pymol_images/3497.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3497&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3497.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3498">3498</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLKSIVSWAKKVL</td><td>MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>80% hemolysis at 20μM</td><td>Guinea-pig</td><td>CCCCHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=3498&amp;di=1" width="5px" height="5px"><img src="pymol_images/3498.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3498&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3498.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3499">3499</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LNLKSIVSWAKKVL</td><td>[Asn<sup>2</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>~5% hemolysis at 30μM</td><td>Guinea-pig</td><td>CCHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3499&amp;di=1" width="5px" height="5px"><img src="pymol_images/3499.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3499&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3499.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3500">3500</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LNLKSIVSWAKKVL</td><td>[Asn<sup>2</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>~4% hemolysis at 20μM</td><td>Guinea-pig</td><td>CCHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3500&amp;di=1" width="5px" height="5px"><img src="pymol_images/3500.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3500&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3500.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3501">3501</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLNSIVSWAKKVL</td><td>[Asn<sup>4</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>&gt;95% hemolysis at 30μM</td><td>Guinea-pig</td><td>CCCCCTTHHHHTTC</td><td align="center"><a href="jmol.php?id=3501&amp;di=1" width="5px" height="5px"><img src="pymol_images/3501.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3501&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3501.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3502">3502</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLNSIVSWAKKVL</td><td>[Asn<sup>4</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>&gt;75% hemolysis at 20μM</td><td>Guinea-pig</td><td>CCCCCTTHHHHTTC</td><td align="center"><a href="jmol.php?id=3502&amp;di=1" width="5px" height="5px"><img src="pymol_images/3502.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3502&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3502.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3503">3503</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLLSIVSWAKKVL</td><td>[Leu<sup>4</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>~80% hemolysis at 30μM</td><td>Guinea-pig</td><td>CCCCSHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3503&amp;di=1" width="5px" height="5px"><img src="pymol_images/3503.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3503&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3503.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3504">3504</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLLSIVSWAKKVL</td><td>[Leu<sup>4</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>&gt;75% hemolysis at 20μM</td><td>Guinea-pig</td><td>CCCCSHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3504&amp;di=1" width="5px" height="5px"><img src="pymol_images/3504.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3504&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3504.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3505">3505</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LNLNSIVSWAKKVL</td><td>[Asn<sup>2</sup>,<sup>4</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>~5% hemolysis at 30μM</td><td>Guinea-pig</td><td>CCCCHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3505&amp;di=1" width="5px" height="5px"><img src="pymol_images/3505.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3505&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3505.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3506">3506</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LNLNSIVSWAKKVL</td><td>[Asn<sup>2</sup>,<sup>4</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>~3% hemolysis at 20μM</td><td>Guinea-pig</td><td>CCCCHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3506&amp;di=1" width="5px" height="5px"><img src="pymol_images/3506.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3506&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3506.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3507">3507</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLKSIVSFAKKVL</td><td>[Phe<sup>9</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>~5% hemolysis at 30μM</td><td>Guinea-pig</td><td>CCCSSTHHHHTTTC</td><td align="center"><a href="jmol.php?id=3507&amp;di=1" width="5px" height="5px"><img src="pymol_images/3507.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3507&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3507.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3508">3508</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLKSIVSFAKKVL</td><td>[Phe<sup>9</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>~3% hemolysis at 20μM</td><td>Guinea-pig</td><td>CCCSSTHHHHTTTC</td><td align="center"><a href="jmol.php?id=3508&amp;di=1" width="5px" height="5px"><img src="pymol_images/3508.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3508&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3508.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3509">3509</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLKSIVSYAKKVL</td><td>[Tyr<sup>9</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>~4% hemolysis at 30μM</td><td>Guinea-pig</td><td>CCSGGGHHHHTTCC</td><td align="center"><a href="jmol.php?id=3509&amp;di=1" width="5px" height="5px"><img src="pymol_images/3509.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3509&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3509.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3510">3510</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLKSIVSYAKKVL</td><td>[Tyr<sup>9</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>~2% hemolysis at 20μM</td><td>Guinea-pig</td><td>CCSGGGHHHHTTCC</td><td align="center"><a href="jmol.php?id=3510&amp;di=1" width="5px" height="5px"><img src="pymol_images/3510.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3510&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3510.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3511">3511</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLKSIVSWALKVL</td><td>[Leu<sup>11</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>&gt;80% hemolysis at 30μM</td><td>Guinea-pig</td><td>CCSCSTHHHHTTCC</td><td align="center"><a href="jmol.php?id=3511&amp;di=1" width="5px" height="5px"><img src="pymol_images/3511.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3511&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3511.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3512">3512</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLKSIVSWALKVL</td><td>[Leu<sup>11</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>&gt;70% hemolysis at 20μM</td><td>Guinea-pig</td><td>CCSCSTHHHHTTCC</td><td align="center"><a href="jmol.php?id=3512&amp;di=1" width="5px" height="5px"><img src="pymol_images/3512.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3512&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3512.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3513">3513</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLKSIVSWAKLVL</td><td>[Leu<sup>12</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>~70% hemolysis at 30μM</td><td>Guinea-pig</td><td>CCCCCTHHHHTTTC</td><td align="center"><a href="jmol.php?id=3513&amp;di=1" width="5px" height="5px"><img src="pymol_images/3513.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3513&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3513.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3514">3514</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLKSIVSWAKLVL</td><td>[Leu<sup>12</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>&gt;55% hemolysis at 20μM</td><td>Guinea-pig</td><td>CCCCCTHHHHTTTC</td><td align="center"><a href="jmol.php?id=3514&amp;di=1" width="5px" height="5px"><img src="pymol_images/3514.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3514&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3514.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3515">3515</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLKSIVSWALLVL</td><td>[Leu<sup>11</sup>,<sup>12</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>~80% hemolysis at 30μM</td><td>Guinea-pig</td><td>CCCCSTHHHHTSCC</td><td align="center"><a href="jmol.php?id=3515&amp;di=1" width="5px" height="5px"><img src="pymol_images/3515.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3515&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3515.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3516">3516</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8896194">8896194</a></td><td align="right">1996</td><td>LKLKSIVSWALLVL</td><td>[Leu<sup>11</sup>,<sup>12</sup>]-MP-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Edema inducer</td><td>~55% hemolysis at 20μM</td><td>Guinea-pig</td><td>CCCCSTHHHHTSCC</td><td align="center"><a href="jmol.php?id=3516&amp;di=1" width="5px" height="5px"><img src="pymol_images/3516.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3516&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3516.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3517">3517</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9026036">9026036</a></td><td align="right">1997</td><td>CIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Dioleoylmelittin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Membrane disturbing</td><td>~20% hemolysis at 1μM</td><td>Dog</td><td>CHHHHHHHHTTCCHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3517&amp;di=1" width="5px" height="5px"><img src="pymol_images/3517.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3517&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3517.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3518">3518</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9026036">9026036</a></td><td align="right">1997</td><td>CIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Dioleoylmelittin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Membrane disturbing</td><td>100% hemolysis at 10μM</td><td>Dog</td><td>CHHHHHHHHTTCCHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3518&amp;di=1" width="5px" height="5px"><img src="pymol_images/3518.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3518&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3518.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3519">3519</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9166783">9166783</a></td><td align="right">1997</td><td>GLGKFLHSAKRFGKAFVGEAMNS</td><td>L<sup>2</sup>R<sup>11</sup>A<sup>20</sup>-M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>EC<sub>50</sub> &gt;1000μM</td><td>Human</td><td>CCSHHHHHHHHHHHHTTCTTCCC</td><td align="center"><a href="jmol.php?id=3519&amp;di=1" width="5px" height="5px"><img src="pymol_images/3519.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3520">3520</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9166783">9166783</a></td><td align="right">1997</td><td>GLGKFLHSAKRFGKAFVGEAMNS</td><td>L<sup>2</sup>R<sup>11</sup>A<sup>20</sup>-M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>12% hemolysis at 150μM</td><td>Human</td><td>CCSHHHHHHHHHHHHTTCTTCCC</td><td align="center"><a href="jmol.php?id=3520&amp;di=1" width="5px" height="5px"><img src="pymol_images/3520.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3521">3521</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9166783">9166783</a></td><td align="right">1997</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>EC<sub>50</sub> =430μM</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=3521&amp;di=1" width="5px" height="5px"><img src="pymol_images/3521.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3522">3522</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9166783">9166783</a></td><td align="right">1997</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>30% hemolysis at 150μM</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=3522&amp;di=1" width="5px" height="5px"><img src="pymol_images/3522.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3523">3523</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9166783">9166783</a></td><td align="right">1997</td><td>GIGKFIHSAKKFGKLFVGEIMNS</td><td>I<sup>6</sup>L<sup>15</sup>-M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>EC<sub>50</sub> =260μM</td><td>Human</td><td>CCSHHHHHHHHHHTCSSSSTTTC</td><td align="center"><a href="jmol.php?id=3523&amp;di=1" width="5px" height="5px"><img src="pymol_images/3523.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3524">3524</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9166783">9166783</a></td><td align="right">1997</td><td>GIGKFIHSAKKFGKLFVGEIMNS</td><td>I<sup>6</sup>L<sup>15</sup>-M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>40% hemolysis at 150μM</td><td>Human</td><td>CCSHHHHHHHHHHTCSSSSTTTC</td><td align="center"><a href="jmol.php?id=3524&amp;di=1" width="5px" height="5px"><img src="pymol_images/3524.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3525">3525</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9166783">9166783</a></td><td align="right">1997</td><td>GIGKFIHAAKKFGKLFIGEIMNS</td><td>I<sup>6</sup>A<sup>8</sup>L<sup>15</sup>I<sup>17</sup>-M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>EC<sub>50</sub> =32μM</td><td>Human</td><td>CCCCTHHHHTTTSSCCCSCGGGC</td><td align="center"><a href="jmol.php?id=3525&amp;di=1" width="5px" height="5px"><img src="pymol_images/3525.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3525&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3525.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3526">3526</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9166783">9166783</a></td><td align="right">1997</td><td>GIGKFIHAAKKFGKLFIGEIMNS</td><td>I<sup>6</sup>A<sup>8</sup>L<sup>15</sup>I<sup>17</sup>-M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>98% hemolysis at 150μM</td><td>Human</td><td>CCCCTHHHHTTTSSCCCSCGGGC</td><td align="center"><a href="jmol.php?id=3526&amp;di=1" width="5px" height="5px"><img src="pymol_images/3526.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3526&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3526.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3527">3527</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11689009">11689009</a></td><td align="right">2001</td><td>GLNALKKVFQGIHEAIKLINNHVQ</td><td>Pseudin-2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>EC<sub>50</sub> &gt;300μM</td><td>Human</td><td>CCHHHHHHHHHHTHHHHTTTTSCC</td><td align="center"><a href="jmol.php?id=3527&amp;di=1" width="5px" height="5px"><img src="pymol_images/3527.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3527&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3527.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3529">3529</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11479030">11479030</a></td><td align="right">2001</td><td>ILQKAVLDCLKAAGSSLSKAAITAIYNKIT</td><td>Dicynthaurin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>0% hemolysis at 25μg/ml</td><td>Human</td><td>CHHHHHHHHHHHHTTCCCTTTHHHHHCCCC</td><td align="center"><a href="jmol.php?id=3529&amp;di=1" width="5px" height="5px"><img src="pymol_images/3529.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3529&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3529.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3530">3530</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11479030">11479030</a></td><td align="right">2001</td><td>ILQKAVLDCLKAAGSSLSKAAITAIYNKIT</td><td>Dicynthaurin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>10% hemolysis at 50μg/ml</td><td>Human</td><td>CHHHHHHHHHHHHTTCCCTTTHHHHHCCCC</td><td align="center"><a href="jmol.php?id=3530&amp;di=1" width="5px" height="5px"><img src="pymol_images/3530.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3530&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3530.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3531">3531</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11479030">11479030</a></td><td align="right">2001</td><td>ILQKAVLDCLKAAGSSLSKAAITAIYNKIT</td><td>Dicynthaurin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>20% hemolysis at 100μg/ml</td><td>Human</td><td>CHHHHHHHHHHHHTTCCCTTTHHHHHCCCC</td><td align="center"><a href="jmol.php?id=3531&amp;di=1" width="5px" height="5px"><img src="pymol_images/3531.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3531&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3531.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3532">3532</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11479030">11479030</a></td><td align="right">2001</td><td>QKAVLDCLKAAGSSLSKAAITAI</td><td>Cynthaurin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml (non hemolytic)</td><td>Human</td><td>CCTHHHHTTTSSSCCCHHHHSCC</td><td align="center"><a href="jmol.php?id=3532&amp;di=1" width="5px" height="5px"><img src="pymol_images/3532.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3532&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3532.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3533">3533</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3299384">3299384</a></td><td align="right">1987</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>Magainin-2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>0% hemolysis at 10μg/ml (non hemolytic)</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=3533&amp;di=1" width="5px" height="5px"><img src="pymol_images/3533.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3534">3534</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005415">12005415</a></td><td align="right">2001</td><td>FLRFIGSVIHGIGHLVHHIGVAL</td><td>Clavaspirin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>~70% hemolysis at 20μg/ml</td><td>Human</td><td>CCTTHHHHTTTHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3534&amp;di=1" width="5px" height="5px"><img src="pymol_images/3534.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3535">3535</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12005415">12005415</a></td><td align="right">2001</td><td>FLRFIGSVIHGIGHLVHHIGVAL</td><td>Clavaspirin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>75% hemolysis at 80μg/ml</td><td>Human</td><td>CCTTHHHHTTTHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3535&amp;di=1" width="5px" height="5px"><img src="pymol_images/3535.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3536">3536</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12067731">12067731</a></td><td align="right">2002</td><td>WLNALLHHGLNCAKGVLA</td><td>18 monomer halocidin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>0% hemolysis at 25μg/ml</td><td>Human</td><td>CHHHHTTSSTTSSCCCCC</td><td align="center"><a href="jmol.php?id=3536&amp;di=1" width="5px" height="5px"><img src="pymol_images/3536.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3536&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3536.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3537">3537</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12067731">12067731</a></td><td align="right">2002</td><td>WLNALLHHGLNCAKGVLA</td><td>18 monomer halocidin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>&lt;10% hemolysis at 50μg/ml</td><td>Human</td><td>CHHHHTTSSTTSSCCCCC</td><td align="center"><a href="jmol.php?id=3537&amp;di=1" width="5px" height="5px"><img src="pymol_images/3537.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3537&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3537.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3538">3538</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12067731">12067731</a></td><td align="right">2002</td><td>WLNALLHHGLNCAKGVLA</td><td>18 monomer halocidin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>~10% hemolysis at 100μg/ml</td><td>Human</td><td>CHHHHTTSSTTSSCCCCC</td><td align="center"><a href="jmol.php?id=3538&amp;di=1" width="5px" height="5px"><img src="pymol_images/3538.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3538&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3538.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3541">3541</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12067731">12067731</a></td><td align="right">2002</td><td>ALLHHGLNCAKGVLA</td><td>15 monomer halocidin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml (non hemolytic)</td><td>Human</td><td>CTTCSSCCGGGGTCC</td><td align="center"><a href="jmol.php?id=3541&amp;di=1" width="5px" height="5px"><img src="pymol_images/3541.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3541&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3541.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3542">3542</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12067731">12067731</a></td><td align="right">2002</td><td>ALLHHGLNCAKGVLAALLHHGLNCAKGVLA</td><td>15 homodimer halocidin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml (non hemolytic)</td><td>Human</td><td>CTTTCCCCCHHHHHHHHHHCSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3542&amp;di=1" width="5px" height="5px"><img src="pymol_images/3542.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3542&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3542.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3543">3543</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FFGWLIKGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>~40% hemolysis at 1μM</td><td>Human</td><td>CCGGGSTTTGGGTCCCCSSCSSSCC</td><td align="center"><a href="jmol.php?id=3543&amp;di=1" width="5px" height="5px"><img src="pymol_images/3543.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3543&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3543.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3544">3544</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FFGWLIKGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>100% hemolysis at 100μM</td><td>Human</td><td>CCGGGSTTTGGGTCCCCSSCSSSCC</td><td align="center"><a href="jmol.php?id=3544&amp;di=1" width="5px" height="5px"><img src="pymol_images/3544.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3544&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3544.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3545">3545</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FFGWLIRGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>&gt;40% hemolysis at 1μM</td><td>Human</td><td>CTTTTCGGGTTSSCCSSSSSCCCCC</td><td align="center"><a href="jmol.php?id=3545&amp;di=1" width="5px" height="5px"><img src="pymol_images/3545.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3545&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3545.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3546">3546</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FFGWLIRGAIHAGKAIHGLIHRRRH</td><td>Chrysophsin-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antibacterial</td><td>100% hemolysis at 100μM</td><td>Human</td><td>CTTTTCGGGTTSSCCSSSSSCCCCC</td><td align="center"><a href="jmol.php?id=3546&amp;di=1" width="5px" height="5px"><img src="pymol_images/3546.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3546&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3546.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3547">3547</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FIGLLISAGKAIHDLIRRRH</td><td>Chrysophsin-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antibacterial</td><td>0% hemolysis at 1μM</td><td>Human</td><td>CCSCCCCSGGGHHHHTTTCC</td><td align="center"><a href="jmol.php?id=3547&amp;di=1" width="5px" height="5px"><img src="pymol_images/3547.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3547&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3547.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3548">3548</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12581207">12581207</a></td><td align="right">2003</td><td>FIGLLISAGKAIHDLIRRRH</td><td>Chrysophsin-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antibacterial</td><td>100% hemolysis at 100μM</td><td>Human</td><td>CCSCCCCSGGGHHHHTTTCC</td><td align="center"><a href="jmol.php?id=3548&amp;di=1" width="5px" height="5px"><img src="pymol_images/3548.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3548&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3548.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3554">3554</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11563967">11563967</a></td><td align="right">2001</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>90.5±0.8% hemolysis at 44.5μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3554&amp;di=1" width="5px" height="5px"><img src="pymol_images/3554.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3555">3555</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11563967">11563967</a></td><td align="right">2001</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>51.4±0.5% hemolysis at 22.2μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3555&amp;di=1" width="5px" height="5px"><img src="pymol_images/3555.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr>'
test3 = '<tr class="even"><td align="right"><a href="display_sub.php?details=3556">3556</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11563967">11563967</a></td><td align="right">2001</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>17.5±1.1% hemolysis at 11.1μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3556&amp;di=1" width="5px" height="5px"><img src="pymol_images/3556.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3557">3557</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11563967">11563967</a></td><td align="right">2001</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>3.7±0.1% hemolysis at 5.5μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3557&amp;di=1" width="5px" height="5px"><img src="pymol_images/3557.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3558">3558</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11563967">11563967</a></td><td align="right">2001</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pandinin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>1.4±0.3% hemolysis at 2.7μM</td><td>Sheep</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3558&amp;di=1" width="5px" height="5px"><img src="pymol_images/3558.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3559">3559</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8223491">8223491</a></td><td align="right">1993</td><td>IIGPVLGLVGSALGGLLKKI</td><td>Peptide B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antibacterial</td><td>Lethal concentration =17.0μM</td><td>Human</td><td>CCSSCHHHHHHHTSSSCSCC</td><td align="center"><a href="jmol.php?id=3559&amp;di=1" width="5px" height="5px"><img src="pymol_images/3559.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3559&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3559.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3561">3561</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2689223">2689223</a></td><td align="right">1989</td><td>KWKLFKKIEKVGQGIGAVLKVLTTGL</td><td>CA(1-13)M(1-13)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antibacterial and Antiparasitic</td><td>Lethal concentration calculated from zone of inhibition&gt;200μM</td><td>Sheep</td><td>CCTTGGGGTTTSCTHHHHHTGGGCCC</td><td align="center"><a href="jmol.php?id=3561&amp;di=1" width="5px" height="5px"><img src="pymol_images/3561.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3561&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3561.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3562">3562</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2689223">2689223</a></td><td align="right">1989</td><td>AVAVVGQATQIAKKWKLFKKIEKVGQ</td><td>CA(25-37)CA(1-13)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antibacterial and Antiparasitic</td><td>Lethal concentration calculated from zone of inhibition&gt;300μM</td><td>Sheep</td><td>CCCCTTHHHHHHHHHHHHHHHTTTTC</td><td align="center"><a href="jmol.php?id=3562&amp;di=1" width="5px" height="5px"><img src="pymol_images/3562.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3562&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3562.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3563">3563</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2689223">2689223</a></td><td align="right">1989</td><td>GIGAVLKVLTTGLKWKLFKKIEKVGQ</td><td>M(1-13)CA(1-13)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antibacterial and Antiparasitic</td><td>Lethal concentration calculated from zone of inhibition80μM</td><td>Sheep</td><td>CGGGSHHHHHHHCCHHHHTTTTTTCC</td><td align="center"><a href="jmol.php?id=3563&amp;di=1" width="5px" height="5px"><img src="pymol_images/3563.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3563&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3563.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3564">3564</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/2689223">2689223</a></td><td align="right">1989</td><td>LISWIKRKRQQGIGAVLKVLTTGL</td><td>M(16-26)M(1-13)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antibacterial and Antiparasitic</td><td>Lethal concentration calculated from zone of inhibition&gt;200μM</td><td>Sheep</td><td>CCHHHHHHHTSCTTTCCCGGGCCC</td><td align="center"><a href="jmol.php?id=3564&amp;di=1" width="5px" height="5px"><img src="pymol_images/3564.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3564&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3564.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3565">3565</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9395500">9395500</a></td><td align="right">1997</td><td>XALWKNMLKGIGKLAGKAALGAVKKLVGAES</td><td>DS3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>X=H</td><td align="right">31</td><td>Antimicrobial and Cytolytic</td><td>No hemolysis at 34μM (non hemolytic)</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3566">3566</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9395500">9395500</a></td><td align="right">1997</td><td>XALWMTLLKKVLKAAAKAALNAVLVGANA</td><td>DS4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>X=H</td><td align="right">29</td><td>Antimicrobial and Cytolytic</td><td>Hemolysis observed at &gt;8.5μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3567">3567</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15304333">15304333</a></td><td align="right">2004</td><td>ACGILHDNCVYVPAQNPCCRGLQCRYGKCLVQV</td><td>PcFK1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antiplasmodial</td><td>No hemolysis at 10μM (non hemolytic)</td><td>Human</td><td>CCSSSSSSCCCCTTCCCCCSSCBSSSSBCCCCC</td><td align="center"><a href="jmol.php?id=3567&amp;di=1" width="5px" height="5px"><img src="pymol_images/3567.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3567&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3567.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3568">3568</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15304333">15304333</a></td><td align="right">2004</td><td>RCLPAGKTCVRGPMRVPCCGSCSQNKCT</td><td>PcFK2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antiplasmodial</td><td>No hemolysis at 10μM (non hemolytic)</td><td>Human</td><td>CCCCSBSSSCBCCCCSSCSSSCCCCTTC</td><td align="center"><a href="jmol.php?id=3568&amp;di=1" width="5px" height="5px"><img src="pymol_images/3568.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3569">3569</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18519720">18519720</a></td><td align="right">2008</td><td>AGYLLGKINLKALAALAKKIL</td><td>TP10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>5% hemolysis at 30μM</td><td>Human</td><td>CCSCCCSSCHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3569&amp;di=1" width="5px" height="5px"><img src="pymol_images/3569.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3569&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3569.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3570">3570</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18984589">18984589</a></td><td align="right">2009</td><td>KKALLOHALHOLALLAOHLAHOLKKA</td><td>HALO</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>O=Ornithine</td><td align="right">26</td><td>Antimicrobial and Antiplasmodial</td><td>HC<sub>50</sub> =179.2μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3571">3571</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18984589">18984589</a></td><td align="right">2009</td><td>ALALLOHALHOLALLAOHLAHOLALA</td><td>HALO-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>O=Ornithine</td><td align="right">26</td><td>Antimicrobial and Antiplasmodial</td><td>HC<sub>50</sub> &lt;5.6μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3572">3572</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18984589">18984589</a></td><td align="right">2009</td><td>FFKKLOHALHOLALLAOHLAHOLKKA</td><td>HALO-F</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>O=Ornithine</td><td align="right">26</td><td>Antimicrobial and Antiplasmodial</td><td>HC<sub>50</sub> &lt;5.6μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3573">3573</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18984589">18984589</a></td><td align="right">2009</td><td>KKALLOHPLHOLALLAOHLAHOLKKA</td><td>HALO-P8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>O=Ornithine</td><td align="right">26</td><td>Antimicrobial and Antiplasmodial</td><td>HC<sub>50</sub> =89.6μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3574">3574</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18984589">18984589</a></td><td align="right">2009</td><td>akklohalhoallalohlahollakk</td><td>D-HALO-rev</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>O=Ornithine</td><td align="right">26</td><td>Antimicrobial and Antiplasmodial</td><td>HC<sub>50</sub> =716.8μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3575">3575</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18984589">18984589</a></td><td align="right">2009</td><td>akklohalhoallalohlpholkkff</td><td>D-HALO-P8Frev</td><td>Amidation</td><td>Free</td><td>Linear</td><td>D</td><td>O=Ornithine</td><td align="right">26</td><td>Antimicrobial and Antiplasmodial</td><td>HC<sub>50</sub> &gt;65.2μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=3575&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3575.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3576">3576</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18332165">18332165</a></td><td align="right">2008</td><td>KILRGVCKKIMRTFLRRISKDILTGKK</td><td>NK-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antiplasmodial</td><td>No hemolysis upto 10μM (non hemolytic)</td><td>Human</td><td>CGGGGGGHHHHHHHGGGGGGTTSSCCC</td><td align="center"><a href="jmol.php?id=3576&amp;di=1" width="5px" height="5px"><img src="pymol_images/3576.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3576&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3576.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3577">3577</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600529">10600529</a></td><td align="right">1999</td><td>RHARHD</td><td>Hannahpep</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">6</td><td>Plasma-anticlotting</td><td>No hemolysis (non hemolytic)</td><td>Human</td><td>CCSCCC</td><td align="center"><a href="jmol.php?id=3577&amp;di=1" width="5px" height="5px"><img src="pymol_images/3577.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3577&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3577.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3578">3578</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600529">10600529</a></td><td align="right">1999</td><td>RHARHD</td><td>Hannahpep</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">6</td><td>Plasma-anticlotting</td><td>No hemolysis (non hemolytic)</td><td>Goat</td><td>CCSCCC</td><td align="center"><a href="jmol.php?id=3578&amp;di=1" width="5px" height="5px"><img src="pymol_images/3578.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3578&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3578.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3579">3579</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600529">10600529</a></td><td align="right">1999</td><td>RHARHD</td><td>Hannahpep</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">6</td><td>Plasma-anticlotting</td><td>No hemolysis (non hemolytic)</td><td>Mouse</td><td>CCSCCC</td><td align="center"><a href="jmol.php?id=3579&amp;di=1" width="5px" height="5px"><img src="pymol_images/3579.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3579&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3579.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3580">3580</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12969798">12969798</a></td><td align="right">2004</td><td>KAVAAKKSPKKAKKPAT</td><td>Oncorhyncin II</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>No hemolysis at 0.8μM(non hemolytic)</td><td>Trout</td><td>CCTTSSCCCCSSCSSCC</td><td align="center"><a href="jmol.php?id=3580&amp;di=1" width="5px" height="5px"><img src="pymol_images/3580.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3580&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3580.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3581">3581</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9048567">9048567</a></td><td align="right">1997</td><td>GIGAVLKVLTTGLPALISWIKRKRQQ</td><td>Melittin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antibacterial</td><td>100% hemolysis at 50μM</td><td>Human</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3581&amp;di=1" width="5px" height="5px"><img src="pymol_images/3581.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3581&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3581.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3582">3582</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9048567">9048567</a></td><td align="right">1997</td><td>GIGAvLKvLTTGLPALiSWIkRKRQQ</td><td>[D]-V<sup>5,8</sup>,I<sup>17</sup>,K<sup>21</sup>-melittin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Antibacterial</td><td>0% hemolysis at 50μM</td><td>Human</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3582&amp;di=1" width="5px" height="5px"><img src="pymol_images/3582.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3582&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3582.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3583">3583</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9048567">9048567</a></td><td align="right">1997</td><td>GIGAvLKvLTTGLPALiSWIkRKRQQ</td><td>[D]-V<sup>5,8</sup>,I<sup>17</sup>,K<sup>21</sup>-melittin</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">26</td><td>Antibacterial</td><td>~1-2% hemolysis at 50μM</td><td>Human</td><td>CCCCCTHHHHTCCHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3583&amp;di=1" width="5px" height="5px"><img src="pymol_images/3583.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3583&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3583.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3584">3584</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9335545">9335545</a></td><td align="right">1997</td><td>GVAKFGKAAAHFGKGWIKEMLNS</td><td>80°M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>EC<sub>25</sub>=307μM</td><td>Human</td><td>CTTSCHHHHTSSCCCHHHHTTTC</td><td align="center"><a href="jmol.php?id=3584&amp;di=1" width="5px" height="5px"><img src="pymol_images/3584.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3584&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3584.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3585">3585</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9335545">9335545</a></td><td align="right">1997</td><td>GIAKFGKAAAHFGKKWVGELMNS</td><td>100°M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>EC<sub>25</sub>=159μM</td><td>Human</td><td>CCGGGCCTHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3585&amp;di=1" width="5px" height="5px"><img src="pymol_images/3585.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3586">3586</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9335545">9335545</a></td><td align="right">1997</td><td>GIGKFLHSAKKFGKAWVGEIMNS</td><td>120°M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>EC<sub>25</sub>=271μM</td><td>Human</td><td>CCTTHHHHHHTTGGGGGTTTTCC</td><td align="center"><a href="jmol.php?id=3586&amp;di=1" width="5px" height="5px"><img src="pymol_images/3586.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3587">3587</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9335545">9335545</a></td><td align="right">1997</td><td>GIGKFLHTLKTFGKKWVGEIMNS</td><td>140°M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>EC<sub>25</sub>=15μM</td><td>Human</td><td>CTTHHHHHHHHTTTTHHHHGGGC</td><td align="center"><a href="jmol.php?id=3587&amp;di=1" width="5px" height="5px"><img src="pymol_images/3587.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3588">3588</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9335545">9335545</a></td><td align="right">1997</td><td>GIGKFLHKVKSFGKSWIGEIMNS</td><td>160°M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>EC<sub>25</sub>=20μM</td><td>Human</td><td>CCCSHHHHHHSSSSCTHHHHTTC</td><td align="center"><a href="jmol.php?id=3588&amp;di=1" width="5px" height="5px"><img src="pymol_images/3588.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3588&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3588.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3589">3589</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9335545">9335545</a></td><td align="right">1997</td><td>GIGKFLHKVGSFIKSWKGEIMNS</td><td>180°M2a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>EC<sub>25</sub>=24μM</td><td>Human</td><td>CHHHHHHHHHHHHHHGGGTSSCC</td><td align="center"><a href="jmol.php?id=3589&amp;di=1" width="5px" height="5px"><img src="pymol_images/3589.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3590">3590</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10387056">10387056</a></td><td align="right">1999</td><td>KWKSFIKKLTSAAKKVVTTAKPLISS</td><td>CP26<sub>p</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC =8μg/ml</td><td>Human</td><td>CCGGGHHHHHHHHHHHTTTSCCSSCC</td><td align="center"><a href="jmol.php?id=3590&amp;di=1" width="5px" height="5px"><img src="pymol_images/3590.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3590&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3590.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3591">3591</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10387056">10387056</a></td><td align="right">1999</td><td>KWKSFLKTFKSAVKTVLHTALKAISS</td><td>V681<sub>n</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC &gt;128μg/ml</td><td>Human</td><td>CCTTTTTHHHHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3591&amp;di=1" width="5px" height="5px"><img src="pymol_images/3591.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3591&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3591.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3592">3592</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10387056">10387056</a></td><td align="right">1999</td><td>KWKSFLRTLKSPAKTVFHTALKAISS</td><td>V25<sub>p</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC &gt;128μg/ml</td><td>Human</td><td>CCTTGGGGTCCCCTTHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3592&amp;di=1" width="5px" height="5px"><img src="pymol_images/3592.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3592&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3592.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3593">3593</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10387056">10387056</a></td><td align="right">1999</td><td>KWKSFLKKLTSAAKKVLTTALKPISS</td><td>V26<sub>p</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC=64μg/ml</td><td>Human</td><td>CHHHHHHHHHHHHHHHHHHTSCCSCC</td><td align="center"><a href="jmol.php?id=3593&amp;di=1" width="5px" height="5px"><img src="pymol_images/3593.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3593&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3593.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3594">3594</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10387056">10387056</a></td><td align="right">1999</td><td>KWKSFLKTLKSPVKTVFYTALKPISS</td><td>V1<sub>pp</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC &gt;128μg/ml</td><td>Human</td><td>CCTTTGGGTSCSSSSCCSSCCCSCCC</td><td align="center"><a href="jmol.php?id=3594&amp;di=1" width="5px" height="5px"><img src="pymol_images/3594.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3594&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3594.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3595">3595</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10387056">10387056</a></td><td align="right">1999</td><td>KWKSFLRTFKSPVRTVFHTALKPISS</td><td>V8<sub>pp</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC =32μg/ml</td><td>Human</td><td>CCHHHHHTTCCCCHHHHTTSCSSCCC</td><td align="center"><a href="jmol.php?id=3595&amp;di=1" width="5px" height="5px"><img src="pymol_images/3595.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3595&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3595.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3596">3596</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10387056">10387056</a></td><td align="right">1999</td><td>KWKSFLKTFKSPVKTVFYTALKPISS</td><td>V14<sub>pp</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC &gt;128μg/ml</td><td>Human</td><td>CCGGGTTTTSCSSSCCSTTTTSSSCC</td><td align="center"><a href="jmol.php?id=3596&amp;di=1" width="5px" height="5px"><img src="pymol_images/3596.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3596&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3596.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3597">3597</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10387056">10387056</a></td><td align="right">1999</td><td>KWKSFLKTFKSPARTVLYTALKPISS</td><td>V68<sub>pp</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC &gt;128μg/ml</td><td>Human</td><td>CCHHHHTTSCSCCGGGCHHHHCCCCC</td><td align="center"><a href="jmol.php?id=3597&amp;di=1" width="5px" height="5px"><img src="pymol_images/3597.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3597&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3597.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3598">3598</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10387056">10387056</a></td><td align="right">1999</td><td>KWKSFLKTFKSPARTVLHTALKPISS</td><td>V31<sub>pp</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC &gt;128μg/ml</td><td>Human</td><td>CCHHHHHSCCSSSCCSGGGSSCSSCC</td><td align="center"><a href="jmol.php?id=3598&amp;di=1" width="5px" height="5px"><img src="pymol_images/3598.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3598&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3598.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3599">3599</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7479693">7479693</a></td><td align="right">1995</td><td>IISTIGKLVKWIIDTV</td><td>Peptide-2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>100% hemolysis at ~25μM</td><td>Guinea-pig</td><td>CCSSHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=3599&amp;di=1" width="5px" height="5px"><img src="pymol_images/3599.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3600">3600</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7479693">7479693</a></td><td align="right">1995</td><td>IISTIGDLVKWIIKTV</td><td>Peptide-3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>1-2% hemolysis at 20μM</td><td>Guinea-pig</td><td>CCCCHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=3600&amp;di=1" width="5px" height="5px"><img src="pymol_images/3600.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3600&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3600.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3601">3601</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7479693">7479693</a></td><td align="right">1995</td><td>IISTIGDLVKWIIKTV</td><td>Peptide-3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>&gt;50% hemolysis at 40μM</td><td>Guinea-pig</td><td>CCCCHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=3601&amp;di=1" width="5px" height="5px"><img src="pymol_images/3601.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3601&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3601.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3602">3602</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7479693">7479693</a></td><td align="right">1995</td><td>IISTIGDLVKWIIKTV</td><td>Peptide-3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>80% hemolysis at 60μM</td><td>Guinea-pig</td><td>CCCCHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=3602&amp;di=1" width="5px" height="5px"><img src="pymol_images/3602.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3602&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3602.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3603">3603</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7479693">7479693</a></td><td align="right">1995</td><td>IISTIGDLVKWIIKTV</td><td>Peptide-3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>~90% hemolysis at 80μM</td><td>Guinea-pig</td><td>CCCCHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=3603&amp;di=1" width="5px" height="5px"><img src="pymol_images/3603.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3603&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3603.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3604">3604</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7479693">7479693</a></td><td align="right">1995</td><td>IISTIGDLVKWIIKTV</td><td>Peptide-3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>100% hemolysis at ~90μM</td><td>Guinea-pig</td><td>CCCCHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=3604&amp;di=1" width="5px" height="5px"><img src="pymol_images/3604.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3604&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3604.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3605">3605</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7479693">7479693</a></td><td align="right">1995</td><td>IISTIGKLVKWIIKTV</td><td>Peptide-4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>50% hemolysis at ~15μM</td><td>Guinea-pig</td><td>CCCCHHHHHHHHCCCC</td><td align="center"><a href="jmol.php?id=3605&amp;di=1" width="5px" height="5px"><img src="pymol_images/3605.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3605&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3605.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3606">3606</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7479693">7479693</a></td><td align="right">1995</td><td>IISTIGKLVKWIIKTV</td><td>Peptide-4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>100% hemolysis at ~20μM</td><td>Guinea-pig</td><td>CCCCHHHHHHHHCCCC</td><td align="center"><a href="jmol.php?id=3606&amp;di=1" width="5px" height="5px"><img src="pymol_images/3606.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3606&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3606.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3607">3607</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22074926">22074926</a></td><td align="right">2012</td><td>CGESCVFIPCITSLAGCSCKNKVCYYDGGSVP</td><td>Parigidin_br1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>41% hemolysis at 40µM </td><td>Human</td><td>CCCCCCCCSSSSSSSSSCCCCCCCCCTTTSCC</td><td align="center"><a href="jmol.php?id=3607&amp;di=1" width="5px" height="5px"><img src="pymol_images/3607.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3607&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3607.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3608">3608</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22074926">22074926</a></td><td align="right">2012</td><td>CGESCVFIPCITSLAGCSCKNKVCYYDGGSVP</td><td>Parigidin_br1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>28% hemolysis at 20µM</td><td>Human</td><td>CCCCCCCCSSSSSSSSSCCCCCCCCCTTTSCC</td><td align="center"><a href="jmol.php?id=3608&amp;di=1" width="5px" height="5px"><img src="pymol_images/3608.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3608&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3608.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3609">3609</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21956830">21956830</a></td><td align="right">2011</td><td>FLKPLFNAALKLLP</td><td>Temporin-Ra</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>1.3% hemolysis at 60µg/ml</td><td>Human</td><td>CCCGGGHHHHTTCC</td><td align="center"><a href="jmol.php?id=3609&amp;di=1" width="5px" height="5px"><img src="pymol_images/3609.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3609&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3609.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3610">3610</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21956830">21956830</a></td><td align="right">2011</td><td>FLPVLAGVLSRA</td><td>Temporin-Rb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>1.1% hemolysis at 60µg/ml</td><td>Human</td><td>CCCCCBTTBCCC</td><td align="center"><a href="jmol.php?id=3610&amp;di=1" width="5px" height="5px"><img src="pymol_images/3610.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3610&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3610.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3612">3612</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7744058">7744058</a></td><td align="right">1995</td><td>KIAEKFSGTRRG</td><td>CGB</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>No hemolysis (Non hemolytic)</td><td>Rabbit</td><td>CHHHHCCGGGGC</td><td align="center"><a href="jmol.php?id=3612&amp;di=1" width="5px" height="5px"><img src="pymol_images/3612.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3612&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3612.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3613">3613</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19088182">19088182</a></td><td align="right">2009</td><td>IFGAIAGLLKNIF</td><td>Meucin-13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>37.7% hemolysis at 6.25µM</td><td>Rabbit</td><td>CCCTHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3613&amp;di=1" width="5px" height="5px"><img src="pymol_images/3613.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3613&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3613.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3614">3614</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19088182">19088182</a></td><td align="right">2009</td><td>IFGAIAGLLKNIF</td><td>Meucin-13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>59.2% hemolysis at 12.5µM</td><td>Rabbit</td><td>CCCTHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3614&amp;di=1" width="5px" height="5px"><img src="pymol_images/3614.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3614&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3614.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3615">3615</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19088182">19088182</a></td><td align="right">2009</td><td>IFGAIAGLLKNIF</td><td>Meucin-13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>&gt;90% hemolysis at 25µM</td><td>Rabbit</td><td>CCCTHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3615&amp;di=1" width="5px" height="5px"><img src="pymol_images/3615.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3615&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3615.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3616">3616</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19088182">19088182</a></td><td align="right">2009</td><td>FFGHLFKLATKIIPSLFQ</td><td>Meucin-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>74% hemolysis at 6.25µM</td><td>Rabbit</td><td>CTTHHHHHGGGGSSSSCC</td><td align="center"><a href="jmol.php?id=3616&amp;di=1" width="5px" height="5px"><img src="pymol_images/3616.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3616&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3616.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3617">3617</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19088182">19088182</a></td><td align="right">2009</td><td>FFGHLFKLATKIIPSLFQ</td><td>Meucin-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>&gt;90% hemolysis at 12.5µM</td><td>Rabbit</td><td>CTTHHHHHGGGGSSSSCC</td><td align="center"><a href="jmol.php?id=3617&amp;di=1" width="5px" height="5px"><img src="pymol_images/3617.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3617&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3617.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3618">3618</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19088182">19088182</a></td><td align="right">2009</td><td>FFGHLFKLATKIIPSLFQ</td><td>Meucin-18</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>&gt;95% hemolysis at 25µM</td><td>Rabbit</td><td>CTTHHHHHGGGGSSSSCC</td><td align="center"><a href="jmol.php?id=3618&amp;di=1" width="5px" height="5px"><img src="pymol_images/3618.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3618&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3618.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3619">3619</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19085906">19085906</a></td><td align="right">2008</td><td>FWGHIWNAVKRVGANALHGAVTGALS</td><td>Halocyntin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>22% hemolysis at 50µM</td><td>Sheep</td><td>CTTTHHHHHTTSEETTEESSCBSSCC</td><td align="center"><a href="jmol.php?id=3619&amp;di=1" width="5px" height="5px"><img src="pymol_images/3619.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3620">3620</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19085906">19085906</a></td><td align="right">2008</td><td>GFWKKVGSAAWGGVKAAAKGAAVGGLNALAKHIQ</td><td>Papillosin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antimicrobial</td><td>0% hemolysis upto 50µM (Non hemolytic)</td><td>Sheep</td><td>CCSSSSSSTTCSTHHHHHHSTTCCTHHHHGGGTC</td><td align="center"><a href="jmol.php?id=3620&amp;di=1" width="5px" height="5px"><img src="pymol_images/3620.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3620&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3620.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3621">3621</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVLPKVMAHMK</td><td>MEP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100µM</td><td>Rat</td><td>CCHHHHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=3621&amp;di=1" width="5px" height="5px"><img src="pymol_images/3621.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3621&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3621.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3622">3622</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVLKKVMAHMK</td><td>MEP-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =27.3µM</td><td>Rat</td><td>CCHHHHTTTHHHHHSSCC</td><td align="center"><a href="jmol.php?id=3622&amp;di=1" width="5px" height="5px"><img src="pymol_images/3622.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3622&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3622.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3623">3623</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVLKVMAHMK</td><td>MEP-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>LC<sub>50</sub> =22.9µM</td><td>Rat</td><td>CCHHHHHHHHHHHSSCC</td><td align="center"><a href="jmol.php?id=3623&amp;di=1" width="5px" height="5px"><img src="pymol_images/3623.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3623&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3623.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3624">3624</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVLAKVMAHMK</td><td>MEP-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =29.7µM</td><td>Rat</td><td>CCHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3624&amp;di=1" width="5px" height="5px"><img src="pymol_images/3624.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3624&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3624.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3625">3625</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVLGKVMAHMK</td><td>MEP-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =41.4µM</td><td>Rat</td><td>CCHHHHHHHHHHHHTSCC</td><td align="center"><a href="jmol.php?id=3625&amp;di=1" width="5px" height="5px"><img src="pymol_images/3625.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3625&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3625.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3626">3626</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>KVMAHMK</td><td>MEP-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">7</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100µM</td><td>Rat</td><td>CCCCCCC</td><td align="center"><a href="jmol.php?id=3626&amp;di=1" width="5px" height="5px"><img src="pymol_images/3626.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3626&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3626.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3627">3627</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>PKVMAHMK</td><td>MEP-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">8</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100µM</td><td>Rat</td><td>CCCCSSCC</td><td align="center"><a href="jmol.php?id=3627&amp;di=1" width="5px" height="5px"><img src="pymol_images/3627.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3627&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3627.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3628">3628</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>LPKVMAHMK</td><td>MEP-7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100µM</td><td>Rat</td><td>CTTSSSSCC</td><td align="center"><a href="jmol.php?id=3628&amp;di=1" width="5px" height="5px"><img src="pymol_images/3628.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3628&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3628.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3629">3629</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>VLPKVMAHMK</td><td>MEP-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100µM</td><td>Rat</td><td>CCGGGTSSCC</td><td align="center"><a href="jmol.php?id=3629&amp;di=1" width="5px" height="5px"><img src="pymol_images/3629.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3629&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3629.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3630">3630</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVLP</td><td>MEP-9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100µM</td><td>Rat</td><td>CCHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3630&amp;di=1" width="5px" height="5px"><img src="pymol_images/3630.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3630&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3630.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3631">3631</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18942691">18942691</a></td><td align="right">2008</td><td>GFLSILKKVL</td><td>MEP-10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100µM</td><td>Rat</td><td>CCHHHHTTTC</td><td align="center"><a href="jmol.php?id=3631&amp;di=1" width="5px" height="5px"><img src="pymol_images/3631.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3631&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3631.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3634">3634</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18000874">18000874</a></td><td align="right">2007</td><td>GLLNGLALRLGKRALKKIIKRLCR</td><td>Cryptonin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>5% hemolysis upto 200µg/ml</td><td>Rat</td><td>CCSSHHHHHTHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3634&amp;di=1" width="5px" height="5px"><img src="pymol_images/3634.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3634&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3634.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3635">3635</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17927208">17927208</a></td><td align="right">2007</td><td>FHPSLWVLIPQYIQLIRKILKSG</td><td>Conolysin-Mt</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Cytolytic</td><td>40% hemolysis at 1µM</td><td>Human</td><td>CCSCCCSSHHHHHHHHTTTTTCC</td><td align="center"><a href="jmol.php?id=3635&amp;di=1" width="5px" height="5px"><img src="pymol_images/3635.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3635&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3635.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3636">3636</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17927208">17927208</a></td><td align="right">2007</td><td>FHPSLWVLIPQYIQLIRKILKSG</td><td>Conolysin-Mt</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Cytolytic</td><td>~90% hemolysis at 10µM</td><td>Human</td><td>CCSCCCSSHHHHHHHHTTTTTCC</td><td align="center"><a href="jmol.php?id=3636&amp;di=1" width="5px" height="5px"><img src="pymol_images/3636.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3636&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3636.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3637">3637</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17884127">17884127</a></td><td align="right">2007</td><td>GLLDFVTGVGKDIFAQLIKQI</td><td>Ocellatin 4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Cytolytic</td><td>HC<sub>50</sub> =14.3 mM</td><td>Human</td><td>CCCCSSSCTTGGGTCGGGGTC</td><td align="center"><a href="jmol.php?id=3637&amp;di=1" width="5px" height="5px"><img src="pymol_images/3637.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3637&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3637.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3638">3638</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17764786">17764786</a></td><td align="right">2007</td><td>SIITMTKEAKLPQLWKQIACRLYNTC</td><td>Pleurain-A1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>Little hemolysis at 200mg/ml</td><td>Human</td><td>CCSSTTSSSSCHHHHHHHTTTTSSCC</td><td align="center"><a href="jmol.php?id=3638&amp;di=1" width="5px" height="5px"><img src="pymol_images/3638.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3638&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3638.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3639">3639</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17764786">17764786</a></td><td align="right">2007</td><td>SIITMTKEAKLPQSWKQIACRLYNTC</td><td>Pleurain-A2</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>Little hemolysis at 200mg/ml</td><td>Human</td><td>CCCCTTSCSCCCCCHHHHGGGTSCCC</td><td align="center"><a href="jmol.php?id=3639&amp;di=1" width="5px" height="5px"><img src="pymol_images/3639.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3639&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3639.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3640">3640</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17764786">17764786</a></td><td align="right">2007</td><td>SIITMTREAKLPQLWKQIACRLYNTC</td><td>Pleurain-A3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>Little hemolysis at 200mg/ml</td><td>Human</td><td>CCTTTTSCSSCHHHHHHHHHTTSCCC</td><td align="center"><a href="jmol.php?id=3640&amp;di=1" width="5px" height="5px"><img src="pymol_images/3640.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3640&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3640.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3641">3641</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17764786">17764786</a></td><td align="right">2007</td><td>SIITTTKEAKLPQLWKQIACRLYNTC</td><td>Pleurain-A4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>Little hemolysis at 200mg/ml</td><td>Human</td><td>CCCCTTCCSSCHHHHHHHHHTTSCCC</td><td align="center"><a href="jmol.php?id=3641&amp;di=1" width="5px" height="5px"><img src="pymol_images/3641.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3641&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3641.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3642">3642</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17698252">17698252</a></td><td align="right">2007</td><td>GLLRASSVWGRKYYVDLAGCAKA</td><td>Odorranin-HP</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>Little hemolysis at 100µg/ml</td><td>Rabbit</td><td>CCCCCCSSSCCCCSCCSSSSSCC</td><td align="center"><a href="jmol.php?id=3642&amp;di=1" width="5px" height="5px"><img src="pymol_images/3642.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3642&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3642.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3643">3643</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17688900">17688900</a></td><td align="right">2007</td><td>FLSLALAALPKFLCLVFKKC</td><td>Brevinin-1DYa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>LC<sub>50</sub> &lt;5µM</td><td>Human</td><td>CHHHHHHHSTTTTCCGGGCC</td><td align="center"><a href="jmol.php?id=3643&amp;di=1" width="5px" height="5px"><img src="pymol_images/3643.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3643&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3643.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3644">3644</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17688900">17688900</a></td><td align="right">2007</td><td>FLSLALAALPKLFCLIFKKC</td><td>Brevinin-1DYb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>LC<sub>50</sub> &lt;5µM</td><td>Human</td><td>CHHHHHTTTTTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3644&amp;di=1" width="5px" height="5px"><img src="pymol_images/3644.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3644&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3644.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3645">3645</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17688900">17688900</a></td><td align="right">2007</td><td>FLPLLLAGLPKLLCLFFKKC</td><td>Brevinin-1DYc</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>LC<sub>50</sub> &lt;5µM</td><td>Human</td><td>CHHHHHSSSCTTTSSSSCCC</td><td align="center"><a href="jmol.php?id=3645&amp;di=1" width="5px" height="5px"><img src="pymol_images/3645.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3645&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3645.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3646">3646</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17272268">17272268</a></td><td align="right">2007</td><td>GFMDTAKNVAKNVAVTLLDNLKCKITKAC</td><td>Odorranain-F1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>15.4±3.5% hemolysis at 50µg/ml</td><td>Human</td><td>CCSCHHHHHHTTCSBTTBCSCCCTTCSCC</td><td align="center"><a href="jmol.php?id=3646&amp;di=1" width="5px" height="5px"><img src="pymol_images/3646.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3646&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3646.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3647">3647</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17272268">17272268</a></td><td align="right">2007</td><td>GLLSGTSVRGSI</td><td>Odorranain-V1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>23.9±4.7% hemolysis at 50µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3648">3648</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17272268">17272268</a></td><td align="right">2007</td><td>GLFGKSSVWGRKYYVDLAGCAKA</td><td>Odorranain-W1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>17.6±2.2% hemolysis at 50µg/ml</td><td>Human</td><td>CCSSCCCSSTTCSSCTTSSSCCC</td><td align="center"><a href="jmol.php?id=3648&amp;di=1" width="5px" height="5px"><img src="pymol_images/3648.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3648&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3648.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3649">3649</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17272268">17272268</a></td><td align="right">2007</td><td>GIFGKILGVGKKVLCGLSGVC</td><td>Odorranain-H1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>21.6±7.3% hemolysis at 50µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3650">3650</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17272268">17272268</a></td><td align="right">2007</td><td>GLLGKILGVEKKVLCGLSGMC</td><td>Nigrocin-OG2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>60±7.1% hemolysis at 50µg/ml</td><td>Human</td><td>CGGGGCCTTCHHHHTTSSSCC</td><td align="center"><a href="jmol.php?id=3650&amp;di=1" width="5px" height="5px"><img src="pymol_images/3650.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3651">3651</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17272268">17272268</a></td><td align="right">2007</td><td>GLLSGILGAGKHIICGLSGLC</td><td>Nigrocin-OG4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>65.5±7.7% hemolysis at 50µg/ml</td><td>Human</td><td>CCCCCCCCCCCCCCCSSSSCC</td><td align="center"><a href="jmol.php?id=3651&amp;di=1" width="5px" height="5px"><img src="pymol_images/3651.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3651&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3651.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3652">3652</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17272268">17272268</a></td><td align="right">2007</td><td>GLLSGILGAGKQKVCGLSGLC</td><td>Nigrocin-OG5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>87.2±2.6% hemolysis at 50µg/ml</td><td>Human</td><td>CCSSSCCSSCCCCCCSSSSCC</td><td align="center"><a href="jmol.php?id=3652&amp;di=1" width="5px" height="5px"><img src="pymol_images/3652.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3652&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3652.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3653">3653</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17272268">17272268</a></td><td align="right">2007</td><td>GLLSGVLGVGKKVLCGLSGLC</td><td>Nigrocin-OG21</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>100% hemolysis at 50µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3655">3655</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8849687">8849687</a></td><td align="right">1996</td><td>ILPWKWPWWPWRR</td><td>Indolicidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>~20% hemolysis at 10µM</td><td>Rat</td><td>CCCCCCSSCSSCC</td><td align="center"><a href="jmol.php?id=3655&amp;di=1" width="5px" height="5px"><img src="pymol_images/3655.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3655&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3655.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3656">3656</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8849687">8849687</a></td><td align="right">1996</td><td>ILPWKWPWWPWRR</td><td>Indolicidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>~90% hemolysis at 20µM</td><td>Rat</td><td>CCCCCCSSCSSCC</td><td align="center"><a href="jmol.php?id=3656&amp;di=1" width="5px" height="5px"><img src="pymol_images/3656.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3656&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3656.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3657">3657</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8849687">8849687</a></td><td align="right">1996</td><td>ILPWKWPWWPWRR</td><td>Indolicidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>~100% hemolysis at 30µM</td><td>Rat</td><td>CCCCCCSSCSSCC</td><td align="center"><a href="jmol.php?id=3657&amp;di=1" width="5px" height="5px"><img src="pymol_images/3657.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3657&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3657.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3658">3658</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8849687">8849687</a></td><td align="right">1996</td><td>ILAWKWAWWAWRR</td><td>ILA</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>~20% hemolysis at 5µM</td><td>Rat</td><td>CGGGCHHHHTCCC</td><td align="center"><a href="jmol.php?id=3658&amp;di=1" width="5px" height="5px"><img src="pymol_images/3658.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3658&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3658.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3659">3659</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8849687">8849687</a></td><td align="right">1996</td><td>ILAWKWAWWAWRR</td><td>ILA</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>100% hemolysis at 20µg/ml</td><td>Rat</td><td>CGGGCHHHHTCCC</td><td align="center"><a href="jmol.php?id=3659&amp;di=1" width="5px" height="5px"><img src="pymol_images/3659.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3659&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3659.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3660">3660</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8849687">8849687</a></td><td align="right">1996</td><td>ILPFKFPFFPFRR</td><td>ILF</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antibacterial</td><td>~20% hemolysis upto 80µg/ml</td><td>Rat</td><td>CCCCCCSCCCCCC</td><td align="center"><a href="jmol.php?id=3660&amp;di=1" width="5px" height="5px"><img src="pymol_images/3660.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3660&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3660.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3661">3661</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8843284">8843284</a></td><td align="right">1996</td><td>KFFKFFKFF</td><td>Synthetic peptide</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antibacterial</td><td>Strongly hemolytic at 40µg/ml</td><td>Human</td><td>CCCCCCCCC</td><td align="center"><a href="jmol.php?id=3661&amp;di=1" width="5px" height="5px"><img src="pymol_images/3661.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3661&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3661.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3662">3662</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>fPVOLfPVOL</td><td>GS</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine</td><td align="right">10</td><td>Antibacterial</td><td>100% hemolysis at 39µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=3662&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3662.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3664">3664</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>yPKyPK</td><td>GS6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">6</td><td>Antibacterial</td><td>100% hemolysis at &gt;1600µg/ml</td><td>Human</td><td>CCCCCC</td><td align="center"><a href="jmol.php?id=3664&amp;di=1" width="5px" height="5px"><img src="pymol_images/3664.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3664&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3664.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3665">3665</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>yPVKyPKL</td><td>GS8</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">8</td><td>Antibacterial</td><td>100% hemolysis at &gt;1600µg/ml</td><td>Human</td><td>CCSSCSCC</td><td align="center"><a href="jmol.php?id=3665&amp;di=1" width="5px" height="5px"><img src="pymol_images/3665.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3665&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3665.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3666">3666</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>yPVKLyPVKL</td><td>GS10</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">10</td><td>Antibacterial</td><td>100% hemolysis at 67µg/ml</td><td>Human</td><td>CCSSCCSTTC</td><td align="center"><a href="jmol.php?id=3666&amp;di=1" width="5px" height="5px"><img src="pymol_images/3666.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3666&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3666.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3667">3667</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>yPVKLKyPKVKL</td><td>GS12</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>100% hemolysis at 250µg/ml</td><td>Human</td><td>CCCCCCCGGGCC</td><td align="center"><a href="jmol.php?id=3667&amp;di=1" width="5px" height="5px"><img src="pymol_images/3667.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3667&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3667.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3668">3668</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>yPVKLKVyPLKVKL</td><td>GS14</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antibacterial</td><td>100% hemolysis at 6µg/ml</td><td>Human</td><td>CCCCCCCCGGGSCC</td><td align="center"><a href="jmol.php?id=3668&amp;di=1" width="5px" height="5px"><img src="pymol_images/3668.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3668&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3668.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3669">3669</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>yPLKVKyPKLKV</td><td>GS12LV</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>100% hemolysis at 92µg/ml</td><td>Human</td><td>CCCCCCCSCCCC</td><td align="center"><a href="jmol.php?id=3669&amp;di=1" width="5px" height="5px"><img src="pymol_images/3669.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3669&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3669.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3670">3670</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>fPVKLKfPKVKL</td><td>GS12F</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antibacterial</td><td>100% hemolysis at 400µg/ml</td><td>Human</td><td>CCCSSSCTTCCC</td><td align="center"><a href="jmol.php?id=3670&amp;di=1" width="5px" height="5px"><img src="pymol_images/3670.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3670&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3670.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3671">3671</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>fPVOLOfPOVOL</td><td>GS12FO</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine</td><td align="right">12</td><td>Antibacterial</td><td>100% hemolysis at 160µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=3671&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3671.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3672">3672</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8810288">8810288</a></td><td align="right">1996</td><td>fPLOLOfPOLOL</td><td>GS12FO/LL</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>Ornithine</td><td align="right">12</td><td>Antibacterial</td><td>100% hemolysis at 170µg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=3672&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3672.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3673">3673</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAVGSALSSSGGQE</td><td>Pardaxin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>&gt;20% hemolysis at 25µM</td><td>Human</td><td>CCCSSCHHHHTSSTTHHHHHHHHHHHTTCSSCC</td><td align="center"><a href="jmol.php?id=3673&amp;di=1" width="5px" height="5px"><img src="pymol_images/3673.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3673&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3673.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3674">3674</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAVGSALSSSGGQE</td><td>Pardaxin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>&gt;50% hemolysis at 50µM</td><td>Human</td><td>CCCSSCHHHHTSSTTHHHHHHHHHHHTTCSSCC</td><td align="center"><a href="jmol.php?id=3674&amp;di=1" width="5px" height="5px"><img src="pymol_images/3674.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3674&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3674.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3675">3675</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAVGSALSSSGGQE</td><td>Pardaxin- [NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antibacterial</td><td>&gt;100% hemolysis at 12µM</td><td>Human</td><td>CCCSSCHHHHTSSTTHHHHHHHHHHHTTCSSCC</td><td align="center"><a href="jmol.php?id=3675&amp;di=1" width="5px" height="5px"><img src="pymol_images/3675.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3676">3676</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAV</td><td>Des-(23-33)-pardaxin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antibacterial</td><td>&gt;100% hemolysis at 10µM</td><td>Human</td><td>CCCCCCHHHHTSHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3676&amp;di=1" width="5px" height="5px"><img src="pymol_images/3676.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3676&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3676.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3677">3677</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAV</td><td>Des-(23-33)-pardaxin-[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antibacterial</td><td>&lt;20% hemolysis at 50µM</td><td>Human</td><td>CCCCCCHHHHTSHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3677&amp;di=1" width="5px" height="5px"><img src="pymol_images/3677.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3678">3678</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>KGFFALIPKIISSPLFKTLLSAV</td><td>Lys-des-(23- 33)-pardaxin-[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antibacterial</td><td>No hemolysis (Non hemolytic)</td><td>Human</td><td>CCSCCSCSSSCCCHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3678&amp;di=1" width="5px" height="5px"><img src="pymol_images/3678.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3679">3679</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>ISSPLFKTLLSAVGSALSSSGGQE</td><td>Des-(I -9)-pardaxin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antibacterial</td><td>No hemolysis (Non hemolytic)</td><td>Human</td><td>CCCHHHHHHHHHHHHHTSCCCCCC</td><td align="center"><a href="jmol.php?id=3679&amp;di=1" width="5px" height="5px"><img src="pymol_images/3679.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3679&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3679.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3680">3680</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>ISSPLFKTLLSAVGSALSSSGGQE</td><td>Des-(I -9)-pardaxin-[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antibacterial</td><td>No hemolysis (Non hemolytic)</td><td>Human</td><td>CCCHHHHHHHHHHHHHTSCCCCCC</td><td align="center"><a href="jmol.php?id=3680&amp;di=1" width="5px" height="5px"><img src="pymol_images/3680.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3681">3681</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIIS</td><td>Des-(12-33)-pardaxin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antibacterial</td><td>No hemolysis (Non hemolytic)</td><td>Human</td><td>CCCCCCCTTCC</td><td align="center"><a href="jmol.php?id=3681&amp;di=1" width="5px" height="5px"><img src="pymol_images/3681.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3681&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3681.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3682">3682</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8620888">8620888</a></td><td align="right">1996</td><td>GFFALIPKIIS</td><td>Des-(I 2-33)-pardaxin-[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>[NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub>]<sub>2</sub></td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antibacterial</td><td>No hemolysis (Non hemolytic)</td><td>Human</td><td>CCCCCCCTTCC</td><td align="center"><a href="jmol.php?id=3682&amp;di=1" width="5px" height="5px"><img src="pymol_images/3682.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3683">3683</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8508915">8508915</a></td><td align="right">1993</td><td>FLPLLAGLAANFLPKIFCKITRKC</td><td>Brevinin-1 E</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>LC<sub>50</sub> =5µM</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHHCSCC</td><td align="center"><a href="jmol.php?id=3683&amp;di=1" width="5px" height="5px"><img src="pymol_images/3683.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3684">3684</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8508915">8508915</a></td><td align="right">1993</td><td>GIMDTLKNLAKTAGKGALQSLLNKASCKLSGQC</td><td>Brevinin-2 E</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;100µM</td><td>Human</td><td>CCTTHHHHHHHTTCCHHHHHHHTTSCSTTSSCC</td><td align="center"><a href="jmol.php?id=3684&amp;di=1" width="5px" height="5px"><img src="pymol_images/3684.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3684&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3684.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3686">3686</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8508915">8508915</a></td><td align="right">1993</td><td>FLPAIAGILSQLF</td><td>A1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LC<sub>50</sub> ~20µM</td><td>Human</td><td>CCCHHHHTTTSCC</td><td align="center"><a href="jmol.php?id=3686&amp;di=1" width="5px" height="5px"><img src="pymol_images/3686.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3686&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3686.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3687">3687</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8045297">8045297</a></td><td align="right">1994</td><td>KLLLKLLKLLLKLLLKLLK</td><td>L<sub>13</sub>K<sub>6</sub></td><td>Dansylation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Cytotoxic</td><td>LD<sub>50</sub> ~2.1µM</td><td>Human</td><td>CTHHHHHHHHHHHHHHSCC</td><td align="center"><a href="jmol.php?id=3687&amp;di=1" width="5px" height="5px"><img src="pymol_images/3687.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3687&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3687.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3688">3688</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8045297">8045297</a></td><td align="right">1994</td><td>LLLKLLKLLLKLLLKLLK</td><td>L<sub>13</sub>K<sub>5</sub></td><td>Dansylation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Cytotoxic</td><td>LD<sub>50</sub> =4.7µM</td><td>Human</td><td>CHHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3688&amp;di=1" width="5px" height="5px"><img src="pymol_images/3688.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3688&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3688.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3689">3689</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8045297">8045297</a></td><td align="right">1994</td><td>KLLKLLLKLLLKLLK</td><td>L<sub>10</sub>K<sub>5</sub></td><td>Dansylation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Cytotoxic</td><td>LD<sub>50</sub> ~0.1µM</td><td>Human</td><td>CTHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3689&amp;di=1" width="5px" height="5px"><img src="pymol_images/3689.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3689&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3689.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3690">3690</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7989335">7989335</a></td><td align="right">1994</td><td>ALWKTMLKKLGTMALHAGKAALGAAADTISQGTQ</td><td>DS s1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antimicrobial</td><td>100% hemolysis at &gt;70µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3691">3691</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7989335">7989335</a></td><td align="right">1994</td><td>ALWFTMLKKLGTMALHAGKAALGAAANTISQGTQ</td><td>DS s2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antimicrobial</td><td>100% hemolysis at 70µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3692">3692</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7989335">7989335</a></td><td align="right">1994</td><td>ALWKNMLKGIGKLAGKAALGAVKKLVGAES</td><td>DS s3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">30</td><td>Antimicrobial</td><td>100% hemolysis at 80µM</td><td>Human</td><td>CTHHHHTTTTTTCHHHHTTHHHHTTSCCCC</td><td align="center"><a href="jmol.php?id=3692&amp;di=1" width="5px" height="5px"><img src="pymol_images/3692.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3692&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3692.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3693">3693</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7989335">7989335</a></td><td align="right">1994</td><td>ALWMTLLKKVLKAAAKAALNAVLVGANA</td><td>DS s4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>100% hemolysis at 1µM</td><td>Human</td><td>CHHHHHHHHHHHHHTTTGGGSSCCCSCC</td><td align="center"><a href="jmol.php?id=3693&amp;di=1" width="5px" height="5px"><img src="pymol_images/3693.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3693&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3693.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3694">3694</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7989335">7989335</a></td><td align="right">1994</td><td>GLWSKIKTAGKSVAKAAAKAAVKAVTNAV</td><td>DS s5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>100% hemolysis at &gt;90µM</td><td>Human</td><td>CCGGGHHHHTGGGHHHHHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3694&amp;di=1" width="5px" height="5px"><img src="pymol_images/3694.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3694&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3694.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3699">3699</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7999137">7999137</a></td><td align="right">1994</td><td>SLFSLIKAGAKFLGKNLLKQGACYAACKASKQC</td><td>Gaegurin 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>0.59% hemolysis at 100µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3700">3700</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7999137">7999137</a></td><td align="right">1994</td><td>GIMSIVKDVAKNAAKEAAKGALSTLSCKLAKTC</td><td>Gaegurin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>0.82% hemolysis at 100µM</td><td>Human</td><td>CCCCHHHHHHHHHHHHHHTTTTTTSSTTTSCCC</td><td align="center"><a href="jmol.php?id=3700&amp;di=1" width="5px" height="5px"><img src="pymol_images/3700.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3700&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3700.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3701">3701</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7999137">7999137</a></td><td align="right">1994</td><td>GIMSIVKDVAKTAAKEAAKGALSTLSCKLAKTC</td><td>Gaegurin 3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>1.20% hemolysis at 100µM</td><td>Human</td><td>CCCCHHHHHHHHHHHHHHHSCCTTTTTTTCCCC</td><td align="center"><a href="jmol.php?id=3701&amp;di=1" width="5px" height="5px"><img src="pymol_images/3701.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3701&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3701.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3703">3703</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7999137">7999137</a></td><td align="right">1994</td><td>FLGALFKVASKVLPSVKCAITKKC</td><td>Gaegurin 5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>1.01% hemolysis at 100µM</td><td>Human</td><td>CTTTTHHHHTSSSSCCCCCGGGCC</td><td align="center"><a href="jmol.php?id=3703&amp;di=1" width="5px" height="5px"><img src="pymol_images/3703.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3703&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3703.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3704">3704</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/7999137">7999137</a></td><td align="right">1994</td><td>FLPLLAGLAANFLPTIICKISYKC</td><td>Gaegurin 6</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>0.86% hemolysis at 100µM</td><td>Human</td><td>CCCCHHHHHHCSSTTSCCCSSCCC</td><td align="center"><a href="jmol.php?id=3704&amp;di=1" width="5px" height="5px"><img src="pymol_images/3704.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3704&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3704.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3705">3705</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8353519">8353519</a></td><td align="right">1993</td><td>SIGSALKKALPVAKKIGKIALPIAKAALP</td><td>Ceratotoxin A</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antibacterial</td><td>Area of inhibition ~25mm2 at 5 nmol</td><td>Human</td><td>CTHHHHTTTCHHHHTTSBCCSSHHHHCBC</td><td align="center"><a href="jmol.php?id=3705&amp;di=1" width="5px" height="5px"><img src="pymol_images/3705.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3705&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3705.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3706">3706</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18295522">18295522</a></td><td align="right">2008</td><td>QLKVDLWGTRSGIQPEQHSSGKSDVRRWRSRY</td><td>Ixosin-B</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>Little hemolysis upto 200µg/ml</td><td>Rabbit</td><td>CCTTCBTTBCCSSCCSCCCSTTCTHHHHTCCC</td><td align="center"><a href="jmol.php?id=3706&amp;di=1" width="5px" height="5px"><img src="pymol_images/3706.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3706&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3706.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3707">3707</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19056441">19056441</a></td><td align="right">2009</td><td>IFGAILPLALGALKNLIK</td><td>Hy-a1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antibacterial</td><td>HC<sub>50</sub> =18µM</td><td>Human</td><td>CTTTSHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3707&amp;di=1" width="5px" height="5px"><img src="pymol_images/3707.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3707&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3707.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3708">3708</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19716842">19716842</a></td><td align="right">2010</td><td>FLPKMSTKLRVPYRRGTKDYH</td><td>Scolopin 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>25±2% hemolysis at 50µg/ml</td><td>Human</td><td>CCCCCCSCCCSSSCSCSSCCC</td><td align="center"><a href="jmol.php?id=3708&amp;di=1" width="5px" height="5px"><img src="pymol_images/3708.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3708&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3708.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3709">3709</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19716842">19716842</a></td><td align="right">2010</td><td>FLPKMSTKLRVPYRRGTKDYH</td><td>Scolopin 1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>21±5% hemolysis at 50µg/ml</td><td>Rabbit</td><td>CCCCCCSCCCSSSCSCSSCCC</td><td align="center"><a href="jmol.php?id=3709&amp;di=1" width="5px" height="5px"><img src="pymol_images/3709.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3709&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3709.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3710">3710</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19716842">19716842</a></td><td align="right">2010</td><td>GILKKFMLHRGTKVYKMRTLSKRSH</td><td>Scolopin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>32±7% hemolysis at 50µg/ml</td><td>Human</td><td>CHHHHHHHHHCCCCCCCCCSSSSCC</td><td align="center"><a href="jmol.php?id=3710&amp;di=1" width="5px" height="5px"><img src="pymol_images/3710.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3711">3711</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19716842">19716842</a></td><td align="right">2010</td><td>GILKKFMLHRGTKVYKMRTLSKRSH</td><td>Scolopin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>37±6% hemolysis at 50µg/ml</td><td>Rabbit</td><td>CHHHHHHHHHCCCCCCCCCSSSSCC</td><td align="center"><a href="jmol.php?id=3711&amp;di=1" width="5px" height="5px"><img src="pymol_images/3711.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3714">3714</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23415652">23415652</a></td><td align="right">2013</td><td>LLGMIPLAISAISALSKL</td><td>Medusin-AC</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>8.7% hemolysis at 32mg/ml</td><td>Horse</td><td>CCSSCHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3714&amp;di=1" width="5px" height="5px"><img src="pymol_images/3714.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3714&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3714.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3715">3715</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23415652">23415652</a></td><td align="right">2013</td><td>LLGMIPVAISAISALSKL</td><td>Medusin-PH</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>1.9% hemolysis at 128mg/ml</td><td>Horse</td><td>CCTTSTTHHHHHTTTCCC</td><td align="center"><a href="jmol.php?id=3715&amp;di=1" width="5px" height="5px"><img src="pymol_images/3715.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3715&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3715.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3716">3716</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23415652">23415652</a></td><td align="right">2013</td><td>LLGMIPLAISAISSLSKL</td><td>Medusin-PD</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>6.9% hemolysis at 64mg/ml</td><td>Horse</td><td>CCSSCGGGGHHHHSSCCC</td><td align="center"><a href="jmol.php?id=3716&amp;di=1" width="5px" height="5px"><img src="pymol_images/3716.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3716&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3716.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3717">3717</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGAILKHIIK</td><td>PNG-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> =119µM</td><td>Human</td><td>CCHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3717&amp;di=1" width="5px" height="5px"><img src="pymol_images/3717.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3717&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3717.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3718">3718</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>NLWAGILKHIIK</td><td>PNG-1/1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3718&amp;di=1" width="5px" height="5px"><img src="pymol_images/3718.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3718&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3718.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3719">3719</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>KNWGAILKHIIK</td><td>PNG-1/2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3719&amp;di=1" width="5px" height="5px"><img src="pymol_images/3719.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3719&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3719.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3720">3720</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LKWGAILKHIIK</td><td>PNG-1/3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CTTHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3720&amp;di=1" width="5px" height="5px"><img src="pymol_images/3720.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3720&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3720.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3721">3721</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNKGAILKHIIK</td><td>PNG-1/4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3721&amp;di=1" width="5px" height="5px"><img src="pymol_images/3721.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3721&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3721.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3722">3722</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWKAILKHIIK</td><td>PNG-1/5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> ~110µM</td><td>Human</td><td>CCHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3722&amp;di=1" width="5px" height="5px"><img src="pymol_images/3722.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3722&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3722.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3723">3723</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGKILKHIIK</td><td>PNG-1/6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;140µM</td><td>Human</td><td>CCHHHHCCTTTC</td><td align="center"><a href="jmol.php?id=3723&amp;di=1" width="5px" height="5px"><img src="pymol_images/3723.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3723&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3723.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3724">3724</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGAKLKHIIK</td><td>PNG-1/7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCCCTTSCCSCC</td><td align="center"><a href="jmol.php?id=3724&amp;di=1" width="5px" height="5px"><img src="pymol_images/3724.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3724&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3724.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3725">3725</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGAIKKHIIK</td><td>PNG-1/8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCTTTTTTCCCC</td><td align="center"><a href="jmol.php?id=3725&amp;di=1" width="5px" height="5px"><img src="pymol_images/3725.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3725&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3725.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3726">3726</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGAILKKIIK</td><td>PNG-1/9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3726&amp;di=1" width="5px" height="5px"><img src="pymol_images/3726.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3726&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3726.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3727">3727</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGAILKHKIK</td><td>PNG-1/10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCHHHHHTCSCC</td><td align="center"><a href="jmol.php?id=3727&amp;di=1" width="5px" height="5px"><img src="pymol_images/3727.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3727&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3727.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3728">3728</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGAILKHIKK</td><td>PNG-1/11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CHHHHTHHHHTC</td><td align="center"><a href="jmol.php?id=3728&amp;di=1" width="5px" height="5px"><img src="pymol_images/3728.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3728&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3728.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3729">3729</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>KNWGKILKHIIK</td><td>PNG-1/12</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCGGGTCHHHHC</td><td align="center"><a href="jmol.php?id=3729&amp;di=1" width="5px" height="5px"><img src="pymol_images/3729.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3729&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3729.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3730">3730</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>KNWKAILKHIIK</td><td>PNG-1/13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCTHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3730&amp;di=1" width="5px" height="5px"><img src="pymol_images/3730.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3730&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3730.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3731">3731</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGAVLKHVVK</td><td>PNG-1/14</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCTHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3731&amp;di=1" width="5px" height="5px"><img src="pymol_images/3731.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3731&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3731.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3732">3732</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGA-Abu-LKH-Abu-Abu-K</td><td>PNG-1/15</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Abu 2 =aminobutyric acid</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3733">3733</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGAALKHAAK</td><td>PNG-1/16</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3733&amp;di=1" width="5px" height="5px"><img src="pymol_images/3733.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3733&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3733.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3734">3734</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGAFLKHFFK</td><td>PNG-1/17</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> =81µM</td><td>Human</td><td>CCHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=3734&amp;di=1" width="5px" height="5px"><img src="pymol_images/3734.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3734&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3734.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3735">3735</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGAGLKHGGK</td><td>PNG-1/18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCCSSSCCCSCC</td><td align="center"><a href="jmol.php?id=3735&amp;di=1" width="5px" height="5px"><img src="pymol_images/3735.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3735&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3735.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3736">3736</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGALLKHLLK</td><td>PNG-1/19</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> =63µM</td><td>Human</td><td>CCHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3736&amp;di=1" width="5px" height="5px"><img src="pymol_images/3736.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3736&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3736.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3737">3737</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGA-Nle-LKH-Nle-Nle-K</td><td>PNG-1/20</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nle =norleucine</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> =40µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3738">3738</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGA-Aca-LKH-Aca-Aca-K</td><td>PNG-1/21</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Aca 1 =aminocyclohexane carboxylic acid</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> =43µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3739">3739</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LNWGAWLKHWWK</td><td>PNG-1/22</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> =132µM</td><td>Human</td><td>CCHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3739&amp;di=1" width="5px" height="5px"><img src="pymol_images/3739.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3739&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3739.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3740">3740</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LDVKKIICVACKIKPNPACKKICPK</td><td>PNG-K</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3741">3741</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LDVKKII-Abu-VA-Abu-KIKPNPA-Abu-KKI-Abu-PK</td><td>PNG-K/1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>Abu 2 =aminobutyric acid</td><td align="right">25</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3742">3742</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LDVKKIICVACKIRPNPACKKICPK</td><td>PNG-R</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>CCCTTCSCCCCCSSSCCCTTCCCCC</td><td align="center"><a href="jmol.php?id=3742&amp;di=1" width="5px" height="5px"><img src="pymol_images/3742.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3743">3743</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23483218">23483218</a></td><td align="right">2013</td><td>LDVKKII-Abu-VA-Abu-KIRPNPA-Abu-KKI-Abu-PK</td><td>PNG-R/1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>Abu 2 =aminobutyric acid</td><td align="right">25</td><td>Antimicrobial</td><td>LC<sub>50</sub> &gt;200µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3744">3744</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23562719">23562719</a></td><td align="right">2013</td><td>KAYSMPRCKGGFRAVMCWL</td><td>Shuchin 3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>1.3% hemolysis at 200µg/ml</td><td>Rabbit</td><td>CCSSTTCSCSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3744&amp;di=1" width="5px" height="5px"><img src="pymol_images/3744.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3744&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3744.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3745">3745</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23562719">23562719</a></td><td align="right">2013</td><td>KAYSTPRCKGLFRALMCWL</td><td>Shuchin 4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>1.2% hemolysis at 200µg/ml</td><td>Rabbit</td><td>CCSSSCTTHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3745&amp;di=1" width="5px" height="5px"><img src="pymol_images/3745.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3745&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3745.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3746">3746</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23562719">23562719</a></td><td align="right">2013</td><td>KAYSTPRCKYLFRAVLCWL</td><td>Shuchin 5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>1.6% hemolysis at 200µg/ml</td><td>Rabbit</td><td>CCSSSCSGGGHHHHGGGCC</td><td align="center"><a href="jmol.php?id=3746&amp;di=1" width="5px" height="5px"><img src="pymol_images/3746.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3746&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3746.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3747">3747</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23562719">23562719</a></td><td align="right">2013</td><td>KAYSMPRCKGGFRAVMCWL</td><td>Shuchin 3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>1.7% hemolysis at 200µg/ml</td><td>Human</td><td>CCSSTTCSCSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3747&amp;di=1" width="5px" height="5px"><img src="pymol_images/3747.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3747&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3747.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3748">3748</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23562719">23562719</a></td><td align="right">2013</td><td>KAYSTPRCKGLFRALMCWL</td><td>Shuchin 4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>2.5% hemolysis at 200µg/ml</td><td>Human</td><td>CCSSSCTTHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3748&amp;di=1" width="5px" height="5px"><img src="pymol_images/3748.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3748&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3748.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3749">3749</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/23562719">23562719</a></td><td align="right">2013</td><td>KAYSTPRCKYLFRAVLCWL</td><td>Shuchin 5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>1.5% hemolysis at 200µg/ml</td><td>Human</td><td>CCSSSCSGGGHHHHGGGCC</td><td align="center"><a href="jmol.php?id=3749&amp;di=1" width="5px" height="5px"><img src="pymol_images/3749.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3749&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3749.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3753">3753</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12354111">12354111</a></td><td align="right">2002</td><td>INLKALAALAKKIL</td><td>Mastoparan</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antibacterial and Antifungal</td><td>~10% hemolysis at 100µM</td><td>Human</td><td>CCHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3753&amp;di=1" width="5px" height="5px"><img src="pymol_images/3753.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3753&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3753.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3758">3758</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10606532">10606532</a></td><td align="right">1999</td><td>KWllLLKlLlLK</td><td>[D]L<sup>3,4,8,10</sup>-K<sub>3</sub>L<sub>8</sub>Wn</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>31% hemolysis at 50μM</td><td>Human</td><td>CHHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=3758&amp;di=1" width="5px" height="5px"><img src="pymol_images/3758.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3758&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3758.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3759">3759</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10606532">10606532</a></td><td align="right">1999</td><td>KLllLWKlLlLK</td><td>[D]L<sup>3,4,8,10</sup>-K<sub>3</sub>L<sub>8</sub>Wm</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>39.4% hemolysis at 50μM</td><td>Human</td><td>CCCHHHHHHTCC</td><td align="center"><a href="jmol.php?id=3759&amp;di=1" width="5px" height="5px"><img src="pymol_images/3759.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3759&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3759.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3760">3760</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10606532">10606532</a></td><td align="right">1999</td><td>KLllLLKlLlWK</td><td>[D]L<sup>3,4,8,10</sup>-K<sub>3</sub>L<sub>8</sub>Wc</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>44.2% hemolysis at 50μM</td><td>Human</td><td>CHHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=3760&amp;di=1" width="5px" height="5px"><img src="pymol_images/3760.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3760&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3760.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3761">3761</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10606532">10606532</a></td><td align="right">1999</td><td>KWllKLKlKlLK</td><td>[D]L<sup>3,4,8,10</sup>-K<sub>5</sub>L<sub>6</sub>Wn</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolysis at 50μM (Non hemolytic)</td><td>Human</td><td>CCCTTSGGGGTC</td><td align="center"><a href="jmol.php?id=3761&amp;di=1" width="5px" height="5px"><img src="pymol_images/3761.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3761&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3761.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3762">3762</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10606532">10606532</a></td><td align="right">1999</td><td>KLllKWKlKlLK</td><td>[D]L<sup>3,4,8,10</sup>-K<sub>5</sub>L<sub>6</sub>Wm</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolysis at 50μM (Non hemolytic)</td><td>Human</td><td>CCCHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3762&amp;di=1" width="5px" height="5px"><img src="pymol_images/3762.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3762&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3762.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3763">3763</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10606532">10606532</a></td><td align="right">1999</td><td>KLllKLKlKlWK</td><td>[D]L<sup>3,4,8,10</sup>-K<sub>5</sub>L<sub>6</sub>Wc</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolysis at 50μM (Non hemolytic)</td><td>Human</td><td>CCHHHHSTTCCC</td><td align="center"><a href="jmol.php?id=3763&amp;di=1" width="5px" height="5px"><img src="pymol_images/3763.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3763&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3763.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3764">3764</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10606532">10606532</a></td><td align="right">1999</td><td>KKWlKLKlKlKK</td><td>[D]L<sup>4,8,10</sup>-K<sub>7</sub>L<sub>4</sub>Wn</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolysis at 50μM (Non hemolytic)</td><td>Human</td><td>CTTCSCCSSCCC</td><td align="center"><a href="jmol.php?id=3764&amp;di=1" width="5px" height="5px"><img src="pymol_images/3764.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3764&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3764.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3765">3765</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10606532">10606532</a></td><td align="right">1999</td><td>KKllKWKlKlKK</td><td>[D]L<sup>3,4,8,10</sup>-K<sub>7</sub>L<sub>4</sub>Wm</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolysis at 50μM (Non hemolytic)</td><td>Human</td><td>CCCSCBTTBSCC</td><td align="center"><a href="jmol.php?id=3765&amp;di=1" width="5px" height="5px"><img src="pymol_images/3765.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3765&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3765.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3766">3766</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10606532">10606532</a></td><td align="right">1999</td><td>KKllKLKlKwKK</td><td>[D]L<sup>3,4,8</sup>-K<sub>7</sub>L<sub>4</sub>Wc</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolysis at 50μM (Non hemolytic)</td><td>Human</td><td>CCCTTCCCCCCC</td><td align="center"><a href="jmol.php?id=3766&amp;di=1" width="5px" height="5px"><img src="pymol_images/3766.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3766&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3766.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3767">3767</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10632060">10632060</a></td><td align="right">1999</td><td>KKVVFKVKFKK</td><td>MP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antifungal and Antibacterial</td><td>0% hemolysis upto 500μg/mL (Non hemolytic)</td><td>Mouse</td><td>CCTTCCCCCCC</td><td align="center"><a href="jmol.php?id=3767&amp;di=1" width="5px" height="5px"><img src="pymol_images/3767.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3767&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3767.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3768">3768</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10632060">10632060</a></td><td align="right">1999</td><td>kΨ[CH<sub>2</sub>NH]KVVFKVKFKk</td><td>MP1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antifungal and Antibacterial</td><td>0% hemolysis upto 500μg/mL (Non hemolytic)</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3769">3769</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10632060">10632060</a></td><td align="right">1999</td><td>KΨ[CH<sub>2</sub>OCONH]KVVFKVKFKk</td><td>MP6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">11</td><td>Antifungal and Antibacterial</td><td>0% hemolysis upto 500μg/mL (Non hemolytic)</td><td>Mouse</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3770">3770</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14636071">14636071</a></td><td align="right">2003</td><td>GALRGCWTKSYPPKPCK</td><td>Ranacyclin T</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>20% hemolysis at 100μM</td><td>Human</td><td>CCCCSSTTSCSCSSCCC</td><td align="center"><a href="jmol.php?id=3770&amp;di=1" width="5px" height="5px"><img src="pymol_images/3770.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3771">3771</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14636071">14636071</a></td><td align="right">2003</td><td>SAPRGCWTKSYPPKPCK</td><td>Ranacyclin E</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>55% hemolysis at 100μM</td><td>Human</td><td>CCSCCSCCCCCCSSCCC</td><td align="center"><a href="jmol.php?id=3771&amp;di=1" width="5px" height="5px"><img src="pymol_images/3771.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3772">3772</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/14636071">14636071</a></td><td align="right">2003</td><td>LVRGCWTKSYPPKPCFVR</td><td>pLR</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>85% hemolysis at 100μM</td><td>Human</td><td>CTTSSCSSSSSSSCSCCC</td><td align="center"><a href="jmol.php?id=3772&amp;di=1" width="5px" height="5px"><img src="pymol_images/3772.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3777">3777</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20656059">20656059</a></td><td align="right">2010</td><td>GMASKAGSIVGKIKIALGAL</td><td>PGLa-B2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>27% hemolysis at 200μM</td><td>Human</td><td>CTTSCSSSSSCCCBTTTBTC</td><td align="center"><a href="jmol.php?id=3777&amp;di=1" width="5px" height="5px"><img src="pymol_images/3777.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3777&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3777.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3778">3778</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20656059">20656059</a></td><td align="right">2010</td><td>GLGSLLGKAFKIGLKTVGKMMGGAPREQ</td><td>CPF-B1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>18% hemolysis at 200μM</td><td>Human</td><td>CGGGTHHHHHHHHHHHHHHHSSCSSSCC</td><td align="center"><a href="jmol.php?id=3778&amp;di=1" width="5px" height="5px"><img src="pymol_images/3778.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3780">3780</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21396976">21396976</a></td><td align="right">2011</td><td>SFPFFPPGICKRLKRC</td><td>Limnonectin-1Fa</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>0% hemolysis upto 160μM (Non hemolytic)</td><td>Horse</td><td>CCCCCCHHHHTTCCCC</td><td align="center"><a href="jmol.php?id=3780&amp;di=1" width="5px" height="5px"><img src="pymol_images/3780.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3780&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3780.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3781">3781</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21396976">21396976</a></td><td align="right">2011</td><td>SFHVFPPWMCKSLKKC</td><td>Limnonectin-1Fb</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>0% hemolysis upto 160μM (Non hemolytic)</td><td>Horse</td><td>CCSSCCSGGGGTCCCC</td><td align="center"><a href="jmol.php?id=3781&amp;di=1" width="5px" height="5px"><img src="pymol_images/3781.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3781&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3781.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3782">3782</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9009216">9009216</a></td><td align="right">1997</td><td>SLSRYAKLANRLANPKLLETFLSKWIG</td><td>SPLN</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>5% hemolysis at 30μg/ml</td><td>Rat</td><td>CCSHHHHHHTTTSSHHHHHHHTGGGTC</td><td align="center"><a href="jmol.php?id=3782&amp;di=1" width="5px" height="5px"><img src="pymol_images/3782.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3782&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3782.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3783">3783</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9009216">9009216</a></td><td align="right">1997</td><td>SLSRYAKLANRLANPKLLETFLSKWIG</td><td>SPLN</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>20% hemolysis at 45μg/ml</td><td>Rat</td><td>CCSHHHHHHTTTSSHHHHHHHTGGGTC</td><td align="center"><a href="jmol.php?id=3783&amp;di=1" width="5px" height="5px"><img src="pymol_images/3783.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3783&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3783.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3784">3784</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9009216">9009216</a></td><td align="right">1997</td><td>SLSRYAKLANRLANPKLLETFLSKWIG</td><td>SPLN</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>30% hemolysis at 60μg/ml</td><td>Rat</td><td>CCSHHHHHHTTTSSHHHHHHHTGGGTC</td><td align="center"><a href="jmol.php?id=3784&amp;di=1" width="5px" height="5px"><img src="pymol_images/3784.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3784&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3784.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3785">3785</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9009216">9009216</a></td><td align="right">1997</td><td>SLSRYAKLANRLANPKLLETFLSKWIG</td><td>SPLN</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>50% hemolysis at 75μg/ml</td><td>Rat</td><td>CCSHHHHHHTTTSSHHHHHHHTGGGTC</td><td align="center"><a href="jmol.php?id=3785&amp;di=1" width="5px" height="5px"><img src="pymol_images/3785.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3785&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3785.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3786">3786</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9009216">9009216</a></td><td align="right">1997</td><td>SLSRYAKLANRLANPKLLETFLSKWIG</td><td>SPLN</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>85% hemolysis at 120μg/ml</td><td>Rat</td><td>CCSHHHHHHTTTSSHHHHHHHTGGGTC</td><td align="center"><a href="jmol.php?id=3786&amp;di=1" width="5px" height="5px"><img src="pymol_images/3786.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3786&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3786.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3787">3787</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9009216">9009216</a></td><td align="right">1997</td><td>SLSRYAKLANRLANPKLLETFLSKWIG</td><td>SPLN</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>90% hemolysis at 150μg/ml</td><td>Rat</td><td>CCSHHHHHHTTTSSHHHHHHHTGGGTC</td><td align="center"><a href="jmol.php?id=3787&amp;di=1" width="5px" height="5px"><img src="pymol_images/3787.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3787&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3787.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3788">3788</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9009216">9009216</a></td><td align="right">1997</td><td>SLSRYAKLANRLANPKLLETFLSKWIG</td><td>SPLN</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>100% hemolysis at 180μg/ml</td><td>Rat</td><td>CCSHHHHHHTTTSSHHHHHHHTGGGTC</td><td align="center"><a href="jmol.php?id=3788&amp;di=1" width="5px" height="5px"><img src="pymol_images/3788.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3788&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3788.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3789">3789</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9010936">9010936</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAVGSALSSSGEQE</td><td>PX</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>100% hemolysis at 14μg/ml</td><td>Human</td><td>CCCSSCCTTTCSHHHHHHHHHHHHHHSSCCSCC</td><td align="center"><a href="jmol.php?id=3789&amp;di=1" width="5px" height="5px"><img src="pymol_images/3789.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3789&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3789.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3790">3790</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9010936">9010936</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTL</td><td>18P</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>No hemolysis (Non hemolytic)</td><td>Human</td><td>CCCCCCTTTTSCCCCCCC</td><td align="center"><a href="jmol.php?id=3790&amp;di=1" width="5px" height="5px"><img src="pymol_images/3790.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3790&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3790.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3791">3791</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9010936">9010936</a></td><td align="right">1996</td><td>GFFALIAKIISSPLFKTL</td><td>18A</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>100% hemolysis at 33μg/ml</td><td>Human</td><td>CCHHHHHHHHSCSCCCCC</td><td align="center"><a href="jmol.php?id=3791&amp;di=1" width="5px" height="5px"><img src="pymol_images/3791.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3791&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3791.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3792">3792</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9010936">9010936</a></td><td align="right">1996</td><td>GFFALIAQIISSPLFQTL</td><td>18Q</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>100% hemolysis at 56μg/ml</td><td>Human</td><td>CHHHHHHHHHHCSSCCCC</td><td align="center"><a href="jmol.php?id=3792&amp;di=1" width="5px" height="5px"><img src="pymol_images/3792.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3792&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3792.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3793">3793</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9108731">9108731</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAVGSALSSSGEQE</td><td>PX1–33</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Cytolytic</td><td>Conc. Required for hemolysis =1.75nmols</td><td>Human</td><td>CCCSSCCTTTCSHHHHHHHHHHHHHHSSCCSCC</td><td align="center"><a href="jmol.php?id=3793&amp;di=1" width="5px" height="5px"><img src="pymol_images/3793.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3793&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3793.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3794">3794</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9108731">9108731</a></td><td align="right">1996</td><td>GFFALIAKIISSPLFKTLLSAVGSALSSSGEQE</td><td>PX1–33 (P7A)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Cytolytic</td><td>Conc. Required for hemolysis =0.6nmols</td><td>Human</td><td>CCSSHHHHHHTSHHHHHHHHHGGGGTSSCCCCC</td><td align="center"><a href="jmol.php?id=3794&amp;di=1" width="5px" height="5px"><img src="pymol_images/3794.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3794&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3794.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3795">3795</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9108731">9108731</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTLLSAVGSAL</td><td>PX1–26</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Cytolytic</td><td>Conc. Required for hemolysis =1.1nmols</td><td>Human</td><td>CCCCCCHHHHTSHHHHHHHHHHSSCC</td><td align="center"><a href="jmol.php?id=3795&amp;di=1" width="5px" height="5px"><img src="pymol_images/3795.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3795&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3795.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3796">3796</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9108731">9108731</a></td><td align="right">1996</td><td>GFFALIPKIISSPLFKTL</td><td>PX1–18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Cytolytic</td><td>No hemolysis (Non hemolysis)</td><td>Human</td><td>CCCCCCTTTTSCCCCCCC</td><td align="center"><a href="jmol.php?id=3796&amp;di=1" width="5px" height="5px"><img src="pymol_images/3796.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3796&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3796.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3797">3797</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9108731">9108731</a></td><td align="right">1996</td><td>GFFALIAKIISSPLFKTL</td><td>PX1–18 (P7A)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Cytolytic</td><td>Conc. Required for hemolysis =7.1nmols</td><td>Human</td><td>CCHHHHHHHHSCSCCCCC</td><td align="center"><a href="jmol.php?id=3797&amp;di=1" width="5px" height="5px"><img src="pymol_images/3797.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3797&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3797.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3798">3798</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9204560">9204560</a></td><td align="right">1997</td><td>frlkfh</td><td>66-10</td><td>Free</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">6</td><td>Antimicrobial</td><td>0% hemolysis at 312 to 0μg/ml</td><td>Human</td><td>CCSSCC</td><td align="center"><a href="jmol.php?id=3798&amp;di=1" width="5px" height="5px"><img src="pymol_images/3798.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3798&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3798.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3799">3799</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9204560">9204560</a></td><td align="right">1997</td><td>frlkfh</td><td>66-10</td><td>Free</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">6</td><td>Antimicrobial</td><td>0% hemolysis at 312 to 0μg/ml</td><td>Bovine</td><td>CCSSCC</td><td align="center"><a href="jmol.php?id=3799&amp;di=1" width="5px" height="5px"><img src="pymol_images/3799.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3799&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3799.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3800">3800</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9204560">9204560</a></td><td align="right">1997</td><td>frlkfh</td><td>66-10</td><td>Free</td><td>Free</td><td>Linear</td><td>D</td><td>None</td><td align="right">6</td><td>Antimicrobial</td><td>0% hemolysis at 312 to 0μg/ml</td><td>Ovine</td><td>CCSSCC</td><td align="center"><a href="jmol.php?id=3800&amp;di=1" width="5px" height="5px"><img src="pymol_images/3800.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3800&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3800.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3801">3801</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9271200">9271200</a></td><td align="right">1997</td><td>RQRVEELSKFSKKGAAARRRK</td><td>Misgurin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>0% hemolysis at 5μg/ml</td><td>Sheep</td><td>CCTTTHHHHTTTTSGGGGTCC</td><td align="center"><a href="jmol.php?id=3801&amp;di=1" width="5px" height="5px"><img src="pymol_images/3801.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3801&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3801.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3802">3802</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9271200">9271200</a></td><td align="right">1997</td><td>RQRVEELSKFSKKGAAARRRK</td><td>Misgurin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>0% hemolysis at 10μg/ml</td><td>Sheep</td><td>CCTTTHHHHTTTTSGGGGTCC</td><td align="center"><a href="jmol.php?id=3802&amp;di=1" width="5px" height="5px"><img src="pymol_images/3802.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3802&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3802.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3803">3803</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9271200">9271200</a></td><td align="right">1997</td><td>RQRVEELSKFSKKGAAARRRK</td><td>Misgurin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>0.23% hemolysis at 25μg/ml</td><td>Sheep</td><td>CCTTTHHHHTTTTSGGGGTCC</td><td align="center"><a href="jmol.php?id=3803&amp;di=1" width="5px" height="5px"><img src="pymol_images/3803.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3803&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3803.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3804">3804</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9271200">9271200</a></td><td align="right">1997</td><td>RQRVEELSKFSKKGAAARRRK</td><td>Misgurin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>0.23% hemolysis at 50μg/ml</td><td>Sheep</td><td>CCTTTHHHHTTTTSGGGGTCC</td><td align="center"><a href="jmol.php?id=3804&amp;di=1" width="5px" height="5px"><img src="pymol_images/3804.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3804&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3804.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3805">3805</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9271200">9271200</a></td><td align="right">1997</td><td>RQRVEELSKFSKKGAAARRRK</td><td>Misgurin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>0.46% hemolysis at 100μg/ml</td><td>Sheep</td><td>CCTTTHHHHTTTTSGGGGTCC</td><td align="center"><a href="jmol.php?id=3805&amp;di=1" width="5px" height="5px"><img src="pymol_images/3805.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3805&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3805.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3806">3806</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9395500">9395500</a></td><td align="right">1997</td><td>HALWKNMLKGIGKLAGKAALGAVKKLVGAES</td><td>DS3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">31</td><td>Cytotoxic</td><td>50% hemolysis at 50μM</td><td>Human</td><td>CTTHHHHHTTTTTSGGGGTTTGGGGTSCSCC</td><td align="center"><a href="jmol.php?id=3806&amp;di=1" width="5px" height="5px"><img src="pymol_images/3806.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3806&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3806.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3807">3807</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9395500">9395500</a></td><td align="right">1997</td><td>HALWMTLLKKVLKAAAKAALNAVLVGANA</td><td>DS4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Cytotoxic</td><td>2.6% hemolysis at 10μM</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=3807&amp;di=1" width="5px" height="5px"><img src="pymol_images/3807.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3807&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3807.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3808">3808</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9518597">9518597</a></td><td align="right">1998</td><td>Aib-P-Aib-A-Aib-AQ-Aib-V-Aib-GL-Aib-PV-Aib-Aib-EQ</td><td>Alam-P14</td><td>Pheol</td><td>Acetylation</td><td>Linear</td><td>L</td><td>Aib = α- methylalanine</td><td align="right">19</td><td>Cytolytic</td><td>EC<sub>50</sub> =30μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3809">3809</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9639668">9639668</a></td><td align="right">1998</td><td>GVVKVSRLKGESLRARL</td><td>bPaAP</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0.42% hemolysis at 200μg/ml</td><td>Human</td><td>CCSCCCCSSSHHHHTTC</td><td align="center"><a href="jmol.php?id=3809&amp;di=1" width="5px" height="5px"><img src="pymol_images/3809.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3810">3810</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9639668">9639668</a></td><td align="right">1998</td><td>IIKVPLKKFKSMREVMRDHGIKAPVVDPATKY</td><td>bPcAP</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>0.36% hemolysis at 200μg/ml</td><td>Human</td><td>CCCSSTTSCTHHHHGGGTSSSSCSCSCSSSCC</td><td align="center"><a href="jmol.php?id=3810&amp;di=1" width="5px" height="5px"><img src="pymol_images/3810.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3810&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3810.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3811">3811</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9639668">9639668</a></td><td align="right">1998</td><td>AVVKVPLKKFKSIRE</td><td>mPcAP</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>0.40% hemolysis at 200μg/ml</td><td>Human</td><td>CCSCCSSTTTTTCCC</td><td align="center"><a href="jmol.php?id=3811&amp;di=1" width="5px" height="5px"><img src="pymol_images/3811.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3811&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3811.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3812">3812</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9639668">9639668</a></td><td align="right">1998</td><td>AVVKVPLKKFKSIRETMKEKGLLEDF</td><td>hPcAP</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>0.49% hemolysis at 200μg/ml</td><td>Human</td><td>CCSCCTTCCCTTTGGGTTTCCCTTCC</td><td align="center"><a href="jmol.php?id=3812&amp;di=1" width="5px" height="5px"><img src="pymol_images/3812.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3812&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3812.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3813">3813</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9756752">9756752</a></td><td align="right">1998</td><td>KKVVFKVKFK</td><td>KSL</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>0% hemolysis at 500μg/ml</td><td>Mouse</td><td>CCCCCCSSCC</td><td align="center"><a href="jmol.php?id=3813&amp;di=1" width="5px" height="5px"><img src="pymol_images/3813.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3813&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3813.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3824">3824</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9824303">9824303</a></td><td align="right">1998</td><td>KGRGKQGGKVRAKAKTRSS</td><td>Parasin I</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0.2% hemolysis at 200μg/ml</td><td>Human</td><td>CCSCCCCCCCSSCCSSSCC</td><td align="center"><a href="jmol.php?id=3824&amp;di=1" width="5px" height="5px"><img src="pymol_images/3824.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3824&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3824.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3826">3826</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9889358">9889358</a></td><td align="right">1998</td><td>KLLLKLLLK</td><td>DnsLK<sub>9</sub></td><td>Free</td><td>Dansylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Cytotoxin</td><td>LD<sub>50</sub> =4.5μM</td><td>Human</td><td>CTHHHHHHC</td><td align="center"><a href="jmol.php?id=3826&amp;di=1" width="5px" height="5px"><img src="pymol_images/3826.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3826&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3826.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3827">3827</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9889358">9889358</a></td><td align="right">1998</td><td>KLLLKLLLK</td><td>DnsLK<sub>9</sub></td><td>Amidation</td><td>Dansylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Cytotoxin</td><td>LD<sub>50</sub> =2.5μM</td><td>Human</td><td>CTHHHHHHC</td><td align="center"><a href="jmol.php?id=3827&amp;di=1" width="5px" height="5px"><img src="pymol_images/3827.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3827&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3827.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3828">3828</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9889358">9889358</a></td><td align="right">1998</td><td>KLLKLLLKLLKLLLKLLLKLLK</td><td>DnsLK<sub>22</sub></td><td>Free</td><td>Dansylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Cytotoxin</td><td>LD<sub>50</sub> =0.3μM</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3828&amp;di=1" width="5px" height="5px"><img src="pymol_images/3828.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3828&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3828.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3829">3829</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9889358">9889358</a></td><td align="right">1998</td><td>KLLKLLLKLLKLLLKLLLKLLK</td><td>DnsLK<sub>22</sub></td><td>Amidation</td><td>Dansylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Cytotoxin</td><td>LD<sub>50</sub> =0.1μM</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHHCC</td><td align="center"><a href="jmol.php?id=3829&amp;di=1" width="5px" height="5px"><img src="pymol_images/3829.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3829&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3829.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3830">3830</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9889358">9889358</a></td><td align="right">1998</td><td>KLLKLLLKLLLKLLK</td><td>LK<sub>15</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Cytotoxin</td><td>LD<sub>50</sub> =0.8μM</td><td>Human</td><td>CTHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3830&amp;di=1" width="5px" height="5px"><img src="pymol_images/3830.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3830&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3830.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3831">3831</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9889358">9889358</a></td><td align="right">1998</td><td>KLLKLLLKLLLKLLK</td><td>DnsLK<sub>15</sub></td><td>Free</td><td>Dansylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Cytotoxin</td><td>LD<sub>50</sub> =0.3μM</td><td>Human</td><td>CTHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3831&amp;di=1" width="5px" height="5px"><img src="pymol_images/3831.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3831&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3831.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3832">3832</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9889358">9889358</a></td><td align="right">1998</td><td>LKLLLLKLLKLKLWK</td><td>r-LK<sub>15</sub></td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Cytotoxin</td><td>LD<sub>50</sub> =5μM</td><td>Human</td><td>CHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3832&amp;di=1" width="5px" height="5px"><img src="pymol_images/3832.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3832&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3832.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3833">3833</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9914515">9914515</a></td><td align="right">1999</td><td>GFFALIPKIISSPLFKTLLSAV</td><td>Par[1-22]</td><td>NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub></td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>22% hemolysis at 50μM</td><td>Human</td><td>CCCCCCHHHHTSHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3833&amp;di=1" width="5px" height="5px"><img src="pymol_images/3833.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3834">3834</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9914515">9914515</a></td><td align="right">1999</td><td>GFFALIpKIISSPLFKTllSAV</td><td>[D]-P<sup>7</sup>L<sup>18,19</sup>-Par[1-22]</td><td>NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub></td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>1% hemolysis at 50μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3835">3835</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/9914515">9914515</a></td><td align="right">1999</td><td>KGFFALIpKIISSPLFKTllSAV</td><td>K<sub>1</sub>[D]-P<sup>7</sup>L<sup>18,19</sup>-Par[1-22]</td><td>NH(CH<sub>2</sub>)<sub>2</sub>NH<sub>2</sub></td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>0% hemolysis at 50μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3836">3836</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10195441">10195441</a></td><td align="right">1999</td><td>PKLLKTFLSKWIG</td><td>SPFK</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolysis at 40μM</td><td>Rat</td><td>CCGGGTTSSTTCC</td><td align="center"><a href="jmol.php?id=3836&amp;di=1" width="5px" height="5px"><img src="pymol_images/3836.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3836&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3836.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3837">3837</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10195441">10195441</a></td><td align="right">2000</td><td>PKLLKKFLKKWIG</td><td>K5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>100% hemolysis at 175μM</td><td>Rat</td><td>CTTHHHHGGGTCC</td><td align="center"><a href="jmol.php?id=3837&amp;di=1" width="5px" height="5px"><img src="pymol_images/3837.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3837&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3837.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3838">3838</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10195441">10195441</a></td><td align="right">2001</td><td>PKLLKTFLSKWKKIG</td><td>WKK</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>100% hemolysis at 85μM</td><td>Rat</td><td>CCSHHHHHHHHTSCC</td><td align="center"><a href="jmol.php?id=3838&amp;di=1" width="5px" height="5px"><img src="pymol_images/3838.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3838&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3838.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3839">3839</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10195441">10195441</a></td><td align="right">2002</td><td>PKLLKFLSKWIG</td><td>K3S</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>100% hemolysis at 63μM</td><td>Rat</td><td>CCTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=3839&amp;di=1" width="5px" height="5px"><img src="pymol_images/3839.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3839&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3839.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3840">3840</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10195441">10195441</a></td><td align="right">2003</td><td>PKLLKTFLKWIG</td><td>K3T</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>100% hemolysis at &gt;330μM</td><td>Rat</td><td>CHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3840&amp;di=1" width="5px" height="5px"><img src="pymol_images/3840.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3840&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3840.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3841">3841</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10195441">10195441</a></td><td align="right">2004</td><td>PKLKTFLSKWIG</td><td>KTF</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolysis at 300μM</td><td>Rat</td><td>CCTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=3841&amp;di=1" width="5px" height="5px"><img src="pymol_images/3841.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3841&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3841.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3842">3842</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10195441">10195441</a></td><td align="right">2005</td><td>PKLLKFLKWIG</td><td>K3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>100% hemolysis at 365μM</td><td>Rat</td><td>CCGGGSSSSCC</td><td align="center"><a href="jmol.php?id=3842&amp;di=1" width="5px" height="5px"><img src="pymol_images/3842.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3842&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3842.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3843">3843</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10217411">10217411</a></td><td align="right">1999</td><td>GLPALISWIKRKRQQG</td><td>MCF</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>100% hemolysis at 100μg/ml</td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3844">3844</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10217411">10217411</a></td><td align="right">1999</td><td>GLKKLISWIKRAAQQG</td><td>MCFA</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>100% hemolysis at 110μg/ml</td><td>Rat</td><td>CCTTHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3844&amp;di=1" width="5px" height="5px"><img src="pymol_images/3844.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3844&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3844.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3863">3863</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10421892">10421892</a></td><td align="right">1999</td><td>WNPFKELERAGQRIRDSIISAAP</td><td>Cecropin D1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml</td><td>Human</td><td>CCCSHHHHHHHHHTTTTTSCCCC</td><td align="center"><a href="jmol.php?id=3863&amp;di=1" width="5px" height="5px"><img src="pymol_images/3863.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3864">3864</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10421892">10421892</a></td><td align="right">1999</td><td>WNPFKELERAGQRVRDAIISAAP</td><td>Cecropin D2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml</td><td>Human</td><td>CCTHHHHHHHHHTTSTTSCSSCC</td><td align="center"><a href="jmol.php?id=3864&amp;di=1" width="5px" height="5px"><img src="pymol_images/3864.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3864&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3864.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3865">3865</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10424354">10424354</a></td><td align="right">1999</td><td>KWKLFKKIGIGKFLHSAKKF</td><td>CA(1-8) – MA(1-12)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis at 25μM</td><td>Human</td><td>CCCCCCCCSSGGGGCTTSCC</td><td align="center"><a href="jmol.php?id=3865&amp;di=1" width="5px" height="5px"><img src="pymol_images/3865.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3865&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3865.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3866">3866</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10424354">10424354</a></td><td align="right">1999</td><td>KWKLFKKIGIGKFLHSAKKF</td><td>CA(1-8) – MA(1-12)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis at 100μM</td><td>Human</td><td>CCCCCCCCSSGGGGCTTSCC</td><td align="center"><a href="jmol.php?id=3866&amp;di=1" width="5px" height="5px"><img src="pymol_images/3866.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3866&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3866.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3867">3867</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10424354">10424354</a></td><td align="right">1999</td><td>KWKLFKKIGIGAVLKVLTTG</td><td>CA(1-8) - ME(1-12)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>3.1% hemolysis at 25μM</td><td>Human</td><td>CCCCCCCCSSSSCCCCSCCC</td><td align="center"><a href="jmol.php?id=3867&amp;di=1" width="5px" height="5px"><img src="pymol_images/3867.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3867&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3867.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3868">3868</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10424354">10424354</a></td><td align="right">1999</td><td>KWKLFKKIGIGAVLKVLTTG</td><td>CA(1-8) - ME(1-12)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>15.5% hemolysis at 100μM</td><td>Human</td><td>CCCCCCCCSSSSCCCCSCCC</td><td align="center"><a href="jmol.php?id=3868&amp;di=1" width="5px" height="5px"><img src="pymol_images/3868.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3868&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3868.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3889">3889</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11318033">11318033</a></td><td align="right">2001</td><td>QlLVDlI</td><td>Lichenysin</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">7</td><td>Chelating agent</td><td>100% hemolysis at 15μM</td><td>Human</td><td>CCSGGGC</td><td align="center"><a href="jmol.php?id=3889&amp;di=1" width="5px" height="5px"><img src="pymol_images/3889.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3889&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3889.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3890">3890</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15996769">15996769</a></td><td align="right">2005</td><td>FLSSIGKILGNLL</td><td>Temporin-1Va</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LD<sub>50</sub> =120μM</td><td>Human</td><td>CCCGGGGGTSCCC</td><td align="center"><a href="jmol.php?id=3890&amp;di=1" width="5px" height="5px"><img src="pymol_images/3890.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3890&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3890.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3891">3891</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15996769">15996769</a></td><td align="right">2005</td><td>FLSIIAKVLGSLF</td><td>Temporin-IVb</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LD<sub>50</sub> =30μM</td><td>Human</td><td>CHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3891&amp;di=1" width="5px" height="5px"><img src="pymol_images/3891.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3891&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3891.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3892">3892</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15996769">15996769</a></td><td align="right">2005</td><td>FLPLVTMLLGKLF</td><td>Temporin-1Vc</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LD<sub>50</sub> =30μM</td><td>Human</td><td>CCCTHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3892&amp;di=1" width="5px" height="5px"><img src="pymol_images/3892.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3893">3893</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKKIFEKVKNLV</td><td>PDD-A</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CCHHHHHHHHTSCC</td><td align="center"><a href="jmol.php?id=3893&amp;di=1" width="5px" height="5px"><img src="pymol_images/3893.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3893&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3893.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3894">3894</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKKIFQKVKNLV</td><td>PDD-A-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> =100μM</td><td>Rat</td><td>CCHHHHHHHTSSCC</td><td align="center"><a href="jmol.php?id=3894&amp;di=1" width="5px" height="5px"><img src="pymol_images/3894.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3894&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3894.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3895">3895</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKKIFEKVKDLV</td><td>PDD-A-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> =100μM</td><td>Rat</td><td>CCHHHHTGGGGGCC</td><td align="center"><a href="jmol.php?id=3895&amp;di=1" width="5px" height="5px"><img src="pymol_images/3895.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3895&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3895.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3896">3896</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>IDWKKIFEKVKNLV</td><td>PDD-A-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CTTCTHHHHTTSCC</td><td align="center"><a href="jmol.php?id=3896&amp;di=1" width="5px" height="5px"><img src="pymol_images/3896.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3896&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3896.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3897">3897</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>IDWKKIFEKVKDLV</td><td>PDD-A-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CCTHHHHGGGSSCC</td><td align="center"><a href="jmol.php?id=3897&amp;di=1" width="5px" height="5px"><img src="pymol_images/3897.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3897&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3897.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3898">3898</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>IDWSKIFEKVKNLV</td><td>PDD-A-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CCHHHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=3898&amp;di=1" width="5px" height="5px"><img src="pymol_images/3898.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3898&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3898.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3899">3899</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWSSIFEKVKNLV</td><td>PDD-A-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CCTHHHHHHHTCCC</td><td align="center"><a href="jmol.php?id=3899&amp;di=1" width="5px" height="5px"><img src="pymol_images/3899.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3899&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3899.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3900">3900</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWSSIFESVKNLV</td><td>PDD-A-7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CCSGGGSTTTTTTC</td><td align="center"><a href="jmol.php?id=3900&amp;di=1" width="5px" height="5px"><img src="pymol_images/3900.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3900&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3900.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3901">3901</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWSSIFESVSNLV</td><td>PDD-A-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CCCCTTGGGGTCCC</td><td align="center"><a href="jmol.php?id=3901&amp;di=1" width="5px" height="5px"><img src="pymol_images/3901.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3901&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3901.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3902">3902</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKKIFEKVSNLV</td><td>PDD-A-9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CCHHHHSGGGSSCC</td><td align="center"><a href="jmol.php?id=3902&amp;di=1" width="5px" height="5px"><img src="pymol_images/3902.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3902&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3902.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3903">3903</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKKIFESVKNLV</td><td>PDD-A-10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CCHHHHHHHSSCCC</td><td align="center"><a href="jmol.php?id=3903&amp;di=1" width="5px" height="5px"><img src="pymol_images/3903.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3903&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3903.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3904">3904</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKSIFEKVKNLV</td><td>PDD-A-11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CCTHHHHHHTTSCC</td><td align="center"><a href="jmol.php?id=3904&amp;di=1" width="5px" height="5px"><img src="pymol_images/3904.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3904&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3904.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3905">3905</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>NIWKKIFEKVKNLV</td><td>PDD-A-12</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CHHHHHHHHHHTCC</td><td align="center"><a href="jmol.php?id=3905&amp;di=1" width="5px" height="5px"><img src="pymol_images/3905.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3905&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3905.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3906">3906</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWLKLGKKILGAL</td><td>PDD-B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> =45μM</td><td>Rat</td><td>CTTHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3906&amp;di=1" width="5px" height="5px"><img src="pymol_images/3906.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3906&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3906.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3907">3907</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWLKLGKKILGAI</td><td>PDD-B-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> =75μM</td><td>Rat</td><td>CCHHHHHHHHSSCC</td><td align="center"><a href="jmol.php?id=3907&amp;di=1" width="5px" height="5px"><img src="pymol_images/3907.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3908">3908</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWLRLGRRILGAL</td><td>PDD-B-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> =25μM</td><td>Rat</td><td>CCTHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=3908&amp;di=1" width="5px" height="5px"><img src="pymol_images/3908.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3909">3909</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INFLKLGKKILGAL</td><td>PDD-B-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;140μM</td><td>Rat</td><td>CCSSTTGGGTTTTC</td><td align="center"><a href="jmol.php?id=3909&amp;di=1" width="5px" height="5px"><img src="pymol_images/3909.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3909&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3909.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3910">3910</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKKLGKKILGAL</td><td>PDD-B-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;140μM</td><td>Rat</td><td>CCHHHHHGGGSSCC</td><td align="center"><a href="jmol.php?id=3910&amp;di=1" width="5px" height="5px"><img src="pymol_images/3910.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3910&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3910.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3911">3911</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INSLKLGKKILGAL</td><td>PDD-B-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CBCCTTGGGSBSCC</td><td align="center"><a href="jmol.php?id=3911&amp;di=1" width="5px" height="5px"><img src="pymol_images/3911.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3911&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3911.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3912">3912</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWLKLGKKMMSAL</td><td>MP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> =100μM</td><td>Rat</td><td>CCSHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3912&amp;di=1" width="5px" height="5px"><img src="pymol_images/3912.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3912&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3912.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3913">3913</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWLKLGKK-Nle-Nle-SAL</td><td>MP-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nle</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> =43μM</td><td>Rat</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3914">3914</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWLKLGKKLLSAL</td><td>MP-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> =44μM</td><td>Rat</td><td>CCHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3914&amp;di=1" width="5px" height="5px"><img src="pymol_images/3914.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3914&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3914.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3915">3915</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWLKLGKKLLSAL</td><td>MP-3</td><td>Amidation</td><td>Fmoc</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> =6μM</td><td>Rat</td><td>CCHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3915&amp;di=1" width="5px" height="5px"><img src="pymol_images/3915.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3915&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3915.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3916">3916</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWLKLGKKLLSAL</td><td>MP-4</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> =44μM</td><td>Rat</td><td>CCHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3916&amp;di=1" width="5px" height="5px"><img src="pymol_images/3916.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3916&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3916.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3917">3917</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWLKLGKKMMSAI</td><td>MP-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CCHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=3917&amp;di=1" width="5px" height="5px"><img src="pymol_images/3917.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3917&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3917.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3918">3918</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>SNWLKLGKKMMSAL</td><td>MP-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CCTHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3918&amp;di=1" width="5px" height="5px"><img src="pymol_images/3918.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3918&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3918.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3919">3919</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWLKLGKKMMSAL</td><td>MP-7</td><td>Amidation</td><td>Fmoc</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> =13μM</td><td>Rat</td><td>CCSHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3919&amp;di=1" width="5px" height="5px"><img src="pymol_images/3919.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3919&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3919.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3920">3920</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWLKLGKKMMSAL</td><td>MP-8</td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CCSHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3920&amp;di=1" width="5px" height="5px"><img src="pymol_images/3920.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3920&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3920.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3921">3921</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWLKLGKKMMSAL</td><td>MP-9</td><td>OH</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CCSHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3921&amp;di=1" width="5px" height="5px"><img src="pymol_images/3921.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3921&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3921.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3922">3922</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKKIASIGKEVLKAL</td><td>PMM</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>IC<sub>50</sub> =80μM</td><td>Rat</td><td>CCHHHHTTTHHHHTTTC</td><td align="center"><a href="jmol.php?id=3922&amp;di=1" width="5px" height="5px"><img src="pymol_images/3922.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3922&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3922.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3923">3923</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKKIASIGKEVLKAI</td><td>PMM-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CCHHHHTTHHHHHTTTC</td><td align="center"><a href="jmol.php?id=3923&amp;di=1" width="5px" height="5px"><img src="pymol_images/3923.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3923&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3923.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3924">3924</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>NWKKIASIGKEVLKAL</td><td>PMM-2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CCTTGGGTHHHHHTTC</td><td align="center"><a href="jmol.php?id=3924&amp;di=1" width="5px" height="5px"><img src="pymol_images/3924.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3924&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3924.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3925">3925</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>WKKIASIGKEVLKAL</td><td>PMM-3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CCGGGSTHHHHSCCC</td><td align="center"><a href="jmol.php?id=3925&amp;di=1" width="5px" height="5px"><img src="pymol_images/3925.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3925&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3925.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3926">3926</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>KKIASIGKEVLKAL</td><td>PMM-4</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CCCSSTHHHHTTTC</td><td align="center"><a href="jmol.php?id=3926&amp;di=1" width="5px" height="5px"><img src="pymol_images/3926.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3926&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3926.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3927">3927</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>KIASIGKEVLKAL</td><td>PMM-5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CCCCHHHHGGGCC</td><td align="center"><a href="jmol.php?id=3927&amp;di=1" width="5px" height="5px"><img src="pymol_images/3927.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3927&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3927.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3928">3928</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKKIASIGKEVLKA</td><td>PMM-6</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CCGGGGGGGGGTTTCC</td><td align="center"><a href="jmol.php?id=3928&amp;di=1" width="5px" height="5px"><img src="pymol_images/3928.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3928&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3928.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3929">3929</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKKIASIGKEVLK</td><td>PMM-7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CCHHHHHHHHTSSCC</td><td align="center"><a href="jmol.php?id=3929&amp;di=1" width="5px" height="5px"><img src="pymol_images/3929.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3929&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3929.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3930">3930</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKKIASIGKEVL</td><td>PMM-8</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CCHHHHTTSSCCCC</td><td align="center"><a href="jmol.php?id=3930&amp;di=1" width="5px" height="5px"><img src="pymol_images/3930.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3930&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3930.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3931">3931</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INQKKIASIGKEV</td><td>PMM-9</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CCSTTSSCCSSCC</td><td align="center"><a href="jmol.php?id=3931&amp;di=1" width="5px" height="5px"><img src="pymol_images/3931.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3931&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3931.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3932">3932</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>IWNKIAKSIGKVLEKAL</td><td>PMM-10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;100μM</td><td>Rat</td><td>CHHHHTSSSCHHHHTTC</td><td align="center"><a href="jmol.php?id=3932&amp;di=1" width="5px" height="5px"><img src="pymol_images/3932.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3932&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3932.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3933">3933</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>INWKKGKEVLKAL</td><td>PMM-11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CBCSSSBGGGGTC</td><td align="center"><a href="jmol.php?id=3933&amp;di=1" width="5px" height="5px"><img src="pymol_images/3933.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3933&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3933.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3934">3934</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>NIWKKIASIAKEVLKAL</td><td>PMM-12</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>IC<sub>50</sub> =32μM</td><td>Rat</td><td>CGGGGGGGHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3934&amp;di=1" width="5px" height="5px"><img src="pymol_images/3934.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3934&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3934.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3935">3935</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>KNWKKIASIGKEVLKAL</td><td>PMM-13</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CCGGGGGGHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3935&amp;di=1" width="5px" height="5px"><img src="pymol_images/3935.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3935&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3935.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3936">3936</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18375018">18375018</a></td><td align="right">2000</td><td>SNWKKIASIGKEVLKAL</td><td>PMM-14</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>IC<sub>50</sub> &gt;&gt;100μM</td><td>Rat</td><td>CCCTTTTTTTHHHHTTC</td><td align="center"><a href="jmol.php?id=3936&amp;di=1" width="5px" height="5px"><img src="pymol_images/3936.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3936&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3936.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3937">3937</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>FKRLEKLFSKIWNWK</td><td>A3-NT</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>MHC &gt;100μg/ml</td><td>Human</td><td>CTTGGGGTTTTCCCC</td><td align="center"><a href="jmol.php?id=3937&amp;di=1" width="5px" height="5px"><img src="pymol_images/3937.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3937&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3937.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3938">3938</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>RWCVYAYVRVRGVLVRYRRCW</td><td>Arenicin-1 (AR-1)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>MHC &gt;1μg/ml</td><td>Human</td><td>CCSCCTTSCCCCCSCCCCCCC</td><td align="center"><a href="jmol.php?id=3938&amp;di=1" width="5px" height="5px"><img src="pymol_images/3938.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3938&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3938.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3939">3939</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>RWSVYAYVRVRGVLVRYRRSW</td><td>AR-1-S</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>MHC &gt;1μg/ml</td><td>Human</td><td>CCCCCTTCCCCCCCCCCCSCC</td><td align="center"><a href="jmol.php?id=3939&amp;di=1" width="5px" height="5px"><img src="pymol_images/3939.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3939&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3939.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3940">3940</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>GKPRPYSPRPTSHPRPIRV</td><td>Drosocin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>MHC &gt;100μg/ml</td><td>Human</td><td>CCSSCCCSSCSCCCCCSCC</td><td align="center"><a href="jmol.php?id=3940&amp;di=1" width="5px" height="5px"><img src="pymol_images/3940.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3940&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3940.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3941">3941</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>LNLKGIFKKVASLLT</td><td>Euminitin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">15</td><td>Antimicrobial</td><td>MHC &lt;1000μg/ml</td><td>Human</td><td>CHHHHHTHHHHTSCC</td><td align="center"><a href="jmol.php?id=3941&amp;di=1" width="5px" height="5px"><img src="pymol_images/3941.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3941&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3941.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3942">3942</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>RVKRVWPLVIRTVIAGYNLYRAIKKK</td><td>Fowlicidin-1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC =1-440μg/ml</td><td>Human</td><td>CCCCCCCCCCTTHHHHSHHHHTTTTC</td><td align="center"><a href="jmol.php?id=3942&amp;di=1" width="5px" height="5px"><img src="pymol_images/3942.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3942&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3942.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3943">3943</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>VOLfPVOLfP</td><td>GS</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>Ornithine</td><td align="right">10</td><td>Antimicrobial</td><td>MHC =12.5μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center"><a href="jmol.php?id=3943&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3943.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3944">3944</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>VKLKVyPLKVKLyP</td><td>GS14</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>MHC =1.5μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=3944&amp;di=1" width="5px" height="5px"><img src="pymol_images/3944.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3944&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3944.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3945">3945</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>VKLkVyPLKVKLyP</td><td>GS14K4</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>MHC =200μg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=3945&amp;di=1" width="5px" height="5px"><img src="pymol_images/3945.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3945&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3945.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3946">3946</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>AKLkAyPLKAKLyP</td><td>GS14K4 V3/A3</td><td>Free</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>MHC &gt;800μg/ml</td><td>Human</td><td>CCCCCCCSSSSCCC</td><td align="center"><a href="jmol.php?id=3946&amp;di=1" width="5px" height="5px"><img src="pymol_images/3946.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3946&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3946.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3947">3947</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>ILPWKWPWWPWRR</td><td>Indolicidin</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>MHC =25μg/ml</td><td>Human</td><td>CCCCCCSSCSSCC</td><td align="center"><a href="jmol.php?id=3947&amp;di=1" width="5px" height="5px"><img src="pymol_images/3947.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3947&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3947.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3948">3948</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>IL-(Nlys)-WKW-(Nlys)-WW-(Nlys)-WRR</td><td>Indolicidin Ink</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nlys</td><td align="right">13</td><td>Antimicrobial</td><td>MHC &gt;200μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3950">3950</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>GVVDILKGAAKDLAGHLATKVMNKL</td><td>Laticeptin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>MHC &lt;400μg/ml</td><td>Human</td><td>CCCCCCCCTHHHHHHHHHHSSSCCC</td><td align="center"><a href="jmol.php?id=3950&amp;di=1" width="5px" height="5px"><img src="pymol_images/3950.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3950&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3950.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3953">3953</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>KILRGVSKKIMRTFLRRILTGKK</td><td>NK23c</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>MHC &lt;10μg/ml</td><td>Human</td><td>CTTTTCCTTTTHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3953&amp;di=1" width="5px" height="5px"><img src="pymol_images/3953.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3954">3954</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>GLLSGILGAGKHIVCGLTGCAKA</td><td>Odorranain-NR</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antimicrobial</td><td>MHC &gt;100μg/ml</td><td>Human</td><td>CCSCSCBCSSSBSSSSSSSSSCC</td><td align="center"><a href="jmol.php?id=3954&amp;di=1" width="5px" height="5px"><img src="pymol_images/3954.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3954&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3954.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3955">3955</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>KWKKLLKKPLLKKLLKKL</td><td>P5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>MHC &gt;100μg/ml</td><td>Human</td><td>CHHHHTHHHHHHHHGGGC</td><td align="center"><a href="jmol.php?id=3955&amp;di=1" width="5px" height="5px"><img src="pymol_images/3955.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3956">3956</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>KWKKLLKKPLLKK</td><td>P10</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>MHC &gt;100μg/ml</td><td>Human</td><td>CCCSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3956&amp;di=1" width="5px" height="5px"><img src="pymol_images/3956.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3956&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3956.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3957">3957</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>KWKKPLLKKLLKKL</td><td>P11</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>MHC &gt;100μg/ml</td><td>Human</td><td>CCCGGGHHHHTTTC</td><td align="center"><a href="jmol.php?id=3957&amp;di=1" width="5px" height="5px"><img src="pymol_images/3957.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3957&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3957.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3958">3958</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>KKTWWKTWWTKWSQPKKKRKV</td><td>Pep-1-K</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>MHC &gt;200μg/ml</td><td>Human</td><td>CCCCHHHHTTTTCSCCTTSCC</td><td align="center"><a href="jmol.php?id=3958&amp;di=1" width="5px" height="5px"><img src="pymol_images/3958.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3958&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3958.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3959">3959</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>GLGSVFGRLARIGRVIPKV</td><td>Pilosulin P1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>MHC &lt;10μg/ml</td><td>Human</td><td>CHHHHTHHHHHHHTTCSCC</td><td align="center"><a href="jmol.php?id=3959&amp;di=1" width="5px" height="5px"><img src="pymol_images/3959.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3960">3960</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>GLLSKFGRLARKLARVIPKV</td><td>Pilosulin P2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>MHC &lt;10μg/ml</td><td>Human</td><td>CTTSHHHHHHHHHTTTSCCC</td><td align="center"><a href="jmol.php?id=3960&amp;di=1" width="5px" height="5px"><img src="pymol_images/3960.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3960&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3960.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3961">3961</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>HNKQEGRDHDKSKGHFHRVVIHHKGGKAH</td><td>SgI-29</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>MHC &gt;200μg/ml</td><td>Human</td><td>CCSCCCSCCCSSSSSSCCSCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=3961&amp;di=1" width="5px" height="5px"><img src="pymol_images/3961.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3961&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3961.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3962">3962</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>SSLLEKGLDGAKKAVGGLGKLGK</td><td>SSL-23</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">22</td><td>Antimicrobial</td><td>MHC &gt;100μg/ml</td><td>Human</td><td>CCHHHHTSCCSGGGSSSCSSCCC</td><td align="center"><a href="jmol.php?id=3962&amp;di=1" width="5px" height="5px"><img src="pymol_images/3962.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3962&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3962.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3963">3963</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>SSLLEKGLDGAKKAVGGLGKLGKDA</td><td>SSL-25</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>MHC &gt;100μg/ml</td><td>Human</td><td>CCSHHHHCCSSSSSSCSSSCCSCCC</td><td align="center"><a href="jmol.php?id=3963&amp;di=1" width="5px" height="5px"><img src="pymol_images/3963.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3963&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3963.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3964">3964</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>SSLLEKGLDGAKKAVGGLGKLGKDAVEDL</td><td>SSL-29</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antimicrobial</td><td>MHC &gt;100μg/ml</td><td>Human</td><td>CCSHHHHSSSSTTTTSCCSSSSSSSSSCC</td><td align="center"><a href="jmol.php?id=3964&amp;di=1" width="5px" height="5px"><img src="pymol_images/3964.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3964&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3964.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3965">3965</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>VRRFPWWWPFLRR</td><td>TP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>MHC &gt;5μg/ml</td><td>Human</td><td>CCCCSSCSCGGGC</td><td align="center"><a href="jmol.php?id=3965&amp;di=1" width="5px" height="5px"><img src="pymol_images/3965.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3965&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3965.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3966">3966</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>VRRFPWWWPFLRR</td><td>TP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>MHC =50μg/ml</td><td>Human</td><td>CCCCSSCSCGGGC</td><td align="center"><a href="jmol.php?id=3966&amp;di=1" width="5px" height="5px"><img src="pymol_images/3966.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3966&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3966.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3967">3967</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>VRRF-(Nphe)-WWW-(Nphe)-FLRR</td><td>TPf</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nphe</td><td align="right">13</td><td>Antimicrobial</td><td>MHC &gt;1μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3968">3968</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>VRRF-(Nlys)-WWW-(Nlys)-FLRR</td><td>TPk</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nlys</td><td align="right">13</td><td>Antimicrobial</td><td>MHC &gt;200μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3969">3969</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>VRRFKWWWKFLRR</td><td>TPK</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>MHC &gt;5μg/ml</td><td>Human</td><td>CCCCHHHHHHTTC</td><td align="center"><a href="jmol.php?id=3969&amp;di=1" width="5px" height="5px"><img src="pymol_images/3969.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3969&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3969.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3970">3970</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>VRRF-(Nleu)-WWW-(Nleu)-FLRR</td><td>TPl</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nleu</td><td align="right">13</td><td>Antimicrobial</td><td>MHC &gt;1μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3971">3971</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>KKFPWWWPFKK</td><td>Tritrpticin STP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">11</td><td>Antimicrobial</td><td>MHC =100μg/ml</td><td>Human</td><td>CCCTTCCSCCC</td><td align="center"><a href="jmol.php?id=3971&amp;di=1" width="5px" height="5px"><img src="pymol_images/3971.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3972">3972</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>KKF-(Nlys)-WWW-(Nlys)-FKK</td><td>Tritrpticin STPk</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nlys</td><td align="right">11</td><td>Antimicrobial</td><td>MHC &gt;200μg/ml</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3973">3973</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>KWKSFLKTFKSAAKTVLHTALKAISS</td><td>V13A<sub>D</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC =31.3μg/ml</td><td>Human</td><td>CCHHHHHHHHHHHHHHHHHHGGGTCC</td><td align="center"><a href="jmol.php?id=3973&amp;di=1" width="5px" height="5px"><img src="pymol_images/3973.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3973&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3973.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3974">3974</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>KWKSFLKTFKSAVKTVLHTALKAISS</td><td>V<sub>681</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC =7.8μg/ml</td><td>Human</td><td>CCTTTTTHHHHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3974&amp;di=1" width="5px" height="5px"><img src="pymol_images/3974.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3974&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3974.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3975">3975</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18568863">18568863</a></td><td align="right">2008</td><td>kwksflktfksavktvlhtalkaiss</td><td>DV<sub>681</sub></td><td>Amidation</td><td>Acetylation</td><td>Linear</td><td>D</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>MHC =7.8μg/ml</td><td>Human</td><td>CCTTTTTHHHHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=3975&amp;di=1" width="5px" height="5px"><img src="pymol_images/3975.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3975&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3975.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3976">3976</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18723059">18723059</a></td><td align="right">2008</td><td>FMPIIGRLMSGSL</td><td>VESP-VB1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>10.2% hemolysis up to 200μg/ml</td><td>Rabbit</td><td>CCCGGGGTCCCCC</td><td align="center"><a href="jmol.php?id=3976&amp;di=1" width="5px" height="5px"><img src="pymol_images/3976.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3976&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3976.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3977">3977</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18723059">18723059</a></td><td align="right">2008</td><td>FMPIIGRLMSGSL</td><td>VESP-VB1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>3.2% hemolysis up to 200μg/ml</td><td>Human</td><td>CCCGGGGTCCCCC</td><td align="center"><a href="jmol.php?id=3977&amp;di=1" width="5px" height="5px"><img src="pymol_images/3977.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3977&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3977.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3978">3978</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18723059">18723059</a></td><td align="right">2008</td><td>INMKASAAVAKKLL</td><td>MP-VB1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>1.7% hemolysis at 200μg/ml</td><td>Rabbit</td><td>CCSSTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=3978&amp;di=1" width="5px" height="5px"><img src="pymol_images/3978.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3978&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3978.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3979">3979</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18723059">18723059</a></td><td align="right">2008</td><td>INMKASAAVAKKLL</td><td>MP-VB1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>2.7% hemolysis at 200μg/ml</td><td>Human</td><td>CCSSTHHHHTTTCC</td><td align="center"><a href="jmol.php?id=3979&amp;di=1" width="5px" height="5px"><img src="pymol_images/3979.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3979&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3979.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3980">3980</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18855761">18855761</a></td><td align="right">2008</td><td>KWKLFKKIPKFLHLAKKF</td><td>P18</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>MHC =50μM</td><td>Human</td><td>CCCSCSCCHHHHTCSSCC</td><td align="center"><a href="jmol.php?id=3980&amp;di=1" width="5px" height="5px"><img src="pymol_images/3980.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3980&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3980.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3981">3981</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18855761">18855761</a></td><td align="right">2008</td><td>KKKLFWKIPKFLHLAKKF</td><td>P18-W<sup>6</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>MHC &lt;200μM</td><td>Human</td><td>CCCSCCCCCHHHHTTTCC</td><td align="center"><a href="jmol.php?id=3981&amp;di=1" width="5px" height="5px"><img src="pymol_images/3981.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3981&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3981.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3982">3982</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18855761">18855761</a></td><td align="right">2008</td><td>KIKLFKKWPKFLHLAKKF</td><td>P18-W<sup>8</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>MHC =200μM</td><td>Human</td><td>CTTCCCCCHHHHTSSCCC</td><td align="center"><a href="jmol.php?id=3982&amp;di=1" width="5px" height="5px"><img src="pymol_images/3982.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3982&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3982.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3983">3983</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18855761">18855761</a></td><td align="right">2008</td><td>KAKLFKKIPKFLHLWKKF</td><td>P18-W<sup>15</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>MHC =200μM</td><td>Human</td><td>CTTCCCCCGGGHHHHSCC</td><td align="center"><a href="jmol.php?id=3983&amp;di=1" width="5px" height="5px"><img src="pymol_images/3983.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3983&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3983.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3984">3984</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18855761">18855761</a></td><td align="right">2008</td><td>KWKLFKKI-Nala-KFLHLAKKF</td><td>P18-Nala<sup>9</sup></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Nala</td><td align="right">18</td><td>Antimicrobial</td><td>MHC =200μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3985">3985</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/18855761">18855761</a></td><td align="right">2008</td><td>KWKLFKKIpKFLHLAKKF</td><td>P18-D-Pro</td><td>Amidation</td><td>Free</td><td>Linear</td><td>Mix</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>MHC &lt;200μM</td><td>Human</td><td>CCCSCSCCHHHHTCSSCC</td><td align="center"><a href="jmol.php?id=3985&amp;di=1" width="5px" height="5px"><img src="pymol_images/3985.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3985&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3985.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3986">3986</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8306981">8306981</a></td><td align="right">1994</td><td>ALWKTMLKKLGTMALHAGKAALGAAADTISQGTQ</td><td>Dermaseptin I</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">34</td><td>Antimicrobial</td><td>100% hemolysis at 100μM</td><td>Rabbit</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3987">3987</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8577744">8577744</a></td><td align="right">1996</td><td>GSKKPVPIIYCNRRTGKCQRM</td><td>Thanatin</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>0% hemolysis at 40μM</td><td>Porcine</td><td>CBCCSSCBCCCCCCCSSCCCC</td><td align="center"><a href="jmol.php?id=3987&amp;di=1" width="5px" height="5px"><img src="pymol_images/3987.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3987&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3987.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3988">3988</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8910461">8910461</a></td><td align="right">1996</td><td>GRFKRFRKKFKKLFKKLSPVIPLLHLG</td><td>BMAP-27</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>3% hemolysis at 10μM</td><td>Human</td><td>CTTTTGGGGGHHHHHHHSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3988&amp;di=1" width="5px" height="5px"><img src="pymol_images/3988.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3988&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3988.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3989">3989</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8910461">8910461</a></td><td align="right">1996</td><td>GRFKRFRKKFKKLFKKLSPVIPLLHLG</td><td>BMAP-27</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>6% hemolysis at 30μM</td><td>Human</td><td>CTTTTGGGGGHHHHHHHSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3989&amp;di=1" width="5px" height="5px"><img src="pymol_images/3989.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3989&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3989.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3990">3990</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8910461">8910461</a></td><td align="right">1996</td><td>GRFKRFRKKFKKLFKKLSPVIPLLHLG</td><td>BMAP-27</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">27</td><td>Antimicrobial</td><td>22% hemolysis at 100μM</td><td>Human</td><td>CTTTTGGGGGHHHHHHHSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=3990&amp;di=1" width="5px" height="5px"><img src="pymol_images/3990.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3990&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3990.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3991">3991</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/8910461">8910461</a></td><td align="right">1996</td><td>GGLRSLGRKILRAWKKYGPIIVPIIRIG</td><td>BMAP-28</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>0% hemolysis at 100μM</td><td>Human</td><td>CCCCSHHHHTTTTGGGTSCSCCCCCCCC</td><td align="center"><a href="jmol.php?id=3991&amp;di=1" width="5px" height="5px"><img src="pymol_images/3991.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3991&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3991.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3993">3993</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16129513">16129513</a></td><td align="right">2005</td><td>IDWKKLLDAAKQIL</td><td>Polybia-MPI</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Cytotoxic</td><td>EC<sub>50</sub> =4.5x10<sup>-5</sup>M</td><td>Rat</td><td>CCHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=3993&amp;di=1" width="5px" height="5px"><img src="pymol_images/3993.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3993&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3993.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3994">3994</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16129513">16129513</a></td><td align="right">2005</td><td>ILGTILGLLKSL</td><td>Polybia-CP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Cytotoxic</td><td>EC<sub>50</sub> =4x10<sup>-6</sup>M</td><td>Rat</td><td>CHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=3994&amp;di=1" width="5px" height="5px"><img src="pymol_images/3994.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3994&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3994.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3995">3995</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16441062">16441062</a></td><td align="right">2005</td><td>CAESCVWIPCTVTALLGCSCSNNVCYNGIP</td><td>Cycloviolacin H4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>HIV inhibitory and Antimicrobial and Cytotoxic and Neurotensin antagonistic and Trypsin inhibitory and Insecticide</td><td>HD<sub>50</sub> =5.5μM</td><td>Human</td><td>CCSSSSCCGGGGGTSBTTBCCCCCSSSSCC</td><td align="center"><a href="jmol.php?id=3995&amp;di=1" width="5px" height="5px"><img src="pymol_images/3995.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3995&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3995.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3996">3996</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16488428">16488428</a></td><td align="right">2006</td><td>SAISCGETCFKFKCYTPRCSCSYPVCK</td><td>Violacin A</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">27</td><td>Uterotonic and Insecticidal and Anti-HIV and Antimicrobial and Anti-neurotensive and Cytotoxic and Hemolytic</td><td>30% hemolysis at 1.5μM</td><td>Human</td><td>CCSCCCSTTCCCTTCCSCCSCSCCCCC</td><td align="center"><a href="jmol.php?id=3996&amp;di=1" width="5px" height="5px"><img src="pymol_images/3996.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3996&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3996.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3997">3997</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CAESCVYIPCTVTALLGCSCSNRVCYNGIP</td><td>Cycloviolacin O1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCSSCCCCTHHHHCSSCCGGGSSCCSCC</td><td align="center"><a href="jmol.php?id=3997&amp;di=1" width="5px" height="5px"><img src="pymol_images/3997.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=3998">3998</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCISSAIGCSCKSKVCYRNGIP</td><td>Cycloviolacin O2</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCSSSCSTTCSSSCSSSSSSCCCSSSCC</td><td align="center"><a href="jmol.php?id=3998&amp;di=1" width="5px" height="5px"><img src="pymol_images/3998.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=3998&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/3998.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=3999">3999</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCLTSAIGCSCKSKVCYRNGIP</td><td>Cycloviolacin O3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4000">4000</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCISSAIGCSCKNKVCYRNGIP</td><td>Cycloviolacin O4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCSCSCTTTTBTTTBCCCCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=4000&amp;di=1" width="5px" height="5px"><img src="pymol_images/4000.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4000&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4000.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4001">4001</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCISSAVGCSCKNKVCYKNGTP</td><td>Cycloviolacin O5</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCSSCCCSTTCSSSSCGGGCCSSCCCSCC</td><td align="center"><a href="jmol.php?id=4001&amp;di=1" width="5px" height="5px"><img src="pymol_images/4001.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4001&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4001.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4002">4002</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCISAAVGCSCKSKVCYKNGTLP</td><td>Cycloviolacin O6</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">31</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCTTSSSCTTTTTTTTTSCCCCCCCCCSCCC</td><td align="center"><a href="jmol.php?id=4002&amp;di=1" width="5px" height="5px"><img src="pymol_images/4002.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4002&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4002.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4003">4003</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCTITALAGCKCKSKVCYNSIP</td><td>Cycloviolacin O7</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCTTSSSCCSHHHHSSSCCCCCCCCCCCC</td><td align="center"><a href="jmol.php?id=4003&amp;di=1" width="5px" height="5px"><img src="pymol_images/4003.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4003&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4003.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4004">4004</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CESCVWIPCISSVVGCSCKSKVCYKNGTLP</td><td>Cycloviolacin O8</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCSSCCCSSSSTTSSSCSSCCCCCSTTCC</td><td align="center"><a href="jmol.php?id=4004&amp;di=1" width="5px" height="5px"><img src="pymol_images/4004.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4004&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4004.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4005">4005</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCLTSAVGCSCKSKVCYRNGIP</td><td>Cycloviolacin O9</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCSTTCCSSCCSBSSSBCCCCCCCSSSCC</td><td align="center"><a href="jmol.php?id=4005&amp;di=1" width="5px" height="5px"><img src="pymol_images/4005.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4005&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4005.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4006">4006</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVYIPCLTSAVGCSCKSKVCYRNGIP</td><td>Cycloviolacin O10</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCSCCCCTTCCCCCTTCSSCCCCSSSCC</td><td align="center"><a href="jmol.php?id=4006&amp;di=1" width="5px" height="5px"><img src="pymol_images/4006.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4006&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4006.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4007">4007</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCISAVVGCSCKSKVCYKNGTLP</td><td>Cycloviolacin O11</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">31</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4008">4008</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVYIPCLTSAIGCSCKSKVCYRNGIP</td><td>Cycloviolacin H1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCSSSSSTTCCSCSCSCSSSSSCCCSSSCC</td><td align="center"><a href="jmol.php?id=4008&amp;di=1" width="5px" height="5px"><img src="pymol_images/4008.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4008&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4008.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4009">4009</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVYIPCISGVIGCSCTDKVCYLNGTP</td><td>Kalata B5</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCTTCCCGGGCSSSSSBTTBCCCCCSCC</td><td align="center"><a href="jmol.php?id=4009&amp;di=1" width="5px" height="5px"><img src="pymol_images/4009.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4009&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4009.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4010">4010</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVWIPCISAALGCSCKNKVCYRNGIP</td><td>Circulin A</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCSSSSCTTTSCTTTTTCCCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=4010&amp;di=1" width="5px" height="5px"><img src="pymol_images/4010.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4011">4011</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVFIPCISTLLGCSCKNKVCYRNGVIP</td><td>Circulin B</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">31</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCSSSSSSTHHHHHTTSCCCCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=4011&amp;di=1" width="5px" height="5px"><img src="pymol_images/4011.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4011&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4011.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4012">4012</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>CGESCVFIPCVTALLGCSCKSKVCYKNSIP</td><td>Cyclopsychotride A</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCSSCHHHHHHHTSCCBTTBCCCCCCCC</td><td align="center"><a href="jmol.php?id=4012&amp;di=1" width="5px" height="5px"><img src="pymol_images/4012.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4012&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4012.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4013">4013</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>VCGETCVGGTCNTPGCSCSRPVCTRNGLP</td><td>Violapeptide I</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCSCGGGTTCSSCCCCCCCCCCCSCCSCC</td><td align="center"><a href="jmol.php?id=4013&amp;di=1" width="5px" height="5px"><img src="pymol_images/4013.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4013&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4013.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4014">4014</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>ICGETCVGGTCNTPGCSCSWPVCTRNGLP</td><td>Cycloviolacin O12</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCCCTTTGGGSCCSSCCSCCCCSSSSSCC</td><td align="center"><a href="jmol.php?id=4014&amp;di=1" width="5px" height="5px"><img src="pymol_images/4014.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4014&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4014.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4015">4015</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>VCGETCFGGTCNTPGCSCTWPICTRDGLP</td><td>Kalata B2</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CBSSSSSSBCSSSCSCBSSSBSSSSCSCC</td><td align="center"><a href="jmol.php?id=4015&amp;di=1" width="5px" height="5px"><img src="pymol_images/4015.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4015&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4015.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4016">4016</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>TCGETCFGGTCNTPGCTCDPWPICTDRGLP</td><td>Kalata B3</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCSSTTSCSSSCCSSCCSSCSTTTTCSSCC</td><td align="center"><a href="jmol.php?id=4016&amp;di=1" width="5px" height="5px"><img src="pymol_images/4016.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4016&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4016.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4017">4017</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>VCGETCVGGTCNTPGCTCSWPVCTRDGLP</td><td>Kalata B4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Anti-HIV</td><td>50% hemolysis at ~20μM</td><td>Human</td><td>CCSSSSHHHHTCCSSCTTCCCBSSSBSCC</td><td align="center"><a href="jmol.php?id=4017&amp;di=1" width="5px" height="5px"><img src="pymol_images/4017.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4017&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4017.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4018">4018</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10600388">10600388</a></td><td align="right">1999</td><td>VCGETCEGGTCNTPGCSCSWPVCTRNGLP</td><td>Kalata S</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Anti-HIV</td><td>51% hemolysis at ~20μM</td><td>Human</td><td>CCSSSCCSSSSCSTTCCCCCCCSSSSSCC</td><td align="center"><a href="jmol.php?id=4018&amp;di=1" width="5px" height="5px"><img src="pymol_images/4018.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4018&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4018.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4019">4019</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17561225">17561225</a></td><td align="right">2007</td><td>GLWSKIKEAAKAAGKAALNAVTGLVNQGDQPS</td><td>Dermaseptin-L1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Antimicrobial</td><td>LC<sub>50</sub> =200μM</td><td>Human</td><td>CCGGGGGHHHHHHHHHHHHHTTTTCCCCCCCC</td><td align="center"><a href="jmol.php?id=4019&amp;di=1" width="5px" height="5px"><img src="pymol_images/4019.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4019&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4019.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4020">4020</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/17561225">17561225</a></td><td align="right">2007</td><td>LLGMIPLAISAISALSKL</td><td>Phylloseptin-L1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">18</td><td>Antimicrobial</td><td>LC<sub>50</sub> =40μM</td><td>Human</td><td>CCSSCHHHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=4020&amp;di=1" width="5px" height="5px"><img src="pymol_images/4020.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4020&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4020.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4021">4021</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>CVISAGWNHKIRCKLTGNC</td><td>Nigroain-B1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CCCSTTCSSCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=4021&amp;di=1" width="5px" height="5px"><img src="pymol_images/4021.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4021&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4021.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4022">4022</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>FKTWKRPPFQTSCSGIIKE</td><td>Nigroain-C1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CCCCCCSSSSCBTTBSSCC</td><td align="center"><a href="jmol.php?id=4022&amp;di=1" width="5px" height="5px"><img src="pymol_images/4022.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4023">4023</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>CVHWQTNPARTSCIGP</td><td>Nigroain-D1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CCCCCSSSSSSCSSCC</td><td align="center"><a href="jmol.php?id=4023&amp;di=1" width="5px" height="5px"><img src="pymol_images/4023.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4023&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4023.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4024">4024</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>DCTRWIIGINGRICRD</td><td>Nigroain-E</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>7.62±1.60% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CCSSSCTTSCCCSSCC</td><td align="center"><a href="jmol.php?id=4024&amp;di=1" width="5px" height="5px"><img src="pymol_images/4024.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4024&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4024.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4025">4025</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>SLWETIKNAGKGFILNILDKIRCKVAGGCKT</td><td>Nigroain-K2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">31</td><td>Antimicrobial</td><td>85.52±3.26% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CHHHHHHHHHSCCCGGGTGGGGTTSSSCCCC</td><td align="center"><a href="jmol.php?id=4025&amp;di=1" width="5px" height="5px"><img src="pymol_images/4025.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4025&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4025.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4026">4026</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>SIRDKIKTIAIDLAKSAGTGVLKTLICKLDKSC</td><td>Rugosin-RN1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">33</td><td>Antimicrobial</td><td>6.69±1.51% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CGGGGGGGSHHHHHTTTSSCTTHHHHSSSCSCC</td><td align="center"><a href="jmol.php?id=4026&amp;di=1" width="5px" height="5px"><img src="pymol_images/4026.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4026&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4026.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4027">4027</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>FLGPIIKIATGILPTAICKILKKC</td><td>Gaegurin-RN3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>0% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CCTTSCTTTSSCCTTHHHHHTTTC</td><td align="center"><a href="jmol.php?id=4027&amp;di=1" width="5px" height="5px"><img src="pymol_images/4027.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4027&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4027.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4028">4028</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19778602">19778602</a></td><td align="right">2010</td><td>FLPLVLGALSGILPKILGK</td><td>Temporin-TN1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antimicrobial</td><td>95.7±4.12% hemolysis at 100μg/ml</td><td>Rabbit</td><td>CCGGGHHHHTTSCCTTSCC</td><td align="center"><a href="jmol.php?id=4028&amp;di=1" width="5px" height="5px"><img src="pymol_images/4028.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4028&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4028.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4029">4029</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>KSLRPRCWIKIKFRCKSLKF</td><td>P1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0.7% hemolysis at 0.0625mg/ml</td><td>Human</td><td>CCCCCCCCCCSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=4029&amp;di=1" width="5px" height="5px"><img src="pymol_images/4029.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4029&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4029.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4030">4030</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>KSLRPRCWIKIKFRCKSLKF</td><td>P1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0.8% hemolysis at 0.125mg/ml</td><td>Human</td><td>CCCCCCCCCCSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=4030&amp;di=1" width="5px" height="5px"><img src="pymol_images/4030.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4030&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4030.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4031">4031</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>KSLRPRCWIKIKFRCKSLKF</td><td>P1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>5.9% hemolysis at 0.25mg/ml</td><td>Human</td><td>CCCCCCCCCCSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=4031&amp;di=1" width="5px" height="5px"><img src="pymol_images/4031.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4031&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4031.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4032">4032</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>KSLRPRCWIKIKFRCKSLKF</td><td>P1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>7.6% hemolysis at 0.5mg/ml</td><td>Human</td><td>CCCCCCCCCCSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=4032&amp;di=1" width="5px" height="5px"><img src="pymol_images/4032.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4032&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4032.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4033">4033</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>KSLRPRCWIKIKFRCKSLKF</td><td>P1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>7.8% hemolysis at 1mg/ml</td><td>Human</td><td>CCCCCCCCCCSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=4033&amp;di=1" width="5px" height="5px"><img src="pymol_images/4033.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4033&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4033.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4034">4034</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>KSLRPRCWIKIKFRCKSLKF</td><td>P1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>1.5% hemolysis at 0.0625mg/ml</td><td>Rabbit</td><td>CCCCCCCCCCSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=4034&amp;di=1" width="5px" height="5px"><img src="pymol_images/4034.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4034&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4034.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4035">4035</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>KSLRPRCWIKIKFRCKSLKF</td><td>P1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>7.2% hemolysis at 0.125mg/ml</td><td>Rabbit</td><td>CCCCCCCCCCSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=4035&amp;di=1" width="5px" height="5px"><img src="pymol_images/4035.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4035&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4035.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4036">4036</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>KSLRPRCWIKIKFRCKSLKF</td><td>P1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>7.8% hemolysis at 0.25mg/ml</td><td>Rabbit</td><td>CCCCCCCCCCSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=4036&amp;di=1" width="5px" height="5px"><img src="pymol_images/4036.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4036&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4036.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4037">4037</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>KSLRPRCWIKIKFRCKSLKF</td><td>P1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>9.5% hemolysis at 0.5mg/ml</td><td>Rabbit</td><td>CCCCCCCCCCSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=4037&amp;di=1" width="5px" height="5px"><img src="pymol_images/4037.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4037&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4037.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4038">4038</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>KSLRPRCWIKIKFRCKSLKF</td><td>P1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>57.1% hemolysis at 1mg/ml</td><td>Rabbit</td><td>CCCCCCCCCCSCCCCCCSCC</td><td align="center"><a href="jmol.php?id=4038&amp;di=1" width="5px" height="5px"><img src="pymol_images/4038.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4038&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4038.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4039">4039</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>RPRCWIKIKFRCKSLKF</td><td>P2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0.6% hemolysis at 0.0625mg/ml</td><td>Human</td><td>CCCCCCCGGGCCSSSCC</td><td align="center"><a href="jmol.php?id=4039&amp;di=1" width="5px" height="5px"><img src="pymol_images/4039.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4039&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4039.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4040">4040</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>RPRCWIKIKFRCKSLKF</td><td>P2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>0.8% hemolysis at 0.125mg/ml</td><td>Human</td><td>CCCCCCCGGGCCSSSCC</td><td align="center"><a href="jmol.php?id=4040&amp;di=1" width="5px" height="5px"><img src="pymol_images/4040.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4040&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4040.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4041">4041</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>RPRCWIKIKFRCKSLKF</td><td>P2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>5.2% hemolysis at 0.25mg/ml</td><td>Human</td><td>CCCCCCCGGGCCSSSCC</td><td align="center"><a href="jmol.php?id=4041&amp;di=1" width="5px" height="5px"><img src="pymol_images/4041.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4041&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4041.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4042">4042</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>RPRCWIKIKFRCKSLKF</td><td>P2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>5.3% hemolysis at 0.5mg/ml</td><td>Human</td><td>CCCCCCCGGGCCSSSCC</td><td align="center"><a href="jmol.php?id=4042&amp;di=1" width="5px" height="5px"><img src="pymol_images/4042.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4042&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4042.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4043">4043</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>RPRCWIKIKFRCKSLKF</td><td>P2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>11.3% hemolysis at 1mg/ml</td><td>Human</td><td>CCCCCCCGGGCCSSSCC</td><td align="center"><a href="jmol.php?id=4043&amp;di=1" width="5px" height="5px"><img src="pymol_images/4043.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4043&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4043.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4044">4044</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>RPRCWIKIKFRCKSLKF</td><td>P2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>2.2% hemolysis at 0.0625mg/ml</td><td>Rabbit</td><td>CCCCCCCGGGCCSSSCC</td><td align="center"><a href="jmol.php?id=4044&amp;di=1" width="5px" height="5px"><img src="pymol_images/4044.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4044&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4044.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4045">4045</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>RPRCWIKIKFRCKSLKF</td><td>P2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>4.7% hemolysis at 0.125mg/ml</td><td>Rabbit</td><td>CCCCCCCGGGCCSSSCC</td><td align="center"><a href="jmol.php?id=4045&amp;di=1" width="5px" height="5px"><img src="pymol_images/4045.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4045&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4045.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4046">4046</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>RPRCWIKIKFRCKSLKF</td><td>P2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>5.2% hemolysis at 0.25mg/ml</td><td>Rabbit</td><td>CCCCCCCGGGCCSSSCC</td><td align="center"><a href="jmol.php?id=4046&amp;di=1" width="5px" height="5px"><img src="pymol_images/4046.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4046&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4046.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4047">4047</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>RPRCWIKIKFRCKSLKF</td><td>P2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>12.4% hemolysis at 0.5mg/ml</td><td>Rabbit</td><td>CCCCCCCGGGCCSSSCC</td><td align="center"><a href="jmol.php?id=4047&amp;di=1" width="5px" height="5px"><img src="pymol_images/4047.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4047&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4047.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4048">4048</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/21644248">21644248</a></td><td align="right">2011</td><td>RPRCWIKIKFRCKSLKF</td><td>P2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>66.3% hemolysis at 1mg/ml</td><td>Rabbit</td><td>CCCCCCCGGGCCSSSCC</td><td align="center"><a href="jmol.php?id=4048&amp;di=1" width="5px" height="5px"><img src="pymol_images/4048.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4048&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4048.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4049">4049</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16675060">16675060</a></td><td align="right">2006</td><td>AVDLAKIANKVLSSLF</td><td>Temporin-1TGb</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">16</td><td>Antimicrobial</td><td>LD<sub>50</sub> =250μM</td><td>Human</td><td>CCCHHHHHHHHTTTCC</td><td align="center"><a href="jmol.php?id=4049&amp;di=1" width="5px" height="5px"><img src="pymol_images/4049.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4049&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4049.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4050">4050</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16675060">16675060</a></td><td align="right">2006</td><td>FLPVILPVIGKLLSGIL</td><td>Temporin-1TGc</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">17</td><td>Antimicrobial</td><td>LD<sub>50</sub> =50μM</td><td>Human</td><td>CCTTTTTHHHHHHTSCC</td><td align="center"><a href="jmol.php?id=4050&amp;di=1" width="5px" height="5px"><img src="pymol_images/4050.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4050&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4050.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4051">4051</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16735513">16735513</a></td><td align="right">2006</td><td>SMWSGMWRRKLKKLRNALKKKLKGE</td><td>Ltc1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">25</td><td>Antimicrobial</td><td>20% hemolysis at 80μM</td><td>Rabbit</td><td>CCSCHHHHHHHHHHHHHTTSTTCCC</td><td align="center"><a href="jmol.php?id=4051&amp;di=1" width="5px" height="5px"><img src="pymol_images/4051.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4051&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4051.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4052">4052</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16735513">16735513</a></td><td align="right">2006</td><td>GLFGKLIKKFGRKAISYAVKKARGKH</td><td>Ltc2a</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">26</td><td>Antimicrobial</td><td>20% hemolysis at 6.0μM</td><td>Rabbit</td><td>CTHHHHHHHHHHHHHHHHHTTTTCCC</td><td align="center"><a href="jmol.php?id=4052&amp;di=1" width="5px" height="5px"><img src="pymol_images/4052.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4052&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4052.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4053">4053</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16735513">16735513</a></td><td align="right">2006</td><td>SWKSMAKKLKEYMEKLKQRA</td><td>Ltc3a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>20% hemolysis at &gt;120μM</td><td>Rabbit</td><td>CHHHHHHHHHHHHTTTTTCC</td><td align="center"><a href="jmol.php?id=4053&amp;di=1" width="5px" height="5px"><img src="pymol_images/4053.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4053&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4053.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4054">4054</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16735513">16735513</a></td><td align="right">2006</td><td>SWASMAKKLKEYMEKLKQRA</td><td>Ltc3b</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>20% hemolysis at &gt;120μM</td><td>Rabbit</td><td>CHHHHHHHHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=4054&amp;di=1" width="5px" height="5px"><img src="pymol_images/4054.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4054&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4054.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4055">4055</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16735513">16735513</a></td><td align="right">2006</td><td>GLKDKFKSMGEKLKQYIQTWKAKF</td><td>Ltc4a</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>20% hemolysis at &gt;120μM</td><td>Rabbit</td><td>CCSCSTTTHHHHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=4055&amp;di=1" width="5px" height="5px"><img src="pymol_images/4055.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4055&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4055.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4056">4056</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16735513">16735513</a></td><td align="right">2006</td><td>SLKDKVKSMGEKLKQYIQTWKAKF</td><td>Ltc4b</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>20% hemolysis at &gt;120μM</td><td>Rabbit</td><td>CCSCCHHHHHHHHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=4056&amp;di=1" width="5px" height="5px"><img src="pymol_images/4056.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4056&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4056.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4057">4057</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16735513">16735513</a></td><td align="right">2006</td><td>GFFGKMKEYFKKFGASFKRRFANLKKRL</td><td>Ltc5</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">28</td><td>Antimicrobial</td><td>20% hemolysis at 40μM</td><td>Rabbit</td><td>CCCTTGGGTTTTSTTTGGGGGGGTTTCC</td><td align="center"><a href="jmol.php?id=4057&amp;di=1" width="5px" height="5px"><img src="pymol_images/4057.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4058">4058</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12804591">12804591</a></td><td align="right">2003</td><td>FLPILGKLLSGIL</td><td>MRP</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">10</td><td>Antimicrobial</td><td>LC<sub>50</sub> = 8μM</td><td>Human</td><td>CCCCTHHHHSCCC</td><td align="center"><a href="jmol.php?id=4058&amp;di=1" width="5px" height="5px"><img src="pymol_images/4058.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4058&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4058.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4063">4063</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328050">15328050</a></td><td align="right">2004</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>7% hemolysis at 1μM</td><td>Guinea-pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=4063&amp;di=1" width="5px" height="5px"><img src="pymol_images/4063.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4064">4064</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328050">15328050</a></td><td align="right">2004</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>50% hemolysis at 6μM</td><td>Guinea-pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=4064&amp;di=1" width="5px" height="5px"><img src="pymol_images/4064.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4065">4065</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328050">15328050</a></td><td align="right">2004</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>85% hemolysis at 12.5μM</td><td>Guinea-pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=4065&amp;di=1" width="5px" height="5px"><img src="pymol_images/4065.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4066">4066</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15328050">15328050</a></td><td align="right">2004</td><td>FWGALAKGALKLIPSLFSSFSKKD</td><td>Pin2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antimicrobial</td><td>100% hemolysis at 25μM</td><td>Guinea-pig</td><td>CCTTSCCSGGGSHHHHHHHHTTCC</td><td align="center"><a href="jmol.php?id=4066&amp;di=1" width="5px" height="5px"><img src="pymol_images/4066.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4071">4071</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIGPLIPHAVKAISDLI</td><td>Kassinatuerin-1</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =65μM</td><td>Human</td><td>CCCCCCSSSSTTTGGGTCCCC</td><td align="center"><a href="jmol.php?id=4071&amp;di=1" width="5px" height="5px"><img src="pymol_images/4071.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4071&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4071.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4072">4072</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>IPHAVKAISDLI</td><td>Kassinatuerin-1 (10-21)fragment</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>0% hemolysis at 400μM</td><td>Human</td><td>CCSSSTTTCCCC</td><td align="center"><a href="jmol.php?id=4072&amp;di=1" width="5px" height="5px"><img src="pymol_images/4072.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4072&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4072.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4073">4073</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIGPLIPHKVKAISDLI</td><td>Kassinatuerin-1 (Lys<sup>13</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>32% hemolysis at 400μM</td><td>Human</td><td>CTTBCSSSBCSSCTTSSCCCC</td><td align="center"><a href="jmol.php?id=4073&amp;di=1" width="5px" height="5px"><img src="pymol_images/4073.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4073&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4073.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4074">4074</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIGPLIPHAVKAISKLI</td><td>Kassinatuerin-1 (Lys<sup>19</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =45μM</td><td>Human</td><td>CCCCCCSSCCHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=4074&amp;di=1" width="5px" height="5px"><img src="pymol_images/4074.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4074&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4074.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4075">4075</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIGPLIPHAVKAIKKLI</td><td>Kassinatuerin-1 (Lys<sup>18</sup>, Lys<sup>19</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =25μM</td><td>Human</td><td>CCCCCCCTTSHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=4075&amp;di=1" width="5px" height="5px"><img src="pymol_images/4075.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4075&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4075.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4076">4076</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIKPLIPHAVKAISKLI</td><td>Kassinatuerin-1 (Lys<sup>7</sup>, Lys<sup>19</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =55μM</td><td>Human</td><td>CCCCCCCCCCTTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=4076&amp;di=1" width="5px" height="5px"><img src="pymol_images/4076.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4076&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4076.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4077">4077</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIKPLIPHAVKAIKKLI</td><td>Kassinatuerin-1 (Lys<sup>7</sup>, Lys<sup>18</sup>, Lys<sup>19</sup>)</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =40μM</td><td>Human</td><td>CCCCCCCCCCHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=4077&amp;di=1" width="5px" height="5px"><img src="pymol_images/4077.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4077&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4077.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4078">4078</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIGPLIPHAVKAISDLI</td><td>Kassinatuerin-1 (COOH)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>39% hemolysis at 400μM</td><td>Human</td><td>CCCCCCSSSSTTTGGGTCCCC</td><td align="center"><a href="jmol.php?id=4078&amp;di=1" width="5px" height="5px"><img src="pymol_images/4078.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4078&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4078.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4079">4079</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIGPLIPHAVKAISKLI</td><td>Kassinatuerin-1 (Lys<sup>19</sup>)(COOH)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =120μM</td><td>Human</td><td>CCCCCCSSCCHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=4079&amp;di=1" width="5px" height="5px"><img src="pymol_images/4079.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4079&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4079.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4080">4080</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIKPLIPHAVKAISKLI</td><td>Kassinatuerin-1 (Lys<sup>7</sup>, Lys<sup>19</sup>)(COOH)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =135μM</td><td>Human</td><td>CCCCCCCCCCTTHHHHHHHHC</td><td align="center"><a href="jmol.php?id=4080&amp;di=1" width="5px" height="5px"><img src="pymol_images/4080.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4080&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4080.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4081">4081</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15885852">15885852</a></td><td align="right">2005</td><td>GFMKYIKPLIPHAVKAIKKLI</td><td>Kassinatuerin-1 (Lys<sup>7</sup>, Lys<sup>18</sup>, Lys<sup>19</sup>) (COOH)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antimicrobial</td><td>LD<sub>50</sub> =200μM</td><td>Human</td><td>CCCCCCCCCCHHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=4081&amp;di=1" width="5px" height="5px"><img src="pymol_images/4081.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4081&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4081.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4082">4082</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>Maganin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antiparasitic</td><td>4% hemolysis at 50μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=4082&amp;di=1" width="5px" height="5px"><img src="pymol_images/4082.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4083">4083</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>Maganin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antiparasitic</td><td>3% hemolysis at 100μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=4083&amp;di=1" width="5px" height="5px"><img src="pymol_images/4083.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4084">4084</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>Maganin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antiparasitic</td><td>3% hemolysis at 200μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=4084&amp;di=1" width="5px" height="5px"><img src="pymol_images/4084.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4085">4085</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>Maganin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antiparasitic</td><td>4% hemolysis at 500μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=4085&amp;di=1" width="5px" height="5px"><img src="pymol_images/4085.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4086">4086</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GIGKFLHSAKKFGKAFVGEIMNS</td><td>Maganin 2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">23</td><td>Antiparasitic</td><td>6% hemolysis at 1000μg/ml</td><td>Human</td><td>CCSHHHHHHTTTCSSGGGGTTTC</td><td align="center"><a href="jmol.php?id=4086&amp;di=1" width="5px" height="5px"><img src="pymol_images/4086.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4087">4087</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GWASKIGQTLGKIAKVGLKELIQPK</td><td>XPF</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antiparasitic</td><td>1% hemolysis at 50μg/ml</td><td>Human</td><td>CCSSHHHHHHHHHGGGTTCCCSSCC</td><td align="center"><a href="jmol.php?id=4087&amp;di=1" width="5px" height="5px"><img src="pymol_images/4087.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4087&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4087.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4088">4088</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GWASKIGQTLGKIAKVGLKELIQPK</td><td>XPF</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antiparasitic</td><td>1% hemolysis at 100μg/ml</td><td>Human</td><td>CCSSHHHHHHHHHGGGTTCCCSSCC</td><td align="center"><a href="jmol.php?id=4088&amp;di=1" width="5px" height="5px"><img src="pymol_images/4088.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4088&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4088.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4089">4089</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GWASKIGQTLGKIAKVGLKELIQPK</td><td>XPF</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antiparasitic</td><td>2% hemolysis at 200μg/ml</td><td>Human</td><td>CCSSHHHHHHHHHGGGTTCCCSSCC</td><td align="center"><a href="jmol.php?id=4089&amp;di=1" width="5px" height="5px"><img src="pymol_images/4089.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4089&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4089.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4090">4090</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GWASKIGQTLGKIAKVGLKELIQPK</td><td>XPF</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antiparasitic</td><td>5% hemolysis at 500μg/ml</td><td>Human</td><td>CCSSHHHHHHHHHGGGTTCCCSSCC</td><td align="center"><a href="jmol.php?id=4090&amp;di=1" width="5px" height="5px"><img src="pymol_images/4090.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4090&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4090.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4091">4091</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GWASKIGQTLGKIAKVGLKELIQPK</td><td>XPF</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">24</td><td>Antiparasitic</td><td>6% hemolysis at 1000μg/ml</td><td>Human</td><td>CCSSHHHHHHHHHGGGTTCCCSSCC</td><td align="center"><a href="jmol.php?id=4091&amp;di=1" width="5px" height="5px"><img src="pymol_images/4091.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4091&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4091.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4092">4092</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GMASKAGAIAGKIAKVALKAL</td><td>PGLa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antiparasitic</td><td>4% hemolysis at 50μg/ml</td><td>Human</td><td>CBSSSBSSHHHHTHHHHHHHC</td><td align="center"><a href="jmol.php?id=4092&amp;di=1" width="5px" height="5px"><img src="pymol_images/4092.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4092&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4092.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4093">4093</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GMASKAGAIAGKIAKVALKAL</td><td>PGLa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antiparasitic</td><td>3% hemolysis at 100μg/ml</td><td>Human</td><td>CBSSSBSSHHHHTHHHHHHHC</td><td align="center"><a href="jmol.php?id=4093&amp;di=1" width="5px" height="5px"><img src="pymol_images/4093.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4093&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4093.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4094">4094</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GMASKAGAIAGKIAKVALKAL</td><td>PGLa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antiparasitic</td><td>4% hemolysis at 200μg/ml</td><td>Human</td><td>CBSSSBSSHHHHTHHHHHHHC</td><td align="center"><a href="jmol.php?id=4094&amp;di=1" width="5px" height="5px"><img src="pymol_images/4094.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4094&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4094.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4095">4095</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GMASKAGAIAGKIAKVALKAL</td><td>PGLa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antiparasitic</td><td>5% hemolysis at 500μg/ml</td><td>Human</td><td>CBSSSBSSHHHHTHHHHHHHC</td><td align="center"><a href="jmol.php?id=4095&amp;di=1" width="5px" height="5px"><img src="pymol_images/4095.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4095&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4095.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4096">4096</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/3125066">3125066</a></td><td align="right">1988</td><td>GMASKAGAIAGKIAKVALKAL</td><td>PGLa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Antiparasitic</td><td>7% hemolysis at 1000μg/ml</td><td>Human</td><td>CBSSSBSSHHHHTHHHHHHHC</td><td align="center"><a href="jmol.php?id=4096&amp;di=1" width="5px" height="5px"><img src="pymol_images/4096.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4096&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4096.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4097">4097</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11897590">11897590</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>P</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>5% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4097&amp;di=1" width="5px" height="5px"><img src="pymol_images/4097.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4097&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4097.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4098">4098</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11937508">11937508</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>K<sub>4</sub>-S4(1-13)<sub>NH2</sub></td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>3% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4098&amp;di=1" width="5px" height="5px"><img src="pymol_images/4098.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4098&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4098.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4099">4099</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11937508">11937508</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>Glycyl-P</td><td>Amidation</td><td>H<sub>2</sub>N-CH<sub>2</sub>-CO-P</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>2% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4099&amp;di=1" width="5px" height="5px"><img src="pymol_images/4099.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4099&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4099.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4100">4100</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11937508">11937508</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>Aminobutyryl-P</td><td>Amidation</td><td>H<sub>2</sub>N-(CH<sub>2</sub>)<sub>3</sub>-CO-P</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>1% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4100&amp;di=1" width="5px" height="5px"><img src="pymol_images/4100.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4101">4101</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11937508">11937508</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>Aminoheptanoyl-P</td><td>Amidation</td><td>H<sub>2</sub>N-(CH<sub>2</sub>)<sub>6</sub>-CO-P</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>1% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4101&amp;di=1" width="5px" height="5px"><img src="pymol_images/4101.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4102">4102</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11937508">11937508</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>Aminolauroyl-P</td><td>Amidation</td><td>H<sub>2</sub>N-(CH<sub>2</sub>)<sub>11</sub>-CO-P</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>4% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4102&amp;di=1" width="5px" height="5px"><img src="pymol_images/4102.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4103">4103</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/11937508">11937508</a></td><td align="right">2002</td><td>ALWKTLLKKVLKA</td><td>Lysyl-P</td><td>Amidation</td><td>H<sub>2</sub>N-C((CH<sub>2</sub>)<sub>4</sub>-NH<sub>2</sub>)H-CO-P</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>5% hemolysis at 10μM</td><td>Human</td><td>CTHHHHGGGSCCC</td><td align="center"><a href="jmol.php?id=4103&amp;di=1" width="5px" height="5px"><img src="pymol_images/4103.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4104">4104</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12379643">12379643</a></td><td align="right">2002</td><td>GLWSTIKQKGKEAAIAAAKAAGQAALGAL</td><td>DS 01</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antiparasitic</td><td>0.41% hemolysis at 4μg/ml</td><td>Human</td><td>CCTTHHHHSCTHHHHHHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=4104&amp;di=1" width="5px" height="5px"><img src="pymol_images/4104.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4104&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4104.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4105">4105</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12379643">12379643</a></td><td align="right">2002</td><td>GLWSTIKQKGKEAAIAAAKAAGQAALGAL</td><td>DS 01</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antiparasitic</td><td>0.54% hemolysis at 8μg/ml</td><td>Human</td><td>CCTTHHHHSCTHHHHHHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=4105&amp;di=1" width="5px" height="5px"><img src="pymol_images/4105.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4105&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4105.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4106">4106</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12379643">12379643</a></td><td align="right">2002</td><td>GLWSTIKQKGKEAAIAAAKAAGQAALGAL</td><td>DS 01</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antiparasitic</td><td>0.81% hemolysis at 16μg/ml</td><td>Human</td><td>CCTTHHHHSCTHHHHHHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=4106&amp;di=1" width="5px" height="5px"><img src="pymol_images/4106.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4106&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4106.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4107">4107</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12379643">12379643</a></td><td align="right">2002</td><td>GLWSTIKQKGKEAAIAAAKAAGQAALGAL</td><td>DS 01</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antiparasitic</td><td>1.08% hemolysis at 32μg/ml</td><td>Human</td><td>CCTTHHHHSCTHHHHHHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=4107&amp;di=1" width="5px" height="5px"><img src="pymol_images/4107.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4107&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4107.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4108">4108</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12379643">12379643</a></td><td align="right">2002</td><td>GLWSTIKQKGKEAAIAAAKAAGQAALGAL</td><td>DS 01</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antiparasitic</td><td>1.36% hemolysis at 64μg/ml</td><td>Human</td><td>CCTTHHHHSCTHHHHHHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=4108&amp;di=1" width="5px" height="5px"><img src="pymol_images/4108.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4108&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4108.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4109">4109</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/12379643">12379643</a></td><td align="right">2002</td><td>GLWSTIKQKGKEAAIAAAKAAGQAALGAL</td><td>DS 01</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">29</td><td>Antiparasitic</td><td>2.17% hemolysis at 128μg/ml</td><td>Human</td><td>CCTTHHHHSCTHHHHHHHHHHHHHTTCCC</td><td align="center"><a href="jmol.php?id=4109&amp;di=1" width="5px" height="5px"><img src="pymol_images/4109.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4109&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4109.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4110">4110</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15513914">15513914</a></td><td align="right">2004</td><td>FLPLIGRVLSGIL</td><td>Temporin A</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>LC<sub>50</sub> &gt;120μM</td><td>Human</td><td>CCCCHHHHTSCCC</td><td align="center"><a href="jmol.php?id=4110&amp;di=1" width="5px" height="5px"><img src="pymol_images/4110.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4110&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4110.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4111">4111</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15513914">15513914</a></td><td align="right">2004</td><td>LLPIVGNLLKSLL</td><td>Temporin B</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antiparasitic</td><td>LC<sub>50</sub> &gt;120μM</td><td>Human</td><td>CCHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=4111&amp;di=1" width="5px" height="5px"><img src="pymol_images/4111.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4111&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4111.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4135">4135</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10441158">10441158</a></td><td align="right">1999</td><td>CGETCVGGTCNTPGCTCSWPVCTRNGLPV</td><td>Kalata B1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">29</td><td>Uterotonic</td><td>HD<sub>50</sub> =50μM</td><td>Human</td><td>CCCGGGTTTSSCCCCCSSSCCCTTCSSCC</td><td align="center"><a href="jmol.php?id=4135&amp;di=1" width="5px" height="5px"><img src="pymol_images/4135.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4135&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4135.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4136">4136</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/10441158">10441158</a></td><td align="right">1999</td><td>CGESCVFIPCISTLLGCSCKNKVCYRNGVIP</td><td>Circulin B</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Uterotonic</td><td>HD<sub>50</sub> =30μM</td><td>Human</td><td>CCCSSSSSSTHHHHHTTSCCCCCCCCCSSCC</td><td align="center"><a href="jmol.php?id=4136&amp;di=1" width="5px" height="5px"><img src="pymol_images/4136.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4136&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4136.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4146">4146</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19635516">19635516</a></td><td align="right">2009</td><td>IPPFIKKVLTTVF</td><td>Temporin-CPa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">12</td><td>Antimicrobial</td><td>LC<sub>50</sub> =220μM</td><td>Human</td><td>CCSGGGGHHHHCC</td><td align="center"><a href="jmol.php?id=4146&amp;di=1" width="5px" height="5px"><img src="pymol_images/4146.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4146&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4146.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4147">4147</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19635516">19635516</a></td><td align="right">2009</td><td>HFLGTLVNLAKKIL</td><td>Temporin-DRa</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>LC<sub>50</sub> =65μM</td><td>Human</td><td>CHHHHHHHHHTTTC</td><td align="center"><a href="jmol.php?id=4147&amp;di=1" width="5px" height="5px"><img src="pymol_images/4147.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4148">4148</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19635516">19635516</a></td><td align="right">2009</td><td>FVQWFSKFLGRIL</td><td>Temporin L</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">13</td><td>Antimicrobial</td><td>LC<sub>50</sub> =2μM</td><td>Human</td><td>CHHHHHHHHHCCC</td><td align="center"><a href="jmol.php?id=4148&amp;di=1" width="5px" height="5px"><img src="pymol_images/4148.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4148&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4148.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4149">4149</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16137634">16137634</a></td><td align="right">2005</td><td>INLKALAALAKALL</td><td>Mastoparan 7</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>100% hemolysis at 25 µM</td><td>Bovine</td><td>CCHHHHHHHHHHHC</td><td align="center"><a href="jmol.php?id=4149&amp;di=1" width="5px" height="5px"><img src="pymol_images/4149.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4149&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4149.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4158">4158</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20564013">20564013</a></td><td align="right">2010</td><td>GLPVCGETCVGGTCNTPGCTCSWPVCTRN</td><td>Kalata B1</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Pesticidal</td><td>HD<sub>50</sub> =20.0±1.1μM </td><td>Human</td><td>CCCCCSSHHHHCCCCCSSCCCCSSCCSCC</td><td align="center"><a href="jmol.php?id=4158&amp;di=1" width="5px" height="5px"><img src="pymol_images/4158.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4158&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4158.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4159">4159</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/20564013">20564013</a></td><td align="right">2010</td><td>GLPVCGETCTLGTCYTQGCTCSWPICKRN</td><td>Kalata B7</td><td>Free</td><td>Free</td><td>Cyclic</td><td>L</td><td>None</td><td align="right">30</td><td>Pesticidal</td><td>HD<sub>50</sub> =55.2±1.3μM </td><td>Human</td><td>CCSSCCCSSCCCCCCSSSSCCCCSSSTTC</td><td align="center"><a href="jmol.php?id=4159&amp;di=1" width="5px" height="5px"><img src="pymol_images/4159.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4159&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4159.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4160">4160</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15009528">15009528</a></td><td align="right">2004</td><td>VKLKVyPLKVKLyP</td><td>GS14K<sub>L</sub>4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>MHC =2µg/ml</td><td>Human</td><td>CTTCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=4160&amp;di=1" width="5px" height="5px"><img src="pymol_images/4160.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4160&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4160.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4161">4161</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15009528">15009528</a></td><td align="right">2004</td><td>VKLLVyPLKVKLyP</td><td>GS14L<sub>L</sub>4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>MHC =3µg/ml</td><td>Human</td><td>CCCEEETTEEECCC</td><td align="center"><a href="jmol.php?id=4161&amp;di=1" width="5px" height="5px"><img src="pymol_images/4161.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4161&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4161.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4162">4162</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15009528">15009528</a></td><td align="right">2004</td><td>VKLFVyPLKVKLyP</td><td>GS14F<sub>L</sub>4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>MHC =3µg/ml</td><td>Human</td><td>CCCCSSCCCCCSCC</td><td align="center"><a href="jmol.php?id=4162&amp;di=1" width="5px" height="5px"><img src="pymol_images/4162.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4162&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4162.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4163">4163</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15009528">15009528</a></td><td align="right">2004</td><td>VKLYVyPLKVKLyP</td><td>GS14Y<sub>L</sub>4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>MHC =8µg/ml</td><td>Human</td><td>CCCSSSSSSCCSCC</td><td align="center"><a href="jmol.php?id=4163&amp;di=1" width="5px" height="5px"><img src="pymol_images/4163.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4163&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4163.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4164">4164</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15009528">15009528</a></td><td align="right">2004</td><td>VKLNVyPLKVKLyP</td><td>GS14N<sub>L</sub>4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>MHC =2µg/ml</td><td>Human</td><td>CCCCSCTTCCCSCC</td><td align="center"><a href="jmol.php?id=4164&amp;di=1" width="5px" height="5px"><img src="pymol_images/4164.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4164&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4164.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4165">4165</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15009528">15009528</a></td><td align="right">2004</td><td>VKLGVyPLKVKLyP</td><td>GS14G4</td><td>Free</td><td>Free</td><td>Cyclic</td><td>Mix</td><td>None</td><td align="right">14</td><td>Antimicrobial</td><td>MHC =3µg/ml</td><td>Human</td><td>CCSTTSSSCCCSCC</td><td align="center"><a href="jmol.php?id=4165&amp;di=1" width="5px" height="5px"><img src="pymol_images/4165.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4165&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4165.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4166">4166</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19111524">19111524</a></td><td align="right">2009</td><td>X-NH-GVATQLLAAYILLFDEYNEKKASAQKDILIKVL</td><td>H-88</td><td>Amidation</td><td>X=H, NBD, Rhodamine</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Cytotoxic</td><td>50% hemolysis at 0.7.2μM </td><td>Human</td><td>CCHHHHHHHHGGGSCTTSGGGGGSSSCCCCCCC</td><td align="center"><a href="jmol.php?id=4166&amp;di=1" width="5px" height="5px"><img src="pymol_images/4166.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4167">4167</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/19111524">19111524</a></td><td align="right">2009</td><td>X-NH-GDATQLLAAYILLFDEYNEKKASAQKDILIKVL</td><td>Mu1-H88</td><td>Amidation</td><td>X=H, NBD, Rhodamine</td><td>Linear</td><td>L</td><td>None</td><td align="right">21</td><td>Cytotoxic</td><td>0% hemolysis at 0.8μM </td><td>Human</td><td>CCHHHHHHHHHHHCGGGCGGGTCSSCCSSSCCC</td><td align="center"><a href="jmol.php?id=4167&amp;di=1" width="5px" height="5px"><img src="pymol_images/4167.png" style="width:60px;height:60px;border:1;"></a></td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4168">4168</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLKVLTTG</td><td>HP(2-9)-ME(1-12)</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis upto 100µM</td><td>Human</td><td>CHHHHTTTCCTTSCCCCSCC</td><td align="center"><a href="jmol.php?id=4168&amp;di=1" width="5px" height="5px"><img src="pymol_images/4168.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4168&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4168.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4169">4169</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTTG</td><td>HM1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>13% hemolysis at 25µM</td><td>Human</td><td>CTTSTTTSSCSSCCCSSTTC</td><td align="center"><a href="jmol.php?id=4169&amp;di=1" width="5px" height="5px"><img src="pymol_images/4169.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4169&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4169.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4170">4170</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTTG</td><td>HM1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>1% hemolysis at 12.5µM</td><td>Human</td><td>CTTSTTTSSCSSCCCSSTTC</td><td align="center"><a href="jmol.php?id=4170&amp;di=1" width="5px" height="5px"><img src="pymol_images/4170.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4170&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4170.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4171">4171</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTTG</td><td>HM1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis at 6.25µM</td><td>Human</td><td>CTTSTTTSSCSSCCCSSTTC</td><td align="center"><a href="jmol.php?id=4171&amp;di=1" width="5px" height="5px"><img src="pymol_images/4171.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4171&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4171.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4172">4172</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLKVLTWG</td><td>HM2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis upto 100µM</td><td>Human</td><td>CTTTCGGGCCSSSCCCCSCC</td><td align="center"><a href="jmol.php?id=4172&amp;di=1" width="5px" height="5px"><img src="pymol_images/4172.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4172&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4172.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4173">4173</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTWG</td><td>HM3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>44% hemolysis at 25µM</td><td>Human</td><td>CGGGGGGSSCTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=4173&amp;di=1" width="5px" height="5px"><img src="pymol_images/4173.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4173&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4173.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4174">4174</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTWG</td><td>HM3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>31% hemolysis at 12.5µM</td><td>Human</td><td>CGGGGGGSSCTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=4174&amp;di=1" width="5px" height="5px"><img src="pymol_images/4174.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4174&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4174.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4175">4175</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTWG</td><td>HM3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>12% hemolysis at 6.25µM</td><td>Human</td><td>CGGGGGGSSCTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=4175&amp;di=1" width="5px" height="5px"><img src="pymol_images/4175.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4175&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4175.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4176">4176</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTWG</td><td>HM3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>4% hemolysis at 3.12µM</td><td>Human</td><td>CGGGGGGSSCTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=4176&amp;di=1" width="5px" height="5px"><img src="pymol_images/4176.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4176&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4176.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4177">4177</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLWVLTWG</td><td>HM3</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis at 1.5µM</td><td>Human</td><td>CGGGGGGSSCTTSSCCCCCC</td><td align="center"><a href="jmol.php?id=4177&amp;di=1" width="5px" height="5px"><img src="pymol_images/4177.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4177&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4177.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4178">4178</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>44% hemolysis at 25µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=4178&amp;di=1" width="5px" height="5px"><img src="pymol_images/4178.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4178&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4178.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4179">4179</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>43% hemolysis at 12.5µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=4179&amp;di=1" width="5px" height="5px"><img src="pymol_images/4179.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4179&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4179.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4180">4180</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>30% hemolysis at 6.25µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=4180&amp;di=1" width="5px" height="5px"><img src="pymol_images/4180.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4180&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4180.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4181">4181</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>29% hemolysis at 3.1µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=4181&amp;di=1" width="5px" height="5px"><img src="pymol_images/4181.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4181&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4181.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4182">4182</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>13% hemolysis at 1.5µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=4182&amp;di=1" width="5px" height="5px"><img src="pymol_images/4182.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4182&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4182.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4183">4183</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AWKVFKRLGIGAVLWVLTWG</td><td>HM4</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis at 0.78µM</td><td>Human</td><td>CCTTGGGTEETTEECCCCCC</td><td align="center"><a href="jmol.php?id=4183&amp;di=1" width="5px" height="5px"><img src="pymol_images/4183.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4183&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4183.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4184">4184</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15127790">15127790</a></td><td align="right">2004</td><td>AKKVFKRLGIGAVLKVLKKG</td><td>HM5</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">20</td><td>Antimicrobial</td><td>0% hemolysis upto 100µM</td><td>Human</td><td>CGGGHHHHCCHHHHHHHTTC</td><td align="center"><a href="jmol.php?id=4184&amp;di=1" width="5px" height="5px"><img src="pymol_images/4184.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4184&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4184.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4189">4189</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22699557">22699557</a></td><td align="right">2012</td><td>IWRIFR</td><td>HFU2</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">6</td><td>Antimicrobial</td><td>MHC &gt;256μM</td><td>Human</td><td>CCCCCC</td><td align="center"><a href="jmol.php?id=4189&amp;di=1" width="5px" height="5px"><img src="pymol_images/4189.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4189&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4189.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4190">4190</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/22699557">22699557</a></td><td align="right">2012</td><td>IWRIFRRIF</td><td>HFU3</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">9</td><td>Antimicrobial</td><td>MHC =101.4μM</td><td>Human</td><td>CCSGGGGTC</td><td align="center"><a href="jmol.php?id=4190&amp;di=1" width="5px" height="5px"><img src="pymol_images/4190.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4190&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4190.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4191">4191</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/15298176">15298176</a></td><td align="right">2004</td><td>FGVLAKVAAHVVPAIAEHF</td><td>[∆<sup>1,2</sup>]Mac</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">19</td><td>Antibacterial</td><td>0% hemolysis upto 100µM</td><td>Rabbit</td><td>CCCGGGGGGTTTGGGGTTC</td><td align="center"><a href="jmol.php?id=4191&amp;di=1" width="5px" height="5px"><img src="pymol_images/4191.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4191&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4191.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4192">4192</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16460023">16460023</a></td><td align="right">2006</td><td>FKIKASKVLDKFGKIVGKVLKQLKKVSAVAKV</td><td>P6_1</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">32</td><td>Hemolytic</td><td>LC<sub>50</sub>=12.6µM</td><td>Human</td><td>CCCSSSGGGTTHHHHHHHHHHHHHHHHHHHTC</td><td align="center"><a href="jmol.php?id=4192&amp;di=1" width="5px" height="5px"><img src="pymol_images/4192.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4192&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4192.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4193">4193</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16460023">16460023</a></td><td align="right">2006</td><td>FSISPGKVLDKFGKIVGKVLKQLKKVSAVAKV</td><td>P6_2</td><td>Free</td><td>Free</td><td>Linear</td><td>L</td><td>None</td><td align="right">31</td><td>Hemolytic</td><td>LC<sub>50</sub>=28.1µM</td><td>Human</td><td>NA</td><td align="center"><a href="jmol.php?id=4193&amp;di=1" width="5px" height="5px"><img src="pymol_images/4193.png" style="width:60px;height:60px;border:1;"></a></td><td align="center"><a href="jmol.php?id=4193&amp;di=2" width="5px" height="5px"><img src="pymol_images_mod/4193.png" style="width:60px;height:60px;border:1;"></a></td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4194">4194</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16235231">16235231</a></td><td align="right">2006</td><td>ZCRRLCYKQRCVTYCRGR</td><td>Gomesin</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>Z= pyroglutamic acid</td><td align="right">18</td><td>Antimicrobial</td><td>~40% hemolysis at 100µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4195">4195</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16235231">16235231</a></td><td align="right">2006</td><td>ZSRRLSYKQRSVTYSRGR</td><td>[Ser<sup>2,6,11,15</sup>]-Gm</td><td>Amidation</td><td>Free</td><td>Linear</td><td>L</td><td>Z= pyroglutamic acid</td><td align="right">18</td><td>Antimicrobial</td><td>&lt;10% hemolysis at 100µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4196">4196</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16235231">16235231</a></td><td align="right">2006</td><td>ZSRRLCYKQRCVTYSRGR</td><td>[Ser<sup>2,15</sup>]-Gm</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>Z= pyroglutamic acid</td><td align="right">18</td><td>Antimicrobial</td><td>&lt;10% hemolysis at 100µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4197">4197</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16235231">16235231</a></td><td align="right">2006</td><td>ZCRRLSYKQRSVTYCRGR</td><td>[Ser<sup>6,11</sup>]-Gm</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>Z= pyroglutamic acid</td><td align="right">18</td><td>Antimicrobial</td><td>&lt;10% hemolysis at 100µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4198">4198</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16235231">16235231</a></td><td align="right">2006</td><td>ZDRRLSYKQRSVTYORGR</td><td>Cyclo(2–15)[Asp<sup>2</sup>, Ser<sup>6, 11</sup>, Orn<sup>15</sup>]-Gm</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>Z= pyroglutamic acid</td><td align="right">18</td><td>Antimicrobial</td><td>&lt;10% hemolysis at 100µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4199">4199</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16235231">16235231</a></td><td align="right">2006</td><td>ZSRRLDYKQROVTYSRGR</td><td>Cyclo(6-11)[Ser<sup>2,15</sup>, Asp<sup>6</sup>, Orn<sup>11</sup>]-Gm</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>Z= pyroglutamic acid</td><td align="right">18</td><td>Antimicrobial</td><td>&lt;10% hemolysis at 100µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="odd"><td align="right"><a href="display_sub.php?details=4200">4200</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16235231">16235231</a></td><td align="right">2006</td><td>ZCRRLEYKQRKVTYCRGR</td><td>Bicyclo(2–15, 6–11)[Cys<sup>2, 15</sup>, Glu<sup>6</sup>, Lys<sup>11</sup>]-Gm</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>Z= pyroglutamic acid</td><td align="right">18</td><td>Antimicrobial</td><td>&lt;10% hemolysis at 100µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr><tr class="even"><td align="right"><a href="display_sub.php?details=4201">4201</a></td><td><a href="http://www.ncbi.nlm.nih.gov/pubmed/16235231">16235231</a></td><td align="right">2006</td><td>ZERRLCYKQRCVTYKRGR</td><td>Bicyclo(2–15, 6–11)[Glu<sup>2</sup>, Cys<sup>6, 11</sup>, Lys<sup>15</sup>]-Gm</td><td>Amidation</td><td>Free</td><td>Cyclic</td><td>L</td><td>Z= pyroglutamic acid</td><td align="right">18</td><td>Antimicrobial</td><td>~32% hemolysis at 100µM</td><td>Human</td><td>NA</td><td align="center">NA</td><td align="center">NA</td></tr>'
test_list0 = test0.split(sep='</td><td>')
test_list1 = test1.split(sep='</td><td>')
test_list2 = test2.split(sep='</td><td>')
test_list3 = test3.split(sep='</td><td>')
test_list_list = [test_list0, test_list1, test_list2, test_list3]

In [152]:
seq =[]
toxic = []
for f in range(4):
    test_list = test_list_list[f]
    for i in range(int((len(test_list)/12))):
        seq_pos = ((2) + 12*i)
        toxic_pos = ((10)+ 12*i)
        seq.append((test_list[seq_pos]).upper())
        toxic.append(test_list[toxic_pos])

In [164]:
df_hemolytik = pd.DataFrame([seq, toxic], index = ['seq', 'toxic'])
df_hemolytik = df_hemolytik.transpose()
valid = []
valid1 =[]
valid2 =[]
for n in range(len(df_hemolytik)):
    if ((df_hemolytik.seq[n].isalpha())== False):
        valid.append(n)
    if ("X") in df_hemolytik.seq[n]:
        valid1.append(n)
    if ("O") in df_hemolytik.seq[n]:
        valid2.append(n)
    if ("Z") in df_hemolytik.seq[n]:
        valid2.append(n)
    
valid = valid + list(set(valid1) - set(valid)) 
valid = valid + list(set(valid2) - set(valid)) 
df_hemolytik = df_hemolytik.drop(valid)
df_hemolytik = df_hemolytik.reset_index(drop=True)

In [165]:
#automatic read out
df_hemolytik['toxic2'] = np.nan
df_hemolytik['toxic3'] = "other"
df_hemolytik['toxic4'] = "other"
df_hemolytik['toxic5'] = "other"
for n in range(len(df_hemolytik)):
    if "-" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic']= re.sub('-[^>]+[%μµnm ]', '', df_hemolytik.toxic[n])
    if "±" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic']= re.sub('±[^>]+[%μµnm ]', '', df_hemolytik.toxic[n])
    if "MHC" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic'] = df_hemolytik.toxic[n].replace('MHC', '15% hem')
    if "M" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "M"
    if "μM" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μM"
    if "µM" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μM"
    if "µg/ml" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μg/ml"
    if "μg/ml" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μg/ml"
    if "μg/mL" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μg/ml"
    if "µg/mL" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μg/ml"        
    if "nM" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "nM"
    if "mg/L" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "mg/L"
    if "mg/ml" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "mg/ml"        
    if "mM" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "mM"
    df_hemolytik.toxic2[n] = re.findall(r"[-+]?\d*\.\d+|\d+", str(df_hemolytik.toxic[n]))
    if (len(df_hemolytik.toxic2[n])==2):
        df_hemolytik.loc[n, 'toxic4'] = (df_hemolytik.toxic2[n])[0]
        df_hemolytik.loc[n, 'toxic5'] = (df_hemolytik.toxic2[n])[1]
df_hemolytik

,seq,toxic,toxic2,toxic3,toxic4,toxic5
0,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,2% hemolysis upto 100μg/ml,"[2, 100]",μg/ml,2,100
1,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,9% hemolysis upto 200μg/ml,"[9, 200]",μg/ml,9,200
2,AAAKAALNAVLVGANA,LC<sub>50</sub> =40μM,"[50, 40]",μM,50,40
3,AAGLAMLFLGILSAAGSTMGARA,~50% hemolysis at 40µM,"[50, 40]",μM,50,40
4,AAGLAMLFLGILSAAGSTMGARA,55M,[55],M,other,other
...,...,...,...,...,...,...
2534,IWRIFR,15% hem &gt;256μM,"[15, 256]",μM,15,256
2535,IWRIFRRIF,15% hem =101.4μM,"[15, 101.4]",μM,15,101.4
2536,FGVLAKVAAHVVPAIAEHF,0% hemolysis upto 100µM,"[0, 100]",μM,0,100
2537,FKIKASKVLDKFGKIVGKVLKQLKKVSAVAKV,LC<sub>50</sub>=12.6µM,"[50, 12.6]",μM,50,12.6


In [166]:
unvalid1= []
unvalid2= []
for n in range(len(df_hemolytik)):
    if ((df_hemolytik.toxic3[n]== 'other')):
        unvalid1.append(n)
    if ((df_hemolytik.toxic4[n]== 'other')):
        unvalid2.append(n)
unvalid = unvalid1 + list(set(unvalid2) - set(unvalid1))
df_hemolytik = df_hemolytik.drop(unvalid)
df_hemolytik.reset_index(drop=True)

,seq,toxic,toxic2,toxic3,toxic4,toxic5
0,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,2% hemolysis upto 100μg/ml,"[2, 100]",μg/ml,2,100
1,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,9% hemolysis upto 200μg/ml,"[9, 200]",μg/ml,9,200
2,AAAKAALNAVLVGANA,LC<sub>50</sub> =40μM,"[50, 40]",μM,50,40
3,AAGLAMLFLGILSAAGSTMGARA,~50% hemolysis at 40µM,"[50, 40]",μM,50,40
4,AARRAARRAARR,HC<sub>50</sub> &gt; 500mg/L,"[50, 500]",mg/L,50,500
...,...,...,...,...,...,...
2026,IWRIFR,15% hem &gt;256μM,"[15, 256]",μM,15,256
2027,IWRIFRRIF,15% hem =101.4μM,"[15, 101.4]",μM,15,101.4
2028,FGVLAKVAAHVVPAIAEHF,0% hemolysis upto 100µM,"[0, 100]",μM,0,100
2029,FKIKASKVLDKFGKIVGKVLKQLKKVSAVAKV,LC<sub>50</sub>=12.6µM,"[50, 12.6]",μM,50,12.6


In [167]:
df_hemolytik = df_hemolytik.reset_index(drop=True)
#types of units that need conversion "μg/ml", "nM", "M", "mM", "mg/L", "mg/ml"
glob = GlobalDescriptor(df_hemolytik.seq.tolist())
glob.calculate_MW(amide=True)
df_hemolytik['MW'] = glob.descriptor
#see what concentrations are in µg/ml instead of µM
mask0 = df_hemolytik["toxic3"] == 'µg/ml'
mask1 = df_hemolytik["toxic3"] == 'mg/L'
mask2 = df_hemolytik["toxic3"] == 'mg/ml'
mask3 = df_hemolytik["toxic3"] == 'μM'
mask4 = df_hemolytik["toxic3"] == 'nM'
mask5 = df_hemolytik["toxic3"] == 'mM'
mask6 = df_hemolytik["toxic3"] == 'M'
#calculate the µM
df_hemolytik["CONCENTRATION_µM"] = 0
df_hemolytik.loc[mask0, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask0])*1000) / df_hemolytik[mask0]['MW']
df_hemolytik.loc[mask1, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask1]).astype(float)*1000) / df_hemolytik[mask1]['MW']
df_hemolytik.loc[mask2, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask2]).astype(float)*1000000) / df_hemolytik[mask2]['MW']
df_hemolytik.loc[mask4, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask4]).astype(float)/1000)
df_hemolytik.loc[mask5, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask5]).astype(float)*1000)
df_hemolytik.loc[mask6, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask6]).astype(float)*1000000)
#fill in values already in µM
df_hemolytik.loc[mask3, "CONCENTRATION_µM"] = df_hemolytik["toxic5"][mask3].astype(float)
df_hemolytik

,seq,toxic,toxic2,toxic3,toxic4,toxic5,MW,CONCENTRATION_µM
0,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,2% hemolysis upto 100μg/ml,"[2, 100]",μg/ml,2,100,3176.71,0
1,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,9% hemolysis upto 200μg/ml,"[9, 200]",μg/ml,9,200,3176.71,0
2,AAAKAALNAVLVGANA,LC<sub>50</sub> =40μM,"[50, 40]",μM,50,40,1423.67,40.00
3,AAGLAMLFLGILSAAGSTMGARA,~50% hemolysis at 40µM,"[50, 40]",μM,50,40,2149.59,40.00
4,AARRAARRAARR,HC<sub>50</sub> &gt; 500mg/L,"[50, 500]",mg/L,50,500,1380.61,362.16
...,...,...,...,...,...,...,...,...
2026,IWRIFR,15% hem &gt;256μM,"[15, 256]",μM,15,256,889.10,256.00
2027,IWRIFRRIF,15% hem =101.4μM,"[15, 101.4]",μM,15,101.4,1305.62,101.40
2028,FGVLAKVAAHVVPAIAEHF,0% hemolysis upto 100µM,"[0, 100]",μM,0,100,1975.35,100.00
2029,FKIKASKVLDKFGKIVGKVLKQLKKVSAVAKV,LC<sub>50</sub>=12.6µM,"[50, 12.6]",μM,50,12.6,3498.40,12.60


In [120]:
#select wether toxicity is positive or negative
#>=5% cell death with <=10uM peptide
#>=10% cell death with <=20uM peptide
#>=15% cell death with <=50uM peptide
#>=10% cell death with <=100uM peptide
#>=30% cell death with <=200uM peptide
#>=50% cell death with <=300uM peptide
#neg if not or 
#<=5% cell death with >=50uM
df_hemolytik["toxic6"] = "~"
for v in range(len(df_hemolytik)):
    if float(df_hemolytik["toxic4"][v]) <= 5:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 50:
            df_hemolytik.loc[v, "toxic6"] = 0
    if float(df_hemolytik["toxic4"][v]) >= 5:
        if float(df_hemolytik["CONCENTRATION_µM"][v]) <= 10:
            df_hemolytik.loc[v, "toxic6"] = 1
        else:
            df_hemolytik.loc[v, "toxic6"] = 0
    if float(df_hemolytik["toxic4"][v]) >= 10:
        if float(df_hemolytik["CONCENTRATION_µM"][v]) <= 20:
            df_hemolytik.loc[v, "toxic6"] = 1
        else:
            df_hemolytik.loc[v, "toxic6"] = 0
    if float(df_hemolytik["toxic4"][v]) >= 15:
        if float(df_hemolytik["CONCENTRATION_µM"][v]) <= 50:
            df_hemolytik.loc[v, "toxic6"] = 1
        else:
            df_hemolytik.loc[v, "toxic6"] = 0
    if float(df_hemolytik["toxic4"][v]) >= 20:
        if float(df_hemolytik["CONCENTRATION_µM"][v]) <= 100:
            df_hemolytik.loc[v, "toxic6"] = 1
        else:
            df_hemolytik.loc[v, "toxic6"] = 0
    if float(df_hemolytik["toxic4"][v]) >= 30:
        if float(df_hemolytik["CONCENTRATION_µM"][v]) <= 200:
            df_hemolytik.loc[v, "toxic6"] = 1
        else:
            df_hemolytik.loc[v, "toxic6"] = 0            
    if float(df_hemolytik["toxic4"][v]) >= 50:
        if float(df_hemolytik["CONCENTRATION_µM"][v]) <= 300:
            df_hemolytik.loc[v, "toxic6"] = 1
        else:
            df_hemolytik.loc[v, "toxic6"] = 0 

KeyError: '[ True False False ... False False False] not found in axis'

In [168]:
#HAPPENN formating of positive/negative
#happenn formating
df_hemolytik["toxic6"] = "~"
for v in range(len(df_hemolytik)):
    if float(df_hemolytik["toxic4"][v]) <= 5:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 30:
            df_hemolytik.loc[v, "toxic6"] = 0        
    if 5 <= float(df_hemolytik["toxic4"][v]) <= 10:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 60:
            df_hemolytik.loc[v, "toxic6"] = 0   
    if 10 <= float(df_hemolytik["toxic4"][v]) <= 15:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 90:
            df_hemolytik.loc[v, "toxic6"] = 0
    if 15 <= float(df_hemolytik["toxic4"][v]) <= 20:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 120:
            df_hemolytik.loc[v, "toxic6"] = 0
    if 20 <= float(df_hemolytik["toxic4"][v]) <= 25:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 180:
            df_hemolytik.loc[v, "toxic6"] = 0  
    if 25 <= float(df_hemolytik["toxic4"][v]) <= 30:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 180:
            df_hemolytik.loc[v, "toxic6"] = 0  
    if 30 <= float(df_hemolytik["toxic4"][v]) <= 35:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 210:
            df_hemolytik.loc[v, "toxic6"] = 0         
    if 35 <= float(df_hemolytik["toxic4"][v]) <= 40:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 240:
            df_hemolytik.loc[v, "toxic6"] = 0  
    if 40 <= float(df_hemolytik["toxic4"][v]) <= 45:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 270:
            df_hemolytik.loc[v, "toxic6"] = 0  
    if float(df_hemolytik["toxic4"][v]) >= 50:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 300:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 55:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 330:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 60:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 360:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 65:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 390:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 70:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 420:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 75:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 450:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 80:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 480:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 85:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 510:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 90:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 540:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 95:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 570:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 100:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 600:
            df_hemolytik.loc[v, "toxic6"] = 1 

In [169]:
unvalid1= []
for n in range(len(df_hemolytik)):
    if ((df_hemolytik.toxic6[n]== '~')):
        unvalid1.append(n)
df_hemolytik = df_hemolytik.drop(unvalid1)
df_hemolytik = df_hemolytik.reset_index(drop=True)

In [171]:
peptides = df_hemolytik["seq"].unique()
final_data = pd.DataFrame(peptides, columns=['seq'])
for a in range(len(df_hemolytik)):
    for n in range(len(peptides)):
        toxic = []
        toxic_count = 0
        if df_hemolytik["seq"][a] == peptides[n]:
            toxic.append(df_hemolytik["toxic6"][a])
            toxic_count = sum(toxic)/len(toxic)
            if toxic_count == len(toxic)/2:
                final_data.loc[n, "toxic"] = "~"
            if toxic_count <= len(toxic)/2:
                final_data.loc[n, "toxic"] = 0
            if toxic_count >= len(toxic)/2:
                final_data.loc[n, "toxic"] = 1

In [174]:

final_data.to_csv(Root + "/Programing/Data/AMP_focussed_data/hemolytic_data.csv",index = False, header=True)

In [172]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef
from lazypredict.Supervised import LazyClassifier
from DNN_features import *
test2 = AAC_df(final_data.seq)
train_X, val_X, train_y, val_y = train_test_split(test2, final_data.toxic, random_state = 445)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=matthews_corrcoef)
#models_train,predictions_train = clf.fit(train_X, train_X, train_y, train_y)
models,predictions = clf.fit(train_X, val_X, train_y, val_y)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 19.56it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,matthews_corrcoef,Time Taken
Model,,,,,,
ExtraTreesClassifier,0.90,0.89,0.89,0.90,0.78,0.17
LabelPropagation,0.88,0.88,0.88,0.88,0.76,0.04
LabelSpreading,0.88,0.88,0.88,0.88,0.76,0.04
XGBClassifier,0.87,0.87,0.87,0.87,0.73,0.08
LGBMClassifier,0.86,0.85,0.85,0.86,0.71,0.07
NuSVC,0.84,0.80,0.80,0.83,0.66,0.03
ExtraTreeClassifier,0.80,0.80,0.80,0.80,0.59,0.01
BaggingClassifier,0.80,0.80,0.80,0.80,0.59,0.06
RandomForestClassifier,0.83,0.80,0.80,0.82,0.63,0.24


In [173]:
final_data.toxic.value_counts()

1.00    446
0.00    239
Name: toxic, dtype: int64